In [1]:
!git clone https://github.com/RyanWangZf/MedCLIP.git

Cloning into 'MedCLIP'...
remote: Enumerating objects: 672, done.
remote: Counting objects: 100% (672/672), done.
remote: Compressing objects: 100% (291/291), done.
remote: Total 672 (delta 385), reused 619 (delta 364), pack-reused 0 (from 0)
Receiving objects: 100% (672/672), 88.06 MiB | 35.97 MiB/s, done.
Resolving deltas: 100% (385/385), done.


In [ ]:
%ls

In [2]:
%cd MedCLIP

/kaggle/working/MedCLIP


In [3]:
!mv examples/run_medclip_pretrain.py run_medclip_pretrain.py

In [25]:
!pip install -r requirements.txt

  Using cached wget-3.2.zip (10 kB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for textaugment>=1.3.4 from https://files.pythonhosted.org/packages/fd/fd/a2a4b69a8fe92e5c79783a408cc393a24994cf425b39ac01cd41453fedc7/textaugment-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for timm>=0.6.11 from https://files.pythonhosted.org/packages/6c/d0/179abca8b984b3deefd996f362b612c39da73b60f685921e6cd58b6125b4/timm-1.0.15-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 896.9 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for transformers<=4.24.0,>=4.23.1 from https://files.pythonhosted.org/packages/a4/df/3248eac2923ceffdf55686ff318e002b558e7c51f6a909dd870cf3185949/transformers-4.24.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 3.8 MB/s eta 0:00:00
  Obtaining dependency information for gensim>=4.0 from https://files.pythonhosted.org/packages/78/4f/f6045d5d

In [26]:
!pip install textblob==0.15.2


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


## **Importing Libraries**

In [1]:
import pdb, os
import random

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

from medclip.modeling_medclip import MedCLIPModel, PromptClassifier, MedCLIPVisionModel, MedCLIPVisionModelViT, MedCLIPVisionModelSwin
from medclip.dataset import ImageTextContrastiveDataset, ZeroShotImageDataset
from medclip.dataset import ImageTextContrastiveCollator, ZeroShotImageCollator
from medclip.losses import ImageTextContrastiveLoss
from medclip.trainer import Trainer
from medclip.evaluator import Evaluator
from medclip import constants
from medclip.prompts import generate_class_prompts, generate_chexpert_class_prompts, generate_covid_class_prompts

## **If you want to use `mendeley` dataset, execute the below cells**

In [3]:
%%writefile local_data/lbp-train-meta.csv
,imgpath,report,No Finding,LBP
0,local_data/mendeley/T1_0002_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1,local_data/mendeley/T1_0002_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2,local_data/mendeley/T1_0002_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
3,local_data/mendeley/T2_0002_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
4,local_data/mendeley/T2_0002_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
5,local_data/mendeley/T2_0002_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
6,local_data/mendeley/T1_0004_D3.png,"Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mildly compressing the thecal sac and encroaching exit nerve roots",0,1
7,local_data/mendeley/T1_0004_D4.png,"Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mildly compressing the thecal sac and encroaching exit nerve roots",0,1
8,local_data/mendeley/T1_0004_D5.png,"Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mildly compressing the thecal sac and encroaching exit nerve roots",0,1
9,local_data/mendeley/T2_0004_D3.png,"Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mildly compressing the thecal sac and encroaching exit nerve roots",0,1
10,local_data/mendeley/T2_0004_D4.png,"Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mildly compressing the thecal sac and encroaching exit nerve roots",0,1
11,local_data/mendeley/T2_0004_D5.png,"Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mildly compressing the thecal sac and encroaching exit nerve roots",0,1
12,local_data/mendeley/T1_0005_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted ar L4/L5 level ,mildly compressing the thecal sac and both exit nerve roots",0,1
13,local_data/mendeley/T1_0005_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted ar L4/L5 level ,mildly compressing the thecal sac and both exit nerve roots",0,1
14,local_data/mendeley/T1_0005_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted ar L4/L5 level ,mildly compressing the thecal sac and both exit nerve roots",0,1
15,local_data/mendeley/T2_0005_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted ar L4/L5 level ,mildly compressing the thecal sac and both exit nerve roots",0,1
16,local_data/mendeley/T2_0005_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted ar L4/L5 level ,mildly compressing the thecal sac and both exit nerve roots",0,1
17,local_data/mendeley/T2_0005_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted ar L4/L5 level ,mildly compressing the thecal sac and both exit nerve roots",0,1
18,local_data/mendeley/T1_0006_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots . Wide base disc bulges noted at L5/S1 level, mild compressing thecal sac and both nerve roots",0,1
19,local_data/mendeley/T1_0006_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots . Wide base disc bulges noted at L5/S1 level, mild compressing thecal sac and both nerve roots",0,1
20,local_data/mendeley/T1_0006_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots . Wide base disc bulges noted at L5/S1 level, mild compressing thecal sac and both nerve roots",0,1
21,local_data/mendeley/T2_0006_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots . Wide base disc bulges noted at L5/S1 level, mild compressing thecal sac and both nerve roots",0,1
22,local_data/mendeley/T2_0006_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots . Wide base disc bulges noted at L5/S1 level, mild compressing thecal sac and both nerve roots",0,1
23,local_data/mendeley/T2_0006_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots . Wide base disc bulges noted at L5/S1 level, mild compressing thecal sac and both nerve roots",0,1
24,local_data/mendeley/T1_0007_D3.png,"LSS MRI diffuse disc bulge noted at L4-L5 level , compressing the thecal sac and exit canals. small central disc protrusion noted at L5-S1 level. no significant spinal canal narrowing noted.",0,1
25,local_data/mendeley/T1_0007_D4.png,"LSS MRI diffuse disc bulge noted at L4-L5 level , compressing the thecal sac and exit canals. small central disc protrusion noted at L5-S1 level. no significant spinal canal narrowing noted.",0,1
26,local_data/mendeley/T1_0007_D5.png,"LSS MRI diffuse disc bulge noted at L4-L5 level , compressing the thecal sac and exit canals. small central disc protrusion noted at L5-S1 level. no significant spinal canal narrowing noted.",0,1
27,local_data/mendeley/T2_0007_D3.png,"LSS MRI diffuse disc bulge noted at L4-L5 level , compressing the thecal sac and exit canals. small central disc protrusion noted at L5-S1 level. no significant spinal canal narrowing noted.",0,1
28,local_data/mendeley/T2_0007_D4.png,"LSS MRI diffuse disc bulge noted at L4-L5 level , compressing the thecal sac and exit canals. small central disc protrusion noted at L5-S1 level. no significant spinal canal narrowing noted.",0,1
29,local_data/mendeley/T2_0007_D5.png,"LSS MRI diffuse disc bulge noted at L4-L5 level , compressing the thecal sac and exit canals. small central disc protrusion noted at L5-S1 level. no significant spinal canal narrowing noted.",0,1
30,local_data/mendeley/T1_0008_D3.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level , with mild compresing thecal sac, NB: ABout 7 cm, mixed pelvic cyst noted, correlate with US .",0,1
31,local_data/mendeley/T1_0008_D4.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level , with mild compresing thecal sac, NB: ABout 7 cm, mixed pelvic cyst noted, correlate with US .",0,1
32,local_data/mendeley/T1_0008_D5.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level , with mild compresing thecal sac, NB: ABout 7 cm, mixed pelvic cyst noted, correlate with US .",0,1
33,local_data/mendeley/T2_0008_D3.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level , with mild compresing thecal sac, NB: ABout 7 cm, mixed pelvic cyst noted, correlate with US .",0,1
34,local_data/mendeley/T2_0008_D4.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level , with mild compresing thecal sac, NB: ABout 7 cm, mixed pelvic cyst noted, correlate with US .",0,1
35,local_data/mendeley/T2_0008_D5.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level , with mild compresing thecal sac, NB: ABout 7 cm, mixed pelvic cyst noted, correlate with US .",0,1
36,local_data/mendeley/T1_0009_D3.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
37,local_data/mendeley/T1_0009_D4.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
38,local_data/mendeley/T1_0009_D5.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
39,local_data/mendeley/T2_0009_D3.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
40,local_data/mendeley/T2_0009_D4.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
41,local_data/mendeley/T2_0009_D5.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
42,local_data/mendeley/T1_0013_D3.png,L4-5 broad based central and left paracentral disc protrusion is noted compressing thecal sac and narrowing left neural foramen./// L5-S1: broad based degenerative central disc bulge is noted indenting thecal sac but not causing significant nerve root compression or foraminal narrowing.//// dehydrated L4-5 and L5-S1 intervetrebral discs./// vertebral bone marrow signal reconversion is noted.,0,1
43,local_data/mendeley/T1_0013_D4.png,L4-5 broad based central and left paracentral disc protrusion is noted compressing thecal sac and narrowing left neural foramen./// L5-S1: broad based degenerative central disc bulge is noted indenting thecal sac but not causing significant nerve root compression or foraminal narrowing.//// dehydrated L4-5 and L5-S1 intervetrebral discs./// vertebral bone marrow signal reconversion is noted.,0,1
44,local_data/mendeley/T1_0013_D5.png,L4-5 broad based central and left paracentral disc protrusion is noted compressing thecal sac and narrowing left neural foramen./// L5-S1: broad based degenerative central disc bulge is noted indenting thecal sac but not causing significant nerve root compression or foraminal narrowing.//// dehydrated L4-5 and L5-S1 intervetrebral discs./// vertebral bone marrow signal reconversion is noted.,0,1
45,local_data/mendeley/T2_0013_D3.png,L4-5 broad based central and left paracentral disc protrusion is noted compressing thecal sac and narrowing left neural foramen./// L5-S1: broad based degenerative central disc bulge is noted indenting thecal sac but not causing significant nerve root compression or foraminal narrowing.//// dehydrated L4-5 and L5-S1 intervetrebral discs./// vertebral bone marrow signal reconversion is noted.,0,1
46,local_data/mendeley/T2_0013_D4.png,L4-5 broad based central and left paracentral disc protrusion is noted compressing thecal sac and narrowing left neural foramen./// L5-S1: broad based degenerative central disc bulge is noted indenting thecal sac but not causing significant nerve root compression or foraminal narrowing.//// dehydrated L4-5 and L5-S1 intervetrebral discs./// vertebral bone marrow signal reconversion is noted.,0,1
47,local_data/mendeley/T2_0013_D5.png,L4-5 broad based central and left paracentral disc protrusion is noted compressing thecal sac and narrowing left neural foramen./// L5-S1: broad based degenerative central disc bulge is noted indenting thecal sac but not causing significant nerve root compression or foraminal narrowing.//// dehydrated L4-5 and L5-S1 intervetrebral discs./// vertebral bone marrow signal reconversion is noted.,0,1
48,local_data/mendeley/T1_0014_D3.png,LSS MRI : Feature of muscle spasm. No significant disc herniation or protrusion seen. Adequate thecal sac,0,1
49,local_data/mendeley/T1_0014_D4.png,LSS MRI : Feature of muscle spasm. No significant disc herniation or protrusion seen. Adequate thecal sac,0,1
50,local_data/mendeley/T1_0014_D5.png,LSS MRI : Feature of muscle spasm. No significant disc herniation or protrusion seen. Adequate thecal sac,0,1
51,local_data/mendeley/T2_0014_D3.png,LSS MRI : Feature of muscle spasm. No significant disc herniation or protrusion seen. Adequate thecal sac,0,1
52,local_data/mendeley/T2_0014_D4.png,LSS MRI : Feature of muscle spasm. No significant disc herniation or protrusion seen. Adequate thecal sac,0,1
53,local_data/mendeley/T2_0014_D5.png,LSS MRI : Feature of muscle spasm. No significant disc herniation or protrusion seen. Adequate thecal sac,0,1
54,local_data/mendeley/T1_0015_D3.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level Small right foraminal disc protrusion with annular tear is noted No significant thecal sac or nerve root compression noted,0,1
55,local_data/mendeley/T1_0015_D4.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level Small right foraminal disc protrusion with annular tear is noted No significant thecal sac or nerve root compression noted,0,1
56,local_data/mendeley/T1_0015_D5.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level Small right foraminal disc protrusion with annular tear is noted No significant thecal sac or nerve root compression noted,0,1
57,local_data/mendeley/T2_0015_D3.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level Small right foraminal disc protrusion with annular tear is noted No significant thecal sac or nerve root compression noted,0,1
58,local_data/mendeley/T2_0015_D4.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level Small right foraminal disc protrusion with annular tear is noted No significant thecal sac or nerve root compression noted,0,1
59,local_data/mendeley/T2_0015_D5.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level Small right foraminal disc protrusion with annular tear is noted No significant thecal sac or nerve root compression noted,0,1
60,local_data/mendeley/T1_0017_D3.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge causing compression of thecal sac, compressing left S1 nerve root within lateral recess, compressing left L5 nerve root within left neural foramen and narrowing right neural foramen./// L4-5: annular disc bulge is noted compressing thecal sac and narrowing both neural foramina/// L3-4: left posterolateral disc bulge is notedslightly narrowing left neural foramen.",0,1
61,local_data/mendeley/T1_0017_D4.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge causing compression of thecal sac, compressing left S1 nerve root within lateral recess, compressing left L5 nerve root within left neural foramen and narrowing right neural foramen./// L4-5: annular disc bulge is noted compressing thecal sac and narrowing both neural foramina/// L3-4: left posterolateral disc bulge is notedslightly narrowing left neural foramen.",0,1
62,local_data/mendeley/T1_0017_D5.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge causing compression of thecal sac, compressing left S1 nerve root within lateral recess, compressing left L5 nerve root within left neural foramen and narrowing right neural foramen./// L4-5: annular disc bulge is noted compressing thecal sac and narrowing both neural foramina/// L3-4: left posterolateral disc bulge is notedslightly narrowing left neural foramen.",0,1
63,local_data/mendeley/T2_0017_D3.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge causing compression of thecal sac, compressing left S1 nerve root within lateral recess, compressing left L5 nerve root within left neural foramen and narrowing right neural foramen./// L4-5: annular disc bulge is noted compressing thecal sac and narrowing both neural foramina/// L3-4: left posterolateral disc bulge is notedslightly narrowing left neural foramen.",0,1
64,local_data/mendeley/T2_0017_D4.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge causing compression of thecal sac, compressing left S1 nerve root within lateral recess, compressing left L5 nerve root within left neural foramen and narrowing right neural foramen./// L4-5: annular disc bulge is noted compressing thecal sac and narrowing both neural foramina/// L3-4: left posterolateral disc bulge is notedslightly narrowing left neural foramen.",0,1
65,local_data/mendeley/T2_0017_D5.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge causing compression of thecal sac, compressing left S1 nerve root within lateral recess, compressing left L5 nerve root within left neural foramen and narrowing right neural foramen./// L4-5: annular disc bulge is noted compressing thecal sac and narrowing both neural foramina/// L3-4: left posterolateral disc bulge is notedslightly narrowing left neural foramen.",0,1
66,local_data/mendeley/T1_0022_D3.png,Lumbosacral MRI Features of muscle spasm Central and left paracentral disc protrusion noted at the level of L4-L5 Significantly compressing the thecal sac and the left exit nerve root and desiccative disc material noted .,0,1
67,local_data/mendeley/T1_0022_D4.png,Lumbosacral MRI Features of muscle spasm Central and left paracentral disc protrusion noted at the level of L4-L5 Significantly compressing the thecal sac and the left exit nerve root and desiccative disc material noted .,0,1
68,local_data/mendeley/T1_0022_D5.png,Lumbosacral MRI Features of muscle spasm Central and left paracentral disc protrusion noted at the level of L4-L5 Significantly compressing the thecal sac and the left exit nerve root and desiccative disc material noted .,0,1
69,local_data/mendeley/T2_0022_D3.png,Lumbosacral MRI Features of muscle spasm Central and left paracentral disc protrusion noted at the level of L4-L5 Significantly compressing the thecal sac and the left exit nerve root and desiccative disc material noted .,0,1
70,local_data/mendeley/T2_0022_D4.png,Lumbosacral MRI Features of muscle spasm Central and left paracentral disc protrusion noted at the level of L4-L5 Significantly compressing the thecal sac and the left exit nerve root and desiccative disc material noted .,0,1
71,local_data/mendeley/T2_0022_D5.png,Lumbosacral MRI Features of muscle spasm Central and left paracentral disc protrusion noted at the level of L4-L5 Significantly compressing the thecal sac and the left exit nerve root and desiccative disc material noted .,0,1
72,local_data/mendeley/T1_0023_D3.png,L4-l5 L5-S1: diffuse disc bulges compressing the thecal sac and encroaching exit canals.,0,1
73,local_data/mendeley/T1_0023_D4.png,L4-l5 L5-S1: diffuse disc bulges compressing the thecal sac and encroaching exit canals.,0,1
74,local_data/mendeley/T1_0023_D5.png,L4-l5 L5-S1: diffuse disc bulges compressing the thecal sac and encroaching exit canals.,0,1
75,local_data/mendeley/T2_0023_D3.png,L4-l5 L5-S1: diffuse disc bulges compressing the thecal sac and encroaching exit canals.,0,1
76,local_data/mendeley/T2_0023_D4.png,L4-l5 L5-S1: diffuse disc bulges compressing the thecal sac and encroaching exit canals.,0,1
77,local_data/mendeley/T2_0023_D5.png,L4-l5 L5-S1: diffuse disc bulges compressing the thecal sac and encroaching exit canals.,0,1
78,local_data/mendeley/T1_0025_D3.png,L4-5: mild broad based central disc bulge is noted indenting thecal sac.// L5-S1: central disc protrusion is noted indenting thecal sac/// diffuse bone marrow signal reconversion is noted,0,1
79,local_data/mendeley/T1_0025_D4.png,L4-5: mild broad based central disc bulge is noted indenting thecal sac.// L5-S1: central disc protrusion is noted indenting thecal sac/// diffuse bone marrow signal reconversion is noted,0,1
80,local_data/mendeley/T1_0025_D5.png,L4-5: mild broad based central disc bulge is noted indenting thecal sac.// L5-S1: central disc protrusion is noted indenting thecal sac/// diffuse bone marrow signal reconversion is noted,0,1
81,local_data/mendeley/T2_0025_D3.png,L4-5: mild broad based central disc bulge is noted indenting thecal sac.// L5-S1: central disc protrusion is noted indenting thecal sac/// diffuse bone marrow signal reconversion is noted,0,1
82,local_data/mendeley/T2_0025_D4.png,L4-5: mild broad based central disc bulge is noted indenting thecal sac.// L5-S1: central disc protrusion is noted indenting thecal sac/// diffuse bone marrow signal reconversion is noted,0,1
83,local_data/mendeley/T2_0025_D5.png,L4-5: mild broad based central disc bulge is noted indenting thecal sac.// L5-S1: central disc protrusion is noted indenting thecal sac/// diffuse bone marrow signal reconversion is noted,0,1
84,local_data/mendeley/T1_0029_D3.png,"MRI OF THE LSS -Feature of muscle spasm. -Central disc protrusion noted at D11/D12, obliterating anterior CSF spaces and compressing spinal cord and encroaching exit nerve canals. -Diffuse disc bulges noted at L4/L5 , mild compressing thecal sac and exit nerve roots . Diffuse disc bulges noted at L5/S1, compressing thecal sac and exit nerve roots more to Lt side",0,1
85,local_data/mendeley/T1_0029_D4.png,"MRI OF THE LSS -Feature of muscle spasm. -Central disc protrusion noted at D11/D12, obliterating anterior CSF spaces and compressing spinal cord and encroaching exit nerve canals. -Diffuse disc bulges noted at L4/L5 , mild compressing thecal sac and exit nerve roots . Diffuse disc bulges noted at L5/S1, compressing thecal sac and exit nerve roots more to Lt side",0,1
86,local_data/mendeley/T1_0029_D5.png,"MRI OF THE LSS -Feature of muscle spasm. -Central disc protrusion noted at D11/D12, obliterating anterior CSF spaces and compressing spinal cord and encroaching exit nerve canals. -Diffuse disc bulges noted at L4/L5 , mild compressing thecal sac and exit nerve roots . Diffuse disc bulges noted at L5/S1, compressing thecal sac and exit nerve roots more to Lt side",0,1
87,local_data/mendeley/T2_0029_D3.png,"MRI OF THE LSS -Feature of muscle spasm. -Central disc protrusion noted at D11/D12, obliterating anterior CSF spaces and compressing spinal cord and encroaching exit nerve canals. -Diffuse disc bulges noted at L4/L5 , mild compressing thecal sac and exit nerve roots . Diffuse disc bulges noted at L5/S1, compressing thecal sac and exit nerve roots more to Lt side",0,1
88,local_data/mendeley/T2_0029_D4.png,"MRI OF THE LSS -Feature of muscle spasm. -Central disc protrusion noted at D11/D12, obliterating anterior CSF spaces and compressing spinal cord and encroaching exit nerve canals. -Diffuse disc bulges noted at L4/L5 , mild compressing thecal sac and exit nerve roots . Diffuse disc bulges noted at L5/S1, compressing thecal sac and exit nerve roots more to Lt side",0,1
89,local_data/mendeley/T2_0029_D5.png,"MRI OF THE LSS -Feature of muscle spasm. -Central disc protrusion noted at D11/D12, obliterating anterior CSF spaces and compressing spinal cord and encroaching exit nerve canals. -Diffuse disc bulges noted at L4/L5 , mild compressing thecal sac and exit nerve roots . Diffuse disc bulges noted at L5/S1, compressing thecal sac and exit nerve roots more to Lt side",0,1
90,local_data/mendeley/T1_0030_D3.png,"LSS MRI Diffuse disc bulge noted at L5-S1 level, more to the Lt side, compressing the thercal sac and Lt exit canal.",0,1
91,local_data/mendeley/T1_0030_D4.png,"LSS MRI Diffuse disc bulge noted at L5-S1 level, more to the Lt side, compressing the thercal sac and Lt exit canal.",0,1
92,local_data/mendeley/T1_0030_D5.png,"LSS MRI Diffuse disc bulge noted at L5-S1 level, more to the Lt side, compressing the thercal sac and Lt exit canal.",0,1
93,local_data/mendeley/T2_0030_D3.png,"LSS MRI Diffuse disc bulge noted at L5-S1 level, more to the Lt side, compressing the thercal sac and Lt exit canal.",0,1
94,local_data/mendeley/T2_0030_D4.png,"LSS MRI Diffuse disc bulge noted at L5-S1 level, more to the Lt side, compressing the thercal sac and Lt exit canal.",0,1
95,local_data/mendeley/T2_0030_D5.png,"LSS MRI Diffuse disc bulge noted at L5-S1 level, more to the Lt side, compressing the thercal sac and Lt exit canal.",0,1
96,local_data/mendeley/T1_0031_D3.png,"LSS MRI : Grade I spondylolithesis of L5/S1 level, with narrowed disc space . Multlevel dissicating disc materials noted Evidence of previus surgery noted at lumbosacra level. Diffuse disc bulges noted at (D12/L1& L4/L5 & L5/S1 levels, compressing (contrast study is asuperior",0,1
97,local_data/mendeley/T1_0031_D4.png,"LSS MRI : Grade I spondylolithesis of L5/S1 level, with narrowed disc space . Multlevel dissicating disc materials noted Evidence of previus surgery noted at lumbosacra level. Diffuse disc bulges noted at (D12/L1& L4/L5 & L5/S1 levels, compressing (contrast study is asuperior",0,1
98,local_data/mendeley/T1_0031_D5.png,"LSS MRI : Grade I spondylolithesis of L5/S1 level, with narrowed disc space . Multlevel dissicating disc materials noted Evidence of previus surgery noted at lumbosacra level. Diffuse disc bulges noted at (D12/L1& L4/L5 & L5/S1 levels, compressing (contrast study is asuperior",0,1
99,local_data/mendeley/T2_0031_D3.png,"LSS MRI : Grade I spondylolithesis of L5/S1 level, with narrowed disc space . Multlevel dissicating disc materials noted Evidence of previus surgery noted at lumbosacra level. Diffuse disc bulges noted at (D12/L1& L4/L5 & L5/S1 levels, compressing (contrast study is asuperior",0,1
100,local_data/mendeley/T2_0031_D4.png,"LSS MRI : Grade I spondylolithesis of L5/S1 level, with narrowed disc space . Multlevel dissicating disc materials noted Evidence of previus surgery noted at lumbosacra level. Diffuse disc bulges noted at (D12/L1& L4/L5 & L5/S1 levels, compressing (contrast study is asuperior",0,1
101,local_data/mendeley/T2_0031_D5.png,"LSS MRI : Grade I spondylolithesis of L5/S1 level, with narrowed disc space . Multlevel dissicating disc materials noted Evidence of previus surgery noted at lumbosacra level. Diffuse disc bulges noted at (D12/L1& L4/L5 & L5/S1 levels, compressing (contrast study is asuperior",0,1
102,local_data/mendeley/T1_0032_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
103,local_data/mendeley/T1_0032_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
104,local_data/mendeley/T1_0032_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
105,local_data/mendeley/T2_0032_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
106,local_data/mendeley/T2_0032_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
107,local_data/mendeley/T2_0032_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
108,local_data/mendeley/T1_0035_D3.png,"LSS MRI (with contrast ):Follow up History of recent & Prevoius operations . Inhomogenous enhancing granulation tissue noted at Lt sided of thecal sac ,compressing the Lt sided thecal sac and exit neural canal (site of operation (L5/S1 )extending posteriorly with small collection . Inhomogenous enhancing granulation noted at L4/L5 left sided .",0,1
109,local_data/mendeley/T1_0035_D4.png,"LSS MRI (with contrast ):Follow up History of recent & Prevoius operations . Inhomogenous enhancing granulation tissue noted at Lt sided of thecal sac ,compressing the Lt sided thecal sac and exit neural canal (site of operation (L5/S1 )extending posteriorly with small collection . Inhomogenous enhancing granulation noted at L4/L5 left sided .",0,1
110,local_data/mendeley/T1_0035_D5.png,"LSS MRI (with contrast ):Follow up History of recent & Prevoius operations . Inhomogenous enhancing granulation tissue noted at Lt sided of thecal sac ,compressing the Lt sided thecal sac and exit neural canal (site of operation (L5/S1 )extending posteriorly with small collection . Inhomogenous enhancing granulation noted at L4/L5 left sided .",0,1
111,local_data/mendeley/T2_0035_D3.png,"LSS MRI (with contrast ):Follow up History of recent & Prevoius operations . Inhomogenous enhancing granulation tissue noted at Lt sided of thecal sac ,compressing the Lt sided thecal sac and exit neural canal (site of operation (L5/S1 )extending posteriorly with small collection . Inhomogenous enhancing granulation noted at L4/L5 left sided .",0,1
112,local_data/mendeley/T2_0035_D4.png,"LSS MRI (with contrast ):Follow up History of recent & Prevoius operations . Inhomogenous enhancing granulation tissue noted at Lt sided of thecal sac ,compressing the Lt sided thecal sac and exit neural canal (site of operation (L5/S1 )extending posteriorly with small collection . Inhomogenous enhancing granulation noted at L4/L5 left sided .",0,1
113,local_data/mendeley/T2_0035_D5.png,"LSS MRI (with contrast ):Follow up History of recent & Prevoius operations . Inhomogenous enhancing granulation tissue noted at Lt sided of thecal sac ,compressing the Lt sided thecal sac and exit neural canal (site of operation (L5/S1 )extending posteriorly with small collection . Inhomogenous enhancing granulation noted at L4/L5 left sided .",0,1
114,local_data/mendeley/T1_0037_D3.png,MRI OF THE LSS Features of muscle spasm noted. No evidence of disc heniation noted. No significant thecal sac or nerve root compression noted.,0,1
115,local_data/mendeley/T1_0037_D4.png,MRI OF THE LSS Features of muscle spasm noted. No evidence of disc heniation noted. No significant thecal sac or nerve root compression noted.,0,1
116,local_data/mendeley/T1_0037_D5.png,MRI OF THE LSS Features of muscle spasm noted. No evidence of disc heniation noted. No significant thecal sac or nerve root compression noted.,0,1
117,local_data/mendeley/T2_0037_D3.png,MRI OF THE LSS Features of muscle spasm noted. No evidence of disc heniation noted. No significant thecal sac or nerve root compression noted.,0,1
118,local_data/mendeley/T2_0037_D4.png,MRI OF THE LSS Features of muscle spasm noted. No evidence of disc heniation noted. No significant thecal sac or nerve root compression noted.,0,1
119,local_data/mendeley/T2_0037_D5.png,MRI OF THE LSS Features of muscle spasm noted. No evidence of disc heniation noted. No significant thecal sac or nerve root compression noted.,0,1
120,local_data/mendeley/T1_0038_D3.png,Lumbosacral MRI Straightening of the lumbar spine mostly due to muscle spasm Dehydrated discal material Right paracentral disc protrusion noted at the L4-5 level compressing the thecal sac Right foraminal disc protrusion noted at the L5-S1 level compressing the right S1 nerve root Incidental finding of subcutaneous lipoma at the level of the L1 vertebra,0,1
121,local_data/mendeley/T1_0038_D4.png,Lumbosacral MRI Straightening of the lumbar spine mostly due to muscle spasm Dehydrated discal material Right paracentral disc protrusion noted at the L4-5 level compressing the thecal sac Right foraminal disc protrusion noted at the L5-S1 level compressing the right S1 nerve root Incidental finding of subcutaneous lipoma at the level of the L1 vertebra,0,1
122,local_data/mendeley/T1_0038_D5.png,Lumbosacral MRI Straightening of the lumbar spine mostly due to muscle spasm Dehydrated discal material Right paracentral disc protrusion noted at the L4-5 level compressing the thecal sac Right foraminal disc protrusion noted at the L5-S1 level compressing the right S1 nerve root Incidental finding of subcutaneous lipoma at the level of the L1 vertebra,0,1
123,local_data/mendeley/T2_0038_D3.png,Lumbosacral MRI Straightening of the lumbar spine mostly due to muscle spasm Dehydrated discal material Right paracentral disc protrusion noted at the L4-5 level compressing the thecal sac Right foraminal disc protrusion noted at the L5-S1 level compressing the right S1 nerve root Incidental finding of subcutaneous lipoma at the level of the L1 vertebra,0,1
124,local_data/mendeley/T2_0038_D4.png,Lumbosacral MRI Straightening of the lumbar spine mostly due to muscle spasm Dehydrated discal material Right paracentral disc protrusion noted at the L4-5 level compressing the thecal sac Right foraminal disc protrusion noted at the L5-S1 level compressing the right S1 nerve root Incidental finding of subcutaneous lipoma at the level of the L1 vertebra,0,1
125,local_data/mendeley/T2_0038_D5.png,Lumbosacral MRI Straightening of the lumbar spine mostly due to muscle spasm Dehydrated discal material Right paracentral disc protrusion noted at the L4-5 level compressing the thecal sac Right foraminal disc protrusion noted at the L5-S1 level compressing the right S1 nerve root Incidental finding of subcutaneous lipoma at the level of the L1 vertebra,0,1
126,local_data/mendeley/T1_0039_D3.png,"LSS MRI Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac. No significant thecal sac or nerve root compression noted. Adequate spinal canal.",0,1
127,local_data/mendeley/T1_0039_D4.png,"LSS MRI Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac. No significant thecal sac or nerve root compression noted. Adequate spinal canal.",0,1
128,local_data/mendeley/T1_0039_D5.png,"LSS MRI Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac. No significant thecal sac or nerve root compression noted. Adequate spinal canal.",0,1
129,local_data/mendeley/T2_0039_D3.png,"LSS MRI Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac. No significant thecal sac or nerve root compression noted. Adequate spinal canal.",0,1
130,local_data/mendeley/T2_0039_D4.png,"LSS MRI Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac. No significant thecal sac or nerve root compression noted. Adequate spinal canal.",0,1
131,local_data/mendeley/T2_0039_D5.png,"LSS MRI Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac. No significant thecal sac or nerve root compression noted. Adequate spinal canal.",0,1
132,local_data/mendeley/T1_0040_D3.png,"Lumbosacral MRI Features of muscle spasm Partially desiccated L4-L5level Central disc protrusion noted at the L4-L5 level, partially caudally migration , significant compressing the thecal sac and encroaching exiting neural canal Widely disc bulge noted at L5-S1 mildly compressing the thecal sac",0,1
133,local_data/mendeley/T1_0040_D4.png,"Lumbosacral MRI Features of muscle spasm Partially desiccated L4-L5level Central disc protrusion noted at the L4-L5 level, partially caudally migration , significant compressing the thecal sac and encroaching exiting neural canal Widely disc bulge noted at L5-S1 mildly compressing the thecal sac",0,1
134,local_data/mendeley/T1_0040_D5.png,"Lumbosacral MRI Features of muscle spasm Partially desiccated L4-L5level Central disc protrusion noted at the L4-L5 level, partially caudally migration , significant compressing the thecal sac and encroaching exiting neural canal Widely disc bulge noted at L5-S1 mildly compressing the thecal sac",0,1
135,local_data/mendeley/T2_0040_D3.png,"Lumbosacral MRI Features of muscle spasm Partially desiccated L4-L5level Central disc protrusion noted at the L4-L5 level, partially caudally migration , significant compressing the thecal sac and encroaching exiting neural canal Widely disc bulge noted at L5-S1 mildly compressing the thecal sac",0,1
136,local_data/mendeley/T2_0040_D4.png,"Lumbosacral MRI Features of muscle spasm Partially desiccated L4-L5level Central disc protrusion noted at the L4-L5 level, partially caudally migration , significant compressing the thecal sac and encroaching exiting neural canal Widely disc bulge noted at L5-S1 mildly compressing the thecal sac",0,1
137,local_data/mendeley/T2_0040_D5.png,"Lumbosacral MRI Features of muscle spasm Partially desiccated L4-L5level Central disc protrusion noted at the L4-L5 level, partially caudally migration , significant compressing the thecal sac and encroaching exiting neural canal Widely disc bulge noted at L5-S1 mildly compressing the thecal sac",0,1
138,local_data/mendeley/T1_0041_D3.png,"Lumbosacral MRI Diffuse disc bulges noted at L4-5 and L3-L4 levels, Abutting the thecal sac With mild encroachment of the left exit neural canal at L4-L5 level Features of muscle spasm noted",0,1
139,local_data/mendeley/T1_0041_D4.png,"Lumbosacral MRI Diffuse disc bulges noted at L4-5 and L3-L4 levels, Abutting the thecal sac With mild encroachment of the left exit neural canal at L4-L5 level Features of muscle spasm noted",0,1
140,local_data/mendeley/T1_0041_D5.png,"Lumbosacral MRI Diffuse disc bulges noted at L4-5 and L3-L4 levels, Abutting the thecal sac With mild encroachment of the left exit neural canal at L4-L5 level Features of muscle spasm noted",0,1
141,local_data/mendeley/T2_0041_D3.png,"Lumbosacral MRI Diffuse disc bulges noted at L4-5 and L3-L4 levels, Abutting the thecal sac With mild encroachment of the left exit neural canal at L4-L5 level Features of muscle spasm noted",0,1
142,local_data/mendeley/T2_0041_D4.png,"Lumbosacral MRI Diffuse disc bulges noted at L4-5 and L3-L4 levels, Abutting the thecal sac With mild encroachment of the left exit neural canal at L4-L5 level Features of muscle spasm noted",0,1
143,local_data/mendeley/T2_0041_D5.png,"Lumbosacral MRI Diffuse disc bulges noted at L4-5 and L3-L4 levels, Abutting the thecal sac With mild encroachment of the left exit neural canal at L4-L5 level Features of muscle spasm noted",0,1
144,local_data/mendeley/T1_0043_D3.png,L4-L5: Central disc protrusion compressing the thecal sac. L5-S1: Rt paracentral disc protrusion abutting Rt S1 nerve root.,0,1
145,local_data/mendeley/T1_0043_D4.png,L4-L5: Central disc protrusion compressing the thecal sac. L5-S1: Rt paracentral disc protrusion abutting Rt S1 nerve root.,0,1
146,local_data/mendeley/T1_0043_D5.png,L4-L5: Central disc protrusion compressing the thecal sac. L5-S1: Rt paracentral disc protrusion abutting Rt S1 nerve root.,0,1
147,local_data/mendeley/T2_0043_D3.png,L4-L5: Central disc protrusion compressing the thecal sac. L5-S1: Rt paracentral disc protrusion abutting Rt S1 nerve root.,0,1
148,local_data/mendeley/T2_0043_D4.png,L4-L5: Central disc protrusion compressing the thecal sac. L5-S1: Rt paracentral disc protrusion abutting Rt S1 nerve root.,0,1
149,local_data/mendeley/T2_0043_D5.png,L4-L5: Central disc protrusion compressing the thecal sac. L5-S1: Rt paracentral disc protrusion abutting Rt S1 nerve root.,0,1
150,local_data/mendeley/T1_0044_D3.png,"LSS MRI (without contrast): -Evidence of recent operation noted at lower level (L5/S1 ), with soft tissue edema. -Central and Rt paracentral disc protrusion notedat L5/S1 level, displacing inferiorly the thecal sac and encraoching Rt S1 exit nerve root. -Diffuse disc bulge noted at L4/L5 level extending to Lt lateral recess, compressing the thecal sac and encroaching upon exit neural canal .",0,1
151,local_data/mendeley/T1_0044_D4.png,"LSS MRI (without contrast): -Evidence of recent operation noted at lower level (L5/S1 ), with soft tissue edema. -Central and Rt paracentral disc protrusion notedat L5/S1 level, displacing inferiorly the thecal sac and encraoching Rt S1 exit nerve root. -Diffuse disc bulge noted at L4/L5 level extending to Lt lateral recess, compressing the thecal sac and encroaching upon exit neural canal .",0,1
152,local_data/mendeley/T1_0044_D5.png,"LSS MRI (without contrast): -Evidence of recent operation noted at lower level (L5/S1 ), with soft tissue edema. -Central and Rt paracentral disc protrusion notedat L5/S1 level, displacing inferiorly the thecal sac and encraoching Rt S1 exit nerve root. -Diffuse disc bulge noted at L4/L5 level extending to Lt lateral recess, compressing the thecal sac and encroaching upon exit neural canal .",0,1
153,local_data/mendeley/T2_0044_D3.png,"LSS MRI (without contrast): -Evidence of recent operation noted at lower level (L5/S1 ), with soft tissue edema. -Central and Rt paracentral disc protrusion notedat L5/S1 level, displacing inferiorly the thecal sac and encraoching Rt S1 exit nerve root. -Diffuse disc bulge noted at L4/L5 level extending to Lt lateral recess, compressing the thecal sac and encroaching upon exit neural canal .",0,1
154,local_data/mendeley/T2_0044_D4.png,"LSS MRI (without contrast): -Evidence of recent operation noted at lower level (L5/S1 ), with soft tissue edema. -Central and Rt paracentral disc protrusion notedat L5/S1 level, displacing inferiorly the thecal sac and encraoching Rt S1 exit nerve root. -Diffuse disc bulge noted at L4/L5 level extending to Lt lateral recess, compressing the thecal sac and encroaching upon exit neural canal .",0,1
155,local_data/mendeley/T2_0044_D5.png,"LSS MRI (without contrast): -Evidence of recent operation noted at lower level (L5/S1 ), with soft tissue edema. -Central and Rt paracentral disc protrusion notedat L5/S1 level, displacing inferiorly the thecal sac and encraoching Rt S1 exit nerve root. -Diffuse disc bulge noted at L4/L5 level extending to Lt lateral recess, compressing the thecal sac and encroaching upon exit neural canal .",0,1
156,local_data/mendeley/T1_0047_D3.png,"LSS MRI : -Feature of muscle spasm. -Degenerative spondylotic chnages noted . -Multilevel diffuse disc bulges , noteceably noted at L3/L4 & L4/L5 level, compresiing thecal sac and both nerve roots associated with ligamnetum flavum hypertrophy , all leading to secandary spinal canal stenosis. Small Lt paracentral disc protrusion noted at L1/L2 level , mild compressing thecal sac NB : Lt renal cyst noted",0,1
157,local_data/mendeley/T1_0047_D4.png,"LSS MRI : -Feature of muscle spasm. -Degenerative spondylotic chnages noted . -Multilevel diffuse disc bulges , noteceably noted at L3/L4 & L4/L5 level, compresiing thecal sac and both nerve roots associated with ligamnetum flavum hypertrophy , all leading to secandary spinal canal stenosis. Small Lt paracentral disc protrusion noted at L1/L2 level , mild compressing thecal sac NB : Lt renal cyst noted",0,1
158,local_data/mendeley/T1_0047_D5.png,"LSS MRI : -Feature of muscle spasm. -Degenerative spondylotic chnages noted . -Multilevel diffuse disc bulges , noteceably noted at L3/L4 & L4/L5 level, compresiing thecal sac and both nerve roots associated with ligamnetum flavum hypertrophy , all leading to secandary spinal canal stenosis. Small Lt paracentral disc protrusion noted at L1/L2 level , mild compressing thecal sac NB : Lt renal cyst noted",0,1
159,local_data/mendeley/T2_0047_D3.png,"LSS MRI : -Feature of muscle spasm. -Degenerative spondylotic chnages noted . -Multilevel diffuse disc bulges , noteceably noted at L3/L4 & L4/L5 level, compresiing thecal sac and both nerve roots associated with ligamnetum flavum hypertrophy , all leading to secandary spinal canal stenosis. Small Lt paracentral disc protrusion noted at L1/L2 level , mild compressing thecal sac NB : Lt renal cyst noted",0,1
160,local_data/mendeley/T2_0047_D4.png,"LSS MRI : -Feature of muscle spasm. -Degenerative spondylotic chnages noted . -Multilevel diffuse disc bulges , noteceably noted at L3/L4 & L4/L5 level, compresiing thecal sac and both nerve roots associated with ligamnetum flavum hypertrophy , all leading to secandary spinal canal stenosis. Small Lt paracentral disc protrusion noted at L1/L2 level , mild compressing thecal sac NB : Lt renal cyst noted",0,1
161,local_data/mendeley/T2_0047_D5.png,"LSS MRI : -Feature of muscle spasm. -Degenerative spondylotic chnages noted . -Multilevel diffuse disc bulges , noteceably noted at L3/L4 & L4/L5 level, compresiing thecal sac and both nerve roots associated with ligamnetum flavum hypertrophy , all leading to secandary spinal canal stenosis. Small Lt paracentral disc protrusion noted at L1/L2 level , mild compressing thecal sac NB : Lt renal cyst noted",0,1
162,local_data/mendeley/T1_0048_D3.png,"LSS MRI : feature of muscle spasm. Difffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both nerve roots",0,1
163,local_data/mendeley/T1_0048_D4.png,"LSS MRI : feature of muscle spasm. Difffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both nerve roots",0,1
164,local_data/mendeley/T1_0048_D5.png,"LSS MRI : feature of muscle spasm. Difffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both nerve roots",0,1
165,local_data/mendeley/T2_0048_D3.png,"LSS MRI : feature of muscle spasm. Difffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both nerve roots",0,1
166,local_data/mendeley/T2_0048_D4.png,"LSS MRI : feature of muscle spasm. Difffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both nerve roots",0,1
167,local_data/mendeley/T2_0048_D5.png,"LSS MRI : feature of muscle spasm. Difffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both nerve roots",0,1
168,local_data/mendeley/T1_0050_D3.png,"L3-L4, L5-S1: Diffuse disc bulges, more to the Rt side, abutting the thecal sac and encroaching Rt exit neural canal. L4-L5: Lt paracentral and foraminal disc protrusion, compressing the thecal sac and Lt exit neural canal.",0,1
169,local_data/mendeley/T1_0050_D4.png,"L3-L4, L5-S1: Diffuse disc bulges, more to the Rt side, abutting the thecal sac and encroaching Rt exit neural canal. L4-L5: Lt paracentral and foraminal disc protrusion, compressing the thecal sac and Lt exit neural canal.",0,1
170,local_data/mendeley/T1_0050_D5.png,"L3-L4, L5-S1: Diffuse disc bulges, more to the Rt side, abutting the thecal sac and encroaching Rt exit neural canal. L4-L5: Lt paracentral and foraminal disc protrusion, compressing the thecal sac and Lt exit neural canal.",0,1
171,local_data/mendeley/T2_0050_D3.png,"L3-L4, L5-S1: Diffuse disc bulges, more to the Rt side, abutting the thecal sac and encroaching Rt exit neural canal. L4-L5: Lt paracentral and foraminal disc protrusion, compressing the thecal sac and Lt exit neural canal.",0,1
172,local_data/mendeley/T2_0050_D4.png,"L3-L4, L5-S1: Diffuse disc bulges, more to the Rt side, abutting the thecal sac and encroaching Rt exit neural canal. L4-L5: Lt paracentral and foraminal disc protrusion, compressing the thecal sac and Lt exit neural canal.",0,1
173,local_data/mendeley/T2_0050_D5.png,"L3-L4, L5-S1: Diffuse disc bulges, more to the Rt side, abutting the thecal sac and encroaching Rt exit neural canal. L4-L5: Lt paracentral and foraminal disc protrusion, compressing the thecal sac and Lt exit neural canal.",0,1
174,local_data/mendeley/T1_0052_D3.png,"Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3/L4 & and L5-S1levels,  mild compressing the thecal sac and mild encroaching exiting neural canal,associated with annular tear at L5-S1",0,1
175,local_data/mendeley/T1_0052_D4.png,"Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3/L4 & and L5-S1levels,  mild compressing the thecal sac and mild encroaching exiting neural canal,associated with annular tear at L5-S1",0,1
176,local_data/mendeley/T1_0052_D5.png,"Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3/L4 & and L5-S1levels,  mild compressing the thecal sac and mild encroaching exiting neural canal,associated with annular tear at L5-S1",0,1
177,local_data/mendeley/T2_0052_D3.png,"Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3/L4 & and L5-S1levels,  mild compressing the thecal sac and mild encroaching exiting neural canal,associated with annular tear at L5-S1",0,1
178,local_data/mendeley/T2_0052_D4.png,"Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3/L4 & and L5-S1levels,  mild compressing the thecal sac and mild encroaching exiting neural canal,associated with annular tear at L5-S1",0,1
179,local_data/mendeley/T2_0052_D5.png,"Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3/L4 & and L5-S1levels,  mild compressing the thecal sac and mild encroaching exiting neural canal,associated with annular tear at L5-S1",0,1
180,local_data/mendeley/T1_0053_D3.png,Lumbosacral MRI Diffuse disc bulges noted at L4-L5 and L5-S1 levels with endplate degeneration Compressing the thecal sac,0,1
181,local_data/mendeley/T1_0053_D4.png,Lumbosacral MRI Diffuse disc bulges noted at L4-L5 and L5-S1 levels with endplate degeneration Compressing the thecal sac,0,1
182,local_data/mendeley/T1_0053_D5.png,Lumbosacral MRI Diffuse disc bulges noted at L4-L5 and L5-S1 levels with endplate degeneration Compressing the thecal sac,0,1
183,local_data/mendeley/T2_0053_D3.png,Lumbosacral MRI Diffuse disc bulges noted at L4-L5 and L5-S1 levels with endplate degeneration Compressing the thecal sac,0,1
184,local_data/mendeley/T2_0053_D4.png,Lumbosacral MRI Diffuse disc bulges noted at L4-L5 and L5-S1 levels with endplate degeneration Compressing the thecal sac,0,1
185,local_data/mendeley/T2_0053_D5.png,Lumbosacral MRI Diffuse disc bulges noted at L4-L5 and L5-S1 levels with endplate degeneration Compressing the thecal sac,0,1
186,local_data/mendeley/T1_0055_D3.png,LSS MRI No evidence of disc herniation noted. No thecal sac or nerve root compression noted,1,0
187,local_data/mendeley/T1_0055_D4.png,LSS MRI No evidence of disc herniation noted. No thecal sac or nerve root compression noted,1,0
188,local_data/mendeley/T1_0055_D5.png,LSS MRI No evidence of disc herniation noted. No thecal sac or nerve root compression noted,1,0
189,local_data/mendeley/T2_0055_D3.png,LSS MRI No evidence of disc herniation noted. No thecal sac or nerve root compression noted,1,0
190,local_data/mendeley/T2_0055_D4.png,LSS MRI No evidence of disc herniation noted. No thecal sac or nerve root compression noted,1,0
191,local_data/mendeley/T2_0055_D5.png,LSS MRI No evidence of disc herniation noted. No thecal sac or nerve root compression noted,1,0
192,local_data/mendeley/T1_0056_D3.png,"LSS MRI Enhancing Lt paracentral soft tissue noted at L5-S1 level, mostly granulation tissue, compressing Lt S1 nerve root and thecal sac. no definite fractures or subluxation noted.",0,1
193,local_data/mendeley/T1_0056_D4.png,"LSS MRI Enhancing Lt paracentral soft tissue noted at L5-S1 level, mostly granulation tissue, compressing Lt S1 nerve root and thecal sac. no definite fractures or subluxation noted.",0,1
194,local_data/mendeley/T1_0056_D5.png,"LSS MRI Enhancing Lt paracentral soft tissue noted at L5-S1 level, mostly granulation tissue, compressing Lt S1 nerve root and thecal sac. no definite fractures or subluxation noted.",0,1
195,local_data/mendeley/T2_0056_D3.png,"LSS MRI Enhancing Lt paracentral soft tissue noted at L5-S1 level, mostly granulation tissue, compressing Lt S1 nerve root and thecal sac. no definite fractures or subluxation noted.",0,1
196,local_data/mendeley/T2_0056_D4.png,"LSS MRI Enhancing Lt paracentral soft tissue noted at L5-S1 level, mostly granulation tissue, compressing Lt S1 nerve root and thecal sac. no definite fractures or subluxation noted.",0,1
197,local_data/mendeley/T2_0056_D5.png,"LSS MRI Enhancing Lt paracentral soft tissue noted at L5-S1 level, mostly granulation tissue, compressing Lt S1 nerve root and thecal sac. no definite fractures or subluxation noted.",0,1
198,local_data/mendeley/T1_0057_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level , mildly compressing the thecal sac, and  exit nerve roots",0,1
199,local_data/mendeley/T1_0057_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level , mildly compressing the thecal sac, and  exit nerve roots",0,1
200,local_data/mendeley/T1_0057_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level , mildly compressing the thecal sac, and  exit nerve roots",0,1
201,local_data/mendeley/T2_0057_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level , mildly compressing the thecal sac, and  exit nerve roots",0,1
202,local_data/mendeley/T2_0057_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level , mildly compressing the thecal sac, and  exit nerve roots",0,1
203,local_data/mendeley/T2_0057_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level , mildly compressing the thecal sac, and  exit nerve roots",0,1
204,local_data/mendeley/T1_0058_D3.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and both nerve roots .  mild diffuse disc bulges noted at L3/L4 level, mild compressing thecal sac.  NB :small tarlove cyst noted at S2 level",0,1
205,local_data/mendeley/T1_0058_D4.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and both nerve roots .  mild diffuse disc bulges noted at L3/L4 level, mild compressing thecal sac.  NB :small tarlove cyst noted at S2 level",0,1
206,local_data/mendeley/T1_0058_D5.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and both nerve roots .  mild diffuse disc bulges noted at L3/L4 level, mild compressing thecal sac.  NB :small tarlove cyst noted at S2 level",0,1
207,local_data/mendeley/T2_0058_D3.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and both nerve roots .  mild diffuse disc bulges noted at L3/L4 level, mild compressing thecal sac.  NB :small tarlove cyst noted at S2 level",0,1
208,local_data/mendeley/T2_0058_D4.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and both nerve roots .  mild diffuse disc bulges noted at L3/L4 level, mild compressing thecal sac.  NB :small tarlove cyst noted at S2 level",0,1
209,local_data/mendeley/T2_0058_D5.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and both nerve roots .  mild diffuse disc bulges noted at L3/L4 level, mild compressing thecal sac.  NB :small tarlove cyst noted at S2 level",0,1
210,local_data/mendeley/T1_0060_D3.png,C SPINE AND LSS MRI Features of muscle spasm. No evidence of disc herniation. No significant thecal sac or nerve root compressioon,0,1
211,local_data/mendeley/T1_0060_D4.png,C SPINE AND LSS MRI Features of muscle spasm. No evidence of disc herniation. No significant thecal sac or nerve root compressioon,0,1
212,local_data/mendeley/T1_0060_D5.png,C SPINE AND LSS MRI Features of muscle spasm. No evidence of disc herniation. No significant thecal sac or nerve root compressioon,0,1
213,local_data/mendeley/T2_0060_D3.png,C SPINE AND LSS MRI Features of muscle spasm. No evidence of disc herniation. No significant thecal sac or nerve root compressioon,0,1
214,local_data/mendeley/T2_0060_D4.png,C SPINE AND LSS MRI Features of muscle spasm. No evidence of disc herniation. No significant thecal sac or nerve root compressioon,0,1
215,local_data/mendeley/T2_0060_D5.png,C SPINE AND LSS MRI Features of muscle spasm. No evidence of disc herniation. No significant thecal sac or nerve root compressioon,0,1
216,local_data/mendeley/T1_0061_D3.png,"LSS MRI Rt paracentral disc protrusion notred at L4-L5 level, compressing the thecal sac and encroaching upon Rt L5 nerrve root. adequate spinal. features of muscle spasm.",0,1
217,local_data/mendeley/T1_0061_D4.png,"LSS MRI Rt paracentral disc protrusion notred at L4-L5 level, compressing the thecal sac and encroaching upon Rt L5 nerrve root. adequate spinal. features of muscle spasm.",0,1
218,local_data/mendeley/T1_0061_D5.png,"LSS MRI Rt paracentral disc protrusion notred at L4-L5 level, compressing the thecal sac and encroaching upon Rt L5 nerrve root. adequate spinal. features of muscle spasm.",0,1
219,local_data/mendeley/T2_0061_D3.png,"LSS MRI Rt paracentral disc protrusion notred at L4-L5 level, compressing the thecal sac and encroaching upon Rt L5 nerrve root. adequate spinal. features of muscle spasm.",0,1
220,local_data/mendeley/T2_0061_D4.png,"LSS MRI Rt paracentral disc protrusion notred at L4-L5 level, compressing the thecal sac and encroaching upon Rt L5 nerrve root. adequate spinal. features of muscle spasm.",0,1
221,local_data/mendeley/T2_0061_D5.png,"LSS MRI Rt paracentral disc protrusion notred at L4-L5 level, compressing the thecal sac and encroaching upon Rt L5 nerrve root. adequate spinal. features of muscle spasm.",0,1
222,local_data/mendeley/T1_0065_D3.png,L4-L5: Diffuse disc bulge compressing the thecal sac and exit neural canals.,0,1
223,local_data/mendeley/T1_0065_D4.png,L4-L5: Diffuse disc bulge compressing the thecal sac and exit neural canals.,0,1
224,local_data/mendeley/T1_0065_D5.png,L4-L5: Diffuse disc bulge compressing the thecal sac and exit neural canals.,0,1
225,local_data/mendeley/T2_0065_D3.png,L4-L5: Diffuse disc bulge compressing the thecal sac and exit neural canals.,0,1
226,local_data/mendeley/T2_0065_D4.png,L4-L5: Diffuse disc bulge compressing the thecal sac and exit neural canals.,0,1
227,local_data/mendeley/T2_0065_D5.png,L4-L5: Diffuse disc bulge compressing the thecal sac and exit neural canals.,0,1
228,local_data/mendeley/T1_0067_D3.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals. L5-S1: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt S1 nerve root.,0,1
229,local_data/mendeley/T1_0067_D4.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals. L5-S1: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt S1 nerve root.,0,1
230,local_data/mendeley/T1_0067_D5.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals. L5-S1: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt S1 nerve root.,0,1
231,local_data/mendeley/T2_0067_D3.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals. L5-S1: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt S1 nerve root.,0,1
232,local_data/mendeley/T2_0067_D4.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals. L5-S1: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt S1 nerve root.,0,1
233,local_data/mendeley/T2_0067_D5.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals. L5-S1: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt S1 nerve root.,0,1
234,local_data/mendeley/T1_0068_D3.png,L.SS MRI :  Central of Lt para central disc protrusion noted at L4/L5 migrating caudally causing thecal sac at Lt nerveroots,0,1
235,local_data/mendeley/T1_0068_D4.png,L.SS MRI :  Central of Lt para central disc protrusion noted at L4/L5 migrating caudally causing thecal sac at Lt nerveroots,0,1
236,local_data/mendeley/T1_0068_D5.png,L.SS MRI :  Central of Lt para central disc protrusion noted at L4/L5 migrating caudally causing thecal sac at Lt nerveroots,0,1
237,local_data/mendeley/T2_0068_D3.png,L.SS MRI :  Central of Lt para central disc protrusion noted at L4/L5 migrating caudally causing thecal sac at Lt nerveroots,0,1
238,local_data/mendeley/T2_0068_D4.png,L.SS MRI :  Central of Lt para central disc protrusion noted at L4/L5 migrating caudally causing thecal sac at Lt nerveroots,0,1
239,local_data/mendeley/T2_0068_D5.png,L.SS MRI :  Central of Lt para central disc protrusion noted at L4/L5 migrating caudally causing thecal sac at Lt nerveroots,0,1
240,local_data/mendeley/T1_0069_D3.png,L5-S1: Lt posterolateral disc protrusion noted. No significant thecal sac or nerve root compression noted.,0,1
241,local_data/mendeley/T1_0069_D4.png,L5-S1: Lt posterolateral disc protrusion noted. No significant thecal sac or nerve root compression noted.,0,1
242,local_data/mendeley/T1_0069_D5.png,L5-S1: Lt posterolateral disc protrusion noted. No significant thecal sac or nerve root compression noted.,0,1
243,local_data/mendeley/T2_0069_D3.png,L5-S1: Lt posterolateral disc protrusion noted. No significant thecal sac or nerve root compression noted.,0,1
244,local_data/mendeley/T2_0069_D4.png,L5-S1: Lt posterolateral disc protrusion noted. No significant thecal sac or nerve root compression noted.,0,1
245,local_data/mendeley/T2_0069_D5.png,L5-S1: Lt posterolateral disc protrusion noted. No significant thecal sac or nerve root compression noted.,0,1
246,local_data/mendeley/T1_0070_D3.png,L5-S1: Diffuse disc bulge abutting the thecal sac. No significant thecal sac or nerve root compression noted.,0,1
247,local_data/mendeley/T1_0070_D4.png,L5-S1: Diffuse disc bulge abutting the thecal sac. No significant thecal sac or nerve root compression noted.,0,1
248,local_data/mendeley/T1_0070_D5.png,L5-S1: Diffuse disc bulge abutting the thecal sac. No significant thecal sac or nerve root compression noted.,0,1
249,local_data/mendeley/T2_0070_D3.png,L5-S1: Diffuse disc bulge abutting the thecal sac. No significant thecal sac or nerve root compression noted.,0,1
250,local_data/mendeley/T2_0070_D4.png,L5-S1: Diffuse disc bulge abutting the thecal sac. No significant thecal sac or nerve root compression noted.,0,1
251,local_data/mendeley/T2_0070_D5.png,L5-S1: Diffuse disc bulge abutting the thecal sac. No significant thecal sac or nerve root compression noted.,0,1
252,local_data/mendeley/T1_0072_D3.png,"LSS MRI :  Feature of muscle spasm .  Multiple schmorls nodules  L3/L4 level , diffuse disc bulge , extending to lateral recess, more to Lt sided, compressing thecal sac and both nerve roots , more to Lt.  L4/L5 & L5/S1levels:  diffuse disc bulge , extending to lateral recess, compressing thecal sac and both nerve roots",0,1
253,local_data/mendeley/T1_0072_D4.png,"LSS MRI :  Feature of muscle spasm .  Multiple schmorls nodules  L3/L4 level , diffuse disc bulge , extending to lateral recess, more to Lt sided, compressing thecal sac and both nerve roots , more to Lt.  L4/L5 & L5/S1levels:  diffuse disc bulge , extending to lateral recess, compressing thecal sac and both nerve roots",0,1
254,local_data/mendeley/T1_0072_D5.png,"LSS MRI :  Feature of muscle spasm .  Multiple schmorls nodules  L3/L4 level , diffuse disc bulge , extending to lateral recess, more to Lt sided, compressing thecal sac and both nerve roots , more to Lt.  L4/L5 & L5/S1levels:  diffuse disc bulge , extending to lateral recess, compressing thecal sac and both nerve roots",0,1
255,local_data/mendeley/T2_0072_D3.png,"LSS MRI :  Feature of muscle spasm .  Multiple schmorls nodules  L3/L4 level , diffuse disc bulge , extending to lateral recess, more to Lt sided, compressing thecal sac and both nerve roots , more to Lt.  L4/L5 & L5/S1levels:  diffuse disc bulge , extending to lateral recess, compressing thecal sac and both nerve roots",0,1
256,local_data/mendeley/T2_0072_D4.png,"LSS MRI :  Feature of muscle spasm .  Multiple schmorls nodules  L3/L4 level , diffuse disc bulge , extending to lateral recess, more to Lt sided, compressing thecal sac and both nerve roots , more to Lt.  L4/L5 & L5/S1levels:  diffuse disc bulge , extending to lateral recess, compressing thecal sac and both nerve roots",0,1
257,local_data/mendeley/T2_0072_D5.png,"LSS MRI :  Feature of muscle spasm .  Multiple schmorls nodules  L3/L4 level , diffuse disc bulge , extending to lateral recess, more to Lt sided, compressing thecal sac and both nerve roots , more to Lt.  L4/L5 & L5/S1levels:  diffuse disc bulge , extending to lateral recess, compressing thecal sac and both nerve roots",0,1
258,local_data/mendeley/T1_0073_D3.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
259,local_data/mendeley/T1_0073_D4.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
260,local_data/mendeley/T1_0073_D5.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
261,local_data/mendeley/T2_0073_D3.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
262,local_data/mendeley/T2_0073_D4.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
263,local_data/mendeley/T2_0073_D5.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
264,local_data/mendeley/T1_0075_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
265,local_data/mendeley/T1_0075_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
266,local_data/mendeley/T1_0075_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
267,local_data/mendeley/T2_0075_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
268,local_data/mendeley/T2_0075_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
269,local_data/mendeley/T2_0075_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
270,local_data/mendeley/T1_0081_D3.png,"LSS MRI :  Feature of muscle spasm.  Wide base posteriocentral disc bulge noted at L4/L5 level, copmressing the thecal sac and encroaching upon exit neural canals.  Diffuse disc bulge noted at L3/L4 level, mildly compressing the thecal sac and both nerve roots",0,1
271,local_data/mendeley/T1_0081_D4.png,"LSS MRI :  Feature of muscle spasm.  Wide base posteriocentral disc bulge noted at L4/L5 level, copmressing the thecal sac and encroaching upon exit neural canals.  Diffuse disc bulge noted at L3/L4 level, mildly compressing the thecal sac and both nerve roots",0,1
272,local_data/mendeley/T1_0081_D5.png,"LSS MRI :  Feature of muscle spasm.  Wide base posteriocentral disc bulge noted at L4/L5 level, copmressing the thecal sac and encroaching upon exit neural canals.  Diffuse disc bulge noted at L3/L4 level, mildly compressing the thecal sac and both nerve roots",0,1
273,local_data/mendeley/T2_0081_D3.png,"LSS MRI :  Feature of muscle spasm.  Wide base posteriocentral disc bulge noted at L4/L5 level, copmressing the thecal sac and encroaching upon exit neural canals.  Diffuse disc bulge noted at L3/L4 level, mildly compressing the thecal sac and both nerve roots",0,1
274,local_data/mendeley/T2_0081_D4.png,"LSS MRI :  Feature of muscle spasm.  Wide base posteriocentral disc bulge noted at L4/L5 level, copmressing the thecal sac and encroaching upon exit neural canals.  Diffuse disc bulge noted at L3/L4 level, mildly compressing the thecal sac and both nerve roots",0,1
275,local_data/mendeley/T2_0081_D5.png,"LSS MRI :  Feature of muscle spasm.  Wide base posteriocentral disc bulge noted at L4/L5 level, copmressing the thecal sac and encroaching upon exit neural canals.  Diffuse disc bulge noted at L3/L4 level, mildly compressing the thecal sac and both nerve roots",0,1
276,local_data/mendeley/T1_0084_D3.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  End plate degenerative changes noted .  Grade I spondylolisthesis of L5 over S1 , with bone marrow edema and narrowed disc spaces with irregular end plates  Bulky Lt iliopsoas muscle with abnormal signal alteration ? post traumatic , infective….  Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac and exit neural canals.  Adequate Spinal canal noted.",0,1
277,local_data/mendeley/T1_0084_D4.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  End plate degenerative changes noted .  Grade I spondylolisthesis of L5 over S1 , with bone marrow edema and narrowed disc spaces with irregular end plates  Bulky Lt iliopsoas muscle with abnormal signal alteration ? post traumatic , infective….  Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac and exit neural canals.  Adequate Spinal canal noted.",0,1
278,local_data/mendeley/T1_0084_D5.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  End plate degenerative changes noted .  Grade I spondylolisthesis of L5 over S1 , with bone marrow edema and narrowed disc spaces with irregular end plates  Bulky Lt iliopsoas muscle with abnormal signal alteration ? post traumatic , infective….  Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac and exit neural canals.  Adequate Spinal canal noted.",0,1
279,local_data/mendeley/T2_0084_D3.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  End plate degenerative changes noted .  Grade I spondylolisthesis of L5 over S1 , with bone marrow edema and narrowed disc spaces with irregular end plates  Bulky Lt iliopsoas muscle with abnormal signal alteration ? post traumatic , infective….  Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac and exit neural canals.  Adequate Spinal canal noted.",0,1
280,local_data/mendeley/T2_0084_D4.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  End plate degenerative changes noted .  Grade I spondylolisthesis of L5 over S1 , with bone marrow edema and narrowed disc spaces with irregular end plates  Bulky Lt iliopsoas muscle with abnormal signal alteration ? post traumatic , infective….  Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac and exit neural canals.  Adequate Spinal canal noted.",0,1
281,local_data/mendeley/T2_0084_D5.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  End plate degenerative changes noted .  Grade I spondylolisthesis of L5 over S1 , with bone marrow edema and narrowed disc spaces with irregular end plates  Bulky Lt iliopsoas muscle with abnormal signal alteration ? post traumatic , infective….  Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac and exit neural canals.  Adequate Spinal canal noted.",0,1
282,local_data/mendeley/T1_0090_D3.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression .  No disc protrusion or herniation seen  NB :  small tarlovs cyst.  pelvic cyst noted ? correalte with US,0,1
283,local_data/mendeley/T1_0090_D4.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression .  No disc protrusion or herniation seen  NB :  small tarlovs cyst.  pelvic cyst noted ? correalte with US,0,1
284,local_data/mendeley/T1_0090_D5.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression .  No disc protrusion or herniation seen  NB :  small tarlovs cyst.  pelvic cyst noted ? correalte with US,0,1
285,local_data/mendeley/T2_0090_D3.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression .  No disc protrusion or herniation seen  NB :  small tarlovs cyst.  pelvic cyst noted ? correalte with US,0,1
286,local_data/mendeley/T2_0090_D4.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression .  No disc protrusion or herniation seen  NB :  small tarlovs cyst.  pelvic cyst noted ? correalte with US,0,1
287,local_data/mendeley/T2_0090_D5.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression .  No disc protrusion or herniation seen  NB :  small tarlovs cyst.  pelvic cyst noted ? correalte with US,0,1
288,local_data/mendeley/T1_0091_D3.png,LSS MRI :  scoliosis of lumbar spine noted.  dissicating disc materials .  Wedging fracture of L2 vertebral body mostly dut to old insult .  Grade II spondylolithesis of L4 over L5 level.  Diffuse disc bulges noted at L5/S1 level mild compressing thecal sac,0,1
289,local_data/mendeley/T1_0091_D4.png,LSS MRI :  scoliosis of lumbar spine noted.  dissicating disc materials .  Wedging fracture of L2 vertebral body mostly dut to old insult .  Grade II spondylolithesis of L4 over L5 level.  Diffuse disc bulges noted at L5/S1 level mild compressing thecal sac,0,1
290,local_data/mendeley/T1_0091_D5.png,LSS MRI :  scoliosis of lumbar spine noted.  dissicating disc materials .  Wedging fracture of L2 vertebral body mostly dut to old insult .  Grade II spondylolithesis of L4 over L5 level.  Diffuse disc bulges noted at L5/S1 level mild compressing thecal sac,0,1
291,local_data/mendeley/T2_0091_D3.png,LSS MRI :  scoliosis of lumbar spine noted.  dissicating disc materials .  Wedging fracture of L2 vertebral body mostly dut to old insult .  Grade II spondylolithesis of L4 over L5 level.  Diffuse disc bulges noted at L5/S1 level mild compressing thecal sac,0,1
292,local_data/mendeley/T2_0091_D4.png,LSS MRI :  scoliosis of lumbar spine noted.  dissicating disc materials .  Wedging fracture of L2 vertebral body mostly dut to old insult .  Grade II spondylolithesis of L4 over L5 level.  Diffuse disc bulges noted at L5/S1 level mild compressing thecal sac,0,1
293,local_data/mendeley/T2_0091_D5.png,LSS MRI :  scoliosis of lumbar spine noted.  dissicating disc materials .  Wedging fracture of L2 vertebral body mostly dut to old insult .  Grade II spondylolithesis of L4 over L5 level.  Diffuse disc bulges noted at L5/S1 level mild compressing thecal sac,0,1
294,local_data/mendeley/T1_0093_D3.png,LSS MRI L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit neural canal. Mild disc bulges noted at L3-L4 and L4-L5 levels.,0,1
295,local_data/mendeley/T1_0093_D4.png,LSS MRI L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit neural canal. Mild disc bulges noted at L3-L4 and L4-L5 levels.,0,1
296,local_data/mendeley/T1_0093_D5.png,LSS MRI L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit neural canal. Mild disc bulges noted at L3-L4 and L4-L5 levels.,0,1
297,local_data/mendeley/T2_0093_D3.png,LSS MRI L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit neural canal. Mild disc bulges noted at L3-L4 and L4-L5 levels.,0,1
298,local_data/mendeley/T2_0093_D4.png,LSS MRI L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit neural canal. Mild disc bulges noted at L3-L4 and L4-L5 levels.,0,1
299,local_data/mendeley/T2_0093_D5.png,LSS MRI L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit neural canal. Mild disc bulges noted at L3-L4 and L4-L5 levels.,0,1
300,local_data/mendeley/T1_0095_D3.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Mild Diffuse disc bulges noted at L4-L5 level,mildly compressing thecal sac and exit nerve root .",0,1
301,local_data/mendeley/T1_0095_D4.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Mild Diffuse disc bulges noted at L4-L5 level,mildly compressing thecal sac and exit nerve root .",0,1
302,local_data/mendeley/T1_0095_D5.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Mild Diffuse disc bulges noted at L4-L5 level,mildly compressing thecal sac and exit nerve root .",0,1
303,local_data/mendeley/T2_0095_D3.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Mild Diffuse disc bulges noted at L4-L5 level,mildly compressing thecal sac and exit nerve root .",0,1
304,local_data/mendeley/T2_0095_D4.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Mild Diffuse disc bulges noted at L4-L5 level,mildly compressing thecal sac and exit nerve root .",0,1
305,local_data/mendeley/T2_0095_D5.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Mild Diffuse disc bulges noted at L4-L5 level,mildly compressing thecal sac and exit nerve root .",0,1
306,local_data/mendeley/T1_0097_D3.png,"LSS MRI: feature of muscle spasm. Wide base focal disc bulge , more to Lt sided, noted at L5/S1 level, compressing thecal sac and both exit nerve roots associated with dehydrated disc material.",0,1
307,local_data/mendeley/T1_0097_D4.png,"LSS MRI: feature of muscle spasm. Wide base focal disc bulge , more to Lt sided, noted at L5/S1 level, compressing thecal sac and both exit nerve roots associated with dehydrated disc material.",0,1
308,local_data/mendeley/T1_0097_D5.png,"LSS MRI: feature of muscle spasm. Wide base focal disc bulge , more to Lt sided, noted at L5/S1 level, compressing thecal sac and both exit nerve roots associated with dehydrated disc material.",0,1
309,local_data/mendeley/T2_0097_D3.png,"LSS MRI: feature of muscle spasm. Wide base focal disc bulge , more to Lt sided, noted at L5/S1 level, compressing thecal sac and both exit nerve roots associated with dehydrated disc material.",0,1
310,local_data/mendeley/T2_0097_D4.png,"LSS MRI: feature of muscle spasm. Wide base focal disc bulge , more to Lt sided, noted at L5/S1 level, compressing thecal sac and both exit nerve roots associated with dehydrated disc material.",0,1
311,local_data/mendeley/T2_0097_D5.png,"LSS MRI: feature of muscle spasm. Wide base focal disc bulge , more to Lt sided, noted at L5/S1 level, compressing thecal sac and both exit nerve roots associated with dehydrated disc material.",0,1
312,local_data/mendeley/T1_0098_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
313,local_data/mendeley/T1_0098_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
314,local_data/mendeley/T1_0098_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
315,local_data/mendeley/T2_0098_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
316,local_data/mendeley/T2_0098_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
317,local_data/mendeley/T2_0098_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
318,local_data/mendeley/T1_0099_D3.png,Lumbosacral MRI Feature of muscle spasm Multilevels diffuse disc bulging noted noticebly at L3-L4 & L4-L5 levels mild compressing thecal sac and exit nerve roots dissicating disc material noted,0,1
319,local_data/mendeley/T1_0099_D4.png,Lumbosacral MRI Feature of muscle spasm Multilevels diffuse disc bulging noted noticebly at L3-L4 & L4-L5 levels mild compressing thecal sac and exit nerve roots dissicating disc material noted,0,1
320,local_data/mendeley/T1_0099_D5.png,Lumbosacral MRI Feature of muscle spasm Multilevels diffuse disc bulging noted noticebly at L3-L4 & L4-L5 levels mild compressing thecal sac and exit nerve roots dissicating disc material noted,0,1
321,local_data/mendeley/T2_0099_D3.png,Lumbosacral MRI Feature of muscle spasm Multilevels diffuse disc bulging noted noticebly at L3-L4 & L4-L5 levels mild compressing thecal sac and exit nerve roots dissicating disc material noted,0,1
322,local_data/mendeley/T2_0099_D4.png,Lumbosacral MRI Feature of muscle spasm Multilevels diffuse disc bulging noted noticebly at L3-L4 & L4-L5 levels mild compressing thecal sac and exit nerve roots dissicating disc material noted,0,1
323,local_data/mendeley/T2_0099_D5.png,Lumbosacral MRI Feature of muscle spasm Multilevels diffuse disc bulging noted noticebly at L3-L4 & L4-L5 levels mild compressing thecal sac and exit nerve roots dissicating disc material noted,0,1
324,local_data/mendeley/T1_0101_D3.png,LSS MRI : Feature of muscle spasm Mild difuse bulge disk  L5-S1 noted No significant thecal sac or nerve roots compression seen.,0,1
325,local_data/mendeley/T1_0101_D4.png,LSS MRI : Feature of muscle spasm Mild difuse bulge disk  L5-S1 noted No significant thecal sac or nerve roots compression seen.,0,1
326,local_data/mendeley/T1_0101_D5.png,LSS MRI : Feature of muscle spasm Mild difuse bulge disk  L5-S1 noted No significant thecal sac or nerve roots compression seen.,0,1
327,local_data/mendeley/T2_0101_D3.png,LSS MRI : Feature of muscle spasm Mild difuse bulge disk  L5-S1 noted No significant thecal sac or nerve roots compression seen.,0,1
328,local_data/mendeley/T2_0101_D4.png,LSS MRI : Feature of muscle spasm Mild difuse bulge disk  L5-S1 noted No significant thecal sac or nerve roots compression seen.,0,1
329,local_data/mendeley/T2_0101_D5.png,LSS MRI : Feature of muscle spasm Mild difuse bulge disk  L5-S1 noted No significant thecal sac or nerve roots compression seen.,0,1
330,local_data/mendeley/T1_0103_D3.png,L4-L5: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt L5 nerve root. Mild disc bulges noted at L3-L4 and L5-S1 levels.,0,1
331,local_data/mendeley/T1_0103_D4.png,L4-L5: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt L5 nerve root. Mild disc bulges noted at L3-L4 and L5-S1 levels.,0,1
332,local_data/mendeley/T1_0103_D5.png,L4-L5: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt L5 nerve root. Mild disc bulges noted at L3-L4 and L5-S1 levels.,0,1
333,local_data/mendeley/T2_0103_D3.png,L4-L5: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt L5 nerve root. Mild disc bulges noted at L3-L4 and L5-S1 levels.,0,1
334,local_data/mendeley/T2_0103_D4.png,L4-L5: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt L5 nerve root. Mild disc bulges noted at L3-L4 and L5-S1 levels.,0,1
335,local_data/mendeley/T2_0103_D5.png,L4-L5: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt L5 nerve root. Mild disc bulges noted at L3-L4 and L5-S1 levels.,0,1
336,local_data/mendeley/T1_0105_D3.png,"LSS MRI :  -Feature of muscle spasm  -Multiple schmorls noted and suggetsive limbus veterba at anterio-superior corner of L4 .  -Mild diffuse disc bulges noted at L4/L5 and L5/S1 levels, mild compressing tehcal sac and both nerve roots .",0,1
337,local_data/mendeley/T1_0105_D4.png,"LSS MRI :  -Feature of muscle spasm  -Multiple schmorls noted and suggetsive limbus veterba at anterio-superior corner of L4 .  -Mild diffuse disc bulges noted at L4/L5 and L5/S1 levels, mild compressing tehcal sac and both nerve roots .",0,1
338,local_data/mendeley/T1_0105_D5.png,"LSS MRI :  -Feature of muscle spasm  -Multiple schmorls noted and suggetsive limbus veterba at anterio-superior corner of L4 .  -Mild diffuse disc bulges noted at L4/L5 and L5/S1 levels, mild compressing tehcal sac and both nerve roots .",0,1
339,local_data/mendeley/T2_0105_D3.png,"LSS MRI :  -Feature of muscle spasm  -Multiple schmorls noted and suggetsive limbus veterba at anterio-superior corner of L4 .  -Mild diffuse disc bulges noted at L4/L5 and L5/S1 levels, mild compressing tehcal sac and both nerve roots .",0,1
340,local_data/mendeley/T2_0105_D4.png,"LSS MRI :  -Feature of muscle spasm  -Multiple schmorls noted and suggetsive limbus veterba at anterio-superior corner of L4 .  -Mild diffuse disc bulges noted at L4/L5 and L5/S1 levels, mild compressing tehcal sac and both nerve roots .",0,1
341,local_data/mendeley/T2_0105_D5.png,"LSS MRI :  -Feature of muscle spasm  -Multiple schmorls noted and suggetsive limbus veterba at anterio-superior corner of L4 .  -Mild diffuse disc bulges noted at L4/L5 and L5/S1 levels, mild compressing tehcal sac and both nerve roots .",0,1
342,local_data/mendeley/T1_0106_D3.png,LSS MRI :  Feature of muscle spasm.  No thecal sac or nerve root compression noted,0,1
343,local_data/mendeley/T1_0106_D4.png,LSS MRI :  Feature of muscle spasm.  No thecal sac or nerve root compression noted,0,1
344,local_data/mendeley/T1_0106_D5.png,LSS MRI :  Feature of muscle spasm.  No thecal sac or nerve root compression noted,0,1
345,local_data/mendeley/T2_0106_D3.png,LSS MRI :  Feature of muscle spasm.  No thecal sac or nerve root compression noted,0,1
346,local_data/mendeley/T2_0106_D4.png,LSS MRI :  Feature of muscle spasm.  No thecal sac or nerve root compression noted,0,1
347,local_data/mendeley/T2_0106_D5.png,LSS MRI :  Feature of muscle spasm.  No thecal sac or nerve root compression noted,0,1
348,local_data/mendeley/T1_0107_D3.png,"LSS MRI :  Feature of muscle spasm.  wide base disc bulges noted at L3/L4 level, compressing thecal sac and encroaching neural exit canals.  Diffuse disc bulges noted at L4/L5 level,mild compressing thecal sac and l exit nerve roots.",0,1
349,local_data/mendeley/T1_0107_D4.png,"LSS MRI :  Feature of muscle spasm.  wide base disc bulges noted at L3/L4 level, compressing thecal sac and encroaching neural exit canals.  Diffuse disc bulges noted at L4/L5 level,mild compressing thecal sac and l exit nerve roots.",0,1
350,local_data/mendeley/T1_0107_D5.png,"LSS MRI :  Feature of muscle spasm.  wide base disc bulges noted at L3/L4 level, compressing thecal sac and encroaching neural exit canals.  Diffuse disc bulges noted at L4/L5 level,mild compressing thecal sac and l exit nerve roots.",0,1
351,local_data/mendeley/T2_0107_D3.png,"LSS MRI :  Feature of muscle spasm.  wide base disc bulges noted at L3/L4 level, compressing thecal sac and encroaching neural exit canals.  Diffuse disc bulges noted at L4/L5 level,mild compressing thecal sac and l exit nerve roots.",0,1
352,local_data/mendeley/T2_0107_D4.png,"LSS MRI :  Feature of muscle spasm.  wide base disc bulges noted at L3/L4 level, compressing thecal sac and encroaching neural exit canals.  Diffuse disc bulges noted at L4/L5 level,mild compressing thecal sac and l exit nerve roots.",0,1
353,local_data/mendeley/T2_0107_D5.png,"LSS MRI :  Feature of muscle spasm.  wide base disc bulges noted at L3/L4 level, compressing thecal sac and encroaching neural exit canals.  Diffuse disc bulges noted at L4/L5 level,mild compressing thecal sac and l exit nerve roots.",0,1
354,local_data/mendeley/T1_0108_D3.png,L4-L5: Central disc protrusion noted compressing the thecal sac.,0,1
355,local_data/mendeley/T1_0108_D4.png,L4-L5: Central disc protrusion noted compressing the thecal sac.,0,1
356,local_data/mendeley/T1_0108_D5.png,L4-L5: Central disc protrusion noted compressing the thecal sac.,0,1
357,local_data/mendeley/T2_0108_D3.png,L4-L5: Central disc protrusion noted compressing the thecal sac.,0,1
358,local_data/mendeley/T2_0108_D4.png,L4-L5: Central disc protrusion noted compressing the thecal sac.,0,1
359,local_data/mendeley/T2_0108_D5.png,L4-L5: Central disc protrusion noted compressing the thecal sac.,0,1
360,local_data/mendeley/T1_0109_D3.png,"LSS MRI : -Feature of muscle spasm. -Grade I spondylolisthesis of L4 over L5 level -Diffuse disc bulge noted at L4/L5 level,largely  compressing the thecal sac extending to lateral recess , associated with ligametum flavum hyperatrophy  with encroaching exit neural canals,  all leading to secandary spinal canal stenosis ,as well as dehydrated disc materials",0,1
361,local_data/mendeley/T1_0109_D4.png,"LSS MRI : -Feature of muscle spasm. -Grade I spondylolisthesis of L4 over L5 level -Diffuse disc bulge noted at L4/L5 level,largely  compressing the thecal sac extending to lateral recess , associated with ligametum flavum hyperatrophy  with encroaching exit neural canals,  all leading to secandary spinal canal stenosis ,as well as dehydrated disc materials",0,1
362,local_data/mendeley/T1_0109_D5.png,"LSS MRI : -Feature of muscle spasm. -Grade I spondylolisthesis of L4 over L5 level -Diffuse disc bulge noted at L4/L5 level,largely  compressing the thecal sac extending to lateral recess , associated with ligametum flavum hyperatrophy  with encroaching exit neural canals,  all leading to secandary spinal canal stenosis ,as well as dehydrated disc materials",0,1
363,local_data/mendeley/T2_0109_D3.png,"LSS MRI : -Feature of muscle spasm. -Grade I spondylolisthesis of L4 over L5 level -Diffuse disc bulge noted at L4/L5 level,largely  compressing the thecal sac extending to lateral recess , associated with ligametum flavum hyperatrophy  with encroaching exit neural canals,  all leading to secandary spinal canal stenosis ,as well as dehydrated disc materials",0,1
364,local_data/mendeley/T2_0109_D4.png,"LSS MRI : -Feature of muscle spasm. -Grade I spondylolisthesis of L4 over L5 level -Diffuse disc bulge noted at L4/L5 level,largely  compressing the thecal sac extending to lateral recess , associated with ligametum flavum hyperatrophy  with encroaching exit neural canals,  all leading to secandary spinal canal stenosis ,as well as dehydrated disc materials",0,1
365,local_data/mendeley/T2_0109_D5.png,"LSS MRI : -Feature of muscle spasm. -Grade I spondylolisthesis of L4 over L5 level -Diffuse disc bulge noted at L4/L5 level,largely  compressing the thecal sac extending to lateral recess , associated with ligametum flavum hyperatrophy  with encroaching exit neural canals,  all leading to secandary spinal canal stenosis ,as well as dehydrated disc materials",0,1
366,local_data/mendeley/T1_0112_D3.png,"L5-S1: Mild RT PARACENTRAL DISC PROTRUSION NOTED, ABUTTING THE THECAL SAC. L4-L5 and L3-L4: Mild disc bulges noted.",0,1
367,local_data/mendeley/T1_0112_D4.png,"L5-S1: Mild RT PARACENTRAL DISC PROTRUSION NOTED, ABUTTING THE THECAL SAC. L4-L5 and L3-L4: Mild disc bulges noted.",0,1
368,local_data/mendeley/T1_0112_D5.png,"L5-S1: Mild RT PARACENTRAL DISC PROTRUSION NOTED, ABUTTING THE THECAL SAC. L4-L5 and L3-L4: Mild disc bulges noted.",0,1
369,local_data/mendeley/T2_0112_D3.png,"L5-S1: Mild RT PARACENTRAL DISC PROTRUSION NOTED, ABUTTING THE THECAL SAC. L4-L5 and L3-L4: Mild disc bulges noted.",0,1
370,local_data/mendeley/T2_0112_D4.png,"L5-S1: Mild RT PARACENTRAL DISC PROTRUSION NOTED, ABUTTING THE THECAL SAC. L4-L5 and L3-L4: Mild disc bulges noted.",0,1
371,local_data/mendeley/T2_0112_D5.png,"L5-S1: Mild RT PARACENTRAL DISC PROTRUSION NOTED, ABUTTING THE THECAL SAC. L4-L5 and L3-L4: Mild disc bulges noted.",0,1
372,local_data/mendeley/T1_0113_D3.png,Lumbosacral MRI  Mild disc bulge noted at L5-S1 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,0,1
373,local_data/mendeley/T1_0113_D4.png,Lumbosacral MRI  Mild disc bulge noted at L5-S1 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,0,1
374,local_data/mendeley/T1_0113_D5.png,Lumbosacral MRI  Mild disc bulge noted at L5-S1 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,0,1
375,local_data/mendeley/T2_0113_D3.png,Lumbosacral MRI  Mild disc bulge noted at L5-S1 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,0,1
376,local_data/mendeley/T2_0113_D4.png,Lumbosacral MRI  Mild disc bulge noted at L5-S1 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,0,1
377,local_data/mendeley/T2_0113_D5.png,Lumbosacral MRI  Mild disc bulge noted at L5-S1 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,0,1
378,local_data/mendeley/T1_0115_D3.png,"LSS MRI  multilevel diffuse disc bulges (L4-S1) compressing the thecal sac, noticeably at L4-L5 where the Lt exit canal also appeared compressed",0,1
379,local_data/mendeley/T1_0115_D4.png,"LSS MRI  multilevel diffuse disc bulges (L4-S1) compressing the thecal sac, noticeably at L4-L5 where the Lt exit canal also appeared compressed",0,1
380,local_data/mendeley/T1_0115_D5.png,"LSS MRI  multilevel diffuse disc bulges (L4-S1) compressing the thecal sac, noticeably at L4-L5 where the Lt exit canal also appeared compressed",0,1
381,local_data/mendeley/T2_0115_D3.png,"LSS MRI  multilevel diffuse disc bulges (L4-S1) compressing the thecal sac, noticeably at L4-L5 where the Lt exit canal also appeared compressed",0,1
382,local_data/mendeley/T2_0115_D4.png,"LSS MRI  multilevel diffuse disc bulges (L4-S1) compressing the thecal sac, noticeably at L4-L5 where the Lt exit canal also appeared compressed",0,1
383,local_data/mendeley/T2_0115_D5.png,"LSS MRI  multilevel diffuse disc bulges (L4-S1) compressing the thecal sac, noticeably at L4-L5 where the Lt exit canal also appeared compressed",0,1
384,local_data/mendeley/T1_0116_D3.png,"LSS MRI Previous surgery noted with transpedicle fixation noted at L3-L4 level. Normal bony alignment. adequate spinal canal with no significant thecal sac or nerve root compression. enhancing posterior parts of L4-L5 disc with ioncreased T2 signal, probably discitis.",0,1
385,local_data/mendeley/T1_0116_D4.png,"LSS MRI Previous surgery noted with transpedicle fixation noted at L3-L4 level. Normal bony alignment. adequate spinal canal with no significant thecal sac or nerve root compression. enhancing posterior parts of L4-L5 disc with ioncreased T2 signal, probably discitis.",0,1
386,local_data/mendeley/T1_0116_D5.png,"LSS MRI Previous surgery noted with transpedicle fixation noted at L3-L4 level. Normal bony alignment. adequate spinal canal with no significant thecal sac or nerve root compression. enhancing posterior parts of L4-L5 disc with ioncreased T2 signal, probably discitis.",0,1
387,local_data/mendeley/T2_0116_D3.png,"LSS MRI Previous surgery noted with transpedicle fixation noted at L3-L4 level. Normal bony alignment. adequate spinal canal with no significant thecal sac or nerve root compression. enhancing posterior parts of L4-L5 disc with ioncreased T2 signal, probably discitis.",0,1
388,local_data/mendeley/T2_0116_D4.png,"LSS MRI Previous surgery noted with transpedicle fixation noted at L3-L4 level. Normal bony alignment. adequate spinal canal with no significant thecal sac or nerve root compression. enhancing posterior parts of L4-L5 disc with ioncreased T2 signal, probably discitis.",0,1
389,local_data/mendeley/T2_0116_D5.png,"LSS MRI Previous surgery noted with transpedicle fixation noted at L3-L4 level. Normal bony alignment. adequate spinal canal with no significant thecal sac or nerve root compression. enhancing posterior parts of L4-L5 disc with ioncreased T2 signal, probably discitis.",0,1
390,local_data/mendeley/T1_0118_D3.png,"L3-L4: Mild disc bulge noted L4-L5: broad based, central and Lt paracentral disc protrusion, compressing the thecal sac and Lt exit neural canal. L5-S1: Central disc protrusion, compressing the thecal sa",0,1
391,local_data/mendeley/T1_0118_D4.png,"L3-L4: Mild disc bulge noted L4-L5: broad based, central and Lt paracentral disc protrusion, compressing the thecal sac and Lt exit neural canal. L5-S1: Central disc protrusion, compressing the thecal sa",0,1
392,local_data/mendeley/T1_0118_D5.png,"L3-L4: Mild disc bulge noted L4-L5: broad based, central and Lt paracentral disc protrusion, compressing the thecal sac and Lt exit neural canal. L5-S1: Central disc protrusion, compressing the thecal sa",0,1
393,local_data/mendeley/T2_0118_D3.png,"L3-L4: Mild disc bulge noted L4-L5: broad based, central and Lt paracentral disc protrusion, compressing the thecal sac and Lt exit neural canal. L5-S1: Central disc protrusion, compressing the thecal sa",0,1
394,local_data/mendeley/T2_0118_D4.png,"L3-L4: Mild disc bulge noted L4-L5: broad based, central and Lt paracentral disc protrusion, compressing the thecal sac and Lt exit neural canal. L5-S1: Central disc protrusion, compressing the thecal sa",0,1
395,local_data/mendeley/T2_0118_D5.png,"L3-L4: Mild disc bulge noted L4-L5: broad based, central and Lt paracentral disc protrusion, compressing the thecal sac and Lt exit neural canal. L5-S1: Central disc protrusion, compressing the thecal sa",0,1
396,local_data/mendeley/T1_0119_D3.png,"LSS MRI  L3 and L2 hemangiomas.  diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.  mild disc bulge noted at L3-L4 level, compressing the thecal sac and encroaching upon the Lt exit canal.",0,1
397,local_data/mendeley/T1_0119_D4.png,"LSS MRI  L3 and L2 hemangiomas.  diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.  mild disc bulge noted at L3-L4 level, compressing the thecal sac and encroaching upon the Lt exit canal.",0,1
398,local_data/mendeley/T1_0119_D5.png,"LSS MRI  L3 and L2 hemangiomas.  diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.  mild disc bulge noted at L3-L4 level, compressing the thecal sac and encroaching upon the Lt exit canal.",0,1
399,local_data/mendeley/T2_0119_D3.png,"LSS MRI  L3 and L2 hemangiomas.  diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.  mild disc bulge noted at L3-L4 level, compressing the thecal sac and encroaching upon the Lt exit canal.",0,1
400,local_data/mendeley/T2_0119_D4.png,"LSS MRI  L3 and L2 hemangiomas.  diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.  mild disc bulge noted at L3-L4 level, compressing the thecal sac and encroaching upon the Lt exit canal.",0,1
401,local_data/mendeley/T2_0119_D5.png,"LSS MRI  L3 and L2 hemangiomas.  diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.  mild disc bulge noted at L3-L4 level, compressing the thecal sac and encroaching upon the Lt exit canal.",0,1
402,local_data/mendeley/T1_0120_D3.png,L4-L5: Rt posterolateral disc protrusion with annular tear compressing Rt exit neural canal.,0,1
403,local_data/mendeley/T1_0120_D4.png,L4-L5: Rt posterolateral disc protrusion with annular tear compressing Rt exit neural canal.,0,1
404,local_data/mendeley/T1_0120_D5.png,L4-L5: Rt posterolateral disc protrusion with annular tear compressing Rt exit neural canal.,0,1
405,local_data/mendeley/T2_0120_D3.png,L4-L5: Rt posterolateral disc protrusion with annular tear compressing Rt exit neural canal.,0,1
406,local_data/mendeley/T2_0120_D4.png,L4-L5: Rt posterolateral disc protrusion with annular tear compressing Rt exit neural canal.,0,1
407,local_data/mendeley/T2_0120_D5.png,L4-L5: Rt posterolateral disc protrusion with annular tear compressing Rt exit neural canal.,0,1
408,local_data/mendeley/T1_0121_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, mildly compressing the thecal sac and exit nerve roots",0,1
409,local_data/mendeley/T1_0121_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, mildly compressing the thecal sac and exit nerve roots",0,1
410,local_data/mendeley/T1_0121_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, mildly compressing the thecal sac and exit nerve roots",0,1
411,local_data/mendeley/T2_0121_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, mildly compressing the thecal sac and exit nerve roots",0,1
412,local_data/mendeley/T2_0121_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, mildly compressing the thecal sac and exit nerve roots",0,1
413,local_data/mendeley/T2_0121_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, mildly compressing the thecal sac and exit nerve roots",0,1
414,local_data/mendeley/T1_0122_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
415,local_data/mendeley/T1_0122_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
416,local_data/mendeley/T1_0122_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
417,local_data/mendeley/T2_0122_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
418,local_data/mendeley/T2_0122_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
419,local_data/mendeley/T2_0122_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
420,local_data/mendeley/T1_0123_D3.png,"LSS MRI :  fea ture of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing thecal sac extending to lateral recess , with encroaching exit neural canals, as well as dehydrated disc materials",0,1
421,local_data/mendeley/T1_0123_D4.png,"LSS MRI :  fea ture of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing thecal sac extending to lateral recess , with encroaching exit neural canals, as well as dehydrated disc materials",0,1
422,local_data/mendeley/T1_0123_D5.png,"LSS MRI :  fea ture of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing thecal sac extending to lateral recess , with encroaching exit neural canals, as well as dehydrated disc materials",0,1
423,local_data/mendeley/T2_0123_D3.png,"LSS MRI :  fea ture of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing thecal sac extending to lateral recess , with encroaching exit neural canals, as well as dehydrated disc materials",0,1
424,local_data/mendeley/T2_0123_D4.png,"LSS MRI :  fea ture of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing thecal sac extending to lateral recess , with encroaching exit neural canals, as well as dehydrated disc materials",0,1
425,local_data/mendeley/T2_0123_D5.png,"LSS MRI :  fea ture of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing thecal sac extending to lateral recess , with encroaching exit neural canals, as well as dehydrated disc materials",0,1
426,local_data/mendeley/T1_0124_D3.png,LSS MRI :  feature of muscle spasm.  Dissicating disc materail at lower level.  Lt foraminal disc protrusion noted at L5/S1 level compressing thecal sac and Lt exit nerve root (S1).  Diffuse disc bulges noted at L4/L5 level mild compressing thecal sac and both nerve roots .,0,1
427,local_data/mendeley/T1_0124_D4.png,LSS MRI :  feature of muscle spasm.  Dissicating disc materail at lower level.  Lt foraminal disc protrusion noted at L5/S1 level compressing thecal sac and Lt exit nerve root (S1).  Diffuse disc bulges noted at L4/L5 level mild compressing thecal sac and both nerve roots .,0,1
428,local_data/mendeley/T1_0124_D5.png,LSS MRI :  feature of muscle spasm.  Dissicating disc materail at lower level.  Lt foraminal disc protrusion noted at L5/S1 level compressing thecal sac and Lt exit nerve root (S1).  Diffuse disc bulges noted at L4/L5 level mild compressing thecal sac and both nerve roots .,0,1
429,local_data/mendeley/T2_0124_D3.png,LSS MRI :  feature of muscle spasm.  Dissicating disc materail at lower level.  Lt foraminal disc protrusion noted at L5/S1 level compressing thecal sac and Lt exit nerve root (S1).  Diffuse disc bulges noted at L4/L5 level mild compressing thecal sac and both nerve roots .,0,1
430,local_data/mendeley/T2_0124_D4.png,LSS MRI :  feature of muscle spasm.  Dissicating disc materail at lower level.  Lt foraminal disc protrusion noted at L5/S1 level compressing thecal sac and Lt exit nerve root (S1).  Diffuse disc bulges noted at L4/L5 level mild compressing thecal sac and both nerve roots .,0,1
431,local_data/mendeley/T2_0124_D5.png,LSS MRI :  feature of muscle spasm.  Dissicating disc materail at lower level.  Lt foraminal disc protrusion noted at L5/S1 level compressing thecal sac and Lt exit nerve root (S1).  Diffuse disc bulges noted at L4/L5 level mild compressing thecal sac and both nerve roots .,0,1
432,local_data/mendeley/T1_0125_D3.png,"LSS MRI:  Lt paracentral ,wide base ,disc bulge noted at L5/S1 level, compressing the thecal sac and exit neural canal.",0,1
433,local_data/mendeley/T1_0125_D4.png,"LSS MRI:  Lt paracentral ,wide base ,disc bulge noted at L5/S1 level, compressing the thecal sac and exit neural canal.",0,1
434,local_data/mendeley/T1_0125_D5.png,"LSS MRI:  Lt paracentral ,wide base ,disc bulge noted at L5/S1 level, compressing the thecal sac and exit neural canal.",0,1
435,local_data/mendeley/T2_0125_D3.png,"LSS MRI:  Lt paracentral ,wide base ,disc bulge noted at L5/S1 level, compressing the thecal sac and exit neural canal.",0,1
436,local_data/mendeley/T2_0125_D4.png,"LSS MRI:  Lt paracentral ,wide base ,disc bulge noted at L5/S1 level, compressing the thecal sac and exit neural canal.",0,1
437,local_data/mendeley/T2_0125_D5.png,"LSS MRI:  Lt paracentral ,wide base ,disc bulge noted at L5/S1 level, compressing the thecal sac and exit neural canal.",0,1
438,local_data/mendeley/T1_0127_D3.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit neural canals.,0,1
439,local_data/mendeley/T1_0127_D4.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit neural canals.,0,1
440,local_data/mendeley/T1_0127_D5.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit neural canals.,0,1
441,local_data/mendeley/T2_0127_D3.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit neural canals.,0,1
442,local_data/mendeley/T2_0127_D4.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit neural canals.,0,1
443,local_data/mendeley/T2_0127_D5.png,L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit neural canals.,0,1
444,local_data/mendeley/T1_0128_D3.png,LSS MRI  small central disc protrusion noted at L5-S1 level.  diffuse disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
445,local_data/mendeley/T1_0128_D4.png,LSS MRI  small central disc protrusion noted at L5-S1 level.  diffuse disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
446,local_data/mendeley/T1_0128_D5.png,LSS MRI  small central disc protrusion noted at L5-S1 level.  diffuse disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
447,local_data/mendeley/T2_0128_D3.png,LSS MRI  small central disc protrusion noted at L5-S1 level.  diffuse disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
448,local_data/mendeley/T2_0128_D4.png,LSS MRI  small central disc protrusion noted at L5-S1 level.  diffuse disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
449,local_data/mendeley/T2_0128_D5.png,LSS MRI  small central disc protrusion noted at L5-S1 level.  diffuse disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
450,local_data/mendeley/T1_0129_D3.png,L3-L4 and L4-L5: diffuse disc bulges compressing the thecal sac and exit canals. L5-S1: Mild disc bulge noted abutting the thecal sac and S1 nerve roots.,0,1
451,local_data/mendeley/T1_0129_D4.png,L3-L4 and L4-L5: diffuse disc bulges compressing the thecal sac and exit canals. L5-S1: Mild disc bulge noted abutting the thecal sac and S1 nerve roots.,0,1
452,local_data/mendeley/T1_0129_D5.png,L3-L4 and L4-L5: diffuse disc bulges compressing the thecal sac and exit canals. L5-S1: Mild disc bulge noted abutting the thecal sac and S1 nerve roots.,0,1
453,local_data/mendeley/T2_0129_D3.png,L3-L4 and L4-L5: diffuse disc bulges compressing the thecal sac and exit canals. L5-S1: Mild disc bulge noted abutting the thecal sac and S1 nerve roots.,0,1
454,local_data/mendeley/T2_0129_D4.png,L3-L4 and L4-L5: diffuse disc bulges compressing the thecal sac and exit canals. L5-S1: Mild disc bulge noted abutting the thecal sac and S1 nerve roots.,0,1
455,local_data/mendeley/T2_0129_D5.png,L3-L4 and L4-L5: diffuse disc bulges compressing the thecal sac and exit canals. L5-S1: Mild disc bulge noted abutting the thecal sac and S1 nerve roots.,0,1
456,local_data/mendeley/T1_0131_D3.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  adequate spinal canal.,0,1
457,local_data/mendeley/T1_0131_D4.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  adequate spinal canal.,0,1
458,local_data/mendeley/T1_0131_D5.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  adequate spinal canal.,0,1
459,local_data/mendeley/T2_0131_D3.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  adequate spinal canal.,0,1
460,local_data/mendeley/T2_0131_D4.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  adequate spinal canal.,0,1
461,local_data/mendeley/T2_0131_D5.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  adequate spinal canal.,0,1
462,local_data/mendeley/T1_0132_D3.png,LSS MRI  Rt paracentral disc protrusion noted compressing the thecal sac and Rt S1 nerve root.,0,1
463,local_data/mendeley/T1_0132_D4.png,LSS MRI  Rt paracentral disc protrusion noted compressing the thecal sac and Rt S1 nerve root.,0,1
464,local_data/mendeley/T1_0132_D5.png,LSS MRI  Rt paracentral disc protrusion noted compressing the thecal sac and Rt S1 nerve root.,0,1
465,local_data/mendeley/T2_0132_D3.png,LSS MRI  Rt paracentral disc protrusion noted compressing the thecal sac and Rt S1 nerve root.,0,1
466,local_data/mendeley/T2_0132_D4.png,LSS MRI  Rt paracentral disc protrusion noted compressing the thecal sac and Rt S1 nerve root.,0,1
467,local_data/mendeley/T2_0132_D5.png,LSS MRI  Rt paracentral disc protrusion noted compressing the thecal sac and Rt S1 nerve root.,0,1
468,local_data/mendeley/T1_0133_D3.png,"LSS MRI :  Feature of muscle spasm.  No thecal sac or exit nerev roots compression seen .  No disc herniation or protrusion noted .  NB :  pelvic kidney noted ,correlate with US .",0,1
469,local_data/mendeley/T1_0133_D4.png,"LSS MRI :  Feature of muscle spasm.  No thecal sac or exit nerev roots compression seen .  No disc herniation or protrusion noted .  NB :  pelvic kidney noted ,correlate with US .",0,1
470,local_data/mendeley/T1_0133_D5.png,"LSS MRI :  Feature of muscle spasm.  No thecal sac or exit nerev roots compression seen .  No disc herniation or protrusion noted .  NB :  pelvic kidney noted ,correlate with US .",0,1
471,local_data/mendeley/T2_0133_D3.png,"LSS MRI :  Feature of muscle spasm.  No thecal sac or exit nerev roots compression seen .  No disc herniation or protrusion noted .  NB :  pelvic kidney noted ,correlate with US .",0,1
472,local_data/mendeley/T2_0133_D4.png,"LSS MRI :  Feature of muscle spasm.  No thecal sac or exit nerev roots compression seen .  No disc herniation or protrusion noted .  NB :  pelvic kidney noted ,correlate with US .",0,1
473,local_data/mendeley/T2_0133_D5.png,"LSS MRI :  Feature of muscle spasm.  No thecal sac or exit nerev roots compression seen .  No disc herniation or protrusion noted .  NB :  pelvic kidney noted ,correlate with US .",0,1
474,local_data/mendeley/T1_0134_D3.png,"LSS MRI :  Feature of muscle spasm.  mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots",0,1
475,local_data/mendeley/T1_0134_D4.png,"LSS MRI :  Feature of muscle spasm.  mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots",0,1
476,local_data/mendeley/T1_0134_D5.png,"LSS MRI :  Feature of muscle spasm.  mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots",0,1
477,local_data/mendeley/T2_0134_D3.png,"LSS MRI :  Feature of muscle spasm.  mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots",0,1
478,local_data/mendeley/T2_0134_D4.png,"LSS MRI :  Feature of muscle spasm.  mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots",0,1
479,local_data/mendeley/T2_0134_D5.png,"LSS MRI :  Feature of muscle spasm.  mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots",0,1
480,local_data/mendeley/T1_0136_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mildly compressing thecalsac and nerve roots .",0,1
481,local_data/mendeley/T1_0136_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mildly compressing thecalsac and nerve roots .",0,1
482,local_data/mendeley/T1_0136_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mildly compressing thecalsac and nerve roots .",0,1
483,local_data/mendeley/T2_0136_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mildly compressing thecalsac and nerve roots .",0,1
484,local_data/mendeley/T2_0136_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mildly compressing thecalsac and nerve roots .",0,1
485,local_data/mendeley/T2_0136_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mildly compressing thecalsac and nerve roots .",0,1
486,local_data/mendeley/T1_0137_D3.png,MRI OF THE C. SPINE  -Features of muscle spasm noted.  -Multilevel dissecating disc materials.  -Wide base disc bulges extending more to Rt side noted at C4/C5 level with obliterating anterior CSF spaces and intending spinal cord and encroaching exit neural canal more to the Rt  -Multilevel small posterior central disc protrusion noted obliterating anterior CSF spaces with annular tears. .  MRI OF THE LSS  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1 .  -Dissecating disc material L5/S1  -L5-S1 level : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
487,local_data/mendeley/T1_0137_D4.png,MRI OF THE C. SPINE  -Features of muscle spasm noted.  -Multilevel dissecating disc materials.  -Wide base disc bulges extending more to Rt side noted at C4/C5 level with obliterating anterior CSF spaces and intending spinal cord and encroaching exit neural canal more to the Rt  -Multilevel small posterior central disc protrusion noted obliterating anterior CSF spaces with annular tears. .  MRI OF THE LSS  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1 .  -Dissecating disc material L5/S1  -L5-S1 level : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
488,local_data/mendeley/T1_0137_D5.png,MRI OF THE C. SPINE  -Features of muscle spasm noted.  -Multilevel dissecating disc materials.  -Wide base disc bulges extending more to Rt side noted at C4/C5 level with obliterating anterior CSF spaces and intending spinal cord and encroaching exit neural canal more to the Rt  -Multilevel small posterior central disc protrusion noted obliterating anterior CSF spaces with annular tears. .  MRI OF THE LSS  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1 .  -Dissecating disc material L5/S1  -L5-S1 level : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
489,local_data/mendeley/T2_0137_D3.png,MRI OF THE C. SPINE  -Features of muscle spasm noted.  -Multilevel dissecating disc materials.  -Wide base disc bulges extending more to Rt side noted at C4/C5 level with obliterating anterior CSF spaces and intending spinal cord and encroaching exit neural canal more to the Rt  -Multilevel small posterior central disc protrusion noted obliterating anterior CSF spaces with annular tears. .  MRI OF THE LSS  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1 .  -Dissecating disc material L5/S1  -L5-S1 level : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
490,local_data/mendeley/T2_0137_D4.png,MRI OF THE C. SPINE  -Features of muscle spasm noted.  -Multilevel dissecating disc materials.  -Wide base disc bulges extending more to Rt side noted at C4/C5 level with obliterating anterior CSF spaces and intending spinal cord and encroaching exit neural canal more to the Rt  -Multilevel small posterior central disc protrusion noted obliterating anterior CSF spaces with annular tears. .  MRI OF THE LSS  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1 .  -Dissecating disc material L5/S1  -L5-S1 level : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
491,local_data/mendeley/T2_0137_D5.png,MRI OF THE C. SPINE  -Features of muscle spasm noted.  -Multilevel dissecating disc materials.  -Wide base disc bulges extending more to Rt side noted at C4/C5 level with obliterating anterior CSF spaces and intending spinal cord and encroaching exit neural canal more to the Rt  -Multilevel small posterior central disc protrusion noted obliterating anterior CSF spaces with annular tears. .  MRI OF THE LSS  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1 .  -Dissecating disc material L5/S1  -L5-S1 level : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
492,local_data/mendeley/T1_0139_D3.png,"LSS MRI  Small Lt paracentral protrusion noted at L5-S1 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.",0,1
493,local_data/mendeley/T1_0139_D4.png,"LSS MRI  Small Lt paracentral protrusion noted at L5-S1 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.",0,1
494,local_data/mendeley/T1_0139_D5.png,"LSS MRI  Small Lt paracentral protrusion noted at L5-S1 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.",0,1
495,local_data/mendeley/T2_0139_D3.png,"LSS MRI  Small Lt paracentral protrusion noted at L5-S1 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.",0,1
496,local_data/mendeley/T2_0139_D4.png,"LSS MRI  Small Lt paracentral protrusion noted at L5-S1 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.",0,1
497,local_data/mendeley/T2_0139_D5.png,"LSS MRI  Small Lt paracentral protrusion noted at L5-S1 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.",0,1
498,local_data/mendeley/T1_0143_D3.png,LSS MRI : Feature of muscle spasm No disc protrusion or herniation noted . No significant thecal sac or nerve roots compression seen .,0,1
499,local_data/mendeley/T1_0143_D4.png,LSS MRI : Feature of muscle spasm No disc protrusion or herniation noted . No significant thecal sac or nerve roots compression seen .,0,1
500,local_data/mendeley/T1_0143_D5.png,LSS MRI : Feature of muscle spasm No disc protrusion or herniation noted . No significant thecal sac or nerve roots compression seen .,0,1
501,local_data/mendeley/T2_0143_D3.png,LSS MRI : Feature of muscle spasm No disc protrusion or herniation noted . No significant thecal sac or nerve roots compression seen .,0,1
502,local_data/mendeley/T2_0143_D4.png,LSS MRI : Feature of muscle spasm No disc protrusion or herniation noted . No significant thecal sac or nerve roots compression seen .,0,1
503,local_data/mendeley/T2_0143_D5.png,LSS MRI : Feature of muscle spasm No disc protrusion or herniation noted . No significant thecal sac or nerve roots compression seen .,0,1
504,local_data/mendeley/T1_0144_D3.png,"LSS MRI : feature of muscle spasm. posterior central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching exit neural canals. Small Rt paracentral disc protrusion noted at D11/D12 level, mildy compressing the thecal sac .",0,1
505,local_data/mendeley/T1_0144_D4.png,"LSS MRI : feature of muscle spasm. posterior central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching exit neural canals. Small Rt paracentral disc protrusion noted at D11/D12 level, mildy compressing the thecal sac .",0,1
506,local_data/mendeley/T1_0144_D5.png,"LSS MRI : feature of muscle spasm. posterior central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching exit neural canals. Small Rt paracentral disc protrusion noted at D11/D12 level, mildy compressing the thecal sac .",0,1
507,local_data/mendeley/T2_0144_D3.png,"LSS MRI : feature of muscle spasm. posterior central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching exit neural canals. Small Rt paracentral disc protrusion noted at D11/D12 level, mildy compressing the thecal sac .",0,1
508,local_data/mendeley/T2_0144_D4.png,"LSS MRI : feature of muscle spasm. posterior central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching exit neural canals. Small Rt paracentral disc protrusion noted at D11/D12 level, mildy compressing the thecal sac .",0,1
509,local_data/mendeley/T2_0144_D5.png,"LSS MRI : feature of muscle spasm. posterior central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching exit neural canals. Small Rt paracentral disc protrusion noted at D11/D12 level, mildy compressing the thecal sac .",0,1
510,local_data/mendeley/T1_0145_D3.png,"LSS MRI  -Feature of muscle spasm.  -Small central disc protrusion , noted at D12/L1 level, obliterating anterior CSF space and indenting spinal cord.  -Partial Dissecating disc material noted at lower levels.  -Wide base disc bulges noted at L4-L5 levels compressing the thecal sac and exit neural canals  -Mild Diffuse disc bulges noted at L5-S1 levels mild compressing the thecal sac and exit neural canals with annular tear.",0,1
511,local_data/mendeley/T1_0145_D4.png,"LSS MRI  -Feature of muscle spasm.  -Small central disc protrusion , noted at D12/L1 level, obliterating anterior CSF space and indenting spinal cord.  -Partial Dissecating disc material noted at lower levels.  -Wide base disc bulges noted at L4-L5 levels compressing the thecal sac and exit neural canals  -Mild Diffuse disc bulges noted at L5-S1 levels mild compressing the thecal sac and exit neural canals with annular tear.",0,1
512,local_data/mendeley/T1_0145_D5.png,"LSS MRI  -Feature of muscle spasm.  -Small central disc protrusion , noted at D12/L1 level, obliterating anterior CSF space and indenting spinal cord.  -Partial Dissecating disc material noted at lower levels.  -Wide base disc bulges noted at L4-L5 levels compressing the thecal sac and exit neural canals  -Mild Diffuse disc bulges noted at L5-S1 levels mild compressing the thecal sac and exit neural canals with annular tear.",0,1
513,local_data/mendeley/T2_0145_D3.png,"LSS MRI  -Feature of muscle spasm.  -Small central disc protrusion , noted at D12/L1 level, obliterating anterior CSF space and indenting spinal cord.  -Partial Dissecating disc material noted at lower levels.  -Wide base disc bulges noted at L4-L5 levels compressing the thecal sac and exit neural canals  -Mild Diffuse disc bulges noted at L5-S1 levels mild compressing the thecal sac and exit neural canals with annular tear.",0,1
514,local_data/mendeley/T2_0145_D4.png,"LSS MRI  -Feature of muscle spasm.  -Small central disc protrusion , noted at D12/L1 level, obliterating anterior CSF space and indenting spinal cord.  -Partial Dissecating disc material noted at lower levels.  -Wide base disc bulges noted at L4-L5 levels compressing the thecal sac and exit neural canals  -Mild Diffuse disc bulges noted at L5-S1 levels mild compressing the thecal sac and exit neural canals with annular tear.",0,1
515,local_data/mendeley/T2_0145_D5.png,"LSS MRI  -Feature of muscle spasm.  -Small central disc protrusion , noted at D12/L1 level, obliterating anterior CSF space and indenting spinal cord.  -Partial Dissecating disc material noted at lower levels.  -Wide base disc bulges noted at L4-L5 levels compressing the thecal sac and exit neural canals  -Mild Diffuse disc bulges noted at L5-S1 levels mild compressing the thecal sac and exit neural canals with annular tear.",0,1
516,local_data/mendeley/T1_0146_D3.png,"LSS MRI : Feature of muscle spasm with schmorls nodules . Posterio-central disc protrusion noted at L5/S1 level, compressing thecal sac and encroaching exit nerve canal  with annular tear and dehtdrated disc material . Diffuse disc bulges note at D12/L1 level, obliterating anterior CSF spaces .",0,1
517,local_data/mendeley/T1_0146_D4.png,"LSS MRI : Feature of muscle spasm with schmorls nodules . Posterio-central disc protrusion noted at L5/S1 level, compressing thecal sac and encroaching exit nerve canal  with annular tear and dehtdrated disc material . Diffuse disc bulges note at D12/L1 level, obliterating anterior CSF spaces .",0,1
518,local_data/mendeley/T1_0146_D5.png,"LSS MRI : Feature of muscle spasm with schmorls nodules . Posterio-central disc protrusion noted at L5/S1 level, compressing thecal sac and encroaching exit nerve canal  with annular tear and dehtdrated disc material . Diffuse disc bulges note at D12/L1 level, obliterating anterior CSF spaces .",0,1
519,local_data/mendeley/T2_0146_D3.png,"LSS MRI : Feature of muscle spasm with schmorls nodules . Posterio-central disc protrusion noted at L5/S1 level, compressing thecal sac and encroaching exit nerve canal  with annular tear and dehtdrated disc material . Diffuse disc bulges note at D12/L1 level, obliterating anterior CSF spaces .",0,1
520,local_data/mendeley/T2_0146_D4.png,"LSS MRI : Feature of muscle spasm with schmorls nodules . Posterio-central disc protrusion noted at L5/S1 level, compressing thecal sac and encroaching exit nerve canal  with annular tear and dehtdrated disc material . Diffuse disc bulges note at D12/L1 level, obliterating anterior CSF spaces .",0,1
521,local_data/mendeley/T2_0146_D5.png,"LSS MRI : Feature of muscle spasm with schmorls nodules . Posterio-central disc protrusion noted at L5/S1 level, compressing thecal sac and encroaching exit nerve canal  with annular tear and dehtdrated disc material . Diffuse disc bulges note at D12/L1 level, obliterating anterior CSF spaces .",0,1
522,local_data/mendeley/T1_0149_D3.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level abutting both S1 nerve roots Diffuse disc bulge noted at L4-L5 level compressing the thecal sac and encroaching upon the exit neural canals,0,1
523,local_data/mendeley/T1_0149_D4.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level abutting both S1 nerve roots Diffuse disc bulge noted at L4-L5 level compressing the thecal sac and encroaching upon the exit neural canals,0,1
524,local_data/mendeley/T1_0149_D5.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level abutting both S1 nerve roots Diffuse disc bulge noted at L4-L5 level compressing the thecal sac and encroaching upon the exit neural canals,0,1
525,local_data/mendeley/T2_0149_D3.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level abutting both S1 nerve roots Diffuse disc bulge noted at L4-L5 level compressing the thecal sac and encroaching upon the exit neural canals,0,1
526,local_data/mendeley/T2_0149_D4.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level abutting both S1 nerve roots Diffuse disc bulge noted at L4-L5 level compressing the thecal sac and encroaching upon the exit neural canals,0,1
527,local_data/mendeley/T2_0149_D5.png,Lumbosacral MRI Mild disc bulge noted at L5-S1 level abutting both S1 nerve roots Diffuse disc bulge noted at L4-L5 level compressing the thecal sac and encroaching upon the exit neural canals,0,1
528,local_data/mendeley/T1_0150_D3.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals. adequate spinal canal.",0,1
529,local_data/mendeley/T1_0150_D4.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals. adequate spinal canal.",0,1
530,local_data/mendeley/T1_0150_D5.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals. adequate spinal canal.",0,1
531,local_data/mendeley/T2_0150_D3.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals. adequate spinal canal.",0,1
532,local_data/mendeley/T2_0150_D4.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals. adequate spinal canal.",0,1
533,local_data/mendeley/T2_0150_D5.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals. adequate spinal canal.",0,1
534,local_data/mendeley/T1_0151_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1  -Dissicating disc material noted at lower levels  -Postreior central disc protrusion noted at L4/L5 level, largely compressing thecal sac more to Lt and encroaching exit neural canal.  -Wide base disc bulge noted at L5/S1 level, compressing thecal sac and nerve roots , with modic type II noted & narrowed disc space",0,1
535,local_data/mendeley/T1_0151_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1  -Dissicating disc material noted at lower levels  -Postreior central disc protrusion noted at L4/L5 level, largely compressing thecal sac more to Lt and encroaching exit neural canal.  -Wide base disc bulge noted at L5/S1 level, compressing thecal sac and nerve roots , with modic type II noted & narrowed disc space",0,1
536,local_data/mendeley/T1_0151_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1  -Dissicating disc material noted at lower levels  -Postreior central disc protrusion noted at L4/L5 level, largely compressing thecal sac more to Lt and encroaching exit neural canal.  -Wide base disc bulge noted at L5/S1 level, compressing thecal sac and nerve roots , with modic type II noted & narrowed disc space",0,1
537,local_data/mendeley/T2_0151_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1  -Dissicating disc material noted at lower levels  -Postreior central disc protrusion noted at L4/L5 level, largely compressing thecal sac more to Lt and encroaching exit neural canal.  -Wide base disc bulge noted at L5/S1 level, compressing thecal sac and nerve roots , with modic type II noted & narrowed disc space",0,1
538,local_data/mendeley/T2_0151_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1  -Dissicating disc material noted at lower levels  -Postreior central disc protrusion noted at L4/L5 level, largely compressing thecal sac more to Lt and encroaching exit neural canal.  -Wide base disc bulge noted at L5/S1 level, compressing thecal sac and nerve roots , with modic type II noted & narrowed disc space",0,1
539,local_data/mendeley/T2_0151_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1  -Dissicating disc material noted at lower levels  -Postreior central disc protrusion noted at L4/L5 level, largely compressing thecal sac more to Lt and encroaching exit neural canal.  -Wide base disc bulge noted at L5/S1 level, compressing thecal sac and nerve roots , with modic type II noted & narrowed disc space",0,1
540,local_data/mendeley/T1_0152_D3.png,"Lumbosacral MRI -Diffuse disc bulge noted at L4-5 level , extending more to the right side compressing the thecal sac and encroaching upon the exiting neural canals, more to Rt sided  -Spondylolisthesis grade 1 seen at L5 over S1 -Diffuse disc bulge noted at L5-S1 mild compressing thecal sac and exiting nerve roots -Feature of muscle spasm with dissicating disc material noted at lower level",0,1
541,local_data/mendeley/T1_0152_D4.png,"Lumbosacral MRI -Diffuse disc bulge noted at L4-5 level , extending more to the right side compressing the thecal sac and encroaching upon the exiting neural canals, more to Rt sided  -Spondylolisthesis grade 1 seen at L5 over S1 -Diffuse disc bulge noted at L5-S1 mild compressing thecal sac and exiting nerve roots -Feature of muscle spasm with dissicating disc material noted at lower level",0,1
542,local_data/mendeley/T1_0152_D5.png,"Lumbosacral MRI -Diffuse disc bulge noted at L4-5 level , extending more to the right side compressing the thecal sac and encroaching upon the exiting neural canals, more to Rt sided  -Spondylolisthesis grade 1 seen at L5 over S1 -Diffuse disc bulge noted at L5-S1 mild compressing thecal sac and exiting nerve roots -Feature of muscle spasm with dissicating disc material noted at lower level",0,1
543,local_data/mendeley/T2_0152_D3.png,"Lumbosacral MRI -Diffuse disc bulge noted at L4-5 level , extending more to the right side compressing the thecal sac and encroaching upon the exiting neural canals, more to Rt sided  -Spondylolisthesis grade 1 seen at L5 over S1 -Diffuse disc bulge noted at L5-S1 mild compressing thecal sac and exiting nerve roots -Feature of muscle spasm with dissicating disc material noted at lower level",0,1
544,local_data/mendeley/T2_0152_D4.png,"Lumbosacral MRI -Diffuse disc bulge noted at L4-5 level , extending more to the right side compressing the thecal sac and encroaching upon the exiting neural canals, more to Rt sided  -Spondylolisthesis grade 1 seen at L5 over S1 -Diffuse disc bulge noted at L5-S1 mild compressing thecal sac and exiting nerve roots -Feature of muscle spasm with dissicating disc material noted at lower level",0,1
545,local_data/mendeley/T2_0152_D5.png,"Lumbosacral MRI -Diffuse disc bulge noted at L4-5 level , extending more to the right side compressing the thecal sac and encroaching upon the exiting neural canals, more to Rt sided  -Spondylolisthesis grade 1 seen at L5 over S1 -Diffuse disc bulge noted at L5-S1 mild compressing thecal sac and exiting nerve roots -Feature of muscle spasm with dissicating disc material noted at lower level",0,1
546,local_data/mendeley/T1_0154_D3.png,"L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, facet joint and ligamentum flavum hypertrophy, compressing the thecal sac, exit canals  and narrowing the spinal canal. L3-L4: diffuse disc bulge compressing the thecal sac and exit canals.",0,1
547,local_data/mendeley/T1_0154_D4.png,"L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, facet joint and ligamentum flavum hypertrophy, compressing the thecal sac, exit canals  and narrowing the spinal canal. L3-L4: diffuse disc bulge compressing the thecal sac and exit canals.",0,1
548,local_data/mendeley/T1_0154_D5.png,"L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, facet joint and ligamentum flavum hypertrophy, compressing the thecal sac, exit canals  and narrowing the spinal canal. L3-L4: diffuse disc bulge compressing the thecal sac and exit canals.",0,1
549,local_data/mendeley/T2_0154_D3.png,"L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, facet joint and ligamentum flavum hypertrophy, compressing the thecal sac, exit canals  and narrowing the spinal canal. L3-L4: diffuse disc bulge compressing the thecal sac and exit canals.",0,1
550,local_data/mendeley/T2_0154_D4.png,"L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, facet joint and ligamentum flavum hypertrophy, compressing the thecal sac, exit canals  and narrowing the spinal canal. L3-L4: diffuse disc bulge compressing the thecal sac and exit canals.",0,1
551,local_data/mendeley/T2_0154_D5.png,"L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, facet joint and ligamentum flavum hypertrophy, compressing the thecal sac, exit canals  and narrowing the spinal canal. L3-L4: diffuse disc bulge compressing the thecal sac and exit canals.",0,1
552,local_data/mendeley/T1_0155_D3.png,"LSS MRI : feature of muscle spasm. Multilevel diffuse disc bulges mainly at L2/L3,L3/L4 & L4/L5 levels compressing thecal sac and both nerve roots",0,1
553,local_data/mendeley/T1_0155_D4.png,"LSS MRI : feature of muscle spasm. Multilevel diffuse disc bulges mainly at L2/L3,L3/L4 & L4/L5 levels compressing thecal sac and both nerve roots",0,1
554,local_data/mendeley/T1_0155_D5.png,"LSS MRI : feature of muscle spasm. Multilevel diffuse disc bulges mainly at L2/L3,L3/L4 & L4/L5 levels compressing thecal sac and both nerve roots",0,1
555,local_data/mendeley/T2_0155_D3.png,"LSS MRI : feature of muscle spasm. Multilevel diffuse disc bulges mainly at L2/L3,L3/L4 & L4/L5 levels compressing thecal sac and both nerve roots",0,1
556,local_data/mendeley/T2_0155_D4.png,"LSS MRI : feature of muscle spasm. Multilevel diffuse disc bulges mainly at L2/L3,L3/L4 & L4/L5 levels compressing thecal sac and both nerve roots",0,1
557,local_data/mendeley/T2_0155_D5.png,"LSS MRI : feature of muscle spasm. Multilevel diffuse disc bulges mainly at L2/L3,L3/L4 & L4/L5 levels compressing thecal sac and both nerve roots",0,1
558,local_data/mendeley/T1_0157_D3.png,mild disc bulge at L4_L5 level.,0,1
559,local_data/mendeley/T1_0157_D4.png,mild disc bulge at L4_L5 level.,0,1
560,local_data/mendeley/T1_0157_D5.png,mild disc bulge at L4_L5 level.,0,1
561,local_data/mendeley/T2_0157_D3.png,mild disc bulge at L4_L5 level.,0,1
562,local_data/mendeley/T2_0157_D4.png,mild disc bulge at L4_L5 level.,0,1
563,local_data/mendeley/T2_0157_D5.png,mild disc bulge at L4_L5 level.,0,1
564,local_data/mendeley/T1_0158_D3.png,"Lumbosacral MRI  -Feature of muscle spasm noted  Modic type II noted at the level of the L4-L5-  -Wide base disc bulge noted at the L4-L5 level ,extending to the lateral recess compressing the thecal sac and encroaching upon exiting neural canals",0,1
565,local_data/mendeley/T1_0158_D4.png,"Lumbosacral MRI  -Feature of muscle spasm noted  Modic type II noted at the level of the L4-L5-  -Wide base disc bulge noted at the L4-L5 level ,extending to the lateral recess compressing the thecal sac and encroaching upon exiting neural canals",0,1
566,local_data/mendeley/T1_0158_D5.png,"Lumbosacral MRI  -Feature of muscle spasm noted  Modic type II noted at the level of the L4-L5-  -Wide base disc bulge noted at the L4-L5 level ,extending to the lateral recess compressing the thecal sac and encroaching upon exiting neural canals",0,1
567,local_data/mendeley/T2_0158_D3.png,"Lumbosacral MRI  -Feature of muscle spasm noted  Modic type II noted at the level of the L4-L5-  -Wide base disc bulge noted at the L4-L5 level ,extending to the lateral recess compressing the thecal sac and encroaching upon exiting neural canals",0,1
568,local_data/mendeley/T2_0158_D4.png,"Lumbosacral MRI  -Feature of muscle spasm noted  Modic type II noted at the level of the L4-L5-  -Wide base disc bulge noted at the L4-L5 level ,extending to the lateral recess compressing the thecal sac and encroaching upon exiting neural canals",0,1
569,local_data/mendeley/T2_0158_D5.png,"Lumbosacral MRI  -Feature of muscle spasm noted  Modic type II noted at the level of the L4-L5-  -Wide base disc bulge noted at the L4-L5 level ,extending to the lateral recess compressing the thecal sac and encroaching upon exiting neural canals",0,1
570,local_data/mendeley/T1_0159_D3.png,Lumbosacral MRI No evidence of disc herniation The thecal sac or nerve root compression noted,1,0
571,local_data/mendeley/T1_0159_D4.png,Lumbosacral MRI No evidence of disc herniation The thecal sac or nerve root compression noted,1,0
572,local_data/mendeley/T1_0159_D5.png,Lumbosacral MRI No evidence of disc herniation The thecal sac or nerve root compression noted,1,0
573,local_data/mendeley/T2_0159_D3.png,Lumbosacral MRI No evidence of disc herniation The thecal sac or nerve root compression noted,1,0
574,local_data/mendeley/T2_0159_D4.png,Lumbosacral MRI No evidence of disc herniation The thecal sac or nerve root compression noted,1,0
575,local_data/mendeley/T2_0159_D5.png,Lumbosacral MRI No evidence of disc herniation The thecal sac or nerve root compression noted,1,0
576,local_data/mendeley/T1_0160_D3.png,feature of muscle spasm.  No significant thecal sac or nerve root compressing,0,1
577,local_data/mendeley/T1_0160_D4.png,feature of muscle spasm.  No significant thecal sac or nerve root compressing,0,1
578,local_data/mendeley/T1_0160_D5.png,feature of muscle spasm.  No significant thecal sac or nerve root compressing,0,1
579,local_data/mendeley/T2_0160_D3.png,feature of muscle spasm.  No significant thecal sac or nerve root compressing,0,1
580,local_data/mendeley/T2_0160_D4.png,feature of muscle spasm.  No significant thecal sac or nerve root compressing,0,1
581,local_data/mendeley/T2_0160_D5.png,feature of muscle spasm.  No significant thecal sac or nerve root compressing,0,1
582,local_data/mendeley/T1_0161_D3.png,LSS MRI :  feature of muscle spasm.  No significant thyecal sac or nerve root compression .  adequate spinal canal,0,1
583,local_data/mendeley/T1_0161_D4.png,LSS MRI :  feature of muscle spasm.  No significant thyecal sac or nerve root compression .  adequate spinal canal,0,1
584,local_data/mendeley/T1_0161_D5.png,LSS MRI :  feature of muscle spasm.  No significant thyecal sac or nerve root compression .  adequate spinal canal,0,1
585,local_data/mendeley/T2_0161_D3.png,LSS MRI :  feature of muscle spasm.  No significant thyecal sac or nerve root compression .  adequate spinal canal,0,1
586,local_data/mendeley/T2_0161_D4.png,LSS MRI :  feature of muscle spasm.  No significant thyecal sac or nerve root compression .  adequate spinal canal,0,1
587,local_data/mendeley/T2_0161_D5.png,LSS MRI :  feature of muscle spasm.  No significant thyecal sac or nerve root compression .  adequate spinal canal,0,1
588,local_data/mendeley/T1_0162_D3.png,LSS MRI :  feature of muscle spasm.  End palte degenerative chnages.  L4/L5 level :narrowed disc space.  Diffuse disc bulges compressing thecal sac and both nerve roots .,0,1
589,local_data/mendeley/T1_0162_D4.png,LSS MRI :  feature of muscle spasm.  End palte degenerative chnages.  L4/L5 level :narrowed disc space.  Diffuse disc bulges compressing thecal sac and both nerve roots .,0,1
590,local_data/mendeley/T1_0162_D5.png,LSS MRI :  feature of muscle spasm.  End palte degenerative chnages.  L4/L5 level :narrowed disc space.  Diffuse disc bulges compressing thecal sac and both nerve roots .,0,1
591,local_data/mendeley/T2_0162_D3.png,LSS MRI :  feature of muscle spasm.  End palte degenerative chnages.  L4/L5 level :narrowed disc space.  Diffuse disc bulges compressing thecal sac and both nerve roots .,0,1
592,local_data/mendeley/T2_0162_D4.png,LSS MRI :  feature of muscle spasm.  End palte degenerative chnages.  L4/L5 level :narrowed disc space.  Diffuse disc bulges compressing thecal sac and both nerve roots .,0,1
593,local_data/mendeley/T2_0162_D5.png,LSS MRI :  feature of muscle spasm.  End palte degenerative chnages.  L4/L5 level :narrowed disc space.  Diffuse disc bulges compressing thecal sac and both nerve roots .,0,1
594,local_data/mendeley/T1_0163_D3.png,L4-L5: Mild disc bulge noted. No significant thecal sac or nerve root compression noted.,0,1
595,local_data/mendeley/T1_0163_D4.png,L4-L5: Mild disc bulge noted. No significant thecal sac or nerve root compression noted.,0,1
596,local_data/mendeley/T1_0163_D5.png,L4-L5: Mild disc bulge noted. No significant thecal sac or nerve root compression noted.,0,1
597,local_data/mendeley/T2_0163_D3.png,L4-L5: Mild disc bulge noted. No significant thecal sac or nerve root compression noted.,0,1
598,local_data/mendeley/T2_0163_D4.png,L4-L5: Mild disc bulge noted. No significant thecal sac or nerve root compression noted.,0,1
599,local_data/mendeley/T2_0163_D5.png,L4-L5: Mild disc bulge noted. No significant thecal sac or nerve root compression noted.,0,1
600,local_data/mendeley/T1_0165_D3.png,No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
601,local_data/mendeley/T1_0165_D4.png,No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
602,local_data/mendeley/T1_0165_D5.png,No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
603,local_data/mendeley/T2_0165_D3.png,No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
604,local_data/mendeley/T2_0165_D4.png,No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
605,local_data/mendeley/T2_0165_D5.png,No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
606,local_data/mendeley/T1_0166_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal  and both nerve roots . NB: lipoma of the filum terminale / filar lipoma is noted",0,1
607,local_data/mendeley/T1_0166_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal  and both nerve roots . NB: lipoma of the filum terminale / filar lipoma is noted",0,1
608,local_data/mendeley/T1_0166_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal  and both nerve roots . NB: lipoma of the filum terminale / filar lipoma is noted",0,1
609,local_data/mendeley/T2_0166_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal  and both nerve roots . NB: lipoma of the filum terminale / filar lipoma is noted",0,1
610,local_data/mendeley/T2_0166_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal  and both nerve roots . NB: lipoma of the filum terminale / filar lipoma is noted",0,1
611,local_data/mendeley/T2_0166_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal  and both nerve roots . NB: lipoma of the filum terminale / filar lipoma is noted",0,1
612,local_data/mendeley/T1_0167_D3.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
613,local_data/mendeley/T1_0167_D4.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
614,local_data/mendeley/T1_0167_D5.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
615,local_data/mendeley/T2_0167_D3.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
616,local_data/mendeley/T2_0167_D4.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
617,local_data/mendeley/T2_0167_D5.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
618,local_data/mendeley/T1_0168_D3.png,"diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit neural canals- more on the Lt side. adequate spinal canal.",0,1
619,local_data/mendeley/T1_0168_D4.png,"diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit neural canals- more on the Lt side. adequate spinal canal.",0,1
620,local_data/mendeley/T1_0168_D5.png,"diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit neural canals- more on the Lt side. adequate spinal canal.",0,1
621,local_data/mendeley/T2_0168_D3.png,"diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit neural canals- more on the Lt side. adequate spinal canal.",0,1
622,local_data/mendeley/T2_0168_D4.png,"diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit neural canals- more on the Lt side. adequate spinal canal.",0,1
623,local_data/mendeley/T2_0168_D5.png,"diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit neural canals- more on the Lt side. adequate spinal canal.",0,1
624,local_data/mendeley/T1_0172_D3.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Desiccated disc material .  Modec type II of L2/L3 with narrowed disc space .  Diffuse disc bulge ,with Rt foraminal disc protrusion noted at L2/L3 level compressing the thecal sac and exit neural canals.  With spinal canal stenosis noted.  Diffuse disc bulge at L3/L4 & L4/L5 levels , mild compressing thecal sac and encroaching exit nerve root , with relatively secondary spinal canal stenosis",0,1
625,local_data/mendeley/T1_0172_D4.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Desiccated disc material .  Modec type II of L2/L3 with narrowed disc space .  Diffuse disc bulge ,with Rt foraminal disc protrusion noted at L2/L3 level compressing the thecal sac and exit neural canals.  With spinal canal stenosis noted.  Diffuse disc bulge at L3/L4 & L4/L5 levels , mild compressing thecal sac and encroaching exit nerve root , with relatively secondary spinal canal stenosis",0,1
626,local_data/mendeley/T1_0172_D5.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Desiccated disc material .  Modec type II of L2/L3 with narrowed disc space .  Diffuse disc bulge ,with Rt foraminal disc protrusion noted at L2/L3 level compressing the thecal sac and exit neural canals.  With spinal canal stenosis noted.  Diffuse disc bulge at L3/L4 & L4/L5 levels , mild compressing thecal sac and encroaching exit nerve root , with relatively secondary spinal canal stenosis",0,1
627,local_data/mendeley/T2_0172_D3.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Desiccated disc material .  Modec type II of L2/L3 with narrowed disc space .  Diffuse disc bulge ,with Rt foraminal disc protrusion noted at L2/L3 level compressing the thecal sac and exit neural canals.  With spinal canal stenosis noted.  Diffuse disc bulge at L3/L4 & L4/L5 levels , mild compressing thecal sac and encroaching exit nerve root , with relatively secondary spinal canal stenosis",0,1
628,local_data/mendeley/T2_0172_D4.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Desiccated disc material .  Modec type II of L2/L3 with narrowed disc space .  Diffuse disc bulge ,with Rt foraminal disc protrusion noted at L2/L3 level compressing the thecal sac and exit neural canals.  With spinal canal stenosis noted.  Diffuse disc bulge at L3/L4 & L4/L5 levels , mild compressing thecal sac and encroaching exit nerve root , with relatively secondary spinal canal stenosis",0,1
629,local_data/mendeley/T2_0172_D5.png,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Desiccated disc material .  Modec type II of L2/L3 with narrowed disc space .  Diffuse disc bulge ,with Rt foraminal disc protrusion noted at L2/L3 level compressing the thecal sac and exit neural canals.  With spinal canal stenosis noted.  Diffuse disc bulge at L3/L4 & L4/L5 levels , mild compressing thecal sac and encroaching exit nerve root , with relatively secondary spinal canal stenosis",0,1
630,local_data/mendeley/T1_0173_D3.png,Old L1 compression fracture noted. No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,0,1
631,local_data/mendeley/T1_0173_D4.png,Old L1 compression fracture noted. No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,0,1
632,local_data/mendeley/T1_0173_D5.png,Old L1 compression fracture noted. No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,0,1
633,local_data/mendeley/T2_0173_D3.png,Old L1 compression fracture noted. No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,0,1
634,local_data/mendeley/T2_0173_D4.png,Old L1 compression fracture noted. No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,0,1
635,local_data/mendeley/T2_0173_D5.png,Old L1 compression fracture noted. No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,0,1
636,local_data/mendeley/T1_0174_D3.png,"LSS MRI : -Feature of muscel spasm. -Dissicating discal materials.   -Modic type II noted L2/L3 level. -Diffuse disc bulges noted at  L2/L3 & L4/L5 levels ,compressing the thecal sac ,extending to  lateral recess -encroaching upon exit neural canals. -Mild disc bulge noted at L3/L4 level, mildy compressing the thecal sac .",0,1
637,local_data/mendeley/T1_0174_D4.png,"LSS MRI : -Feature of muscel spasm. -Dissicating discal materials.   -Modic type II noted L2/L3 level. -Diffuse disc bulges noted at  L2/L3 & L4/L5 levels ,compressing the thecal sac ,extending to  lateral recess -encroaching upon exit neural canals. -Mild disc bulge noted at L3/L4 level, mildy compressing the thecal sac .",0,1
638,local_data/mendeley/T1_0174_D5.png,"LSS MRI : -Feature of muscel spasm. -Dissicating discal materials.   -Modic type II noted L2/L3 level. -Diffuse disc bulges noted at  L2/L3 & L4/L5 levels ,compressing the thecal sac ,extending to  lateral recess -encroaching upon exit neural canals. -Mild disc bulge noted at L3/L4 level, mildy compressing the thecal sac .",0,1
639,local_data/mendeley/T2_0174_D3.png,"LSS MRI : -Feature of muscel spasm. -Dissicating discal materials.   -Modic type II noted L2/L3 level. -Diffuse disc bulges noted at  L2/L3 & L4/L5 levels ,compressing the thecal sac ,extending to  lateral recess -encroaching upon exit neural canals. -Mild disc bulge noted at L3/L4 level, mildy compressing the thecal sac .",0,1
640,local_data/mendeley/T2_0174_D4.png,"LSS MRI : -Feature of muscel spasm. -Dissicating discal materials.   -Modic type II noted L2/L3 level. -Diffuse disc bulges noted at  L2/L3 & L4/L5 levels ,compressing the thecal sac ,extending to  lateral recess -encroaching upon exit neural canals. -Mild disc bulge noted at L3/L4 level, mildy compressing the thecal sac .",0,1
641,local_data/mendeley/T2_0174_D5.png,"LSS MRI : -Feature of muscel spasm. -Dissicating discal materials.   -Modic type II noted L2/L3 level. -Diffuse disc bulges noted at  L2/L3 & L4/L5 levels ,compressing the thecal sac ,extending to  lateral recess -encroaching upon exit neural canals. -Mild disc bulge noted at L3/L4 level, mildy compressing the thecal sac .",0,1
642,local_data/mendeley/T1_0175_D3.png,"LSS MRI :  Feature of muscle spasm.  dissicating disc materials noted .  Multilevels diffuse disc bulges noted mainly at L4/L5& L5/S1 levels, compressing thecal and encroaching exit neural canal with relatively ligamentum flavum hyperatrophy at L4/L5 level.",0,1
643,local_data/mendeley/T1_0175_D4.png,"LSS MRI :  Feature of muscle spasm.  dissicating disc materials noted .  Multilevels diffuse disc bulges noted mainly at L4/L5& L5/S1 levels, compressing thecal and encroaching exit neural canal with relatively ligamentum flavum hyperatrophy at L4/L5 level.",0,1
644,local_data/mendeley/T1_0175_D5.png,"LSS MRI :  Feature of muscle spasm.  dissicating disc materials noted .  Multilevels diffuse disc bulges noted mainly at L4/L5& L5/S1 levels, compressing thecal and encroaching exit neural canal with relatively ligamentum flavum hyperatrophy at L4/L5 level.",0,1
645,local_data/mendeley/T2_0175_D3.png,"LSS MRI :  Feature of muscle spasm.  dissicating disc materials noted .  Multilevels diffuse disc bulges noted mainly at L4/L5& L5/S1 levels, compressing thecal and encroaching exit neural canal with relatively ligamentum flavum hyperatrophy at L4/L5 level.",0,1
646,local_data/mendeley/T2_0175_D4.png,"LSS MRI :  Feature of muscle spasm.  dissicating disc materials noted .  Multilevels diffuse disc bulges noted mainly at L4/L5& L5/S1 levels, compressing thecal and encroaching exit neural canal with relatively ligamentum flavum hyperatrophy at L4/L5 level.",0,1
647,local_data/mendeley/T2_0175_D5.png,"LSS MRI :  Feature of muscle spasm.  dissicating disc materials noted .  Multilevels diffuse disc bulges noted mainly at L4/L5& L5/S1 levels, compressing thecal and encroaching exit neural canal with relatively ligamentum flavum hyperatrophy at L4/L5 level.",0,1
648,local_data/mendeley/T1_0176_D3.png,"LSS MRI  L4-L5: Diffuse disc bulge, more to the Lt side, compressing the thecal sac and Lt exit canal.  Adequate spinal canal.",0,1
649,local_data/mendeley/T1_0176_D4.png,"LSS MRI  L4-L5: Diffuse disc bulge, more to the Lt side, compressing the thecal sac and Lt exit canal.  Adequate spinal canal.",0,1
650,local_data/mendeley/T1_0176_D5.png,"LSS MRI  L4-L5: Diffuse disc bulge, more to the Lt side, compressing the thecal sac and Lt exit canal.  Adequate spinal canal.",0,1
651,local_data/mendeley/T2_0176_D3.png,"LSS MRI  L4-L5: Diffuse disc bulge, more to the Lt side, compressing the thecal sac and Lt exit canal.  Adequate spinal canal.",0,1
652,local_data/mendeley/T2_0176_D4.png,"LSS MRI  L4-L5: Diffuse disc bulge, more to the Lt side, compressing the thecal sac and Lt exit canal.  Adequate spinal canal.",0,1
653,local_data/mendeley/T2_0176_D5.png,"LSS MRI  L4-L5: Diffuse disc bulge, more to the Lt side, compressing the thecal sac and Lt exit canal.  Adequate spinal canal.",0,1
654,local_data/mendeley/T1_0177_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
655,local_data/mendeley/T1_0177_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
656,local_data/mendeley/T1_0177_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
657,local_data/mendeley/T2_0177_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
658,local_data/mendeley/T2_0177_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
659,local_data/mendeley/T2_0177_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
660,local_data/mendeley/T1_0178_D3.png,"Lumbosacral MRI Left paracentral disc protrusion noted at T12 - L1, Compressing the thecal sac and encroaching on the left exit neural canal mild, Diffuse disc bulges noted at L3-L4 and L4-L5 level",0,1
661,local_data/mendeley/T1_0178_D4.png,"Lumbosacral MRI Left paracentral disc protrusion noted at T12 - L1, Compressing the thecal sac and encroaching on the left exit neural canal mild, Diffuse disc bulges noted at L3-L4 and L4-L5 level",0,1
662,local_data/mendeley/T1_0178_D5.png,"Lumbosacral MRI Left paracentral disc protrusion noted at T12 - L1, Compressing the thecal sac and encroaching on the left exit neural canal mild, Diffuse disc bulges noted at L3-L4 and L4-L5 level",0,1
663,local_data/mendeley/T2_0178_D3.png,"Lumbosacral MRI Left paracentral disc protrusion noted at T12 - L1, Compressing the thecal sac and encroaching on the left exit neural canal mild, Diffuse disc bulges noted at L3-L4 and L4-L5 level",0,1
664,local_data/mendeley/T2_0178_D4.png,"Lumbosacral MRI Left paracentral disc protrusion noted at T12 - L1, Compressing the thecal sac and encroaching on the left exit neural canal mild, Diffuse disc bulges noted at L3-L4 and L4-L5 level",0,1
665,local_data/mendeley/T2_0178_D5.png,"Lumbosacral MRI Left paracentral disc protrusion noted at T12 - L1, Compressing the thecal sac and encroaching on the left exit neural canal mild, Diffuse disc bulges noted at L3-L4 and L4-L5 level",0,1
666,local_data/mendeley/T1_0179_D3.png,mild disc bulge noted at L5-S1 level.  adequate spinal canal.,0,1
667,local_data/mendeley/T1_0179_D4.png,mild disc bulge noted at L5-S1 level.  adequate spinal canal.,0,1
668,local_data/mendeley/T1_0179_D5.png,mild disc bulge noted at L5-S1 level.  adequate spinal canal.,0,1
669,local_data/mendeley/T2_0179_D3.png,mild disc bulge noted at L5-S1 level.  adequate spinal canal.,0,1
670,local_data/mendeley/T2_0179_D4.png,mild disc bulge noted at L5-S1 level.  adequate spinal canal.,0,1
671,local_data/mendeley/T2_0179_D5.png,mild disc bulge noted at L5-S1 level.  adequate spinal canal.,0,1
672,local_data/mendeley/T1_0180_D3.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level abutting thecal sac.  No significant thecal sac or nerve roots compression noted .,0,1
673,local_data/mendeley/T1_0180_D4.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level abutting thecal sac.  No significant thecal sac or nerve roots compression noted .,0,1
674,local_data/mendeley/T1_0180_D5.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level abutting thecal sac.  No significant thecal sac or nerve roots compression noted .,0,1
675,local_data/mendeley/T2_0180_D3.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level abutting thecal sac.  No significant thecal sac or nerve roots compression noted .,0,1
676,local_data/mendeley/T2_0180_D4.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level abutting thecal sac.  No significant thecal sac or nerve roots compression noted .,0,1
677,local_data/mendeley/T2_0180_D5.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level abutting thecal sac.  No significant thecal sac or nerve roots compression noted .,0,1
678,local_data/mendeley/T1_0181_D3.png,"LSS MRI : Feature of muscle spasm. dissicating disc materials . Multilevel diffuse disc bulges noted , notceably at L4/L5 & L5/S1, level, compressing the thecal sac and encroaching exit neural canal , associated with ligamentum flavum hyperatrophy at L4/L5 level with secandary spinal canal stenosis. NB : small Rt renal  cyst",0,1
679,local_data/mendeley/T1_0181_D4.png,"LSS MRI : Feature of muscle spasm. dissicating disc materials . Multilevel diffuse disc bulges noted , notceably at L4/L5 & L5/S1, level, compressing the thecal sac and encroaching exit neural canal , associated with ligamentum flavum hyperatrophy at L4/L5 level with secandary spinal canal stenosis. NB : small Rt renal  cyst",0,1
680,local_data/mendeley/T1_0181_D5.png,"LSS MRI : Feature of muscle spasm. dissicating disc materials . Multilevel diffuse disc bulges noted , notceably at L4/L5 & L5/S1, level, compressing the thecal sac and encroaching exit neural canal , associated with ligamentum flavum hyperatrophy at L4/L5 level with secandary spinal canal stenosis. NB : small Rt renal  cyst",0,1
681,local_data/mendeley/T2_0181_D3.png,"LSS MRI : Feature of muscle spasm. dissicating disc materials . Multilevel diffuse disc bulges noted , notceably at L4/L5 & L5/S1, level, compressing the thecal sac and encroaching exit neural canal , associated with ligamentum flavum hyperatrophy at L4/L5 level with secandary spinal canal stenosis. NB : small Rt renal  cyst",0,1
682,local_data/mendeley/T2_0181_D4.png,"LSS MRI : Feature of muscle spasm. dissicating disc materials . Multilevel diffuse disc bulges noted , notceably at L4/L5 & L5/S1, level, compressing the thecal sac and encroaching exit neural canal , associated with ligamentum flavum hyperatrophy at L4/L5 level with secandary spinal canal stenosis. NB : small Rt renal  cyst",0,1
683,local_data/mendeley/T2_0181_D5.png,"LSS MRI : Feature of muscle spasm. dissicating disc materials . Multilevel diffuse disc bulges noted , notceably at L4/L5 & L5/S1, level, compressing the thecal sac and encroaching exit neural canal , associated with ligamentum flavum hyperatrophy at L4/L5 level with secandary spinal canal stenosis. NB : small Rt renal  cyst",0,1
684,local_data/mendeley/T1_0183_D3.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
685,local_data/mendeley/T1_0183_D4.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
686,local_data/mendeley/T1_0183_D5.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
687,local_data/mendeley/T2_0183_D3.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
688,local_data/mendeley/T2_0183_D4.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
689,local_data/mendeley/T2_0183_D5.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
690,local_data/mendeley/T1_0185_D3.png,"LSS MRI : -Feature of muscle spasm. Diffuse decreased bone marrow signal intensitie (obese patient ). - Diffuse disc bulge noted, L4/L5 level, mildy compressing thecal sac and encroaching both exit neural canals",0,1
691,local_data/mendeley/T1_0185_D4.png,"LSS MRI : -Feature of muscle spasm. Diffuse decreased bone marrow signal intensitie (obese patient ). - Diffuse disc bulge noted, L4/L5 level, mildy compressing thecal sac and encroaching both exit neural canals",0,1
692,local_data/mendeley/T1_0185_D5.png,"LSS MRI : -Feature of muscle spasm. Diffuse decreased bone marrow signal intensitie (obese patient ). - Diffuse disc bulge noted, L4/L5 level, mildy compressing thecal sac and encroaching both exit neural canals",0,1
693,local_data/mendeley/T2_0185_D3.png,"LSS MRI : -Feature of muscle spasm. Diffuse decreased bone marrow signal intensitie (obese patient ). - Diffuse disc bulge noted, L4/L5 level, mildy compressing thecal sac and encroaching both exit neural canals",0,1
694,local_data/mendeley/T2_0185_D4.png,"LSS MRI : -Feature of muscle spasm. Diffuse decreased bone marrow signal intensitie (obese patient ). - Diffuse disc bulge noted, L4/L5 level, mildy compressing thecal sac and encroaching both exit neural canals",0,1
695,local_data/mendeley/T2_0185_D5.png,"LSS MRI : -Feature of muscle spasm. Diffuse decreased bone marrow signal intensitie (obese patient ). - Diffuse disc bulge noted, L4/L5 level, mildy compressing thecal sac and encroaching both exit neural canals",0,1
696,local_data/mendeley/T1_0186_D3.png,"LSS MRI : .Feature of muscle spasm. Dissicating disc materials noted . .Multilevel diffuse disc bulges, notceably noted at L3/L4 & L4/L5 levels, compressing the thecal sac and encroaching upon exit neural canals .",0,1
697,local_data/mendeley/T1_0186_D4.png,"LSS MRI : .Feature of muscle spasm. Dissicating disc materials noted . .Multilevel diffuse disc bulges, notceably noted at L3/L4 & L4/L5 levels, compressing the thecal sac and encroaching upon exit neural canals .",0,1
698,local_data/mendeley/T1_0186_D5.png,"LSS MRI : .Feature of muscle spasm. Dissicating disc materials noted . .Multilevel diffuse disc bulges, notceably noted at L3/L4 & L4/L5 levels, compressing the thecal sac and encroaching upon exit neural canals .",0,1
699,local_data/mendeley/T2_0186_D3.png,"LSS MRI : .Feature of muscle spasm. Dissicating disc materials noted . .Multilevel diffuse disc bulges, notceably noted at L3/L4 & L4/L5 levels, compressing the thecal sac and encroaching upon exit neural canals .",0,1
700,local_data/mendeley/T2_0186_D4.png,"LSS MRI : .Feature of muscle spasm. Dissicating disc materials noted . .Multilevel diffuse disc bulges, notceably noted at L3/L4 & L4/L5 levels, compressing the thecal sac and encroaching upon exit neural canals .",0,1
701,local_data/mendeley/T2_0186_D5.png,"LSS MRI : .Feature of muscle spasm. Dissicating disc materials noted . .Multilevel diffuse disc bulges, notceably noted at L3/L4 & L4/L5 levels, compressing the thecal sac and encroaching upon exit neural canals .",0,1
702,local_data/mendeley/T1_0187_D3.png,"LSS MRI  diffuse disc bulges noted at L4-L5 and L5-S1 levels, compressing the thecal sac and exit canals.  the spinal canal appeared adequate.  L1 hemangioma.",0,1
703,local_data/mendeley/T1_0187_D4.png,"LSS MRI  diffuse disc bulges noted at L4-L5 and L5-S1 levels, compressing the thecal sac and exit canals.  the spinal canal appeared adequate.  L1 hemangioma.",0,1
704,local_data/mendeley/T1_0187_D5.png,"LSS MRI  diffuse disc bulges noted at L4-L5 and L5-S1 levels, compressing the thecal sac and exit canals.  the spinal canal appeared adequate.  L1 hemangioma.",0,1
705,local_data/mendeley/T2_0187_D3.png,"LSS MRI  diffuse disc bulges noted at L4-L5 and L5-S1 levels, compressing the thecal sac and exit canals.  the spinal canal appeared adequate.  L1 hemangioma.",0,1
706,local_data/mendeley/T2_0187_D4.png,"LSS MRI  diffuse disc bulges noted at L4-L5 and L5-S1 levels, compressing the thecal sac and exit canals.  the spinal canal appeared adequate.  L1 hemangioma.",0,1
707,local_data/mendeley/T2_0187_D5.png,"LSS MRI  diffuse disc bulges noted at L4-L5 and L5-S1 levels, compressing the thecal sac and exit canals.  the spinal canal appeared adequate.  L1 hemangioma.",0,1
708,local_data/mendeley/T1_0188_D3.png,LSS MRI : Feature of muscle spasm. Adequate thecal sac . No disc protrusion or herniation seen .,0,1
709,local_data/mendeley/T1_0188_D4.png,LSS MRI : Feature of muscle spasm. Adequate thecal sac . No disc protrusion or herniation seen .,0,1
710,local_data/mendeley/T1_0188_D5.png,LSS MRI : Feature of muscle spasm. Adequate thecal sac . No disc protrusion or herniation seen .,0,1
711,local_data/mendeley/T2_0188_D3.png,LSS MRI : Feature of muscle spasm. Adequate thecal sac . No disc protrusion or herniation seen .,0,1
712,local_data/mendeley/T2_0188_D4.png,LSS MRI : Feature of muscle spasm. Adequate thecal sac . No disc protrusion or herniation seen .,0,1
713,local_data/mendeley/T2_0188_D5.png,LSS MRI : Feature of muscle spasm. Adequate thecal sac . No disc protrusion or herniation seen .,0,1
714,local_data/mendeley/T1_0192_D3.png,Multilevel diffuse disc bulges noted with facet joint hypertrophy compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
715,local_data/mendeley/T1_0192_D4.png,Multilevel diffuse disc bulges noted with facet joint hypertrophy compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
716,local_data/mendeley/T1_0192_D5.png,Multilevel diffuse disc bulges noted with facet joint hypertrophy compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
717,local_data/mendeley/T2_0192_D3.png,Multilevel diffuse disc bulges noted with facet joint hypertrophy compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
718,local_data/mendeley/T2_0192_D4.png,Multilevel diffuse disc bulges noted with facet joint hypertrophy compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
719,local_data/mendeley/T2_0192_D5.png,Multilevel diffuse disc bulges noted with facet joint hypertrophy compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
720,local_data/mendeley/T1_0193_D3.png,"LSS MRI  Features of muscle spasm noted.  multilevel schmorls nodule noted  Lt foraminal disc protrusion noted at L4-L5 level, compressing the thecal sac and Lt L5 NERVE ROOT.",0,1
721,local_data/mendeley/T1_0193_D4.png,"LSS MRI  Features of muscle spasm noted.  multilevel schmorls nodule noted  Lt foraminal disc protrusion noted at L4-L5 level, compressing the thecal sac and Lt L5 NERVE ROOT.",0,1
722,local_data/mendeley/T1_0193_D5.png,"LSS MRI  Features of muscle spasm noted.  multilevel schmorls nodule noted  Lt foraminal disc protrusion noted at L4-L5 level, compressing the thecal sac and Lt L5 NERVE ROOT.",0,1
723,local_data/mendeley/T2_0193_D3.png,"LSS MRI  Features of muscle spasm noted.  multilevel schmorls nodule noted  Lt foraminal disc protrusion noted at L4-L5 level, compressing the thecal sac and Lt L5 NERVE ROOT.",0,1
724,local_data/mendeley/T2_0193_D4.png,"LSS MRI  Features of muscle spasm noted.  multilevel schmorls nodule noted  Lt foraminal disc protrusion noted at L4-L5 level, compressing the thecal sac and Lt L5 NERVE ROOT.",0,1
725,local_data/mendeley/T2_0193_D5.png,"LSS MRI  Features of muscle spasm noted.  multilevel schmorls nodule noted  Lt foraminal disc protrusion noted at L4-L5 level, compressing the thecal sac and Lt L5 NERVE ROOT.",0,1
726,local_data/mendeley/T1_0196_D3.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
727,local_data/mendeley/T1_0196_D4.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
728,local_data/mendeley/T1_0196_D5.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
729,local_data/mendeley/T2_0196_D3.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
730,local_data/mendeley/T2_0196_D4.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
731,local_data/mendeley/T2_0196_D5.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
732,local_data/mendeley/T1_0197_D3.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
733,local_data/mendeley/T1_0197_D4.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
734,local_data/mendeley/T1_0197_D5.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
735,local_data/mendeley/T2_0197_D3.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
736,local_data/mendeley/T2_0197_D4.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
737,local_data/mendeley/T2_0197_D5.png,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1
738,local_data/mendeley/T1_0199_D3.png,"LSS MRI : .Feaure of muscle spasm. .Multilevels diffuse disc bulges noted nortceably at L2/L3 & L4/L5 level, compressing the thecal sac and enroaching upon exit neural canals.   anterior wedging of D12 vertebral body  associated  with multiple Schmorl's nodes, and end plate irregularity , Dorsal spine MRI is advsied to rule out, Schuerman's disease .",0,1
739,local_data/mendeley/T1_0199_D4.png,"LSS MRI : .Feaure of muscle spasm. .Multilevels diffuse disc bulges noted nortceably at L2/L3 & L4/L5 level, compressing the thecal sac and enroaching upon exit neural canals.   anterior wedging of D12 vertebral body  associated  with multiple Schmorl's nodes, and end plate irregularity , Dorsal spine MRI is advsied to rule out, Schuerman's disease .",0,1
740,local_data/mendeley/T1_0199_D5.png,"LSS MRI : .Feaure of muscle spasm. .Multilevels diffuse disc bulges noted nortceably at L2/L3 & L4/L5 level, compressing the thecal sac and enroaching upon exit neural canals.   anterior wedging of D12 vertebral body  associated  with multiple Schmorl's nodes, and end plate irregularity , Dorsal spine MRI is advsied to rule out, Schuerman's disease .",0,1
741,local_data/mendeley/T2_0199_D3.png,"LSS MRI : .Feaure of muscle spasm. .Multilevels diffuse disc bulges noted nortceably at L2/L3 & L4/L5 level, compressing the thecal sac and enroaching upon exit neural canals.   anterior wedging of D12 vertebral body  associated  with multiple Schmorl's nodes, and end plate irregularity , Dorsal spine MRI is advsied to rule out, Schuerman's disease .",0,1
742,local_data/mendeley/T2_0199_D4.png,"LSS MRI : .Feaure of muscle spasm. .Multilevels diffuse disc bulges noted nortceably at L2/L3 & L4/L5 level, compressing the thecal sac and enroaching upon exit neural canals.   anterior wedging of D12 vertebral body  associated  with multiple Schmorl's nodes, and end plate irregularity , Dorsal spine MRI is advsied to rule out, Schuerman's disease .",0,1
743,local_data/mendeley/T2_0199_D5.png,"LSS MRI : .Feaure of muscle spasm. .Multilevels diffuse disc bulges noted nortceably at L2/L3 & L4/L5 level, compressing the thecal sac and enroaching upon exit neural canals.   anterior wedging of D12 vertebral body  associated  with multiple Schmorl's nodes, and end plate irregularity , Dorsal spine MRI is advsied to rule out, Schuerman's disease .",0,1
744,local_data/mendeley/T1_0201_D3.png,"LSS MRI : Feature of muscle spasm. Central and to the Lt,  disc protrusion noted  at L5/S1 level relatively compressing thecal sac and encroaching exit neural canal , with dissicating disc material",0,1
745,local_data/mendeley/T1_0201_D4.png,"LSS MRI : Feature of muscle spasm. Central and to the Lt,  disc protrusion noted  at L5/S1 level relatively compressing thecal sac and encroaching exit neural canal , with dissicating disc material",0,1
746,local_data/mendeley/T1_0201_D5.png,"LSS MRI : Feature of muscle spasm. Central and to the Lt,  disc protrusion noted  at L5/S1 level relatively compressing thecal sac and encroaching exit neural canal , with dissicating disc material",0,1
747,local_data/mendeley/T2_0201_D3.png,"LSS MRI : Feature of muscle spasm. Central and to the Lt,  disc protrusion noted  at L5/S1 level relatively compressing thecal sac and encroaching exit neural canal , with dissicating disc material",0,1
748,local_data/mendeley/T2_0201_D4.png,"LSS MRI : Feature of muscle spasm. Central and to the Lt,  disc protrusion noted  at L5/S1 level relatively compressing thecal sac and encroaching exit neural canal , with dissicating disc material",0,1
749,local_data/mendeley/T2_0201_D5.png,"LSS MRI : Feature of muscle spasm. Central and to the Lt,  disc protrusion noted  at L5/S1 level relatively compressing thecal sac and encroaching exit neural canal , with dissicating disc material",0,1
750,local_data/mendeley/T1_0203_D3.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
751,local_data/mendeley/T1_0203_D4.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
752,local_data/mendeley/T1_0203_D5.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
753,local_data/mendeley/T2_0203_D3.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
754,local_data/mendeley/T2_0203_D4.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
755,local_data/mendeley/T2_0203_D5.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
756,local_data/mendeley/T1_0204_D3.png,"LSS MRI Central disc protrusions noted at L3-L4, L4-L5 and L5-S1 levels, compressing the thecal sac. mild disc bulge noted at L2-L3 level.",0,1
757,local_data/mendeley/T1_0204_D4.png,"LSS MRI Central disc protrusions noted at L3-L4, L4-L5 and L5-S1 levels, compressing the thecal sac. mild disc bulge noted at L2-L3 level.",0,1
758,local_data/mendeley/T1_0204_D5.png,"LSS MRI Central disc protrusions noted at L3-L4, L4-L5 and L5-S1 levels, compressing the thecal sac. mild disc bulge noted at L2-L3 level.",0,1
759,local_data/mendeley/T2_0204_D3.png,"LSS MRI Central disc protrusions noted at L3-L4, L4-L5 and L5-S1 levels, compressing the thecal sac. mild disc bulge noted at L2-L3 level.",0,1
760,local_data/mendeley/T2_0204_D4.png,"LSS MRI Central disc protrusions noted at L3-L4, L4-L5 and L5-S1 levels, compressing the thecal sac. mild disc bulge noted at L2-L3 level.",0,1
761,local_data/mendeley/T2_0204_D5.png,"LSS MRI Central disc protrusions noted at L3-L4, L4-L5 and L5-S1 levels, compressing the thecal sac. mild disc bulge noted at L2-L3 level.",0,1
762,local_data/mendeley/T1_0205_D3.png,"LSS MRI : .Feature of muscle spasm. .Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac extending to lateral recess , more to Rt sided with encroaching upon exit neural canal. .Mild disc bulge noted at L5/S1 level, mildy compressing thecal sac.",0,1
763,local_data/mendeley/T1_0205_D4.png,"LSS MRI : .Feature of muscle spasm. .Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac extending to lateral recess , more to Rt sided with encroaching upon exit neural canal. .Mild disc bulge noted at L5/S1 level, mildy compressing thecal sac.",0,1
764,local_data/mendeley/T1_0205_D5.png,"LSS MRI : .Feature of muscle spasm. .Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac extending to lateral recess , more to Rt sided with encroaching upon exit neural canal. .Mild disc bulge noted at L5/S1 level, mildy compressing thecal sac.",0,1
765,local_data/mendeley/T2_0205_D3.png,"LSS MRI : .Feature of muscle spasm. .Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac extending to lateral recess , more to Rt sided with encroaching upon exit neural canal. .Mild disc bulge noted at L5/S1 level, mildy compressing thecal sac.",0,1
766,local_data/mendeley/T2_0205_D4.png,"LSS MRI : .Feature of muscle spasm. .Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac extending to lateral recess , more to Rt sided with encroaching upon exit neural canal. .Mild disc bulge noted at L5/S1 level, mildy compressing thecal sac.",0,1
767,local_data/mendeley/T2_0205_D5.png,"LSS MRI : .Feature of muscle spasm. .Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac extending to lateral recess , more to Rt sided with encroaching upon exit neural canal. .Mild disc bulge noted at L5/S1 level, mildy compressing thecal sac.",0,1
768,local_data/mendeley/T1_0207_D3.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy and hypertrophied ligament flava, compressing the thecal sac and exit neural canals with narrowed spinal canal.",0,1
769,local_data/mendeley/T1_0207_D4.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy and hypertrophied ligament flava, compressing the thecal sac and exit neural canals with narrowed spinal canal.",0,1
770,local_data/mendeley/T1_0207_D5.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy and hypertrophied ligament flava, compressing the thecal sac and exit neural canals with narrowed spinal canal.",0,1
771,local_data/mendeley/T2_0207_D3.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy and hypertrophied ligament flava, compressing the thecal sac and exit neural canals with narrowed spinal canal.",0,1
772,local_data/mendeley/T2_0207_D4.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy and hypertrophied ligament flava, compressing the thecal sac and exit neural canals with narrowed spinal canal.",0,1
773,local_data/mendeley/T2_0207_D5.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy and hypertrophied ligament flava, compressing the thecal sac and exit neural canals with narrowed spinal canal.",0,1
774,local_data/mendeley/T1_0208_D3.png,LSS MRI  L5-S1: Rt paracentral disc protrusion compressing Rt S1 nerve root.  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals with narrow spinal canal.  L3-L4: Mild disc bulge noted.,0,1
775,local_data/mendeley/T1_0208_D4.png,LSS MRI  L5-S1: Rt paracentral disc protrusion compressing Rt S1 nerve root.  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals with narrow spinal canal.  L3-L4: Mild disc bulge noted.,0,1
776,local_data/mendeley/T1_0208_D5.png,LSS MRI  L5-S1: Rt paracentral disc protrusion compressing Rt S1 nerve root.  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals with narrow spinal canal.  L3-L4: Mild disc bulge noted.,0,1
777,local_data/mendeley/T2_0208_D3.png,LSS MRI  L5-S1: Rt paracentral disc protrusion compressing Rt S1 nerve root.  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals with narrow spinal canal.  L3-L4: Mild disc bulge noted.,0,1
778,local_data/mendeley/T2_0208_D4.png,LSS MRI  L5-S1: Rt paracentral disc protrusion compressing Rt S1 nerve root.  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals with narrow spinal canal.  L3-L4: Mild disc bulge noted.,0,1
779,local_data/mendeley/T2_0208_D5.png,LSS MRI  L5-S1: Rt paracentral disc protrusion compressing Rt S1 nerve root.  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals with narrow spinal canal.  L3-L4: Mild disc bulge noted.,0,1
780,local_data/mendeley/T1_0209_D3.png,"L2 osteoprotic compression fracture with no retrobulge into spinal canal.  No evidence of disc herniation or thecal sac compression.  high T2 signal intensity involving the sacral ala (? post trauma, inflamatory,..).",0,1
781,local_data/mendeley/T1_0209_D4.png,"L2 osteoprotic compression fracture with no retrobulge into spinal canal.  No evidence of disc herniation or thecal sac compression.  high T2 signal intensity involving the sacral ala (? post trauma, inflamatory,..).",0,1
782,local_data/mendeley/T1_0209_D5.png,"L2 osteoprotic compression fracture with no retrobulge into spinal canal.  No evidence of disc herniation or thecal sac compression.  high T2 signal intensity involving the sacral ala (? post trauma, inflamatory,..).",0,1
783,local_data/mendeley/T2_0209_D3.png,"L2 osteoprotic compression fracture with no retrobulge into spinal canal.  No evidence of disc herniation or thecal sac compression.  high T2 signal intensity involving the sacral ala (? post trauma, inflamatory,..).",0,1
784,local_data/mendeley/T2_0209_D4.png,"L2 osteoprotic compression fracture with no retrobulge into spinal canal.  No evidence of disc herniation or thecal sac compression.  high T2 signal intensity involving the sacral ala (? post trauma, inflamatory,..).",0,1
785,local_data/mendeley/T2_0209_D5.png,"L2 osteoprotic compression fracture with no retrobulge into spinal canal.  No evidence of disc herniation or thecal sac compression.  high T2 signal intensity involving the sacral ala (? post trauma, inflamatory,..).",0,1
786,local_data/mendeley/T1_0210_D3.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
787,local_data/mendeley/T1_0210_D4.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
788,local_data/mendeley/T1_0210_D5.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
789,local_data/mendeley/T2_0210_D3.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
790,local_data/mendeley/T2_0210_D4.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
791,local_data/mendeley/T2_0210_D5.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
792,local_data/mendeley/T1_0212_D3.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
793,local_data/mendeley/T1_0212_D4.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
794,local_data/mendeley/T1_0212_D5.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
795,local_data/mendeley/T2_0212_D3.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
796,local_data/mendeley/T2_0212_D4.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
797,local_data/mendeley/T2_0212_D5.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
798,local_data/mendeley/T1_0215_D3.png,"LSS MRI  Mild disc bulge noted at L5-S1 level, abutting the thecal sac and S1 nerve roots.  Adequate spinal canal.",0,1
799,local_data/mendeley/T1_0215_D4.png,"LSS MRI  Mild disc bulge noted at L5-S1 level, abutting the thecal sac and S1 nerve roots.  Adequate spinal canal.",0,1
800,local_data/mendeley/T1_0215_D5.png,"LSS MRI  Mild disc bulge noted at L5-S1 level, abutting the thecal sac and S1 nerve roots.  Adequate spinal canal.",0,1
801,local_data/mendeley/T2_0215_D3.png,"LSS MRI  Mild disc bulge noted at L5-S1 level, abutting the thecal sac and S1 nerve roots.  Adequate spinal canal.",0,1
802,local_data/mendeley/T2_0215_D4.png,"LSS MRI  Mild disc bulge noted at L5-S1 level, abutting the thecal sac and S1 nerve roots.  Adequate spinal canal.",0,1
803,local_data/mendeley/T2_0215_D5.png,"LSS MRI  Mild disc bulge noted at L5-S1 level, abutting the thecal sac and S1 nerve roots.  Adequate spinal canal.",0,1
804,local_data/mendeley/T1_0216_D3.png,"LSS MRI:  feature of muscle spasm.  Dissicating disc mayetrials.  Lt paracentral disc protruion noted at D12/L1 level, indenting spinal cord.  Multilevel diffuse disc bulges noted at L3/L4&L4/L5 levels , compressing thecal sac and encroaching exit neural canals .",0,1
805,local_data/mendeley/T1_0216_D4.png,"LSS MRI:  feature of muscle spasm.  Dissicating disc mayetrials.  Lt paracentral disc protruion noted at D12/L1 level, indenting spinal cord.  Multilevel diffuse disc bulges noted at L3/L4&L4/L5 levels , compressing thecal sac and encroaching exit neural canals .",0,1
806,local_data/mendeley/T1_0216_D5.png,"LSS MRI:  feature of muscle spasm.  Dissicating disc mayetrials.  Lt paracentral disc protruion noted at D12/L1 level, indenting spinal cord.  Multilevel diffuse disc bulges noted at L3/L4&L4/L5 levels , compressing thecal sac and encroaching exit neural canals .",0,1
807,local_data/mendeley/T2_0216_D3.png,"LSS MRI:  feature of muscle spasm.  Dissicating disc mayetrials.  Lt paracentral disc protruion noted at D12/L1 level, indenting spinal cord.  Multilevel diffuse disc bulges noted at L3/L4&L4/L5 levels , compressing thecal sac and encroaching exit neural canals .",0,1
808,local_data/mendeley/T2_0216_D4.png,"LSS MRI:  feature of muscle spasm.  Dissicating disc mayetrials.  Lt paracentral disc protruion noted at D12/L1 level, indenting spinal cord.  Multilevel diffuse disc bulges noted at L3/L4&L4/L5 levels , compressing thecal sac and encroaching exit neural canals .",0,1
809,local_data/mendeley/T2_0216_D5.png,"LSS MRI:  feature of muscle spasm.  Dissicating disc mayetrials.  Lt paracentral disc protruion noted at D12/L1 level, indenting spinal cord.  Multilevel diffuse disc bulges noted at L3/L4&L4/L5 levels , compressing thecal sac and encroaching exit neural canals .",0,1
810,local_data/mendeley/T1_0217_D3.png,L3-L4: Rt foraminal disc protrusion compressing Rt L4 nerve root. Multilevel mild disc bulges noted elsewhere.,0,1
811,local_data/mendeley/T1_0217_D4.png,L3-L4: Rt foraminal disc protrusion compressing Rt L4 nerve root. Multilevel mild disc bulges noted elsewhere.,0,1
812,local_data/mendeley/T1_0217_D5.png,L3-L4: Rt foraminal disc protrusion compressing Rt L4 nerve root. Multilevel mild disc bulges noted elsewhere.,0,1
813,local_data/mendeley/T2_0217_D3.png,L3-L4: Rt foraminal disc protrusion compressing Rt L4 nerve root. Multilevel mild disc bulges noted elsewhere.,0,1
814,local_data/mendeley/T2_0217_D4.png,L3-L4: Rt foraminal disc protrusion compressing Rt L4 nerve root. Multilevel mild disc bulges noted elsewhere.,0,1
815,local_data/mendeley/T2_0217_D5.png,L3-L4: Rt foraminal disc protrusion compressing Rt L4 nerve root. Multilevel mild disc bulges noted elsewhere.,0,1
816,local_data/mendeley/T1_0218_D3.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level. mildly compressing the thecal sac and exit nerve roots,0,1
817,local_data/mendeley/T1_0218_D4.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level. mildly compressing the thecal sac and exit nerve roots,0,1
818,local_data/mendeley/T1_0218_D5.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level. mildly compressing the thecal sac and exit nerve roots,0,1
819,local_data/mendeley/T2_0218_D3.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level. mildly compressing the thecal sac and exit nerve roots,0,1
820,local_data/mendeley/T2_0218_D4.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level. mildly compressing the thecal sac and exit nerve roots,0,1
821,local_data/mendeley/T2_0218_D5.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level. mildly compressing the thecal sac and exit nerve roots,0,1
822,local_data/mendeley/T1_0219_D3.png,"L5-S1: Mild disc bulge noted. L4-L5: Diffuse disc bulge with Lt posterolateral annular tear, abutting the thecal sac and encroaching Lt exit canal.",0,1
823,local_data/mendeley/T1_0219_D4.png,"L5-S1: Mild disc bulge noted. L4-L5: Diffuse disc bulge with Lt posterolateral annular tear, abutting the thecal sac and encroaching Lt exit canal.",0,1
824,local_data/mendeley/T1_0219_D5.png,"L5-S1: Mild disc bulge noted. L4-L5: Diffuse disc bulge with Lt posterolateral annular tear, abutting the thecal sac and encroaching Lt exit canal.",0,1
825,local_data/mendeley/T2_0219_D3.png,"L5-S1: Mild disc bulge noted. L4-L5: Diffuse disc bulge with Lt posterolateral annular tear, abutting the thecal sac and encroaching Lt exit canal.",0,1
826,local_data/mendeley/T2_0219_D4.png,"L5-S1: Mild disc bulge noted. L4-L5: Diffuse disc bulge with Lt posterolateral annular tear, abutting the thecal sac and encroaching Lt exit canal.",0,1
827,local_data/mendeley/T2_0219_D5.png,"L5-S1: Mild disc bulge noted. L4-L5: Diffuse disc bulge with Lt posterolateral annular tear, abutting the thecal sac and encroaching Lt exit canal.",0,1
828,local_data/mendeley/T1_0220_D3.png,LSS MRI  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4 AND L4-L5: Mild disc bulges noted.,0,1
829,local_data/mendeley/T1_0220_D4.png,LSS MRI  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4 AND L4-L5: Mild disc bulges noted.,0,1
830,local_data/mendeley/T1_0220_D5.png,LSS MRI  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4 AND L4-L5: Mild disc bulges noted.,0,1
831,local_data/mendeley/T2_0220_D3.png,LSS MRI  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4 AND L4-L5: Mild disc bulges noted.,0,1
832,local_data/mendeley/T2_0220_D4.png,LSS MRI  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4 AND L4-L5: Mild disc bulges noted.,0,1
833,local_data/mendeley/T2_0220_D5.png,LSS MRI  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4 AND L4-L5: Mild disc bulges noted.,0,1
834,local_data/mendeley/T1_0221_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
835,local_data/mendeley/T1_0221_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
836,local_data/mendeley/T1_0221_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
837,local_data/mendeley/T2_0221_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
838,local_data/mendeley/T2_0221_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
839,local_data/mendeley/T2_0221_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
840,local_data/mendeley/T1_0224_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildy compressint the thecal sac and exit nerve root.",0,1
841,local_data/mendeley/T1_0224_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildy compressint the thecal sac and exit nerve root.",0,1
842,local_data/mendeley/T1_0224_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildy compressint the thecal sac and exit nerve root.",0,1
843,local_data/mendeley/T2_0224_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildy compressint the thecal sac and exit nerve root.",0,1
844,local_data/mendeley/T2_0224_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildy compressint the thecal sac and exit nerve root.",0,1
845,local_data/mendeley/T2_0224_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildy compressint the thecal sac and exit nerve root.",0,1
846,local_data/mendeley/T1_0226_D3.png,LSS MRI Multilevel mild disc bulges noted. no significant thecal sac or nerve root compression noted. adequate spinal canal.,0,1
847,local_data/mendeley/T1_0226_D4.png,LSS MRI Multilevel mild disc bulges noted. no significant thecal sac or nerve root compression noted. adequate spinal canal.,0,1
848,local_data/mendeley/T1_0226_D5.png,LSS MRI Multilevel mild disc bulges noted. no significant thecal sac or nerve root compression noted. adequate spinal canal.,0,1
849,local_data/mendeley/T2_0226_D3.png,LSS MRI Multilevel mild disc bulges noted. no significant thecal sac or nerve root compression noted. adequate spinal canal.,0,1
850,local_data/mendeley/T2_0226_D4.png,LSS MRI Multilevel mild disc bulges noted. no significant thecal sac or nerve root compression noted. adequate spinal canal.,0,1
851,local_data/mendeley/T2_0226_D5.png,LSS MRI Multilevel mild disc bulges noted. no significant thecal sac or nerve root compression noted. adequate spinal canal.,0,1
852,local_data/mendeley/T1_0227_D3.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
853,local_data/mendeley/T1_0227_D4.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
854,local_data/mendeley/T1_0227_D5.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
855,local_data/mendeley/T2_0227_D3.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
856,local_data/mendeley/T2_0227_D4.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
857,local_data/mendeley/T2_0227_D5.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
858,local_data/mendeley/T1_0232_D3.png,Lumbosacral MRI  Diffuse disc bulge more to the left side at L5-S1 level compressing the thecal sac and left exit neural canal  Diffuse disc bulge noted at L3-L4 level compressing the thecal sac and encroaching upon exit neural canals,0,1
859,local_data/mendeley/T1_0232_D4.png,Lumbosacral MRI  Diffuse disc bulge more to the left side at L5-S1 level compressing the thecal sac and left exit neural canal  Diffuse disc bulge noted at L3-L4 level compressing the thecal sac and encroaching upon exit neural canals,0,1
860,local_data/mendeley/T1_0232_D5.png,Lumbosacral MRI  Diffuse disc bulge more to the left side at L5-S1 level compressing the thecal sac and left exit neural canal  Diffuse disc bulge noted at L3-L4 level compressing the thecal sac and encroaching upon exit neural canals,0,1
861,local_data/mendeley/T2_0232_D3.png,Lumbosacral MRI  Diffuse disc bulge more to the left side at L5-S1 level compressing the thecal sac and left exit neural canal  Diffuse disc bulge noted at L3-L4 level compressing the thecal sac and encroaching upon exit neural canals,0,1
862,local_data/mendeley/T2_0232_D4.png,Lumbosacral MRI  Diffuse disc bulge more to the left side at L5-S1 level compressing the thecal sac and left exit neural canal  Diffuse disc bulge noted at L3-L4 level compressing the thecal sac and encroaching upon exit neural canals,0,1
863,local_data/mendeley/T2_0232_D5.png,Lumbosacral MRI  Diffuse disc bulge more to the left side at L5-S1 level compressing the thecal sac and left exit neural canal  Diffuse disc bulge noted at L3-L4 level compressing the thecal sac and encroaching upon exit neural canals,0,1
864,local_data/mendeley/T1_0233_D3.png,"LSS MRI  Feature of muscle spasm.  Partial dissecating disc materials.  Multilevel diffuse disc bulges noted at L2/L3,L3/L4&L5/S1 level, compressing thecal sac and encroaching exit neural canal",0,1
865,local_data/mendeley/T1_0233_D4.png,"LSS MRI  Feature of muscle spasm.  Partial dissecating disc materials.  Multilevel diffuse disc bulges noted at L2/L3,L3/L4&L5/S1 level, compressing thecal sac and encroaching exit neural canal",0,1
866,local_data/mendeley/T1_0233_D5.png,"LSS MRI  Feature of muscle spasm.  Partial dissecating disc materials.  Multilevel diffuse disc bulges noted at L2/L3,L3/L4&L5/S1 level, compressing thecal sac and encroaching exit neural canal",0,1
867,local_data/mendeley/T2_0233_D3.png,"LSS MRI  Feature of muscle spasm.  Partial dissecating disc materials.  Multilevel diffuse disc bulges noted at L2/L3,L3/L4&L5/S1 level, compressing thecal sac and encroaching exit neural canal",0,1
868,local_data/mendeley/T2_0233_D4.png,"LSS MRI  Feature of muscle spasm.  Partial dissecating disc materials.  Multilevel diffuse disc bulges noted at L2/L3,L3/L4&L5/S1 level, compressing thecal sac and encroaching exit neural canal",0,1
869,local_data/mendeley/T2_0233_D5.png,"LSS MRI  Feature of muscle spasm.  Partial dissecating disc materials.  Multilevel diffuse disc bulges noted at L2/L3,L3/L4&L5/S1 level, compressing thecal sac and encroaching exit neural canal",0,1
870,local_data/mendeley/T1_0234_D3.png,"LSS MRI:  -Evidence of bone marrow edema involving L1 vertebral body , extending to the Lt sided posterior element ?? bone isotope scan is advised,  -Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots , with ligamentum flavum hyperatrophy , leading to secandary spinal canal stensois .  Narrowed disc space of L5/S1 level, with diffuse disc bulge , more to Lt sided compressing thecal sac and nerve roots.  D12 vertebral body hemangioma ?",0,1
871,local_data/mendeley/T1_0234_D4.png,"LSS MRI:  -Evidence of bone marrow edema involving L1 vertebral body , extending to the Lt sided posterior element ?? bone isotope scan is advised,  -Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots , with ligamentum flavum hyperatrophy , leading to secandary spinal canal stensois .  Narrowed disc space of L5/S1 level, with diffuse disc bulge , more to Lt sided compressing thecal sac and nerve roots.  D12 vertebral body hemangioma ?",0,1
872,local_data/mendeley/T1_0234_D5.png,"LSS MRI:  -Evidence of bone marrow edema involving L1 vertebral body , extending to the Lt sided posterior element ?? bone isotope scan is advised,  -Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots , with ligamentum flavum hyperatrophy , leading to secandary spinal canal stensois .  Narrowed disc space of L5/S1 level, with diffuse disc bulge , more to Lt sided compressing thecal sac and nerve roots.  D12 vertebral body hemangioma ?",0,1
873,local_data/mendeley/T2_0234_D3.png,"LSS MRI:  -Evidence of bone marrow edema involving L1 vertebral body , extending to the Lt sided posterior element ?? bone isotope scan is advised,  -Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots , with ligamentum flavum hyperatrophy , leading to secandary spinal canal stensois .  Narrowed disc space of L5/S1 level, with diffuse disc bulge , more to Lt sided compressing thecal sac and nerve roots.  D12 vertebral body hemangioma ?",0,1
874,local_data/mendeley/T2_0234_D4.png,"LSS MRI:  -Evidence of bone marrow edema involving L1 vertebral body , extending to the Lt sided posterior element ?? bone isotope scan is advised,  -Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots , with ligamentum flavum hyperatrophy , leading to secandary spinal canal stensois .  Narrowed disc space of L5/S1 level, with diffuse disc bulge , more to Lt sided compressing thecal sac and nerve roots.  D12 vertebral body hemangioma ?",0,1
875,local_data/mendeley/T2_0234_D5.png,"LSS MRI:  -Evidence of bone marrow edema involving L1 vertebral body , extending to the Lt sided posterior element ?? bone isotope scan is advised,  -Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots , with ligamentum flavum hyperatrophy , leading to secandary spinal canal stensois .  Narrowed disc space of L5/S1 level, with diffuse disc bulge , more to Lt sided compressing thecal sac and nerve roots.  D12 vertebral body hemangioma ?",0,1
876,local_data/mendeley/T1_0235_D3.png,Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and right exiting neural canal Diffuse disc bulge noted at L4-L5 level encroaching  upon the thecal sac and exiting neural canals,0,1
877,local_data/mendeley/T1_0235_D4.png,Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and right exiting neural canal Diffuse disc bulge noted at L4-L5 level encroaching  upon the thecal sac and exiting neural canals,0,1
878,local_data/mendeley/T1_0235_D5.png,Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and right exiting neural canal Diffuse disc bulge noted at L4-L5 level encroaching  upon the thecal sac and exiting neural canals,0,1
879,local_data/mendeley/T2_0235_D3.png,Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and right exiting neural canal Diffuse disc bulge noted at L4-L5 level encroaching  upon the thecal sac and exiting neural canals,0,1
880,local_data/mendeley/T2_0235_D4.png,Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and right exiting neural canal Diffuse disc bulge noted at L4-L5 level encroaching  upon the thecal sac and exiting neural canals,0,1
881,local_data/mendeley/T2_0235_D5.png,Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and right exiting neural canal Diffuse disc bulge noted at L4-L5 level encroaching  upon the thecal sac and exiting neural canals,0,1
882,local_data/mendeley/T1_0236_D3.png,LSS MRI : feature of muscle spasm. multiple schmorls nodules. Left  sided foraminal lumbar disc protrusion  noted at L5/S1 level compressing Lt sided thecal sac and the exiting nerve root.,0,1
883,local_data/mendeley/T1_0236_D4.png,LSS MRI : feature of muscle spasm. multiple schmorls nodules. Left  sided foraminal lumbar disc protrusion  noted at L5/S1 level compressing Lt sided thecal sac and the exiting nerve root.,0,1
884,local_data/mendeley/T1_0236_D5.png,LSS MRI : feature of muscle spasm. multiple schmorls nodules. Left  sided foraminal lumbar disc protrusion  noted at L5/S1 level compressing Lt sided thecal sac and the exiting nerve root.,0,1
885,local_data/mendeley/T2_0236_D3.png,LSS MRI : feature of muscle spasm. multiple schmorls nodules. Left  sided foraminal lumbar disc protrusion  noted at L5/S1 level compressing Lt sided thecal sac and the exiting nerve root.,0,1
886,local_data/mendeley/T2_0236_D4.png,LSS MRI : feature of muscle spasm. multiple schmorls nodules. Left  sided foraminal lumbar disc protrusion  noted at L5/S1 level compressing Lt sided thecal sac and the exiting nerve root.,0,1
887,local_data/mendeley/T2_0236_D5.png,LSS MRI : feature of muscle spasm. multiple schmorls nodules. Left  sided foraminal lumbar disc protrusion  noted at L5/S1 level compressing Lt sided thecal sac and the exiting nerve root.,0,1
888,local_data/mendeley/T1_0239_D3.png,"LSS MRI L5-S1 level: Lt paracentral disc protrusion noted, compressing the thecal sac and abutting the Lt S1 nerve root.",0,1
889,local_data/mendeley/T1_0239_D4.png,"LSS MRI L5-S1 level: Lt paracentral disc protrusion noted, compressing the thecal sac and abutting the Lt S1 nerve root.",0,1
890,local_data/mendeley/T1_0239_D5.png,"LSS MRI L5-S1 level: Lt paracentral disc protrusion noted, compressing the thecal sac and abutting the Lt S1 nerve root.",0,1
891,local_data/mendeley/T2_0239_D3.png,"LSS MRI L5-S1 level: Lt paracentral disc protrusion noted, compressing the thecal sac and abutting the Lt S1 nerve root.",0,1
892,local_data/mendeley/T2_0239_D4.png,"LSS MRI L5-S1 level: Lt paracentral disc protrusion noted, compressing the thecal sac and abutting the Lt S1 nerve root.",0,1
893,local_data/mendeley/T2_0239_D5.png,"LSS MRI L5-S1 level: Lt paracentral disc protrusion noted, compressing the thecal sac and abutting the Lt S1 nerve root.",0,1
894,local_data/mendeley/T1_0240_D3.png,"LSS MRI  L5-S1: central and more to the Lt side, disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  L4-L5: diffuse disc bulge noted with Rt foraminal disc protrusion, compressing the thecal sac and Rt L5 nerve root",0,1
895,local_data/mendeley/T1_0240_D4.png,"LSS MRI  L5-S1: central and more to the Lt side, disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  L4-L5: diffuse disc bulge noted with Rt foraminal disc protrusion, compressing the thecal sac and Rt L5 nerve root",0,1
896,local_data/mendeley/T1_0240_D5.png,"LSS MRI  L5-S1: central and more to the Lt side, disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  L4-L5: diffuse disc bulge noted with Rt foraminal disc protrusion, compressing the thecal sac and Rt L5 nerve root",0,1
897,local_data/mendeley/T2_0240_D3.png,"LSS MRI  L5-S1: central and more to the Lt side, disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  L4-L5: diffuse disc bulge noted with Rt foraminal disc protrusion, compressing the thecal sac and Rt L5 nerve root",0,1
898,local_data/mendeley/T2_0240_D4.png,"LSS MRI  L5-S1: central and more to the Lt side, disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  L4-L5: diffuse disc bulge noted with Rt foraminal disc protrusion, compressing the thecal sac and Rt L5 nerve root",0,1
899,local_data/mendeley/T2_0240_D5.png,"LSS MRI  L5-S1: central and more to the Lt side, disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  L4-L5: diffuse disc bulge noted with Rt foraminal disc protrusion, compressing the thecal sac and Rt L5 nerve root",0,1
900,local_data/mendeley/T1_0242_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 levels, mild compressing the thecal sac and both nerve roots .",0,1
901,local_data/mendeley/T1_0242_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 levels, mild compressing the thecal sac and both nerve roots .",0,1
902,local_data/mendeley/T1_0242_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 levels, mild compressing the thecal sac and both nerve roots .",0,1
903,local_data/mendeley/T2_0242_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 levels, mild compressing the thecal sac and both nerve roots .",0,1
904,local_data/mendeley/T2_0242_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 levels, mild compressing the thecal sac and both nerve roots .",0,1
905,local_data/mendeley/T2_0242_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 levels, mild compressing the thecal sac and both nerve roots .",0,1
906,local_data/mendeley/T1_0244_D3.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
907,local_data/mendeley/T1_0244_D4.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
908,local_data/mendeley/T1_0244_D5.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
909,local_data/mendeley/T2_0244_D3.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
910,local_data/mendeley/T2_0244_D4.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
911,local_data/mendeley/T2_0244_D5.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
912,local_data/mendeley/T1_0246_D3.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
913,local_data/mendeley/T1_0246_D4.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
914,local_data/mendeley/T1_0246_D5.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
915,local_data/mendeley/T2_0246_D3.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
916,local_data/mendeley/T2_0246_D4.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
917,local_data/mendeley/T2_0246_D5.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
918,local_data/mendeley/T1_0247_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L3/L4& L4/L5 levels, mildly compressing the thecal sac and exit nerve roots",0,1
919,local_data/mendeley/T1_0247_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L3/L4& L4/L5 levels, mildly compressing the thecal sac and exit nerve roots",0,1
920,local_data/mendeley/T1_0247_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L3/L4& L4/L5 levels, mildly compressing the thecal sac and exit nerve roots",0,1
921,local_data/mendeley/T2_0247_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L3/L4& L4/L5 levels, mildly compressing the thecal sac and exit nerve roots",0,1
922,local_data/mendeley/T2_0247_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L3/L4& L4/L5 levels, mildly compressing the thecal sac and exit nerve roots",0,1
923,local_data/mendeley/T2_0247_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L3/L4& L4/L5 levels, mildly compressing the thecal sac and exit nerve roots",0,1
924,local_data/mendeley/T1_0249_D3.png,"MRI OF THE LSS  Feature of muscle spasm.  Spondylolisthesis of L5 over L5 level , with Modec type II endplate .as well as dissecated disc material.  Diffuse disc bulge noted at L4/L5 level compressing thecal sac and both exit nerve root .  Mild diffuse disc bulge noted at L3/L4 level ,mild compressing thecal sac .",0,1
925,local_data/mendeley/T1_0249_D4.png,"MRI OF THE LSS  Feature of muscle spasm.  Spondylolisthesis of L5 over L5 level , with Modec type II endplate .as well as dissecated disc material.  Diffuse disc bulge noted at L4/L5 level compressing thecal sac and both exit nerve root .  Mild diffuse disc bulge noted at L3/L4 level ,mild compressing thecal sac .",0,1
926,local_data/mendeley/T1_0249_D5.png,"MRI OF THE LSS  Feature of muscle spasm.  Spondylolisthesis of L5 over L5 level , with Modec type II endplate .as well as dissecated disc material.  Diffuse disc bulge noted at L4/L5 level compressing thecal sac and both exit nerve root .  Mild diffuse disc bulge noted at L3/L4 level ,mild compressing thecal sac .",0,1
927,local_data/mendeley/T2_0249_D3.png,"MRI OF THE LSS  Feature of muscle spasm.  Spondylolisthesis of L5 over L5 level , with Modec type II endplate .as well as dissecated disc material.  Diffuse disc bulge noted at L4/L5 level compressing thecal sac and both exit nerve root .  Mild diffuse disc bulge noted at L3/L4 level ,mild compressing thecal sac .",0,1
928,local_data/mendeley/T2_0249_D4.png,"MRI OF THE LSS  Feature of muscle spasm.  Spondylolisthesis of L5 over L5 level , with Modec type II endplate .as well as dissecated disc material.  Diffuse disc bulge noted at L4/L5 level compressing thecal sac and both exit nerve root .  Mild diffuse disc bulge noted at L3/L4 level ,mild compressing thecal sac .",0,1
929,local_data/mendeley/T2_0249_D5.png,"MRI OF THE LSS  Feature of muscle spasm.  Spondylolisthesis of L5 over L5 level , with Modec type II endplate .as well as dissecated disc material.  Diffuse disc bulge noted at L4/L5 level compressing thecal sac and both exit nerve root .  Mild diffuse disc bulge noted at L3/L4 level ,mild compressing thecal sac .",0,1
930,local_data/mendeley/T1_0251_D3.png,"LSS MRI : Feature of muscle spasm. Multilevel diffuse disc bulge noted , noticeably at  L3/L4&  at L4/L5 levels, relatively compressing  thecal sac and both exit nerve roots . NB:Tarlov cysts noted mainly at Rt side",0,1
931,local_data/mendeley/T1_0251_D4.png,"LSS MRI : Feature of muscle spasm. Multilevel diffuse disc bulge noted , noticeably at  L3/L4&  at L4/L5 levels, relatively compressing  thecal sac and both exit nerve roots . NB:Tarlov cysts noted mainly at Rt side",0,1
932,local_data/mendeley/T1_0251_D5.png,"LSS MRI : Feature of muscle spasm. Multilevel diffuse disc bulge noted , noticeably at  L3/L4&  at L4/L5 levels, relatively compressing  thecal sac and both exit nerve roots . NB:Tarlov cysts noted mainly at Rt side",0,1
933,local_data/mendeley/T2_0251_D3.png,"LSS MRI : Feature of muscle spasm. Multilevel diffuse disc bulge noted , noticeably at  L3/L4&  at L4/L5 levels, relatively compressing  thecal sac and both exit nerve roots . NB:Tarlov cysts noted mainly at Rt side",0,1
934,local_data/mendeley/T2_0251_D4.png,"LSS MRI : Feature of muscle spasm. Multilevel diffuse disc bulge noted , noticeably at  L3/L4&  at L4/L5 levels, relatively compressing  thecal sac and both exit nerve roots . NB:Tarlov cysts noted mainly at Rt side",0,1
935,local_data/mendeley/T2_0251_D5.png,"LSS MRI : Feature of muscle spasm. Multilevel diffuse disc bulge noted , noticeably at  L3/L4&  at L4/L5 levels, relatively compressing  thecal sac and both exit nerve roots . NB:Tarlov cysts noted mainly at Rt side",0,1
936,local_data/mendeley/T1_0252_D3.png,"LSS MRI L Diffusely reduced signal of the bone marrow noted , correalte with bone isotope scan ?. faeture of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and both nerve roots .",0,1
937,local_data/mendeley/T1_0252_D4.png,"LSS MRI L Diffusely reduced signal of the bone marrow noted , correalte with bone isotope scan ?. faeture of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and both nerve roots .",0,1
938,local_data/mendeley/T1_0252_D5.png,"LSS MRI L Diffusely reduced signal of the bone marrow noted , correalte with bone isotope scan ?. faeture of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and both nerve roots .",0,1
939,local_data/mendeley/T2_0252_D3.png,"LSS MRI L Diffusely reduced signal of the bone marrow noted , correalte with bone isotope scan ?. faeture of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and both nerve roots .",0,1
940,local_data/mendeley/T2_0252_D4.png,"LSS MRI L Diffusely reduced signal of the bone marrow noted , correalte with bone isotope scan ?. faeture of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and both nerve roots .",0,1
941,local_data/mendeley/T2_0252_D5.png,"LSS MRI L Diffusely reduced signal of the bone marrow noted , correalte with bone isotope scan ?. faeture of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and both nerve roots .",0,1
942,local_data/mendeley/T1_0253_D3.png,"Lumbosacral MRI Diffuse disc bulge noted at L3-L4 level significantly compressing the thecal sac ,extending to Lt lateral recess    and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac nerve roots",0,1
943,local_data/mendeley/T1_0253_D4.png,"Lumbosacral MRI Diffuse disc bulge noted at L3-L4 level significantly compressing the thecal sac ,extending to Lt lateral recess    and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac nerve roots",0,1
944,local_data/mendeley/T1_0253_D5.png,"Lumbosacral MRI Diffuse disc bulge noted at L3-L4 level significantly compressing the thecal sac ,extending to Lt lateral recess    and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac nerve roots",0,1
945,local_data/mendeley/T2_0253_D3.png,"Lumbosacral MRI Diffuse disc bulge noted at L3-L4 level significantly compressing the thecal sac ,extending to Lt lateral recess    and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac nerve roots",0,1
946,local_data/mendeley/T2_0253_D4.png,"Lumbosacral MRI Diffuse disc bulge noted at L3-L4 level significantly compressing the thecal sac ,extending to Lt lateral recess    and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac nerve roots",0,1
947,local_data/mendeley/T2_0253_D5.png,"Lumbosacral MRI Diffuse disc bulge noted at L3-L4 level significantly compressing the thecal sac ,extending to Lt lateral recess    and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac nerve roots",0,1
948,local_data/mendeley/T1_0255_D3.png,"Feature of muscle spasm.  Multilevel dissecating disc materials.  Diffuse disc bulge noted at L3/L4 level , compressing thecal and exit nerve roots more to Rt .  Diffuse disc bulge noted at L4/L5 compressing the thecal sac and exit nerves root more to Lt .",0,1
949,local_data/mendeley/T1_0255_D4.png,"Feature of muscle spasm.  Multilevel dissecating disc materials.  Diffuse disc bulge noted at L3/L4 level , compressing thecal and exit nerve roots more to Rt .  Diffuse disc bulge noted at L4/L5 compressing the thecal sac and exit nerves root more to Lt .",0,1
950,local_data/mendeley/T1_0255_D5.png,"Feature of muscle spasm.  Multilevel dissecating disc materials.  Diffuse disc bulge noted at L3/L4 level , compressing thecal and exit nerve roots more to Rt .  Diffuse disc bulge noted at L4/L5 compressing the thecal sac and exit nerves root more to Lt .",0,1
951,local_data/mendeley/T2_0255_D3.png,"Feature of muscle spasm.  Multilevel dissecating disc materials.  Diffuse disc bulge noted at L3/L4 level , compressing thecal and exit nerve roots more to Rt .  Diffuse disc bulge noted at L4/L5 compressing the thecal sac and exit nerves root more to Lt .",0,1
952,local_data/mendeley/T2_0255_D4.png,"Feature of muscle spasm.  Multilevel dissecating disc materials.  Diffuse disc bulge noted at L3/L4 level , compressing thecal and exit nerve roots more to Rt .  Diffuse disc bulge noted at L4/L5 compressing the thecal sac and exit nerves root more to Lt .",0,1
953,local_data/mendeley/T2_0255_D5.png,"Feature of muscle spasm.  Multilevel dissecating disc materials.  Diffuse disc bulge noted at L3/L4 level , compressing thecal and exit nerve roots more to Rt .  Diffuse disc bulge noted at L4/L5 compressing the thecal sac and exit nerves root more to Lt .",0,1
954,local_data/mendeley/T1_0256_D3.png,D SPINE MRI Normal bony alignment. No significant spinal cord or nerve root compression noted..,1,0
955,local_data/mendeley/T1_0256_D4.png,D SPINE MRI Normal bony alignment. No significant spinal cord or nerve root compression noted..,1,0
956,local_data/mendeley/T1_0256_D5.png,D SPINE MRI Normal bony alignment. No significant spinal cord or nerve root compression noted..,1,0
957,local_data/mendeley/T2_0256_D3.png,D SPINE MRI Normal bony alignment. No significant spinal cord or nerve root compression noted..,1,0
958,local_data/mendeley/T2_0256_D4.png,D SPINE MRI Normal bony alignment. No significant spinal cord or nerve root compression noted..,1,0
959,local_data/mendeley/T2_0256_D5.png,D SPINE MRI Normal bony alignment. No significant spinal cord or nerve root compression noted..,1,0
960,local_data/mendeley/T1_0257_D3.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac.  the spinal canal appeared adequate.",0,1
961,local_data/mendeley/T1_0257_D4.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac.  the spinal canal appeared adequate.",0,1
962,local_data/mendeley/T1_0257_D5.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac.  the spinal canal appeared adequate.",0,1
963,local_data/mendeley/T2_0257_D3.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac.  the spinal canal appeared adequate.",0,1
964,local_data/mendeley/T2_0257_D4.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac.  the spinal canal appeared adequate.",0,1
965,local_data/mendeley/T2_0257_D5.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac.  the spinal canal appeared adequate.",0,1
966,local_data/mendeley/T1_0258_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and both nerve roots",0,1
967,local_data/mendeley/T1_0258_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and both nerve roots",0,1
968,local_data/mendeley/T1_0258_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and both nerve roots",0,1
969,local_data/mendeley/T2_0258_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and both nerve roots",0,1
970,local_data/mendeley/T2_0258_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and both nerve roots",0,1
971,local_data/mendeley/T2_0258_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and both nerve roots",0,1
972,local_data/mendeley/T1_0259_D3.png,"MRI OF THE LUMBAR SPINE :  Feature of muscle spasm.  L4/L5 level: central and Rt paracentral disc protrusion noted ,significantly compressing the thecal sac and exit neural canals, associated with annular tear .  L5/S1 level: Rt paracentral disc protrusion noted compressing the thecal sac and exit neural canals.",0,1
973,local_data/mendeley/T1_0259_D4.png,"MRI OF THE LUMBAR SPINE :  Feature of muscle spasm.  L4/L5 level: central and Rt paracentral disc protrusion noted ,significantly compressing the thecal sac and exit neural canals, associated with annular tear .  L5/S1 level: Rt paracentral disc protrusion noted compressing the thecal sac and exit neural canals.",0,1
974,local_data/mendeley/T1_0259_D5.png,"MRI OF THE LUMBAR SPINE :  Feature of muscle spasm.  L4/L5 level: central and Rt paracentral disc protrusion noted ,significantly compressing the thecal sac and exit neural canals, associated with annular tear .  L5/S1 level: Rt paracentral disc protrusion noted compressing the thecal sac and exit neural canals.",0,1
975,local_data/mendeley/T2_0259_D3.png,"MRI OF THE LUMBAR SPINE :  Feature of muscle spasm.  L4/L5 level: central and Rt paracentral disc protrusion noted ,significantly compressing the thecal sac and exit neural canals, associated with annular tear .  L5/S1 level: Rt paracentral disc protrusion noted compressing the thecal sac and exit neural canals.",0,1
976,local_data/mendeley/T2_0259_D4.png,"MRI OF THE LUMBAR SPINE :  Feature of muscle spasm.  L4/L5 level: central and Rt paracentral disc protrusion noted ,significantly compressing the thecal sac and exit neural canals, associated with annular tear .  L5/S1 level: Rt paracentral disc protrusion noted compressing the thecal sac and exit neural canals.",0,1
977,local_data/mendeley/T2_0259_D5.png,"MRI OF THE LUMBAR SPINE :  Feature of muscle spasm.  L4/L5 level: central and Rt paracentral disc protrusion noted ,significantly compressing the thecal sac and exit neural canals, associated with annular tear .  L5/S1 level: Rt paracentral disc protrusion noted compressing the thecal sac and exit neural canals.",0,1
978,local_data/mendeley/T1_0263_D3.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolisthesis of L5 over S1 level  Dissicating disc materails noted,  Multilevel diffuse disc bulges, notceably at L4/L5 level,compressing thecal sac and nerve roots .  About 3*1.5 cm, Tarlov cyst noted at level of S3 , with impression on posterior aspect.",0,1
979,local_data/mendeley/T1_0263_D4.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolisthesis of L5 over S1 level  Dissicating disc materails noted,  Multilevel diffuse disc bulges, notceably at L4/L5 level,compressing thecal sac and nerve roots .  About 3*1.5 cm, Tarlov cyst noted at level of S3 , with impression on posterior aspect.",0,1
980,local_data/mendeley/T1_0263_D5.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolisthesis of L5 over S1 level  Dissicating disc materails noted,  Multilevel diffuse disc bulges, notceably at L4/L5 level,compressing thecal sac and nerve roots .  About 3*1.5 cm, Tarlov cyst noted at level of S3 , with impression on posterior aspect.",0,1
981,local_data/mendeley/T2_0263_D3.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolisthesis of L5 over S1 level  Dissicating disc materails noted,  Multilevel diffuse disc bulges, notceably at L4/L5 level,compressing thecal sac and nerve roots .  About 3*1.5 cm, Tarlov cyst noted at level of S3 , with impression on posterior aspect.",0,1
982,local_data/mendeley/T2_0263_D4.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolisthesis of L5 over S1 level  Dissicating disc materails noted,  Multilevel diffuse disc bulges, notceably at L4/L5 level,compressing thecal sac and nerve roots .  About 3*1.5 cm, Tarlov cyst noted at level of S3 , with impression on posterior aspect.",0,1
983,local_data/mendeley/T2_0263_D5.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolisthesis of L5 over S1 level  Dissicating disc materails noted,  Multilevel diffuse disc bulges, notceably at L4/L5 level,compressing thecal sac and nerve roots .  About 3*1.5 cm, Tarlov cyst noted at level of S3 , with impression on posterior aspect.",0,1
984,local_data/mendeley/T1_0264_D3.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
985,local_data/mendeley/T1_0264_D4.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
986,local_data/mendeley/T1_0264_D5.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
987,local_data/mendeley/T2_0264_D3.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
988,local_data/mendeley/T2_0264_D4.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
989,local_data/mendeley/T2_0264_D5.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
990,local_data/mendeley/T1_0265_D3.png,"LSS MRI:  Feature of muscle spasm.  Dissicating disc materals at multilevels.  Endpalte modec type II noted  Large L2 veretbral body hemangioma as well assmaller one s noted.  Posteriocentral disc protruion noted at L5/S1 level, compressing thecal sac and encroaching both neural canals.  Diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots.",0,1
991,local_data/mendeley/T1_0265_D4.png,"LSS MRI:  Feature of muscle spasm.  Dissicating disc materals at multilevels.  Endpalte modec type II noted  Large L2 veretbral body hemangioma as well assmaller one s noted.  Posteriocentral disc protruion noted at L5/S1 level, compressing thecal sac and encroaching both neural canals.  Diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots.",0,1
992,local_data/mendeley/T1_0265_D5.png,"LSS MRI:  Feature of muscle spasm.  Dissicating disc materals at multilevels.  Endpalte modec type II noted  Large L2 veretbral body hemangioma as well assmaller one s noted.  Posteriocentral disc protruion noted at L5/S1 level, compressing thecal sac and encroaching both neural canals.  Diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots.",0,1
993,local_data/mendeley/T2_0265_D3.png,"LSS MRI:  Feature of muscle spasm.  Dissicating disc materals at multilevels.  Endpalte modec type II noted  Large L2 veretbral body hemangioma as well assmaller one s noted.  Posteriocentral disc protruion noted at L5/S1 level, compressing thecal sac and encroaching both neural canals.  Diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots.",0,1
994,local_data/mendeley/T2_0265_D4.png,"LSS MRI:  Feature of muscle spasm.  Dissicating disc materals at multilevels.  Endpalte modec type II noted  Large L2 veretbral body hemangioma as well assmaller one s noted.  Posteriocentral disc protruion noted at L5/S1 level, compressing thecal sac and encroaching both neural canals.  Diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots.",0,1
995,local_data/mendeley/T2_0265_D5.png,"LSS MRI:  Feature of muscle spasm.  Dissicating disc materals at multilevels.  Endpalte modec type II noted  Large L2 veretbral body hemangioma as well assmaller one s noted.  Posteriocentral disc protruion noted at L5/S1 level, compressing thecal sac and encroaching both neural canals.  Diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots.",0,1
996,local_data/mendeley/T1_0266_D3.png,"L4-L5: Lt paracentral disc protrusion, compressing the thecal sac  T12-L1: Diffuse disc bulge noted abutting the cord",0,1
997,local_data/mendeley/T1_0266_D4.png,"L4-L5: Lt paracentral disc protrusion, compressing the thecal sac  T12-L1: Diffuse disc bulge noted abutting the cord",0,1
998,local_data/mendeley/T1_0266_D5.png,"L4-L5: Lt paracentral disc protrusion, compressing the thecal sac  T12-L1: Diffuse disc bulge noted abutting the cord",0,1
999,local_data/mendeley/T2_0266_D3.png,"L4-L5: Lt paracentral disc protrusion, compressing the thecal sac  T12-L1: Diffuse disc bulge noted abutting the cord",0,1
1000,local_data/mendeley/T2_0266_D4.png,"L4-L5: Lt paracentral disc protrusion, compressing the thecal sac  T12-L1: Diffuse disc bulge noted abutting the cord",0,1
1001,local_data/mendeley/T2_0266_D5.png,"L4-L5: Lt paracentral disc protrusion, compressing the thecal sac  T12-L1: Diffuse disc bulge noted abutting the cord",0,1
1002,local_data/mendeley/T1_0267_D3.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
1003,local_data/mendeley/T1_0267_D4.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
1004,local_data/mendeley/T1_0267_D5.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
1005,local_data/mendeley/T2_0267_D3.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
1006,local_data/mendeley/T2_0267_D4.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
1007,local_data/mendeley/T2_0267_D5.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
1008,local_data/mendeley/T1_0269_D3.png,LSS MRI :  Feature of musclespasm.  No significant thecal sac or nerve roots compression seen,0,1
1009,local_data/mendeley/T1_0269_D4.png,LSS MRI :  Feature of musclespasm.  No significant thecal sac or nerve roots compression seen,0,1
1010,local_data/mendeley/T1_0269_D5.png,LSS MRI :  Feature of musclespasm.  No significant thecal sac or nerve roots compression seen,0,1
1011,local_data/mendeley/T2_0269_D3.png,LSS MRI :  Feature of musclespasm.  No significant thecal sac or nerve roots compression seen,0,1
1012,local_data/mendeley/T2_0269_D4.png,LSS MRI :  Feature of musclespasm.  No significant thecal sac or nerve roots compression seen,0,1
1013,local_data/mendeley/T2_0269_D5.png,LSS MRI :  Feature of musclespasm.  No significant thecal sac or nerve roots compression seen,0,1
1014,local_data/mendeley/T1_0270_D3.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc material at lower levels. -Central disc protrusion noted at L4/L5 level, mild caudally migration, compressing thecal sac and both nerve rootys . -Central and to the the Rt  disc protrusion noted at L5/S1 level, compressing thecal sac and nerve roots",0,1
1015,local_data/mendeley/T1_0270_D4.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc material at lower levels. -Central disc protrusion noted at L4/L5 level, mild caudally migration, compressing thecal sac and both nerve rootys . -Central and to the the Rt  disc protrusion noted at L5/S1 level, compressing thecal sac and nerve roots",0,1
1016,local_data/mendeley/T1_0270_D5.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc material at lower levels. -Central disc protrusion noted at L4/L5 level, mild caudally migration, compressing thecal sac and both nerve rootys . -Central and to the the Rt  disc protrusion noted at L5/S1 level, compressing thecal sac and nerve roots",0,1
1017,local_data/mendeley/T2_0270_D3.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc material at lower levels. -Central disc protrusion noted at L4/L5 level, mild caudally migration, compressing thecal sac and both nerve rootys . -Central and to the the Rt  disc protrusion noted at L5/S1 level, compressing thecal sac and nerve roots",0,1
1018,local_data/mendeley/T2_0270_D4.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc material at lower levels. -Central disc protrusion noted at L4/L5 level, mild caudally migration, compressing thecal sac and both nerve rootys . -Central and to the the Rt  disc protrusion noted at L5/S1 level, compressing thecal sac and nerve roots",0,1
1019,local_data/mendeley/T2_0270_D5.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc material at lower levels. -Central disc protrusion noted at L4/L5 level, mild caudally migration, compressing thecal sac and both nerve rootys . -Central and to the the Rt  disc protrusion noted at L5/S1 level, compressing thecal sac and nerve roots",0,1
1020,local_data/mendeley/T1_0272_D3.png,Cervical spine MRI  Features of muscle spasm  No significant thecal sac or nerve root compression  Normal signal intensity of the spinal cord  Lumbosacral MRI  Small vertebral hemangioma of L3  Diffuse disc bulge noted at the L4-L5 mild comparison thecal sac,0,1
1021,local_data/mendeley/T1_0272_D4.png,Cervical spine MRI  Features of muscle spasm  No significant thecal sac or nerve root compression  Normal signal intensity of the spinal cord  Lumbosacral MRI  Small vertebral hemangioma of L3  Diffuse disc bulge noted at the L4-L5 mild comparison thecal sac,0,1
1022,local_data/mendeley/T1_0272_D5.png,Cervical spine MRI  Features of muscle spasm  No significant thecal sac or nerve root compression  Normal signal intensity of the spinal cord  Lumbosacral MRI  Small vertebral hemangioma of L3  Diffuse disc bulge noted at the L4-L5 mild comparison thecal sac,0,1
1023,local_data/mendeley/T2_0272_D3.png,Cervical spine MRI  Features of muscle spasm  No significant thecal sac or nerve root compression  Normal signal intensity of the spinal cord  Lumbosacral MRI  Small vertebral hemangioma of L3  Diffuse disc bulge noted at the L4-L5 mild comparison thecal sac,0,1
1024,local_data/mendeley/T2_0272_D4.png,Cervical spine MRI  Features of muscle spasm  No significant thecal sac or nerve root compression  Normal signal intensity of the spinal cord  Lumbosacral MRI  Small vertebral hemangioma of L3  Diffuse disc bulge noted at the L4-L5 mild comparison thecal sac,0,1
1025,local_data/mendeley/T2_0272_D5.png,Cervical spine MRI  Features of muscle spasm  No significant thecal sac or nerve root compression  Normal signal intensity of the spinal cord  Lumbosacral MRI  Small vertebral hemangioma of L3  Diffuse disc bulge noted at the L4-L5 mild comparison thecal sac,0,1
1026,local_data/mendeley/T1_0276_D3.png,"LSS MRI : Feature of muscle spasm. Central and to left disc protrusion noted at L2/L3 level, compressing thecal sac and encroaching exit neural canal . Multilevel diffuse disc bulges noted, noteceably at L4/L5&L5/S1 levels , compressing thecal sac and both nerve roots NB:Cystic dilaition of Rt pelvicalyceal syetems ? correalte with renal US",0,1
1027,local_data/mendeley/T1_0276_D4.png,"LSS MRI : Feature of muscle spasm. Central and to left disc protrusion noted at L2/L3 level, compressing thecal sac and encroaching exit neural canal . Multilevel diffuse disc bulges noted, noteceably at L4/L5&L5/S1 levels , compressing thecal sac and both nerve roots NB:Cystic dilaition of Rt pelvicalyceal syetems ? correalte with renal US",0,1
1028,local_data/mendeley/T1_0276_D5.png,"LSS MRI : Feature of muscle spasm. Central and to left disc protrusion noted at L2/L3 level, compressing thecal sac and encroaching exit neural canal . Multilevel diffuse disc bulges noted, noteceably at L4/L5&L5/S1 levels , compressing thecal sac and both nerve roots NB:Cystic dilaition of Rt pelvicalyceal syetems ? correalte with renal US",0,1
1029,local_data/mendeley/T2_0276_D3.png,"LSS MRI : Feature of muscle spasm. Central and to left disc protrusion noted at L2/L3 level, compressing thecal sac and encroaching exit neural canal . Multilevel diffuse disc bulges noted, noteceably at L4/L5&L5/S1 levels , compressing thecal sac and both nerve roots NB:Cystic dilaition of Rt pelvicalyceal syetems ? correalte with renal US",0,1
1030,local_data/mendeley/T2_0276_D4.png,"LSS MRI : Feature of muscle spasm. Central and to left disc protrusion noted at L2/L3 level, compressing thecal sac and encroaching exit neural canal . Multilevel diffuse disc bulges noted, noteceably at L4/L5&L5/S1 levels , compressing thecal sac and both nerve roots NB:Cystic dilaition of Rt pelvicalyceal syetems ? correalte with renal US",0,1
1031,local_data/mendeley/T2_0276_D5.png,"LSS MRI : Feature of muscle spasm. Central and to left disc protrusion noted at L2/L3 level, compressing thecal sac and encroaching exit neural canal . Multilevel diffuse disc bulges noted, noteceably at L4/L5&L5/S1 levels , compressing thecal sac and both nerve roots NB:Cystic dilaition of Rt pelvicalyceal syetems ? correalte with renal US",0,1
1032,local_data/mendeley/T1_0277_D3.png,"LSS MRI : feature of muscle spasm. Diffuse disc bulges noted   L4/L5 & L5/S1 levels , mild compressing thecal sac and nerve roots",0,1
1033,local_data/mendeley/T1_0277_D4.png,"LSS MRI : feature of muscle spasm. Diffuse disc bulges noted   L4/L5 & L5/S1 levels , mild compressing thecal sac and nerve roots",0,1
1034,local_data/mendeley/T1_0277_D5.png,"LSS MRI : feature of muscle spasm. Diffuse disc bulges noted   L4/L5 & L5/S1 levels , mild compressing thecal sac and nerve roots",0,1
1035,local_data/mendeley/T2_0277_D3.png,"LSS MRI : feature of muscle spasm. Diffuse disc bulges noted   L4/L5 & L5/S1 levels , mild compressing thecal sac and nerve roots",0,1
1036,local_data/mendeley/T2_0277_D4.png,"LSS MRI : feature of muscle spasm. Diffuse disc bulges noted   L4/L5 & L5/S1 levels , mild compressing thecal sac and nerve roots",0,1
1037,local_data/mendeley/T2_0277_D5.png,"LSS MRI : feature of muscle spasm. Diffuse disc bulges noted   L4/L5 & L5/S1 levels , mild compressing thecal sac and nerve roots",0,1
1038,local_data/mendeley/T1_0279_D3.png,LSS MRI Small central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
1039,local_data/mendeley/T1_0279_D4.png,LSS MRI Small central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
1040,local_data/mendeley/T1_0279_D5.png,LSS MRI Small central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
1041,local_data/mendeley/T2_0279_D3.png,LSS MRI Small central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
1042,local_data/mendeley/T2_0279_D4.png,LSS MRI Small central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
1043,local_data/mendeley/T2_0279_D5.png,LSS MRI Small central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
1044,local_data/mendeley/T1_0281_D3.png,"L4-L5 AND L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1045,local_data/mendeley/T1_0281_D4.png,"L4-L5 AND L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1046,local_data/mendeley/T1_0281_D5.png,"L4-L5 AND L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1047,local_data/mendeley/T2_0281_D3.png,"L4-L5 AND L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1048,local_data/mendeley/T2_0281_D4.png,"L4-L5 AND L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1049,local_data/mendeley/T2_0281_D5.png,"L4-L5 AND L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1050,local_data/mendeley/T1_0282_D3.png,L5-S1: Central disc protrusion noted compressing the thecal sac and abutting S1 nerve roots.,0,1
1051,local_data/mendeley/T1_0282_D4.png,L5-S1: Central disc protrusion noted compressing the thecal sac and abutting S1 nerve roots.,0,1
1052,local_data/mendeley/T1_0282_D5.png,L5-S1: Central disc protrusion noted compressing the thecal sac and abutting S1 nerve roots.,0,1
1053,local_data/mendeley/T2_0282_D3.png,L5-S1: Central disc protrusion noted compressing the thecal sac and abutting S1 nerve roots.,0,1
1054,local_data/mendeley/T2_0282_D4.png,L5-S1: Central disc protrusion noted compressing the thecal sac and abutting S1 nerve roots.,0,1
1055,local_data/mendeley/T2_0282_D5.png,L5-S1: Central disc protrusion noted compressing the thecal sac and abutting S1 nerve roots.,0,1
1056,local_data/mendeley/T1_0283_D3.png,"LSS MRI : -Feature of muscle spasm. -wide base disc bulge noted at L5/S1 level, compressing the theca sac and both nerve roots . -Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroching exit neural canal.",0,1
1057,local_data/mendeley/T1_0283_D4.png,"LSS MRI : -Feature of muscle spasm. -wide base disc bulge noted at L5/S1 level, compressing the theca sac and both nerve roots . -Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroching exit neural canal.",0,1
1058,local_data/mendeley/T1_0283_D5.png,"LSS MRI : -Feature of muscle spasm. -wide base disc bulge noted at L5/S1 level, compressing the theca sac and both nerve roots . -Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroching exit neural canal.",0,1
1059,local_data/mendeley/T2_0283_D3.png,"LSS MRI : -Feature of muscle spasm. -wide base disc bulge noted at L5/S1 level, compressing the theca sac and both nerve roots . -Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroching exit neural canal.",0,1
1060,local_data/mendeley/T2_0283_D4.png,"LSS MRI : -Feature of muscle spasm. -wide base disc bulge noted at L5/S1 level, compressing the theca sac and both nerve roots . -Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroching exit neural canal.",0,1
1061,local_data/mendeley/T2_0283_D5.png,"LSS MRI : -Feature of muscle spasm. -wide base disc bulge noted at L5/S1 level, compressing the theca sac and both nerve roots . -Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroching exit neural canal.",0,1
1062,local_data/mendeley/T1_0284_D3.png,LSS MRI Feature of muscle spasm Small posterior central disc protrusion noted at L5-S1 level compressing thecal sac and exit  nerve roots(dehydrated disc matreial noted) Diffuse disc bulge noted at the L4-L5 mild compressing thecal sac and encroaching both neural canals,0,1
1063,local_data/mendeley/T1_0284_D4.png,LSS MRI Feature of muscle spasm Small posterior central disc protrusion noted at L5-S1 level compressing thecal sac and exit  nerve roots(dehydrated disc matreial noted) Diffuse disc bulge noted at the L4-L5 mild compressing thecal sac and encroaching both neural canals,0,1
1064,local_data/mendeley/T1_0284_D5.png,LSS MRI Feature of muscle spasm Small posterior central disc protrusion noted at L5-S1 level compressing thecal sac and exit  nerve roots(dehydrated disc matreial noted) Diffuse disc bulge noted at the L4-L5 mild compressing thecal sac and encroaching both neural canals,0,1
1065,local_data/mendeley/T2_0284_D3.png,LSS MRI Feature of muscle spasm Small posterior central disc protrusion noted at L5-S1 level compressing thecal sac and exit  nerve roots(dehydrated disc matreial noted) Diffuse disc bulge noted at the L4-L5 mild compressing thecal sac and encroaching both neural canals,0,1
1066,local_data/mendeley/T2_0284_D4.png,LSS MRI Feature of muscle spasm Small posterior central disc protrusion noted at L5-S1 level compressing thecal sac and exit  nerve roots(dehydrated disc matreial noted) Diffuse disc bulge noted at the L4-L5 mild compressing thecal sac and encroaching both neural canals,0,1
1067,local_data/mendeley/T2_0284_D5.png,LSS MRI Feature of muscle spasm Small posterior central disc protrusion noted at L5-S1 level compressing thecal sac and exit  nerve roots(dehydrated disc matreial noted) Diffuse disc bulge noted at the L4-L5 mild compressing thecal sac and encroaching both neural canals,0,1
1068,local_data/mendeley/T1_0285_D3.png,"LSS MRI : Feature of muscle spasm. Partial loss of L1 vertebral body hieght  anteriorly , with no bone marrow edema , mostly old fracture, with no definite thecal sac or exit neural canal compression . -Focal bony bruise noted at S3-S4 vertebral bodies? post traumatic  -Mild diffuse disc bulge noted at L4/L5 level , mild compressing thecal and exit nerve roots ,",0,1
1069,local_data/mendeley/T1_0285_D4.png,"LSS MRI : Feature of muscle spasm. Partial loss of L1 vertebral body hieght  anteriorly , with no bone marrow edema , mostly old fracture, with no definite thecal sac or exit neural canal compression . -Focal bony bruise noted at S3-S4 vertebral bodies? post traumatic  -Mild diffuse disc bulge noted at L4/L5 level , mild compressing thecal and exit nerve roots ,",0,1
1070,local_data/mendeley/T1_0285_D5.png,"LSS MRI : Feature of muscle spasm. Partial loss of L1 vertebral body hieght  anteriorly , with no bone marrow edema , mostly old fracture, with no definite thecal sac or exit neural canal compression . -Focal bony bruise noted at S3-S4 vertebral bodies? post traumatic  -Mild diffuse disc bulge noted at L4/L5 level , mild compressing thecal and exit nerve roots ,",0,1
1071,local_data/mendeley/T2_0285_D3.png,"LSS MRI : Feature of muscle spasm. Partial loss of L1 vertebral body hieght  anteriorly , with no bone marrow edema , mostly old fracture, with no definite thecal sac or exit neural canal compression . -Focal bony bruise noted at S3-S4 vertebral bodies? post traumatic  -Mild diffuse disc bulge noted at L4/L5 level , mild compressing thecal and exit nerve roots ,",0,1
1072,local_data/mendeley/T2_0285_D4.png,"LSS MRI : Feature of muscle spasm. Partial loss of L1 vertebral body hieght  anteriorly , with no bone marrow edema , mostly old fracture, with no definite thecal sac or exit neural canal compression . -Focal bony bruise noted at S3-S4 vertebral bodies? post traumatic  -Mild diffuse disc bulge noted at L4/L5 level , mild compressing thecal and exit nerve roots ,",0,1
1073,local_data/mendeley/T2_0285_D5.png,"LSS MRI : Feature of muscle spasm. Partial loss of L1 vertebral body hieght  anteriorly , with no bone marrow edema , mostly old fracture, with no definite thecal sac or exit neural canal compression . -Focal bony bruise noted at S3-S4 vertebral bodies? post traumatic  -Mild diffuse disc bulge noted at L4/L5 level , mild compressing thecal and exit nerve roots ,",0,1
1074,local_data/mendeley/T1_0286_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -L1 vertebral body hemangioma noted  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing the thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  NB:  Small size Rt kidney (correalte with US )",0,1
1075,local_data/mendeley/T1_0286_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -L1 vertebral body hemangioma noted  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing the thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  NB:  Small size Rt kidney (correalte with US )",0,1
1076,local_data/mendeley/T1_0286_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -L1 vertebral body hemangioma noted  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing the thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  NB:  Small size Rt kidney (correalte with US )",0,1
1077,local_data/mendeley/T2_0286_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -L1 vertebral body hemangioma noted  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing the thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  NB:  Small size Rt kidney (correalte with US )",0,1
1078,local_data/mendeley/T2_0286_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -L1 vertebral body hemangioma noted  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing the thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  NB:  Small size Rt kidney (correalte with US )",0,1
1079,local_data/mendeley/T2_0286_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -L1 vertebral body hemangioma noted  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing the thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  NB:  Small size Rt kidney (correalte with US )",0,1
1080,local_data/mendeley/T1_0287_D3.png,MRI OF THE LSS  Feature of muscle spasm.  Dissecating disc material L5/S 1  L4/L5& L5-S1 levels : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
1081,local_data/mendeley/T1_0287_D4.png,MRI OF THE LSS  Feature of muscle spasm.  Dissecating disc material L5/S 1  L4/L5& L5-S1 levels : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
1082,local_data/mendeley/T1_0287_D5.png,MRI OF THE LSS  Feature of muscle spasm.  Dissecating disc material L5/S 1  L4/L5& L5-S1 levels : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
1083,local_data/mendeley/T2_0287_D3.png,MRI OF THE LSS  Feature of muscle spasm.  Dissecating disc material L5/S 1  L4/L5& L5-S1 levels : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
1084,local_data/mendeley/T2_0287_D4.png,MRI OF THE LSS  Feature of muscle spasm.  Dissecating disc material L5/S 1  L4/L5& L5-S1 levels : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
1085,local_data/mendeley/T2_0287_D5.png,MRI OF THE LSS  Feature of muscle spasm.  Dissecating disc material L5/S 1  L4/L5& L5-S1 levels : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1
1086,local_data/mendeley/T1_0288_D3.png,"LSS MRI : -Feature of muscle spasm. -small vertebral body hemangioma of L4 . -Central and Lt paracentral disc protruion, noted at L4/L5 level , migrated caudally , compressing thecal sac and Lt nerve roots.",0,1
1087,local_data/mendeley/T1_0288_D4.png,"LSS MRI : -Feature of muscle spasm. -small vertebral body hemangioma of L4 . -Central and Lt paracentral disc protruion, noted at L4/L5 level , migrated caudally , compressing thecal sac and Lt nerve roots.",0,1
1088,local_data/mendeley/T1_0288_D5.png,"LSS MRI : -Feature of muscle spasm. -small vertebral body hemangioma of L4 . -Central and Lt paracentral disc protruion, noted at L4/L5 level , migrated caudally , compressing thecal sac and Lt nerve roots.",0,1
1089,local_data/mendeley/T2_0288_D3.png,"LSS MRI : -Feature of muscle spasm. -small vertebral body hemangioma of L4 . -Central and Lt paracentral disc protruion, noted at L4/L5 level , migrated caudally , compressing thecal sac and Lt nerve roots.",0,1
1090,local_data/mendeley/T2_0288_D4.png,"LSS MRI : -Feature of muscle spasm. -small vertebral body hemangioma of L4 . -Central and Lt paracentral disc protruion, noted at L4/L5 level , migrated caudally , compressing thecal sac and Lt nerve roots.",0,1
1091,local_data/mendeley/T2_0288_D5.png,"LSS MRI : -Feature of muscle spasm. -small vertebral body hemangioma of L4 . -Central and Lt paracentral disc protruion, noted at L4/L5 level , migrated caudally , compressing thecal sac and Lt nerve roots.",0,1
1092,local_data/mendeley/T1_0290_D3.png,LSS MRI :  Diffuse disc bulge noted at L3/L4&&L4/S1 level mild compressing thecal sac and exit nerve roots .,0,1
1093,local_data/mendeley/T1_0290_D4.png,LSS MRI :  Diffuse disc bulge noted at L3/L4&&L4/S1 level mild compressing thecal sac and exit nerve roots .,0,1
1094,local_data/mendeley/T1_0290_D5.png,LSS MRI :  Diffuse disc bulge noted at L3/L4&&L4/S1 level mild compressing thecal sac and exit nerve roots .,0,1
1095,local_data/mendeley/T2_0290_D3.png,LSS MRI :  Diffuse disc bulge noted at L3/L4&&L4/S1 level mild compressing thecal sac and exit nerve roots .,0,1
1096,local_data/mendeley/T2_0290_D4.png,LSS MRI :  Diffuse disc bulge noted at L3/L4&&L4/S1 level mild compressing thecal sac and exit nerve roots .,0,1
1097,local_data/mendeley/T2_0290_D5.png,LSS MRI :  Diffuse disc bulge noted at L3/L4&&L4/S1 level mild compressing thecal sac and exit nerve roots .,0,1
1098,local_data/mendeley/T1_0292_D3.png,"Multilevel mild disc bulges noted, largest at L4-L5 level, where it encroaches the exit neural canals.",0,1
1099,local_data/mendeley/T1_0292_D4.png,"Multilevel mild disc bulges noted, largest at L4-L5 level, where it encroaches the exit neural canals.",0,1
1100,local_data/mendeley/T1_0292_D5.png,"Multilevel mild disc bulges noted, largest at L4-L5 level, where it encroaches the exit neural canals.",0,1
1101,local_data/mendeley/T2_0292_D3.png,"Multilevel mild disc bulges noted, largest at L4-L5 level, where it encroaches the exit neural canals.",0,1
1102,local_data/mendeley/T2_0292_D4.png,"Multilevel mild disc bulges noted, largest at L4-L5 level, where it encroaches the exit neural canals.",0,1
1103,local_data/mendeley/T2_0292_D5.png,"Multilevel mild disc bulges noted, largest at L4-L5 level, where it encroaches the exit neural canals.",0,1
1104,local_data/mendeley/T1_0293_D3.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1105,local_data/mendeley/T1_0293_D4.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1106,local_data/mendeley/T1_0293_D5.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1107,local_data/mendeley/T2_0293_D3.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1108,local_data/mendeley/T2_0293_D4.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1109,local_data/mendeley/T2_0293_D5.png,"Multilevel diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1110,local_data/mendeley/T1_0295_D3.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
1111,local_data/mendeley/T1_0295_D4.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
1112,local_data/mendeley/T1_0295_D5.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
1113,local_data/mendeley/T2_0295_D3.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
1114,local_data/mendeley/T2_0295_D4.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
1115,local_data/mendeley/T2_0295_D5.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
1116,local_data/mendeley/T1_0296_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, compressing the  thecal sac and encroaching both exit neural canal , more to Lt sided.",0,1
1117,local_data/mendeley/T1_0296_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, compressing the  thecal sac and encroaching both exit neural canal , more to Lt sided.",0,1
1118,local_data/mendeley/T1_0296_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, compressing the  thecal sac and encroaching both exit neural canal , more to Lt sided.",0,1
1119,local_data/mendeley/T2_0296_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, compressing the  thecal sac and encroaching both exit neural canal , more to Lt sided.",0,1
1120,local_data/mendeley/T2_0296_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, compressing the  thecal sac and encroaching both exit neural canal , more to Lt sided.",0,1
1121,local_data/mendeley/T2_0296_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, compressing the  thecal sac and encroaching both exit neural canal , more to Lt sided.",0,1
1122,local_data/mendeley/T1_0297_D3.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5&L5/S1 levels, mild compressing thecal sac  and both nerve roots",0,1
1123,local_data/mendeley/T1_0297_D4.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5&L5/S1 levels, mild compressing thecal sac  and both nerve roots",0,1
1124,local_data/mendeley/T1_0297_D5.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5&L5/S1 levels, mild compressing thecal sac  and both nerve roots",0,1
1125,local_data/mendeley/T2_0297_D3.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5&L5/S1 levels, mild compressing thecal sac  and both nerve roots",0,1
1126,local_data/mendeley/T2_0297_D4.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5&L5/S1 levels, mild compressing thecal sac  and both nerve roots",0,1
1127,local_data/mendeley/T2_0297_D5.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5&L5/S1 levels, mild compressing thecal sac  and both nerve roots",0,1
1128,local_data/mendeley/T1_0299_D3.png,"LSS MRI  Multilevel diffuse disc bulges, abutting the thecal sac and exit canals.- L3-L4 & L4-L5.  no significant thecal sac or nerve root compression.  Adequate spinaL canal.",0,1
1129,local_data/mendeley/T1_0299_D4.png,"LSS MRI  Multilevel diffuse disc bulges, abutting the thecal sac and exit canals.- L3-L4 & L4-L5.  no significant thecal sac or nerve root compression.  Adequate spinaL canal.",0,1
1130,local_data/mendeley/T1_0299_D5.png,"LSS MRI  Multilevel diffuse disc bulges, abutting the thecal sac and exit canals.- L3-L4 & L4-L5.  no significant thecal sac or nerve root compression.  Adequate spinaL canal.",0,1
1131,local_data/mendeley/T2_0299_D3.png,"LSS MRI  Multilevel diffuse disc bulges, abutting the thecal sac and exit canals.- L3-L4 & L4-L5.  no significant thecal sac or nerve root compression.  Adequate spinaL canal.",0,1
1132,local_data/mendeley/T2_0299_D4.png,"LSS MRI  Multilevel diffuse disc bulges, abutting the thecal sac and exit canals.- L3-L4 & L4-L5.  no significant thecal sac or nerve root compression.  Adequate spinaL canal.",0,1
1133,local_data/mendeley/T2_0299_D5.png,"LSS MRI  Multilevel diffuse disc bulges, abutting the thecal sac and exit canals.- L3-L4 & L4-L5.  no significant thecal sac or nerve root compression.  Adequate spinaL canal.",0,1
1134,local_data/mendeley/T1_0300_D3.png,LSS MRI :  Feature of muscle spasm.  no significant thecal sac or nerve roots compression .  Tarlov cyst noted at level of S1/S2 level,0,1
1135,local_data/mendeley/T1_0300_D4.png,LSS MRI :  Feature of muscle spasm.  no significant thecal sac or nerve roots compression .  Tarlov cyst noted at level of S1/S2 level,0,1
1136,local_data/mendeley/T1_0300_D5.png,LSS MRI :  Feature of muscle spasm.  no significant thecal sac or nerve roots compression .  Tarlov cyst noted at level of S1/S2 level,0,1
1137,local_data/mendeley/T2_0300_D3.png,LSS MRI :  Feature of muscle spasm.  no significant thecal sac or nerve roots compression .  Tarlov cyst noted at level of S1/S2 level,0,1
1138,local_data/mendeley/T2_0300_D4.png,LSS MRI :  Feature of muscle spasm.  no significant thecal sac or nerve roots compression .  Tarlov cyst noted at level of S1/S2 level,0,1
1139,local_data/mendeley/T2_0300_D5.png,LSS MRI :  Feature of muscle spasm.  no significant thecal sac or nerve roots compression .  Tarlov cyst noted at level of S1/S2 level,0,1
1140,local_data/mendeley/T1_0303_D3.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted. -Diffuse disc bulge with small Lt paracentral disc protrusion noted at L2/L3 level, compressing the thecal sac and exit nerve roots . -Diffuse disc bulges noted at L3/L4& L4/L5 levels , compressing the thecal sac and encroaching upon exit neural canals .",0,1
1141,local_data/mendeley/T1_0303_D4.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted. -Diffuse disc bulge with small Lt paracentral disc protrusion noted at L2/L3 level, compressing the thecal sac and exit nerve roots . -Diffuse disc bulges noted at L3/L4& L4/L5 levels , compressing the thecal sac and encroaching upon exit neural canals .",0,1
1142,local_data/mendeley/T1_0303_D5.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted. -Diffuse disc bulge with small Lt paracentral disc protrusion noted at L2/L3 level, compressing the thecal sac and exit nerve roots . -Diffuse disc bulges noted at L3/L4& L4/L5 levels , compressing the thecal sac and encroaching upon exit neural canals .",0,1
1143,local_data/mendeley/T2_0303_D3.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted. -Diffuse disc bulge with small Lt paracentral disc protrusion noted at L2/L3 level, compressing the thecal sac and exit nerve roots . -Diffuse disc bulges noted at L3/L4& L4/L5 levels , compressing the thecal sac and encroaching upon exit neural canals .",0,1
1144,local_data/mendeley/T2_0303_D4.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted. -Diffuse disc bulge with small Lt paracentral disc protrusion noted at L2/L3 level, compressing the thecal sac and exit nerve roots . -Diffuse disc bulges noted at L3/L4& L4/L5 levels , compressing the thecal sac and encroaching upon exit neural canals .",0,1
1145,local_data/mendeley/T2_0303_D5.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted. -Diffuse disc bulge with small Lt paracentral disc protrusion noted at L2/L3 level, compressing the thecal sac and exit nerve roots . -Diffuse disc bulges noted at L3/L4& L4/L5 levels , compressing the thecal sac and encroaching upon exit neural canals .",0,1
1146,local_data/mendeley/T1_0306_D3.png,Lumbosacral MRI  L3-L4  Diffuse disc bulge with annular tear more to the right side compressing the thecal sac  L4-L5  Diffuse disc bulge with left paracentral and foraminal protrusion compressing the thecal sac and exit neural canals  L5-S1  Diffuse bulge with annular tear compressing the thecal sac and exit neural canals more to the right side,0,1
1147,local_data/mendeley/T1_0306_D4.png,Lumbosacral MRI  L3-L4  Diffuse disc bulge with annular tear more to the right side compressing the thecal sac  L4-L5  Diffuse disc bulge with left paracentral and foraminal protrusion compressing the thecal sac and exit neural canals  L5-S1  Diffuse bulge with annular tear compressing the thecal sac and exit neural canals more to the right side,0,1
1148,local_data/mendeley/T1_0306_D5.png,Lumbosacral MRI  L3-L4  Diffuse disc bulge with annular tear more to the right side compressing the thecal sac  L4-L5  Diffuse disc bulge with left paracentral and foraminal protrusion compressing the thecal sac and exit neural canals  L5-S1  Diffuse bulge with annular tear compressing the thecal sac and exit neural canals more to the right side,0,1
1149,local_data/mendeley/T2_0306_D3.png,Lumbosacral MRI  L3-L4  Diffuse disc bulge with annular tear more to the right side compressing the thecal sac  L4-L5  Diffuse disc bulge with left paracentral and foraminal protrusion compressing the thecal sac and exit neural canals  L5-S1  Diffuse bulge with annular tear compressing the thecal sac and exit neural canals more to the right side,0,1
1150,local_data/mendeley/T2_0306_D4.png,Lumbosacral MRI  L3-L4  Diffuse disc bulge with annular tear more to the right side compressing the thecal sac  L4-L5  Diffuse disc bulge with left paracentral and foraminal protrusion compressing the thecal sac and exit neural canals  L5-S1  Diffuse bulge with annular tear compressing the thecal sac and exit neural canals more to the right side,0,1
1151,local_data/mendeley/T2_0306_D5.png,Lumbosacral MRI  L3-L4  Diffuse disc bulge with annular tear more to the right side compressing the thecal sac  L4-L5  Diffuse disc bulge with left paracentral and foraminal protrusion compressing the thecal sac and exit neural canals  L5-S1  Diffuse bulge with annular tear compressing the thecal sac and exit neural canals more to the right side,0,1
1152,local_data/mendeley/T1_0307_D3.png,"LSS MRI:  Muscle spasm noted.  Desiccated disc material noted  Multilevel diffuse disc bulge noted, notecibly at L3-L4 and L4-L5 Mildly compressing the thecal sac and significantly encroaching exiting neural canals",0,1
1153,local_data/mendeley/T1_0307_D4.png,"LSS MRI:  Muscle spasm noted.  Desiccated disc material noted  Multilevel diffuse disc bulge noted, notecibly at L3-L4 and L4-L5 Mildly compressing the thecal sac and significantly encroaching exiting neural canals",0,1
1154,local_data/mendeley/T1_0307_D5.png,"LSS MRI:  Muscle spasm noted.  Desiccated disc material noted  Multilevel diffuse disc bulge noted, notecibly at L3-L4 and L4-L5 Mildly compressing the thecal sac and significantly encroaching exiting neural canals",0,1
1155,local_data/mendeley/T2_0307_D3.png,"LSS MRI:  Muscle spasm noted.  Desiccated disc material noted  Multilevel diffuse disc bulge noted, notecibly at L3-L4 and L4-L5 Mildly compressing the thecal sac and significantly encroaching exiting neural canals",0,1
1156,local_data/mendeley/T2_0307_D4.png,"LSS MRI:  Muscle spasm noted.  Desiccated disc material noted  Multilevel diffuse disc bulge noted, notecibly at L3-L4 and L4-L5 Mildly compressing the thecal sac and significantly encroaching exiting neural canals",0,1
1157,local_data/mendeley/T2_0307_D5.png,"LSS MRI:  Muscle spasm noted.  Desiccated disc material noted  Multilevel diffuse disc bulge noted, notecibly at L3-L4 and L4-L5 Mildly compressing the thecal sac and significantly encroaching exiting neural canals",0,1
1158,local_data/mendeley/T1_0309_D3.png,Lumbosacral MRI  No evidence of disc Herniation  No significant thecal sac or nerve root compression noted,1,0
1159,local_data/mendeley/T1_0309_D4.png,Lumbosacral MRI  No evidence of disc Herniation  No significant thecal sac or nerve root compression noted,1,0
1160,local_data/mendeley/T1_0309_D5.png,Lumbosacral MRI  No evidence of disc Herniation  No significant thecal sac or nerve root compression noted,1,0
1161,local_data/mendeley/T2_0309_D3.png,Lumbosacral MRI  No evidence of disc Herniation  No significant thecal sac or nerve root compression noted,1,0
1162,local_data/mendeley/T2_0309_D4.png,Lumbosacral MRI  No evidence of disc Herniation  No significant thecal sac or nerve root compression noted,1,0
1163,local_data/mendeley/T2_0309_D5.png,Lumbosacral MRI  No evidence of disc Herniation  No significant thecal sac or nerve root compression noted,1,0
1164,local_data/mendeley/T1_0310_D3.png,Lumbosacral MRI Features of muscle spasm No significant thecal sac or nerve root compression No disc protrusion or herniation noted,0,1
1165,local_data/mendeley/T1_0310_D4.png,Lumbosacral MRI Features of muscle spasm No significant thecal sac or nerve root compression No disc protrusion or herniation noted,0,1
1166,local_data/mendeley/T1_0310_D5.png,Lumbosacral MRI Features of muscle spasm No significant thecal sac or nerve root compression No disc protrusion or herniation noted,0,1
1167,local_data/mendeley/T2_0310_D3.png,Lumbosacral MRI Features of muscle spasm No significant thecal sac or nerve root compression No disc protrusion or herniation noted,0,1
1168,local_data/mendeley/T2_0310_D4.png,Lumbosacral MRI Features of muscle spasm No significant thecal sac or nerve root compression No disc protrusion or herniation noted,0,1
1169,local_data/mendeley/T2_0310_D5.png,Lumbosacral MRI Features of muscle spasm No significant thecal sac or nerve root compression No disc protrusion or herniation noted,0,1
1170,local_data/mendeley/T1_0313_D3.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression.  No disc protrusion or herniation seen,0,1
1171,local_data/mendeley/T1_0313_D4.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression.  No disc protrusion or herniation seen,0,1
1172,local_data/mendeley/T1_0313_D5.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression.  No disc protrusion or herniation seen,0,1
1173,local_data/mendeley/T2_0313_D3.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression.  No disc protrusion or herniation seen,0,1
1174,local_data/mendeley/T2_0313_D4.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression.  No disc protrusion or herniation seen,0,1
1175,local_data/mendeley/T2_0313_D5.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression.  No disc protrusion or herniation seen,0,1
1176,local_data/mendeley/T1_0317_D3.png,"LSS MRI  Small Rt paracentral disc protrusion noted at L5-S1 level, mildly compressing the thecal sac.  adequate spinal canal",0,1
1177,local_data/mendeley/T1_0317_D4.png,"LSS MRI  Small Rt paracentral disc protrusion noted at L5-S1 level, mildly compressing the thecal sac.  adequate spinal canal",0,1
1178,local_data/mendeley/T1_0317_D5.png,"LSS MRI  Small Rt paracentral disc protrusion noted at L5-S1 level, mildly compressing the thecal sac.  adequate spinal canal",0,1
1179,local_data/mendeley/T2_0317_D3.png,"LSS MRI  Small Rt paracentral disc protrusion noted at L5-S1 level, mildly compressing the thecal sac.  adequate spinal canal",0,1
1180,local_data/mendeley/T2_0317_D4.png,"LSS MRI  Small Rt paracentral disc protrusion noted at L5-S1 level, mildly compressing the thecal sac.  adequate spinal canal",0,1
1181,local_data/mendeley/T2_0317_D5.png,"LSS MRI  Small Rt paracentral disc protrusion noted at L5-S1 level, mildly compressing the thecal sac.  adequate spinal canal",0,1
1182,local_data/mendeley/T1_0319_D3.png,"LSS MRI WITH CONTRAST  Previous surgery noted.  mild Rt paracentral inhomogenous enhancement noted at L4-L5 level, mostly postoperative granulatiobn/fibrous tissue.",0,1
1183,local_data/mendeley/T1_0319_D4.png,"LSS MRI WITH CONTRAST  Previous surgery noted.  mild Rt paracentral inhomogenous enhancement noted at L4-L5 level, mostly postoperative granulatiobn/fibrous tissue.",0,1
1184,local_data/mendeley/T1_0319_D5.png,"LSS MRI WITH CONTRAST  Previous surgery noted.  mild Rt paracentral inhomogenous enhancement noted at L4-L5 level, mostly postoperative granulatiobn/fibrous tissue.",0,1
1185,local_data/mendeley/T2_0319_D3.png,"LSS MRI WITH CONTRAST  Previous surgery noted.  mild Rt paracentral inhomogenous enhancement noted at L4-L5 level, mostly postoperative granulatiobn/fibrous tissue.",0,1
1186,local_data/mendeley/T2_0319_D4.png,"LSS MRI WITH CONTRAST  Previous surgery noted.  mild Rt paracentral inhomogenous enhancement noted at L4-L5 level, mostly postoperative granulatiobn/fibrous tissue.",0,1
1187,local_data/mendeley/T2_0319_D5.png,"LSS MRI WITH CONTRAST  Previous surgery noted.  mild Rt paracentral inhomogenous enhancement noted at L4-L5 level, mostly postoperative granulatiobn/fibrous tissue.",0,1
1188,local_data/mendeley/T1_0320_D3.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
1189,local_data/mendeley/T1_0320_D4.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
1190,local_data/mendeley/T1_0320_D5.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
1191,local_data/mendeley/T2_0320_D3.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
1192,local_data/mendeley/T2_0320_D4.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
1193,local_data/mendeley/T2_0320_D5.png,LSS MRI  Mild disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1
1194,local_data/mendeley/T1_0322_D3.png,"LSS MRI :  Feature of muscle spasm.  Adequate thecal sac.  No significant disc protrusion or herniation seen .  Diffuse disc bulge noted at D11/D12, obliterating anterior CSf spaces .",0,1
1195,local_data/mendeley/T1_0322_D4.png,"LSS MRI :  Feature of muscle spasm.  Adequate thecal sac.  No significant disc protrusion or herniation seen .  Diffuse disc bulge noted at D11/D12, obliterating anterior CSf spaces .",0,1
1196,local_data/mendeley/T1_0322_D5.png,"LSS MRI :  Feature of muscle spasm.  Adequate thecal sac.  No significant disc protrusion or herniation seen .  Diffuse disc bulge noted at D11/D12, obliterating anterior CSf spaces .",0,1
1197,local_data/mendeley/T2_0322_D3.png,"LSS MRI :  Feature of muscle spasm.  Adequate thecal sac.  No significant disc protrusion or herniation seen .  Diffuse disc bulge noted at D11/D12, obliterating anterior CSf spaces .",0,1
1198,local_data/mendeley/T2_0322_D4.png,"LSS MRI :  Feature of muscle spasm.  Adequate thecal sac.  No significant disc protrusion or herniation seen .  Diffuse disc bulge noted at D11/D12, obliterating anterior CSf spaces .",0,1
1199,local_data/mendeley/T2_0322_D5.png,"LSS MRI :  Feature of muscle spasm.  Adequate thecal sac.  No significant disc protrusion or herniation seen .  Diffuse disc bulge noted at D11/D12, obliterating anterior CSf spaces .",0,1
1200,local_data/mendeley/T1_0323_D3.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals.,0,1
1201,local_data/mendeley/T1_0323_D4.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals.,0,1
1202,local_data/mendeley/T1_0323_D5.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals.,0,1
1203,local_data/mendeley/T2_0323_D3.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals.,0,1
1204,local_data/mendeley/T2_0323_D4.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals.,0,1
1205,local_data/mendeley/T2_0323_D5.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals.,0,1
1206,local_data/mendeley/T1_0324_D3.png,MRI OF THE LSS  Feature of muscle spasm.  Dissicating disc at L4/L5 level  Lt foraminal disc protrusion noted at L4-L5 level compressing Lt sided (thecal sac and exit nerve canal),0,1
1207,local_data/mendeley/T1_0324_D4.png,MRI OF THE LSS  Feature of muscle spasm.  Dissicating disc at L4/L5 level  Lt foraminal disc protrusion noted at L4-L5 level compressing Lt sided (thecal sac and exit nerve canal),0,1
1208,local_data/mendeley/T1_0324_D5.png,MRI OF THE LSS  Feature of muscle spasm.  Dissicating disc at L4/L5 level  Lt foraminal disc protrusion noted at L4-L5 level compressing Lt sided (thecal sac and exit nerve canal),0,1
1209,local_data/mendeley/T2_0324_D3.png,MRI OF THE LSS  Feature of muscle spasm.  Dissicating disc at L4/L5 level  Lt foraminal disc protrusion noted at L4-L5 level compressing Lt sided (thecal sac and exit nerve canal),0,1
1210,local_data/mendeley/T2_0324_D4.png,MRI OF THE LSS  Feature of muscle spasm.  Dissicating disc at L4/L5 level  Lt foraminal disc protrusion noted at L4-L5 level compressing Lt sided (thecal sac and exit nerve canal),0,1
1211,local_data/mendeley/T2_0324_D5.png,MRI OF THE LSS  Feature of muscle spasm.  Dissicating disc at L4/L5 level  Lt foraminal disc protrusion noted at L4-L5 level compressing Lt sided (thecal sac and exit nerve canal),0,1
1212,local_data/mendeley/T1_0325_D3.png,LSS MRI  L2 hemngioma.  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
1213,local_data/mendeley/T1_0325_D4.png,LSS MRI  L2 hemngioma.  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
1214,local_data/mendeley/T1_0325_D5.png,LSS MRI  L2 hemngioma.  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
1215,local_data/mendeley/T2_0325_D3.png,LSS MRI  L2 hemngioma.  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
1216,local_data/mendeley/T2_0325_D4.png,LSS MRI  L2 hemngioma.  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
1217,local_data/mendeley/T2_0325_D5.png,LSS MRI  L2 hemngioma.  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
1218,local_data/mendeley/T1_0329_D3.png,LSS MRI  No significant changes noted since 04/01/2016,0,1
1219,local_data/mendeley/T1_0329_D4.png,LSS MRI  No significant changes noted since 04/01/2016,0,1
1220,local_data/mendeley/T1_0329_D5.png,LSS MRI  No significant changes noted since 04/01/2016,0,1
1221,local_data/mendeley/T2_0329_D3.png,LSS MRI  No significant changes noted since 04/01/2016,0,1
1222,local_data/mendeley/T2_0329_D4.png,LSS MRI  No significant changes noted since 04/01/2016,0,1
1223,local_data/mendeley/T2_0329_D5.png,LSS MRI  No significant changes noted since 04/01/2016,0,1
1224,local_data/mendeley/T1_0332_D3.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
1225,local_data/mendeley/T1_0332_D4.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
1226,local_data/mendeley/T1_0332_D5.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
1227,local_data/mendeley/T2_0332_D3.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
1228,local_data/mendeley/T2_0332_D4.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
1229,local_data/mendeley/T2_0332_D5.png,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0
1230,local_data/mendeley/T1_0333_D3.png,"LSS MRI :  Preveius surgery noted at lower lumbar spine.  Degenerative spondolytic changes .  Lt paracentral disc protrusion noted at L3/L4 level, compressing thecal sac and encroaching exit neural canal.  -Small abnormal signal alteration noted just posterior to L4 vertebral body appeared low on T1&T2 WI, compressing thecal sac and encroaching upon exit neural canal?appearances suggetsive sequestrated disc , calcifiaction ...ect, contrast study is advsied .  -Narrowed L5/S1 level with irrigular end plates and vacuume phenomina .",0,1
1231,local_data/mendeley/T1_0333_D4.png,"LSS MRI :  Preveius surgery noted at lower lumbar spine.  Degenerative spondolytic changes .  Lt paracentral disc protrusion noted at L3/L4 level, compressing thecal sac and encroaching exit neural canal.  -Small abnormal signal alteration noted just posterior to L4 vertebral body appeared low on T1&T2 WI, compressing thecal sac and encroaching upon exit neural canal?appearances suggetsive sequestrated disc , calcifiaction ...ect, contrast study is advsied .  -Narrowed L5/S1 level with irrigular end plates and vacuume phenomina .",0,1
1232,local_data/mendeley/T1_0333_D5.png,"LSS MRI :  Preveius surgery noted at lower lumbar spine.  Degenerative spondolytic changes .  Lt paracentral disc protrusion noted at L3/L4 level, compressing thecal sac and encroaching exit neural canal.  -Small abnormal signal alteration noted just posterior to L4 vertebral body appeared low on T1&T2 WI, compressing thecal sac and encroaching upon exit neural canal?appearances suggetsive sequestrated disc , calcifiaction ...ect, contrast study is advsied .  -Narrowed L5/S1 level with irrigular end plates and vacuume phenomina .",0,1
1233,local_data/mendeley/T2_0333_D3.png,"LSS MRI :  Preveius surgery noted at lower lumbar spine.  Degenerative spondolytic changes .  Lt paracentral disc protrusion noted at L3/L4 level, compressing thecal sac and encroaching exit neural canal.  -Small abnormal signal alteration noted just posterior to L4 vertebral body appeared low on T1&T2 WI, compressing thecal sac and encroaching upon exit neural canal?appearances suggetsive sequestrated disc , calcifiaction ...ect, contrast study is advsied .  -Narrowed L5/S1 level with irrigular end plates and vacuume phenomina .",0,1
1234,local_data/mendeley/T2_0333_D4.png,"LSS MRI :  Preveius surgery noted at lower lumbar spine.  Degenerative spondolytic changes .  Lt paracentral disc protrusion noted at L3/L4 level, compressing thecal sac and encroaching exit neural canal.  -Small abnormal signal alteration noted just posterior to L4 vertebral body appeared low on T1&T2 WI, compressing thecal sac and encroaching upon exit neural canal?appearances suggetsive sequestrated disc , calcifiaction ...ect, contrast study is advsied .  -Narrowed L5/S1 level with irrigular end plates and vacuume phenomina .",0,1
1235,local_data/mendeley/T2_0333_D5.png,"LSS MRI :  Preveius surgery noted at lower lumbar spine.  Degenerative spondolytic changes .  Lt paracentral disc protrusion noted at L3/L4 level, compressing thecal sac and encroaching exit neural canal.  -Small abnormal signal alteration noted just posterior to L4 vertebral body appeared low on T1&T2 WI, compressing thecal sac and encroaching upon exit neural canal?appearances suggetsive sequestrated disc , calcifiaction ...ect, contrast study is advsied .  -Narrowed L5/S1 level with irrigular end plates and vacuume phenomina .",0,1
1236,local_data/mendeley/T1_0334_D3.png,"LSS MRI L4-L5: Diffuse disc bulge noted, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L3-L4: diffuse disc bulge noted compressing the thecal sac and exit canals.  multilevel disc bulges noted elswere with end plate degeneration and schmorls nodules.",0,1
1237,local_data/mendeley/T1_0334_D4.png,"LSS MRI L4-L5: Diffuse disc bulge noted, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L3-L4: diffuse disc bulge noted compressing the thecal sac and exit canals.  multilevel disc bulges noted elswere with end plate degeneration and schmorls nodules.",0,1
1238,local_data/mendeley/T1_0334_D5.png,"LSS MRI L4-L5: Diffuse disc bulge noted, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L3-L4: diffuse disc bulge noted compressing the thecal sac and exit canals.  multilevel disc bulges noted elswere with end plate degeneration and schmorls nodules.",0,1
1239,local_data/mendeley/T2_0334_D3.png,"LSS MRI L4-L5: Diffuse disc bulge noted, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L3-L4: diffuse disc bulge noted compressing the thecal sac and exit canals.  multilevel disc bulges noted elswere with end plate degeneration and schmorls nodules.",0,1
1240,local_data/mendeley/T2_0334_D4.png,"LSS MRI L4-L5: Diffuse disc bulge noted, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L3-L4: diffuse disc bulge noted compressing the thecal sac and exit canals.  multilevel disc bulges noted elswere with end plate degeneration and schmorls nodules.",0,1
1241,local_data/mendeley/T2_0334_D5.png,"LSS MRI L4-L5: Diffuse disc bulge noted, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L3-L4: diffuse disc bulge noted compressing the thecal sac and exit canals.  multilevel disc bulges noted elswere with end plate degeneration and schmorls nodules.",0,1
1242,local_data/mendeley/T1_0335_D3.png,"LSS MRI :  Feature of muscle spasm.  Lt paracentral wide base disc bulge noted at L4/L5 level , compressing the Rt sided thecal sac and exit nerve root",0,1
1243,local_data/mendeley/T1_0335_D4.png,"LSS MRI :  Feature of muscle spasm.  Lt paracentral wide base disc bulge noted at L4/L5 level , compressing the Rt sided thecal sac and exit nerve root",0,1
1244,local_data/mendeley/T1_0335_D5.png,"LSS MRI :  Feature of muscle spasm.  Lt paracentral wide base disc bulge noted at L4/L5 level , compressing the Rt sided thecal sac and exit nerve root",0,1
1245,local_data/mendeley/T2_0335_D3.png,"LSS MRI :  Feature of muscle spasm.  Lt paracentral wide base disc bulge noted at L4/L5 level , compressing the Rt sided thecal sac and exit nerve root",0,1
1246,local_data/mendeley/T2_0335_D4.png,"LSS MRI :  Feature of muscle spasm.  Lt paracentral wide base disc bulge noted at L4/L5 level , compressing the Rt sided thecal sac and exit nerve root",0,1
1247,local_data/mendeley/T2_0335_D5.png,"LSS MRI :  Feature of muscle spasm.  Lt paracentral wide base disc bulge noted at L4/L5 level , compressing the Rt sided thecal sac and exit nerve root",0,1
1248,local_data/mendeley/T1_0336_D3.png,"LSS MRI :  -Multilevel dissicating disc materials noted .  -Degenrative spondolytic chnages  -Diffuse disc bulges noted mostly at L4/L5 level , compressing the thecal sac and encroaching exit neural canals associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis .  -Diffuse disc bulges noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1249,local_data/mendeley/T1_0336_D4.png,"LSS MRI :  -Multilevel dissicating disc materials noted .  -Degenrative spondolytic chnages  -Diffuse disc bulges noted mostly at L4/L5 level , compressing the thecal sac and encroaching exit neural canals associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis .  -Diffuse disc bulges noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1250,local_data/mendeley/T1_0336_D5.png,"LSS MRI :  -Multilevel dissicating disc materials noted .  -Degenrative spondolytic chnages  -Diffuse disc bulges noted mostly at L4/L5 level , compressing the thecal sac and encroaching exit neural canals associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis .  -Diffuse disc bulges noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1251,local_data/mendeley/T2_0336_D3.png,"LSS MRI :  -Multilevel dissicating disc materials noted .  -Degenrative spondolytic chnages  -Diffuse disc bulges noted mostly at L4/L5 level , compressing the thecal sac and encroaching exit neural canals associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis .  -Diffuse disc bulges noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1252,local_data/mendeley/T2_0336_D4.png,"LSS MRI :  -Multilevel dissicating disc materials noted .  -Degenrative spondolytic chnages  -Diffuse disc bulges noted mostly at L4/L5 level , compressing the thecal sac and encroaching exit neural canals associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis .  -Diffuse disc bulges noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1253,local_data/mendeley/T2_0336_D5.png,"LSS MRI :  -Multilevel dissicating disc materials noted .  -Degenrative spondolytic chnages  -Diffuse disc bulges noted mostly at L4/L5 level , compressing the thecal sac and encroaching exit neural canals associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis .  -Diffuse disc bulges noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1254,local_data/mendeley/T1_0338_D3.png,"LSS MRI  mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  the spinal canal appeared adquate.",0,1
1255,local_data/mendeley/T1_0338_D4.png,"LSS MRI  mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  the spinal canal appeared adquate.",0,1
1256,local_data/mendeley/T1_0338_D5.png,"LSS MRI  mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  the spinal canal appeared adquate.",0,1
1257,local_data/mendeley/T2_0338_D3.png,"LSS MRI  mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  the spinal canal appeared adquate.",0,1
1258,local_data/mendeley/T2_0338_D4.png,"LSS MRI  mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  the spinal canal appeared adquate.",0,1
1259,local_data/mendeley/T2_0338_D5.png,"LSS MRI  mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  the spinal canal appeared adquate.",0,1
1260,local_data/mendeley/T1_0339_D3.png,LSS MRI :  Feature of muscle spasm.  No definite thecal sca or nerve roots compression noted .  No disc herniation or protrusion noted .,0,1
1261,local_data/mendeley/T1_0339_D4.png,LSS MRI :  Feature of muscle spasm.  No definite thecal sca or nerve roots compression noted .  No disc herniation or protrusion noted .,0,1
1262,local_data/mendeley/T1_0339_D5.png,LSS MRI :  Feature of muscle spasm.  No definite thecal sca or nerve roots compression noted .  No disc herniation or protrusion noted .,0,1
1263,local_data/mendeley/T2_0339_D3.png,LSS MRI :  Feature of muscle spasm.  No definite thecal sca or nerve roots compression noted .  No disc herniation or protrusion noted .,0,1
1264,local_data/mendeley/T2_0339_D4.png,LSS MRI :  Feature of muscle spasm.  No definite thecal sca or nerve roots compression noted .  No disc herniation or protrusion noted .,0,1
1265,local_data/mendeley/T2_0339_D5.png,LSS MRI :  Feature of muscle spasm.  No definite thecal sca or nerve roots compression noted .  No disc herniation or protrusion noted .,0,1
1266,local_data/mendeley/T1_0340_D3.png,LSS MRI :  Feature of muscle spasm noted  Central and left paracentral disc protrusion noted at L3-L4 and L4-5 disc space compressing the thecal sac and exite nerve roots  Dissicating disc material noted,0,1
1267,local_data/mendeley/T1_0340_D4.png,LSS MRI :  Feature of muscle spasm noted  Central and left paracentral disc protrusion noted at L3-L4 and L4-5 disc space compressing the thecal sac and exite nerve roots  Dissicating disc material noted,0,1
1268,local_data/mendeley/T1_0340_D5.png,LSS MRI :  Feature of muscle spasm noted  Central and left paracentral disc protrusion noted at L3-L4 and L4-5 disc space compressing the thecal sac and exite nerve roots  Dissicating disc material noted,0,1
1269,local_data/mendeley/T2_0340_D3.png,LSS MRI :  Feature of muscle spasm noted  Central and left paracentral disc protrusion noted at L3-L4 and L4-5 disc space compressing the thecal sac and exite nerve roots  Dissicating disc material noted,0,1
1270,local_data/mendeley/T2_0340_D4.png,LSS MRI :  Feature of muscle spasm noted  Central and left paracentral disc protrusion noted at L3-L4 and L4-5 disc space compressing the thecal sac and exite nerve roots  Dissicating disc material noted,0,1
1271,local_data/mendeley/T2_0340_D5.png,LSS MRI :  Feature of muscle spasm noted  Central and left paracentral disc protrusion noted at L3-L4 and L4-5 disc space compressing the thecal sac and exite nerve roots  Dissicating disc material noted,0,1
1272,local_data/mendeley/T1_0341_D3.png,"L2-L3, L3-L4, L4-L5: diffuse disc bulges compressing the thecal sac and encroaching exit canals.",0,1
1273,local_data/mendeley/T1_0341_D4.png,"L2-L3, L3-L4, L4-L5: diffuse disc bulges compressing the thecal sac and encroaching exit canals.",0,1
1274,local_data/mendeley/T1_0341_D5.png,"L2-L3, L3-L4, L4-L5: diffuse disc bulges compressing the thecal sac and encroaching exit canals.",0,1
1275,local_data/mendeley/T2_0341_D3.png,"L2-L3, L3-L4, L4-L5: diffuse disc bulges compressing the thecal sac and encroaching exit canals.",0,1
1276,local_data/mendeley/T2_0341_D4.png,"L2-L3, L3-L4, L4-L5: diffuse disc bulges compressing the thecal sac and encroaching exit canals.",0,1
1277,local_data/mendeley/T2_0341_D5.png,"L2-L3, L3-L4, L4-L5: diffuse disc bulges compressing the thecal sac and encroaching exit canals.",0,1
1278,local_data/mendeley/T1_0342_D3.png,"LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1279,local_data/mendeley/T1_0342_D4.png,"LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1280,local_data/mendeley/T1_0342_D5.png,"LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1281,local_data/mendeley/T2_0342_D3.png,"LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1282,local_data/mendeley/T2_0342_D4.png,"LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1283,local_data/mendeley/T2_0342_D5.png,"LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1284,local_data/mendeley/T1_0343_D3.png,L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal. L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals.,0,1
1285,local_data/mendeley/T1_0343_D4.png,L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal. L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals.,0,1
1286,local_data/mendeley/T1_0343_D5.png,L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal. L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals.,0,1
1287,local_data/mendeley/T2_0343_D3.png,L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal. L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals.,0,1
1288,local_data/mendeley/T2_0343_D4.png,L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal. L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals.,0,1
1289,local_data/mendeley/T2_0343_D5.png,L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal. L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals.,0,1
1290,local_data/mendeley/T1_0344_D3.png,Previous surgery noted with transpedicle fixation. Enhancing granulation tissue compressing the thecal sac.,0,1
1291,local_data/mendeley/T1_0344_D4.png,Previous surgery noted with transpedicle fixation. Enhancing granulation tissue compressing the thecal sac.,0,1
1292,local_data/mendeley/T1_0344_D5.png,Previous surgery noted with transpedicle fixation. Enhancing granulation tissue compressing the thecal sac.,0,1
1293,local_data/mendeley/T2_0344_D3.png,Previous surgery noted with transpedicle fixation. Enhancing granulation tissue compressing the thecal sac.,0,1
1294,local_data/mendeley/T2_0344_D4.png,Previous surgery noted with transpedicle fixation. Enhancing granulation tissue compressing the thecal sac.,0,1
1295,local_data/mendeley/T2_0344_D5.png,Previous surgery noted with transpedicle fixation. Enhancing granulation tissue compressing the thecal sac.,0,1
1296,local_data/mendeley/T1_0346_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1297,local_data/mendeley/T1_0346_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1298,local_data/mendeley/T1_0346_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1299,local_data/mendeley/T2_0346_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1300,local_data/mendeley/T2_0346_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1301,local_data/mendeley/T2_0346_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1302,local_data/mendeley/T1_0347_D3.png,"LSS MRI  Broad based central disc protrusion noted at L4-L5 level, largely compressng the thecal sac and encroaching upon the exit neural canals.  Diffuse disc bulge noted at L3-L4 level, compressing the thecal sac and exit canals.",0,1
1303,local_data/mendeley/T1_0347_D4.png,"LSS MRI  Broad based central disc protrusion noted at L4-L5 level, largely compressng the thecal sac and encroaching upon the exit neural canals.  Diffuse disc bulge noted at L3-L4 level, compressing the thecal sac and exit canals.",0,1
1304,local_data/mendeley/T1_0347_D5.png,"LSS MRI  Broad based central disc protrusion noted at L4-L5 level, largely compressng the thecal sac and encroaching upon the exit neural canals.  Diffuse disc bulge noted at L3-L4 level, compressing the thecal sac and exit canals.",0,1
1305,local_data/mendeley/T2_0347_D3.png,"LSS MRI  Broad based central disc protrusion noted at L4-L5 level, largely compressng the thecal sac and encroaching upon the exit neural canals.  Diffuse disc bulge noted at L3-L4 level, compressing the thecal sac and exit canals.",0,1
1306,local_data/mendeley/T2_0347_D4.png,"LSS MRI  Broad based central disc protrusion noted at L4-L5 level, largely compressng the thecal sac and encroaching upon the exit neural canals.  Diffuse disc bulge noted at L3-L4 level, compressing the thecal sac and exit canals.",0,1
1307,local_data/mendeley/T2_0347_D5.png,"LSS MRI  Broad based central disc protrusion noted at L4-L5 level, largely compressng the thecal sac and encroaching upon the exit neural canals.  Diffuse disc bulge noted at L3-L4 level, compressing the thecal sac and exit canals.",0,1
1308,local_data/mendeley/T1_0348_D3.png,"LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted mainly at L3/L4 & L4/L5 level, mildly compressing the thecal sac and exit nerve.",0,1
1309,local_data/mendeley/T1_0348_D4.png,"LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted mainly at L3/L4 & L4/L5 level, mildly compressing the thecal sac and exit nerve.",0,1
1310,local_data/mendeley/T1_0348_D5.png,"LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted mainly at L3/L4 & L4/L5 level, mildly compressing the thecal sac and exit nerve.",0,1
1311,local_data/mendeley/T2_0348_D3.png,"LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted mainly at L3/L4 & L4/L5 level, mildly compressing the thecal sac and exit nerve.",0,1
1312,local_data/mendeley/T2_0348_D4.png,"LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted mainly at L3/L4 & L4/L5 level, mildly compressing the thecal sac and exit nerve.",0,1
1313,local_data/mendeley/T2_0348_D5.png,"LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted mainly at L3/L4 & L4/L5 level, mildly compressing the thecal sac and exit nerve.",0,1
1314,local_data/mendeley/T1_0349_D3.png,LSS MRI  dehydrated L4-L5 discal material with features of muscle spasm.  no evidence of disc herniation.  no significant thecal sac or nerve root compression noted.,0,1
1315,local_data/mendeley/T1_0349_D4.png,LSS MRI  dehydrated L4-L5 discal material with features of muscle spasm.  no evidence of disc herniation.  no significant thecal sac or nerve root compression noted.,0,1
1316,local_data/mendeley/T1_0349_D5.png,LSS MRI  dehydrated L4-L5 discal material with features of muscle spasm.  no evidence of disc herniation.  no significant thecal sac or nerve root compression noted.,0,1
1317,local_data/mendeley/T2_0349_D3.png,LSS MRI  dehydrated L4-L5 discal material with features of muscle spasm.  no evidence of disc herniation.  no significant thecal sac or nerve root compression noted.,0,1
1318,local_data/mendeley/T2_0349_D4.png,LSS MRI  dehydrated L4-L5 discal material with features of muscle spasm.  no evidence of disc herniation.  no significant thecal sac or nerve root compression noted.,0,1
1319,local_data/mendeley/T2_0349_D5.png,LSS MRI  dehydrated L4-L5 discal material with features of muscle spasm.  no evidence of disc herniation.  no significant thecal sac or nerve root compression noted.,0,1
1320,local_data/mendeley/T1_0352_D3.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1321,local_data/mendeley/T1_0352_D4.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1322,local_data/mendeley/T1_0352_D5.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1323,local_data/mendeley/T2_0352_D3.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1324,local_data/mendeley/T2_0352_D4.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1325,local_data/mendeley/T2_0352_D5.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1326,local_data/mendeley/T1_0354_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan.  Adequate thecal sac with no exit nerve root compression .  No evidence of disc protrusion or herniation seen .",0,1
1327,local_data/mendeley/T1_0354_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan.  Adequate thecal sac with no exit nerve root compression .  No evidence of disc protrusion or herniation seen .",0,1
1328,local_data/mendeley/T1_0354_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan.  Adequate thecal sac with no exit nerve root compression .  No evidence of disc protrusion or herniation seen .",0,1
1329,local_data/mendeley/T2_0354_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan.  Adequate thecal sac with no exit nerve root compression .  No evidence of disc protrusion or herniation seen .",0,1
1330,local_data/mendeley/T2_0354_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan.  Adequate thecal sac with no exit nerve root compression .  No evidence of disc protrusion or herniation seen .",0,1
1331,local_data/mendeley/T2_0354_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan.  Adequate thecal sac with no exit nerve root compression .  No evidence of disc protrusion or herniation seen .",0,1
1332,local_data/mendeley/T1_0355_D3.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1333,local_data/mendeley/T1_0355_D4.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1334,local_data/mendeley/T1_0355_D5.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1335,local_data/mendeley/T2_0355_D3.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1336,local_data/mendeley/T2_0355_D4.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1337,local_data/mendeley/T2_0355_D5.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1338,local_data/mendeley/T1_0359_D3.png,LSS MRI :  feature of muscle spasm.  adequate thecal sac.  No significant disc protruion seen,1,0
1339,local_data/mendeley/T1_0359_D4.png,LSS MRI :  feature of muscle spasm.  adequate thecal sac.  No significant disc protruion seen,1,0
1340,local_data/mendeley/T1_0359_D5.png,LSS MRI :  feature of muscle spasm.  adequate thecal sac.  No significant disc protruion seen,1,0
1341,local_data/mendeley/T2_0359_D3.png,LSS MRI :  feature of muscle spasm.  adequate thecal sac.  No significant disc protruion seen,1,0
1342,local_data/mendeley/T2_0359_D4.png,LSS MRI :  feature of muscle spasm.  adequate thecal sac.  No significant disc protruion seen,1,0
1343,local_data/mendeley/T2_0359_D5.png,LSS MRI :  feature of muscle spasm.  adequate thecal sac.  No significant disc protruion seen,1,0
1344,local_data/mendeley/T1_0362_D3.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac  the spinal canal appeared adquate.",0,1
1345,local_data/mendeley/T1_0362_D4.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac  the spinal canal appeared adquate.",0,1
1346,local_data/mendeley/T1_0362_D5.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac  the spinal canal appeared adquate.",0,1
1347,local_data/mendeley/T2_0362_D3.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac  the spinal canal appeared adquate.",0,1
1348,local_data/mendeley/T2_0362_D4.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac  the spinal canal appeared adquate.",0,1
1349,local_data/mendeley/T2_0362_D5.png,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac  the spinal canal appeared adquate.",0,1
1350,local_data/mendeley/T1_0363_D3.png,"LSS MRI :  Feature of muscle spasm.  There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised .  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1351,local_data/mendeley/T1_0363_D4.png,"LSS MRI :  Feature of muscle spasm.  There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised .  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1352,local_data/mendeley/T1_0363_D5.png,"LSS MRI :  Feature of muscle spasm.  There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised .  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1353,local_data/mendeley/T2_0363_D3.png,"LSS MRI :  Feature of muscle spasm.  There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised .  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1354,local_data/mendeley/T2_0363_D4.png,"LSS MRI :  Feature of muscle spasm.  There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised .  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1355,local_data/mendeley/T2_0363_D5.png,"LSS MRI :  Feature of muscle spasm.  There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised .  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1356,local_data/mendeley/T1_0365_D3.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1357,local_data/mendeley/T1_0365_D4.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1358,local_data/mendeley/T1_0365_D5.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1359,local_data/mendeley/T2_0365_D3.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1360,local_data/mendeley/T2_0365_D4.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1361,local_data/mendeley/T2_0365_D5.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1362,local_data/mendeley/T1_0368_D3.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1363,local_data/mendeley/T1_0368_D4.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1364,local_data/mendeley/T1_0368_D5.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1365,local_data/mendeley/T2_0368_D3.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1366,local_data/mendeley/T2_0368_D4.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1367,local_data/mendeley/T2_0368_D5.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1368,local_data/mendeley/T1_0371_D3.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1369,local_data/mendeley/T1_0371_D4.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1370,local_data/mendeley/T1_0371_D5.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1371,local_data/mendeley/T2_0371_D3.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1372,local_data/mendeley/T2_0371_D4.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1373,local_data/mendeley/T2_0371_D5.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1374,local_data/mendeley/T1_0372_D3.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1 0,1,0
1375,local_data/mendeley/T1_0372_D4.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1 0,1,0
1376,local_data/mendeley/T1_0372_D5.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1 0,1,0
1377,local_data/mendeley/T2_0372_D3.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1 0,1,0
1378,local_data/mendeley/T2_0372_D4.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1 0,1,0
1379,local_data/mendeley/T2_0372_D5.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1 0,1,0
1380,local_data/mendeley/T1_0373_D3.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1,0,1
1381,local_data/mendeley/T1_0373_D4.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1,0,1
1382,local_data/mendeley/T1_0373_D5.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1,0,1
1383,local_data/mendeley/T2_0373_D3.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1,0,1
1384,local_data/mendeley/T2_0373_D4.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1,0,1
1385,local_data/mendeley/T2_0373_D5.png,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1,0,1
1386,local_data/mendeley/T1_0375_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.  Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal.  NB:Multiple small tarlovs cysts noted",0,1
1387,local_data/mendeley/T1_0375_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.  Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal.  NB:Multiple small tarlovs cysts noted",0,1
1388,local_data/mendeley/T1_0375_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.  Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal.  NB:Multiple small tarlovs cysts noted",0,1
1389,local_data/mendeley/T2_0375_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.  Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal.  NB:Multiple small tarlovs cysts noted",0,1
1390,local_data/mendeley/T2_0375_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.  Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal.  NB:Multiple small tarlovs cysts noted",0,1
1391,local_data/mendeley/T2_0375_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.  Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal.  NB:Multiple small tarlovs cysts noted",0,1
1392,local_data/mendeley/T1_0377_D3.png,"LSS MRI  Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root.  adequate spinal canal.",0,1
1393,local_data/mendeley/T1_0377_D4.png,"LSS MRI  Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root.  adequate spinal canal.",0,1
1394,local_data/mendeley/T1_0377_D5.png,"LSS MRI  Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root.  adequate spinal canal.",0,1
1395,local_data/mendeley/T2_0377_D3.png,"LSS MRI  Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root.  adequate spinal canal.",0,1
1396,local_data/mendeley/T2_0377_D4.png,"LSS MRI  Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root.  adequate spinal canal.",0,1
1397,local_data/mendeley/T2_0377_D5.png,"LSS MRI  Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root.  adequate spinal canal.",0,1
1398,local_data/mendeley/T1_0381_D3.png,"LSS MRI  L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals.  L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1399,local_data/mendeley/T1_0381_D4.png,"LSS MRI  L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals.  L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1400,local_data/mendeley/T1_0381_D5.png,"LSS MRI  L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals.  L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1401,local_data/mendeley/T2_0381_D3.png,"LSS MRI  L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals.  L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1402,local_data/mendeley/T2_0381_D4.png,"LSS MRI  L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals.  L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1403,local_data/mendeley/T2_0381_D5.png,"LSS MRI  L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals.  L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1404,local_data/mendeley/T1_0382_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan.  -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal.  Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1405,local_data/mendeley/T1_0382_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan.  -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal.  Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1406,local_data/mendeley/T1_0382_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan.  -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal.  Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1407,local_data/mendeley/T2_0382_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan.  -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal.  Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1408,local_data/mendeley/T2_0382_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan.  -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal.  Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1409,local_data/mendeley/T2_0382_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan.  -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal.  Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1410,local_data/mendeley/T1_0383_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1411,local_data/mendeley/T1_0383_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1412,local_data/mendeley/T1_0383_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1413,local_data/mendeley/T2_0383_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1414,local_data/mendeley/T2_0383_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1415,local_data/mendeley/T2_0383_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1416,local_data/mendeley/T1_0384_D3.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression.,1,0
1417,local_data/mendeley/T1_0384_D4.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression.,1,0
1418,local_data/mendeley/T1_0384_D5.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression.,1,0
1419,local_data/mendeley/T2_0384_D3.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression.,1,0
1420,local_data/mendeley/T2_0384_D4.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression.,1,0
1421,local_data/mendeley/T2_0384_D5.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression.,1,0
1422,local_data/mendeley/T1_0385_D3.png,"LSS MRI :  There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1423,local_data/mendeley/T1_0385_D4.png,"LSS MRI :  There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1424,local_data/mendeley/T1_0385_D5.png,"LSS MRI :  There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1425,local_data/mendeley/T2_0385_D3.png,"LSS MRI :  There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1426,local_data/mendeley/T2_0385_D4.png,"LSS MRI :  There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1427,local_data/mendeley/T2_0385_D5.png,"LSS MRI :  There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1428,local_data/mendeley/T1_0386_D3.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1429,local_data/mendeley/T1_0386_D4.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1430,local_data/mendeley/T1_0386_D5.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1431,local_data/mendeley/T2_0386_D3.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1432,local_data/mendeley/T2_0386_D4.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1433,local_data/mendeley/T2_0386_D5.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1434,local_data/mendeley/T1_0387_D3.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1435,local_data/mendeley/T1_0387_D4.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1436,local_data/mendeley/T1_0387_D5.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1437,local_data/mendeley/T2_0387_D3.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1438,local_data/mendeley/T2_0387_D4.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1439,local_data/mendeley/T2_0387_D5.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1440,local_data/mendeley/T1_0388_D3.png,"Feature of muscle spasm.  L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1441,local_data/mendeley/T1_0388_D4.png,"Feature of muscle spasm.  L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1442,local_data/mendeley/T1_0388_D5.png,"Feature of muscle spasm.  L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1443,local_data/mendeley/T2_0388_D3.png,"Feature of muscle spasm.  L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1444,local_data/mendeley/T2_0388_D4.png,"Feature of muscle spasm.  L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1445,local_data/mendeley/T2_0388_D5.png,"Feature of muscle spasm.  L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1446,local_data/mendeley/T1_0389_D3.png,MRI LSS  Feature of muscle spasm  Mild diffuse disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted,1,0
1447,local_data/mendeley/T1_0389_D4.png,MRI LSS  Feature of muscle spasm  Mild diffuse disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted,1,0
1448,local_data/mendeley/T1_0389_D5.png,MRI LSS  Feature of muscle spasm  Mild diffuse disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted,1,0
1449,local_data/mendeley/T2_0389_D3.png,MRI LSS  Feature of muscle spasm  Mild diffuse disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted,1,0
1450,local_data/mendeley/T2_0389_D4.png,MRI LSS  Feature of muscle spasm  Mild diffuse disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted,1,0
1451,local_data/mendeley/T2_0389_D5.png,MRI LSS  Feature of muscle spasm  Mild diffuse disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted,1,0
1452,local_data/mendeley/T1_0392_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Dgenerative spondolytic changes  -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals.  -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1453,local_data/mendeley/T1_0392_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Dgenerative spondolytic changes  -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals.  -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1454,local_data/mendeley/T1_0392_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Dgenerative spondolytic changes  -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals.  -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1455,local_data/mendeley/T2_0392_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Dgenerative spondolytic changes  -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals.  -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1456,local_data/mendeley/T2_0392_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Dgenerative spondolytic changes  -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals.  -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1457,local_data/mendeley/T2_0392_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Dgenerative spondolytic changes  -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals.  -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1458,local_data/mendeley/T1_0393_D3.png,LSS MRI :  faeture of muscle spasm.  No definite thecal sac or nerev root compression .  Small tarlov cyst noted,1,0
1459,local_data/mendeley/T1_0393_D4.png,LSS MRI :  faeture of muscle spasm.  No definite thecal sac or nerev root compression .  Small tarlov cyst noted,1,0
1460,local_data/mendeley/T1_0393_D5.png,LSS MRI :  faeture of muscle spasm.  No definite thecal sac or nerev root compression .  Small tarlov cyst noted,1,0
1461,local_data/mendeley/T2_0393_D3.png,LSS MRI :  faeture of muscle spasm.  No definite thecal sac or nerev root compression .  Small tarlov cyst noted,1,0
1462,local_data/mendeley/T2_0393_D4.png,LSS MRI :  faeture of muscle spasm.  No definite thecal sac or nerev root compression .  Small tarlov cyst noted,1,0
1463,local_data/mendeley/T2_0393_D5.png,LSS MRI :  faeture of muscle spasm.  No definite thecal sac or nerev root compression .  Small tarlov cyst noted,1,0
1464,local_data/mendeley/T1_0396_D3.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1465,local_data/mendeley/T1_0396_D4.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1466,local_data/mendeley/T1_0396_D5.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1467,local_data/mendeley/T2_0396_D3.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1468,local_data/mendeley/T2_0396_D4.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1469,local_data/mendeley/T2_0396_D5.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1470,local_data/mendeley/T1_0397_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1471,local_data/mendeley/T1_0397_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1472,local_data/mendeley/T1_0397_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1473,local_data/mendeley/T2_0397_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1474,local_data/mendeley/T2_0397_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1475,local_data/mendeley/T2_0397_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1476,local_data/mendeley/T1_0399_D3.png,"LSS MRI :  Dissicating disc materials.  Grdae I spondylolisthesis of L4 over L5 level.  widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1477,local_data/mendeley/T1_0399_D4.png,"LSS MRI :  Dissicating disc materials.  Grdae I spondylolisthesis of L4 over L5 level.  widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1478,local_data/mendeley/T1_0399_D5.png,"LSS MRI :  Dissicating disc materials.  Grdae I spondylolisthesis of L4 over L5 level.  widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1479,local_data/mendeley/T2_0399_D3.png,"LSS MRI :  Dissicating disc materials.  Grdae I spondylolisthesis of L4 over L5 level.  widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1480,local_data/mendeley/T2_0399_D4.png,"LSS MRI :  Dissicating disc materials.  Grdae I spondylolisthesis of L4 over L5 level.  widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1481,local_data/mendeley/T2_0399_D5.png,"LSS MRI :  Dissicating disc materials.  Grdae I spondylolisthesis of L4 over L5 level.  widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1482,local_data/mendeley/T1_0400_D3.png,Lumbosacral MRI  Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1483,local_data/mendeley/T1_0400_D4.png,Lumbosacral MRI  Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1484,local_data/mendeley/T1_0400_D5.png,Lumbosacral MRI  Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1485,local_data/mendeley/T2_0400_D3.png,Lumbosacral MRI  Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1486,local_data/mendeley/T2_0400_D4.png,Lumbosacral MRI  Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1487,local_data/mendeley/T2_0400_D5.png,Lumbosacral MRI  Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1488,local_data/mendeley/T1_0352_D3.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1489,local_data/mendeley/T1_0352_D4.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1490,local_data/mendeley/T1_0352_D5.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1491,local_data/mendeley/T2_0352_D3.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1492,local_data/mendeley/T2_0352_D4.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1493,local_data/mendeley/T2_0352_D5.png,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1
1494,local_data/mendeley/T1_0354_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan. Adequate thecal sac with no exit nerve root compression . No evidence of disc protrusion or herniation seen .",0,1
1495,local_data/mendeley/T1_0354_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan. Adequate thecal sac with no exit nerve root compression . No evidence of disc protrusion or herniation seen .",0,1
1496,local_data/mendeley/T1_0354_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan. Adequate thecal sac with no exit nerve root compression . No evidence of disc protrusion or herniation seen .",0,1
1497,local_data/mendeley/T2_0354_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan. Adequate thecal sac with no exit nerve root compression . No evidence of disc protrusion or herniation seen .",0,1
1498,local_data/mendeley/T2_0354_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan. Adequate thecal sac with no exit nerve root compression . No evidence of disc protrusion or herniation seen .",0,1
1499,local_data/mendeley/T2_0354_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan. Adequate thecal sac with no exit nerve root compression . No evidence of disc protrusion or herniation seen .",0,1
1500,local_data/mendeley/T1_0355_D3.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1501,local_data/mendeley/T1_0355_D4.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1502,local_data/mendeley/T1_0355_D5.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1503,local_data/mendeley/T2_0355_D3.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1504,local_data/mendeley/T2_0355_D4.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1505,local_data/mendeley/T2_0355_D5.png,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1
1506,local_data/mendeley/T1_0359_D3.png,LSS MRI : feature of muscle spasm. adequate thecal sac. No significant disc protruion seen,1,0
1507,local_data/mendeley/T1_0359_D4.png,LSS MRI : feature of muscle spasm. adequate thecal sac. No significant disc protruion seen,1,0
1508,local_data/mendeley/T1_0359_D5.png,LSS MRI : feature of muscle spasm. adequate thecal sac. No significant disc protruion seen,1,0
1509,local_data/mendeley/T2_0359_D3.png,LSS MRI : feature of muscle spasm. adequate thecal sac. No significant disc protruion seen,1,0
1510,local_data/mendeley/T2_0359_D4.png,LSS MRI : feature of muscle spasm. adequate thecal sac. No significant disc protruion seen,1,0
1511,local_data/mendeley/T2_0359_D5.png,LSS MRI : feature of muscle spasm. adequate thecal sac. No significant disc protruion seen,1,0
1512,local_data/mendeley/T1_0362_D3.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac the spinal canal appeared adquate.",0,1
1513,local_data/mendeley/T1_0362_D4.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac the spinal canal appeared adquate.",0,1
1514,local_data/mendeley/T1_0362_D5.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac the spinal canal appeared adquate.",0,1
1515,local_data/mendeley/T2_0362_D3.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac the spinal canal appeared adquate.",0,1
1516,local_data/mendeley/T2_0362_D4.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac the spinal canal appeared adquate.",0,1
1517,local_data/mendeley/T2_0362_D5.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac the spinal canal appeared adquate.",0,1
1518,local_data/mendeley/T1_0363_D3.png,"LSS MRI : Feature of muscle spasm. There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised . Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1519,local_data/mendeley/T1_0363_D4.png,"LSS MRI : Feature of muscle spasm. There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised . Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1520,local_data/mendeley/T1_0363_D5.png,"LSS MRI : Feature of muscle spasm. There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised . Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1521,local_data/mendeley/T2_0363_D3.png,"LSS MRI : Feature of muscle spasm. There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised . Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1522,local_data/mendeley/T2_0363_D4.png,"LSS MRI : Feature of muscle spasm. There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised . Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1523,local_data/mendeley/T2_0363_D5.png,"LSS MRI : Feature of muscle spasm. There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised . Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1
1524,local_data/mendeley/T1_0365_D3.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1525,local_data/mendeley/T1_0365_D4.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1526,local_data/mendeley/T1_0365_D5.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1527,local_data/mendeley/T2_0365_D3.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1528,local_data/mendeley/T2_0365_D4.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1529,local_data/mendeley/T2_0365_D5.png,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1
1530,local_data/mendeley/T1_0368_D3.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1531,local_data/mendeley/T1_0368_D4.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1532,local_data/mendeley/T1_0368_D5.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1533,local_data/mendeley/T2_0368_D3.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1534,local_data/mendeley/T2_0368_D4.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1535,local_data/mendeley/T2_0368_D5.png,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1
1536,local_data/mendeley/T1_0371_D3.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1537,local_data/mendeley/T1_0371_D4.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1538,local_data/mendeley/T1_0371_D5.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1539,local_data/mendeley/T2_0371_D3.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1540,local_data/mendeley/T2_0371_D4.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1541,local_data/mendeley/T2_0371_D5.png,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1
1542,local_data/mendeley/T1_0372_D3.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1543,local_data/mendeley/T1_0372_D4.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1544,local_data/mendeley/T1_0372_D5.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1545,local_data/mendeley/T2_0372_D3.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1546,local_data/mendeley/T2_0372_D4.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1547,local_data/mendeley/T2_0372_D5.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1548,local_data/mendeley/T1_0373_D3.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1549,local_data/mendeley/T1_0373_D4.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1550,local_data/mendeley/T1_0373_D5.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1551,local_data/mendeley/T2_0373_D3.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1552,local_data/mendeley/T2_0373_D4.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1553,local_data/mendeley/T2_0373_D5.png,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1
1554,local_data/mendeley/T1_0375_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal. Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal. NB:Multiple small tarlovs cysts noted",0,1
1555,local_data/mendeley/T1_0375_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal. Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal. NB:Multiple small tarlovs cysts noted",0,1
1556,local_data/mendeley/T1_0375_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal. Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal. NB:Multiple small tarlovs cysts noted",0,1
1557,local_data/mendeley/T2_0375_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal. Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal. NB:Multiple small tarlovs cysts noted",0,1
1558,local_data/mendeley/T2_0375_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal. Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal. NB:Multiple small tarlovs cysts noted",0,1
1559,local_data/mendeley/T2_0375_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal. Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal. NB:Multiple small tarlovs cysts noted",0,1
1560,local_data/mendeley/T1_0377_D3.png,"LSS MRI Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root. adequate spinal canal.",0,1
1561,local_data/mendeley/T1_0377_D4.png,"LSS MRI Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root. adequate spinal canal.",0,1
1562,local_data/mendeley/T1_0377_D5.png,"LSS MRI Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root. adequate spinal canal.",0,1
1563,local_data/mendeley/T2_0377_D3.png,"LSS MRI Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root. adequate spinal canal.",0,1
1564,local_data/mendeley/T2_0377_D4.png,"LSS MRI Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root. adequate spinal canal.",0,1
1565,local_data/mendeley/T2_0377_D5.png,"LSS MRI Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root. adequate spinal canal.",0,1
1566,local_data/mendeley/T1_0381_D3.png,"LSS MRI L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal. L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals. L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1567,local_data/mendeley/T1_0381_D4.png,"LSS MRI L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal. L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals. L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1568,local_data/mendeley/T1_0381_D5.png,"LSS MRI L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal. L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals. L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1569,local_data/mendeley/T2_0381_D3.png,"LSS MRI L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal. L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals. L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1570,local_data/mendeley/T2_0381_D4.png,"LSS MRI L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal. L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals. L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1571,local_data/mendeley/T2_0381_D5.png,"LSS MRI L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal. L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals. L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1
1572,local_data/mendeley/T1_0382_D3.png,"LSS MRI : -Feature of muscle spasm. -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan. -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal. Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1573,local_data/mendeley/T1_0382_D4.png,"LSS MRI : -Feature of muscle spasm. -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan. -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal. Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1574,local_data/mendeley/T1_0382_D5.png,"LSS MRI : -Feature of muscle spasm. -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan. -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal. Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1575,local_data/mendeley/T2_0382_D3.png,"LSS MRI : -Feature of muscle spasm. -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan. -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal. Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1576,local_data/mendeley/T2_0382_D4.png,"LSS MRI : -Feature of muscle spasm. -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan. -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal. Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1577,local_data/mendeley/T2_0382_D5.png,"LSS MRI : -Feature of muscle spasm. -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan. -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal. Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1
1578,local_data/mendeley/T1_0383_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1579,local_data/mendeley/T1_0383_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1580,local_data/mendeley/T1_0383_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1581,local_data/mendeley/T2_0383_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1582,local_data/mendeley/T2_0383_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1583,local_data/mendeley/T2_0383_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1
1584,local_data/mendeley/T1_0384_D3.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
1585,local_data/mendeley/T1_0384_D4.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
1586,local_data/mendeley/T1_0384_D5.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
1587,local_data/mendeley/T2_0384_D3.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
1588,local_data/mendeley/T2_0384_D4.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
1589,local_data/mendeley/T2_0384_D5.png,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0
1590,local_data/mendeley/T1_0385_D3.png,"LSS MRI : There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1591,local_data/mendeley/T1_0385_D4.png,"LSS MRI : There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1592,local_data/mendeley/T1_0385_D5.png,"LSS MRI : There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1593,local_data/mendeley/T2_0385_D3.png,"LSS MRI : There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1594,local_data/mendeley/T2_0385_D4.png,"LSS MRI : There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1595,local_data/mendeley/T2_0385_D5.png,"LSS MRI : There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1
1596,local_data/mendeley/T1_0386_D3.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1597,local_data/mendeley/T1_0386_D4.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1598,local_data/mendeley/T1_0386_D5.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1599,local_data/mendeley/T2_0386_D3.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1600,local_data/mendeley/T2_0386_D4.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1601,local_data/mendeley/T2_0386_D5.png,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
1602,local_data/mendeley/T1_0387_D3.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1603,local_data/mendeley/T1_0387_D4.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1604,local_data/mendeley/T1_0387_D5.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1605,local_data/mendeley/T2_0387_D3.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1606,local_data/mendeley/T2_0387_D4.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1607,local_data/mendeley/T2_0387_D5.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1
1608,local_data/mendeley/T1_0388_D3.png,"Feature of muscle spasm. L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1609,local_data/mendeley/T1_0388_D4.png,"Feature of muscle spasm. L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1610,local_data/mendeley/T1_0388_D5.png,"Feature of muscle spasm. L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1611,local_data/mendeley/T2_0388_D3.png,"Feature of muscle spasm. L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1612,local_data/mendeley/T2_0388_D4.png,"Feature of muscle spasm. L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1613,local_data/mendeley/T2_0388_D5.png,"Feature of muscle spasm. L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1
1614,local_data/mendeley/T1_0389_D3.png,MRI LSS Feature of muscle spasm Mild diffuse disc bulge noted at L4-L5 level No significant thecal sac or nerve root compression noted,1,0
1615,local_data/mendeley/T1_0389_D4.png,MRI LSS Feature of muscle spasm Mild diffuse disc bulge noted at L4-L5 level No significant thecal sac or nerve root compression noted,1,0
1616,local_data/mendeley/T1_0389_D5.png,MRI LSS Feature of muscle spasm Mild diffuse disc bulge noted at L4-L5 level No significant thecal sac or nerve root compression noted,1,0
1617,local_data/mendeley/T2_0389_D3.png,MRI LSS Feature of muscle spasm Mild diffuse disc bulge noted at L4-L5 level No significant thecal sac or nerve root compression noted,1,0
1618,local_data/mendeley/T2_0389_D4.png,MRI LSS Feature of muscle spasm Mild diffuse disc bulge noted at L4-L5 level No significant thecal sac or nerve root compression noted,1,0
1619,local_data/mendeley/T2_0389_D5.png,MRI LSS Feature of muscle spasm Mild diffuse disc bulge noted at L4-L5 level No significant thecal sac or nerve root compression noted,1,0
1620,local_data/mendeley/T1_0392_D3.png,"LSS MRI : -Feature of muscle spasm. -Dgenerative spondolytic changes -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals. -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1621,local_data/mendeley/T1_0392_D4.png,"LSS MRI : -Feature of muscle spasm. -Dgenerative spondolytic changes -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals. -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1622,local_data/mendeley/T1_0392_D5.png,"LSS MRI : -Feature of muscle spasm. -Dgenerative spondolytic changes -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals. -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1623,local_data/mendeley/T2_0392_D3.png,"LSS MRI : -Feature of muscle spasm. -Dgenerative spondolytic changes -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals. -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1624,local_data/mendeley/T2_0392_D4.png,"LSS MRI : -Feature of muscle spasm. -Dgenerative spondolytic changes -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals. -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1625,local_data/mendeley/T2_0392_D5.png,"LSS MRI : -Feature of muscle spasm. -Dgenerative spondolytic changes -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals. -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1
1626,local_data/mendeley/T1_0393_D3.png,LSS MRI : faeture of muscle spasm. No definite thecal sac or nerev root compression . Small tarlov cyst noted,1,0
1627,local_data/mendeley/T1_0393_D4.png,LSS MRI : faeture of muscle spasm. No definite thecal sac or nerev root compression . Small tarlov cyst noted,1,0
1628,local_data/mendeley/T1_0393_D5.png,LSS MRI : faeture of muscle spasm. No definite thecal sac or nerev root compression . Small tarlov cyst noted,1,0
1629,local_data/mendeley/T2_0393_D3.png,LSS MRI : faeture of muscle spasm. No definite thecal sac or nerev root compression . Small tarlov cyst noted,1,0
1630,local_data/mendeley/T2_0393_D4.png,LSS MRI : faeture of muscle spasm. No definite thecal sac or nerev root compression . Small tarlov cyst noted,1,0
1631,local_data/mendeley/T2_0393_D5.png,LSS MRI : faeture of muscle spasm. No definite thecal sac or nerev root compression . Small tarlov cyst noted,1,0
1632,local_data/mendeley/T1_0396_D3.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1633,local_data/mendeley/T1_0396_D4.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1634,local_data/mendeley/T1_0396_D5.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1635,local_data/mendeley/T2_0396_D3.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1636,local_data/mendeley/T2_0396_D4.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1637,local_data/mendeley/T2_0396_D5.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1
1638,local_data/mendeley/T1_0397_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1639,local_data/mendeley/T1_0397_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1640,local_data/mendeley/T1_0397_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1641,local_data/mendeley/T2_0397_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1642,local_data/mendeley/T2_0397_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1643,local_data/mendeley/T2_0397_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1
1644,local_data/mendeley/T1_0399_D3.png,"LSS MRI : Dissicating disc materials. Grdae I spondylolisthesis of L4 over L5 level. widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1645,local_data/mendeley/T1_0399_D4.png,"LSS MRI : Dissicating disc materials. Grdae I spondylolisthesis of L4 over L5 level. widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1646,local_data/mendeley/T1_0399_D5.png,"LSS MRI : Dissicating disc materials. Grdae I spondylolisthesis of L4 over L5 level. widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1647,local_data/mendeley/T2_0399_D3.png,"LSS MRI : Dissicating disc materials. Grdae I spondylolisthesis of L4 over L5 level. widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1648,local_data/mendeley/T2_0399_D4.png,"LSS MRI : Dissicating disc materials. Grdae I spondylolisthesis of L4 over L5 level. widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1649,local_data/mendeley/T2_0399_D5.png,"LSS MRI : Dissicating disc materials. Grdae I spondylolisthesis of L4 over L5 level. widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1
1650,local_data/mendeley/T1_0400_D3.png,Lumbosacral MRI Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1651,local_data/mendeley/T1_0400_D4.png,Lumbosacral MRI Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1652,local_data/mendeley/T1_0400_D5.png,Lumbosacral MRI Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1653,local_data/mendeley/T2_0400_D3.png,Lumbosacral MRI Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1654,local_data/mendeley/T2_0400_D4.png,Lumbosacral MRI Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1655,local_data/mendeley/T2_0400_D5.png,Lumbosacral MRI Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1
1656,local_data/mendeley/T1_0402_D3.png,"LSS MRI  fearture of muscle spasm.  L5/S1 level :  Central disc protrusion noted , migrateing caudally , more to Lt side, largely compressing the thecal sac and exit nerve roots  dissicating disc materials .  Modic type II noted .",0,1
1657,local_data/mendeley/T1_0402_D4.png,"LSS MRI  fearture of muscle spasm.  L5/S1 level :  Central disc protrusion noted , migrateing caudally , more to Lt side, largely compressing the thecal sac and exit nerve roots  dissicating disc materials .  Modic type II noted .",0,1
1658,local_data/mendeley/T1_0402_D5.png,"LSS MRI  fearture of muscle spasm.  L5/S1 level :  Central disc protrusion noted , migrateing caudally , more to Lt side, largely compressing the thecal sac and exit nerve roots  dissicating disc materials .  Modic type II noted .",0,1
1659,local_data/mendeley/T2_0402_D3.png,"LSS MRI  fearture of muscle spasm.  L5/S1 level :  Central disc protrusion noted , migrateing caudally , more to Lt side, largely compressing the thecal sac and exit nerve roots  dissicating disc materials .  Modic type II noted .",0,1
1660,local_data/mendeley/T2_0402_D4.png,"LSS MRI  fearture of muscle spasm.  L5/S1 level :  Central disc protrusion noted , migrateing caudally , more to Lt side, largely compressing the thecal sac and exit nerve roots  dissicating disc materials .  Modic type II noted .",0,1
1661,local_data/mendeley/T2_0402_D5.png,"LSS MRI  fearture of muscle spasm.  L5/S1 level :  Central disc protrusion noted , migrateing caudally , more to Lt side, largely compressing the thecal sac and exit nerve roots  dissicating disc materials .  Modic type II noted .",0,1
1662,local_data/mendeley/T1_0403_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1663,local_data/mendeley/T1_0403_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1664,local_data/mendeley/T1_0403_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1665,local_data/mendeley/T2_0403_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1666,local_data/mendeley/T2_0403_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1667,local_data/mendeley/T2_0403_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1668,local_data/mendeley/T1_0404_D3.png,"LSS MRI :  Feature of Muscle spasm.  Diffuse disc bulges noted noticeably at L3/L4/&L4/L5 levels, compressing thecal sac and encroaqching both exit neural canals, associated with ligamentum flavum hyperatrophy at L4/L5 level.",0,1
1669,local_data/mendeley/T1_0404_D4.png,"LSS MRI :  Feature of Muscle spasm.  Diffuse disc bulges noted noticeably at L3/L4/&L4/L5 levels, compressing thecal sac and encroaqching both exit neural canals, associated with ligamentum flavum hyperatrophy at L4/L5 level.",0,1
1670,local_data/mendeley/T1_0404_D5.png,"LSS MRI :  Feature of Muscle spasm.  Diffuse disc bulges noted noticeably at L3/L4/&L4/L5 levels, compressing thecal sac and encroaqching both exit neural canals, associated with ligamentum flavum hyperatrophy at L4/L5 level.",0,1
1671,local_data/mendeley/T2_0404_D3.png,"LSS MRI :  Feature of Muscle spasm.  Diffuse disc bulges noted noticeably at L3/L4/&L4/L5 levels, compressing thecal sac and encroaqching both exit neural canals, associated with ligamentum flavum hyperatrophy at L4/L5 level.",0,1
1672,local_data/mendeley/T2_0404_D4.png,"LSS MRI :  Feature of Muscle spasm.  Diffuse disc bulges noted noticeably at L3/L4/&L4/L5 levels, compressing thecal sac and encroaqching both exit neural canals, associated with ligamentum flavum hyperatrophy at L4/L5 level.",0,1
1673,local_data/mendeley/T2_0404_D5.png,"LSS MRI :  Feature of Muscle spasm.  Diffuse disc bulges noted noticeably at L3/L4/&L4/L5 levels, compressing thecal sac and encroaqching both exit neural canals, associated with ligamentum flavum hyperatrophy at L4/L5 level.",0,1
1674,local_data/mendeley/T1_0405_D3.png,"LSS MRI  Multilevel diffuse disc bulges (L2-L3, L3-L4, L4-L5), compressing the thecal sac and exit canals.  multilevel end plate degenerative changes",0,1
1675,local_data/mendeley/T1_0405_D4.png,"LSS MRI  Multilevel diffuse disc bulges (L2-L3, L3-L4, L4-L5), compressing the thecal sac and exit canals.  multilevel end plate degenerative changes",0,1
1676,local_data/mendeley/T1_0405_D5.png,"LSS MRI  Multilevel diffuse disc bulges (L2-L3, L3-L4, L4-L5), compressing the thecal sac and exit canals.  multilevel end plate degenerative changes",0,1
1677,local_data/mendeley/T2_0405_D3.png,"LSS MRI  Multilevel diffuse disc bulges (L2-L3, L3-L4, L4-L5), compressing the thecal sac and exit canals.  multilevel end plate degenerative changes",0,1
1678,local_data/mendeley/T2_0405_D4.png,"LSS MRI  Multilevel diffuse disc bulges (L2-L3, L3-L4, L4-L5), compressing the thecal sac and exit canals.  multilevel end plate degenerative changes",0,1
1679,local_data/mendeley/T2_0405_D5.png,"LSS MRI  Multilevel diffuse disc bulges (L2-L3, L3-L4, L4-L5), compressing the thecal sac and exit canals.  multilevel end plate degenerative changes",0,1
1680,local_data/mendeley/T1_0406_D3.png,LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level mildly compressing thecal sac,0,1
1681,local_data/mendeley/T1_0406_D4.png,LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level mildly compressing thecal sac,0,1
1682,local_data/mendeley/T1_0406_D5.png,LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level mildly compressing thecal sac,0,1
1683,local_data/mendeley/T2_0406_D3.png,LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level mildly compressing thecal sac,0,1
1684,local_data/mendeley/T2_0406_D4.png,LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level mildly compressing thecal sac,0,1
1685,local_data/mendeley/T2_0406_D5.png,LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level mildly compressing thecal sac,0,1
1686,local_data/mendeley/T1_0407_D3.png,"LSS MRI  Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  Adequate spinal canal.",0,1
1687,local_data/mendeley/T1_0407_D4.png,"LSS MRI  Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  Adequate spinal canal.",0,1
1688,local_data/mendeley/T1_0407_D5.png,"LSS MRI  Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  Adequate spinal canal.",0,1
1689,local_data/mendeley/T2_0407_D3.png,"LSS MRI  Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  Adequate spinal canal.",0,1
1690,local_data/mendeley/T2_0407_D4.png,"LSS MRI  Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  Adequate spinal canal.",0,1
1691,local_data/mendeley/T2_0407_D5.png,"LSS MRI  Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  Adequate spinal canal.",0,1
1692,local_data/mendeley/T1_0408_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1693,local_data/mendeley/T1_0408_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1694,local_data/mendeley/T1_0408_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1695,local_data/mendeley/T2_0408_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1696,local_data/mendeley/T2_0408_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1697,local_data/mendeley/T2_0408_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1698,local_data/mendeley/T1_0409_D3.png,"LSS MRI :  faeture of muscle spasm.  L4/L5 level :(disscating disc material ):  Central and Lt paracentral disc protruion migrating dawnwoard ,compressing thecal sac and Lt exit nerve roots .",0,1
1699,local_data/mendeley/T1_0409_D4.png,"LSS MRI :  faeture of muscle spasm.  L4/L5 level :(disscating disc material ):  Central and Lt paracentral disc protruion migrating dawnwoard ,compressing thecal sac and Lt exit nerve roots .",0,1
1700,local_data/mendeley/T1_0409_D5.png,"LSS MRI :  faeture of muscle spasm.  L4/L5 level :(disscating disc material ):  Central and Lt paracentral disc protruion migrating dawnwoard ,compressing thecal sac and Lt exit nerve roots .",0,1
1701,local_data/mendeley/T2_0409_D3.png,"LSS MRI :  faeture of muscle spasm.  L4/L5 level :(disscating disc material ):  Central and Lt paracentral disc protruion migrating dawnwoard ,compressing thecal sac and Lt exit nerve roots .",0,1
1702,local_data/mendeley/T2_0409_D4.png,"LSS MRI :  faeture of muscle spasm.  L4/L5 level :(disscating disc material ):  Central and Lt paracentral disc protruion migrating dawnwoard ,compressing thecal sac and Lt exit nerve roots .",0,1
1703,local_data/mendeley/T2_0409_D5.png,"LSS MRI :  faeture of muscle spasm.  L4/L5 level :(disscating disc material ):  Central and Lt paracentral disc protruion migrating dawnwoard ,compressing thecal sac and Lt exit nerve roots .",0,1
1704,local_data/mendeley/T1_0410_D3.png,"Compression fracture of L5 vertebra with retrobulge into spinal canal, compressing the thecal sac.",0,1
1705,local_data/mendeley/T1_0410_D4.png,"Compression fracture of L5 vertebra with retrobulge into spinal canal, compressing the thecal sac.",0,1
1706,local_data/mendeley/T1_0410_D5.png,"Compression fracture of L5 vertebra with retrobulge into spinal canal, compressing the thecal sac.",0,1
1707,local_data/mendeley/T2_0410_D3.png,"Compression fracture of L5 vertebra with retrobulge into spinal canal, compressing the thecal sac.",0,1
1708,local_data/mendeley/T2_0410_D4.png,"Compression fracture of L5 vertebra with retrobulge into spinal canal, compressing the thecal sac.",0,1
1709,local_data/mendeley/T2_0410_D5.png,"Compression fracture of L5 vertebra with retrobulge into spinal canal, compressing the thecal sac.",0,1
1710,local_data/mendeley/T1_0411_D3.png,Lumbosacral MRI  Central disc protrusion noted at L4-L5 level compressing the thecal sac  Mild disc bulge noted at L5-S1 level  Adequate spinal canal  Features of muscle spasm noted,0,1
1711,local_data/mendeley/T1_0411_D4.png,Lumbosacral MRI  Central disc protrusion noted at L4-L5 level compressing the thecal sac  Mild disc bulge noted at L5-S1 level  Adequate spinal canal  Features of muscle spasm noted,0,1
1712,local_data/mendeley/T1_0411_D5.png,Lumbosacral MRI  Central disc protrusion noted at L4-L5 level compressing the thecal sac  Mild disc bulge noted at L5-S1 level  Adequate spinal canal  Features of muscle spasm noted,0,1
1713,local_data/mendeley/T2_0411_D3.png,Lumbosacral MRI  Central disc protrusion noted at L4-L5 level compressing the thecal sac  Mild disc bulge noted at L5-S1 level  Adequate spinal canal  Features of muscle spasm noted,0,1
1714,local_data/mendeley/T2_0411_D4.png,Lumbosacral MRI  Central disc protrusion noted at L4-L5 level compressing the thecal sac  Mild disc bulge noted at L5-S1 level  Adequate spinal canal  Features of muscle spasm noted,0,1
1715,local_data/mendeley/T2_0411_D5.png,Lumbosacral MRI  Central disc protrusion noted at L4-L5 level compressing the thecal sac  Mild disc bulge noted at L5-S1 level  Adequate spinal canal  Features of muscle spasm noted,0,1
1716,local_data/mendeley/T1_0412_D3.png,"LSS MRI :  wide base , Rt paracentral ,disc bulge noted at L4/L5 level , compressing thecal sac and Rt exit nerve root.  posterior central disc protrusion noted at L5/S1 level , with mild cephalic migration, compressing the thecal sac and encroaching exit neuralcanal.  NB:Lt Pelvic kidney noted , correalte with ultrasound",0,1
1717,local_data/mendeley/T1_0412_D4.png,"LSS MRI :  wide base , Rt paracentral ,disc bulge noted at L4/L5 level , compressing thecal sac and Rt exit nerve root.  posterior central disc protrusion noted at L5/S1 level , with mild cephalic migration, compressing the thecal sac and encroaching exit neuralcanal.  NB:Lt Pelvic kidney noted , correalte with ultrasound",0,1
1718,local_data/mendeley/T1_0412_D5.png,"LSS MRI :  wide base , Rt paracentral ,disc bulge noted at L4/L5 level , compressing thecal sac and Rt exit nerve root.  posterior central disc protrusion noted at L5/S1 level , with mild cephalic migration, compressing the thecal sac and encroaching exit neuralcanal.  NB:Lt Pelvic kidney noted , correalte with ultrasound",0,1
1719,local_data/mendeley/T2_0412_D3.png,"LSS MRI :  wide base , Rt paracentral ,disc bulge noted at L4/L5 level , compressing thecal sac and Rt exit nerve root.  posterior central disc protrusion noted at L5/S1 level , with mild cephalic migration, compressing the thecal sac and encroaching exit neuralcanal.  NB:Lt Pelvic kidney noted , correalte with ultrasound",0,1
1720,local_data/mendeley/T2_0412_D4.png,"LSS MRI :  wide base , Rt paracentral ,disc bulge noted at L4/L5 level , compressing thecal sac and Rt exit nerve root.  posterior central disc protrusion noted at L5/S1 level , with mild cephalic migration, compressing the thecal sac and encroaching exit neuralcanal.  NB:Lt Pelvic kidney noted , correalte with ultrasound",0,1
1721,local_data/mendeley/T2_0412_D5.png,"LSS MRI :  wide base , Rt paracentral ,disc bulge noted at L4/L5 level , compressing thecal sac and Rt exit nerve root.  posterior central disc protrusion noted at L5/S1 level , with mild cephalic migration, compressing the thecal sac and encroaching exit neuralcanal.  NB:Lt Pelvic kidney noted , correalte with ultrasound",0,1
1722,local_data/mendeley/T1_0413_D3.png,LSS MRI  No evidence of disc herniation noted.  No thecal sac or nerve root compression noted.,1,0
1723,local_data/mendeley/T1_0413_D4.png,LSS MRI  No evidence of disc herniation noted.  No thecal sac or nerve root compression noted.,1,0
1724,local_data/mendeley/T1_0413_D5.png,LSS MRI  No evidence of disc herniation noted.  No thecal sac or nerve root compression noted.,1,0
1725,local_data/mendeley/T2_0413_D3.png,LSS MRI  No evidence of disc herniation noted.  No thecal sac or nerve root compression noted.,1,0
1726,local_data/mendeley/T2_0413_D4.png,LSS MRI  No evidence of disc herniation noted.  No thecal sac or nerve root compression noted.,1,0
1727,local_data/mendeley/T2_0413_D5.png,LSS MRI  No evidence of disc herniation noted.  No thecal sac or nerve root compression noted.,1,0
1728,local_data/mendeley/T1_0414_D3.png,"LSS MRI :  faetuer of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, extending to lateral recess, compressing thecal sac and exit neural canals with annular tear .",0,1
1729,local_data/mendeley/T1_0414_D4.png,"LSS MRI :  faetuer of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, extending to lateral recess, compressing thecal sac and exit neural canals with annular tear .",0,1
1730,local_data/mendeley/T1_0414_D5.png,"LSS MRI :  faetuer of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, extending to lateral recess, compressing thecal sac and exit neural canals with annular tear .",0,1
1731,local_data/mendeley/T2_0414_D3.png,"LSS MRI :  faetuer of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, extending to lateral recess, compressing thecal sac and exit neural canals with annular tear .",0,1
1732,local_data/mendeley/T2_0414_D4.png,"LSS MRI :  faetuer of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, extending to lateral recess, compressing thecal sac and exit neural canals with annular tear .",0,1
1733,local_data/mendeley/T2_0414_D5.png,"LSS MRI :  faetuer of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, extending to lateral recess, compressing thecal sac and exit neural canals with annular tear .",0,1
1734,local_data/mendeley/T1_0415_D3.png,"L3-L4, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level where mild spondylolisthesis noted, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
1735,local_data/mendeley/T1_0415_D4.png,"L3-L4, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level where mild spondylolisthesis noted, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
1736,local_data/mendeley/T1_0415_D5.png,"L3-L4, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level where mild spondylolisthesis noted, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
1737,local_data/mendeley/T2_0415_D3.png,"L3-L4, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level where mild spondylolisthesis noted, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
1738,local_data/mendeley/T2_0415_D4.png,"L3-L4, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level where mild spondylolisthesis noted, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
1739,local_data/mendeley/T2_0415_D5.png,"L3-L4, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level where mild spondylolisthesis noted, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
1740,local_data/mendeley/T1_0416_D3.png,"LSS MRI  L5-S1: Mild disc bulge with Lt paracentral annular tear, compressing the thecal sac and encroaching upon Lt exit neural canal.  L4-L5: Mild central disc protrusion noted compressing the thecal sac.  L3-L4: Mild disc bulge with posterocentral tear.",0,1
1741,local_data/mendeley/T1_0416_D4.png,"LSS MRI  L5-S1: Mild disc bulge with Lt paracentral annular tear, compressing the thecal sac and encroaching upon Lt exit neural canal.  L4-L5: Mild central disc protrusion noted compressing the thecal sac.  L3-L4: Mild disc bulge with posterocentral tear.",0,1
1742,local_data/mendeley/T1_0416_D5.png,"LSS MRI  L5-S1: Mild disc bulge with Lt paracentral annular tear, compressing the thecal sac and encroaching upon Lt exit neural canal.  L4-L5: Mild central disc protrusion noted compressing the thecal sac.  L3-L4: Mild disc bulge with posterocentral tear.",0,1
1743,local_data/mendeley/T2_0416_D3.png,"LSS MRI  L5-S1: Mild disc bulge with Lt paracentral annular tear, compressing the thecal sac and encroaching upon Lt exit neural canal.  L4-L5: Mild central disc protrusion noted compressing the thecal sac.  L3-L4: Mild disc bulge with posterocentral tear.",0,1
1744,local_data/mendeley/T2_0416_D4.png,"LSS MRI  L5-S1: Mild disc bulge with Lt paracentral annular tear, compressing the thecal sac and encroaching upon Lt exit neural canal.  L4-L5: Mild central disc protrusion noted compressing the thecal sac.  L3-L4: Mild disc bulge with posterocentral tear.",0,1
1745,local_data/mendeley/T2_0416_D5.png,"LSS MRI  L5-S1: Mild disc bulge with Lt paracentral annular tear, compressing the thecal sac and encroaching upon Lt exit neural canal.  L4-L5: Mild central disc protrusion noted compressing the thecal sac.  L3-L4: Mild disc bulge with posterocentral tear.",0,1
1746,local_data/mendeley/T1_0417_D3.png,"LSS MRI  Small central disc protrusions noted at L4-L5 amnd L5-S1 levels, compressing the thecal sac.  adequate spinal canal.",0,1
1747,local_data/mendeley/T1_0417_D4.png,"LSS MRI  Small central disc protrusions noted at L4-L5 amnd L5-S1 levels, compressing the thecal sac.  adequate spinal canal.",0,1
1748,local_data/mendeley/T1_0417_D5.png,"LSS MRI  Small central disc protrusions noted at L4-L5 amnd L5-S1 levels, compressing the thecal sac.  adequate spinal canal.",0,1
1749,local_data/mendeley/T2_0417_D3.png,"LSS MRI  Small central disc protrusions noted at L4-L5 amnd L5-S1 levels, compressing the thecal sac.  adequate spinal canal.",0,1
1750,local_data/mendeley/T2_0417_D4.png,"LSS MRI  Small central disc protrusions noted at L4-L5 amnd L5-S1 levels, compressing the thecal sac.  adequate spinal canal.",0,1
1751,local_data/mendeley/T2_0417_D5.png,"LSS MRI  Small central disc protrusions noted at L4-L5 amnd L5-S1 levels, compressing the thecal sac.  adequate spinal canal.",0,1
1752,local_data/mendeley/T1_0419_D3.png,L3-L4 and L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals L5-S1: Mild disc bulge noted.,0,1
1753,local_data/mendeley/T1_0419_D4.png,L3-L4 and L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals L5-S1: Mild disc bulge noted.,0,1
1754,local_data/mendeley/T1_0419_D5.png,L3-L4 and L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals L5-S1: Mild disc bulge noted.,0,1
1755,local_data/mendeley/T2_0419_D3.png,L3-L4 and L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals L5-S1: Mild disc bulge noted.,0,1
1756,local_data/mendeley/T2_0419_D4.png,L3-L4 and L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals L5-S1: Mild disc bulge noted.,0,1
1757,local_data/mendeley/T2_0419_D5.png,L3-L4 and L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals L5-S1: Mild disc bulge noted.,0,1
1758,local_data/mendeley/T1_0420_D3.png,LSS MRI :  Feature of muscle spasm.  No significant disc protrusion or herniation seen.  No thecal sac or exit nerve root compression noted .  Preserved vertebral bodies hieght and bone marrow signal intensity seen.,1,0
1759,local_data/mendeley/T1_0420_D4.png,LSS MRI :  Feature of muscle spasm.  No significant disc protrusion or herniation seen.  No thecal sac or exit nerve root compression noted .  Preserved vertebral bodies hieght and bone marrow signal intensity seen.,1,0
1760,local_data/mendeley/T1_0420_D5.png,LSS MRI :  Feature of muscle spasm.  No significant disc protrusion or herniation seen.  No thecal sac or exit nerve root compression noted .  Preserved vertebral bodies hieght and bone marrow signal intensity seen.,1,0
1761,local_data/mendeley/T2_0420_D3.png,LSS MRI :  Feature of muscle spasm.  No significant disc protrusion or herniation seen.  No thecal sac or exit nerve root compression noted .  Preserved vertebral bodies hieght and bone marrow signal intensity seen.,1,0
1762,local_data/mendeley/T2_0420_D4.png,LSS MRI :  Feature of muscle spasm.  No significant disc protrusion or herniation seen.  No thecal sac or exit nerve root compression noted .  Preserved vertebral bodies hieght and bone marrow signal intensity seen.,1,0
1763,local_data/mendeley/T2_0420_D5.png,LSS MRI :  Feature of muscle spasm.  No significant disc protrusion or herniation seen.  No thecal sac or exit nerve root compression noted .  Preserved vertebral bodies hieght and bone marrow signal intensity seen.,1,0
1764,local_data/mendeley/T1_0421_D3.png,L4-L5: Mild spondylolisthesis with secondary disc bulge abutting the thecal sac and encroaching exit canals. L1-L2: Posterior osteophytes noted abutting the thecal sac,0,1
1765,local_data/mendeley/T1_0421_D4.png,L4-L5: Mild spondylolisthesis with secondary disc bulge abutting the thecal sac and encroaching exit canals. L1-L2: Posterior osteophytes noted abutting the thecal sac,0,1
1766,local_data/mendeley/T1_0421_D5.png,L4-L5: Mild spondylolisthesis with secondary disc bulge abutting the thecal sac and encroaching exit canals. L1-L2: Posterior osteophytes noted abutting the thecal sac,0,1
1767,local_data/mendeley/T2_0421_D3.png,L4-L5: Mild spondylolisthesis with secondary disc bulge abutting the thecal sac and encroaching exit canals. L1-L2: Posterior osteophytes noted abutting the thecal sac,0,1
1768,local_data/mendeley/T2_0421_D4.png,L4-L5: Mild spondylolisthesis with secondary disc bulge abutting the thecal sac and encroaching exit canals. L1-L2: Posterior osteophytes noted abutting the thecal sac,0,1
1769,local_data/mendeley/T2_0421_D5.png,L4-L5: Mild spondylolisthesis with secondary disc bulge abutting the thecal sac and encroaching exit canals. L1-L2: Posterior osteophytes noted abutting the thecal sac,0,1
1770,local_data/mendeley/T1_0423_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, compressing thecal sac and both nerve roots .",0,1
1771,local_data/mendeley/T1_0423_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, compressing thecal sac and both nerve roots .",0,1
1772,local_data/mendeley/T1_0423_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, compressing thecal sac and both nerve roots .",0,1
1773,local_data/mendeley/T2_0423_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, compressing thecal sac and both nerve roots .",0,1
1774,local_data/mendeley/T2_0423_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, compressing thecal sac and both nerve roots .",0,1
1775,local_data/mendeley/T2_0423_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, compressing thecal sac and both nerve roots .",0,1
1776,local_data/mendeley/T1_0424_D3.png,"LSS MRI :  Cetral & Rt paracentral disc protrusion noted at L4/L5 level , largely compressing the Rt sided thecal sac and exit nerve root.",0,1
1777,local_data/mendeley/T1_0424_D4.png,"LSS MRI :  Cetral & Rt paracentral disc protrusion noted at L4/L5 level , largely compressing the Rt sided thecal sac and exit nerve root.",0,1
1778,local_data/mendeley/T1_0424_D5.png,"LSS MRI :  Cetral & Rt paracentral disc protrusion noted at L4/L5 level , largely compressing the Rt sided thecal sac and exit nerve root.",0,1
1779,local_data/mendeley/T2_0424_D3.png,"LSS MRI :  Cetral & Rt paracentral disc protrusion noted at L4/L5 level , largely compressing the Rt sided thecal sac and exit nerve root.",0,1
1780,local_data/mendeley/T2_0424_D4.png,"LSS MRI :  Cetral & Rt paracentral disc protrusion noted at L4/L5 level , largely compressing the Rt sided thecal sac and exit nerve root.",0,1
1781,local_data/mendeley/T2_0424_D5.png,"LSS MRI :  Cetral & Rt paracentral disc protrusion noted at L4/L5 level , largely compressing the Rt sided thecal sac and exit nerve root.",0,1
1782,local_data/mendeley/T1_0425_D3.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals.",0,1
1783,local_data/mendeley/T1_0425_D4.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals.",0,1
1784,local_data/mendeley/T1_0425_D5.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals.",0,1
1785,local_data/mendeley/T2_0425_D3.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals.",0,1
1786,local_data/mendeley/T2_0425_D4.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals.",0,1
1787,local_data/mendeley/T2_0425_D5.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals.",0,1
1788,local_data/mendeley/T1_0426_D3.png,"LSS MRI :  Lt paracentral disc protrusion noted at L4/L5 level, largely compressing (Lt) thecal sca and exit nerve root",0,1
1789,local_data/mendeley/T1_0426_D4.png,"LSS MRI :  Lt paracentral disc protrusion noted at L4/L5 level, largely compressing (Lt) thecal sca and exit nerve root",0,1
1790,local_data/mendeley/T1_0426_D5.png,"LSS MRI :  Lt paracentral disc protrusion noted at L4/L5 level, largely compressing (Lt) thecal sca and exit nerve root",0,1
1791,local_data/mendeley/T2_0426_D3.png,"LSS MRI :  Lt paracentral disc protrusion noted at L4/L5 level, largely compressing (Lt) thecal sca and exit nerve root",0,1
1792,local_data/mendeley/T2_0426_D4.png,"LSS MRI :  Lt paracentral disc protrusion noted at L4/L5 level, largely compressing (Lt) thecal sca and exit nerve root",0,1
1793,local_data/mendeley/T2_0426_D5.png,"LSS MRI :  Lt paracentral disc protrusion noted at L4/L5 level, largely compressing (Lt) thecal sca and exit nerve root",0,1
1794,local_data/mendeley/T1_0427_D3.png,"LSS MRI :  .Feature of muscle spasm.  .posterior central disc protrusion noted at L4/L5 level, largely compressing the thecal sac and enrcoaching exit neural canal.",0,1
1795,local_data/mendeley/T1_0427_D4.png,"LSS MRI :  .Feature of muscle spasm.  .posterior central disc protrusion noted at L4/L5 level, largely compressing the thecal sac and enrcoaching exit neural canal.",0,1
1796,local_data/mendeley/T1_0427_D5.png,"LSS MRI :  .Feature of muscle spasm.  .posterior central disc protrusion noted at L4/L5 level, largely compressing the thecal sac and enrcoaching exit neural canal.",0,1
1797,local_data/mendeley/T2_0427_D3.png,"LSS MRI :  .Feature of muscle spasm.  .posterior central disc protrusion noted at L4/L5 level, largely compressing the thecal sac and enrcoaching exit neural canal.",0,1
1798,local_data/mendeley/T2_0427_D4.png,"LSS MRI :  .Feature of muscle spasm.  .posterior central disc protrusion noted at L4/L5 level, largely compressing the thecal sac and enrcoaching exit neural canal.",0,1
1799,local_data/mendeley/T2_0427_D5.png,"LSS MRI :  .Feature of muscle spasm.  .posterior central disc protrusion noted at L4/L5 level, largely compressing the thecal sac and enrcoaching exit neural canal.",0,1
1800,local_data/mendeley/T1_0428_D3.png,"C.SPINE MRI :  Feature of muscle spasm.  Wide base disc bulge noted at C5/C6 level, obliterating anterior CSF spaces , and encroachong exit neural canal.  LSS MRI :  Feature of muscle spasm.  Dehydrated disc materials at lower levels.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and encroaching exit neural canal with annular tear .",0,1
1801,local_data/mendeley/T1_0428_D4.png,"C.SPINE MRI :  Feature of muscle spasm.  Wide base disc bulge noted at C5/C6 level, obliterating anterior CSF spaces , and encroachong exit neural canal.  LSS MRI :  Feature of muscle spasm.  Dehydrated disc materials at lower levels.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and encroaching exit neural canal with annular tear .",0,1
1802,local_data/mendeley/T1_0428_D5.png,"C.SPINE MRI :  Feature of muscle spasm.  Wide base disc bulge noted at C5/C6 level, obliterating anterior CSF spaces , and encroachong exit neural canal.  LSS MRI :  Feature of muscle spasm.  Dehydrated disc materials at lower levels.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and encroaching exit neural canal with annular tear .",0,1
1803,local_data/mendeley/T2_0428_D3.png,"C.SPINE MRI :  Feature of muscle spasm.  Wide base disc bulge noted at C5/C6 level, obliterating anterior CSF spaces , and encroachong exit neural canal.  LSS MRI :  Feature of muscle spasm.  Dehydrated disc materials at lower levels.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and encroaching exit neural canal with annular tear .",0,1
1804,local_data/mendeley/T2_0428_D4.png,"C.SPINE MRI :  Feature of muscle spasm.  Wide base disc bulge noted at C5/C6 level, obliterating anterior CSF spaces , and encroachong exit neural canal.  LSS MRI :  Feature of muscle spasm.  Dehydrated disc materials at lower levels.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and encroaching exit neural canal with annular tear .",0,1
1805,local_data/mendeley/T2_0428_D5.png,"C.SPINE MRI :  Feature of muscle spasm.  Wide base disc bulge noted at C5/C6 level, obliterating anterior CSF spaces , and encroachong exit neural canal.  LSS MRI :  Feature of muscle spasm.  Dehydrated disc materials at lower levels.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and encroaching exit neural canal with annular tear .",0,1
1806,local_data/mendeley/T1_0429_D3.png,"LSS MRI :  feature of muscle spasm.  multiple schmorls nodules .  dissicating disc material at L5/S1 level.  Rt paracentral disc protruion noted at L5/S1 level ,compression thecal sac and Rt exit nerve root.",0,1
1807,local_data/mendeley/T1_0429_D4.png,"LSS MRI :  feature of muscle spasm.  multiple schmorls nodules .  dissicating disc material at L5/S1 level.  Rt paracentral disc protruion noted at L5/S1 level ,compression thecal sac and Rt exit nerve root.",0,1
1808,local_data/mendeley/T1_0429_D5.png,"LSS MRI :  feature of muscle spasm.  multiple schmorls nodules .  dissicating disc material at L5/S1 level.  Rt paracentral disc protruion noted at L5/S1 level ,compression thecal sac and Rt exit nerve root.",0,1
1809,local_data/mendeley/T2_0429_D3.png,"LSS MRI :  feature of muscle spasm.  multiple schmorls nodules .  dissicating disc material at L5/S1 level.  Rt paracentral disc protruion noted at L5/S1 level ,compression thecal sac and Rt exit nerve root.",0,1
1810,local_data/mendeley/T2_0429_D4.png,"LSS MRI :  feature of muscle spasm.  multiple schmorls nodules .  dissicating disc material at L5/S1 level.  Rt paracentral disc protruion noted at L5/S1 level ,compression thecal sac and Rt exit nerve root.",0,1
1811,local_data/mendeley/T2_0429_D5.png,"LSS MRI :  feature of muscle spasm.  multiple schmorls nodules .  dissicating disc material at L5/S1 level.  Rt paracentral disc protruion noted at L5/S1 level ,compression thecal sac and Rt exit nerve root.",0,1
1812,local_data/mendeley/T1_0430_D3.png,"LSS MRI :  FEATure of muscle spasm.  Diffuse disc bulge with Lt para central disc protrusion & sequestration , noted at L5/S1, largely compressing the thecal sac and encroaching exit neural canal, with narrowed disc space and dehydrated disc material.",0,1
1813,local_data/mendeley/T1_0430_D4.png,"LSS MRI :  FEATure of muscle spasm.  Diffuse disc bulge with Lt para central disc protrusion & sequestration , noted at L5/S1, largely compressing the thecal sac and encroaching exit neural canal, with narrowed disc space and dehydrated disc material.",0,1
1814,local_data/mendeley/T1_0430_D5.png,"LSS MRI :  FEATure of muscle spasm.  Diffuse disc bulge with Lt para central disc protrusion & sequestration , noted at L5/S1, largely compressing the thecal sac and encroaching exit neural canal, with narrowed disc space and dehydrated disc material.",0,1
1815,local_data/mendeley/T2_0430_D3.png,"LSS MRI :  FEATure of muscle spasm.  Diffuse disc bulge with Lt para central disc protrusion & sequestration , noted at L5/S1, largely compressing the thecal sac and encroaching exit neural canal, with narrowed disc space and dehydrated disc material.",0,1
1816,local_data/mendeley/T2_0430_D4.png,"LSS MRI :  FEATure of muscle spasm.  Diffuse disc bulge with Lt para central disc protrusion & sequestration , noted at L5/S1, largely compressing the thecal sac and encroaching exit neural canal, with narrowed disc space and dehydrated disc material.",0,1
1817,local_data/mendeley/T2_0430_D5.png,"LSS MRI :  FEATure of muscle spasm.  Diffuse disc bulge with Lt para central disc protrusion & sequestration , noted at L5/S1, largely compressing the thecal sac and encroaching exit neural canal, with narrowed disc space and dehydrated disc material.",0,1
1818,local_data/mendeley/T1_0431_D3.png,LSS MRI  diffuse disc bulges noted at L4-L5 levels compressing the thecal sac.  mild spondylolisthesis of L4 over L5 vertebrae.,0,1
1819,local_data/mendeley/T1_0431_D4.png,LSS MRI  diffuse disc bulges noted at L4-L5 levels compressing the thecal sac.  mild spondylolisthesis of L4 over L5 vertebrae.,0,1
1820,local_data/mendeley/T1_0431_D5.png,LSS MRI  diffuse disc bulges noted at L4-L5 levels compressing the thecal sac.  mild spondylolisthesis of L4 over L5 vertebrae.,0,1
1821,local_data/mendeley/T2_0431_D3.png,LSS MRI  diffuse disc bulges noted at L4-L5 levels compressing the thecal sac.  mild spondylolisthesis of L4 over L5 vertebrae.,0,1
1822,local_data/mendeley/T2_0431_D4.png,LSS MRI  diffuse disc bulges noted at L4-L5 levels compressing the thecal sac.  mild spondylolisthesis of L4 over L5 vertebrae.,0,1
1823,local_data/mendeley/T2_0431_D5.png,LSS MRI  diffuse disc bulges noted at L4-L5 levels compressing the thecal sac.  mild spondylolisthesis of L4 over L5 vertebrae.,0,1
1824,local_data/mendeley/T1_0432_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level , mild compresing thecal sac and nerve roots",0,1
1825,local_data/mendeley/T1_0432_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level , mild compresing thecal sac and nerve roots",0,1
1826,local_data/mendeley/T1_0432_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level , mild compresing thecal sac and nerve roots",0,1
1827,local_data/mendeley/T2_0432_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level , mild compresing thecal sac and nerve roots",0,1
1828,local_data/mendeley/T2_0432_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level , mild compresing thecal sac and nerve roots",0,1
1829,local_data/mendeley/T2_0432_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level , mild compresing thecal sac and nerve roots",0,1
1830,local_data/mendeley/T1_0434_D3.png,Lumbosacral MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,1,0
1831,local_data/mendeley/T1_0434_D4.png,Lumbosacral MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,1,0
1832,local_data/mendeley/T1_0434_D5.png,Lumbosacral MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,1,0
1833,local_data/mendeley/T2_0434_D3.png,Lumbosacral MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,1,0
1834,local_data/mendeley/T2_0434_D4.png,Lumbosacral MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,1,0
1835,local_data/mendeley/T2_0434_D5.png,Lumbosacral MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,1,0
1836,local_data/mendeley/T1_0435_D3.png,Lumbosacral MRI  No evidence of disc herniation  No thecal sac or nerve root compression  Adequate spinal canal,1,0
1837,local_data/mendeley/T1_0435_D4.png,Lumbosacral MRI  No evidence of disc herniation  No thecal sac or nerve root compression  Adequate spinal canal,1,0
1838,local_data/mendeley/T1_0435_D5.png,Lumbosacral MRI  No evidence of disc herniation  No thecal sac or nerve root compression  Adequate spinal canal,1,0
1839,local_data/mendeley/T2_0435_D3.png,Lumbosacral MRI  No evidence of disc herniation  No thecal sac or nerve root compression  Adequate spinal canal,1,0
1840,local_data/mendeley/T2_0435_D4.png,Lumbosacral MRI  No evidence of disc herniation  No thecal sac or nerve root compression  Adequate spinal canal,1,0
1841,local_data/mendeley/T2_0435_D5.png,Lumbosacral MRI  No evidence of disc herniation  No thecal sac or nerve root compression  Adequate spinal canal,1,0
1842,local_data/mendeley/T1_0439_D3.png,"Lumbosacral MRI:  Feature of muscle spasm  Dissicating lower lumbar disc material noted  L4-L5 level:  Central and right paracentral disc protrusion noted with cephalic migration as well as soft tissue density noted mostly sequestrated disc,  Compressing the thecal sac and encroaching the right exit neural canals.  L5-S1 level:  Wide based disc bulge noted compressing the thecal sac and both nerve root.  Contrast study LSS is advised",0,1
1843,local_data/mendeley/T1_0439_D4.png,"Lumbosacral MRI:  Feature of muscle spasm  Dissicating lower lumbar disc material noted  L4-L5 level:  Central and right paracentral disc protrusion noted with cephalic migration as well as soft tissue density noted mostly sequestrated disc,  Compressing the thecal sac and encroaching the right exit neural canals.  L5-S1 level:  Wide based disc bulge noted compressing the thecal sac and both nerve root.  Contrast study LSS is advised",0,1
1844,local_data/mendeley/T1_0439_D5.png,"Lumbosacral MRI:  Feature of muscle spasm  Dissicating lower lumbar disc material noted  L4-L5 level:  Central and right paracentral disc protrusion noted with cephalic migration as well as soft tissue density noted mostly sequestrated disc,  Compressing the thecal sac and encroaching the right exit neural canals.  L5-S1 level:  Wide based disc bulge noted compressing the thecal sac and both nerve root.  Contrast study LSS is advised",0,1
1845,local_data/mendeley/T2_0439_D3.png,"Lumbosacral MRI:  Feature of muscle spasm  Dissicating lower lumbar disc material noted  L4-L5 level:  Central and right paracentral disc protrusion noted with cephalic migration as well as soft tissue density noted mostly sequestrated disc,  Compressing the thecal sac and encroaching the right exit neural canals.  L5-S1 level:  Wide based disc bulge noted compressing the thecal sac and both nerve root.  Contrast study LSS is advised",0,1
1846,local_data/mendeley/T2_0439_D4.png,"Lumbosacral MRI:  Feature of muscle spasm  Dissicating lower lumbar disc material noted  L4-L5 level:  Central and right paracentral disc protrusion noted with cephalic migration as well as soft tissue density noted mostly sequestrated disc,  Compressing the thecal sac and encroaching the right exit neural canals.  L5-S1 level:  Wide based disc bulge noted compressing the thecal sac and both nerve root.  Contrast study LSS is advised",0,1
1847,local_data/mendeley/T2_0439_D5.png,"Lumbosacral MRI:  Feature of muscle spasm  Dissicating lower lumbar disc material noted  L4-L5 level:  Central and right paracentral disc protrusion noted with cephalic migration as well as soft tissue density noted mostly sequestrated disc,  Compressing the thecal sac and encroaching the right exit neural canals.  L5-S1 level:  Wide based disc bulge noted compressing the thecal sac and both nerve root.  Contrast study LSS is advised",0,1
1848,local_data/mendeley/T1_0442_D3.png,"LSS MRI :  Feature of muscle spasm.-  -multiple schmorls nodules noted  -Dissicating disc materials at lower levels.  -posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and exit nerve roots more to Lt side.  -Diffuse disc bulge with small central protrusion noted at L4/L5 level compressing thecal sac and both nerve roots .",0,1
1849,local_data/mendeley/T1_0442_D4.png,"LSS MRI :  Feature of muscle spasm.-  -multiple schmorls nodules noted  -Dissicating disc materials at lower levels.  -posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and exit nerve roots more to Lt side.  -Diffuse disc bulge with small central protrusion noted at L4/L5 level compressing thecal sac and both nerve roots .",0,1
1850,local_data/mendeley/T1_0442_D5.png,"LSS MRI :  Feature of muscle spasm.-  -multiple schmorls nodules noted  -Dissicating disc materials at lower levels.  -posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and exit nerve roots more to Lt side.  -Diffuse disc bulge with small central protrusion noted at L4/L5 level compressing thecal sac and both nerve roots .",0,1
1851,local_data/mendeley/T2_0442_D3.png,"LSS MRI :  Feature of muscle spasm.-  -multiple schmorls nodules noted  -Dissicating disc materials at lower levels.  -posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and exit nerve roots more to Lt side.  -Diffuse disc bulge with small central protrusion noted at L4/L5 level compressing thecal sac and both nerve roots .",0,1
1852,local_data/mendeley/T2_0442_D4.png,"LSS MRI :  Feature of muscle spasm.-  -multiple schmorls nodules noted  -Dissicating disc materials at lower levels.  -posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and exit nerve roots more to Lt side.  -Diffuse disc bulge with small central protrusion noted at L4/L5 level compressing thecal sac and both nerve roots .",0,1
1853,local_data/mendeley/T2_0442_D5.png,"LSS MRI :  Feature of muscle spasm.-  -multiple schmorls nodules noted  -Dissicating disc materials at lower levels.  -posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and exit nerve roots more to Lt side.  -Diffuse disc bulge with small central protrusion noted at L4/L5 level compressing thecal sac and both nerve roots .",0,1
1854,local_data/mendeley/T1_0443_D3.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing thecal sac and encroaching exit neural canals.  SI JOINTS MRI:  Normal smooth articular surface of both scaro iliac joints  Normal bone marrow signals intensities.  No bony erosion or peri-osteal reaction.  Intact visualised muscles",0,1
1855,local_data/mendeley/T1_0443_D4.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing thecal sac and encroaching exit neural canals.  SI JOINTS MRI:  Normal smooth articular surface of both scaro iliac joints  Normal bone marrow signals intensities.  No bony erosion or peri-osteal reaction.  Intact visualised muscles",0,1
1856,local_data/mendeley/T1_0443_D5.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing thecal sac and encroaching exit neural canals.  SI JOINTS MRI:  Normal smooth articular surface of both scaro iliac joints  Normal bone marrow signals intensities.  No bony erosion or peri-osteal reaction.  Intact visualised muscles",0,1
1857,local_data/mendeley/T2_0443_D3.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing thecal sac and encroaching exit neural canals.  SI JOINTS MRI:  Normal smooth articular surface of both scaro iliac joints  Normal bone marrow signals intensities.  No bony erosion or peri-osteal reaction.  Intact visualised muscles",0,1
1858,local_data/mendeley/T2_0443_D4.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing thecal sac and encroaching exit neural canals.  SI JOINTS MRI:  Normal smooth articular surface of both scaro iliac joints  Normal bone marrow signals intensities.  No bony erosion or peri-osteal reaction.  Intact visualised muscles",0,1
1859,local_data/mendeley/T2_0443_D5.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing thecal sac and encroaching exit neural canals.  SI JOINTS MRI:  Normal smooth articular surface of both scaro iliac joints  Normal bone marrow signals intensities.  No bony erosion or peri-osteal reaction.  Intact visualised muscles",0,1
1860,local_data/mendeley/T1_0445_D3.png,"Nild diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and exit nerve root.  Featrure of muscle spam.",0,1
1861,local_data/mendeley/T1_0445_D4.png,"Nild diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and exit nerve root.  Featrure of muscle spam.",0,1
1862,local_data/mendeley/T1_0445_D5.png,"Nild diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and exit nerve root.  Featrure of muscle spam.",0,1
1863,local_data/mendeley/T2_0445_D3.png,"Nild diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and exit nerve root.  Featrure of muscle spam.",0,1
1864,local_data/mendeley/T2_0445_D4.png,"Nild diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and exit nerve root.  Featrure of muscle spam.",0,1
1865,local_data/mendeley/T2_0445_D5.png,"Nild diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and exit nerve root.  Featrure of muscle spam.",0,1
1866,local_data/mendeley/T1_0447_D3.png,"LSS MRI :  Evidnece of tranpedicular fixation screws noted at lower dorsal and upper lumbar spines with fractures verretbal bodies (post traumatic ).  -Diffuse disc bulges noted at L4/L5 level compressing thecal sac and both nerve roots associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis, as well as and to lesser degree at L3/L4 level.  Grade I spondylolithesis of L4 over L5 level",0,1
1867,local_data/mendeley/T1_0447_D4.png,"LSS MRI :  Evidnece of tranpedicular fixation screws noted at lower dorsal and upper lumbar spines with fractures verretbal bodies (post traumatic ).  -Diffuse disc bulges noted at L4/L5 level compressing thecal sac and both nerve roots associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis, as well as and to lesser degree at L3/L4 level.  Grade I spondylolithesis of L4 over L5 level",0,1
1868,local_data/mendeley/T1_0447_D5.png,"LSS MRI :  Evidnece of tranpedicular fixation screws noted at lower dorsal and upper lumbar spines with fractures verretbal bodies (post traumatic ).  -Diffuse disc bulges noted at L4/L5 level compressing thecal sac and both nerve roots associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis, as well as and to lesser degree at L3/L4 level.  Grade I spondylolithesis of L4 over L5 level",0,1
1869,local_data/mendeley/T2_0447_D3.png,"LSS MRI :  Evidnece of tranpedicular fixation screws noted at lower dorsal and upper lumbar spines with fractures verretbal bodies (post traumatic ).  -Diffuse disc bulges noted at L4/L5 level compressing thecal sac and both nerve roots associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis, as well as and to lesser degree at L3/L4 level.  Grade I spondylolithesis of L4 over L5 level",0,1
1870,local_data/mendeley/T2_0447_D4.png,"LSS MRI :  Evidnece of tranpedicular fixation screws noted at lower dorsal and upper lumbar spines with fractures verretbal bodies (post traumatic ).  -Diffuse disc bulges noted at L4/L5 level compressing thecal sac and both nerve roots associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis, as well as and to lesser degree at L3/L4 level.  Grade I spondylolithesis of L4 over L5 level",0,1
1871,local_data/mendeley/T2_0447_D5.png,"LSS MRI :  Evidnece of tranpedicular fixation screws noted at lower dorsal and upper lumbar spines with fractures verretbal bodies (post traumatic ).  -Diffuse disc bulges noted at L4/L5 level compressing thecal sac and both nerve roots associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis, as well as and to lesser degree at L3/L4 level.  Grade I spondylolithesis of L4 over L5 level",0,1
1872,local_data/mendeley/T1_0448_D3.png,LSS MRI :  feature of muscle spasm.  No thecal sac or nerve roots compression seen.  No significant disc protrusion or herniation noted .,1,0
1873,local_data/mendeley/T1_0448_D4.png,LSS MRI :  feature of muscle spasm.  No thecal sac or nerve roots compression seen.  No significant disc protrusion or herniation noted .,1,0
1874,local_data/mendeley/T1_0448_D5.png,LSS MRI :  feature of muscle spasm.  No thecal sac or nerve roots compression seen.  No significant disc protrusion or herniation noted .,1,0
1875,local_data/mendeley/T2_0448_D3.png,LSS MRI :  feature of muscle spasm.  No thecal sac or nerve roots compression seen.  No significant disc protrusion or herniation noted .,1,0
1876,local_data/mendeley/T2_0448_D4.png,LSS MRI :  feature of muscle spasm.  No thecal sac or nerve roots compression seen.  No significant disc protrusion or herniation noted .,1,0
1877,local_data/mendeley/T2_0448_D5.png,LSS MRI :  feature of muscle spasm.  No thecal sac or nerve roots compression seen.  No significant disc protrusion or herniation noted .,1,0
1878,local_data/mendeley/T1_0449_D3.png,Lumbosacral MRI  Mild disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,1,0
1879,local_data/mendeley/T1_0449_D4.png,Lumbosacral MRI  Mild disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,1,0
1880,local_data/mendeley/T1_0449_D5.png,Lumbosacral MRI  Mild disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,1,0
1881,local_data/mendeley/T2_0449_D3.png,Lumbosacral MRI  Mild disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,1,0
1882,local_data/mendeley/T2_0449_D4.png,Lumbosacral MRI  Mild disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,1,0
1883,local_data/mendeley/T2_0449_D5.png,Lumbosacral MRI  Mild disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,1,0
1884,local_data/mendeley/T1_0450_D3.png,"LSS MRI :  Feature of muscle spasm.  -Lt paracentral disc protrusion noted at L5/S1 level, compressing Lt thecal sac and exit nerve root.(dissicaring disc material noted )  -Wide base disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1885,local_data/mendeley/T1_0450_D4.png,"LSS MRI :  Feature of muscle spasm.  -Lt paracentral disc protrusion noted at L5/S1 level, compressing Lt thecal sac and exit nerve root.(dissicaring disc material noted )  -Wide base disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1886,local_data/mendeley/T1_0450_D5.png,"LSS MRI :  Feature of muscle spasm.  -Lt paracentral disc protrusion noted at L5/S1 level, compressing Lt thecal sac and exit nerve root.(dissicaring disc material noted )  -Wide base disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1887,local_data/mendeley/T2_0450_D3.png,"LSS MRI :  Feature of muscle spasm.  -Lt paracentral disc protrusion noted at L5/S1 level, compressing Lt thecal sac and exit nerve root.(dissicaring disc material noted )  -Wide base disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1888,local_data/mendeley/T2_0450_D4.png,"LSS MRI :  Feature of muscle spasm.  -Lt paracentral disc protrusion noted at L5/S1 level, compressing Lt thecal sac and exit nerve root.(dissicaring disc material noted )  -Wide base disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1889,local_data/mendeley/T2_0450_D5.png,"LSS MRI :  Feature of muscle spasm.  -Lt paracentral disc protrusion noted at L5/S1 level, compressing Lt thecal sac and exit nerve root.(dissicaring disc material noted )  -Wide base disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1
1890,local_data/mendeley/T1_0451_D3.png,"Lumbosacral MRI  Central and left paracentral disc protrusion noted at L4/L5 level , largely compressing the thecal sac and encroaching upon exiting neural canals.",0,1
1891,local_data/mendeley/T1_0451_D4.png,"Lumbosacral MRI  Central and left paracentral disc protrusion noted at L4/L5 level , largely compressing the thecal sac and encroaching upon exiting neural canals.",0,1
1892,local_data/mendeley/T1_0451_D5.png,"Lumbosacral MRI  Central and left paracentral disc protrusion noted at L4/L5 level , largely compressing the thecal sac and encroaching upon exiting neural canals.",0,1
1893,local_data/mendeley/T2_0451_D3.png,"Lumbosacral MRI  Central and left paracentral disc protrusion noted at L4/L5 level , largely compressing the thecal sac and encroaching upon exiting neural canals.",0,1
1894,local_data/mendeley/T2_0451_D4.png,"Lumbosacral MRI  Central and left paracentral disc protrusion noted at L4/L5 level , largely compressing the thecal sac and encroaching upon exiting neural canals.",0,1
1895,local_data/mendeley/T2_0451_D5.png,"Lumbosacral MRI  Central and left paracentral disc protrusion noted at L4/L5 level , largely compressing the thecal sac and encroaching upon exiting neural canals.",0,1
1896,local_data/mendeley/T1_0452_D3.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material noted at L5/S1 level.  wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and encroaching upon exit neural canals",0,1
1897,local_data/mendeley/T1_0452_D4.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material noted at L5/S1 level.  wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and encroaching upon exit neural canals",0,1
1898,local_data/mendeley/T1_0452_D5.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material noted at L5/S1 level.  wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and encroaching upon exit neural canals",0,1
1899,local_data/mendeley/T2_0452_D3.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material noted at L5/S1 level.  wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and encroaching upon exit neural canals",0,1
1900,local_data/mendeley/T2_0452_D4.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material noted at L5/S1 level.  wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and encroaching upon exit neural canals",0,1
1901,local_data/mendeley/T2_0452_D5.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material noted at L5/S1 level.  wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and encroaching upon exit neural canals",0,1
1902,local_data/mendeley/T1_0454_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1903,local_data/mendeley/T1_0454_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1904,local_data/mendeley/T1_0454_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1905,local_data/mendeley/T2_0454_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1906,local_data/mendeley/T2_0454_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1907,local_data/mendeley/T2_0454_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
1908,local_data/mendeley/T1_0455_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Multiple schmorls nodules.  -Diffuse disc bulge noted at L4/L5 level, extending to lateral recess , more to Rt sided ,compressing the thecal sac and both nerve roots .  -Diffuse disc bulges with small posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1909,local_data/mendeley/T1_0455_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Multiple schmorls nodules.  -Diffuse disc bulge noted at L4/L5 level, extending to lateral recess , more to Rt sided ,compressing the thecal sac and both nerve roots .  -Diffuse disc bulges with small posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1910,local_data/mendeley/T1_0455_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Multiple schmorls nodules.  -Diffuse disc bulge noted at L4/L5 level, extending to lateral recess , more to Rt sided ,compressing the thecal sac and both nerve roots .  -Diffuse disc bulges with small posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1911,local_data/mendeley/T2_0455_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Multiple schmorls nodules.  -Diffuse disc bulge noted at L4/L5 level, extending to lateral recess , more to Rt sided ,compressing the thecal sac and both nerve roots .  -Diffuse disc bulges with small posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1912,local_data/mendeley/T2_0455_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Multiple schmorls nodules.  -Diffuse disc bulge noted at L4/L5 level, extending to lateral recess , more to Rt sided ,compressing the thecal sac and both nerve roots .  -Diffuse disc bulges with small posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1913,local_data/mendeley/T2_0455_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Multiple schmorls nodules.  -Diffuse disc bulge noted at L4/L5 level, extending to lateral recess , more to Rt sided ,compressing the thecal sac and both nerve roots .  -Diffuse disc bulges with small posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1
1914,local_data/mendeley/T1_0456_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  -Diffuse disc bulges with small Rt paracentral disc protrusion noted at L5/S1 level , compressing thecal sac and exit nerve roots",0,1
1915,local_data/mendeley/T1_0456_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  -Diffuse disc bulges with small Rt paracentral disc protrusion noted at L5/S1 level , compressing thecal sac and exit nerve roots",0,1
1916,local_data/mendeley/T1_0456_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  -Diffuse disc bulges with small Rt paracentral disc protrusion noted at L5/S1 level , compressing thecal sac and exit nerve roots",0,1
1917,local_data/mendeley/T2_0456_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  -Diffuse disc bulges with small Rt paracentral disc protrusion noted at L5/S1 level , compressing thecal sac and exit nerve roots",0,1
1918,local_data/mendeley/T2_0456_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  -Diffuse disc bulges with small Rt paracentral disc protrusion noted at L5/S1 level , compressing thecal sac and exit nerve roots",0,1
1919,local_data/mendeley/T2_0456_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  -Diffuse disc bulges with small Rt paracentral disc protrusion noted at L5/S1 level , compressing thecal sac and exit nerve roots",0,1
1920,local_data/mendeley/T1_0458_D3.png,"LSS MRI :  Feature of muscle spasm.  central slightly to Rt disc protrusion noted at L5/S1 , compressing Lt thecal sac and exit nerve root.",0,1
1921,local_data/mendeley/T1_0458_D4.png,"LSS MRI :  Feature of muscle spasm.  central slightly to Rt disc protrusion noted at L5/S1 , compressing Lt thecal sac and exit nerve root.",0,1
1922,local_data/mendeley/T1_0458_D5.png,"LSS MRI :  Feature of muscle spasm.  central slightly to Rt disc protrusion noted at L5/S1 , compressing Lt thecal sac and exit nerve root.",0,1
1923,local_data/mendeley/T2_0458_D3.png,"LSS MRI :  Feature of muscle spasm.  central slightly to Rt disc protrusion noted at L5/S1 , compressing Lt thecal sac and exit nerve root.",0,1
1924,local_data/mendeley/T2_0458_D4.png,"LSS MRI :  Feature of muscle spasm.  central slightly to Rt disc protrusion noted at L5/S1 , compressing Lt thecal sac and exit nerve root.",0,1
1925,local_data/mendeley/T2_0458_D5.png,"LSS MRI :  Feature of muscle spasm.  central slightly to Rt disc protrusion noted at L5/S1 , compressing Lt thecal sac and exit nerve root.",0,1
1926,local_data/mendeley/T1_0459_D3.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level .  No significant thecal sac or nerve root compression notred,1,0
1927,local_data/mendeley/T1_0459_D4.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level .  No significant thecal sac or nerve root compression notred,1,0
1928,local_data/mendeley/T1_0459_D5.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level .  No significant thecal sac or nerve root compression notred,1,0
1929,local_data/mendeley/T2_0459_D3.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level .  No significant thecal sac or nerve root compression notred,1,0
1930,local_data/mendeley/T2_0459_D4.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level .  No significant thecal sac or nerve root compression notred,1,0
1931,local_data/mendeley/T2_0459_D5.png,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level .  No significant thecal sac or nerve root compression notred,1,0
1932,local_data/mendeley/T1_0460_D3.png,LSS MRI  No evidence of disc herniation noted.  No significant thecal sac or nerve root compression noted.,1,0
1933,local_data/mendeley/T1_0460_D4.png,LSS MRI  No evidence of disc herniation noted.  No significant thecal sac or nerve root compression noted.,1,0
1934,local_data/mendeley/T1_0460_D5.png,LSS MRI  No evidence of disc herniation noted.  No significant thecal sac or nerve root compression noted.,1,0
1935,local_data/mendeley/T2_0460_D3.png,LSS MRI  No evidence of disc herniation noted.  No significant thecal sac or nerve root compression noted.,1,0
1936,local_data/mendeley/T2_0460_D4.png,LSS MRI  No evidence of disc herniation noted.  No significant thecal sac or nerve root compression noted.,1,0
1937,local_data/mendeley/T2_0460_D5.png,LSS MRI  No evidence of disc herniation noted.  No significant thecal sac or nerve root compression noted.,1,0
1938,local_data/mendeley/T1_0461_D3.png,"LSS MRI  No evidence of disc herniation.  no thecal sac or nerve root compresion noted.  small hypointense T1 and T2 lesion noted, probably schmorls nodule",1,0
1939,local_data/mendeley/T1_0461_D4.png,"LSS MRI  No evidence of disc herniation.  no thecal sac or nerve root compresion noted.  small hypointense T1 and T2 lesion noted, probably schmorls nodule",1,0
1940,local_data/mendeley/T1_0461_D5.png,"LSS MRI  No evidence of disc herniation.  no thecal sac or nerve root compresion noted.  small hypointense T1 and T2 lesion noted, probably schmorls nodule",1,0
1941,local_data/mendeley/T2_0461_D3.png,"LSS MRI  No evidence of disc herniation.  no thecal sac or nerve root compresion noted.  small hypointense T1 and T2 lesion noted, probably schmorls nodule",1,0
1942,local_data/mendeley/T2_0461_D4.png,"LSS MRI  No evidence of disc herniation.  no thecal sac or nerve root compresion noted.  small hypointense T1 and T2 lesion noted, probably schmorls nodule",1,0
1943,local_data/mendeley/T2_0461_D5.png,"LSS MRI  No evidence of disc herniation.  no thecal sac or nerve root compresion noted.  small hypointense T1 and T2 lesion noted, probably schmorls nodule",1,0
1944,local_data/mendeley/T1_0462_D3.png,"LSS MRI  Mild Lt paracentral disc protrusion noted at L1-L2 level, compressing the thecal sac.  no evidence of disc herniation at other levels.",1,0
1945,local_data/mendeley/T1_0462_D4.png,"LSS MRI  Mild Lt paracentral disc protrusion noted at L1-L2 level, compressing the thecal sac.  no evidence of disc herniation at other levels.",1,0
1946,local_data/mendeley/T1_0462_D5.png,"LSS MRI  Mild Lt paracentral disc protrusion noted at L1-L2 level, compressing the thecal sac.  no evidence of disc herniation at other levels.",1,0
1947,local_data/mendeley/T2_0462_D3.png,"LSS MRI  Mild Lt paracentral disc protrusion noted at L1-L2 level, compressing the thecal sac.  no evidence of disc herniation at other levels.",1,0
1948,local_data/mendeley/T2_0462_D4.png,"LSS MRI  Mild Lt paracentral disc protrusion noted at L1-L2 level, compressing the thecal sac.  no evidence of disc herniation at other levels.",1,0
1949,local_data/mendeley/T2_0462_D5.png,"LSS MRI  Mild Lt paracentral disc protrusion noted at L1-L2 level, compressing the thecal sac.  no evidence of disc herniation at other levels.",1,0
1950,local_data/mendeley/T1_0466_D3.png,LSS MRI:  feature of muscle spasm.  No significant thecal sac or nerve roots.  NB:relatively Decreased bone marrow signals noted ? correalte with bone isotope scan,1,0
1951,local_data/mendeley/T1_0466_D4.png,LSS MRI:  feature of muscle spasm.  No significant thecal sac or nerve roots.  NB:relatively Decreased bone marrow signals noted ? correalte with bone isotope scan,1,0
1952,local_data/mendeley/T1_0466_D5.png,LSS MRI:  feature of muscle spasm.  No significant thecal sac or nerve roots.  NB:relatively Decreased bone marrow signals noted ? correalte with bone isotope scan,1,0
1953,local_data/mendeley/T2_0466_D3.png,LSS MRI:  feature of muscle spasm.  No significant thecal sac or nerve roots.  NB:relatively Decreased bone marrow signals noted ? correalte with bone isotope scan,1,0
1954,local_data/mendeley/T2_0466_D4.png,LSS MRI:  feature of muscle spasm.  No significant thecal sac or nerve roots.  NB:relatively Decreased bone marrow signals noted ? correalte with bone isotope scan,1,0
1955,local_data/mendeley/T2_0466_D5.png,LSS MRI:  feature of muscle spasm.  No significant thecal sac or nerve roots.  NB:relatively Decreased bone marrow signals noted ? correalte with bone isotope scan,1,0
1956,local_data/mendeley/T1_0467_D3.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL,0,1
1957,local_data/mendeley/T1_0467_D4.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL,0,1
1958,local_data/mendeley/T1_0467_D5.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL,0,1
1959,local_data/mendeley/T2_0467_D3.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL,0,1
1960,local_data/mendeley/T2_0467_D4.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL,0,1
1961,local_data/mendeley/T2_0467_D5.png,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL,0,1
1962,local_data/mendeley/T1_0468_D3.png,"Cervical MRI :  -feature of muscle spasm.  -dehydrated discal materials.  -diffuse disc bulges noted at C5/C6 & C6/C7 levels , compressing anterior CSF spaces and encroaching upon both exit nerve canals.  LSS MRI :  -Diffuse disc bulge noted at L4/L5 level compressing the thecal sac and both exit nerve roots.  NB:the sacral canal is occupied by a central extradural and non divided cyst with fine wall and without neural element or herniation of meninges.The cyst is very expansive but doses not transgress margins of the sacrum  No connection is visualized between the cyst and the thecal sac.  There is no other significant abnormality.  appearances mostly represent occult intrasacral meningocele (OIM) or sacral meningeal cyst ..",0,1
1963,local_data/mendeley/T1_0468_D4.png,"Cervical MRI :  -feature of muscle spasm.  -dehydrated discal materials.  -diffuse disc bulges noted at C5/C6 & C6/C7 levels , compressing anterior CSF spaces and encroaching upon both exit nerve canals.  LSS MRI :  -Diffuse disc bulge noted at L4/L5 level compressing the thecal sac and both exit nerve roots.  NB:the sacral canal is occupied by a central extradural and non divided cyst with fine wall and without neural element or herniation of meninges.The cyst is very expansive but doses not transgress margins of the sacrum  No connection is visualized between the cyst and the thecal sac.  There is no other significant abnormality.  appearances mostly represent occult intrasacral meningocele (OIM) or sacral meningeal cyst ..",0,1
1964,local_data/mendeley/T1_0468_D5.png,"Cervical MRI :  -feature of muscle spasm.  -dehydrated discal materials.  -diffuse disc bulges noted at C5/C6 & C6/C7 levels , compressing anterior CSF spaces and encroaching upon both exit nerve canals.  LSS MRI :  -Diffuse disc bulge noted at L4/L5 level compressing the thecal sac and both exit nerve roots.  NB:the sacral canal is occupied by a central extradural and non divided cyst with fine wall and without neural element or herniation of meninges.The cyst is very expansive but doses not transgress margins of the sacrum  No connection is visualized between the cyst and the thecal sac.  There is no other significant abnormality.  appearances mostly represent occult intrasacral meningocele (OIM) or sacral meningeal cyst ..",0,1
1965,local_data/mendeley/T2_0468_D3.png,"Cervical MRI :  -feature of muscle spasm.  -dehydrated discal materials.  -diffuse disc bulges noted at C5/C6 & C6/C7 levels , compressing anterior CSF spaces and encroaching upon both exit nerve canals.  LSS MRI :  -Diffuse disc bulge noted at L4/L5 level compressing the thecal sac and both exit nerve roots.  NB:the sacral canal is occupied by a central extradural and non divided cyst with fine wall and without neural element or herniation of meninges.The cyst is very expansive but doses not transgress margins of the sacrum  No connection is visualized between the cyst and the thecal sac.  There is no other significant abnormality.  appearances mostly represent occult intrasacral meningocele (OIM) or sacral meningeal cyst ..",0,1
1966,local_data/mendeley/T2_0468_D4.png,"Cervical MRI :  -feature of muscle spasm.  -dehydrated discal materials.  -diffuse disc bulges noted at C5/C6 & C6/C7 levels , compressing anterior CSF spaces and encroaching upon both exit nerve canals.  LSS MRI :  -Diffuse disc bulge noted at L4/L5 level compressing the thecal sac and both exit nerve roots.  NB:the sacral canal is occupied by a central extradural and non divided cyst with fine wall and without neural element or herniation of meninges.The cyst is very expansive but doses not transgress margins of the sacrum  No connection is visualized between the cyst and the thecal sac.  There is no other significant abnormality.  appearances mostly represent occult intrasacral meningocele (OIM) or sacral meningeal cyst ..",0,1
1967,local_data/mendeley/T2_0468_D5.png,"Cervical MRI :  -feature of muscle spasm.  -dehydrated discal materials.  -diffuse disc bulges noted at C5/C6 & C6/C7 levels , compressing anterior CSF spaces and encroaching upon both exit nerve canals.  LSS MRI :  -Diffuse disc bulge noted at L4/L5 level compressing the thecal sac and both exit nerve roots.  NB:the sacral canal is occupied by a central extradural and non divided cyst with fine wall and without neural element or herniation of meninges.The cyst is very expansive but doses not transgress margins of the sacrum  No connection is visualized between the cyst and the thecal sac.  There is no other significant abnormality.  appearances mostly represent occult intrasacral meningocele (OIM) or sacral meningeal cyst ..",0,1
1968,local_data/mendeley/T1_0470_D3.png,LSS MRI  No evidence of disc herniation.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,1,0
1969,local_data/mendeley/T1_0470_D4.png,LSS MRI  No evidence of disc herniation.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,1,0
1970,local_data/mendeley/T1_0470_D5.png,LSS MRI  No evidence of disc herniation.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,1,0
1971,local_data/mendeley/T2_0470_D3.png,LSS MRI  No evidence of disc herniation.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,1,0
1972,local_data/mendeley/T2_0470_D4.png,LSS MRI  No evidence of disc herniation.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,1,0
1973,local_data/mendeley/T2_0470_D5.png,LSS MRI  No evidence of disc herniation.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,1,0
1974,local_data/mendeley/T1_0471_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1975,local_data/mendeley/T1_0471_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1976,local_data/mendeley/T1_0471_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1977,local_data/mendeley/T2_0471_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1978,local_data/mendeley/T2_0471_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1979,local_data/mendeley/T2_0471_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
1980,local_data/mendeley/T1_0473_D3.png,"LSS MRI  Small central disc protrusion noted at L5-S1 level, compressing the thecal sac.  adequate spinal canal.",0,1
1981,local_data/mendeley/T1_0473_D4.png,"LSS MRI  Small central disc protrusion noted at L5-S1 level, compressing the thecal sac.  adequate spinal canal.",0,1
1982,local_data/mendeley/T1_0473_D5.png,"LSS MRI  Small central disc protrusion noted at L5-S1 level, compressing the thecal sac.  adequate spinal canal.",0,1
1983,local_data/mendeley/T2_0473_D3.png,"LSS MRI  Small central disc protrusion noted at L5-S1 level, compressing the thecal sac.  adequate spinal canal.",0,1
1984,local_data/mendeley/T2_0473_D4.png,"LSS MRI  Small central disc protrusion noted at L5-S1 level, compressing the thecal sac.  adequate spinal canal.",0,1
1985,local_data/mendeley/T2_0473_D5.png,"LSS MRI  Small central disc protrusion noted at L5-S1 level, compressing the thecal sac.  adequate spinal canal.",0,1
1986,local_data/mendeley/T1_0475_D3.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1987,local_data/mendeley/T1_0475_D4.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1988,local_data/mendeley/T1_0475_D5.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1989,local_data/mendeley/T2_0475_D3.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1990,local_data/mendeley/T2_0475_D4.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1991,local_data/mendeley/T2_0475_D5.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1992,local_data/mendeley/T1_0476_D3.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1993,local_data/mendeley/T1_0476_D4.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1994,local_data/mendeley/T1_0476_D5.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1995,local_data/mendeley/T2_0476_D3.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1996,local_data/mendeley/T2_0476_D4.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1997,local_data/mendeley/T2_0476_D5.png,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1
1998,local_data/mendeley/T1_0477_D3.png,"L5-S1: Lt paracentral disc protrusion, compressing the thecal sac. L4-L5: Mild disc bulge with annular tear.",0,1
1999,local_data/mendeley/T1_0477_D4.png,"L5-S1: Lt paracentral disc protrusion, compressing the thecal sac. L4-L5: Mild disc bulge with annular tear.",0,1
2000,local_data/mendeley/T1_0477_D5.png,"L5-S1: Lt paracentral disc protrusion, compressing the thecal sac. L4-L5: Mild disc bulge with annular tear.",0,1
2001,local_data/mendeley/T2_0477_D3.png,"L5-S1: Lt paracentral disc protrusion, compressing the thecal sac. L4-L5: Mild disc bulge with annular tear.",0,1
2002,local_data/mendeley/T2_0477_D4.png,"L5-S1: Lt paracentral disc protrusion, compressing the thecal sac. L4-L5: Mild disc bulge with annular tear.",0,1
2003,local_data/mendeley/T2_0477_D5.png,"L5-S1: Lt paracentral disc protrusion, compressing the thecal sac. L4-L5: Mild disc bulge with annular tear.",0,1
2004,local_data/mendeley/T1_0479_D3.png,"LSS MRI :  Feature of muscle spasmز  Relatively generalised decreased bone marrow signals ? correalte with bone isotope scan..  diffuse disc bulge, noted at L4/L5 level, compressing the thecal sac and exit neural canals",0,1
2005,local_data/mendeley/T1_0479_D4.png,"LSS MRI :  Feature of muscle spasmز  Relatively generalised decreased bone marrow signals ? correalte with bone isotope scan..  diffuse disc bulge, noted at L4/L5 level, compressing the thecal sac and exit neural canals",0,1
2006,local_data/mendeley/T1_0479_D5.png,"LSS MRI :  Feature of muscle spasmز  Relatively generalised decreased bone marrow signals ? correalte with bone isotope scan..  diffuse disc bulge, noted at L4/L5 level, compressing the thecal sac and exit neural canals",0,1
2007,local_data/mendeley/T2_0479_D3.png,"LSS MRI :  Feature of muscle spasmز  Relatively generalised decreased bone marrow signals ? correalte with bone isotope scan..  diffuse disc bulge, noted at L4/L5 level, compressing the thecal sac and exit neural canals",0,1
2008,local_data/mendeley/T2_0479_D4.png,"LSS MRI :  Feature of muscle spasmز  Relatively generalised decreased bone marrow signals ? correalte with bone isotope scan..  diffuse disc bulge, noted at L4/L5 level, compressing the thecal sac and exit neural canals",0,1
2009,local_data/mendeley/T2_0479_D5.png,"LSS MRI :  Feature of muscle spasmز  Relatively generalised decreased bone marrow signals ? correalte with bone isotope scan..  diffuse disc bulge, noted at L4/L5 level, compressing the thecal sac and exit neural canals",0,1
2010,local_data/mendeley/T1_0482_D3.png,"LSS MRI :  -There is anterior wedging fracture of L5 vertebral body noted with bone marrow edema involving the rest of vertebral body extending to pedicles with evidence of paravertebral soft tissue componant with no posterior bulges ??fracture , however bone isotope scan is advised to rule out secandary depostis.  -Multlevel diffuse disc bulges noted , at L2/L3 , L3/L4 & L5/S1 levels ,compresiing the thecal sac and both exit nerve roots.  grade I spodylolisthesis of L5 over S1 level",0,1
2011,local_data/mendeley/T1_0482_D4.png,"LSS MRI :  -There is anterior wedging fracture of L5 vertebral body noted with bone marrow edema involving the rest of vertebral body extending to pedicles with evidence of paravertebral soft tissue componant with no posterior bulges ??fracture , however bone isotope scan is advised to rule out secandary depostis.  -Multlevel diffuse disc bulges noted , at L2/L3 , L3/L4 & L5/S1 levels ,compresiing the thecal sac and both exit nerve roots.  grade I spodylolisthesis of L5 over S1 level",0,1
2012,local_data/mendeley/T1_0482_D5.png,"LSS MRI :  -There is anterior wedging fracture of L5 vertebral body noted with bone marrow edema involving the rest of vertebral body extending to pedicles with evidence of paravertebral soft tissue componant with no posterior bulges ??fracture , however bone isotope scan is advised to rule out secandary depostis.  -Multlevel diffuse disc bulges noted , at L2/L3 , L3/L4 & L5/S1 levels ,compresiing the thecal sac and both exit nerve roots.  grade I spodylolisthesis of L5 over S1 level",0,1
2013,local_data/mendeley/T2_0482_D3.png,"LSS MRI :  -There is anterior wedging fracture of L5 vertebral body noted with bone marrow edema involving the rest of vertebral body extending to pedicles with evidence of paravertebral soft tissue componant with no posterior bulges ??fracture , however bone isotope scan is advised to rule out secandary depostis.  -Multlevel diffuse disc bulges noted , at L2/L3 , L3/L4 & L5/S1 levels ,compresiing the thecal sac and both exit nerve roots.  grade I spodylolisthesis of L5 over S1 level",0,1
2014,local_data/mendeley/T2_0482_D4.png,"LSS MRI :  -There is anterior wedging fracture of L5 vertebral body noted with bone marrow edema involving the rest of vertebral body extending to pedicles with evidence of paravertebral soft tissue componant with no posterior bulges ??fracture , however bone isotope scan is advised to rule out secandary depostis.  -Multlevel diffuse disc bulges noted , at L2/L3 , L3/L4 & L5/S1 levels ,compresiing the thecal sac and both exit nerve roots.  grade I spodylolisthesis of L5 over S1 level",0,1
2015,local_data/mendeley/T2_0482_D5.png,"LSS MRI :  -There is anterior wedging fracture of L5 vertebral body noted with bone marrow edema involving the rest of vertebral body extending to pedicles with evidence of paravertebral soft tissue componant with no posterior bulges ??fracture , however bone isotope scan is advised to rule out secandary depostis.  -Multlevel diffuse disc bulges noted , at L2/L3 , L3/L4 & L5/S1 levels ,compresiing the thecal sac and both exit nerve roots.  grade I spodylolisthesis of L5 over S1 level",0,1
2016,local_data/mendeley/T1_0484_D3.png,"HIP JOINT & SIJ MRI :  Feature of Lt sided scaroilitis noted(infective = septic sacroiliatis ,TB, Brucellosis ...etc), correalte with lab results and needs clinical correaltion .  Preserved Both femoral head noted  LSS MRI :  Feature of muscle spasm .  No significant thecal sac or nerve roots compression seen .  Evidence of left sided gluteal signal alteration noted (old insult )",1,0
2017,local_data/mendeley/T1_0484_D4.png,"HIP JOINT & SIJ MRI :  Feature of Lt sided scaroilitis noted(infective = septic sacroiliatis ,TB, Brucellosis ...etc), correalte with lab results and needs clinical correaltion .  Preserved Both femoral head noted  LSS MRI :  Feature of muscle spasm .  No significant thecal sac or nerve roots compression seen .  Evidence of left sided gluteal signal alteration noted (old insult )",1,0
2018,local_data/mendeley/T1_0484_D5.png,"HIP JOINT & SIJ MRI :  Feature of Lt sided scaroilitis noted(infective = septic sacroiliatis ,TB, Brucellosis ...etc), correalte with lab results and needs clinical correaltion .  Preserved Both femoral head noted  LSS MRI :  Feature of muscle spasm .  No significant thecal sac or nerve roots compression seen .  Evidence of left sided gluteal signal alteration noted (old insult )",1,0
2019,local_data/mendeley/T2_0484_D3.png,"HIP JOINT & SIJ MRI :  Feature of Lt sided scaroilitis noted(infective = septic sacroiliatis ,TB, Brucellosis ...etc), correalte with lab results and needs clinical correaltion .  Preserved Both femoral head noted  LSS MRI :  Feature of muscle spasm .  No significant thecal sac or nerve roots compression seen .  Evidence of left sided gluteal signal alteration noted (old insult )",1,0
2020,local_data/mendeley/T2_0484_D4.png,"HIP JOINT & SIJ MRI :  Feature of Lt sided scaroilitis noted(infective = septic sacroiliatis ,TB, Brucellosis ...etc), correalte with lab results and needs clinical correaltion .  Preserved Both femoral head noted  LSS MRI :  Feature of muscle spasm .  No significant thecal sac or nerve roots compression seen .  Evidence of left sided gluteal signal alteration noted (old insult )",1,0
2021,local_data/mendeley/T2_0484_D5.png,"HIP JOINT & SIJ MRI :  Feature of Lt sided scaroilitis noted(infective = septic sacroiliatis ,TB, Brucellosis ...etc), correalte with lab results and needs clinical correaltion .  Preserved Both femoral head noted  LSS MRI :  Feature of muscle spasm .  No significant thecal sac or nerve roots compression seen .  Evidence of left sided gluteal signal alteration noted (old insult )",1,0
2022,local_data/mendeley/T1_0487_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2023,local_data/mendeley/T1_0487_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2024,local_data/mendeley/T1_0487_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2025,local_data/mendeley/T2_0487_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2026,local_data/mendeley/T2_0487_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2027,local_data/mendeley/T2_0487_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2028,local_data/mendeley/T1_0488_D3.png,"LSS MRI  diffuse disc bulge with annular tear noted at L4-L5 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.",1,0
2029,local_data/mendeley/T1_0488_D4.png,"LSS MRI  diffuse disc bulge with annular tear noted at L4-L5 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.",1,0
2030,local_data/mendeley/T1_0488_D5.png,"LSS MRI  diffuse disc bulge with annular tear noted at L4-L5 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.",1,0
2031,local_data/mendeley/T2_0488_D3.png,"LSS MRI  diffuse disc bulge with annular tear noted at L4-L5 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.",1,0
2032,local_data/mendeley/T2_0488_D4.png,"LSS MRI  diffuse disc bulge with annular tear noted at L4-L5 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.",1,0
2033,local_data/mendeley/T2_0488_D5.png,"LSS MRI  diffuse disc bulge with annular tear noted at L4-L5 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.",1,0
2034,local_data/mendeley/T1_0493_D3.png,Lumbosacral MRI  Diffuse disc bulge noted at L4-L5 level  Mildly compressing the thecal sac and the exit neural canals  The spinal canal appears adequate,0,1
2035,local_data/mendeley/T1_0493_D4.png,Lumbosacral MRI  Diffuse disc bulge noted at L4-L5 level  Mildly compressing the thecal sac and the exit neural canals  The spinal canal appears adequate,0,1
2036,local_data/mendeley/T1_0493_D5.png,Lumbosacral MRI  Diffuse disc bulge noted at L4-L5 level  Mildly compressing the thecal sac and the exit neural canals  The spinal canal appears adequate,0,1
2037,local_data/mendeley/T2_0493_D3.png,Lumbosacral MRI  Diffuse disc bulge noted at L4-L5 level  Mildly compressing the thecal sac and the exit neural canals  The spinal canal appears adequate,0,1
2038,local_data/mendeley/T2_0493_D4.png,Lumbosacral MRI  Diffuse disc bulge noted at L4-L5 level  Mildly compressing the thecal sac and the exit neural canals  The spinal canal appears adequate,0,1
2039,local_data/mendeley/T2_0493_D5.png,Lumbosacral MRI  Diffuse disc bulge noted at L4-L5 level  Mildly compressing the thecal sac and the exit neural canals  The spinal canal appears adequate,0,1
2040,local_data/mendeley/T1_0494_D3.png,"Multilevel diffuse disc bulges compressing the thecal sac and encroaching exit neural canals, largest at L2-L3 level.",0,1
2041,local_data/mendeley/T1_0494_D4.png,"Multilevel diffuse disc bulges compressing the thecal sac and encroaching exit neural canals, largest at L2-L3 level.",0,1
2042,local_data/mendeley/T1_0494_D5.png,"Multilevel diffuse disc bulges compressing the thecal sac and encroaching exit neural canals, largest at L2-L3 level.",0,1
2043,local_data/mendeley/T2_0494_D3.png,"Multilevel diffuse disc bulges compressing the thecal sac and encroaching exit neural canals, largest at L2-L3 level.",0,1
2044,local_data/mendeley/T2_0494_D4.png,"Multilevel diffuse disc bulges compressing the thecal sac and encroaching exit neural canals, largest at L2-L3 level.",0,1
2045,local_data/mendeley/T2_0494_D5.png,"Multilevel diffuse disc bulges compressing the thecal sac and encroaching exit neural canals, largest at L2-L3 level.",0,1
2046,local_data/mendeley/T1_0496_D3.png,"Lumbosacral MRI  Multilevel diffuse disc bulges noted at L5-S1, L4-L5 and L3-L4 levels  Compressing the thecal sac and exiting neural canals  the Spinal canal appears narrowed at the L4-L5 leve, By disc bulge and the facet joint hypertrophy  Multilevel endplate degeneration noted",0,1
2047,local_data/mendeley/T1_0496_D4.png,"Lumbosacral MRI  Multilevel diffuse disc bulges noted at L5-S1, L4-L5 and L3-L4 levels  Compressing the thecal sac and exiting neural canals  the Spinal canal appears narrowed at the L4-L5 leve, By disc bulge and the facet joint hypertrophy  Multilevel endplate degeneration noted",0,1
2048,local_data/mendeley/T1_0496_D5.png,"Lumbosacral MRI  Multilevel diffuse disc bulges noted at L5-S1, L4-L5 and L3-L4 levels  Compressing the thecal sac and exiting neural canals  the Spinal canal appears narrowed at the L4-L5 leve, By disc bulge and the facet joint hypertrophy  Multilevel endplate degeneration noted",0,1
2049,local_data/mendeley/T2_0496_D3.png,"Lumbosacral MRI  Multilevel diffuse disc bulges noted at L5-S1, L4-L5 and L3-L4 levels  Compressing the thecal sac and exiting neural canals  the Spinal canal appears narrowed at the L4-L5 leve, By disc bulge and the facet joint hypertrophy  Multilevel endplate degeneration noted",0,1
2050,local_data/mendeley/T2_0496_D4.png,"Lumbosacral MRI  Multilevel diffuse disc bulges noted at L5-S1, L4-L5 and L3-L4 levels  Compressing the thecal sac and exiting neural canals  the Spinal canal appears narrowed at the L4-L5 leve, By disc bulge and the facet joint hypertrophy  Multilevel endplate degeneration noted",0,1
2051,local_data/mendeley/T2_0496_D5.png,"Lumbosacral MRI  Multilevel diffuse disc bulges noted at L5-S1, L4-L5 and L3-L4 levels  Compressing the thecal sac and exiting neural canals  the Spinal canal appears narrowed at the L4-L5 leve, By disc bulge and the facet joint hypertrophy  Multilevel endplate degeneration noted",0,1
2052,local_data/mendeley/T1_0499_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2053,local_data/mendeley/T1_0499_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2054,local_data/mendeley/T1_0499_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2055,local_data/mendeley/T2_0499_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2056,local_data/mendeley/T2_0499_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2057,local_data/mendeley/T2_0499_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2058,local_data/mendeley/T1_0500_D3.png,LSS MRI  Mild disc bulge noted at L2-L3 level abutting the thecal sac.  features of muscle spasm.,0,1
2059,local_data/mendeley/T1_0500_D4.png,LSS MRI  Mild disc bulge noted at L2-L3 level abutting the thecal sac.  features of muscle spasm.,0,1
2060,local_data/mendeley/T1_0500_D5.png,LSS MRI  Mild disc bulge noted at L2-L3 level abutting the thecal sac.  features of muscle spasm.,0,1
2061,local_data/mendeley/T2_0500_D3.png,LSS MRI  Mild disc bulge noted at L2-L3 level abutting the thecal sac.  features of muscle spasm.,0,1
2062,local_data/mendeley/T2_0500_D4.png,LSS MRI  Mild disc bulge noted at L2-L3 level abutting the thecal sac.  features of muscle spasm.,0,1
2063,local_data/mendeley/T2_0500_D5.png,LSS MRI  Mild disc bulge noted at L2-L3 level abutting the thecal sac.  features of muscle spasm.,0,1
2064,local_data/mendeley/T1_0501_D3.png,"LSS MRI : feature of degenerative chaanges . Mutlilevel diffuse disc bulges notceably at L3/L4 &L4/L5 levels, compressing the thecal sac and encroaching exit neural canal associated with ligamtum flavum hyperatrophy , all leading to secandary spinal canal stenosis.",0,1
2065,local_data/mendeley/T1_0501_D4.png,"LSS MRI : feature of degenerative chaanges . Mutlilevel diffuse disc bulges notceably at L3/L4 &L4/L5 levels, compressing the thecal sac and encroaching exit neural canal associated with ligamtum flavum hyperatrophy , all leading to secandary spinal canal stenosis.",0,1
2066,local_data/mendeley/T1_0501_D5.png,"LSS MRI : feature of degenerative chaanges . Mutlilevel diffuse disc bulges notceably at L3/L4 &L4/L5 levels, compressing the thecal sac and encroaching exit neural canal associated with ligamtum flavum hyperatrophy , all leading to secandary spinal canal stenosis.",0,1
2067,local_data/mendeley/T2_0501_D3.png,"LSS MRI : feature of degenerative chaanges . Mutlilevel diffuse disc bulges notceably at L3/L4 &L4/L5 levels, compressing the thecal sac and encroaching exit neural canal associated with ligamtum flavum hyperatrophy , all leading to secandary spinal canal stenosis.",0,1
2068,local_data/mendeley/T2_0501_D4.png,"LSS MRI : feature of degenerative chaanges . Mutlilevel diffuse disc bulges notceably at L3/L4 &L4/L5 levels, compressing the thecal sac and encroaching exit neural canal associated with ligamtum flavum hyperatrophy , all leading to secandary spinal canal stenosis.",0,1
2069,local_data/mendeley/T2_0501_D5.png,"LSS MRI : feature of degenerative chaanges . Mutlilevel diffuse disc bulges notceably at L3/L4 &L4/L5 levels, compressing the thecal sac and encroaching exit neural canal associated with ligamtum flavum hyperatrophy , all leading to secandary spinal canal stenosis.",0,1
2070,local_data/mendeley/T1_0504_D3.png,"LSS MRI : Feature of muscle spasm. Central and Lt paracentral disc prortusion noted at L5/S1 level, compressing the thecal sac and Lt (S1) exit nerve root.",0,1
2071,local_data/mendeley/T1_0504_D4.png,"LSS MRI : Feature of muscle spasm. Central and Lt paracentral disc prortusion noted at L5/S1 level, compressing the thecal sac and Lt (S1) exit nerve root.",0,1
2072,local_data/mendeley/T1_0504_D5.png,"LSS MRI : Feature of muscle spasm. Central and Lt paracentral disc prortusion noted at L5/S1 level, compressing the thecal sac and Lt (S1) exit nerve root.",0,1
2073,local_data/mendeley/T2_0504_D3.png,"LSS MRI : Feature of muscle spasm. Central and Lt paracentral disc prortusion noted at L5/S1 level, compressing the thecal sac and Lt (S1) exit nerve root.",0,1
2074,local_data/mendeley/T2_0504_D4.png,"LSS MRI : Feature of muscle spasm. Central and Lt paracentral disc prortusion noted at L5/S1 level, compressing the thecal sac and Lt (S1) exit nerve root.",0,1
2075,local_data/mendeley/T2_0504_D5.png,"LSS MRI : Feature of muscle spasm. Central and Lt paracentral disc prortusion noted at L5/S1 level, compressing the thecal sac and Lt (S1) exit nerve root.",0,1
2076,local_data/mendeley/T1_0506_D3.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac. features of muscle spasm.",0,1
2077,local_data/mendeley/T1_0506_D4.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac. features of muscle spasm.",0,1
2078,local_data/mendeley/T1_0506_D5.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac. features of muscle spasm.",0,1
2079,local_data/mendeley/T2_0506_D3.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac. features of muscle spasm.",0,1
2080,local_data/mendeley/T2_0506_D4.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac. features of muscle spasm.",0,1
2081,local_data/mendeley/T2_0506_D5.png,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac. features of muscle spasm.",0,1
2082,local_data/mendeley/T1_0510_D3.png,"LSS MRI : Feature of muscle spasm. Partial dissicating disc material noted at L4/L5 level Diffuse disc bulg noted at L4/L5 level, compressing thecal sac andd both nerve roots",0,1
2083,local_data/mendeley/T1_0510_D4.png,"LSS MRI : Feature of muscle spasm. Partial dissicating disc material noted at L4/L5 level Diffuse disc bulg noted at L4/L5 level, compressing thecal sac andd both nerve roots",0,1
2084,local_data/mendeley/T1_0510_D5.png,"LSS MRI : Feature of muscle spasm. Partial dissicating disc material noted at L4/L5 level Diffuse disc bulg noted at L4/L5 level, compressing thecal sac andd both nerve roots",0,1
2085,local_data/mendeley/T2_0510_D3.png,"LSS MRI : Feature of muscle spasm. Partial dissicating disc material noted at L4/L5 level Diffuse disc bulg noted at L4/L5 level, compressing thecal sac andd both nerve roots",0,1
2086,local_data/mendeley/T2_0510_D4.png,"LSS MRI : Feature of muscle spasm. Partial dissicating disc material noted at L4/L5 level Diffuse disc bulg noted at L4/L5 level, compressing thecal sac andd both nerve roots",0,1
2087,local_data/mendeley/T2_0510_D5.png,"LSS MRI : Feature of muscle spasm. Partial dissicating disc material noted at L4/L5 level Diffuse disc bulg noted at L4/L5 level, compressing thecal sac andd both nerve roots",0,1
2088,local_data/mendeley/T1_0512_D3.png,"Lumbosacral MRI Feature of muscle spasm Desiccative disc material noted at the lower levels Multiple Schmorl's nodule noted Modic type II noted at the L5-S1 L4-L5 leve:l wide based disc bulge noted ,mildly compressing the thecal sac and exiting nerve root L5-S1 level: Diffuse disc bulge noted compressing the thecal sac and encroaching upon the neural canals",0,1
2089,local_data/mendeley/T1_0512_D4.png,"Lumbosacral MRI Feature of muscle spasm Desiccative disc material noted at the lower levels Multiple Schmorl's nodule noted Modic type II noted at the L5-S1 L4-L5 leve:l wide based disc bulge noted ,mildly compressing the thecal sac and exiting nerve root L5-S1 level: Diffuse disc bulge noted compressing the thecal sac and encroaching upon the neural canals",0,1
2090,local_data/mendeley/T1_0512_D5.png,"Lumbosacral MRI Feature of muscle spasm Desiccative disc material noted at the lower levels Multiple Schmorl's nodule noted Modic type II noted at the L5-S1 L4-L5 leve:l wide based disc bulge noted ,mildly compressing the thecal sac and exiting nerve root L5-S1 level: Diffuse disc bulge noted compressing the thecal sac and encroaching upon the neural canals",0,1
2091,local_data/mendeley/T2_0512_D3.png,"Lumbosacral MRI Feature of muscle spasm Desiccative disc material noted at the lower levels Multiple Schmorl's nodule noted Modic type II noted at the L5-S1 L4-L5 leve:l wide based disc bulge noted ,mildly compressing the thecal sac and exiting nerve root L5-S1 level: Diffuse disc bulge noted compressing the thecal sac and encroaching upon the neural canals",0,1
2092,local_data/mendeley/T2_0512_D4.png,"Lumbosacral MRI Feature of muscle spasm Desiccative disc material noted at the lower levels Multiple Schmorl's nodule noted Modic type II noted at the L5-S1 L4-L5 leve:l wide based disc bulge noted ,mildly compressing the thecal sac and exiting nerve root L5-S1 level: Diffuse disc bulge noted compressing the thecal sac and encroaching upon the neural canals",0,1
2093,local_data/mendeley/T2_0512_D5.png,"Lumbosacral MRI Feature of muscle spasm Desiccative disc material noted at the lower levels Multiple Schmorl's nodule noted Modic type II noted at the L5-S1 L4-L5 leve:l wide based disc bulge noted ,mildly compressing the thecal sac and exiting nerve root L5-S1 level: Diffuse disc bulge noted compressing the thecal sac and encroaching upon the neural canals",0,1
2094,local_data/mendeley/T1_0514_D3.png,"LSS MRI : feature of muscle spasm. diffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both exit nerve roots .",0,1
2095,local_data/mendeley/T1_0514_D4.png,"LSS MRI : feature of muscle spasm. diffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both exit nerve roots .",0,1
2096,local_data/mendeley/T1_0514_D5.png,"LSS MRI : feature of muscle spasm. diffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both exit nerve roots .",0,1
2097,local_data/mendeley/T2_0514_D3.png,"LSS MRI : feature of muscle spasm. diffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both exit nerve roots .",0,1
2098,local_data/mendeley/T2_0514_D4.png,"LSS MRI : feature of muscle spasm. diffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both exit nerve roots .",0,1
2099,local_data/mendeley/T2_0514_D5.png,"LSS MRI : feature of muscle spasm. diffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both exit nerve roots .",0,1
2100,local_data/mendeley/T1_0515_D3.png,"MRI OF THE LUMBAR SPINE : Feature of muscle spasm. L4/L5 level: diffuse bulge noted, with annular tear mild compressing the thecal sac . No significant thecal sac compression . RT KNEE MRI : Tear of posterior horn of medial Meniscus. Joint effusion noted . No other abnormality seen",0,1
2101,local_data/mendeley/T1_0515_D4.png,"MRI OF THE LUMBAR SPINE : Feature of muscle spasm. L4/L5 level: diffuse bulge noted, with annular tear mild compressing the thecal sac . No significant thecal sac compression . RT KNEE MRI : Tear of posterior horn of medial Meniscus. Joint effusion noted . No other abnormality seen",0,1
2102,local_data/mendeley/T1_0515_D5.png,"MRI OF THE LUMBAR SPINE : Feature of muscle spasm. L4/L5 level: diffuse bulge noted, with annular tear mild compressing the thecal sac . No significant thecal sac compression . RT KNEE MRI : Tear of posterior horn of medial Meniscus. Joint effusion noted . No other abnormality seen",0,1
2103,local_data/mendeley/T2_0515_D3.png,"MRI OF THE LUMBAR SPINE : Feature of muscle spasm. L4/L5 level: diffuse bulge noted, with annular tear mild compressing the thecal sac . No significant thecal sac compression . RT KNEE MRI : Tear of posterior horn of medial Meniscus. Joint effusion noted . No other abnormality seen",0,1
2104,local_data/mendeley/T2_0515_D4.png,"MRI OF THE LUMBAR SPINE : Feature of muscle spasm. L4/L5 level: diffuse bulge noted, with annular tear mild compressing the thecal sac . No significant thecal sac compression . RT KNEE MRI : Tear of posterior horn of medial Meniscus. Joint effusion noted . No other abnormality seen",0,1
2105,local_data/mendeley/T2_0515_D5.png,"MRI OF THE LUMBAR SPINE : Feature of muscle spasm. L4/L5 level: diffuse bulge noted, with annular tear mild compressing the thecal sac . No significant thecal sac compression . RT KNEE MRI : Tear of posterior horn of medial Meniscus. Joint effusion noted . No other abnormality seen",0,1
2106,local_data/mendeley/T1_0516_D3.png,"Multilevel Diffuse disc bulges with facet joint hypertrophy noted compressing the thecal sac and exit neural canals, noticeably at L4-L5 level.",0,1
2107,local_data/mendeley/T1_0516_D4.png,"Multilevel Diffuse disc bulges with facet joint hypertrophy noted compressing the thecal sac and exit neural canals, noticeably at L4-L5 level.",0,1
2108,local_data/mendeley/T1_0516_D5.png,"Multilevel Diffuse disc bulges with facet joint hypertrophy noted compressing the thecal sac and exit neural canals, noticeably at L4-L5 level.",0,1
2109,local_data/mendeley/T2_0516_D3.png,"Multilevel Diffuse disc bulges with facet joint hypertrophy noted compressing the thecal sac and exit neural canals, noticeably at L4-L5 level.",0,1
2110,local_data/mendeley/T2_0516_D4.png,"Multilevel Diffuse disc bulges with facet joint hypertrophy noted compressing the thecal sac and exit neural canals, noticeably at L4-L5 level.",0,1
2111,local_data/mendeley/T2_0516_D5.png,"Multilevel Diffuse disc bulges with facet joint hypertrophy noted compressing the thecal sac and exit neural canals, noticeably at L4-L5 level.",0,1
2112,local_data/mendeley/T1_0517_D3.png,LSS MRI Mild diosc bulges noted at L4-L5 and L5-S1 levels. No sigificant thecal sac or nerve root compressuion noted. adequate spinal canal,0,1
2113,local_data/mendeley/T1_0517_D4.png,LSS MRI Mild diosc bulges noted at L4-L5 and L5-S1 levels. No sigificant thecal sac or nerve root compressuion noted. adequate spinal canal,0,1
2114,local_data/mendeley/T1_0517_D5.png,LSS MRI Mild diosc bulges noted at L4-L5 and L5-S1 levels. No sigificant thecal sac or nerve root compressuion noted. adequate spinal canal,0,1
2115,local_data/mendeley/T2_0517_D3.png,LSS MRI Mild diosc bulges noted at L4-L5 and L5-S1 levels. No sigificant thecal sac or nerve root compressuion noted. adequate spinal canal,0,1
2116,local_data/mendeley/T2_0517_D4.png,LSS MRI Mild diosc bulges noted at L4-L5 and L5-S1 levels. No sigificant thecal sac or nerve root compressuion noted. adequate spinal canal,0,1
2117,local_data/mendeley/T2_0517_D5.png,LSS MRI Mild diosc bulges noted at L4-L5 and L5-S1 levels. No sigificant thecal sac or nerve root compressuion noted. adequate spinal canal,0,1
2118,local_data/mendeley/T1_0520_D3.png,"LSS MRI L5-S1: Rt paracentral disc protrusion, compressing the thecal sac and Rt exit neural canal. mild disc bulge noted at L4-L5 level.",0,1
2119,local_data/mendeley/T1_0520_D4.png,"LSS MRI L5-S1: Rt paracentral disc protrusion, compressing the thecal sac and Rt exit neural canal. mild disc bulge noted at L4-L5 level.",0,1
2120,local_data/mendeley/T1_0520_D5.png,"LSS MRI L5-S1: Rt paracentral disc protrusion, compressing the thecal sac and Rt exit neural canal. mild disc bulge noted at L4-L5 level.",0,1
2121,local_data/mendeley/T2_0520_D3.png,"LSS MRI L5-S1: Rt paracentral disc protrusion, compressing the thecal sac and Rt exit neural canal. mild disc bulge noted at L4-L5 level.",0,1
2122,local_data/mendeley/T2_0520_D4.png,"LSS MRI L5-S1: Rt paracentral disc protrusion, compressing the thecal sac and Rt exit neural canal. mild disc bulge noted at L4-L5 level.",0,1
2123,local_data/mendeley/T2_0520_D5.png,"LSS MRI L5-S1: Rt paracentral disc protrusion, compressing the thecal sac and Rt exit neural canal. mild disc bulge noted at L4-L5 level.",0,1
2124,local_data/mendeley/T1_0523_D3.png,L5-S1: Lt paracentral disc protrusion compressing the thecal sac and Lt exit neural canal.,0,1
2125,local_data/mendeley/T1_0523_D4.png,L5-S1: Lt paracentral disc protrusion compressing the thecal sac and Lt exit neural canal.,0,1
2126,local_data/mendeley/T1_0523_D5.png,L5-S1: Lt paracentral disc protrusion compressing the thecal sac and Lt exit neural canal.,0,1
2127,local_data/mendeley/T2_0523_D3.png,L5-S1: Lt paracentral disc protrusion compressing the thecal sac and Lt exit neural canal.,0,1
2128,local_data/mendeley/T2_0523_D4.png,L5-S1: Lt paracentral disc protrusion compressing the thecal sac and Lt exit neural canal.,0,1
2129,local_data/mendeley/T2_0523_D5.png,L5-S1: Lt paracentral disc protrusion compressing the thecal sac and Lt exit neural canal.,0,1
2130,local_data/mendeley/T1_0524_D3.png,"LSS MRI : Feature of muscle spasm. Multiple schomrls nodules Wide base disc bulge noted at L4/L5 level, compressing thecal sac more at Rt side with encroaching exit neural canal . Mild diffuse disc bulge noted at L5/S1 level, mild compressing thecal sca and both nerve roots",0,1
2131,local_data/mendeley/T1_0524_D4.png,"LSS MRI : Feature of muscle spasm. Multiple schomrls nodules Wide base disc bulge noted at L4/L5 level, compressing thecal sac more at Rt side with encroaching exit neural canal . Mild diffuse disc bulge noted at L5/S1 level, mild compressing thecal sca and both nerve roots",0,1
2132,local_data/mendeley/T1_0524_D5.png,"LSS MRI : Feature of muscle spasm. Multiple schomrls nodules Wide base disc bulge noted at L4/L5 level, compressing thecal sac more at Rt side with encroaching exit neural canal . Mild diffuse disc bulge noted at L5/S1 level, mild compressing thecal sca and both nerve roots",0,1
2133,local_data/mendeley/T2_0524_D3.png,"LSS MRI : Feature of muscle spasm. Multiple schomrls nodules Wide base disc bulge noted at L4/L5 level, compressing thecal sac more at Rt side with encroaching exit neural canal . Mild diffuse disc bulge noted at L5/S1 level, mild compressing thecal sca and both nerve roots",0,1
2134,local_data/mendeley/T2_0524_D4.png,"LSS MRI : Feature of muscle spasm. Multiple schomrls nodules Wide base disc bulge noted at L4/L5 level, compressing thecal sac more at Rt side with encroaching exit neural canal . Mild diffuse disc bulge noted at L5/S1 level, mild compressing thecal sca and both nerve roots",0,1
2135,local_data/mendeley/T2_0524_D5.png,"LSS MRI : Feature of muscle spasm. Multiple schomrls nodules Wide base disc bulge noted at L4/L5 level, compressing thecal sac more at Rt side with encroaching exit neural canal . Mild diffuse disc bulge noted at L5/S1 level, mild compressing thecal sca and both nerve roots",0,1
2136,local_data/mendeley/T1_0525_D3.png,"C.SPINE MRI : feature of muscle spasm. Rt paracentral disc protrusion with annular tear noted at C5/C6 level, ablitertaing anterior CSF space with indenting spinal cord with mild encroaching exit neural canal. LSS MRI (contrast): Evidnece of previus opertaion at Rt sided (L4/L5 level): Central and RT paracentral inhomogenous enhancing struture noted at L4/L5 level, compressing thecal sac and Rt exit neural canal , mostly post opertaive fibrous -granulation tissue",0,1
2137,local_data/mendeley/T1_0525_D4.png,"C.SPINE MRI : feature of muscle spasm. Rt paracentral disc protrusion with annular tear noted at C5/C6 level, ablitertaing anterior CSF space with indenting spinal cord with mild encroaching exit neural canal. LSS MRI (contrast): Evidnece of previus opertaion at Rt sided (L4/L5 level): Central and RT paracentral inhomogenous enhancing struture noted at L4/L5 level, compressing thecal sac and Rt exit neural canal , mostly post opertaive fibrous -granulation tissue",0,1
2138,local_data/mendeley/T1_0525_D5.png,"C.SPINE MRI : feature of muscle spasm. Rt paracentral disc protrusion with annular tear noted at C5/C6 level, ablitertaing anterior CSF space with indenting spinal cord with mild encroaching exit neural canal. LSS MRI (contrast): Evidnece of previus opertaion at Rt sided (L4/L5 level): Central and RT paracentral inhomogenous enhancing struture noted at L4/L5 level, compressing thecal sac and Rt exit neural canal , mostly post opertaive fibrous -granulation tissue",0,1
2139,local_data/mendeley/T2_0525_D3.png,"C.SPINE MRI : feature of muscle spasm. Rt paracentral disc protrusion with annular tear noted at C5/C6 level, ablitertaing anterior CSF space with indenting spinal cord with mild encroaching exit neural canal. LSS MRI (contrast): Evidnece of previus opertaion at Rt sided (L4/L5 level): Central and RT paracentral inhomogenous enhancing struture noted at L4/L5 level, compressing thecal sac and Rt exit neural canal , mostly post opertaive fibrous -granulation tissue",0,1
2140,local_data/mendeley/T2_0525_D4.png,"C.SPINE MRI : feature of muscle spasm. Rt paracentral disc protrusion with annular tear noted at C5/C6 level, ablitertaing anterior CSF space with indenting spinal cord with mild encroaching exit neural canal. LSS MRI (contrast): Evidnece of previus opertaion at Rt sided (L4/L5 level): Central and RT paracentral inhomogenous enhancing struture noted at L4/L5 level, compressing thecal sac and Rt exit neural canal , mostly post opertaive fibrous -granulation tissue",0,1
2141,local_data/mendeley/T2_0525_D5.png,"C.SPINE MRI : feature of muscle spasm. Rt paracentral disc protrusion with annular tear noted at C5/C6 level, ablitertaing anterior CSF space with indenting spinal cord with mild encroaching exit neural canal. LSS MRI (contrast): Evidnece of previus opertaion at Rt sided (L4/L5 level): Central and RT paracentral inhomogenous enhancing struture noted at L4/L5 level, compressing thecal sac and Rt exit neural canal , mostly post opertaive fibrous -granulation tissue",0,1
2142,local_data/mendeley/T1_0526_D3.png,LSS MRI no evidence of disc herniation. no thecal sac or nerve root compression.,1,0
2143,local_data/mendeley/T1_0526_D4.png,LSS MRI no evidence of disc herniation. no thecal sac or nerve root compression.,1,0
2144,local_data/mendeley/T1_0526_D5.png,LSS MRI no evidence of disc herniation. no thecal sac or nerve root compression.,1,0
2145,local_data/mendeley/T2_0526_D3.png,LSS MRI no evidence of disc herniation. no thecal sac or nerve root compression.,1,0
2146,local_data/mendeley/T2_0526_D4.png,LSS MRI no evidence of disc herniation. no thecal sac or nerve root compression.,1,0
2147,local_data/mendeley/T2_0526_D5.png,LSS MRI no evidence of disc herniation. no thecal sac or nerve root compression.,1,0
2148,local_data/mendeley/T1_0527_D3.png,"Lumbosacral MRI Features of muscle spasm Dissicating disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3, L3-L4 and L4-L5 levels compressing thecal sac and encroaching exiting neural canals Relatively secondary spinal Stenosis noted at the L4-L5 Level",0,1
2149,local_data/mendeley/T1_0527_D4.png,"Lumbosacral MRI Features of muscle spasm Dissicating disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3, L3-L4 and L4-L5 levels compressing thecal sac and encroaching exiting neural canals Relatively secondary spinal Stenosis noted at the L4-L5 Level",0,1
2150,local_data/mendeley/T1_0527_D5.png,"Lumbosacral MRI Features of muscle spasm Dissicating disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3, L3-L4 and L4-L5 levels compressing thecal sac and encroaching exiting neural canals Relatively secondary spinal Stenosis noted at the L4-L5 Level",0,1
2151,local_data/mendeley/T2_0527_D3.png,"Lumbosacral MRI Features of muscle spasm Dissicating disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3, L3-L4 and L4-L5 levels compressing thecal sac and encroaching exiting neural canals Relatively secondary spinal Stenosis noted at the L4-L5 Level",0,1
2152,local_data/mendeley/T2_0527_D4.png,"Lumbosacral MRI Features of muscle spasm Dissicating disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3, L3-L4 and L4-L5 levels compressing thecal sac and encroaching exiting neural canals Relatively secondary spinal Stenosis noted at the L4-L5 Level",0,1
2153,local_data/mendeley/T2_0527_D5.png,"Lumbosacral MRI Features of muscle spasm Dissicating disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3, L3-L4 and L4-L5 levels compressing thecal sac and encroaching exiting neural canals Relatively secondary spinal Stenosis noted at the L4-L5 Level",0,1
2154,local_data/mendeley/T1_0528_D3.png,"LSS MRI : features of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots. .diffusely reduced signal of bone marrow noted, mostly non-specific - however, correlate with bone isotope scan .",0,1
2155,local_data/mendeley/T1_0528_D4.png,"LSS MRI : features of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots. .diffusely reduced signal of bone marrow noted, mostly non-specific - however, correlate with bone isotope scan .",0,1
2156,local_data/mendeley/T1_0528_D5.png,"LSS MRI : features of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots. .diffusely reduced signal of bone marrow noted, mostly non-specific - however, correlate with bone isotope scan .",0,1
2157,local_data/mendeley/T2_0528_D3.png,"LSS MRI : features of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots. .diffusely reduced signal of bone marrow noted, mostly non-specific - however, correlate with bone isotope scan .",0,1
2158,local_data/mendeley/T2_0528_D4.png,"LSS MRI : features of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots. .diffusely reduced signal of bone marrow noted, mostly non-specific - however, correlate with bone isotope scan .",0,1
2159,local_data/mendeley/T2_0528_D5.png,"LSS MRI : features of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots. .diffusely reduced signal of bone marrow noted, mostly non-specific - however, correlate with bone isotope scan .",0,1
2160,local_data/mendeley/T1_0531_D3.png,"LSS MRI : Feature of muscle spasm.- -Dissicating disc materials noetd at L4/L5&L5/S1 levels. Posteriocentral disc protrusion noted at L5/S1 level, largely compressing the thecal and encroaching both neural canals. mild diffuse disc bulge noted at L4/L5 level.",0,1
2161,local_data/mendeley/T1_0531_D4.png,"LSS MRI : Feature of muscle spasm.- -Dissicating disc materials noetd at L4/L5&L5/S1 levels. Posteriocentral disc protrusion noted at L5/S1 level, largely compressing the thecal and encroaching both neural canals. mild diffuse disc bulge noted at L4/L5 level.",0,1
2162,local_data/mendeley/T1_0531_D5.png,"LSS MRI : Feature of muscle spasm.- -Dissicating disc materials noetd at L4/L5&L5/S1 levels. Posteriocentral disc protrusion noted at L5/S1 level, largely compressing the thecal and encroaching both neural canals. mild diffuse disc bulge noted at L4/L5 level.",0,1
2163,local_data/mendeley/T2_0531_D3.png,"LSS MRI : Feature of muscle spasm.- -Dissicating disc materials noetd at L4/L5&L5/S1 levels. Posteriocentral disc protrusion noted at L5/S1 level, largely compressing the thecal and encroaching both neural canals. mild diffuse disc bulge noted at L4/L5 level.",0,1
2164,local_data/mendeley/T2_0531_D4.png,"LSS MRI : Feature of muscle spasm.- -Dissicating disc materials noetd at L4/L5&L5/S1 levels. Posteriocentral disc protrusion noted at L5/S1 level, largely compressing the thecal and encroaching both neural canals. mild diffuse disc bulge noted at L4/L5 level.",0,1
2165,local_data/mendeley/T2_0531_D5.png,"LSS MRI : Feature of muscle spasm.- -Dissicating disc materials noetd at L4/L5&L5/S1 levels. Posteriocentral disc protrusion noted at L5/S1 level, largely compressing the thecal and encroaching both neural canals. mild diffuse disc bulge noted at L4/L5 level.",0,1
2166,local_data/mendeley/T1_0532_D3.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials .- -Rt para central disc protrotion , slight caudal migratition noted at L4/L5 level, compressing Rt thecal sac & L5 nerve root .",0,1
2167,local_data/mendeley/T1_0532_D4.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials .- -Rt para central disc protrotion , slight caudal migratition noted at L4/L5 level, compressing Rt thecal sac & L5 nerve root .",0,1
2168,local_data/mendeley/T1_0532_D5.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials .- -Rt para central disc protrotion , slight caudal migratition noted at L4/L5 level, compressing Rt thecal sac & L5 nerve root .",0,1
2169,local_data/mendeley/T2_0532_D3.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials .- -Rt para central disc protrotion , slight caudal migratition noted at L4/L5 level, compressing Rt thecal sac & L5 nerve root .",0,1
2170,local_data/mendeley/T2_0532_D4.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials .- -Rt para central disc protrotion , slight caudal migratition noted at L4/L5 level, compressing Rt thecal sac & L5 nerve root .",0,1
2171,local_data/mendeley/T2_0532_D5.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials .- -Rt para central disc protrotion , slight caudal migratition noted at L4/L5 level, compressing Rt thecal sac & L5 nerve root .",0,1
2172,local_data/mendeley/T1_0533_D3.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
2173,local_data/mendeley/T1_0533_D4.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
2174,local_data/mendeley/T1_0533_D5.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
2175,local_data/mendeley/T2_0533_D3.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
2176,local_data/mendeley/T2_0533_D4.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
2177,local_data/mendeley/T2_0533_D5.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
2178,local_data/mendeley/T1_0534_D3.png,"LSS MRI : Feature of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and exit nerve roots associated with annular tear",0,1
2179,local_data/mendeley/T1_0534_D4.png,"LSS MRI : Feature of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and exit nerve roots associated with annular tear",0,1
2180,local_data/mendeley/T1_0534_D5.png,"LSS MRI : Feature of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and exit nerve roots associated with annular tear",0,1
2181,local_data/mendeley/T2_0534_D3.png,"LSS MRI : Feature of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and exit nerve roots associated with annular tear",0,1
2182,local_data/mendeley/T2_0534_D4.png,"LSS MRI : Feature of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and exit nerve roots associated with annular tear",0,1
2183,local_data/mendeley/T2_0534_D5.png,"LSS MRI : Feature of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and exit nerve roots associated with annular tear",0,1
2184,local_data/mendeley/T1_0536_D3.png,L4-l5: Diffuse disc bulge noted abutting the thecal sac and encroaching exit canals.,0,1
2185,local_data/mendeley/T1_0536_D4.png,L4-l5: Diffuse disc bulge noted abutting the thecal sac and encroaching exit canals.,0,1
2186,local_data/mendeley/T1_0536_D5.png,L4-l5: Diffuse disc bulge noted abutting the thecal sac and encroaching exit canals.,0,1
2187,local_data/mendeley/T2_0536_D3.png,L4-l5: Diffuse disc bulge noted abutting the thecal sac and encroaching exit canals.,0,1
2188,local_data/mendeley/T2_0536_D4.png,L4-l5: Diffuse disc bulge noted abutting the thecal sac and encroaching exit canals.,0,1
2189,local_data/mendeley/T2_0536_D5.png,L4-l5: Diffuse disc bulge noted abutting the thecal sac and encroaching exit canals.,0,1
2190,local_data/mendeley/T1_0537_D3.png,"LSS MRI : Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac.",0,1
2191,local_data/mendeley/T1_0537_D4.png,"LSS MRI : Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac.",0,1
2192,local_data/mendeley/T1_0537_D5.png,"LSS MRI : Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac.",0,1
2193,local_data/mendeley/T2_0537_D3.png,"LSS MRI : Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac.",0,1
2194,local_data/mendeley/T2_0537_D4.png,"LSS MRI : Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac.",0,1
2195,local_data/mendeley/T2_0537_D5.png,"LSS MRI : Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac.",0,1
2196,local_data/mendeley/T1_0538_D3.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve root compression seen . No disc herniation or protruion noted.,1,0
2197,local_data/mendeley/T1_0538_D4.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve root compression seen . No disc herniation or protruion noted.,1,0
2198,local_data/mendeley/T1_0538_D5.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve root compression seen . No disc herniation or protruion noted.,1,0
2199,local_data/mendeley/T2_0538_D3.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve root compression seen . No disc herniation or protruion noted.,1,0
2200,local_data/mendeley/T2_0538_D4.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve root compression seen . No disc herniation or protruion noted.,1,0
2201,local_data/mendeley/T2_0538_D5.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve root compression seen . No disc herniation or protruion noted.,1,0
2202,local_data/mendeley/T1_0539_D3.png,"L5-S1, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
2203,local_data/mendeley/T1_0539_D4.png,"L5-S1, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
2204,local_data/mendeley/T1_0539_D5.png,"L5-S1, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
2205,local_data/mendeley/T2_0539_D3.png,"L5-S1, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
2206,local_data/mendeley/T2_0539_D4.png,"L5-S1, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
2207,local_data/mendeley/T2_0539_D5.png,"L5-S1, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
2208,local_data/mendeley/T1_0540_D3.png,LSS MRI No evidence of disc herniation .No significant thecal sac or nerve root compression noted.,1,0
2209,local_data/mendeley/T1_0540_D4.png,LSS MRI No evidence of disc herniation .No significant thecal sac or nerve root compression noted.,1,0
2210,local_data/mendeley/T1_0540_D5.png,LSS MRI No evidence of disc herniation .No significant thecal sac or nerve root compression noted.,1,0
2211,local_data/mendeley/T2_0540_D3.png,LSS MRI No evidence of disc herniation .No significant thecal sac or nerve root compression noted.,1,0
2212,local_data/mendeley/T2_0540_D4.png,LSS MRI No evidence of disc herniation .No significant thecal sac or nerve root compression noted.,1,0
2213,local_data/mendeley/T2_0540_D5.png,LSS MRI No evidence of disc herniation .No significant thecal sac or nerve root compression noted.,1,0
2214,local_data/mendeley/T1_0542_D3.png,"LSS MRI : Diffuse disc bulge with Central and to the Rt disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.",0,1
2215,local_data/mendeley/T1_0542_D4.png,"LSS MRI : Diffuse disc bulge with Central and to the Rt disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.",0,1
2216,local_data/mendeley/T1_0542_D5.png,"LSS MRI : Diffuse disc bulge with Central and to the Rt disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.",0,1
2217,local_data/mendeley/T2_0542_D3.png,"LSS MRI : Diffuse disc bulge with Central and to the Rt disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.",0,1
2218,local_data/mendeley/T2_0542_D4.png,"LSS MRI : Diffuse disc bulge with Central and to the Rt disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.",0,1
2219,local_data/mendeley/T2_0542_D5.png,"LSS MRI : Diffuse disc bulge with Central and to the Rt disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.",0,1
2220,local_data/mendeley/T1_0545_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2221,local_data/mendeley/T1_0545_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2222,local_data/mendeley/T1_0545_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2223,local_data/mendeley/T2_0545_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2224,local_data/mendeley/T2_0545_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2225,local_data/mendeley/T2_0545_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
2226,local_data/mendeley/T1_0546_D3.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mild cpmpressing thecal sac and nervce roots . NB : -About 4 cm, sub mucosal fibroid noted .",0,1
2227,local_data/mendeley/T1_0546_D4.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mild cpmpressing thecal sac and nervce roots . NB : -About 4 cm, sub mucosal fibroid noted .",0,1
2228,local_data/mendeley/T1_0546_D5.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mild cpmpressing thecal sac and nervce roots . NB : -About 4 cm, sub mucosal fibroid noted .",0,1
2229,local_data/mendeley/T2_0546_D3.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mild cpmpressing thecal sac and nervce roots . NB : -About 4 cm, sub mucosal fibroid noted .",0,1
2230,local_data/mendeley/T2_0546_D4.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mild cpmpressing thecal sac and nervce roots . NB : -About 4 cm, sub mucosal fibroid noted .",0,1
2231,local_data/mendeley/T2_0546_D5.png,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mild cpmpressing thecal sac and nervce roots . NB : -About 4 cm, sub mucosal fibroid noted .",0,1
2232,local_data/mendeley/T1_0547_D3.png,Lumbosacral MRI There is inhomogeneously enhancing soft tissue in the left paracentral and foraminal area at L4-L5 level Mostly fibrosis compressing the thecal sac and left exit neural canal,0,1
2233,local_data/mendeley/T1_0547_D4.png,Lumbosacral MRI There is inhomogeneously enhancing soft tissue in the left paracentral and foraminal area at L4-L5 level Mostly fibrosis compressing the thecal sac and left exit neural canal,0,1
2234,local_data/mendeley/T1_0547_D5.png,Lumbosacral MRI There is inhomogeneously enhancing soft tissue in the left paracentral and foraminal area at L4-L5 level Mostly fibrosis compressing the thecal sac and left exit neural canal,0,1
2235,local_data/mendeley/T2_0547_D3.png,Lumbosacral MRI There is inhomogeneously enhancing soft tissue in the left paracentral and foraminal area at L4-L5 level Mostly fibrosis compressing the thecal sac and left exit neural canal,0,1
2236,local_data/mendeley/T2_0547_D4.png,Lumbosacral MRI There is inhomogeneously enhancing soft tissue in the left paracentral and foraminal area at L4-L5 level Mostly fibrosis compressing the thecal sac and left exit neural canal,0,1
2237,local_data/mendeley/T2_0547_D5.png,Lumbosacral MRI There is inhomogeneously enhancing soft tissue in the left paracentral and foraminal area at L4-L5 level Mostly fibrosis compressing the thecal sac and left exit neural canal,0,1
2238,local_data/mendeley/T1_0548_D3.png,"LSS MRI Features of muscle spasm. Mild disc bulge noted at L5-S1 level, abutting the thecal sac. No signicant thecal sac or nerve root compression noted.",0,1
2239,local_data/mendeley/T1_0548_D4.png,"LSS MRI Features of muscle spasm. Mild disc bulge noted at L5-S1 level, abutting the thecal sac. No signicant thecal sac or nerve root compression noted.",0,1
2240,local_data/mendeley/T1_0548_D5.png,"LSS MRI Features of muscle spasm. Mild disc bulge noted at L5-S1 level, abutting the thecal sac. No signicant thecal sac or nerve root compression noted.",0,1
2241,local_data/mendeley/T2_0548_D3.png,"LSS MRI Features of muscle spasm. Mild disc bulge noted at L5-S1 level, abutting the thecal sac. No signicant thecal sac or nerve root compression noted.",0,1
2242,local_data/mendeley/T2_0548_D4.png,"LSS MRI Features of muscle spasm. Mild disc bulge noted at L5-S1 level, abutting the thecal sac. No signicant thecal sac or nerve root compression noted.",0,1
2243,local_data/mendeley/T2_0548_D5.png,"LSS MRI Features of muscle spasm. Mild disc bulge noted at L5-S1 level, abutting the thecal sac. No signicant thecal sac or nerve root compression noted.",0,1
2244,local_data/mendeley/T1_0549_D3.png,L3-L4: Posterior osteophytes abutting the thecal sac with narrow disc space.,0,1
2245,local_data/mendeley/T1_0549_D4.png,L3-L4: Posterior osteophytes abutting the thecal sac with narrow disc space.,0,1
2246,local_data/mendeley/T1_0549_D5.png,L3-L4: Posterior osteophytes abutting the thecal sac with narrow disc space.,0,1
2247,local_data/mendeley/T2_0549_D3.png,L3-L4: Posterior osteophytes abutting the thecal sac with narrow disc space.,0,1
2248,local_data/mendeley/T2_0549_D4.png,L3-L4: Posterior osteophytes abutting the thecal sac with narrow disc space.,0,1
2249,local_data/mendeley/T2_0549_D5.png,L3-L4: Posterior osteophytes abutting the thecal sac with narrow disc space.,0,1
2250,local_data/mendeley/T1_0550_D3.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit canals.",0,1
2251,local_data/mendeley/T1_0550_D4.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit canals.",0,1
2252,local_data/mendeley/T1_0550_D5.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit canals.",0,1
2253,local_data/mendeley/T2_0550_D3.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit canals.",0,1
2254,local_data/mendeley/T2_0550_D4.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit canals.",0,1
2255,local_data/mendeley/T2_0550_D5.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit canals.",0,1
2256,local_data/mendeley/T1_0551_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2257,local_data/mendeley/T1_0551_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2258,local_data/mendeley/T1_0551_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2259,local_data/mendeley/T2_0551_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2260,local_data/mendeley/T2_0551_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2261,local_data/mendeley/T2_0551_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
2262,local_data/mendeley/T1_0555_D3.png,Lumbosacral spine MRI No evidence of disc herniation No significant thecal sac or nerve root compression noted Adequate spinal canal,1,0
2263,local_data/mendeley/T1_0555_D4.png,Lumbosacral spine MRI No evidence of disc herniation No significant thecal sac or nerve root compression noted Adequate spinal canal,1,0
2264,local_data/mendeley/T1_0555_D5.png,Lumbosacral spine MRI No evidence of disc herniation No significant thecal sac or nerve root compression noted Adequate spinal canal,1,0
2265,local_data/mendeley/T2_0555_D3.png,Lumbosacral spine MRI No evidence of disc herniation No significant thecal sac or nerve root compression noted Adequate spinal canal,1,0
2266,local_data/mendeley/T2_0555_D4.png,Lumbosacral spine MRI No evidence of disc herniation No significant thecal sac or nerve root compression noted Adequate spinal canal,1,0
2267,local_data/mendeley/T2_0555_D5.png,Lumbosacral spine MRI No evidence of disc herniation No significant thecal sac or nerve root compression noted Adequate spinal canal,1,0
2268,local_data/mendeley/T1_0556_D3.png,"LSS MRI : Feature of muscle spasm. .Central disc protrusion more to Rt sided noted at L5/S1 level ,largely compressing the thecal sac and exit nerve roots with narrowed disc space . .Diffuse disc bulge noted at L4/L5 level, mildly compressing the tehcal sac and both nerve roots associated with annular tear .",0,1
2269,local_data/mendeley/T1_0556_D4.png,"LSS MRI : Feature of muscle spasm. .Central disc protrusion more to Rt sided noted at L5/S1 level ,largely compressing the thecal sac and exit nerve roots with narrowed disc space . .Diffuse disc bulge noted at L4/L5 level, mildly compressing the tehcal sac and both nerve roots associated with annular tear .",0,1
2270,local_data/mendeley/T1_0556_D5.png,"LSS MRI : Feature of muscle spasm. .Central disc protrusion more to Rt sided noted at L5/S1 level ,largely compressing the thecal sac and exit nerve roots with narrowed disc space . .Diffuse disc bulge noted at L4/L5 level, mildly compressing the tehcal sac and both nerve roots associated with annular tear .",0,1
2271,local_data/mendeley/T2_0556_D3.png,"LSS MRI : Feature of muscle spasm. .Central disc protrusion more to Rt sided noted at L5/S1 level ,largely compressing the thecal sac and exit nerve roots with narrowed disc space . .Diffuse disc bulge noted at L4/L5 level, mildly compressing the tehcal sac and both nerve roots associated with annular tear .",0,1
2272,local_data/mendeley/T2_0556_D4.png,"LSS MRI : Feature of muscle spasm. .Central disc protrusion more to Rt sided noted at L5/S1 level ,largely compressing the thecal sac and exit nerve roots with narrowed disc space . .Diffuse disc bulge noted at L4/L5 level, mildly compressing the tehcal sac and both nerve roots associated with annular tear .",0,1
2273,local_data/mendeley/T2_0556_D5.png,"LSS MRI : Feature of muscle spasm. .Central disc protrusion more to Rt sided noted at L5/S1 level ,largely compressing the thecal sac and exit nerve roots with narrowed disc space . .Diffuse disc bulge noted at L4/L5 level, mildly compressing the tehcal sac and both nerve roots associated with annular tear .",0,1
2274,local_data/mendeley/T1_0557_D3.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
2275,local_data/mendeley/T1_0557_D4.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
2276,local_data/mendeley/T1_0557_D5.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
2277,local_data/mendeley/T2_0557_D3.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
2278,local_data/mendeley/T2_0557_D4.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
2279,local_data/mendeley/T2_0557_D5.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
2280,local_data/mendeley/T1_0559_D3.png,L3-L4: Mild Rt paracentral disc protrusion abutting the thecal sac. L4-L5: Mild central disc protrusion abutting the thecal sac. L5-S1: Mild Lt paracentral disc protrusion abutting the thecal sac.,0,1
2281,local_data/mendeley/T1_0559_D4.png,L3-L4: Mild Rt paracentral disc protrusion abutting the thecal sac. L4-L5: Mild central disc protrusion abutting the thecal sac. L5-S1: Mild Lt paracentral disc protrusion abutting the thecal sac.,0,1
2282,local_data/mendeley/T1_0559_D5.png,L3-L4: Mild Rt paracentral disc protrusion abutting the thecal sac. L4-L5: Mild central disc protrusion abutting the thecal sac. L5-S1: Mild Lt paracentral disc protrusion abutting the thecal sac.,0,1
2283,local_data/mendeley/T2_0559_D3.png,L3-L4: Mild Rt paracentral disc protrusion abutting the thecal sac. L4-L5: Mild central disc protrusion abutting the thecal sac. L5-S1: Mild Lt paracentral disc protrusion abutting the thecal sac.,0,1
2284,local_data/mendeley/T2_0559_D4.png,L3-L4: Mild Rt paracentral disc protrusion abutting the thecal sac. L4-L5: Mild central disc protrusion abutting the thecal sac. L5-S1: Mild Lt paracentral disc protrusion abutting the thecal sac.,0,1
2285,local_data/mendeley/T2_0559_D5.png,L3-L4: Mild Rt paracentral disc protrusion abutting the thecal sac. L4-L5: Mild central disc protrusion abutting the thecal sac. L5-S1: Mild Lt paracentral disc protrusion abutting the thecal sac.,0,1
2286,local_data/mendeley/T1_0561_D3.png,. MRI LSS Feature of muscle spasm Dissicating disc material noted at L4-L5 and L5-S1 wide base disc bulge noted at the L4-L5 level there is mild compressing thecal sac and encroaching the neural canals,0,1
2287,local_data/mendeley/T1_0561_D4.png,. MRI LSS Feature of muscle spasm Dissicating disc material noted at L4-L5 and L5-S1 wide base disc bulge noted at the L4-L5 level there is mild compressing thecal sac and encroaching the neural canals,0,1
2288,local_data/mendeley/T1_0561_D5.png,. MRI LSS Feature of muscle spasm Dissicating disc material noted at L4-L5 and L5-S1 wide base disc bulge noted at the L4-L5 level there is mild compressing thecal sac and encroaching the neural canals,0,1
2289,local_data/mendeley/T2_0561_D3.png,. MRI LSS Feature of muscle spasm Dissicating disc material noted at L4-L5 and L5-S1 wide base disc bulge noted at the L4-L5 level there is mild compressing thecal sac and encroaching the neural canals,0,1
2290,local_data/mendeley/T2_0561_D4.png,. MRI LSS Feature of muscle spasm Dissicating disc material noted at L4-L5 and L5-S1 wide base disc bulge noted at the L4-L5 level there is mild compressing thecal sac and encroaching the neural canals,0,1
2291,local_data/mendeley/T2_0561_D5.png,. MRI LSS Feature of muscle spasm Dissicating disc material noted at L4-L5 and L5-S1 wide base disc bulge noted at the L4-L5 level there is mild compressing thecal sac and encroaching the neural canals,0,1
2292,local_data/mendeley/T1_0563_D3.png,"L4-L5: Diffuse disc bulge with Lt paracentral disc protrusion, compressing the thecal sac and encroaching Lt exit canal. L5-S1: Mild disc bulge noted.",0,1
2293,local_data/mendeley/T1_0563_D4.png,"L4-L5: Diffuse disc bulge with Lt paracentral disc protrusion, compressing the thecal sac and encroaching Lt exit canal. L5-S1: Mild disc bulge noted.",0,1
2294,local_data/mendeley/T1_0563_D5.png,"L4-L5: Diffuse disc bulge with Lt paracentral disc protrusion, compressing the thecal sac and encroaching Lt exit canal. L5-S1: Mild disc bulge noted.",0,1
2295,local_data/mendeley/T2_0563_D3.png,"L4-L5: Diffuse disc bulge with Lt paracentral disc protrusion, compressing the thecal sac and encroaching Lt exit canal. L5-S1: Mild disc bulge noted.",0,1
2296,local_data/mendeley/T2_0563_D4.png,"L4-L5: Diffuse disc bulge with Lt paracentral disc protrusion, compressing the thecal sac and encroaching Lt exit canal. L5-S1: Mild disc bulge noted.",0,1
2297,local_data/mendeley/T2_0563_D5.png,"L4-L5: Diffuse disc bulge with Lt paracentral disc protrusion, compressing the thecal sac and encroaching Lt exit canal. L5-S1: Mild disc bulge noted.",0,1
2298,local_data/mendeley/T1_0564_D3.png,Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3-L4 and L4-5 levels with mild compressing thecal sac and encroaching upon exiting neural canals Small vertebral bodies hemangioma noted,0,1
2299,local_data/mendeley/T1_0564_D4.png,Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3-L4 and L4-5 levels with mild compressing thecal sac and encroaching upon exiting neural canals Small vertebral bodies hemangioma noted,0,1
2300,local_data/mendeley/T1_0564_D5.png,Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3-L4 and L4-5 levels with mild compressing thecal sac and encroaching upon exiting neural canals Small vertebral bodies hemangioma noted,0,1
2301,local_data/mendeley/T2_0564_D3.png,Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3-L4 and L4-5 levels with mild compressing thecal sac and encroaching upon exiting neural canals Small vertebral bodies hemangioma noted,0,1
2302,local_data/mendeley/T2_0564_D4.png,Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3-L4 and L4-5 levels with mild compressing thecal sac and encroaching upon exiting neural canals Small vertebral bodies hemangioma noted,0,1
2303,local_data/mendeley/T2_0564_D5.png,Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3-L4 and L4-5 levels with mild compressing thecal sac and encroaching upon exiting neural canals Small vertebral bodies hemangioma noted,0,1
2304,local_data/mendeley/T1_0568_D3.png,"LSS MRI : Feature of muscle spasm . diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and nerve roots .",0,1
2305,local_data/mendeley/T1_0568_D4.png,"LSS MRI : Feature of muscle spasm . diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and nerve roots .",0,1
2306,local_data/mendeley/T1_0568_D5.png,"LSS MRI : Feature of muscle spasm . diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and nerve roots .",0,1
2307,local_data/mendeley/T2_0568_D3.png,"LSS MRI : Feature of muscle spasm . diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and nerve roots .",0,1
2308,local_data/mendeley/T2_0568_D4.png,"LSS MRI : Feature of muscle spasm . diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and nerve roots .",0,1
2309,local_data/mendeley/T2_0568_D5.png,"LSS MRI : Feature of muscle spasm . diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and nerve roots .",0,1
2310,local_data/mendeley/T1_0569_D3.png,"LSS MRI Mutlivel diffuse disc bulges noted in the lower LSS, noticeably at L4-L5, compressing the thecal sac and encroaching upon exit canals",0,1
2311,local_data/mendeley/T1_0569_D4.png,"LSS MRI Mutlivel diffuse disc bulges noted in the lower LSS, noticeably at L4-L5, compressing the thecal sac and encroaching upon exit canals",0,1
2312,local_data/mendeley/T1_0569_D5.png,"LSS MRI Mutlivel diffuse disc bulges noted in the lower LSS, noticeably at L4-L5, compressing the thecal sac and encroaching upon exit canals",0,1
2313,local_data/mendeley/T2_0569_D3.png,"LSS MRI Mutlivel diffuse disc bulges noted in the lower LSS, noticeably at L4-L5, compressing the thecal sac and encroaching upon exit canals",0,1
2314,local_data/mendeley/T2_0569_D4.png,"LSS MRI Mutlivel diffuse disc bulges noted in the lower LSS, noticeably at L4-L5, compressing the thecal sac and encroaching upon exit canals",0,1
2315,local_data/mendeley/T2_0569_D5.png,"LSS MRI Mutlivel diffuse disc bulges noted in the lower LSS, noticeably at L4-L5, compressing the thecal sac and encroaching upon exit canals",0,1
2316,local_data/mendeley/T1_0572_D3.png,"L3-L4, L4-L5 AND L5-S1: Diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
2317,local_data/mendeley/T1_0572_D4.png,"L3-L4, L4-L5 AND L5-S1: Diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
2318,local_data/mendeley/T1_0572_D5.png,"L3-L4, L4-L5 AND L5-S1: Diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
2319,local_data/mendeley/T2_0572_D3.png,"L3-L4, L4-L5 AND L5-S1: Diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
2320,local_data/mendeley/T2_0572_D4.png,"L3-L4, L4-L5 AND L5-S1: Diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
2321,local_data/mendeley/T2_0572_D5.png,"L3-L4, L4-L5 AND L5-S1: Diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
2322,local_data/mendeley/T1_0575_D3.png,"MRI OF THE L. SPINE Feature of muscle spasm. Desiccating disc materials with degenerative changes . Diffuse disc bulge noted at L4/L5 ,with posterior osteophytes compressing thecal sac and both exit nerve roots, associated with ligamnetum flavum hypertrophy , all leading to secondary spinal canal stenosis . Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and exit nerve roost D11/D12. showed ligamnetum flavum hyperatrophy",0,1
2323,local_data/mendeley/T1_0575_D4.png,"MRI OF THE L. SPINE Feature of muscle spasm. Desiccating disc materials with degenerative changes . Diffuse disc bulge noted at L4/L5 ,with posterior osteophytes compressing thecal sac and both exit nerve roots, associated with ligamnetum flavum hypertrophy , all leading to secondary spinal canal stenosis . Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and exit nerve roost D11/D12. showed ligamnetum flavum hyperatrophy",0,1
2324,local_data/mendeley/T1_0575_D5.png,"MRI OF THE L. SPINE Feature of muscle spasm. Desiccating disc materials with degenerative changes . Diffuse disc bulge noted at L4/L5 ,with posterior osteophytes compressing thecal sac and both exit nerve roots, associated with ligamnetum flavum hypertrophy , all leading to secondary spinal canal stenosis . Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and exit nerve roost D11/D12. showed ligamnetum flavum hyperatrophy",0,1
2325,local_data/mendeley/T2_0575_D3.png,"MRI OF THE L. SPINE Feature of muscle spasm. Desiccating disc materials with degenerative changes . Diffuse disc bulge noted at L4/L5 ,with posterior osteophytes compressing thecal sac and both exit nerve roots, associated with ligamnetum flavum hypertrophy , all leading to secondary spinal canal stenosis . Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and exit nerve roost D11/D12. showed ligamnetum flavum hyperatrophy",0,1
2326,local_data/mendeley/T2_0575_D4.png,"MRI OF THE L. SPINE Feature of muscle spasm. Desiccating disc materials with degenerative changes . Diffuse disc bulge noted at L4/L5 ,with posterior osteophytes compressing thecal sac and both exit nerve roots, associated with ligamnetum flavum hypertrophy , all leading to secondary spinal canal stenosis . Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and exit nerve roost D11/D12. showed ligamnetum flavum hyperatrophy",0,1
2327,local_data/mendeley/T2_0575_D5.png,"MRI OF THE L. SPINE Feature of muscle spasm. Desiccating disc materials with degenerative changes . Diffuse disc bulge noted at L4/L5 ,with posterior osteophytes compressing thecal sac and both exit nerve roots, associated with ligamnetum flavum hypertrophy , all leading to secondary spinal canal stenosis . Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and exit nerve roost D11/D12. showed ligamnetum flavum hyperatrophy",0,1

Overwriting local_data/lbp-train-meta.csv


In [4]:
%%writefile local_data/lbp-valid-meta.csv
,imgpath,report,No Finding,LBP
0,local_data/mendeley/T1_0010_D3.png,LSS MRI no evidence of disc herniation. no significant thecal sac or nerve root compression noted.,1,0
1,local_data/mendeley/T1_0010_D4.png,LSS MRI no evidence of disc herniation. no significant thecal sac or nerve root compression noted.,1,0
2,local_data/mendeley/T1_0010_D5.png,LSS MRI no evidence of disc herniation. no significant thecal sac or nerve root compression noted.,1,0
3,local_data/mendeley/T2_0010_D3.png,LSS MRI no evidence of disc herniation. no significant thecal sac or nerve root compression noted.,1,0
4,local_data/mendeley/T2_0010_D4.png,LSS MRI no evidence of disc herniation. no significant thecal sac or nerve root compression noted.,1,0
5,local_data/mendeley/T2_0010_D5.png,LSS MRI no evidence of disc herniation. no significant thecal sac or nerve root compression noted.,1,0
6,local_data/mendeley/T1_0011_D3.png,"LSS MRI : About 3*2 cm lesion with inhomogenous signal intensity noted just posterior to L1 veretbral body ,largely compressing the Rt side of the thecal sac. contrast study is advsied. Diffuse disc bulge noted at L4/L5 level, compresing the thecal sac and encroaching upon both neural canals. Wide base disc bulge noted at L5/S1 level, extending to Lt lateral recess , compressin the thecal sac and nerve roots, more to left side, associated with ligamnetum flavum hyperatrophy.",0,1
7,local_data/mendeley/T1_0011_D4.png,"LSS MRI : About 3*2 cm lesion with inhomogenous signal intensity noted just posterior to L1 veretbral body ,largely compressing the Rt side of the thecal sac. contrast study is advsied. Diffuse disc bulge noted at L4/L5 level, compresing the thecal sac and encroaching upon both neural canals. Wide base disc bulge noted at L5/S1 level, extending to Lt lateral recess , compressin the thecal sac and nerve roots, more to left side, associated with ligamnetum flavum hyperatrophy.",0,1
8,local_data/mendeley/T1_0011_D5.png,"LSS MRI : About 3*2 cm lesion with inhomogenous signal intensity noted just posterior to L1 veretbral body ,largely compressing the Rt side of the thecal sac. contrast study is advsied. Diffuse disc bulge noted at L4/L5 level, compresing the thecal sac and encroaching upon both neural canals. Wide base disc bulge noted at L5/S1 level, extending to Lt lateral recess , compressin the thecal sac and nerve roots, more to left side, associated with ligamnetum flavum hyperatrophy.",0,1
9,local_data/mendeley/T2_0011_D3.png,"LSS MRI : About 3*2 cm lesion with inhomogenous signal intensity noted just posterior to L1 veretbral body ,largely compressing the Rt side of the thecal sac. contrast study is advsied. Diffuse disc bulge noted at L4/L5 level, compresing the thecal sac and encroaching upon both neural canals. Wide base disc bulge noted at L5/S1 level, extending to Lt lateral recess , compressin the thecal sac and nerve roots, more to left side, associated with ligamnetum flavum hyperatrophy.",0,1
10,local_data/mendeley/T2_0011_D4.png,"LSS MRI : About 3*2 cm lesion with inhomogenous signal intensity noted just posterior to L1 veretbral body ,largely compressing the Rt side of the thecal sac. contrast study is advsied. Diffuse disc bulge noted at L4/L5 level, compresing the thecal sac and encroaching upon both neural canals. Wide base disc bulge noted at L5/S1 level, extending to Lt lateral recess , compressin the thecal sac and nerve roots, more to left side, associated with ligamnetum flavum hyperatrophy.",0,1
11,local_data/mendeley/T2_0011_D5.png,"LSS MRI : About 3*2 cm lesion with inhomogenous signal intensity noted just posterior to L1 veretbral body ,largely compressing the Rt side of the thecal sac. contrast study is advsied. Diffuse disc bulge noted at L4/L5 level, compresing the thecal sac and encroaching upon both neural canals. Wide base disc bulge noted at L5/S1 level, extending to Lt lateral recess , compressin the thecal sac and nerve roots, more to left side, associated with ligamnetum flavum hyperatrophy.",0,1
12,local_data/mendeley/T1_0012_D3.png,"LSS MRI Rt paracentral disc protrusion noted at L4-L5 level, compressing the thecal sac. Adequate spinal canal.",0,1
13,local_data/mendeley/T1_0012_D4.png,"LSS MRI Rt paracentral disc protrusion noted at L4-L5 level, compressing the thecal sac. Adequate spinal canal.",0,1
14,local_data/mendeley/T1_0012_D5.png,"LSS MRI Rt paracentral disc protrusion noted at L4-L5 level, compressing the thecal sac. Adequate spinal canal.",0,1
15,local_data/mendeley/T2_0012_D3.png,"LSS MRI Rt paracentral disc protrusion noted at L4-L5 level, compressing the thecal sac. Adequate spinal canal.",0,1
16,local_data/mendeley/T2_0012_D4.png,"LSS MRI Rt paracentral disc protrusion noted at L4-L5 level, compressing the thecal sac. Adequate spinal canal.",0,1
17,local_data/mendeley/T2_0012_D5.png,"LSS MRI Rt paracentral disc protrusion noted at L4-L5 level, compressing the thecal sac. Adequate spinal canal.",0,1
18,local_data/mendeley/T1_0016_D3.png,"Lumbosacral MRI Features of muscle spasm Dissicating lower disc space noted -Central and left paracentral disc protrusion noted at L4-L5 level compressing the thecal sac and Lt exit nerve root -Diffuse Disc bulges noted at the L5-S1 level , mild compressing the thecal sac and exiting nerve root.",0,1
19,local_data/mendeley/T1_0016_D4.png,"Lumbosacral MRI Features of muscle spasm Dissicating lower disc space noted -Central and left paracentral disc protrusion noted at L4-L5 level compressing the thecal sac and Lt exit nerve root -Diffuse Disc bulges noted at the L5-S1 level , mild compressing the thecal sac and exiting nerve root.",0,1
20,local_data/mendeley/T1_0016_D5.png,"Lumbosacral MRI Features of muscle spasm Dissicating lower disc space noted -Central and left paracentral disc protrusion noted at L4-L5 level compressing the thecal sac and Lt exit nerve root -Diffuse Disc bulges noted at the L5-S1 level , mild compressing the thecal sac and exiting nerve root.",0,1
21,local_data/mendeley/T2_0016_D3.png,"Lumbosacral MRI Features of muscle spasm Dissicating lower disc space noted -Central and left paracentral disc protrusion noted at L4-L5 level compressing the thecal sac and Lt exit nerve root -Diffuse Disc bulges noted at the L5-S1 level , mild compressing the thecal sac and exiting nerve root.",0,1
22,local_data/mendeley/T2_0016_D4.png,"Lumbosacral MRI Features of muscle spasm Dissicating lower disc space noted -Central and left paracentral disc protrusion noted at L4-L5 level compressing the thecal sac and Lt exit nerve root -Diffuse Disc bulges noted at the L5-S1 level , mild compressing the thecal sac and exiting nerve root.",0,1
23,local_data/mendeley/T2_0016_D5.png,"Lumbosacral MRI Features of muscle spasm Dissicating lower disc space noted -Central and left paracentral disc protrusion noted at L4-L5 level compressing the thecal sac and Lt exit nerve root -Diffuse Disc bulges noted at the L5-S1 level , mild compressing the thecal sac and exiting nerve root.",0,1
24,local_data/mendeley/T1_0019_D3.png,L1-2: central disc bulge is noted indenting thecal sac/// L3-4: right posterolateral disc bulge is noted with end-plate osteophyte formation causing compression of right nerve root within neural foramen. L5-S1: right paramedian disc bulge is noted indenting thecal sac.,0,1
25,local_data/mendeley/T1_0019_D4.png,L1-2: central disc bulge is noted indenting thecal sac/// L3-4: right posterolateral disc bulge is noted with end-plate osteophyte formation causing compression of right nerve root within neural foramen. L5-S1: right paramedian disc bulge is noted indenting thecal sac.,0,1
26,local_data/mendeley/T1_0019_D5.png,L1-2: central disc bulge is noted indenting thecal sac/// L3-4: right posterolateral disc bulge is noted with end-plate osteophyte formation causing compression of right nerve root within neural foramen. L5-S1: right paramedian disc bulge is noted indenting thecal sac.,0,1
27,local_data/mendeley/T2_0019_D3.png,L1-2: central disc bulge is noted indenting thecal sac/// L3-4: right posterolateral disc bulge is noted with end-plate osteophyte formation causing compression of right nerve root within neural foramen. L5-S1: right paramedian disc bulge is noted indenting thecal sac.,0,1
28,local_data/mendeley/T2_0019_D4.png,L1-2: central disc bulge is noted indenting thecal sac/// L3-4: right posterolateral disc bulge is noted with end-plate osteophyte formation causing compression of right nerve root within neural foramen. L5-S1: right paramedian disc bulge is noted indenting thecal sac.,0,1
29,local_data/mendeley/T2_0019_D5.png,L1-2: central disc bulge is noted indenting thecal sac/// L3-4: right posterolateral disc bulge is noted with end-plate osteophyte formation causing compression of right nerve root within neural foramen. L5-S1: right paramedian disc bulge is noted indenting thecal sac.,0,1
30,local_data/mendeley/T1_0021_D3.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge with end-plate osteophytes formation causing compression of thecal sac, compression of left S1 nerve root within lateral recess, and narrowing of neural foramina.",0,1
31,local_data/mendeley/T1_0021_D4.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge with end-plate osteophytes formation causing compression of thecal sac, compression of left S1 nerve root within lateral recess, and narrowing of neural foramina.",0,1
32,local_data/mendeley/T1_0021_D5.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge with end-plate osteophytes formation causing compression of thecal sac, compression of left S1 nerve root within lateral recess, and narrowing of neural foramina.",0,1
33,local_data/mendeley/T2_0021_D3.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge with end-plate osteophytes formation causing compression of thecal sac, compression of left S1 nerve root within lateral recess, and narrowing of neural foramina.",0,1
34,local_data/mendeley/T2_0021_D4.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge with end-plate osteophytes formation causing compression of thecal sac, compression of left S1 nerve root within lateral recess, and narrowing of neural foramina.",0,1
35,local_data/mendeley/T2_0021_D5.png,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge with end-plate osteophytes formation causing compression of thecal sac, compression of left S1 nerve root within lateral recess, and narrowing of neural foramina.",0,1
36,local_data/mendeley/T1_0024_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
37,local_data/mendeley/T1_0024_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
38,local_data/mendeley/T1_0024_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
39,local_data/mendeley/T2_0024_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
40,local_data/mendeley/T2_0024_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
41,local_data/mendeley/T2_0024_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0
42,local_data/mendeley/T1_0026_D3.png,LSS MRI No evidence of disc herniation. No signidant thecal sac or nerve root compression noted.,1,0
43,local_data/mendeley/T1_0026_D4.png,LSS MRI No evidence of disc herniation. No signidant thecal sac or nerve root compression noted.,1,0
44,local_data/mendeley/T1_0026_D5.png,LSS MRI No evidence of disc herniation. No signidant thecal sac or nerve root compression noted.,1,0
45,local_data/mendeley/T2_0026_D3.png,LSS MRI No evidence of disc herniation. No signidant thecal sac or nerve root compression noted.,1,0
46,local_data/mendeley/T2_0026_D4.png,LSS MRI No evidence of disc herniation. No signidant thecal sac or nerve root compression noted.,1,0
47,local_data/mendeley/T2_0026_D5.png,LSS MRI No evidence of disc herniation. No signidant thecal sac or nerve root compression noted.,1,0
48,local_data/mendeley/T1_0033_D3.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
49,local_data/mendeley/T1_0033_D4.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
50,local_data/mendeley/T1_0033_D5.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
51,local_data/mendeley/T2_0033_D3.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
52,local_data/mendeley/T2_0033_D4.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
53,local_data/mendeley/T2_0033_D5.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
54,local_data/mendeley/T1_0046_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
55,local_data/mendeley/T1_0046_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
56,local_data/mendeley/T1_0046_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
57,local_data/mendeley/T2_0046_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
58,local_data/mendeley/T2_0046_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
59,local_data/mendeley/T2_0046_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
60,local_data/mendeley/T1_0062_D3.png,"LSS MRI L4-L5: Lt paracentral disc protrusion noted, compressing the tyhecal sac and Lt exit canal.",0,1
61,local_data/mendeley/T1_0062_D4.png,"LSS MRI L4-L5: Lt paracentral disc protrusion noted, compressing the tyhecal sac and Lt exit canal.",0,1
62,local_data/mendeley/T1_0062_D5.png,"LSS MRI L4-L5: Lt paracentral disc protrusion noted, compressing the tyhecal sac and Lt exit canal.",0,1
63,local_data/mendeley/T2_0062_D3.png,"LSS MRI L4-L5: Lt paracentral disc protrusion noted, compressing the tyhecal sac and Lt exit canal.",0,1
64,local_data/mendeley/T2_0062_D4.png,"LSS MRI L4-L5: Lt paracentral disc protrusion noted, compressing the tyhecal sac and Lt exit canal.",0,1
65,local_data/mendeley/T2_0062_D5.png,"LSS MRI L4-L5: Lt paracentral disc protrusion noted, compressing the tyhecal sac and Lt exit canal.",0,1
66,local_data/mendeley/T1_0063_D3.png,"LSS MRI:  Feature of muscle spasm.  Mild diffuse disc bulges noted at L4/L5&L5/S1 levels , mild compressing thecal sac and exit nerve roots .",0,1
67,local_data/mendeley/T1_0063_D4.png,"LSS MRI:  Feature of muscle spasm.  Mild diffuse disc bulges noted at L4/L5&L5/S1 levels , mild compressing thecal sac and exit nerve roots .",0,1
68,local_data/mendeley/T1_0063_D5.png,"LSS MRI:  Feature of muscle spasm.  Mild diffuse disc bulges noted at L4/L5&L5/S1 levels , mild compressing thecal sac and exit nerve roots .",0,1
69,local_data/mendeley/T2_0063_D3.png,"LSS MRI:  Feature of muscle spasm.  Mild diffuse disc bulges noted at L4/L5&L5/S1 levels , mild compressing thecal sac and exit nerve roots .",0,1
70,local_data/mendeley/T2_0063_D4.png,"LSS MRI:  Feature of muscle spasm.  Mild diffuse disc bulges noted at L4/L5&L5/S1 levels , mild compressing thecal sac and exit nerve roots .",0,1
71,local_data/mendeley/T2_0063_D5.png,"LSS MRI:  Feature of muscle spasm.  Mild diffuse disc bulges noted at L4/L5&L5/S1 levels , mild compressing thecal sac and exit nerve roots .",0,1
72,local_data/mendeley/T1_0071_D3.png,Multilevel diffuse disc bulges noted compressing the thecal sac and exit canals. Multilevel schmorl's nodules noted.,0,1
73,local_data/mendeley/T1_0071_D4.png,Multilevel diffuse disc bulges noted compressing the thecal sac and exit canals. Multilevel schmorl's nodules noted.,0,1
74,local_data/mendeley/T1_0071_D5.png,Multilevel diffuse disc bulges noted compressing the thecal sac and exit canals. Multilevel schmorl's nodules noted.,0,1
75,local_data/mendeley/T2_0071_D3.png,Multilevel diffuse disc bulges noted compressing the thecal sac and exit canals. Multilevel schmorl's nodules noted.,0,1
76,local_data/mendeley/T2_0071_D4.png,Multilevel diffuse disc bulges noted compressing the thecal sac and exit canals. Multilevel schmorl's nodules noted.,0,1
77,local_data/mendeley/T2_0071_D5.png,Multilevel diffuse disc bulges noted compressing the thecal sac and exit canals. Multilevel schmorl's nodules noted.,0,1
78,local_data/mendeley/T1_0074_D3.png,"Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level, compressing the thecal sac and right exit neural canal Diffuse disc bulges noted at L4-L5 and L3-L4 levels compressing the thecal sac and exiting neural canals Multilevel endplate degeneration noted",0,1
79,local_data/mendeley/T1_0074_D4.png,"Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level, compressing the thecal sac and right exit neural canal Diffuse disc bulges noted at L4-L5 and L3-L4 levels compressing the thecal sac and exiting neural canals Multilevel endplate degeneration noted",0,1
80,local_data/mendeley/T1_0074_D5.png,"Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level, compressing the thecal sac and right exit neural canal Diffuse disc bulges noted at L4-L5 and L3-L4 levels compressing the thecal sac and exiting neural canals Multilevel endplate degeneration noted",0,1
81,local_data/mendeley/T2_0074_D3.png,"Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level, compressing the thecal sac and right exit neural canal Diffuse disc bulges noted at L4-L5 and L3-L4 levels compressing the thecal sac and exiting neural canals Multilevel endplate degeneration noted",0,1
82,local_data/mendeley/T2_0074_D4.png,"Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level, compressing the thecal sac and right exit neural canal Diffuse disc bulges noted at L4-L5 and L3-L4 levels compressing the thecal sac and exiting neural canals Multilevel endplate degeneration noted",0,1
83,local_data/mendeley/T2_0074_D5.png,"Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level, compressing the thecal sac and right exit neural canal Diffuse disc bulges noted at L4-L5 and L3-L4 levels compressing the thecal sac and exiting neural canals Multilevel endplate degeneration noted",0,1
84,local_data/mendeley/T1_0082_D3.png,LSS MRI : Feature of muscle spasm. No evidence of Thecal sac or nerve root roots compression,0,1
85,local_data/mendeley/T1_0082_D4.png,LSS MRI : Feature of muscle spasm. No evidence of Thecal sac or nerve root roots compression,0,1
86,local_data/mendeley/T1_0082_D5.png,LSS MRI : Feature of muscle spasm. No evidence of Thecal sac or nerve root roots compression,0,1
87,local_data/mendeley/T2_0082_D3.png,LSS MRI : Feature of muscle spasm. No evidence of Thecal sac or nerve root roots compression,0,1
88,local_data/mendeley/T2_0082_D4.png,LSS MRI : Feature of muscle spasm. No evidence of Thecal sac or nerve root roots compression,0,1
89,local_data/mendeley/T2_0082_D5.png,LSS MRI : Feature of muscle spasm. No evidence of Thecal sac or nerve root roots compression,0,1
90,local_data/mendeley/T1_0083_D3.png,"Lumbosacral MRI Diffuse disc bulge noted at L4-5 level compressing the thecal sac and exit neural canals Left paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and left exit neural,",0,1
91,local_data/mendeley/T1_0083_D4.png,"Lumbosacral MRI Diffuse disc bulge noted at L4-5 level compressing the thecal sac and exit neural canals Left paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and left exit neural,",0,1
92,local_data/mendeley/T1_0083_D5.png,"Lumbosacral MRI Diffuse disc bulge noted at L4-5 level compressing the thecal sac and exit neural canals Left paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and left exit neural,",0,1
93,local_data/mendeley/T2_0083_D3.png,"Lumbosacral MRI Diffuse disc bulge noted at L4-5 level compressing the thecal sac and exit neural canals Left paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and left exit neural,",0,1
94,local_data/mendeley/T2_0083_D4.png,"Lumbosacral MRI Diffuse disc bulge noted at L4-5 level compressing the thecal sac and exit neural canals Left paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and left exit neural,",0,1
95,local_data/mendeley/T2_0083_D5.png,"Lumbosacral MRI Diffuse disc bulge noted at L4-5 level compressing the thecal sac and exit neural canals Left paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and left exit neural,",0,1
96,local_data/mendeley/T1_0087_D3.png,no evidence of disc heniation. no significant thecal sac or nerve root compression.,1,0
97,local_data/mendeley/T1_0087_D4.png,no evidence of disc heniation. no significant thecal sac or nerve root compression.,1,0
98,local_data/mendeley/T1_0087_D5.png,no evidence of disc heniation. no significant thecal sac or nerve root compression.,1,0
99,local_data/mendeley/T2_0087_D3.png,no evidence of disc heniation. no significant thecal sac or nerve root compression.,1,0
100,local_data/mendeley/T2_0087_D4.png,no evidence of disc heniation. no significant thecal sac or nerve root compression.,1,0
101,local_data/mendeley/T2_0087_D5.png,no evidence of disc heniation. no significant thecal sac or nerve root compression.,1,0
102,local_data/mendeley/T1_0092_D3.png,"LSS MRI : faeture of muscle spasm. there is Posterio-central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching both exit neural canal",0,1
103,local_data/mendeley/T1_0092_D4.png,"LSS MRI : faeture of muscle spasm. there is Posterio-central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching both exit neural canal",0,1
104,local_data/mendeley/T1_0092_D5.png,"LSS MRI : faeture of muscle spasm. there is Posterio-central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching both exit neural canal",0,1
105,local_data/mendeley/T2_0092_D3.png,"LSS MRI : faeture of muscle spasm. there is Posterio-central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching both exit neural canal",0,1
106,local_data/mendeley/T2_0092_D4.png,"LSS MRI : faeture of muscle spasm. there is Posterio-central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching both exit neural canal",0,1
107,local_data/mendeley/T2_0092_D5.png,"LSS MRI : faeture of muscle spasm. there is Posterio-central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching both exit neural canal",0,1
108,local_data/mendeley/T1_0094_D3.png,LSS MRI Features of muscle spasm. mild disc bulge noted at L4-L5 level.,0,1
109,local_data/mendeley/T1_0094_D4.png,LSS MRI Features of muscle spasm. mild disc bulge noted at L4-L5 level.,0,1
110,local_data/mendeley/T1_0094_D5.png,LSS MRI Features of muscle spasm. mild disc bulge noted at L4-L5 level.,0,1
111,local_data/mendeley/T2_0094_D3.png,LSS MRI Features of muscle spasm. mild disc bulge noted at L4-L5 level.,0,1
112,local_data/mendeley/T2_0094_D4.png,LSS MRI Features of muscle spasm. mild disc bulge noted at L4-L5 level.,0,1
113,local_data/mendeley/T2_0094_D5.png,LSS MRI Features of muscle spasm. mild disc bulge noted at L4-L5 level.,0,1
114,local_data/mendeley/T1_0100_D3.png,"LSS MRI Diffuse disc bulge noted at L3-L4 level, more to the Lt side, compressing the thecal sac and exit canals.  Features of muscle spasm.",0,1
115,local_data/mendeley/T1_0100_D4.png,"LSS MRI Diffuse disc bulge noted at L3-L4 level, more to the Lt side, compressing the thecal sac and exit canals.  Features of muscle spasm.",0,1
116,local_data/mendeley/T1_0100_D5.png,"LSS MRI Diffuse disc bulge noted at L3-L4 level, more to the Lt side, compressing the thecal sac and exit canals.  Features of muscle spasm.",0,1
117,local_data/mendeley/T2_0100_D3.png,"LSS MRI Diffuse disc bulge noted at L3-L4 level, more to the Lt side, compressing the thecal sac and exit canals.  Features of muscle spasm.",0,1
118,local_data/mendeley/T2_0100_D4.png,"LSS MRI Diffuse disc bulge noted at L3-L4 level, more to the Lt side, compressing the thecal sac and exit canals.  Features of muscle spasm.",0,1
119,local_data/mendeley/T2_0100_D5.png,"LSS MRI Diffuse disc bulge noted at L3-L4 level, more to the Lt side, compressing the thecal sac and exit canals.  Features of muscle spasm.",0,1
120,local_data/mendeley/T1_0117_D3.png,LSS MRI  Multilevel degenerative end plate changes noted.  mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.  tarlov cyst noted at S2 level,0,1
121,local_data/mendeley/T1_0117_D4.png,LSS MRI  Multilevel degenerative end plate changes noted.  mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.  tarlov cyst noted at S2 level,0,1
122,local_data/mendeley/T1_0117_D5.png,LSS MRI  Multilevel degenerative end plate changes noted.  mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.  tarlov cyst noted at S2 level,0,1
123,local_data/mendeley/T2_0117_D3.png,LSS MRI  Multilevel degenerative end plate changes noted.  mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.  tarlov cyst noted at S2 level,0,1
124,local_data/mendeley/T2_0117_D4.png,LSS MRI  Multilevel degenerative end plate changes noted.  mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.  tarlov cyst noted at S2 level,0,1
125,local_data/mendeley/T2_0117_D5.png,LSS MRI  Multilevel degenerative end plate changes noted.  mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.  tarlov cyst noted at S2 level,0,1
126,local_data/mendeley/T1_0130_D3.png,L3-L4 and L4-L5: Mild disc bulges noted. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,0,1
127,local_data/mendeley/T1_0130_D4.png,L3-L4 and L4-L5: Mild disc bulges noted. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,0,1
128,local_data/mendeley/T1_0130_D5.png,L3-L4 and L4-L5: Mild disc bulges noted. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,0,1
129,local_data/mendeley/T2_0130_D3.png,L3-L4 and L4-L5: Mild disc bulges noted. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,0,1
130,local_data/mendeley/T2_0130_D4.png,L3-L4 and L4-L5: Mild disc bulges noted. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,0,1
131,local_data/mendeley/T2_0130_D5.png,L3-L4 and L4-L5: Mild disc bulges noted. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,0,1
132,local_data/mendeley/T1_0140_D3.png,"LSS MRI Central and to the Rt side, disc protrusion noted at L5-S1 level,  compressing the thecal sac and encroaching upon the Rt exit canal. Features of muscle spasm.",0,1
133,local_data/mendeley/T1_0140_D4.png,"LSS MRI Central and to the Rt side, disc protrusion noted at L5-S1 level,  compressing the thecal sac and encroaching upon the Rt exit canal. Features of muscle spasm.",0,1
134,local_data/mendeley/T1_0140_D5.png,"LSS MRI Central and to the Rt side, disc protrusion noted at L5-S1 level,  compressing the thecal sac and encroaching upon the Rt exit canal. Features of muscle spasm.",0,1
135,local_data/mendeley/T2_0140_D3.png,"LSS MRI Central and to the Rt side, disc protrusion noted at L5-S1 level,  compressing the thecal sac and encroaching upon the Rt exit canal. Features of muscle spasm.",0,1
136,local_data/mendeley/T2_0140_D4.png,"LSS MRI Central and to the Rt side, disc protrusion noted at L5-S1 level,  compressing the thecal sac and encroaching upon the Rt exit canal. Features of muscle spasm.",0,1
137,local_data/mendeley/T2_0140_D5.png,"LSS MRI Central and to the Rt side, disc protrusion noted at L5-S1 level,  compressing the thecal sac and encroaching upon the Rt exit canal. Features of muscle spasm.",0,1
138,local_data/mendeley/T1_0147_D3.png,"L3-L4, L4-L5, L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
139,local_data/mendeley/T1_0147_D4.png,"L3-L4, L4-L5, L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
140,local_data/mendeley/T1_0147_D5.png,"L3-L4, L4-L5, L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
141,local_data/mendeley/T2_0147_D3.png,"L3-L4, L4-L5, L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
142,local_data/mendeley/T2_0147_D4.png,"L3-L4, L4-L5, L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
143,local_data/mendeley/T2_0147_D5.png,"L3-L4, L4-L5, L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
144,local_data/mendeley/T1_0148_D3.png,"L4-L5: Broad based disc protrusion, more to the Lt side, compressing the thecal sac and exit canals (largely the Lt one).",0,1
145,local_data/mendeley/T1_0148_D4.png,"L4-L5: Broad based disc protrusion, more to the Lt side, compressing the thecal sac and exit canals (largely the Lt one).",0,1
146,local_data/mendeley/T1_0148_D5.png,"L4-L5: Broad based disc protrusion, more to the Lt side, compressing the thecal sac and exit canals (largely the Lt one).",0,1
147,local_data/mendeley/T2_0148_D3.png,"L4-L5: Broad based disc protrusion, more to the Lt side, compressing the thecal sac and exit canals (largely the Lt one).",0,1
148,local_data/mendeley/T2_0148_D4.png,"L4-L5: Broad based disc protrusion, more to the Lt side, compressing the thecal sac and exit canals (largely the Lt one).",0,1
149,local_data/mendeley/T2_0148_D5.png,"L4-L5: Broad based disc protrusion, more to the Lt side, compressing the thecal sac and exit canals (largely the Lt one).",0,1
150,local_data/mendeley/T1_0153_D3.png,"LSS MRI L4-L5: Diffuse disc bulge compresing the thecal sac and exit canals. L5-S1: Mild disc bulge, more to the Lt side, encroaching on Lt exit canal. Adequate spinal canal.",0,1
151,local_data/mendeley/T1_0153_D4.png,"LSS MRI L4-L5: Diffuse disc bulge compresing the thecal sac and exit canals. L5-S1: Mild disc bulge, more to the Lt side, encroaching on Lt exit canal. Adequate spinal canal.",0,1
152,local_data/mendeley/T1_0153_D5.png,"LSS MRI L4-L5: Diffuse disc bulge compresing the thecal sac and exit canals. L5-S1: Mild disc bulge, more to the Lt side, encroaching on Lt exit canal. Adequate spinal canal.",0,1
153,local_data/mendeley/T2_0153_D3.png,"LSS MRI L4-L5: Diffuse disc bulge compresing the thecal sac and exit canals. L5-S1: Mild disc bulge, more to the Lt side, encroaching on Lt exit canal. Adequate spinal canal.",0,1
154,local_data/mendeley/T2_0153_D4.png,"LSS MRI L4-L5: Diffuse disc bulge compresing the thecal sac and exit canals. L5-S1: Mild disc bulge, more to the Lt side, encroaching on Lt exit canal. Adequate spinal canal.",0,1
155,local_data/mendeley/T2_0153_D5.png,"LSS MRI L4-L5: Diffuse disc bulge compresing the thecal sac and exit canals. L5-S1: Mild disc bulge, more to the Lt side, encroaching on Lt exit canal. Adequate spinal canal.",0,1
156,local_data/mendeley/T1_0164_D3.png,MRI LSS:  feature of muscle spasm  Endplate Modic typeI noted at L4/L5 level with dehyadrated disc material.  Central and to the Rt disc protrusion noted at the L4-L5 level significantly compression thecal sac and exit nerve roots,0,1
157,local_data/mendeley/T1_0164_D4.png,MRI LSS:  feature of muscle spasm  Endplate Modic typeI noted at L4/L5 level with dehyadrated disc material.  Central and to the Rt disc protrusion noted at the L4-L5 level significantly compression thecal sac and exit nerve roots,0,1
158,local_data/mendeley/T1_0164_D5.png,MRI LSS:  feature of muscle spasm  Endplate Modic typeI noted at L4/L5 level with dehyadrated disc material.  Central and to the Rt disc protrusion noted at the L4-L5 level significantly compression thecal sac and exit nerve roots,0,1
159,local_data/mendeley/T2_0164_D3.png,MRI LSS:  feature of muscle spasm  Endplate Modic typeI noted at L4/L5 level with dehyadrated disc material.  Central and to the Rt disc protrusion noted at the L4-L5 level significantly compression thecal sac and exit nerve roots,0,1
160,local_data/mendeley/T2_0164_D4.png,MRI LSS:  feature of muscle spasm  Endplate Modic typeI noted at L4/L5 level with dehyadrated disc material.  Central and to the Rt disc protrusion noted at the L4-L5 level significantly compression thecal sac and exit nerve roots,0,1
161,local_data/mendeley/T2_0164_D5.png,MRI LSS:  feature of muscle spasm  Endplate Modic typeI noted at L4/L5 level with dehyadrated disc material.  Central and to the Rt disc protrusion noted at the L4-L5 level significantly compression thecal sac and exit nerve roots,0,1
162,local_data/mendeley/T1_0169_D3.png,"LSS MRI :  L3/L4 level :wide based disc bulge noted with a small soft tissue lesion noted at Lt foraminal region ? neurogenic tumour - sequestrated disc . contrast study is recommended  L4/L5 level: diffuse disc bulge , extended to Rt lateral recess , mild compressing thecal sca and encroaching exit nerve root more to RT .",0,1
163,local_data/mendeley/T1_0169_D4.png,"LSS MRI :  L3/L4 level :wide based disc bulge noted with a small soft tissue lesion noted at Lt foraminal region ? neurogenic tumour - sequestrated disc . contrast study is recommended  L4/L5 level: diffuse disc bulge , extended to Rt lateral recess , mild compressing thecal sca and encroaching exit nerve root more to RT .",0,1
164,local_data/mendeley/T1_0169_D5.png,"LSS MRI :  L3/L4 level :wide based disc bulge noted with a small soft tissue lesion noted at Lt foraminal region ? neurogenic tumour - sequestrated disc . contrast study is recommended  L4/L5 level: diffuse disc bulge , extended to Rt lateral recess , mild compressing thecal sca and encroaching exit nerve root more to RT .",0,1
165,local_data/mendeley/T2_0169_D3.png,"LSS MRI :  L3/L4 level :wide based disc bulge noted with a small soft tissue lesion noted at Lt foraminal region ? neurogenic tumour - sequestrated disc . contrast study is recommended  L4/L5 level: diffuse disc bulge , extended to Rt lateral recess , mild compressing thecal sca and encroaching exit nerve root more to RT .",0,1
166,local_data/mendeley/T2_0169_D4.png,"LSS MRI :  L3/L4 level :wide based disc bulge noted with a small soft tissue lesion noted at Lt foraminal region ? neurogenic tumour - sequestrated disc . contrast study is recommended  L4/L5 level: diffuse disc bulge , extended to Rt lateral recess , mild compressing thecal sca and encroaching exit nerve root more to RT .",0,1
167,local_data/mendeley/T2_0169_D5.png,"LSS MRI :  L3/L4 level :wide based disc bulge noted with a small soft tissue lesion noted at Lt foraminal region ? neurogenic tumour - sequestrated disc . contrast study is recommended  L4/L5 level: diffuse disc bulge , extended to Rt lateral recess , mild compressing thecal sca and encroaching exit nerve root more to RT .",0,1
168,local_data/mendeley/T1_0170_D3.png,"LSS MRI  Diffuse disc bulges, at L4-L5 and L5-S1 levels, compressing the thecal sac and exit neural canals.",0,1
169,local_data/mendeley/T1_0170_D4.png,"LSS MRI  Diffuse disc bulges, at L4-L5 and L5-S1 levels, compressing the thecal sac and exit neural canals.",0,1
170,local_data/mendeley/T1_0170_D5.png,"LSS MRI  Diffuse disc bulges, at L4-L5 and L5-S1 levels, compressing the thecal sac and exit neural canals.",0,1
171,local_data/mendeley/T2_0170_D3.png,"LSS MRI  Diffuse disc bulges, at L4-L5 and L5-S1 levels, compressing the thecal sac and exit neural canals.",0,1
172,local_data/mendeley/T2_0170_D4.png,"LSS MRI  Diffuse disc bulges, at L4-L5 and L5-S1 levels, compressing the thecal sac and exit neural canals.",0,1
173,local_data/mendeley/T2_0170_D5.png,"LSS MRI  Diffuse disc bulges, at L4-L5 and L5-S1 levels, compressing the thecal sac and exit neural canals.",0,1
174,local_data/mendeley/T1_0184_D3.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
175,local_data/mendeley/T1_0184_D4.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
176,local_data/mendeley/T1_0184_D5.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
177,local_data/mendeley/T2_0184_D3.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
178,local_data/mendeley/T2_0184_D4.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
179,local_data/mendeley/T2_0184_D5.png,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1
180,local_data/mendeley/T1_0190_D3.png,"MRI OF THE L. SPINE  Feature of muscle spasm.  Dissecating disc material at L5/S1 with narrowed disc space .  Diffuse disc bulge noted at L5/S1 ,mild compressing thecal sac and exit nerve roots.",0,1
181,local_data/mendeley/T1_0190_D4.png,"MRI OF THE L. SPINE  Feature of muscle spasm.  Dissecating disc material at L5/S1 with narrowed disc space .  Diffuse disc bulge noted at L5/S1 ,mild compressing thecal sac and exit nerve roots.",0,1
182,local_data/mendeley/T1_0190_D5.png,"MRI OF THE L. SPINE  Feature of muscle spasm.  Dissecating disc material at L5/S1 with narrowed disc space .  Diffuse disc bulge noted at L5/S1 ,mild compressing thecal sac and exit nerve roots.",0,1
183,local_data/mendeley/T2_0190_D3.png,"MRI OF THE L. SPINE  Feature of muscle spasm.  Dissecating disc material at L5/S1 with narrowed disc space .  Diffuse disc bulge noted at L5/S1 ,mild compressing thecal sac and exit nerve roots.",0,1
184,local_data/mendeley/T2_0190_D4.png,"MRI OF THE L. SPINE  Feature of muscle spasm.  Dissecating disc material at L5/S1 with narrowed disc space .  Diffuse disc bulge noted at L5/S1 ,mild compressing thecal sac and exit nerve roots.",0,1
185,local_data/mendeley/T2_0190_D5.png,"MRI OF THE L. SPINE  Feature of muscle spasm.  Dissecating disc material at L5/S1 with narrowed disc space .  Diffuse disc bulge noted at L5/S1 ,mild compressing thecal sac and exit nerve roots.",0,1
186,local_data/mendeley/T1_0191_D3.png,Multilevel diffuse disc bulges with facet joint hypertrophy and ligamentum hypertrophy. At L4-L5 level mild spondylolisthesis noted. Spinal canal appeared narrow at L3-L4 and L4-L5 levels.,0,1
187,local_data/mendeley/T1_0191_D4.png,Multilevel diffuse disc bulges with facet joint hypertrophy and ligamentum hypertrophy. At L4-L5 level mild spondylolisthesis noted. Spinal canal appeared narrow at L3-L4 and L4-L5 levels.,0,1
188,local_data/mendeley/T1_0191_D5.png,Multilevel diffuse disc bulges with facet joint hypertrophy and ligamentum hypertrophy. At L4-L5 level mild spondylolisthesis noted. Spinal canal appeared narrow at L3-L4 and L4-L5 levels.,0,1
189,local_data/mendeley/T2_0191_D3.png,Multilevel diffuse disc bulges with facet joint hypertrophy and ligamentum hypertrophy. At L4-L5 level mild spondylolisthesis noted. Spinal canal appeared narrow at L3-L4 and L4-L5 levels.,0,1
190,local_data/mendeley/T2_0191_D4.png,Multilevel diffuse disc bulges with facet joint hypertrophy and ligamentum hypertrophy. At L4-L5 level mild spondylolisthesis noted. Spinal canal appeared narrow at L3-L4 and L4-L5 levels.,0,1
191,local_data/mendeley/T2_0191_D5.png,Multilevel diffuse disc bulges with facet joint hypertrophy and ligamentum hypertrophy. At L4-L5 level mild spondylolisthesis noted. Spinal canal appeared narrow at L3-L4 and L4-L5 levels.,0,1
192,local_data/mendeley/T1_0194_D3.png,"LSS MRI  Small Rt parantral disc protrusion noted at L4-L5 level, comressing the thecal sac.  adequate spinal canal",0,1
193,local_data/mendeley/T1_0194_D4.png,"LSS MRI  Small Rt parantral disc protrusion noted at L4-L5 level, comressing the thecal sac.  adequate spinal canal",0,1
194,local_data/mendeley/T1_0194_D5.png,"LSS MRI  Small Rt parantral disc protrusion noted at L4-L5 level, comressing the thecal sac.  adequate spinal canal",0,1
195,local_data/mendeley/T2_0194_D3.png,"LSS MRI  Small Rt parantral disc protrusion noted at L4-L5 level, comressing the thecal sac.  adequate spinal canal",0,1
196,local_data/mendeley/T2_0194_D4.png,"LSS MRI  Small Rt parantral disc protrusion noted at L4-L5 level, comressing the thecal sac.  adequate spinal canal",0,1
197,local_data/mendeley/T2_0194_D5.png,"LSS MRI  Small Rt parantral disc protrusion noted at L4-L5 level, comressing the thecal sac.  adequate spinal canal",0,1
198,local_data/mendeley/T1_0213_D3.png,MRI LSS Feature of muscle spasm noted Diffuse disc bulge noted at L5-S1 level mild compressing thecal sac and exiting nerve root Dehydrated  disc material noted at the same level,0,1
199,local_data/mendeley/T1_0213_D4.png,MRI LSS Feature of muscle spasm noted Diffuse disc bulge noted at L5-S1 level mild compressing thecal sac and exiting nerve root Dehydrated  disc material noted at the same level,0,1
200,local_data/mendeley/T1_0213_D5.png,MRI LSS Feature of muscle spasm noted Diffuse disc bulge noted at L5-S1 level mild compressing thecal sac and exiting nerve root Dehydrated  disc material noted at the same level,0,1
201,local_data/mendeley/T2_0213_D3.png,MRI LSS Feature of muscle spasm noted Diffuse disc bulge noted at L5-S1 level mild compressing thecal sac and exiting nerve root Dehydrated  disc material noted at the same level,0,1
202,local_data/mendeley/T2_0213_D4.png,MRI LSS Feature of muscle spasm noted Diffuse disc bulge noted at L5-S1 level mild compressing thecal sac and exiting nerve root Dehydrated  disc material noted at the same level,0,1
203,local_data/mendeley/T2_0213_D5.png,MRI LSS Feature of muscle spasm noted Diffuse disc bulge noted at L5-S1 level mild compressing thecal sac and exiting nerve root Dehydrated  disc material noted at the same level,0,1
204,local_data/mendeley/T1_0222_D3.png,L4-L5 AND L5-S1: diffuse disc bulges compressing the thecal sac and exit canals. L3-L4: Mild disc bulge noted.,0,1
205,local_data/mendeley/T1_0222_D4.png,L4-L5 AND L5-S1: diffuse disc bulges compressing the thecal sac and exit canals. L3-L4: Mild disc bulge noted.,0,1
206,local_data/mendeley/T1_0222_D5.png,L4-L5 AND L5-S1: diffuse disc bulges compressing the thecal sac and exit canals. L3-L4: Mild disc bulge noted.,0,1
207,local_data/mendeley/T2_0222_D3.png,L4-L5 AND L5-S1: diffuse disc bulges compressing the thecal sac and exit canals. L3-L4: Mild disc bulge noted.,0,1
208,local_data/mendeley/T2_0222_D4.png,L4-L5 AND L5-S1: diffuse disc bulges compressing the thecal sac and exit canals. L3-L4: Mild disc bulge noted.,0,1
209,local_data/mendeley/T2_0222_D5.png,L4-L5 AND L5-S1: diffuse disc bulges compressing the thecal sac and exit canals. L3-L4: Mild disc bulge noted.,0,1
210,local_data/mendeley/T1_0223_D3.png,LSS MRI :  Feature of muscel spasm.  Multilevel end plates degenerative changes.  schmorles nodules noted .  No significant disc protrusion or herniation seen.  No thecal sac or nerve roots compression.,0,1
211,local_data/mendeley/T1_0223_D4.png,LSS MRI :  Feature of muscel spasm.  Multilevel end plates degenerative changes.  schmorles nodules noted .  No significant disc protrusion or herniation seen.  No thecal sac or nerve roots compression.,0,1
212,local_data/mendeley/T1_0223_D5.png,LSS MRI :  Feature of muscel spasm.  Multilevel end plates degenerative changes.  schmorles nodules noted .  No significant disc protrusion or herniation seen.  No thecal sac or nerve roots compression.,0,1
213,local_data/mendeley/T2_0223_D3.png,LSS MRI :  Feature of muscel spasm.  Multilevel end plates degenerative changes.  schmorles nodules noted .  No significant disc protrusion or herniation seen.  No thecal sac or nerve roots compression.,0,1
214,local_data/mendeley/T2_0223_D4.png,LSS MRI :  Feature of muscel spasm.  Multilevel end plates degenerative changes.  schmorles nodules noted .  No significant disc protrusion or herniation seen.  No thecal sac or nerve roots compression.,0,1
215,local_data/mendeley/T2_0223_D5.png,LSS MRI :  Feature of muscel spasm.  Multilevel end plates degenerative changes.  schmorles nodules noted .  No significant disc protrusion or herniation seen.  No thecal sac or nerve roots compression.,0,1
216,local_data/mendeley/T1_0229_D3.png,"LSS MRI : Dissicating disc materials . Narrowed disc spaces noted at L2/L3& L3/L4 levels . Grade I spondylolisthesis of L2 over L3 level   Multilevel diffuse disc bulges noticeably at L3/L4 level ,relatively compressing thecal sac and both nerve roots more at Rt sided .",0,1
217,local_data/mendeley/T1_0229_D4.png,"LSS MRI : Dissicating disc materials . Narrowed disc spaces noted at L2/L3& L3/L4 levels . Grade I spondylolisthesis of L2 over L3 level   Multilevel diffuse disc bulges noticeably at L3/L4 level ,relatively compressing thecal sac and both nerve roots more at Rt sided .",0,1
218,local_data/mendeley/T1_0229_D5.png,"LSS MRI : Dissicating disc materials . Narrowed disc spaces noted at L2/L3& L3/L4 levels . Grade I spondylolisthesis of L2 over L3 level   Multilevel diffuse disc bulges noticeably at L3/L4 level ,relatively compressing thecal sac and both nerve roots more at Rt sided .",0,1
219,local_data/mendeley/T2_0229_D3.png,"LSS MRI : Dissicating disc materials . Narrowed disc spaces noted at L2/L3& L3/L4 levels . Grade I spondylolisthesis of L2 over L3 level   Multilevel diffuse disc bulges noticeably at L3/L4 level ,relatively compressing thecal sac and both nerve roots more at Rt sided .",0,1
220,local_data/mendeley/T2_0229_D4.png,"LSS MRI : Dissicating disc materials . Narrowed disc spaces noted at L2/L3& L3/L4 levels . Grade I spondylolisthesis of L2 over L3 level   Multilevel diffuse disc bulges noticeably at L3/L4 level ,relatively compressing thecal sac and both nerve roots more at Rt sided .",0,1
221,local_data/mendeley/T2_0229_D5.png,"LSS MRI : Dissicating disc materials . Narrowed disc spaces noted at L2/L3& L3/L4 levels . Grade I spondylolisthesis of L2 over L3 level   Multilevel diffuse disc bulges noticeably at L3/L4 level ,relatively compressing thecal sac and both nerve roots more at Rt sided .",0,1
222,local_data/mendeley/T1_0241_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots .",0,1
223,local_data/mendeley/T1_0241_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots .",0,1
224,local_data/mendeley/T1_0241_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots .",0,1
225,local_data/mendeley/T2_0241_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots .",0,1
226,local_data/mendeley/T2_0241_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots .",0,1
227,local_data/mendeley/T2_0241_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots .",0,1
228,local_data/mendeley/T1_0245_D3.png,"L4-L5: Central disc protrusion, compressing the thecal sac.",0,1
229,local_data/mendeley/T1_0245_D4.png,"L4-L5: Central disc protrusion, compressing the thecal sac.",0,1
230,local_data/mendeley/T1_0245_D5.png,"L4-L5: Central disc protrusion, compressing the thecal sac.",0,1
231,local_data/mendeley/T2_0245_D3.png,"L4-L5: Central disc protrusion, compressing the thecal sac.",0,1
232,local_data/mendeley/T2_0245_D4.png,"L4-L5: Central disc protrusion, compressing the thecal sac.",0,1
233,local_data/mendeley/T2_0245_D5.png,"L4-L5: Central disc protrusion, compressing the thecal sac.",0,1
234,local_data/mendeley/T1_0248_D3.png,LSS MRI  mild disc bulge noted at L5-S1 level.  No significant thecal sac or nerve root compresion noted.,0,1
235,local_data/mendeley/T1_0248_D4.png,LSS MRI  mild disc bulge noted at L5-S1 level.  No significant thecal sac or nerve root compresion noted.,0,1
236,local_data/mendeley/T1_0248_D5.png,LSS MRI  mild disc bulge noted at L5-S1 level.  No significant thecal sac or nerve root compresion noted.,0,1
237,local_data/mendeley/T2_0248_D3.png,LSS MRI  mild disc bulge noted at L5-S1 level.  No significant thecal sac or nerve root compresion noted.,0,1
238,local_data/mendeley/T2_0248_D4.png,LSS MRI  mild disc bulge noted at L5-S1 level.  No significant thecal sac or nerve root compresion noted.,0,1
239,local_data/mendeley/T2_0248_D5.png,LSS MRI  mild disc bulge noted at L5-S1 level.  No significant thecal sac or nerve root compresion noted.,0,1
240,local_data/mendeley/T1_0260_D3.png,"LSS MRI :  feature of muscle spasm.  Small focal central disc protrusion noted at L5/S1 level, mild compressing thecal sac .",0,1
241,local_data/mendeley/T1_0260_D4.png,"LSS MRI :  feature of muscle spasm.  Small focal central disc protrusion noted at L5/S1 level, mild compressing thecal sac .",0,1
242,local_data/mendeley/T1_0260_D5.png,"LSS MRI :  feature of muscle spasm.  Small focal central disc protrusion noted at L5/S1 level, mild compressing thecal sac .",0,1
243,local_data/mendeley/T2_0260_D3.png,"LSS MRI :  feature of muscle spasm.  Small focal central disc protrusion noted at L5/S1 level, mild compressing thecal sac .",0,1
244,local_data/mendeley/T2_0260_D4.png,"LSS MRI :  feature of muscle spasm.  Small focal central disc protrusion noted at L5/S1 level, mild compressing thecal sac .",0,1
245,local_data/mendeley/T2_0260_D5.png,"LSS MRI :  feature of muscle spasm.  Small focal central disc protrusion noted at L5/S1 level, mild compressing thecal sac .",0,1
246,local_data/mendeley/T1_0298_D3.png,LSS MRI  Feature of muscle spasm.  Mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,0,1
247,local_data/mendeley/T1_0298_D4.png,LSS MRI  Feature of muscle spasm.  Mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,0,1
248,local_data/mendeley/T1_0298_D5.png,LSS MRI  Feature of muscle spasm.  Mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,0,1
249,local_data/mendeley/T2_0298_D3.png,LSS MRI  Feature of muscle spasm.  Mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,0,1
250,local_data/mendeley/T2_0298_D4.png,LSS MRI  Feature of muscle spasm.  Mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,0,1
251,local_data/mendeley/T2_0298_D5.png,LSS MRI  Feature of muscle spasm.  Mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,0,1
252,local_data/mendeley/T1_0301_D3.png,LSS MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,0,1
253,local_data/mendeley/T1_0301_D4.png,LSS MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,0,1
254,local_data/mendeley/T1_0301_D5.png,LSS MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,0,1
255,local_data/mendeley/T2_0301_D3.png,LSS MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,0,1
256,local_data/mendeley/T2_0301_D4.png,LSS MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,0,1
257,local_data/mendeley/T2_0301_D5.png,LSS MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,0,1
258,local_data/mendeley/T1_0304_D3.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
259,local_data/mendeley/T1_0304_D4.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
260,local_data/mendeley/T1_0304_D5.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
261,local_data/mendeley/T2_0304_D3.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
262,local_data/mendeley/T2_0304_D4.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
263,local_data/mendeley/T2_0304_D5.png,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0
264,local_data/mendeley/T1_0305_D3.png,"LSS MRI :  Central and Lt paracentral disc protrusion noted at L5/S1 level , largely compressing the thecal sac and S1 Lt exit nerve root.  NB:  small tarlov cyst noted at S2 level .",0,1
265,local_data/mendeley/T1_0305_D4.png,"LSS MRI :  Central and Lt paracentral disc protrusion noted at L5/S1 level , largely compressing the thecal sac and S1 Lt exit nerve root.  NB:  small tarlov cyst noted at S2 level .",0,1
266,local_data/mendeley/T1_0305_D5.png,"LSS MRI :  Central and Lt paracentral disc protrusion noted at L5/S1 level , largely compressing the thecal sac and S1 Lt exit nerve root.  NB:  small tarlov cyst noted at S2 level .",0,1
267,local_data/mendeley/T2_0305_D3.png,"LSS MRI :  Central and Lt paracentral disc protrusion noted at L5/S1 level , largely compressing the thecal sac and S1 Lt exit nerve root.  NB:  small tarlov cyst noted at S2 level .",0,1
268,local_data/mendeley/T2_0305_D4.png,"LSS MRI :  Central and Lt paracentral disc protrusion noted at L5/S1 level , largely compressing the thecal sac and S1 Lt exit nerve root.  NB:  small tarlov cyst noted at S2 level .",0,1
269,local_data/mendeley/T2_0305_D5.png,"LSS MRI :  Central and Lt paracentral disc protrusion noted at L5/S1 level , largely compressing the thecal sac and S1 Lt exit nerve root.  NB:  small tarlov cyst noted at S2 level .",0,1
270,local_data/mendeley/T1_0311_D3.png,LSS MRI : Feature of muscle spasm. Adequate  thecal sac. No significant  disc protrusion or herniation seen .,0,1
271,local_data/mendeley/T1_0311_D4.png,LSS MRI : Feature of muscle spasm. Adequate  thecal sac. No significant  disc protrusion or herniation seen .,0,1
272,local_data/mendeley/T1_0311_D5.png,LSS MRI : Feature of muscle spasm. Adequate  thecal sac. No significant  disc protrusion or herniation seen .,0,1
273,local_data/mendeley/T2_0311_D3.png,LSS MRI : Feature of muscle spasm. Adequate  thecal sac. No significant  disc protrusion or herniation seen .,0,1
274,local_data/mendeley/T2_0311_D4.png,LSS MRI : Feature of muscle spasm. Adequate  thecal sac. No significant  disc protrusion or herniation seen .,0,1
275,local_data/mendeley/T2_0311_D5.png,LSS MRI : Feature of muscle spasm. Adequate  thecal sac. No significant  disc protrusion or herniation seen .,0,1
276,local_data/mendeley/T1_0316_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 & L5/S1 levels , mildly compressing the thecal sac and encroaching both nerve roots .",0,1
277,local_data/mendeley/T1_0316_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 & L5/S1 levels , mildly compressing the thecal sac and encroaching both nerve roots .",0,1
278,local_data/mendeley/T1_0316_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 & L5/S1 levels , mildly compressing the thecal sac and encroaching both nerve roots .",0,1
279,local_data/mendeley/T2_0316_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 & L5/S1 levels , mildly compressing the thecal sac and encroaching both nerve roots .",0,1
280,local_data/mendeley/T2_0316_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 & L5/S1 levels , mildly compressing the thecal sac and encroaching both nerve roots .",0,1
281,local_data/mendeley/T2_0316_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 & L5/S1 levels , mildly compressing the thecal sac and encroaching both nerve roots .",0,1
282,local_data/mendeley/T1_0326_D3.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals. L3-L4: Diffuse disc bulge noted abutting the thecal sac",0,1
283,local_data/mendeley/T1_0326_D4.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals. L3-L4: Diffuse disc bulge noted abutting the thecal sac",0,1
284,local_data/mendeley/T1_0326_D5.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals. L3-L4: Diffuse disc bulge noted abutting the thecal sac",0,1
285,local_data/mendeley/T2_0326_D3.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals. L3-L4: Diffuse disc bulge noted abutting the thecal sac",0,1
286,local_data/mendeley/T2_0326_D4.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals. L3-L4: Diffuse disc bulge noted abutting the thecal sac",0,1
287,local_data/mendeley/T2_0326_D5.png,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals. L3-L4: Diffuse disc bulge noted abutting the thecal sac",0,1
288,local_data/mendeley/T1_0345_D3.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, abitting thecal sac.",0,1
289,local_data/mendeley/T1_0345_D4.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, abitting thecal sac.",0,1
290,local_data/mendeley/T1_0345_D5.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, abitting thecal sac.",0,1
291,local_data/mendeley/T2_0345_D3.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, abitting thecal sac.",0,1
292,local_data/mendeley/T2_0345_D4.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, abitting thecal sac.",0,1
293,local_data/mendeley/T2_0345_D5.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, abitting thecal sac.",0,1
294,local_data/mendeley/T1_0350_D3.png,L5-S1: diffuse disc bulge compressing the thecal sac and exit canals.,0,1
295,local_data/mendeley/T1_0350_D4.png,L5-S1: diffuse disc bulge compressing the thecal sac and exit canals.,0,1
296,local_data/mendeley/T1_0350_D5.png,L5-S1: diffuse disc bulge compressing the thecal sac and exit canals.,0,1
297,local_data/mendeley/T2_0350_D3.png,L5-S1: diffuse disc bulge compressing the thecal sac and exit canals.,0,1
298,local_data/mendeley/T2_0350_D4.png,L5-S1: diffuse disc bulge compressing the thecal sac and exit canals.,0,1
299,local_data/mendeley/T2_0350_D5.png,L5-S1: diffuse disc bulge compressing the thecal sac and exit canals.,0,1
300,local_data/mendeley/T1_0353_D3.png,"LSS MRI  L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals.  L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
301,local_data/mendeley/T1_0353_D4.png,"LSS MRI  L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals.  L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
302,local_data/mendeley/T1_0353_D5.png,"LSS MRI  L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals.  L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
303,local_data/mendeley/T2_0353_D3.png,"LSS MRI  L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals.  L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
304,local_data/mendeley/T2_0353_D4.png,"LSS MRI  L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals.  L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
305,local_data/mendeley/T2_0353_D5.png,"LSS MRI  L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals.  L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
306,local_data/mendeley/T1_0358_D3.png,Lumbosacral MRI  Features of muscle spasm  Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
307,local_data/mendeley/T1_0358_D4.png,Lumbosacral MRI  Features of muscle spasm  Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
308,local_data/mendeley/T1_0358_D5.png,Lumbosacral MRI  Features of muscle spasm  Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
309,local_data/mendeley/T2_0358_D3.png,Lumbosacral MRI  Features of muscle spasm  Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
310,local_data/mendeley/T2_0358_D4.png,Lumbosacral MRI  Features of muscle spasm  Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
311,local_data/mendeley/T2_0358_D5.png,Lumbosacral MRI  Features of muscle spasm  Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
312,local_data/mendeley/T1_0378_D3.png,MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
313,local_data/mendeley/T1_0378_D4.png,MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
314,local_data/mendeley/T1_0378_D5.png,MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
315,local_data/mendeley/T2_0378_D3.png,MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
316,local_data/mendeley/T2_0378_D4.png,MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
317,local_data/mendeley/T2_0378_D5.png,MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
318,local_data/mendeley/T1_0379_D3.png,"LSS MRI :  Feature of muscle spasm.  Vertebral bodies hemangioma noted at L2&L3 .  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
319,local_data/mendeley/T1_0379_D4.png,"LSS MRI :  Feature of muscle spasm.  Vertebral bodies hemangioma noted at L2&L3 .  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
320,local_data/mendeley/T1_0379_D5.png,"LSS MRI :  Feature of muscle spasm.  Vertebral bodies hemangioma noted at L2&L3 .  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
321,local_data/mendeley/T2_0379_D3.png,"LSS MRI :  Feature of muscle spasm.  Vertebral bodies hemangioma noted at L2&L3 .  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
322,local_data/mendeley/T2_0379_D4.png,"LSS MRI :  Feature of muscle spasm.  Vertebral bodies hemangioma noted at L2&L3 .  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
323,local_data/mendeley/T2_0379_D5.png,"LSS MRI :  Feature of muscle spasm.  Vertebral bodies hemangioma noted at L2&L3 .  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
324,local_data/mendeley/T1_0394_D3.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
325,local_data/mendeley/T1_0394_D4.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
326,local_data/mendeley/T1_0394_D5.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
327,local_data/mendeley/T2_0394_D3.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
328,local_data/mendeley/T2_0394_D4.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
329,local_data/mendeley/T2_0394_D5.png,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0
330,local_data/mendeley/T1_0395_D3.png,LSS MRI  Multilevel mild disc bulges noted.  Adequate spinal canal.  No significant thecal sac or nerve root compression noted.,1,0
331,local_data/mendeley/T1_0395_D4.png,LSS MRI  Multilevel mild disc bulges noted.  Adequate spinal canal.  No significant thecal sac or nerve root compression noted.,1,0
332,local_data/mendeley/T1_0395_D5.png,LSS MRI  Multilevel mild disc bulges noted.  Adequate spinal canal.  No significant thecal sac or nerve root compression noted.,1,0
333,local_data/mendeley/T2_0395_D3.png,LSS MRI  Multilevel mild disc bulges noted.  Adequate spinal canal.  No significant thecal sac or nerve root compression noted.,1,0
334,local_data/mendeley/T2_0395_D4.png,LSS MRI  Multilevel mild disc bulges noted.  Adequate spinal canal.  No significant thecal sac or nerve root compression noted.,1,0
335,local_data/mendeley/T2_0395_D5.png,LSS MRI  Multilevel mild disc bulges noted.  Adequate spinal canal.  No significant thecal sac or nerve root compression noted.,1,0
336,local_data/mendeley/T1_0401_D3.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted.  multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots .  scoliosis of Lumbar spine noted, convexity to Ltside",0,1
337,local_data/mendeley/T1_0401_D4.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted.  multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots .  scoliosis of Lumbar spine noted, convexity to Ltside",0,1
338,local_data/mendeley/T1_0401_D5.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted.  multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots .  scoliosis of Lumbar spine noted, convexity to Ltside",0,1
339,local_data/mendeley/T2_0401_D3.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted.  multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots .  scoliosis of Lumbar spine noted, convexity to Ltside",0,1
340,local_data/mendeley/T2_0401_D4.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted.  multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots .  scoliosis of Lumbar spine noted, convexity to Ltside",0,1
341,local_data/mendeley/T2_0401_D5.png,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted.  multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots .  scoliosis of Lumbar spine noted, convexity to Ltside",0,1
342,local_data/mendeley/T1_0353_D3.png,"LSS MRI L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals. L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
343,local_data/mendeley/T1_0353_D4.png,"LSS MRI L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals. L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
344,local_data/mendeley/T1_0353_D5.png,"LSS MRI L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals. L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
345,local_data/mendeley/T2_0353_D3.png,"LSS MRI L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals. L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
346,local_data/mendeley/T2_0353_D4.png,"LSS MRI L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals. L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
347,local_data/mendeley/T2_0353_D5.png,"LSS MRI L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals. L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1
348,local_data/mendeley/T1_0358_D3.png,Lumbosacral MRI Features of muscle spasm Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
349,local_data/mendeley/T1_0358_D4.png,Lumbosacral MRI Features of muscle spasm Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
350,local_data/mendeley/T1_0358_D5.png,Lumbosacral MRI Features of muscle spasm Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
351,local_data/mendeley/T2_0358_D3.png,Lumbosacral MRI Features of muscle spasm Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
352,local_data/mendeley/T2_0358_D4.png,Lumbosacral MRI Features of muscle spasm Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
353,local_data/mendeley/T2_0358_D5.png,Lumbosacral MRI Features of muscle spasm Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1
354,local_data/mendeley/T1_0378_D3.png,MRI OF THE LUMBOSACRAL SPINE Feature of muscle spasm. Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
355,local_data/mendeley/T1_0378_D4.png,MRI OF THE LUMBOSACRAL SPINE Feature of muscle spasm. Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
356,local_data/mendeley/T1_0378_D5.png,MRI OF THE LUMBOSACRAL SPINE Feature of muscle spasm. Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
357,local_data/mendeley/T2_0378_D3.png,MRI OF THE LUMBOSACRAL SPINE Feature of muscle spasm. Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
358,local_data/mendeley/T2_0378_D4.png,MRI OF THE LUMBOSACRAL SPINE Feature of muscle spasm. Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
359,local_data/mendeley/T2_0378_D5.png,MRI OF THE LUMBOSACRAL SPINE Feature of muscle spasm. Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1
360,local_data/mendeley/T1_0379_D3.png,"LSS MRI : Feature of muscle spasm. Vertebral bodies hemangioma noted at L2&L3 . Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
361,local_data/mendeley/T1_0379_D4.png,"LSS MRI : Feature of muscle spasm. Vertebral bodies hemangioma noted at L2&L3 . Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
362,local_data/mendeley/T1_0379_D5.png,"LSS MRI : Feature of muscle spasm. Vertebral bodies hemangioma noted at L2&L3 . Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
363,local_data/mendeley/T2_0379_D3.png,"LSS MRI : Feature of muscle spasm. Vertebral bodies hemangioma noted at L2&L3 . Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
364,local_data/mendeley/T2_0379_D4.png,"LSS MRI : Feature of muscle spasm. Vertebral bodies hemangioma noted at L2&L3 . Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
365,local_data/mendeley/T2_0379_D5.png,"LSS MRI : Feature of muscle spasm. Vertebral bodies hemangioma noted at L2&L3 . Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1
366,local_data/mendeley/T1_0394_D3.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
367,local_data/mendeley/T1_0394_D4.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
368,local_data/mendeley/T1_0394_D5.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
369,local_data/mendeley/T2_0394_D3.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
370,local_data/mendeley/T2_0394_D4.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
371,local_data/mendeley/T2_0394_D5.png,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0
372,local_data/mendeley/T1_0395_D3.png,LSS MRI Multilevel mild disc bulges noted. Adequate spinal canal. No significant thecal sac or nerve root compression noted.,1,0
373,local_data/mendeley/T1_0395_D4.png,LSS MRI Multilevel mild disc bulges noted. Adequate spinal canal. No significant thecal sac or nerve root compression noted.,1,0
374,local_data/mendeley/T1_0395_D5.png,LSS MRI Multilevel mild disc bulges noted. Adequate spinal canal. No significant thecal sac or nerve root compression noted.,1,0
375,local_data/mendeley/T2_0395_D3.png,LSS MRI Multilevel mild disc bulges noted. Adequate spinal canal. No significant thecal sac or nerve root compression noted.,1,0
376,local_data/mendeley/T2_0395_D4.png,LSS MRI Multilevel mild disc bulges noted. Adequate spinal canal. No significant thecal sac or nerve root compression noted.,1,0
377,local_data/mendeley/T2_0395_D5.png,LSS MRI Multilevel mild disc bulges noted. Adequate spinal canal. No significant thecal sac or nerve root compression noted.,1,0
378,local_data/mendeley/T1_0401_D3.png,"LSS MRI : Feature of muscle spasm. Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted. multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots . scoliosis of Lumbar spine noted, convexity to Ltside",0,1
379,local_data/mendeley/T1_0401_D4.png,"LSS MRI : Feature of muscle spasm. Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted. multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots . scoliosis of Lumbar spine noted, convexity to Ltside",0,1
380,local_data/mendeley/T1_0401_D5.png,"LSS MRI : Feature of muscle spasm. Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted. multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots . scoliosis of Lumbar spine noted, convexity to Ltside",0,1
381,local_data/mendeley/T2_0401_D3.png,"LSS MRI : Feature of muscle spasm. Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted. multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots . scoliosis of Lumbar spine noted, convexity to Ltside",0,1
382,local_data/mendeley/T2_0401_D4.png,"LSS MRI : Feature of muscle spasm. Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted. multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots . scoliosis of Lumbar spine noted, convexity to Ltside",0,1
383,local_data/mendeley/T2_0401_D5.png,"LSS MRI : Feature of muscle spasm. Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted. multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots . scoliosis of Lumbar spine noted, convexity to Ltside",0,1
384,local_data/mendeley/T1_0433_D3.png,"LSS MRI :  faeture of muscle spasm.  wide base disc bulge ,with annular tear noted at L5/S1 level, compressing teh thecal sac and both nerve roots",0,1
385,local_data/mendeley/T1_0433_D4.png,"LSS MRI :  faeture of muscle spasm.  wide base disc bulge ,with annular tear noted at L5/S1 level, compressing teh thecal sac and both nerve roots",0,1
386,local_data/mendeley/T1_0433_D5.png,"LSS MRI :  faeture of muscle spasm.  wide base disc bulge ,with annular tear noted at L5/S1 level, compressing teh thecal sac and both nerve roots",0,1
387,local_data/mendeley/T2_0433_D3.png,"LSS MRI :  faeture of muscle spasm.  wide base disc bulge ,with annular tear noted at L5/S1 level, compressing teh thecal sac and both nerve roots",0,1
388,local_data/mendeley/T2_0433_D4.png,"LSS MRI :  faeture of muscle spasm.  wide base disc bulge ,with annular tear noted at L5/S1 level, compressing teh thecal sac and both nerve roots",0,1
389,local_data/mendeley/T2_0433_D5.png,"LSS MRI :  faeture of muscle spasm.  wide base disc bulge ,with annular tear noted at L5/S1 level, compressing teh thecal sac and both nerve roots",0,1
390,local_data/mendeley/T1_0436_D3.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing the thecal sac and encroaching exit neural canals .",0,1
391,local_data/mendeley/T1_0436_D4.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing the thecal sac and encroaching exit neural canals .",0,1
392,local_data/mendeley/T1_0436_D5.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing the thecal sac and encroaching exit neural canals .",0,1
393,local_data/mendeley/T2_0436_D3.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing the thecal sac and encroaching exit neural canals .",0,1
394,local_data/mendeley/T2_0436_D4.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing the thecal sac and encroaching exit neural canals .",0,1
395,local_data/mendeley/T2_0436_D5.png,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing the thecal sac and encroaching exit neural canals .",0,1
396,local_data/mendeley/T1_0438_D3.png,"LSS MRI  Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and encroaching upon exit canals.  mild disc bulge noted at L5-S1 level.",0,1
397,local_data/mendeley/T1_0438_D4.png,"LSS MRI  Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and encroaching upon exit canals.  mild disc bulge noted at L5-S1 level.",0,1
398,local_data/mendeley/T1_0438_D5.png,"LSS MRI  Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and encroaching upon exit canals.  mild disc bulge noted at L5-S1 level.",0,1
399,local_data/mendeley/T2_0438_D3.png,"LSS MRI  Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and encroaching upon exit canals.  mild disc bulge noted at L5-S1 level.",0,1
400,local_data/mendeley/T2_0438_D4.png,"LSS MRI  Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and encroaching upon exit canals.  mild disc bulge noted at L5-S1 level.",0,1
401,local_data/mendeley/T2_0438_D5.png,"LSS MRI  Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and encroaching upon exit canals.  mild disc bulge noted at L5-S1 level.",0,1
402,local_data/mendeley/T1_0441_D3.png,"LSS MRI  L4-L5: diffuse disc bulge, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L5-S1: Mild disc bulge noted.",0,1
403,local_data/mendeley/T1_0441_D4.png,"LSS MRI  L4-L5: diffuse disc bulge, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L5-S1: Mild disc bulge noted.",0,1
404,local_data/mendeley/T1_0441_D5.png,"LSS MRI  L4-L5: diffuse disc bulge, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L5-S1: Mild disc bulge noted.",0,1
405,local_data/mendeley/T2_0441_D3.png,"LSS MRI  L4-L5: diffuse disc bulge, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L5-S1: Mild disc bulge noted.",0,1
406,local_data/mendeley/T2_0441_D4.png,"LSS MRI  L4-L5: diffuse disc bulge, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L5-S1: Mild disc bulge noted.",0,1
407,local_data/mendeley/T2_0441_D5.png,"LSS MRI  L4-L5: diffuse disc bulge, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L5-S1: Mild disc bulge noted.",0,1
408,local_data/mendeley/T1_0464_D3.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
409,local_data/mendeley/T1_0464_D4.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
410,local_data/mendeley/T1_0464_D5.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
411,local_data/mendeley/T2_0464_D3.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
412,local_data/mendeley/T2_0464_D4.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
413,local_data/mendeley/T2_0464_D5.png,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0
414,local_data/mendeley/T1_0472_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, compressing the thecal sac and encroaching both exit neural canals",0,1
415,local_data/mendeley/T1_0472_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, compressing the thecal sac and encroaching both exit neural canals",0,1
416,local_data/mendeley/T1_0472_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, compressing the thecal sac and encroaching both exit neural canals",0,1
417,local_data/mendeley/T2_0472_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, compressing the thecal sac and encroaching both exit neural canals",0,1
418,local_data/mendeley/T2_0472_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, compressing the thecal sac and encroaching both exit neural canals",0,1
419,local_data/mendeley/T2_0472_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, compressing the thecal sac and encroaching both exit neural canals",0,1
420,local_data/mendeley/T1_0492_D3.png,"LSS MRI  diffuse disc bulge, more to the Rt side, noted at L4-L5 level, compressing the thecal sac and Rt exit neural canal.",0,1
421,local_data/mendeley/T1_0492_D4.png,"LSS MRI  diffuse disc bulge, more to the Rt side, noted at L4-L5 level, compressing the thecal sac and Rt exit neural canal.",0,1
422,local_data/mendeley/T1_0492_D5.png,"LSS MRI  diffuse disc bulge, more to the Rt side, noted at L4-L5 level, compressing the thecal sac and Rt exit neural canal.",0,1
423,local_data/mendeley/T2_0492_D3.png,"LSS MRI  diffuse disc bulge, more to the Rt side, noted at L4-L5 level, compressing the thecal sac and Rt exit neural canal.",0,1
424,local_data/mendeley/T2_0492_D4.png,"LSS MRI  diffuse disc bulge, more to the Rt side, noted at L4-L5 level, compressing the thecal sac and Rt exit neural canal.",0,1
425,local_data/mendeley/T2_0492_D5.png,"LSS MRI  diffuse disc bulge, more to the Rt side, noted at L4-L5 level, compressing the thecal sac and Rt exit neural canal.",0,1
426,local_data/mendeley/T1_0495_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Diffuse disc bulge noted at L5/S1 level, extending to lateral recess, more to Lt the sided the thecal sac and encraoching upon exit neural canals.  -Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and nerve roots associated with annular tear",0,1
427,local_data/mendeley/T1_0495_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Diffuse disc bulge noted at L5/S1 level, extending to lateral recess, more to Lt the sided the thecal sac and encraoching upon exit neural canals.  -Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and nerve roots associated with annular tear",0,1
428,local_data/mendeley/T1_0495_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Diffuse disc bulge noted at L5/S1 level, extending to lateral recess, more to Lt the sided the thecal sac and encraoching upon exit neural canals.  -Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and nerve roots associated with annular tear",0,1
429,local_data/mendeley/T2_0495_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Diffuse disc bulge noted at L5/S1 level, extending to lateral recess, more to Lt the sided the thecal sac and encraoching upon exit neural canals.  -Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and nerve roots associated with annular tear",0,1
430,local_data/mendeley/T2_0495_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Diffuse disc bulge noted at L5/S1 level, extending to lateral recess, more to Lt the sided the thecal sac and encraoching upon exit neural canals.  -Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and nerve roots associated with annular tear",0,1
431,local_data/mendeley/T2_0495_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Diffuse disc bulge noted at L5/S1 level, extending to lateral recess, more to Lt the sided the thecal sac and encraoching upon exit neural canals.  -Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and nerve roots associated with annular tear",0,1
432,local_data/mendeley/T1_0497_D3.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material at L5/S1 level .  Diffuse disc bulge noted at L5/S1 level, compressing thecal sac and encroaching both exit neural canal.  Mild diffuse disc bulge noted at L4/L5 level mild comprssing thecal sac & nerve roots .",0,1
433,local_data/mendeley/T1_0497_D4.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material at L5/S1 level .  Diffuse disc bulge noted at L5/S1 level, compressing thecal sac and encroaching both exit neural canal.  Mild diffuse disc bulge noted at L4/L5 level mild comprssing thecal sac & nerve roots .",0,1
434,local_data/mendeley/T1_0497_D5.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material at L5/S1 level .  Diffuse disc bulge noted at L5/S1 level, compressing thecal sac and encroaching both exit neural canal.  Mild diffuse disc bulge noted at L4/L5 level mild comprssing thecal sac & nerve roots .",0,1
435,local_data/mendeley/T2_0497_D3.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material at L5/S1 level .  Diffuse disc bulge noted at L5/S1 level, compressing thecal sac and encroaching both exit neural canal.  Mild diffuse disc bulge noted at L4/L5 level mild comprssing thecal sac & nerve roots .",0,1
436,local_data/mendeley/T2_0497_D4.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material at L5/S1 level .  Diffuse disc bulge noted at L5/S1 level, compressing thecal sac and encroaching both exit neural canal.  Mild diffuse disc bulge noted at L4/L5 level mild comprssing thecal sac & nerve roots .",0,1
437,local_data/mendeley/T2_0497_D5.png,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material at L5/S1 level .  Diffuse disc bulge noted at L5/S1 level, compressing thecal sac and encroaching both exit neural canal.  Mild diffuse disc bulge noted at L4/L5 level mild comprssing thecal sac & nerve roots .",0,1
438,local_data/mendeley/T1_0503_D3.png,"LSS MRI : .Feature of muscle spasm. .dehydrated disc materials noted at lower levels . .wide base posterior central disc bulge with annular tear noted at L4/L5 level, extending to Lt lateral recess, compressing the thecal sac and encroaching exit neural canal. .Mild diffuse dic bulges noted at L3/L4 & L5/mpressing the thecal sac",0,1
439,local_data/mendeley/T1_0503_D4.png,"LSS MRI : .Feature of muscle spasm. .dehydrated disc materials noted at lower levels . .wide base posterior central disc bulge with annular tear noted at L4/L5 level, extending to Lt lateral recess, compressing the thecal sac and encroaching exit neural canal. .Mild diffuse dic bulges noted at L3/L4 & L5/mpressing the thecal sac",0,1
440,local_data/mendeley/T1_0503_D5.png,"LSS MRI : .Feature of muscle spasm. .dehydrated disc materials noted at lower levels . .wide base posterior central disc bulge with annular tear noted at L4/L5 level, extending to Lt lateral recess, compressing the thecal sac and encroaching exit neural canal. .Mild diffuse dic bulges noted at L3/L4 & L5/mpressing the thecal sac",0,1
441,local_data/mendeley/T2_0503_D3.png,"LSS MRI : .Feature of muscle spasm. .dehydrated disc materials noted at lower levels . .wide base posterior central disc bulge with annular tear noted at L4/L5 level, extending to Lt lateral recess, compressing the thecal sac and encroaching exit neural canal. .Mild diffuse dic bulges noted at L3/L4 & L5/mpressing the thecal sac",0,1
442,local_data/mendeley/T2_0503_D4.png,"LSS MRI : .Feature of muscle spasm. .dehydrated disc materials noted at lower levels . .wide base posterior central disc bulge with annular tear noted at L4/L5 level, extending to Lt lateral recess, compressing the thecal sac and encroaching exit neural canal. .Mild diffuse dic bulges noted at L3/L4 & L5/mpressing the thecal sac",0,1
443,local_data/mendeley/T2_0503_D5.png,"LSS MRI : .Feature of muscle spasm. .dehydrated disc materials noted at lower levels . .wide base posterior central disc bulge with annular tear noted at L4/L5 level, extending to Lt lateral recess, compressing the thecal sac and encroaching exit neural canal. .Mild diffuse dic bulges noted at L3/L4 & L5/mpressing the thecal sac",0,1
444,local_data/mendeley/T1_0507_D3.png,"Lumbosacral MRI Central disc protrusion and more to the right side, noted at L4-L5 level, largely compressing the thecal sac and right exit neural canal Left paracentral disc protrusion noted at L2-L3 level, compressing the left side of the thecal sac",0,1
445,local_data/mendeley/T1_0507_D4.png,"Lumbosacral MRI Central disc protrusion and more to the right side, noted at L4-L5 level, largely compressing the thecal sac and right exit neural canal Left paracentral disc protrusion noted at L2-L3 level, compressing the left side of the thecal sac",0,1
446,local_data/mendeley/T1_0507_D5.png,"Lumbosacral MRI Central disc protrusion and more to the right side, noted at L4-L5 level, largely compressing the thecal sac and right exit neural canal Left paracentral disc protrusion noted at L2-L3 level, compressing the left side of the thecal sac",0,1
447,local_data/mendeley/T2_0507_D3.png,"Lumbosacral MRI Central disc protrusion and more to the right side, noted at L4-L5 level, largely compressing the thecal sac and right exit neural canal Left paracentral disc protrusion noted at L2-L3 level, compressing the left side of the thecal sac",0,1
448,local_data/mendeley/T2_0507_D4.png,"Lumbosacral MRI Central disc protrusion and more to the right side, noted at L4-L5 level, largely compressing the thecal sac and right exit neural canal Left paracentral disc protrusion noted at L2-L3 level, compressing the left side of the thecal sac",0,1
449,local_data/mendeley/T2_0507_D5.png,"Lumbosacral MRI Central disc protrusion and more to the right side, noted at L4-L5 level, largely compressing the thecal sac and right exit neural canal Left paracentral disc protrusion noted at L2-L3 level, compressing the left side of the thecal sac",0,1
450,local_data/mendeley/T1_0509_D3.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials at L4/L5 & L5/S1 levels Diffuse disc bulge noted at L4/L5 level, compressing the theca sac and encroaching upon exit neural canal , with annular tear . -Small posterior central disc bulge noted at L5/S1 level, mildly compressing the thecal sac and exit nerve roots",0,1
451,local_data/mendeley/T1_0509_D4.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials at L4/L5 & L5/S1 levels Diffuse disc bulge noted at L4/L5 level, compressing the theca sac and encroaching upon exit neural canal , with annular tear . -Small posterior central disc bulge noted at L5/S1 level, mildly compressing the thecal sac and exit nerve roots",0,1
452,local_data/mendeley/T1_0509_D5.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials at L4/L5 & L5/S1 levels Diffuse disc bulge noted at L4/L5 level, compressing the theca sac and encroaching upon exit neural canal , with annular tear . -Small posterior central disc bulge noted at L5/S1 level, mildly compressing the thecal sac and exit nerve roots",0,1
453,local_data/mendeley/T2_0509_D3.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials at L4/L5 & L5/S1 levels Diffuse disc bulge noted at L4/L5 level, compressing the theca sac and encroaching upon exit neural canal , with annular tear . -Small posterior central disc bulge noted at L5/S1 level, mildly compressing the thecal sac and exit nerve roots",0,1
454,local_data/mendeley/T2_0509_D4.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials at L4/L5 & L5/S1 levels Diffuse disc bulge noted at L4/L5 level, compressing the theca sac and encroaching upon exit neural canal , with annular tear . -Small posterior central disc bulge noted at L5/S1 level, mildly compressing the thecal sac and exit nerve roots",0,1
455,local_data/mendeley/T2_0509_D5.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials at L4/L5 & L5/S1 levels Diffuse disc bulge noted at L4/L5 level, compressing the theca sac and encroaching upon exit neural canal , with annular tear . -Small posterior central disc bulge noted at L5/S1 level, mildly compressing the thecal sac and exit nerve roots",0,1
456,local_data/mendeley/T1_0521_D3.png,LSS AND PELVIS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted. preserved SI joints. features of PID noted.,1,0
457,local_data/mendeley/T1_0521_D4.png,LSS AND PELVIS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted. preserved SI joints. features of PID noted.,1,0
458,local_data/mendeley/T1_0521_D5.png,LSS AND PELVIS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted. preserved SI joints. features of PID noted.,1,0
459,local_data/mendeley/T2_0521_D3.png,LSS AND PELVIS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted. preserved SI joints. features of PID noted.,1,0
460,local_data/mendeley/T2_0521_D4.png,LSS AND PELVIS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted. preserved SI joints. features of PID noted.,1,0
461,local_data/mendeley/T2_0521_D5.png,LSS AND PELVIS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted. preserved SI joints. features of PID noted.,1,0
462,local_data/mendeley/T1_0522_D3.png,"LSS MRI L4-L5: Diffuse disc bulge noted, compressing the thecal nsac and both exit canals. L5-S1: Mild disc bulge noted",0,1
463,local_data/mendeley/T1_0522_D4.png,"LSS MRI L4-L5: Diffuse disc bulge noted, compressing the thecal nsac and both exit canals. L5-S1: Mild disc bulge noted",0,1
464,local_data/mendeley/T1_0522_D5.png,"LSS MRI L4-L5: Diffuse disc bulge noted, compressing the thecal nsac and both exit canals. L5-S1: Mild disc bulge noted",0,1
465,local_data/mendeley/T2_0522_D3.png,"LSS MRI L4-L5: Diffuse disc bulge noted, compressing the thecal nsac and both exit canals. L5-S1: Mild disc bulge noted",0,1
466,local_data/mendeley/T2_0522_D4.png,"LSS MRI L4-L5: Diffuse disc bulge noted, compressing the thecal nsac and both exit canals. L5-S1: Mild disc bulge noted",0,1
467,local_data/mendeley/T2_0522_D5.png,"LSS MRI L4-L5: Diffuse disc bulge noted, compressing the thecal nsac and both exit canals. L5-S1: Mild disc bulge noted",0,1
468,local_data/mendeley/T1_0529_D3.png,C4-C5 mild disc bulge noted compressing anterior CSF space with features iof m. spasm. L4-L5: Diffuse disc bulge compressing the thecal sac and exit canals.,0,1
469,local_data/mendeley/T1_0529_D4.png,C4-C5 mild disc bulge noted compressing anterior CSF space with features iof m. spasm. L4-L5: Diffuse disc bulge compressing the thecal sac and exit canals.,0,1
470,local_data/mendeley/T1_0529_D5.png,C4-C5 mild disc bulge noted compressing anterior CSF space with features iof m. spasm. L4-L5: Diffuse disc bulge compressing the thecal sac and exit canals.,0,1
471,local_data/mendeley/T2_0529_D3.png,C4-C5 mild disc bulge noted compressing anterior CSF space with features iof m. spasm. L4-L5: Diffuse disc bulge compressing the thecal sac and exit canals.,0,1
472,local_data/mendeley/T2_0529_D4.png,C4-C5 mild disc bulge noted compressing anterior CSF space with features iof m. spasm. L4-L5: Diffuse disc bulge compressing the thecal sac and exit canals.,0,1
473,local_data/mendeley/T2_0529_D5.png,C4-C5 mild disc bulge noted compressing anterior CSF space with features iof m. spasm. L4-L5: Diffuse disc bulge compressing the thecal sac and exit canals.,0,1
474,local_data/mendeley/T1_0541_D3.png,Lumbosacral MRI Mild disc bulge at the L5-S1 level Abutting the thecal sac No significant thecal sac or nerve root compression noted,0,1
475,local_data/mendeley/T1_0541_D4.png,Lumbosacral MRI Mild disc bulge at the L5-S1 level Abutting the thecal sac No significant thecal sac or nerve root compression noted,0,1
476,local_data/mendeley/T1_0541_D5.png,Lumbosacral MRI Mild disc bulge at the L5-S1 level Abutting the thecal sac No significant thecal sac or nerve root compression noted,0,1
477,local_data/mendeley/T2_0541_D3.png,Lumbosacral MRI Mild disc bulge at the L5-S1 level Abutting the thecal sac No significant thecal sac or nerve root compression noted,0,1
478,local_data/mendeley/T2_0541_D4.png,Lumbosacral MRI Mild disc bulge at the L5-S1 level Abutting the thecal sac No significant thecal sac or nerve root compression noted,0,1
479,local_data/mendeley/T2_0541_D5.png,Lumbosacral MRI Mild disc bulge at the L5-S1 level Abutting the thecal sac No significant thecal sac or nerve root compression noted,0,1
480,local_data/mendeley/T1_0552_D3.png,Mild disc bulge noted at L4-L5 and L1-L2 level. No significant thecal sac or nerve root compression noted.,0,1
481,local_data/mendeley/T1_0552_D4.png,Mild disc bulge noted at L4-L5 and L1-L2 level. No significant thecal sac or nerve root compression noted.,0,1
482,local_data/mendeley/T1_0552_D5.png,Mild disc bulge noted at L4-L5 and L1-L2 level. No significant thecal sac or nerve root compression noted.,0,1
483,local_data/mendeley/T2_0552_D3.png,Mild disc bulge noted at L4-L5 and L1-L2 level. No significant thecal sac or nerve root compression noted.,0,1
484,local_data/mendeley/T2_0552_D4.png,Mild disc bulge noted at L4-L5 and L1-L2 level. No significant thecal sac or nerve root compression noted.,0,1
485,local_data/mendeley/T2_0552_D5.png,Mild disc bulge noted at L4-L5 and L1-L2 level. No significant thecal sac or nerve root compression noted.,0,1
486,local_data/mendeley/T1_0558_D3.png,"LSS MRI : Feature of muscel spasm. Generalised decreased bone marrow signals intensities noted ? correalte with bone isotope scan. Mild diffuse disc bulges noted at L4/L5 level , abutting thecal sac .",0,1
487,local_data/mendeley/T1_0558_D4.png,"LSS MRI : Feature of muscel spasm. Generalised decreased bone marrow signals intensities noted ? correalte with bone isotope scan. Mild diffuse disc bulges noted at L4/L5 level , abutting thecal sac .",0,1
488,local_data/mendeley/T1_0558_D5.png,"LSS MRI : Feature of muscel spasm. Generalised decreased bone marrow signals intensities noted ? correalte with bone isotope scan. Mild diffuse disc bulges noted at L4/L5 level , abutting thecal sac .",0,1
489,local_data/mendeley/T2_0558_D3.png,"LSS MRI : Feature of muscel spasm. Generalised decreased bone marrow signals intensities noted ? correalte with bone isotope scan. Mild diffuse disc bulges noted at L4/L5 level , abutting thecal sac .",0,1
490,local_data/mendeley/T2_0558_D4.png,"LSS MRI : Feature of muscel spasm. Generalised decreased bone marrow signals intensities noted ? correalte with bone isotope scan. Mild diffuse disc bulges noted at L4/L5 level , abutting thecal sac .",0,1
491,local_data/mendeley/T2_0558_D5.png,"LSS MRI : Feature of muscel spasm. Generalised decreased bone marrow signals intensities noted ? correalte with bone isotope scan. Mild diffuse disc bulges noted at L4/L5 level , abutting thecal sac .",0,1
492,local_data/mendeley/T1_0570_D3.png,"L3-L4, L4-L5: Mild disc bulges noted compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
493,local_data/mendeley/T1_0570_D4.png,"L3-L4, L4-L5: Mild disc bulges noted compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
494,local_data/mendeley/T1_0570_D5.png,"L3-L4, L4-L5: Mild disc bulges noted compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
495,local_data/mendeley/T2_0570_D3.png,"L3-L4, L4-L5: Mild disc bulges noted compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
496,local_data/mendeley/T2_0570_D4.png,"L3-L4, L4-L5: Mild disc bulges noted compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
497,local_data/mendeley/T2_0570_D5.png,"L3-L4, L4-L5: Mild disc bulges noted compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1
498,local_data/mendeley/T1_0571_D3.png,"LSS MRI : wedging fractures of L2& L4 vetrebal bodies , with bone marrow edema without significant thecal sac. Multilevel diffuse disc bulges notecibly at L4/L5 levels , compressing thecal sac and both exit nerves roots with relatively secandary spinal canal stenosis",0,1
499,local_data/mendeley/T1_0571_D4.png,"LSS MRI : wedging fractures of L2& L4 vetrebal bodies , with bone marrow edema without significant thecal sac. Multilevel diffuse disc bulges notecibly at L4/L5 levels , compressing thecal sac and both exit nerves roots with relatively secandary spinal canal stenosis",0,1
500,local_data/mendeley/T1_0571_D5.png,"LSS MRI : wedging fractures of L2& L4 vetrebal bodies , with bone marrow edema without significant thecal sac. Multilevel diffuse disc bulges notecibly at L4/L5 levels , compressing thecal sac and both exit nerves roots with relatively secandary spinal canal stenosis",0,1
501,local_data/mendeley/T2_0571_D3.png,"LSS MRI : wedging fractures of L2& L4 vetrebal bodies , with bone marrow edema without significant thecal sac. Multilevel diffuse disc bulges notecibly at L4/L5 levels , compressing thecal sac and both exit nerves roots with relatively secandary spinal canal stenosis",0,1
502,local_data/mendeley/T2_0571_D4.png,"LSS MRI : wedging fractures of L2& L4 vetrebal bodies , with bone marrow edema without significant thecal sac. Multilevel diffuse disc bulges notecibly at L4/L5 levels , compressing thecal sac and both exit nerves roots with relatively secandary spinal canal stenosis",0,1
503,local_data/mendeley/T2_0571_D5.png,"LSS MRI : wedging fractures of L2& L4 vetrebal bodies , with bone marrow edema without significant thecal sac. Multilevel diffuse disc bulges notecibly at L4/L5 levels , compressing thecal sac and both exit nerves roots with relatively secandary spinal canal stenosis",0,1

Overwriting local_data/lbp-valid-meta.csv


In [5]:
%%writefile local_data/lbp-test-meta.csv
,imgpath,report,No Finding,LBP
0,local_data/mendeley/T1_0001_D3.png,"L4-5: degenerative annular disc bulge is noted more to the left side compressing thecal sac, compressing left nerve root and narrowing right neural foramen. // Evidence of hyperintense signal within the annulus fibrosus at left paramedian/posterolateral area which probably represents a torn annulus.",0,1
1,local_data/mendeley/T1_0001_D4.png,"L4-5: degenerative annular disc bulge is noted more to the left side compressing thecal sac, compressing left nerve root and narrowing right neural foramen. // Evidence of hyperintense signal within the annulus fibrosus at left paramedian/posterolateral area which probably represents a torn annulus.",0,1
2,local_data/mendeley/T1_0001_D5.png,"L4-5: degenerative annular disc bulge is noted more to the left side compressing thecal sac, compressing left nerve root and narrowing right neural foramen. // Evidence of hyperintense signal within the annulus fibrosus at left paramedian/posterolateral area which probably represents a torn annulus.",0,1
3,local_data/mendeley/T2_0001_D3.png,"L4-5: degenerative annular disc bulge is noted more to the left side compressing thecal sac, compressing left nerve root and narrowing right neural foramen. // Evidence of hyperintense signal within the annulus fibrosus at left paramedian/posterolateral area which probably represents a torn annulus.",0,1
4,local_data/mendeley/T2_0001_D4.png,"L4-5: degenerative annular disc bulge is noted more to the left side compressing thecal sac, compressing left nerve root and narrowing right neural foramen. // Evidence of hyperintense signal within the annulus fibrosus at left paramedian/posterolateral area which probably represents a torn annulus.",0,1
5,local_data/mendeley/T2_0001_D5.png,"L4-5: degenerative annular disc bulge is noted more to the left side compressing thecal sac, compressing left nerve root and narrowing right neural foramen. // Evidence of hyperintense signal within the annulus fibrosus at left paramedian/posterolateral area which probably represents a torn annulus.",0,1
6,local_data/mendeley/T1_0003_D3.png,LSS MRI Features of muscle spasm. small central disc protrusion noted at L5-S1 level abutting the thecal sac. no significant thecal sac or nerve root compression noted.,0,1
7,local_data/mendeley/T1_0003_D4.png,LSS MRI Features of muscle spasm. small central disc protrusion noted at L5-S1 level abutting the thecal sac. no significant thecal sac or nerve root compression noted.,0,1
8,local_data/mendeley/T1_0003_D5.png,LSS MRI Features of muscle spasm. small central disc protrusion noted at L5-S1 level abutting the thecal sac. no significant thecal sac or nerve root compression noted.,0,1
9,local_data/mendeley/T2_0003_D3.png,LSS MRI Features of muscle spasm. small central disc protrusion noted at L5-S1 level abutting the thecal sac. no significant thecal sac or nerve root compression noted.,0,1
10,local_data/mendeley/T2_0003_D4.png,LSS MRI Features of muscle spasm. small central disc protrusion noted at L5-S1 level abutting the thecal sac. no significant thecal sac or nerve root compression noted.,0,1
11,local_data/mendeley/T2_0003_D5.png,LSS MRI Features of muscle spasm. small central disc protrusion noted at L5-S1 level abutting the thecal sac. no significant thecal sac or nerve root compression noted.,0,1
12,local_data/mendeley/T1_0020_D3.png,"Compression fracture is noted involving D12 body with compression of thecal sac and indentation of spinal cord at this level and maximum posterior displacement of posterior border of vertebral body by about 6.5mm. This fracture is believed to be pathological due to the mottled appearance of vertebral bodies. L4-5: secondary lumbar canal stenosis is noted due to degenerative annular disc bulge along with hypertrophy of facet joints and ligamenta flaca. compression of right L4 nerve root within right neural foramen due to the above mentioned disc bulge is also noted. L5-S1: degenerative annular disc bulge more toward the right side causing compression of thecal sac, compression of right L5 nerve root within right neural foramen and narrowing of left neural foramen. Diffusely altered signal of vertebral bone marrow; further evaluation is advised.",0,1
13,local_data/mendeley/T1_0020_D4.png,"Compression fracture is noted involving D12 body with compression of thecal sac and indentation of spinal cord at this level and maximum posterior displacement of posterior border of vertebral body by about 6.5mm. This fracture is believed to be pathological due to the mottled appearance of vertebral bodies. L4-5: secondary lumbar canal stenosis is noted due to degenerative annular disc bulge along with hypertrophy of facet joints and ligamenta flaca. compression of right L4 nerve root within right neural foramen due to the above mentioned disc bulge is also noted. L5-S1: degenerative annular disc bulge more toward the right side causing compression of thecal sac, compression of right L5 nerve root within right neural foramen and narrowing of left neural foramen. Diffusely altered signal of vertebral bone marrow; further evaluation is advised.",0,1
14,local_data/mendeley/T1_0020_D5.png,"Compression fracture is noted involving D12 body with compression of thecal sac and indentation of spinal cord at this level and maximum posterior displacement of posterior border of vertebral body by about 6.5mm. This fracture is believed to be pathological due to the mottled appearance of vertebral bodies. L4-5: secondary lumbar canal stenosis is noted due to degenerative annular disc bulge along with hypertrophy of facet joints and ligamenta flaca. compression of right L4 nerve root within right neural foramen due to the above mentioned disc bulge is also noted. L5-S1: degenerative annular disc bulge more toward the right side causing compression of thecal sac, compression of right L5 nerve root within right neural foramen and narrowing of left neural foramen. Diffusely altered signal of vertebral bone marrow; further evaluation is advised.",0,1
15,local_data/mendeley/T2_0020_D3.png,"Compression fracture is noted involving D12 body with compression of thecal sac and indentation of spinal cord at this level and maximum posterior displacement of posterior border of vertebral body by about 6.5mm. This fracture is believed to be pathological due to the mottled appearance of vertebral bodies. L4-5: secondary lumbar canal stenosis is noted due to degenerative annular disc bulge along with hypertrophy of facet joints and ligamenta flaca. compression of right L4 nerve root within right neural foramen due to the above mentioned disc bulge is also noted. L5-S1: degenerative annular disc bulge more toward the right side causing compression of thecal sac, compression of right L5 nerve root within right neural foramen and narrowing of left neural foramen. Diffusely altered signal of vertebral bone marrow; further evaluation is advised.",0,1
16,local_data/mendeley/T2_0020_D4.png,"Compression fracture is noted involving D12 body with compression of thecal sac and indentation of spinal cord at this level and maximum posterior displacement of posterior border of vertebral body by about 6.5mm. This fracture is believed to be pathological due to the mottled appearance of vertebral bodies. L4-5: secondary lumbar canal stenosis is noted due to degenerative annular disc bulge along with hypertrophy of facet joints and ligamenta flaca. compression of right L4 nerve root within right neural foramen due to the above mentioned disc bulge is also noted. L5-S1: degenerative annular disc bulge more toward the right side causing compression of thecal sac, compression of right L5 nerve root within right neural foramen and narrowing of left neural foramen. Diffusely altered signal of vertebral bone marrow; further evaluation is advised.",0,1
17,local_data/mendeley/T2_0020_D5.png,"Compression fracture is noted involving D12 body with compression of thecal sac and indentation of spinal cord at this level and maximum posterior displacement of posterior border of vertebral body by about 6.5mm. This fracture is believed to be pathological due to the mottled appearance of vertebral bodies. L4-5: secondary lumbar canal stenosis is noted due to degenerative annular disc bulge along with hypertrophy of facet joints and ligamenta flaca. compression of right L4 nerve root within right neural foramen due to the above mentioned disc bulge is also noted. L5-S1: degenerative annular disc bulge more toward the right side causing compression of thecal sac, compression of right L5 nerve root within right neural foramen and narrowing of left neural foramen. Diffusely altered signal of vertebral bone marrow; further evaluation is advised.",0,1
18,local_data/mendeley/T1_0027_D3.png,L2-3: broad based right paramedin disc extrusion is noted slightly migrating downwards,0,1
19,local_data/mendeley/T1_0027_D4.png,L2-3: broad based right paramedin disc extrusion is noted slightly migrating downwards,0,1
20,local_data/mendeley/T1_0027_D5.png,L2-3: broad based right paramedin disc extrusion is noted slightly migrating downwards,0,1
21,local_data/mendeley/T2_0027_D3.png,L2-3: broad based right paramedin disc extrusion is noted slightly migrating downwards,0,1
22,local_data/mendeley/T2_0027_D4.png,L2-3: broad based right paramedin disc extrusion is noted slightly migrating downwards,0,1
23,local_data/mendeley/T2_0027_D5.png,L2-3: broad based right paramedin disc extrusion is noted slightly migrating downwards,0,1
24,local_data/mendeley/T1_0034_D3.png,L4-L5: Mild central disc protrusion noted abutting the thecal sac.,0,1
25,local_data/mendeley/T1_0034_D4.png,L4-L5: Mild central disc protrusion noted abutting the thecal sac.,0,1
26,local_data/mendeley/T1_0034_D5.png,L4-L5: Mild central disc protrusion noted abutting the thecal sac.,0,1
27,local_data/mendeley/T2_0034_D3.png,L4-L5: Mild central disc protrusion noted abutting the thecal sac.,0,1
28,local_data/mendeley/T2_0034_D4.png,L4-L5: Mild central disc protrusion noted abutting the thecal sac.,0,1
29,local_data/mendeley/T2_0034_D5.png,L4-L5: Mild central disc protrusion noted abutting the thecal sac.,0,1
30,local_data/mendeley/T1_0036_D3.png,"LSS MRI : There is sequeatrated disc fracgement (ring enhancing ) noted just posterior to L5 vertebral body below L4/L5 level ,at Lt paracentral thecal sac compressing Lt sided thecal sac and exit nerve roots",0,1
31,local_data/mendeley/T1_0036_D4.png,"LSS MRI : There is sequeatrated disc fracgement (ring enhancing ) noted just posterior to L5 vertebral body below L4/L5 level ,at Lt paracentral thecal sac compressing Lt sided thecal sac and exit nerve roots",0,1
32,local_data/mendeley/T1_0036_D5.png,"LSS MRI : There is sequeatrated disc fracgement (ring enhancing ) noted just posterior to L5 vertebral body below L4/L5 level ,at Lt paracentral thecal sac compressing Lt sided thecal sac and exit nerve roots",0,1
33,local_data/mendeley/T2_0036_D3.png,"LSS MRI : There is sequeatrated disc fracgement (ring enhancing ) noted just posterior to L5 vertebral body below L4/L5 level ,at Lt paracentral thecal sac compressing Lt sided thecal sac and exit nerve roots",0,1
34,local_data/mendeley/T2_0036_D4.png,"LSS MRI : There is sequeatrated disc fracgement (ring enhancing ) noted just posterior to L5 vertebral body below L4/L5 level ,at Lt paracentral thecal sac compressing Lt sided thecal sac and exit nerve roots",0,1
35,local_data/mendeley/T2_0036_D5.png,"LSS MRI : There is sequeatrated disc fracgement (ring enhancing ) noted just posterior to L5 vertebral body below L4/L5 level ,at Lt paracentral thecal sac compressing Lt sided thecal sac and exit nerve roots",0,1
36,local_data/mendeley/T1_0042_D3.png,MRI OF THE LSS -Feature of muscle spasm. diffuse disc bulge noted at L3/L4& L4/L5 level mildly compressing thecal sac and exit nerve roots,0,1
37,local_data/mendeley/T1_0042_D4.png,MRI OF THE LSS -Feature of muscle spasm. diffuse disc bulge noted at L3/L4& L4/L5 level mildly compressing thecal sac and exit nerve roots,0,1
38,local_data/mendeley/T1_0042_D5.png,MRI OF THE LSS -Feature of muscle spasm. diffuse disc bulge noted at L3/L4& L4/L5 level mildly compressing thecal sac and exit nerve roots,0,1
39,local_data/mendeley/T2_0042_D3.png,MRI OF THE LSS -Feature of muscle spasm. diffuse disc bulge noted at L3/L4& L4/L5 level mildly compressing thecal sac and exit nerve roots,0,1
40,local_data/mendeley/T2_0042_D4.png,MRI OF THE LSS -Feature of muscle spasm. diffuse disc bulge noted at L3/L4& L4/L5 level mildly compressing thecal sac and exit nerve roots,0,1
41,local_data/mendeley/T2_0042_D5.png,MRI OF THE LSS -Feature of muscle spasm. diffuse disc bulge noted at L3/L4& L4/L5 level mildly compressing thecal sac and exit nerve roots,0,1
42,local_data/mendeley/T1_0051_D3.png,LSS MRI Mild disc bulges noted at L5-S1 and L2-L3 levels abutting the thecal sac. Adequate spinal canal.,0,1
43,local_data/mendeley/T1_0051_D4.png,LSS MRI Mild disc bulges noted at L5-S1 and L2-L3 levels abutting the thecal sac. Adequate spinal canal.,0,1
44,local_data/mendeley/T1_0051_D5.png,LSS MRI Mild disc bulges noted at L5-S1 and L2-L3 levels abutting the thecal sac. Adequate spinal canal.,0,1
45,local_data/mendeley/T2_0051_D3.png,LSS MRI Mild disc bulges noted at L5-S1 and L2-L3 levels abutting the thecal sac. Adequate spinal canal.,0,1
46,local_data/mendeley/T2_0051_D4.png,LSS MRI Mild disc bulges noted at L5-S1 and L2-L3 levels abutting the thecal sac. Adequate spinal canal.,0,1
47,local_data/mendeley/T2_0051_D5.png,LSS MRI Mild disc bulges noted at L5-S1 and L2-L3 levels abutting the thecal sac. Adequate spinal canal.,0,1
48,local_data/mendeley/T1_0064_D3.png,Lumbosacral MRI No evidence of disc herniation No thecal sac or nerve root compression noted,1,0
49,local_data/mendeley/T1_0064_D4.png,Lumbosacral MRI No evidence of disc herniation No thecal sac or nerve root compression noted,1,0
50,local_data/mendeley/T1_0064_D5.png,Lumbosacral MRI No evidence of disc herniation No thecal sac or nerve root compression noted,1,0
51,local_data/mendeley/T2_0064_D3.png,Lumbosacral MRI No evidence of disc herniation No thecal sac or nerve root compression noted,1,0
52,local_data/mendeley/T2_0064_D4.png,Lumbosacral MRI No evidence of disc herniation No thecal sac or nerve root compression noted,1,0
53,local_data/mendeley/T2_0064_D5.png,Lumbosacral MRI No evidence of disc herniation No thecal sac or nerve root compression noted,1,0
54,local_data/mendeley/T1_0077_D3.png,LSS MRI mild disc bulge noted at L4-L5 level abutting the thecal sac.  no significant thecal sac or nerve root compression noted.,0,1
55,local_data/mendeley/T1_0077_D4.png,LSS MRI mild disc bulge noted at L4-L5 level abutting the thecal sac.  no significant thecal sac or nerve root compression noted.,0,1
56,local_data/mendeley/T1_0077_D5.png,LSS MRI mild disc bulge noted at L4-L5 level abutting the thecal sac.  no significant thecal sac or nerve root compression noted.,0,1
57,local_data/mendeley/T2_0077_D3.png,LSS MRI mild disc bulge noted at L4-L5 level abutting the thecal sac.  no significant thecal sac or nerve root compression noted.,0,1
58,local_data/mendeley/T2_0077_D4.png,LSS MRI mild disc bulge noted at L4-L5 level abutting the thecal sac.  no significant thecal sac or nerve root compression noted.,0,1
59,local_data/mendeley/T2_0077_D5.png,LSS MRI mild disc bulge noted at L4-L5 level abutting the thecal sac.  no significant thecal sac or nerve root compression noted.,0,1
60,local_data/mendeley/T1_0078_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
61,local_data/mendeley/T1_0078_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
62,local_data/mendeley/T1_0078_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
63,local_data/mendeley/T2_0078_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
64,local_data/mendeley/T2_0078_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
65,local_data/mendeley/T2_0078_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
66,local_data/mendeley/T1_0080_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
67,local_data/mendeley/T1_0080_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
68,local_data/mendeley/T1_0080_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
69,local_data/mendeley/T2_0080_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
70,local_data/mendeley/T2_0080_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
71,local_data/mendeley/T2_0080_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
72,local_data/mendeley/T1_0085_D3.png,LSS MRI Mild disc bulge noted at L5-S1 level with Rt posterolateral tear.  No significant thecal sac or nerve root compression noted.  Adequate spinal canal.,0,1
73,local_data/mendeley/T1_0085_D4.png,LSS MRI Mild disc bulge noted at L5-S1 level with Rt posterolateral tear.  No significant thecal sac or nerve root compression noted.  Adequate spinal canal.,0,1
74,local_data/mendeley/T1_0085_D5.png,LSS MRI Mild disc bulge noted at L5-S1 level with Rt posterolateral tear.  No significant thecal sac or nerve root compression noted.  Adequate spinal canal.,0,1
75,local_data/mendeley/T2_0085_D3.png,LSS MRI Mild disc bulge noted at L5-S1 level with Rt posterolateral tear.  No significant thecal sac or nerve root compression noted.  Adequate spinal canal.,0,1
76,local_data/mendeley/T2_0085_D4.png,LSS MRI Mild disc bulge noted at L5-S1 level with Rt posterolateral tear.  No significant thecal sac or nerve root compression noted.  Adequate spinal canal.,0,1
77,local_data/mendeley/T2_0085_D5.png,LSS MRI Mild disc bulge noted at L5-S1 level with Rt posterolateral tear.  No significant thecal sac or nerve root compression noted.  Adequate spinal canal.,0,1
78,local_data/mendeley/T1_0086_D3.png,"LSS MRI Multilevel, diffuse disc bulges noted (L2-L3, L3-L4 and L4-L5 levels), mildly compressing the thecal sac.",0,1
79,local_data/mendeley/T1_0086_D4.png,"LSS MRI Multilevel, diffuse disc bulges noted (L2-L3, L3-L4 and L4-L5 levels), mildly compressing the thecal sac.",0,1
80,local_data/mendeley/T1_0086_D5.png,"LSS MRI Multilevel, diffuse disc bulges noted (L2-L3, L3-L4 and L4-L5 levels), mildly compressing the thecal sac.",0,1
81,local_data/mendeley/T2_0086_D3.png,"LSS MRI Multilevel, diffuse disc bulges noted (L2-L3, L3-L4 and L4-L5 levels), mildly compressing the thecal sac.",0,1
82,local_data/mendeley/T2_0086_D4.png,"LSS MRI Multilevel, diffuse disc bulges noted (L2-L3, L3-L4 and L4-L5 levels), mildly compressing the thecal sac.",0,1
83,local_data/mendeley/T2_0086_D5.png,"LSS MRI Multilevel, diffuse disc bulges noted (L2-L3, L3-L4 and L4-L5 levels), mildly compressing the thecal sac.",0,1
84,local_data/mendeley/T1_0088_D3.png,"LSS MRI :  Feature of muscle spasm.  L4/L5 level :  Partial dissicating disc material noted ,with diffuse disc bulge causing mild compressing thecal sac and encroaching exit neural canals",0,1
85,local_data/mendeley/T1_0088_D4.png,"LSS MRI :  Feature of muscle spasm.  L4/L5 level :  Partial dissicating disc material noted ,with diffuse disc bulge causing mild compressing thecal sac and encroaching exit neural canals",0,1
86,local_data/mendeley/T1_0088_D5.png,"LSS MRI :  Feature of muscle spasm.  L4/L5 level :  Partial dissicating disc material noted ,with diffuse disc bulge causing mild compressing thecal sac and encroaching exit neural canals",0,1
87,local_data/mendeley/T2_0088_D3.png,"LSS MRI :  Feature of muscle spasm.  L4/L5 level :  Partial dissicating disc material noted ,with diffuse disc bulge causing mild compressing thecal sac and encroaching exit neural canals",0,1
88,local_data/mendeley/T2_0088_D4.png,"LSS MRI :  Feature of muscle spasm.  L4/L5 level :  Partial dissicating disc material noted ,with diffuse disc bulge causing mild compressing thecal sac and encroaching exit neural canals",0,1
89,local_data/mendeley/T2_0088_D5.png,"LSS MRI :  Feature of muscle spasm.  L4/L5 level :  Partial dissicating disc material noted ,with diffuse disc bulge causing mild compressing thecal sac and encroaching exit neural canals",0,1
90,local_data/mendeley/T1_0089_D3.png,"L3-L4: Central disc protrusion noted, largely compressing the thecal sac. L4-L5: Rt paracentral disc protrusion noted, compressing the thecal sac and Rt exit neural canal. L5-S1: Mild disc bulge noted.",0,1
91,local_data/mendeley/T1_0089_D4.png,"L3-L4: Central disc protrusion noted, largely compressing the thecal sac. L4-L5: Rt paracentral disc protrusion noted, compressing the thecal sac and Rt exit neural canal. L5-S1: Mild disc bulge noted.",0,1
92,local_data/mendeley/T1_0089_D5.png,"L3-L4: Central disc protrusion noted, largely compressing the thecal sac. L4-L5: Rt paracentral disc protrusion noted, compressing the thecal sac and Rt exit neural canal. L5-S1: Mild disc bulge noted.",0,1
93,local_data/mendeley/T2_0089_D3.png,"L3-L4: Central disc protrusion noted, largely compressing the thecal sac. L4-L5: Rt paracentral disc protrusion noted, compressing the thecal sac and Rt exit neural canal. L5-S1: Mild disc bulge noted.",0,1
94,local_data/mendeley/T2_0089_D4.png,"L3-L4: Central disc protrusion noted, largely compressing the thecal sac. L4-L5: Rt paracentral disc protrusion noted, compressing the thecal sac and Rt exit neural canal. L5-S1: Mild disc bulge noted.",0,1
95,local_data/mendeley/T2_0089_D5.png,"L3-L4: Central disc protrusion noted, largely compressing the thecal sac. L4-L5: Rt paracentral disc protrusion noted, compressing the thecal sac and Rt exit neural canal. L5-S1: Mild disc bulge noted.",0,1
96,local_data/mendeley/T1_0096_D3.png,"LSS MRI Lt paracentral disc protrusion noted at L5-S1 level, abutting the thecal sac and Lt S1 nerve root.  adequate spinal canal.",0,1
97,local_data/mendeley/T1_0096_D4.png,"LSS MRI Lt paracentral disc protrusion noted at L5-S1 level, abutting the thecal sac and Lt S1 nerve root.  adequate spinal canal.",0,1
98,local_data/mendeley/T1_0096_D5.png,"LSS MRI Lt paracentral disc protrusion noted at L5-S1 level, abutting the thecal sac and Lt S1 nerve root.  adequate spinal canal.",0,1
99,local_data/mendeley/T2_0096_D3.png,"LSS MRI Lt paracentral disc protrusion noted at L5-S1 level, abutting the thecal sac and Lt S1 nerve root.  adequate spinal canal.",0,1
100,local_data/mendeley/T2_0096_D4.png,"LSS MRI Lt paracentral disc protrusion noted at L5-S1 level, abutting the thecal sac and Lt S1 nerve root.  adequate spinal canal.",0,1
101,local_data/mendeley/T2_0096_D5.png,"LSS MRI Lt paracentral disc protrusion noted at L5-S1 level, abutting the thecal sac and Lt S1 nerve root.  adequate spinal canal.",0,1
102,local_data/mendeley/T1_0104_D3.png,MRI of the lumbosacral spine  No evidence of disc herniation  No thecal sac or nerve root compression noted  Adequate spinal canal,1,0
103,local_data/mendeley/T1_0104_D4.png,MRI of the lumbosacral spine  No evidence of disc herniation  No thecal sac or nerve root compression noted  Adequate spinal canal,1,0
104,local_data/mendeley/T1_0104_D5.png,MRI of the lumbosacral spine  No evidence of disc herniation  No thecal sac or nerve root compression noted  Adequate spinal canal,1,0
105,local_data/mendeley/T2_0104_D3.png,MRI of the lumbosacral spine  No evidence of disc herniation  No thecal sac or nerve root compression noted  Adequate spinal canal,1,0
106,local_data/mendeley/T2_0104_D4.png,MRI of the lumbosacral spine  No evidence of disc herniation  No thecal sac or nerve root compression noted  Adequate spinal canal,1,0
107,local_data/mendeley/T2_0104_D5.png,MRI of the lumbosacral spine  No evidence of disc herniation  No thecal sac or nerve root compression noted  Adequate spinal canal,1,0
108,local_data/mendeley/T1_0114_D3.png,"LSS MRI  L5-S1: Lt paracentral and foraminal disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  features of muscle spasm noted.",0,1
109,local_data/mendeley/T1_0114_D4.png,"LSS MRI  L5-S1: Lt paracentral and foraminal disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  features of muscle spasm noted.",0,1
110,local_data/mendeley/T1_0114_D5.png,"LSS MRI  L5-S1: Lt paracentral and foraminal disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  features of muscle spasm noted.",0,1
111,local_data/mendeley/T2_0114_D3.png,"LSS MRI  L5-S1: Lt paracentral and foraminal disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  features of muscle spasm noted.",0,1
112,local_data/mendeley/T2_0114_D4.png,"LSS MRI  L5-S1: Lt paracentral and foraminal disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  features of muscle spasm noted.",0,1
113,local_data/mendeley/T2_0114_D5.png,"LSS MRI  L5-S1: Lt paracentral and foraminal disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  features of muscle spasm noted.",0,1
114,local_data/mendeley/T1_0126_D3.png,"LSS MRI : Degenerative changes. Multilevel diffuse bulges noteceably at L2/L3, L3/L4&L4/5 levels, compressing the thecal sac and encroaching both exit nerve roots , associated with ligamentum flavum hyperatrophy, all leading to secandary spinal canal stenosis",0,1
115,local_data/mendeley/T1_0126_D4.png,"LSS MRI : Degenerative changes. Multilevel diffuse bulges noteceably at L2/L3, L3/L4&L4/5 levels, compressing the thecal sac and encroaching both exit nerve roots , associated with ligamentum flavum hyperatrophy, all leading to secandary spinal canal stenosis",0,1
116,local_data/mendeley/T1_0126_D5.png,"LSS MRI : Degenerative changes. Multilevel diffuse bulges noteceably at L2/L3, L3/L4&L4/5 levels, compressing the thecal sac and encroaching both exit nerve roots , associated with ligamentum flavum hyperatrophy, all leading to secandary spinal canal stenosis",0,1
117,local_data/mendeley/T2_0126_D3.png,"LSS MRI : Degenerative changes. Multilevel diffuse bulges noteceably at L2/L3, L3/L4&L4/5 levels, compressing the thecal sac and encroaching both exit nerve roots , associated with ligamentum flavum hyperatrophy, all leading to secandary spinal canal stenosis",0,1
118,local_data/mendeley/T2_0126_D4.png,"LSS MRI : Degenerative changes. Multilevel diffuse bulges noteceably at L2/L3, L3/L4&L4/5 levels, compressing the thecal sac and encroaching both exit nerve roots , associated with ligamentum flavum hyperatrophy, all leading to secandary spinal canal stenosis",0,1
119,local_data/mendeley/T2_0126_D5.png,"LSS MRI : Degenerative changes. Multilevel diffuse bulges noteceably at L2/L3, L3/L4&L4/5 levels, compressing the thecal sac and encroaching both exit nerve roots , associated with ligamentum flavum hyperatrophy, all leading to secandary spinal canal stenosis",0,1
120,local_data/mendeley/T1_0138_D3.png,"LSS MRI :  Feature of muscle spasm.  Small Lt paracentral disc protruion noted at D12/L1 , mildly compressing Lt exit nerve root.",0,1
121,local_data/mendeley/T1_0138_D4.png,"LSS MRI :  Feature of muscle spasm.  Small Lt paracentral disc protruion noted at D12/L1 , mildly compressing Lt exit nerve root.",0,1
122,local_data/mendeley/T1_0138_D5.png,"LSS MRI :  Feature of muscle spasm.  Small Lt paracentral disc protruion noted at D12/L1 , mildly compressing Lt exit nerve root.",0,1
123,local_data/mendeley/T2_0138_D3.png,"LSS MRI :  Feature of muscle spasm.  Small Lt paracentral disc protruion noted at D12/L1 , mildly compressing Lt exit nerve root.",0,1
124,local_data/mendeley/T2_0138_D4.png,"LSS MRI :  Feature of muscle spasm.  Small Lt paracentral disc protruion noted at D12/L1 , mildly compressing Lt exit nerve root.",0,1
125,local_data/mendeley/T2_0138_D5.png,"LSS MRI :  Feature of muscle spasm.  Small Lt paracentral disc protruion noted at D12/L1 , mildly compressing Lt exit nerve root.",0,1
126,local_data/mendeley/T1_0156_D3.png,LSS and pelvis MRI:  Evidence of the previous surgery noted  No evidence of significant thecal sac or nerve root compression noted  Adequate thecal sac,1,0
127,local_data/mendeley/T1_0156_D4.png,LSS and pelvis MRI:  Evidence of the previous surgery noted  No evidence of significant thecal sac or nerve root compression noted  Adequate thecal sac,1,0
128,local_data/mendeley/T1_0156_D5.png,LSS and pelvis MRI:  Evidence of the previous surgery noted  No evidence of significant thecal sac or nerve root compression noted  Adequate thecal sac,1,0
129,local_data/mendeley/T2_0156_D3.png,LSS and pelvis MRI:  Evidence of the previous surgery noted  No evidence of significant thecal sac or nerve root compression noted  Adequate thecal sac,1,0
130,local_data/mendeley/T2_0156_D4.png,LSS and pelvis MRI:  Evidence of the previous surgery noted  No evidence of significant thecal sac or nerve root compression noted  Adequate thecal sac,1,0
131,local_data/mendeley/T2_0156_D5.png,LSS and pelvis MRI:  Evidence of the previous surgery noted  No evidence of significant thecal sac or nerve root compression noted  Adequate thecal sac,1,0
132,local_data/mendeley/T1_0171_D3.png,"LSS MRI :  Feature of muscle spasm.  Wide base focal disc bulge noted at L5/S1 level, compressing the thecal sac with annular tear and mild encroaching upon exit neural canal.",0,1
133,local_data/mendeley/T1_0171_D4.png,"LSS MRI :  Feature of muscle spasm.  Wide base focal disc bulge noted at L5/S1 level, compressing the thecal sac with annular tear and mild encroaching upon exit neural canal.",0,1
134,local_data/mendeley/T1_0171_D5.png,"LSS MRI :  Feature of muscle spasm.  Wide base focal disc bulge noted at L5/S1 level, compressing the thecal sac with annular tear and mild encroaching upon exit neural canal.",0,1
135,local_data/mendeley/T2_0171_D3.png,"LSS MRI :  Feature of muscle spasm.  Wide base focal disc bulge noted at L5/S1 level, compressing the thecal sac with annular tear and mild encroaching upon exit neural canal.",0,1
136,local_data/mendeley/T2_0171_D4.png,"LSS MRI :  Feature of muscle spasm.  Wide base focal disc bulge noted at L5/S1 level, compressing the thecal sac with annular tear and mild encroaching upon exit neural canal.",0,1
137,local_data/mendeley/T2_0171_D5.png,"LSS MRI :  Feature of muscle spasm.  Wide base focal disc bulge noted at L5/S1 level, compressing the thecal sac with annular tear and mild encroaching upon exit neural canal.",0,1
138,local_data/mendeley/T1_0195_D3.png,Lumbosacral MRI Features of muscle spasm Multilevel dissicating  disc material Modic Type II at the L5-S1 Hemangioma of the L1 vertebral body -Diffuse disc bulge with right paracentral disc extrusion noted at the L5-S1 compressing the thecal sac and encroaching upon  neural canal -Diffuse disc bulge noted at the L4-L5 mild compressing the thecal sac and there-,0,1
139,local_data/mendeley/T1_0195_D4.png,Lumbosacral MRI Features of muscle spasm Multilevel dissicating  disc material Modic Type II at the L5-S1 Hemangioma of the L1 vertebral body -Diffuse disc bulge with right paracentral disc extrusion noted at the L5-S1 compressing the thecal sac and encroaching upon  neural canal -Diffuse disc bulge noted at the L4-L5 mild compressing the thecal sac and there-,0,1
140,local_data/mendeley/T1_0195_D5.png,Lumbosacral MRI Features of muscle spasm Multilevel dissicating  disc material Modic Type II at the L5-S1 Hemangioma of the L1 vertebral body -Diffuse disc bulge with right paracentral disc extrusion noted at the L5-S1 compressing the thecal sac and encroaching upon  neural canal -Diffuse disc bulge noted at the L4-L5 mild compressing the thecal sac and there-,0,1
141,local_data/mendeley/T2_0195_D3.png,Lumbosacral MRI Features of muscle spasm Multilevel dissicating  disc material Modic Type II at the L5-S1 Hemangioma of the L1 vertebral body -Diffuse disc bulge with right paracentral disc extrusion noted at the L5-S1 compressing the thecal sac and encroaching upon  neural canal -Diffuse disc bulge noted at the L4-L5 mild compressing the thecal sac and there-,0,1
142,local_data/mendeley/T2_0195_D4.png,Lumbosacral MRI Features of muscle spasm Multilevel dissicating  disc material Modic Type II at the L5-S1 Hemangioma of the L1 vertebral body -Diffuse disc bulge with right paracentral disc extrusion noted at the L5-S1 compressing the thecal sac and encroaching upon  neural canal -Diffuse disc bulge noted at the L4-L5 mild compressing the thecal sac and there-,0,1
143,local_data/mendeley/T2_0195_D5.png,Lumbosacral MRI Features of muscle spasm Multilevel dissicating  disc material Modic Type II at the L5-S1 Hemangioma of the L1 vertebral body -Diffuse disc bulge with right paracentral disc extrusion noted at the L5-S1 compressing the thecal sac and encroaching upon  neural canal -Diffuse disc bulge noted at the L4-L5 mild compressing the thecal sac and there-,0,1
144,local_data/mendeley/T1_0198_D3.png,Rt paracentral disc protrusion noted compressing the thecal sac.,0,1
145,local_data/mendeley/T1_0198_D4.png,Rt paracentral disc protrusion noted compressing the thecal sac.,0,1
146,local_data/mendeley/T1_0198_D5.png,Rt paracentral disc protrusion noted compressing the thecal sac.,0,1
147,local_data/mendeley/T2_0198_D3.png,Rt paracentral disc protrusion noted compressing the thecal sac.,0,1
148,local_data/mendeley/T2_0198_D4.png,Rt paracentral disc protrusion noted compressing the thecal sac.,0,1
149,local_data/mendeley/T2_0198_D5.png,Rt paracentral disc protrusion noted compressing the thecal sac.,0,1
150,local_data/mendeley/T1_0200_D3.png,L5-S1: Mild Lt paracentral disc protrusion compressing the thecal sac. L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals. Mild multilevel disc bulges noted elsewhere.,0,1
151,local_data/mendeley/T1_0200_D4.png,L5-S1: Mild Lt paracentral disc protrusion compressing the thecal sac. L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals. Mild multilevel disc bulges noted elsewhere.,0,1
152,local_data/mendeley/T1_0200_D5.png,L5-S1: Mild Lt paracentral disc protrusion compressing the thecal sac. L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals. Mild multilevel disc bulges noted elsewhere.,0,1
153,local_data/mendeley/T2_0200_D3.png,L5-S1: Mild Lt paracentral disc protrusion compressing the thecal sac. L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals. Mild multilevel disc bulges noted elsewhere.,0,1
154,local_data/mendeley/T2_0200_D4.png,L5-S1: Mild Lt paracentral disc protrusion compressing the thecal sac. L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals. Mild multilevel disc bulges noted elsewhere.,0,1
155,local_data/mendeley/T2_0200_D5.png,L5-S1: Mild Lt paracentral disc protrusion compressing the thecal sac. L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals. Mild multilevel disc bulges noted elsewhere.,0,1
156,local_data/mendeley/T1_0202_D3.png,L3-L4 AND L5-S1: Mild disc bulges noted. No significant thecal sac or nerve root compression noted.,0,1
157,local_data/mendeley/T1_0202_D4.png,L3-L4 AND L5-S1: Mild disc bulges noted. No significant thecal sac or nerve root compression noted.,0,1
158,local_data/mendeley/T1_0202_D5.png,L3-L4 AND L5-S1: Mild disc bulges noted. No significant thecal sac or nerve root compression noted.,0,1
159,local_data/mendeley/T2_0202_D3.png,L3-L4 AND L5-S1: Mild disc bulges noted. No significant thecal sac or nerve root compression noted.,0,1
160,local_data/mendeley/T2_0202_D4.png,L3-L4 AND L5-S1: Mild disc bulges noted. No significant thecal sac or nerve root compression noted.,0,1
161,local_data/mendeley/T2_0202_D5.png,L3-L4 AND L5-S1: Mild disc bulges noted. No significant thecal sac or nerve root compression noted.,0,1
162,local_data/mendeley/T1_0211_D3.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
163,local_data/mendeley/T1_0211_D4.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
164,local_data/mendeley/T1_0211_D5.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
165,local_data/mendeley/T2_0211_D3.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
166,local_data/mendeley/T2_0211_D4.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
167,local_data/mendeley/T2_0211_D5.png,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1
168,local_data/mendeley/T1_0214_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildly compressing the thecal sac and  both exit nerve roots",0,1
169,local_data/mendeley/T1_0214_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildly compressing the thecal sac and  both exit nerve roots",0,1
170,local_data/mendeley/T1_0214_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildly compressing the thecal sac and  both exit nerve roots",0,1
171,local_data/mendeley/T2_0214_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildly compressing the thecal sac and  both exit nerve roots",0,1
172,local_data/mendeley/T2_0214_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildly compressing the thecal sac and  both exit nerve roots",0,1
173,local_data/mendeley/T2_0214_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildly compressing the thecal sac and  both exit nerve roots",0,1
174,local_data/mendeley/T1_0228_D3.png,L5-S1: Mild disc bulge noted.,0,1
175,local_data/mendeley/T1_0228_D4.png,L5-S1: Mild disc bulge noted.,0,1
176,local_data/mendeley/T1_0228_D5.png,L5-S1: Mild disc bulge noted.,0,1
177,local_data/mendeley/T2_0228_D3.png,L5-S1: Mild disc bulge noted.,0,1
178,local_data/mendeley/T2_0228_D4.png,L5-S1: Mild disc bulge noted.,0,1
179,local_data/mendeley/T2_0228_D5.png,L5-S1: Mild disc bulge noted.,0,1
180,local_data/mendeley/T1_0230_D3.png,Marked degenerative changes with kyphoscoliosis. Spondylolisthesis noted at L2-L3 level.,0,1
181,local_data/mendeley/T1_0230_D4.png,Marked degenerative changes with kyphoscoliosis. Spondylolisthesis noted at L2-L3 level.,0,1
182,local_data/mendeley/T1_0230_D5.png,Marked degenerative changes with kyphoscoliosis. Spondylolisthesis noted at L2-L3 level.,0,1
183,local_data/mendeley/T2_0230_D3.png,Marked degenerative changes with kyphoscoliosis. Spondylolisthesis noted at L2-L3 level.,0,1
184,local_data/mendeley/T2_0230_D4.png,Marked degenerative changes with kyphoscoliosis. Spondylolisthesis noted at L2-L3 level.,0,1
185,local_data/mendeley/T2_0230_D5.png,Marked degenerative changes with kyphoscoliosis. Spondylolisthesis noted at L2-L3 level.,0,1
186,local_data/mendeley/T1_0243_D3.png,LSS MRI  -feature of muscle spasm.  -Modec type II noted at L4/L5 level.  -Diffuse disc bulges noted at L4-L5&L5/S1 levels compressing the thecal sac and exit neural canals.,0,1
187,local_data/mendeley/T1_0243_D4.png,LSS MRI  -feature of muscle spasm.  -Modec type II noted at L4/L5 level.  -Diffuse disc bulges noted at L4-L5&L5/S1 levels compressing the thecal sac and exit neural canals.,0,1
188,local_data/mendeley/T1_0243_D5.png,LSS MRI  -feature of muscle spasm.  -Modec type II noted at L4/L5 level.  -Diffuse disc bulges noted at L4-L5&L5/S1 levels compressing the thecal sac and exit neural canals.,0,1
189,local_data/mendeley/T2_0243_D3.png,LSS MRI  -feature of muscle spasm.  -Modec type II noted at L4/L5 level.  -Diffuse disc bulges noted at L4-L5&L5/S1 levels compressing the thecal sac and exit neural canals.,0,1
190,local_data/mendeley/T2_0243_D4.png,LSS MRI  -feature of muscle spasm.  -Modec type II noted at L4/L5 level.  -Diffuse disc bulges noted at L4-L5&L5/S1 levels compressing the thecal sac and exit neural canals.,0,1
191,local_data/mendeley/T2_0243_D5.png,LSS MRI  -feature of muscle spasm.  -Modec type II noted at L4/L5 level.  -Diffuse disc bulges noted at L4-L5&L5/S1 levels compressing the thecal sac and exit neural canals.,0,1
192,local_data/mendeley/T1_0250_D3.png,"LSS MRI :  Feature of muscle spasm.  L1/L2 level:  posterior central disc protrusion largly compressing thecal sac and both exit nerve roots.  L5/S1 level:  Lt paracentral disc protruion , migrating caudally ,compressing thecal sca and Lt exit nerve roots.",0,1
193,local_data/mendeley/T1_0250_D4.png,"LSS MRI :  Feature of muscle spasm.  L1/L2 level:  posterior central disc protrusion largly compressing thecal sac and both exit nerve roots.  L5/S1 level:  Lt paracentral disc protruion , migrating caudally ,compressing thecal sca and Lt exit nerve roots.",0,1
194,local_data/mendeley/T1_0250_D5.png,"LSS MRI :  Feature of muscle spasm.  L1/L2 level:  posterior central disc protrusion largly compressing thecal sac and both exit nerve roots.  L5/S1 level:  Lt paracentral disc protruion , migrating caudally ,compressing thecal sca and Lt exit nerve roots.",0,1
195,local_data/mendeley/T2_0250_D3.png,"LSS MRI :  Feature of muscle spasm.  L1/L2 level:  posterior central disc protrusion largly compressing thecal sac and both exit nerve roots.  L5/S1 level:  Lt paracentral disc protruion , migrating caudally ,compressing thecal sca and Lt exit nerve roots.",0,1
196,local_data/mendeley/T2_0250_D4.png,"LSS MRI :  Feature of muscle spasm.  L1/L2 level:  posterior central disc protrusion largly compressing thecal sac and both exit nerve roots.  L5/S1 level:  Lt paracentral disc protruion , migrating caudally ,compressing thecal sca and Lt exit nerve roots.",0,1
197,local_data/mendeley/T2_0250_D5.png,"LSS MRI :  Feature of muscle spasm.  L1/L2 level:  posterior central disc protrusion largly compressing thecal sac and both exit nerve roots.  L5/S1 level:  Lt paracentral disc protruion , migrating caudally ,compressing thecal sca and Lt exit nerve roots.",0,1
198,local_data/mendeley/T1_0254_D3.png,"LSS MRI L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, compressing the thecal sac and exit canals with narrow spinal canal. diffuse disc bulges noted also at L3-L4 and L2-L3 levels..",0,1
199,local_data/mendeley/T1_0254_D4.png,"LSS MRI L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, compressing the thecal sac and exit canals with narrow spinal canal. diffuse disc bulges noted also at L3-L4 and L2-L3 levels..",0,1
200,local_data/mendeley/T1_0254_D5.png,"LSS MRI L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, compressing the thecal sac and exit canals with narrow spinal canal. diffuse disc bulges noted also at L3-L4 and L2-L3 levels..",0,1
201,local_data/mendeley/T2_0254_D3.png,"LSS MRI L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, compressing the thecal sac and exit canals with narrow spinal canal. diffuse disc bulges noted also at L3-L4 and L2-L3 levels..",0,1
202,local_data/mendeley/T2_0254_D4.png,"LSS MRI L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, compressing the thecal sac and exit canals with narrow spinal canal. diffuse disc bulges noted also at L3-L4 and L2-L3 levels..",0,1
203,local_data/mendeley/T2_0254_D5.png,"LSS MRI L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, compressing the thecal sac and exit canals with narrow spinal canal. diffuse disc bulges noted also at L3-L4 and L2-L3 levels..",0,1
204,local_data/mendeley/T1_0261_D3.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
205,local_data/mendeley/T1_0261_D4.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
206,local_data/mendeley/T1_0261_D5.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
207,local_data/mendeley/T2_0261_D3.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
208,local_data/mendeley/T2_0261_D4.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
209,local_data/mendeley/T2_0261_D5.png,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1
210,local_data/mendeley/T1_0268_D3.png,"MRI OF THE LSS  - feature of muscle spasm.  Small Lt foraminal disc protrusion noted at L4-L5 level , compressing the Lt sided nerves canals.",0,1
211,local_data/mendeley/T1_0268_D4.png,"MRI OF THE LSS  - feature of muscle spasm.  Small Lt foraminal disc protrusion noted at L4-L5 level , compressing the Lt sided nerves canals.",0,1
212,local_data/mendeley/T1_0268_D5.png,"MRI OF THE LSS  - feature of muscle spasm.  Small Lt foraminal disc protrusion noted at L4-L5 level , compressing the Lt sided nerves canals.",0,1
213,local_data/mendeley/T2_0268_D3.png,"MRI OF THE LSS  - feature of muscle spasm.  Small Lt foraminal disc protrusion noted at L4-L5 level , compressing the Lt sided nerves canals.",0,1
214,local_data/mendeley/T2_0268_D4.png,"MRI OF THE LSS  - feature of muscle spasm.  Small Lt foraminal disc protrusion noted at L4-L5 level , compressing the Lt sided nerves canals.",0,1
215,local_data/mendeley/T2_0268_D5.png,"MRI OF THE LSS  - feature of muscle spasm.  Small Lt foraminal disc protrusion noted at L4-L5 level , compressing the Lt sided nerves canals.",0,1
216,local_data/mendeley/T1_0271_D3.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit neural canals.",0,1
217,local_data/mendeley/T1_0271_D4.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit neural canals.",0,1
218,local_data/mendeley/T1_0271_D5.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit neural canals.",0,1
219,local_data/mendeley/T2_0271_D3.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit neural canals.",0,1
220,local_data/mendeley/T2_0271_D4.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit neural canals.",0,1
221,local_data/mendeley/T2_0271_D5.png,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit neural canals.",0,1
222,local_data/mendeley/T1_0274_D3.png,L3-L4: Narrowed disc space with mild disc bulge. L4-L5: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal.,0,1
223,local_data/mendeley/T1_0274_D4.png,L3-L4: Narrowed disc space with mild disc bulge. L4-L5: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal.,0,1
224,local_data/mendeley/T1_0274_D5.png,L3-L4: Narrowed disc space with mild disc bulge. L4-L5: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal.,0,1
225,local_data/mendeley/T2_0274_D3.png,L3-L4: Narrowed disc space with mild disc bulge. L4-L5: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal.,0,1
226,local_data/mendeley/T2_0274_D4.png,L3-L4: Narrowed disc space with mild disc bulge. L4-L5: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal.,0,1
227,local_data/mendeley/T2_0274_D5.png,L3-L4: Narrowed disc space with mild disc bulge. L4-L5: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal.,0,1
228,local_data/mendeley/T1_0275_D3.png,Lumbosacral MRI Feature of muscle spasm noted Small posterior central disc protrusion noted at L5-S1 mildly indenting thecal sac and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 mild compressing the thecal sac and the needle roots,0,1
229,local_data/mendeley/T1_0275_D4.png,Lumbosacral MRI Feature of muscle spasm noted Small posterior central disc protrusion noted at L5-S1 mildly indenting thecal sac and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 mild compressing the thecal sac and the needle roots,0,1
230,local_data/mendeley/T1_0275_D5.png,Lumbosacral MRI Feature of muscle spasm noted Small posterior central disc protrusion noted at L5-S1 mildly indenting thecal sac and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 mild compressing the thecal sac and the needle roots,0,1
231,local_data/mendeley/T2_0275_D3.png,Lumbosacral MRI Feature of muscle spasm noted Small posterior central disc protrusion noted at L5-S1 mildly indenting thecal sac and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 mild compressing the thecal sac and the needle roots,0,1
232,local_data/mendeley/T2_0275_D4.png,Lumbosacral MRI Feature of muscle spasm noted Small posterior central disc protrusion noted at L5-S1 mildly indenting thecal sac and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 mild compressing the thecal sac and the needle roots,0,1
233,local_data/mendeley/T2_0275_D5.png,Lumbosacral MRI Feature of muscle spasm noted Small posterior central disc protrusion noted at L5-S1 mildly indenting thecal sac and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 mild compressing the thecal sac and the needle roots,0,1
234,local_data/mendeley/T1_0278_D3.png,"LSS MRI  Diffuse disc bulge noted at L5-S1 level, more to the Rt side, compressing the thecal sac and Rt exit canal.",0,1
235,local_data/mendeley/T1_0278_D4.png,"LSS MRI  Diffuse disc bulge noted at L5-S1 level, more to the Rt side, compressing the thecal sac and Rt exit canal.",0,1
236,local_data/mendeley/T1_0278_D5.png,"LSS MRI  Diffuse disc bulge noted at L5-S1 level, more to the Rt side, compressing the thecal sac and Rt exit canal.",0,1
237,local_data/mendeley/T2_0278_D3.png,"LSS MRI  Diffuse disc bulge noted at L5-S1 level, more to the Rt side, compressing the thecal sac and Rt exit canal.",0,1
238,local_data/mendeley/T2_0278_D4.png,"LSS MRI  Diffuse disc bulge noted at L5-S1 level, more to the Rt side, compressing the thecal sac and Rt exit canal.",0,1
239,local_data/mendeley/T2_0278_D5.png,"LSS MRI  Diffuse disc bulge noted at L5-S1 level, more to the Rt side, compressing the thecal sac and Rt exit canal.",0,1
240,local_data/mendeley/T1_0280_D3.png,"MRI OF THE L. SPINE  Feature of muscle spasm.Desiccating disc materials.  Diffuse disc bulge noted at L4/L5 & L5/S1 levels , mild compressing thecal sac and both exit nerve roots, annular tear.",0,1
241,local_data/mendeley/T1_0280_D4.png,"MRI OF THE L. SPINE  Feature of muscle spasm.Desiccating disc materials.  Diffuse disc bulge noted at L4/L5 & L5/S1 levels , mild compressing thecal sac and both exit nerve roots, annular tear.",0,1
242,local_data/mendeley/T1_0280_D5.png,"MRI OF THE L. SPINE  Feature of muscle spasm.Desiccating disc materials.  Diffuse disc bulge noted at L4/L5 & L5/S1 levels , mild compressing thecal sac and both exit nerve roots, annular tear.",0,1
243,local_data/mendeley/T2_0280_D3.png,"MRI OF THE L. SPINE  Feature of muscle spasm.Desiccating disc materials.  Diffuse disc bulge noted at L4/L5 & L5/S1 levels , mild compressing thecal sac and both exit nerve roots, annular tear.",0,1
244,local_data/mendeley/T2_0280_D4.png,"MRI OF THE L. SPINE  Feature of muscle spasm.Desiccating disc materials.  Diffuse disc bulge noted at L4/L5 & L5/S1 levels , mild compressing thecal sac and both exit nerve roots, annular tear.",0,1
245,local_data/mendeley/T2_0280_D5.png,"MRI OF THE L. SPINE  Feature of muscle spasm.Desiccating disc materials.  Diffuse disc bulge noted at L4/L5 & L5/S1 levels , mild compressing thecal sac and both exit nerve roots, annular tear.",0,1
246,local_data/mendeley/T1_0291_D3.png,LSS MRI L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4: Mild disc bulge noted.  Adequate spinal canal,0,1
247,local_data/mendeley/T1_0291_D4.png,LSS MRI L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4: Mild disc bulge noted.  Adequate spinal canal,0,1
248,local_data/mendeley/T1_0291_D5.png,LSS MRI L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4: Mild disc bulge noted.  Adequate spinal canal,0,1
249,local_data/mendeley/T2_0291_D3.png,LSS MRI L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4: Mild disc bulge noted.  Adequate spinal canal,0,1
250,local_data/mendeley/T2_0291_D4.png,LSS MRI L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4: Mild disc bulge noted.  Adequate spinal canal,0,1
251,local_data/mendeley/T2_0291_D5.png,LSS MRI L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4: Mild disc bulge noted.  Adequate spinal canal,0,1
252,local_data/mendeley/T1_0294_D3.png,"Multilevel mild disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level (Lt side), abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
253,local_data/mendeley/T1_0294_D4.png,"Multilevel mild disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level (Lt side), abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
254,local_data/mendeley/T1_0294_D5.png,"Multilevel mild disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level (Lt side), abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
255,local_data/mendeley/T2_0294_D3.png,"Multilevel mild disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level (Lt side), abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
256,local_data/mendeley/T2_0294_D4.png,"Multilevel mild disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level (Lt side), abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
257,local_data/mendeley/T2_0294_D5.png,"Multilevel mild disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level (Lt side), abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1
258,local_data/mendeley/T1_0308_D3.png,"LSS MRI  L5-S1: diffuse disc bulge, more to the Lt side.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit canals.  about 1.3 cm, Lt paracentral soft tissue fullness noted posterior to lower L4 vertebra, probably detached disc material. Contrast re-evaluation is recommended.",0,1
259,local_data/mendeley/T1_0308_D4.png,"LSS MRI  L5-S1: diffuse disc bulge, more to the Lt side.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit canals.  about 1.3 cm, Lt paracentral soft tissue fullness noted posterior to lower L4 vertebra, probably detached disc material. Contrast re-evaluation is recommended.",0,1
260,local_data/mendeley/T1_0308_D5.png,"LSS MRI  L5-S1: diffuse disc bulge, more to the Lt side.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit canals.  about 1.3 cm, Lt paracentral soft tissue fullness noted posterior to lower L4 vertebra, probably detached disc material. Contrast re-evaluation is recommended.",0,1
261,local_data/mendeley/T2_0308_D3.png,"LSS MRI  L5-S1: diffuse disc bulge, more to the Lt side.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit canals.  about 1.3 cm, Lt paracentral soft tissue fullness noted posterior to lower L4 vertebra, probably detached disc material. Contrast re-evaluation is recommended.",0,1
262,local_data/mendeley/T2_0308_D4.png,"LSS MRI  L5-S1: diffuse disc bulge, more to the Lt side.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit canals.  about 1.3 cm, Lt paracentral soft tissue fullness noted posterior to lower L4 vertebra, probably detached disc material. Contrast re-evaluation is recommended.",0,1
263,local_data/mendeley/T2_0308_D5.png,"LSS MRI  L5-S1: diffuse disc bulge, more to the Lt side.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit canals.  about 1.3 cm, Lt paracentral soft tissue fullness noted posterior to lower L4 vertebra, probably detached disc material. Contrast re-evaluation is recommended.",0,1
264,local_data/mendeley/T1_0314_D3.png,"LSS MRI : Feature of muscle spasm. Multiple vertebral bodies hemangiomas noted . Diffuse disc bulge noted at L4/L5 level, compression the thecal sac and encroaching both exit neural canals.associated with ligamentum flavum hyperatrophy . .Diffuse disc bulge noted at L3/L4 level, mild compressing thecall sac and nerve roots .",0,1
265,local_data/mendeley/T1_0314_D4.png,"LSS MRI : Feature of muscle spasm. Multiple vertebral bodies hemangiomas noted . Diffuse disc bulge noted at L4/L5 level, compression the thecal sac and encroaching both exit neural canals.associated with ligamentum flavum hyperatrophy . .Diffuse disc bulge noted at L3/L4 level, mild compressing thecall sac and nerve roots .",0,1
266,local_data/mendeley/T1_0314_D5.png,"LSS MRI : Feature of muscle spasm. Multiple vertebral bodies hemangiomas noted . Diffuse disc bulge noted at L4/L5 level, compression the thecal sac and encroaching both exit neural canals.associated with ligamentum flavum hyperatrophy . .Diffuse disc bulge noted at L3/L4 level, mild compressing thecall sac and nerve roots .",0,1
267,local_data/mendeley/T2_0314_D3.png,"LSS MRI : Feature of muscle spasm. Multiple vertebral bodies hemangiomas noted . Diffuse disc bulge noted at L4/L5 level, compression the thecal sac and encroaching both exit neural canals.associated with ligamentum flavum hyperatrophy . .Diffuse disc bulge noted at L3/L4 level, mild compressing thecall sac and nerve roots .",0,1
268,local_data/mendeley/T2_0314_D4.png,"LSS MRI : Feature of muscle spasm. Multiple vertebral bodies hemangiomas noted . Diffuse disc bulge noted at L4/L5 level, compression the thecal sac and encroaching both exit neural canals.associated with ligamentum flavum hyperatrophy . .Diffuse disc bulge noted at L3/L4 level, mild compressing thecall sac and nerve roots .",0,1
269,local_data/mendeley/T2_0314_D5.png,"LSS MRI : Feature of muscle spasm. Multiple vertebral bodies hemangiomas noted . Diffuse disc bulge noted at L4/L5 level, compression the thecal sac and encroaching both exit neural canals.associated with ligamentum flavum hyperatrophy . .Diffuse disc bulge noted at L3/L4 level, mild compressing thecall sac and nerve roots .",0,1
270,local_data/mendeley/T1_0318_D3.png,LSS MRI :  Grade I spondylolithesis of L4 over L5 level noted with end palate degenertaive change.,0,1
271,local_data/mendeley/T1_0318_D4.png,LSS MRI :  Grade I spondylolithesis of L4 over L5 level noted with end palate degenertaive change.,0,1
272,local_data/mendeley/T1_0318_D5.png,LSS MRI :  Grade I spondylolithesis of L4 over L5 level noted with end palate degenertaive change.,0,1
273,local_data/mendeley/T2_0318_D3.png,LSS MRI :  Grade I spondylolithesis of L4 over L5 level noted with end palate degenertaive change.,0,1
274,local_data/mendeley/T2_0318_D4.png,LSS MRI :  Grade I spondylolithesis of L4 over L5 level noted with end palate degenertaive change.,0,1
275,local_data/mendeley/T2_0318_D5.png,LSS MRI :  Grade I spondylolithesis of L4 over L5 level noted with end palate degenertaive change.,0,1
276,local_data/mendeley/T1_0331_D3.png,L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals.,0,1
277,local_data/mendeley/T1_0331_D4.png,L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals.,0,1
278,local_data/mendeley/T1_0331_D5.png,L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals.,0,1
279,local_data/mendeley/T2_0331_D3.png,L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals.,0,1
280,local_data/mendeley/T2_0331_D4.png,L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals.,0,1
281,local_data/mendeley/T2_0331_D5.png,L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals.,0,1
282,local_data/mendeley/T1_0337_D3.png,Lumbosacral MRI  Mild disc bulges noted at L4-5 and L5-S1 levels  No significant thecal sac or nerve root compression noted  Multilevel Schmorl's nodules noted  Adequate spinal canal,0,1
283,local_data/mendeley/T1_0337_D4.png,Lumbosacral MRI  Mild disc bulges noted at L4-5 and L5-S1 levels  No significant thecal sac or nerve root compression noted  Multilevel Schmorl's nodules noted  Adequate spinal canal,0,1
284,local_data/mendeley/T1_0337_D5.png,Lumbosacral MRI  Mild disc bulges noted at L4-5 and L5-S1 levels  No significant thecal sac or nerve root compression noted  Multilevel Schmorl's nodules noted  Adequate spinal canal,0,1
285,local_data/mendeley/T2_0337_D3.png,Lumbosacral MRI  Mild disc bulges noted at L4-5 and L5-S1 levels  No significant thecal sac or nerve root compression noted  Multilevel Schmorl's nodules noted  Adequate spinal canal,0,1
286,local_data/mendeley/T2_0337_D4.png,Lumbosacral MRI  Mild disc bulges noted at L4-5 and L5-S1 levels  No significant thecal sac or nerve root compression noted  Multilevel Schmorl's nodules noted  Adequate spinal canal,0,1
287,local_data/mendeley/T2_0337_D5.png,Lumbosacral MRI  Mild disc bulges noted at L4-5 and L5-S1 levels  No significant thecal sac or nerve root compression noted  Multilevel Schmorl's nodules noted  Adequate spinal canal,0,1
288,local_data/mendeley/T1_0351_D3.png,LSS MRI: Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
289,local_data/mendeley/T1_0351_D4.png,LSS MRI: Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
290,local_data/mendeley/T1_0351_D5.png,LSS MRI: Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
291,local_data/mendeley/T2_0351_D3.png,LSS MRI: Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
292,local_data/mendeley/T2_0351_D4.png,LSS MRI: Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
293,local_data/mendeley/T2_0351_D5.png,LSS MRI: Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
294,local_data/mendeley/T1_0356_D3.png,"LSS MRI  L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate.  large anterior bulge of the same abovementioned disc noted.  posterior osteophytes noted.  Type I end plate degeneration noted.",0,1
295,local_data/mendeley/T1_0356_D4.png,"LSS MRI  L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate.  large anterior bulge of the same abovementioned disc noted.  posterior osteophytes noted.  Type I end plate degeneration noted.",0,1
296,local_data/mendeley/T1_0356_D5.png,"LSS MRI  L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate.  large anterior bulge of the same abovementioned disc noted.  posterior osteophytes noted.  Type I end plate degeneration noted.",0,1
297,local_data/mendeley/T2_0356_D3.png,"LSS MRI  L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate.  large anterior bulge of the same abovementioned disc noted.  posterior osteophytes noted.  Type I end plate degeneration noted.",0,1
298,local_data/mendeley/T2_0356_D4.png,"LSS MRI  L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate.  large anterior bulge of the same abovementioned disc noted.  posterior osteophytes noted.  Type I end plate degeneration noted.",0,1
299,local_data/mendeley/T2_0356_D5.png,"LSS MRI  L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate.  large anterior bulge of the same abovementioned disc noted.  posterior osteophytes noted.  Type I end plate degeneration noted.",0,1
300,local_data/mendeley/T1_0357_D3.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression noted .  No disc protrusion or herniation seen .,1,0
301,local_data/mendeley/T1_0357_D4.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression noted .  No disc protrusion or herniation seen .,1,0
302,local_data/mendeley/T1_0357_D5.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression noted .  No disc protrusion or herniation seen .,1,0
303,local_data/mendeley/T2_0357_D3.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression noted .  No disc protrusion or herniation seen .,1,0
304,local_data/mendeley/T2_0357_D4.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression noted .  No disc protrusion or herniation seen .,1,0
305,local_data/mendeley/T2_0357_D5.png,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression noted .  No disc protrusion or herniation seen .,1,0
306,local_data/mendeley/T1_0364_D3.png,"LSS MRI :  Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
307,local_data/mendeley/T1_0364_D4.png,"LSS MRI :  Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
308,local_data/mendeley/T1_0364_D5.png,"LSS MRI :  Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
309,local_data/mendeley/T2_0364_D3.png,"LSS MRI :  Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
310,local_data/mendeley/T2_0364_D4.png,"LSS MRI :  Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
311,local_data/mendeley/T2_0364_D5.png,"LSS MRI :  Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
312,local_data/mendeley/T1_0366_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
313,local_data/mendeley/T1_0366_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
314,local_data/mendeley/T1_0366_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
315,local_data/mendeley/T2_0366_D3.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
316,local_data/mendeley/T2_0366_D4.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
317,local_data/mendeley/T2_0366_D5.png,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
318,local_data/mendeley/T1_0367_D3.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
319,local_data/mendeley/T1_0367_D4.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
320,local_data/mendeley/T1_0367_D5.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
321,local_data/mendeley/T2_0367_D3.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
322,local_data/mendeley/T2_0367_D4.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
323,local_data/mendeley/T2_0367_D5.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
324,local_data/mendeley/T1_0370_D3.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
325,local_data/mendeley/T1_0370_D4.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
326,local_data/mendeley/T1_0370_D5.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
327,local_data/mendeley/T2_0370_D3.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
328,local_data/mendeley/T2_0370_D4.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
329,local_data/mendeley/T2_0370_D5.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
330,local_data/mendeley/T1_0380_D3.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
331,local_data/mendeley/T1_0380_D4.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
332,local_data/mendeley/T1_0380_D5.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
333,local_data/mendeley/T2_0380_D3.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
334,local_data/mendeley/T2_0380_D4.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
335,local_data/mendeley/T2_0380_D5.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
336,local_data/mendeley/T1_0390_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
337,local_data/mendeley/T1_0390_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
338,local_data/mendeley/T1_0390_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
339,local_data/mendeley/T2_0390_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
340,local_data/mendeley/T2_0390_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
341,local_data/mendeley/T2_0390_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
342,local_data/mendeley/T1_0391_D3.png,"LSS MRI :  feature of muscle spasm.  schorls nodules noted .  Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
343,local_data/mendeley/T1_0391_D4.png,"LSS MRI :  feature of muscle spasm.  schorls nodules noted .  Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
344,local_data/mendeley/T1_0391_D5.png,"LSS MRI :  feature of muscle spasm.  schorls nodules noted .  Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
345,local_data/mendeley/T2_0391_D3.png,"LSS MRI :  feature of muscle spasm.  schorls nodules noted .  Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
346,local_data/mendeley/T2_0391_D4.png,"LSS MRI :  feature of muscle spasm.  schorls nodules noted .  Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
347,local_data/mendeley/T2_0391_D5.png,"LSS MRI :  feature of muscle spasm.  schorls nodules noted .  Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
348,local_data/mendeley/T1_0351_D3.png,LSS MRI Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
349,local_data/mendeley/T1_0351_D4.png,LSS MRI Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
350,local_data/mendeley/T1_0351_D5.png,LSS MRI Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
351,local_data/mendeley/T2_0351_D3.png,LSS MRI Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
352,local_data/mendeley/T2_0351_D4.png,LSS MRI Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
353,local_data/mendeley/T2_0351_D5.png,LSS MRI Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1
354,local_data/mendeley/T1_0356_D3.png,"LSS MRI L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate. large anterior bulge of the same abovementioned disc noted. posterior osteophytes noted. Type I end plate degeneration noted.",0,1
355,local_data/mendeley/T1_0356_D4.png,"LSS MRI L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate. large anterior bulge of the same abovementioned disc noted. posterior osteophytes noted. Type I end plate degeneration noted.",0,1
356,local_data/mendeley/T1_0356_D5.png,"LSS MRI L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate. large anterior bulge of the same abovementioned disc noted. posterior osteophytes noted. Type I end plate degeneration noted.",0,1
357,local_data/mendeley/T2_0356_D3.png,"LSS MRI L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate. large anterior bulge of the same abovementioned disc noted. posterior osteophytes noted. Type I end plate degeneration noted.",0,1
358,local_data/mendeley/T2_0356_D4.png,"LSS MRI L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate. large anterior bulge of the same abovementioned disc noted. posterior osteophytes noted. Type I end plate degeneration noted.",0,1
359,local_data/mendeley/T2_0356_D5.png,"LSS MRI L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate. large anterior bulge of the same abovementioned disc noted. posterior osteophytes noted. Type I end plate degeneration noted.",0,1
360,local_data/mendeley/T1_0357_D3.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve roots compression noted . No disc protrusion or herniation seen .,1,0
361,local_data/mendeley/T1_0357_D4.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve roots compression noted . No disc protrusion or herniation seen .,1,0
362,local_data/mendeley/T1_0357_D5.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve roots compression noted . No disc protrusion or herniation seen .,1,0
363,local_data/mendeley/T2_0357_D3.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve roots compression noted . No disc protrusion or herniation seen .,1,0
364,local_data/mendeley/T2_0357_D4.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve roots compression noted . No disc protrusion or herniation seen .,1,0
365,local_data/mendeley/T2_0357_D5.png,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve roots compression noted . No disc protrusion or herniation seen .,1,0
366,local_data/mendeley/T1_0364_D3.png,"LSS MRI : Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
367,local_data/mendeley/T1_0364_D4.png,"LSS MRI : Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
368,local_data/mendeley/T1_0364_D5.png,"LSS MRI : Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
369,local_data/mendeley/T2_0364_D3.png,"LSS MRI : Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
370,local_data/mendeley/T2_0364_D4.png,"LSS MRI : Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
371,local_data/mendeley/T2_0364_D5.png,"LSS MRI : Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1
372,local_data/mendeley/T1_0366_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
373,local_data/mendeley/T1_0366_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
374,local_data/mendeley/T1_0366_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
375,local_data/mendeley/T2_0366_D3.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
376,local_data/mendeley/T2_0366_D4.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
377,local_data/mendeley/T2_0366_D5.png,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1
378,local_data/mendeley/T1_0367_D3.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
379,local_data/mendeley/T1_0367_D4.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
380,local_data/mendeley/T1_0367_D5.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
381,local_data/mendeley/T2_0367_D3.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
382,local_data/mendeley/T2_0367_D4.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
383,local_data/mendeley/T2_0367_D5.png,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
384,local_data/mendeley/T1_0370_D3.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
385,local_data/mendeley/T1_0370_D4.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
386,local_data/mendeley/T1_0370_D5.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
387,local_data/mendeley/T2_0370_D3.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
388,local_data/mendeley/T2_0370_D4.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
389,local_data/mendeley/T2_0370_D5.png,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1
390,local_data/mendeley/T1_0380_D3.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
391,local_data/mendeley/T1_0380_D4.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
392,local_data/mendeley/T1_0380_D5.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
393,local_data/mendeley/T2_0380_D3.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
394,local_data/mendeley/T2_0380_D4.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
395,local_data/mendeley/T2_0380_D5.png,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1
396,local_data/mendeley/T1_0390_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
397,local_data/mendeley/T1_0390_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
398,local_data/mendeley/T1_0390_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
399,local_data/mendeley/T2_0390_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
400,local_data/mendeley/T2_0390_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
401,local_data/mendeley/T2_0390_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0
402,local_data/mendeley/T1_0391_D3.png,"LSS MRI : feature of muscle spasm. schorls nodules noted . Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
403,local_data/mendeley/T1_0391_D4.png,"LSS MRI : feature of muscle spasm. schorls nodules noted . Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
404,local_data/mendeley/T1_0391_D5.png,"LSS MRI : feature of muscle spasm. schorls nodules noted . Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
405,local_data/mendeley/T2_0391_D3.png,"LSS MRI : feature of muscle spasm. schorls nodules noted . Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
406,local_data/mendeley/T2_0391_D4.png,"LSS MRI : feature of muscle spasm. schorls nodules noted . Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
407,local_data/mendeley/T2_0391_D5.png,"LSS MRI : feature of muscle spasm. schorls nodules noted . Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1
408,local_data/mendeley/T1_0422_D3.png,"LSS MRI  Mild disc bulge noted at L4-L5 level, abutting the thecal sac and encroaching upon the exit canals.  L2 hemangioma.  3 cm, Lt ovarian cyst.",0,1
409,local_data/mendeley/T1_0422_D4.png,"LSS MRI  Mild disc bulge noted at L4-L5 level, abutting the thecal sac and encroaching upon the exit canals.  L2 hemangioma.  3 cm, Lt ovarian cyst.",0,1
410,local_data/mendeley/T1_0422_D5.png,"LSS MRI  Mild disc bulge noted at L4-L5 level, abutting the thecal sac and encroaching upon the exit canals.  L2 hemangioma.  3 cm, Lt ovarian cyst.",0,1
411,local_data/mendeley/T2_0422_D3.png,"LSS MRI  Mild disc bulge noted at L4-L5 level, abutting the thecal sac and encroaching upon the exit canals.  L2 hemangioma.  3 cm, Lt ovarian cyst.",0,1
412,local_data/mendeley/T2_0422_D4.png,"LSS MRI  Mild disc bulge noted at L4-L5 level, abutting the thecal sac and encroaching upon the exit canals.  L2 hemangioma.  3 cm, Lt ovarian cyst.",0,1
413,local_data/mendeley/T2_0422_D5.png,"LSS MRI  Mild disc bulge noted at L4-L5 level, abutting the thecal sac and encroaching upon the exit canals.  L2 hemangioma.  3 cm, Lt ovarian cyst.",0,1
414,local_data/mendeley/T1_0437_D3.png,"LSS MRI :  feature of muscle spasm.  diffuse disc bulges noted at L4/L5 levels, extending to lateral recess, compressing the thecal sac and both nerve roots",0,1
415,local_data/mendeley/T1_0437_D4.png,"LSS MRI :  feature of muscle spasm.  diffuse disc bulges noted at L4/L5 levels, extending to lateral recess, compressing the thecal sac and both nerve roots",0,1
416,local_data/mendeley/T1_0437_D5.png,"LSS MRI :  feature of muscle spasm.  diffuse disc bulges noted at L4/L5 levels, extending to lateral recess, compressing the thecal sac and both nerve roots",0,1
417,local_data/mendeley/T2_0437_D3.png,"LSS MRI :  feature of muscle spasm.  diffuse disc bulges noted at L4/L5 levels, extending to lateral recess, compressing the thecal sac and both nerve roots",0,1
418,local_data/mendeley/T2_0437_D4.png,"LSS MRI :  feature of muscle spasm.  diffuse disc bulges noted at L4/L5 levels, extending to lateral recess, compressing the thecal sac and both nerve roots",0,1
419,local_data/mendeley/T2_0437_D5.png,"LSS MRI :  feature of muscle spasm.  diffuse disc bulges noted at L4/L5 levels, extending to lateral recess, compressing the thecal sac and both nerve roots",0,1
420,local_data/mendeley/T1_0444_D3.png,"LSS MRI :  posterior central disc protrusion noted at L5/S1 level, crossing & migrating caudally to Rt thecal sac, largely compressing the thecal sac and Rt S1 nerve root.",0,1
421,local_data/mendeley/T1_0444_D4.png,"LSS MRI :  posterior central disc protrusion noted at L5/S1 level, crossing & migrating caudally to Rt thecal sac, largely compressing the thecal sac and Rt S1 nerve root.",0,1
422,local_data/mendeley/T1_0444_D5.png,"LSS MRI :  posterior central disc protrusion noted at L5/S1 level, crossing & migrating caudally to Rt thecal sac, largely compressing the thecal sac and Rt S1 nerve root.",0,1
423,local_data/mendeley/T2_0444_D3.png,"LSS MRI :  posterior central disc protrusion noted at L5/S1 level, crossing & migrating caudally to Rt thecal sac, largely compressing the thecal sac and Rt S1 nerve root.",0,1
424,local_data/mendeley/T2_0444_D4.png,"LSS MRI :  posterior central disc protrusion noted at L5/S1 level, crossing & migrating caudally to Rt thecal sac, largely compressing the thecal sac and Rt S1 nerve root.",0,1
425,local_data/mendeley/T2_0444_D5.png,"LSS MRI :  posterior central disc protrusion noted at L5/S1 level, crossing & migrating caudally to Rt thecal sac, largely compressing the thecal sac and Rt S1 nerve root.",0,1
426,local_data/mendeley/T1_0446_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Discating disc material with degeneraive changes  -Multilevel diffuse disc bulges noted , notceably at L3/L4 level , compressing  thecal sac and encroaching upon exit neural canals asoociated with ligementum flavum hyperatrophy , all leading to secandary spinal canal stenosis",0,1
427,local_data/mendeley/T1_0446_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Discating disc material with degeneraive changes  -Multilevel diffuse disc bulges noted , notceably at L3/L4 level , compressing  thecal sac and encroaching upon exit neural canals asoociated with ligementum flavum hyperatrophy , all leading to secandary spinal canal stenosis",0,1
428,local_data/mendeley/T1_0446_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Discating disc material with degeneraive changes  -Multilevel diffuse disc bulges noted , notceably at L3/L4 level , compressing  thecal sac and encroaching upon exit neural canals asoociated with ligementum flavum hyperatrophy , all leading to secandary spinal canal stenosis",0,1
429,local_data/mendeley/T2_0446_D3.png,"LSS MRI :  -Feature of muscle spasm.  -Discating disc material with degeneraive changes  -Multilevel diffuse disc bulges noted , notceably at L3/L4 level , compressing  thecal sac and encroaching upon exit neural canals asoociated with ligementum flavum hyperatrophy , all leading to secandary spinal canal stenosis",0,1
430,local_data/mendeley/T2_0446_D4.png,"LSS MRI :  -Feature of muscle spasm.  -Discating disc material with degeneraive changes  -Multilevel diffuse disc bulges noted , notceably at L3/L4 level , compressing  thecal sac and encroaching upon exit neural canals asoociated with ligementum flavum hyperatrophy , all leading to secandary spinal canal stenosis",0,1
431,local_data/mendeley/T2_0446_D5.png,"LSS MRI :  -Feature of muscle spasm.  -Discating disc material with degeneraive changes  -Multilevel diffuse disc bulges noted , notceably at L3/L4 level , compressing  thecal sac and encroaching upon exit neural canals asoociated with ligementum flavum hyperatrophy , all leading to secandary spinal canal stenosis",0,1
432,local_data/mendeley/T1_0453_D3.png,LSS MRI  Central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
433,local_data/mendeley/T1_0453_D4.png,LSS MRI  Central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
434,local_data/mendeley/T1_0453_D5.png,LSS MRI  Central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
435,local_data/mendeley/T2_0453_D3.png,LSS MRI  Central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
436,local_data/mendeley/T2_0453_D4.png,LSS MRI  Central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
437,local_data/mendeley/T2_0453_D5.png,LSS MRI  Central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1
438,local_data/mendeley/T1_0457_D3.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots .  Mild disc bulges noted at L5/S1 level.",0,1
439,local_data/mendeley/T1_0457_D4.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots .  Mild disc bulges noted at L5/S1 level.",0,1
440,local_data/mendeley/T1_0457_D5.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots .  Mild disc bulges noted at L5/S1 level.",0,1
441,local_data/mendeley/T2_0457_D3.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots .  Mild disc bulges noted at L5/S1 level.",0,1
442,local_data/mendeley/T2_0457_D4.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots .  Mild disc bulges noted at L5/S1 level.",0,1
443,local_data/mendeley/T2_0457_D5.png,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots .  Mild disc bulges noted at L5/S1 level.",0,1
444,local_data/mendeley/T1_0469_D3.png,"LSS MRI :  -Degenerative changes noted with dissicating disc materials.  -Multilevel diffuse disc bulges noted , noteceably at L4/L5 level and L2/L3 levels, compressing the thecal sca and encroaching upon exit neural canals , associated with ligamentum flavum hyperatrophy , relatively causing secandary spinal canal stnosis NB:  Small Rt renal cyst noted .",0,1
445,local_data/mendeley/T1_0469_D4.png,"LSS MRI :  -Degenerative changes noted with dissicating disc materials.  -Multilevel diffuse disc bulges noted , noteceably at L4/L5 level and L2/L3 levels, compressing the thecal sca and encroaching upon exit neural canals , associated with ligamentum flavum hyperatrophy , relatively causing secandary spinal canal stnosis NB:  Small Rt renal cyst noted .",0,1
446,local_data/mendeley/T1_0469_D5.png,"LSS MRI :  -Degenerative changes noted with dissicating disc materials.  -Multilevel diffuse disc bulges noted , noteceably at L4/L5 level and L2/L3 levels, compressing the thecal sca and encroaching upon exit neural canals , associated with ligamentum flavum hyperatrophy , relatively causing secandary spinal canal stnosis NB:  Small Rt renal cyst noted .",0,1
447,local_data/mendeley/T2_0469_D3.png,"LSS MRI :  -Degenerative changes noted with dissicating disc materials.  -Multilevel diffuse disc bulges noted , noteceably at L4/L5 level and L2/L3 levels, compressing the thecal sca and encroaching upon exit neural canals , associated with ligamentum flavum hyperatrophy , relatively causing secandary spinal canal stnosis NB:  Small Rt renal cyst noted .",0,1
448,local_data/mendeley/T2_0469_D4.png,"LSS MRI :  -Degenerative changes noted with dissicating disc materials.  -Multilevel diffuse disc bulges noted , noteceably at L4/L5 level and L2/L3 levels, compressing the thecal sca and encroaching upon exit neural canals , associated with ligamentum flavum hyperatrophy , relatively causing secandary spinal canal stnosis NB:  Small Rt renal cyst noted .",0,1
449,local_data/mendeley/T2_0469_D5.png,"LSS MRI :  -Degenerative changes noted with dissicating disc materials.  -Multilevel diffuse disc bulges noted , noteceably at L4/L5 level and L2/L3 levels, compressing the thecal sca and encroaching upon exit neural canals , associated with ligamentum flavum hyperatrophy , relatively causing secandary spinal canal stnosis NB:  Small Rt renal cyst noted .",0,1
450,local_data/mendeley/T1_0474_D3.png,"LSS MRI :  Feature of Muscle spasm.  Modec Type I , noted at L5/S1 level.  Diffuse disc bulges noted L5/S1 level , compressing thecal sac and both exit nerve roots.",0,1
451,local_data/mendeley/T1_0474_D4.png,"LSS MRI :  Feature of Muscle spasm.  Modec Type I , noted at L5/S1 level.  Diffuse disc bulges noted L5/S1 level , compressing thecal sac and both exit nerve roots.",0,1
452,local_data/mendeley/T1_0474_D5.png,"LSS MRI :  Feature of Muscle spasm.  Modec Type I , noted at L5/S1 level.  Diffuse disc bulges noted L5/S1 level , compressing thecal sac and both exit nerve roots.",0,1
453,local_data/mendeley/T2_0474_D3.png,"LSS MRI :  Feature of Muscle spasm.  Modec Type I , noted at L5/S1 level.  Diffuse disc bulges noted L5/S1 level , compressing thecal sac and both exit nerve roots.",0,1
454,local_data/mendeley/T2_0474_D4.png,"LSS MRI :  Feature of Muscle spasm.  Modec Type I , noted at L5/S1 level.  Diffuse disc bulges noted L5/S1 level , compressing thecal sac and both exit nerve roots.",0,1
455,local_data/mendeley/T2_0474_D5.png,"LSS MRI :  Feature of Muscle spasm.  Modec Type I , noted at L5/S1 level.  Diffuse disc bulges noted L5/S1 level , compressing thecal sac and both exit nerve roots.",0,1
456,local_data/mendeley/T1_0486_D3.png,Lumbosacral MRI  Mild disc bulge noted at L3-L4 level  No evidence of disc Herniation at other levels noted  Adequate the spinal canal,1,0
457,local_data/mendeley/T1_0486_D4.png,Lumbosacral MRI  Mild disc bulge noted at L3-L4 level  No evidence of disc Herniation at other levels noted  Adequate the spinal canal,1,0
458,local_data/mendeley/T1_0486_D5.png,Lumbosacral MRI  Mild disc bulge noted at L3-L4 level  No evidence of disc Herniation at other levels noted  Adequate the spinal canal,1,0
459,local_data/mendeley/T2_0486_D3.png,Lumbosacral MRI  Mild disc bulge noted at L3-L4 level  No evidence of disc Herniation at other levels noted  Adequate the spinal canal,1,0
460,local_data/mendeley/T2_0486_D4.png,Lumbosacral MRI  Mild disc bulge noted at L3-L4 level  No evidence of disc Herniation at other levels noted  Adequate the spinal canal,1,0
461,local_data/mendeley/T2_0486_D5.png,Lumbosacral MRI  Mild disc bulge noted at L3-L4 level  No evidence of disc Herniation at other levels noted  Adequate the spinal canal,1,0
462,local_data/mendeley/T1_0502_D3.png,"LSS MRI : -Feature of muscle spasm. -Modic type II end plate degenerative changes of L4/L5 level noted . -Posterior central disc protrusion noted at L4/L5 level , compressing the thecal sca extending to lateral recess encroaching upon both exit neural canals . -Diffuse disc bulge with annular tear noted at L3/L4 level, mild compressing thecal sac and exit nerve roots NB :About 4 cm , ovarian cyst noted .",0,1
463,local_data/mendeley/T1_0502_D4.png,"LSS MRI : -Feature of muscle spasm. -Modic type II end plate degenerative changes of L4/L5 level noted . -Posterior central disc protrusion noted at L4/L5 level , compressing the thecal sca extending to lateral recess encroaching upon both exit neural canals . -Diffuse disc bulge with annular tear noted at L3/L4 level, mild compressing thecal sac and exit nerve roots NB :About 4 cm , ovarian cyst noted .",0,1
464,local_data/mendeley/T1_0502_D5.png,"LSS MRI : -Feature of muscle spasm. -Modic type II end plate degenerative changes of L4/L5 level noted . -Posterior central disc protrusion noted at L4/L5 level , compressing the thecal sca extending to lateral recess encroaching upon both exit neural canals . -Diffuse disc bulge with annular tear noted at L3/L4 level, mild compressing thecal sac and exit nerve roots NB :About 4 cm , ovarian cyst noted .",0,1
465,local_data/mendeley/T2_0502_D3.png,"LSS MRI : -Feature of muscle spasm. -Modic type II end plate degenerative changes of L4/L5 level noted . -Posterior central disc protrusion noted at L4/L5 level , compressing the thecal sca extending to lateral recess encroaching upon both exit neural canals . -Diffuse disc bulge with annular tear noted at L3/L4 level, mild compressing thecal sac and exit nerve roots NB :About 4 cm , ovarian cyst noted .",0,1
466,local_data/mendeley/T2_0502_D4.png,"LSS MRI : -Feature of muscle spasm. -Modic type II end plate degenerative changes of L4/L5 level noted . -Posterior central disc protrusion noted at L4/L5 level , compressing the thecal sca extending to lateral recess encroaching upon both exit neural canals . -Diffuse disc bulge with annular tear noted at L3/L4 level, mild compressing thecal sac and exit nerve roots NB :About 4 cm , ovarian cyst noted .",0,1
467,local_data/mendeley/T2_0502_D5.png,"LSS MRI : -Feature of muscle spasm. -Modic type II end plate degenerative changes of L4/L5 level noted . -Posterior central disc protrusion noted at L4/L5 level , compressing the thecal sca extending to lateral recess encroaching upon both exit neural canals . -Diffuse disc bulge with annular tear noted at L3/L4 level, mild compressing thecal sac and exit nerve roots NB :About 4 cm , ovarian cyst noted .",0,1
468,local_data/mendeley/T1_0505_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
469,local_data/mendeley/T1_0505_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
470,local_data/mendeley/T1_0505_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
471,local_data/mendeley/T2_0505_D3.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
472,local_data/mendeley/T2_0505_D4.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
473,local_data/mendeley/T2_0505_D5.png,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0
474,local_data/mendeley/T1_0508_D3.png,"LSS MRI : Feature of muscle spasm. Dissicating disc material at L4/L5 level: Diffuse disc bulge noted at L4/L5 level , extending to lateral recess, compressing thecal sac , and largely encroaching upon exit neural canal associated with annular tear .",0,1
475,local_data/mendeley/T1_0508_D4.png,"LSS MRI : Feature of muscle spasm. Dissicating disc material at L4/L5 level: Diffuse disc bulge noted at L4/L5 level , extending to lateral recess, compressing thecal sac , and largely encroaching upon exit neural canal associated with annular tear .",0,1
476,local_data/mendeley/T1_0508_D5.png,"LSS MRI : Feature of muscle spasm. Dissicating disc material at L4/L5 level: Diffuse disc bulge noted at L4/L5 level , extending to lateral recess, compressing thecal sac , and largely encroaching upon exit neural canal associated with annular tear .",0,1
477,local_data/mendeley/T2_0508_D3.png,"LSS MRI : Feature of muscle spasm. Dissicating disc material at L4/L5 level: Diffuse disc bulge noted at L4/L5 level , extending to lateral recess, compressing thecal sac , and largely encroaching upon exit neural canal associated with annular tear .",0,1
478,local_data/mendeley/T2_0508_D4.png,"LSS MRI : Feature of muscle spasm. Dissicating disc material at L4/L5 level: Diffuse disc bulge noted at L4/L5 level , extending to lateral recess, compressing thecal sac , and largely encroaching upon exit neural canal associated with annular tear .",0,1
479,local_data/mendeley/T2_0508_D5.png,"LSS MRI : Feature of muscle spasm. Dissicating disc material at L4/L5 level: Diffuse disc bulge noted at L4/L5 level , extending to lateral recess, compressing thecal sac , and largely encroaching upon exit neural canal associated with annular tear .",0,1
480,local_data/mendeley/T1_0518_D3.png,LSS MRI Multilevel mild disc bulges noted. No significant thecal sac or nerve root compression noted. Tarlov cyst- S2 level.,0,1
481,local_data/mendeley/T1_0518_D4.png,LSS MRI Multilevel mild disc bulges noted. No significant thecal sac or nerve root compression noted. Tarlov cyst- S2 level.,0,1
482,local_data/mendeley/T1_0518_D5.png,LSS MRI Multilevel mild disc bulges noted. No significant thecal sac or nerve root compression noted. Tarlov cyst- S2 level.,0,1
483,local_data/mendeley/T2_0518_D3.png,LSS MRI Multilevel mild disc bulges noted. No significant thecal sac or nerve root compression noted. Tarlov cyst- S2 level.,0,1
484,local_data/mendeley/T2_0518_D4.png,LSS MRI Multilevel mild disc bulges noted. No significant thecal sac or nerve root compression noted. Tarlov cyst- S2 level.,0,1
485,local_data/mendeley/T2_0518_D5.png,LSS MRI Multilevel mild disc bulges noted. No significant thecal sac or nerve root compression noted. Tarlov cyst- S2 level.,0,1
486,local_data/mendeley/T1_0519_D3.png,"LSS MRI: Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 level, mildly compressing thecal sac and exit nerve roots",0,1
487,local_data/mendeley/T1_0519_D4.png,"LSS MRI: Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 level, mildly compressing thecal sac and exit nerve roots",0,1
488,local_data/mendeley/T1_0519_D5.png,"LSS MRI: Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 level, mildly compressing thecal sac and exit nerve roots",0,1
489,local_data/mendeley/T2_0519_D3.png,"LSS MRI: Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 level, mildly compressing thecal sac and exit nerve roots",0,1
490,local_data/mendeley/T2_0519_D4.png,"LSS MRI: Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 level, mildly compressing thecal sac and exit nerve roots",0,1
491,local_data/mendeley/T2_0519_D5.png,"LSS MRI: Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 level, mildly compressing thecal sac and exit nerve roots",0,1
492,local_data/mendeley/T1_0543_D3.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted . -Narrowed disc space of L5/S1 level -Multilevel diffuse disc bulges noteceably at L4/L5 &L5/S1 levels, compressing the thecal sac and encroaching upon exit neural canals with ligamentum flavum hyperatrophy. NB:Small hypointense lesion noted at S2 vertebral body , mostly bony scleorois like bony island .",0,1
493,local_data/mendeley/T1_0543_D4.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted . -Narrowed disc space of L5/S1 level -Multilevel diffuse disc bulges noteceably at L4/L5 &L5/S1 levels, compressing the thecal sac and encroaching upon exit neural canals with ligamentum flavum hyperatrophy. NB:Small hypointense lesion noted at S2 vertebral body , mostly bony scleorois like bony island .",0,1
494,local_data/mendeley/T1_0543_D5.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted . -Narrowed disc space of L5/S1 level -Multilevel diffuse disc bulges noteceably at L4/L5 &L5/S1 levels, compressing the thecal sac and encroaching upon exit neural canals with ligamentum flavum hyperatrophy. NB:Small hypointense lesion noted at S2 vertebral body , mostly bony scleorois like bony island .",0,1
495,local_data/mendeley/T2_0543_D3.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted . -Narrowed disc space of L5/S1 level -Multilevel diffuse disc bulges noteceably at L4/L5 &L5/S1 levels, compressing the thecal sac and encroaching upon exit neural canals with ligamentum flavum hyperatrophy. NB:Small hypointense lesion noted at S2 vertebral body , mostly bony scleorois like bony island .",0,1
496,local_data/mendeley/T2_0543_D4.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted . -Narrowed disc space of L5/S1 level -Multilevel diffuse disc bulges noteceably at L4/L5 &L5/S1 levels, compressing the thecal sac and encroaching upon exit neural canals with ligamentum flavum hyperatrophy. NB:Small hypointense lesion noted at S2 vertebral body , mostly bony scleorois like bony island .",0,1
497,local_data/mendeley/T2_0543_D5.png,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted . -Narrowed disc space of L5/S1 level -Multilevel diffuse disc bulges noteceably at L4/L5 &L5/S1 levels, compressing the thecal sac and encroaching upon exit neural canals with ligamentum flavum hyperatrophy. NB:Small hypointense lesion noted at S2 vertebral body , mostly bony scleorois like bony island .",0,1
498,local_data/mendeley/T1_0560_D3.png,"LSS MRI : faeture of muscle spasm. L5/S1 level :(dissicating disc material ): Central disc protruion relatively upwatd migration ,mild compressing thecal sac and exit nerve root .",0,1
499,local_data/mendeley/T1_0560_D4.png,"LSS MRI : faeture of muscle spasm. L5/S1 level :(dissicating disc material ): Central disc protruion relatively upwatd migration ,mild compressing thecal sac and exit nerve root .",0,1
500,local_data/mendeley/T1_0560_D5.png,"LSS MRI : faeture of muscle spasm. L5/S1 level :(dissicating disc material ): Central disc protruion relatively upwatd migration ,mild compressing thecal sac and exit nerve root .",0,1
501,local_data/mendeley/T2_0560_D3.png,"LSS MRI : faeture of muscle spasm. L5/S1 level :(dissicating disc material ): Central disc protruion relatively upwatd migration ,mild compressing thecal sac and exit nerve root .",0,1
502,local_data/mendeley/T2_0560_D4.png,"LSS MRI : faeture of muscle spasm. L5/S1 level :(dissicating disc material ): Central disc protruion relatively upwatd migration ,mild compressing thecal sac and exit nerve root .",0,1
503,local_data/mendeley/T2_0560_D5.png,"LSS MRI : faeture of muscle spasm. L5/S1 level :(dissicating disc material ): Central disc protruion relatively upwatd migration ,mild compressing thecal sac and exit nerve root .",0,1
504,local_data/mendeley/T1_0562_D3.png,Mild disc bulges noted at L23-L3 and L4-L5 levels.,0,1
505,local_data/mendeley/T1_0562_D4.png,Mild disc bulges noted at L23-L3 and L4-L5 levels.,0,1
506,local_data/mendeley/T1_0562_D5.png,Mild disc bulges noted at L23-L3 and L4-L5 levels.,0,1
507,local_data/mendeley/T2_0562_D3.png,Mild disc bulges noted at L23-L3 and L4-L5 levels.,0,1
508,local_data/mendeley/T2_0562_D4.png,Mild disc bulges noted at L23-L3 and L4-L5 levels.,0,1
509,local_data/mendeley/T2_0562_D5.png,Mild disc bulges noted at L23-L3 and L4-L5 levels.,0,1
510,local_data/mendeley/T1_0565_D3.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
511,local_data/mendeley/T1_0565_D4.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
512,local_data/mendeley/T1_0565_D5.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
513,local_data/mendeley/T2_0565_D3.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
514,local_data/mendeley/T2_0565_D4.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
515,local_data/mendeley/T2_0565_D5.png,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1
516,local_data/mendeley/T1_0567_D3.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
517,local_data/mendeley/T1_0567_D4.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
518,local_data/mendeley/T1_0567_D5.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
519,local_data/mendeley/T2_0567_D3.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
520,local_data/mendeley/T2_0567_D4.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
521,local_data/mendeley/T2_0567_D5.png,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1
522,local_data/mendeley/T1_0573_D3.png,"LSS MRI : Feature of muscle spasm. Preveius operation at L4/L5 level ( fenestration). Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroaching upon exit neural canal with annular tear . .No definite abnormal enhancing lesion could be seen .",0,1
523,local_data/mendeley/T1_0573_D4.png,"LSS MRI : Feature of muscle spasm. Preveius operation at L4/L5 level ( fenestration). Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroaching upon exit neural canal with annular tear . .No definite abnormal enhancing lesion could be seen .",0,1
524,local_data/mendeley/T1_0573_D5.png,"LSS MRI : Feature of muscle spasm. Preveius operation at L4/L5 level ( fenestration). Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroaching upon exit neural canal with annular tear . .No definite abnormal enhancing lesion could be seen .",0,1
525,local_data/mendeley/T2_0573_D3.png,"LSS MRI : Feature of muscle spasm. Preveius operation at L4/L5 level ( fenestration). Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroaching upon exit neural canal with annular tear . .No definite abnormal enhancing lesion could be seen .",0,1
526,local_data/mendeley/T2_0573_D4.png,"LSS MRI : Feature of muscle spasm. Preveius operation at L4/L5 level ( fenestration). Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroaching upon exit neural canal with annular tear . .No definite abnormal enhancing lesion could be seen .",0,1
527,local_data/mendeley/T2_0573_D5.png,"LSS MRI : Feature of muscle spasm. Preveius operation at L4/L5 level ( fenestration). Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroaching upon exit neural canal with annular tear . .No definite abnormal enhancing lesion could be seen .",0,1

Overwriting local_data/lbp-test-meta.csv


In [6]:
%%writefile local_data/sentence-label.csv
,Reports,No Finding,LBP
1,No abnormalities detected in the lumbar spine.,1,0
2,The MRI shows normal spinal alignment with no disc herniation.,1,0
3,There are no signs of nerve compression or other abnormalities.,1,0
4,The lumbar vertebrae are intact with no evidence of fractures.,1,0
5,No bulging or herniation of intervertebral discs noted.,1,0
6,No abnormal findings in the lumbar region on imaging.,1,0
7,The MRI reveals a normal spine with no pathological changes.,1,0
8,No signs of degenerative disc disease or spinal stenosis.,1,0
9,The vertebral bodies and intervertebral discs are normal.,1,0
10,There is no evidence of scoliosis or other deformities.,1,0
11,The spinal cord appears normal with no signs of injury.,1,0
12,There is no evidence of osteoarthritis in the lumbar spine.,1,0
13,The MRI shows normal curvature and alignment of the spine.,1,0
14,There are no abnormal disc spaces or vertebral body changes.,1,0
15,No signs of bone fractures or tumors in the lumbar region.,1,0
16,The intervertebral discs appear healthy and well-aligned.,1,0
17,There is no evidence of swelling or inflammation in the spine.,1,0
18,No signs of spinal cord compression or other abnormalities.,1,0
19,The MRI scan shows no signs of disc degeneration.,1,0
20,There is no bulging or rupturing of the intervertebral discs.,1,0
21,No pathological findings detected in the lumbar MRI.,1,0
22,The vertebrae in the lumbar spine are healthy and intact.,1,0
23,There is no evidence of herniation or spinal deformity.,1,0
24,Normal intervertebral disc spacing is observed in the MRI.,1,0
25,No signs of abnormal bone growth or spurs in the lumbar region.,1,0
26,The spine appears healthy with no signs of degeneration.,1,0
27,The MRI reveals a normal disc-to-vertebral body ratio.,1,0
28,There are no abnormal changes to the vertebral column.,1,0
29,No evidence of scoliosis or abnormal curvature in the lumbar spine.,1,0
30,The lumbar spine is structurally intact with no herniation.,1,0
31,The discs in the lumbar spine show no signs of bulging or prolapse.,1,0
32,The MRI confirms a normal and healthy lumbar spine.,1,0
33,There are no irregularities in the spinal alignment or bone structure.,1,0
34,No signs of inflammation in the lumbar intervertebral discs.,1,0
35,The MRI scan does not show any nerve compression.,1,0
36,The lumbar spine appears symmetrical with normal curvature.,1,0
37,No abnormalities in the soft tissues surrounding the spine.,1,0
38,There are no signs of narrowing of the spinal canal.,1,0
39,The MRI results show a normal lumbar vertebral body structure.,1,0
40,No signs of disc bulging or damage to the spinal cord.,1,0
41,The vertebral bodies are intact with no signs of fractures.,1,0
42,The spinal column appears normal with no deformities.,1,0
43,There are no lesions or masses in the lumbar region.,1,0
44,The lumbar discs are well-spaced and free from degeneration.,1,0
45,No abnormal bone formations are present in the MRI.,1,0
46,The MRI reveals a healthy spinal alignment with no evidence of damage.,1,0
47,There are no signs of disc herniation or prolapse in the lumbar spine.,1,0
48,The lumbar spine shows no evidence of wear and tear.,1,0
49,No abnormalities in the spine's bony structures.,1,0
50,The lumbar region shows normal joint and bone structures.,1,0
51,There are no unusual growths or abnormalities in the lumbar MRI.,1,0
52,The MRI confirms the absence of degenerative changes in the spine.,1,0
53,No evidence of narrowing or compression in the spinal canal.,1,0
54,The vertebral joints show no signs of arthritis or degeneration.,1,0
55,The lumbar MRI appears unremarkable with no pathological findings.,1,0
56,The lumbar spinal cord demonstrates no abnormal signal.,1,0
57,There is normal signal intensity in the intervertebral discs.,1,0
58,No evidence of disc desiccation or collapse.,1,0
59,No posterior disc protrusion is noted in any lumbar levels.,1,0
60,The lumbar vertebral alignment is within normal limits.,1,0
61,No mass effect is present within the spinal canal.,1,0
62,No abnormal enhancement detected post-contrast administration.,1,0
63,No abnormalities identified in paraspinal soft tissues.,1,0
64,The sacroiliac joints appear unremarkable.,1,0
65,No significant disc space narrowing is seen.,1,0
66,The bony structures appear normal in density and contour.,1,0
67,No evidence of spondylolisthesis in the lumbar region.,1,0
68,The thecal sac is normal in size without compression.,1,0
69,The lumbar neural foramina are widely patent.,1,0
70,No abnormal marrow signal in the lumbar vertebrae.,1,0
71,There is no epidural or paraspinal mass.,1,0
72,The conus medullaris is normally positioned.,1,0
73,No abnormal motion artifact distorting image quality.,1,0
74,The vertebral endplates appear intact and regular.,1,0
75,No Schmorl's nodes or endplate irregularities observed.,1,0
76,No abnormal soft tissue masses detected.,1,0
77,No signs of infection or inflammatory process in the lumbar spine.,1,0
78,Normal spinal canal diameter throughout lumbar region.,1,0
79,The MRI shows maintained lumbar lordosis.,1,0
80,No evidence of traumatic injury in the lumbar area.,1,0
81,The facet joints appear well-maintained without degeneration.,1,0
82,The discs demonstrate normal hydration signal on T2 images.,1,0
83,No evidence of congenital anomalies in the lumbar spine.,1,0
84,No signs of subluxation or instability are present.,1,0
85,The posterior elements of the spine are intact and normal.,1,0
86,The posterior longitudinal ligament appears intact.,1,0
87,The anterior longitudinal ligament shows no abnormality.,1,0
88,No vertebral collapse or compression fractures noted.,1,0
89,No signs of previous spinal surgery or instrumentation.,1,0
90,The MRI reveals no suspicious lesions in the lumbar spine.,1,0
91,The lumbar interspinous ligaments appear intact.,1,0
92,No foraminal narrowing at any lumbar levels.,1,0
93,No abnormality noted in adjacent pelvic structures.,1,0
94,The vertebral cortical margins are intact and smooth.,1,0
95,No abnormal signal is seen in adjacent muscles.,1,0
96,The epidural fat planes are preserved and symmetrical.,1,0
97,No spinal cord tethering or abnormal positioning detected.,1,0
98,The MRI scan reveals no acute or chronic pathology.,1,0
99,"No spinal masses, cysts, or hemangiomas detected.",1,0
100,"No disc herniation, protrusion, or extrusion identified.",1,0
101,The MRI reveals signs of disc herniation in the lumbar spine.,0,1
102,There is evidence of nerve root compression in the lower back.,0,1
103,Degenerative disc disease is seen in the lumbar region.,0,1
104,The imaging shows signs of spinal stenosis in the lumbar spine.,0,1
105,The lumbar vertebrae show signs of mild fractures or degeneration.,0,1
106,There is bulging of the intervertebral disc at L4-L5.,0,1
107,MRI suggests a herniated disc at L5-S1 with nerve root impingement.,0,1
108,Signs of lumbar spondylosis and facet joint osteoarthritis.,0,1
109,There is evidence of a lumbar disc bulge causing nerve compression.,0,1
110,The MRI indicates a possible spinal cord abnormality due to herniated disc.,0,1
111,The MRI scan shows disc degeneration at L3-L4.,0,1
112,There are signs of nerve compression at L4-L5 and L5-S1.,0,1
113,Spinal canal narrowing is observed in the lumbar region.,0,1
114,The MRI reveals significant herniation at the L4-L5 disc.,0,1
115,Degenerative changes in the lumbar spine are noted.,0,1
116,There is a narrowing of the intervertebral space at L5-S1.,0,1
117,The lumbar vertebrae show signs of wear and tear.,0,1
118,Signs of facet joint arthritis in the lumbar spine.,0,1
119,The MRI shows evidence of spondylolisthesis at L5.,0,1
120,There is abnormal disc bulging with nerve compression in the lumbar spine.,0,1
121,The MRI shows lumbar spine deformities due to spondylosis.,0,1
122,Herniated discs are observed at multiple lumbar levels.,0,1
123,There is evidence of lumbar spinal instability due to disc damage.,0,1
124,The imaging shows signs of spinal stenosis causing nerve compression.,0,1
125,There is moderate degeneration of the lumbar intervertebral discs.,0,1
126,The MRI suggests lumbar spine fractures with mild displacement.,0,1
127,The vertebral bodies show signs of degenerative changes in the lumbar spine.,0,1
128,There is significant disc herniation at L3-L4 affecting the nerve roots.,0,1
129,Signs of severe disc degeneration and nerve impingement at L5-S1.,0,1
130,The MRI scan shows severe degenerative changes in the lumbar spine.,0,1
131,There is bulging of the disc at L4-L5 with mild spinal stenosis.,0,1
132,The MRI shows evidence of spinal canal narrowing at L2-L3.,0,1
133,The lumbar spine shows signs of severe degenerative changes.,0,1
134,A herniated disc is pressing on the spinal nerve at L5-S1.,0,1
135,MRI indicates moderate degeneration at L3-L4 with narrowing of the disc space.,0,1
136,The MRI shows a herniated lumbar disc causing nerve root impingement.,0,1
137,There is evidence of facet joint arthritis at L4-L5 and L5-S1.,0,1
138,The MRI suggests degenerative changes in the lower lumbar spine.,0,1
139,Disc bulging is observed at L3-L4 and L5-S1 levels.,0,1
140,There is nerve compression and spinal stenosis at multiple lumbar levels.,0,1
141,The MRI indicates advanced degenerative disc disease in the lumbar spine.,0,1
142,There is evidence of chronic lumbar pain from disc herniation.,0,1
143,The MRI scan shows significant disc protrusion at L4-L5.,0,1
144,The lumbar spine shows disc herniation with nerve root compression.,0,1
145,There is severe stenosis of the lumbar spinal canal.,0,1
146,The MRI indicates bulging discs causing pressure on the spinal nerves.,0,1
147,There are signs of chronic lumbar pain and disk degeneration.,0,1
148,The MRI suggests herniated discs causing spinal cord pressure.,0,1
149,Moderate narrowing of the lumbar neural foramina is present.,0,1
150,Degenerative spondylolisthesis is observed at L4-L5.,0,1
151,Disc desiccation is evident at multiple lumbar levels.,0,1
152,Endplate changes are noted at L5-S1 due to chronic degeneration.,0,1
153,Facet joint hypertrophy is seen at L3-L4.,0,1
154,Spinal canal effacement is observed due to disc protrusion.,0,1
155,A disc extrusion compressing the thecal sac is present at L5-S1.,0,1
156,There is moderate nerve impingement bilaterally at L4-L5.,0,1
157,Anterior disc osteophyte complexes are present at L3-L4.,0,1
158,A lumbar disc herniation is causing foraminal narrowing at L4-L5.,0,1
159,The MRI shows degenerative retrolisthesis at L2-L3.,0,1
160,Disc protrusions are seen at three consecutive lumbar levels.,0,1
161,The lumbar MRI reveals paracentral disc herniation at L5-S1.,0,1
162,There is posterior disc displacement causing mild nerve impingement.,0,1
163,A broad-based disc bulge is compressing the nerve roots at L3-L4.,0,1
164,There is evidence of mild spondylolisthesis at L5-S1.,0,1
165,The MRI reveals prominent disc degeneration throughout the lumbar spine.,0,1
166,Advanced degenerative disc changes are evident at L4-L5.,0,1
167,A right paracentral disc herniation is compressing the L5 nerve.,0,1
168,There is posterior vertebral body osteophyte formation.,0,1
169,The imaging shows lumbar disc prolapse with thecal sac compression.,0,1
170,The MRI scan reveals bilateral foraminal stenosis.,0,1
171,There is ligamentum flavum thickening contributing to canal stenosis.,0,1
172,Disc space collapse is noted at L5-S1.,0,1
173,MRI reveals a large disc protrusion affecting neural structures.,0,1
174,A large disc herniation at L3-L4 compresses the adjacent nerve root.,0,1
175,The MRI reveals facet joint hypertrophy contributing to spinal stenosis.,0,1
176,Evidence of disc protrusion is causing significant nerve impingement.,0,1
177,A lumbar vertebral body compression fracture is seen at L2.,0,1
178,The lumbar discs show marked desiccation with disc space narrowing.,0,1
179,There is bilateral nerve root compression at L5-S1.,0,1
180,The MRI shows lumbar disc prolapse at L4-L5.,0,1
181,A sequestered disc fragment is compressing the left L5 nerve root.,0,1
182,There is degenerative retrolisthesis at L3-L4 with associated stenosis.,0,1
183,Posterior disc herniation is observed at L2-L3 with mild canal narrowing.,0,1
184,MRI reveals advanced discogenic changes at multiple lumbar levels.,0,1
185,Disc extrusion at L5-S1 is impinging on the thecal sac.,0,1
186,An annular tear is identified at the L4-L5 intervertebral disc.,0,1
187,Significant disc bulging is seen at the L3-L4 level.,0,1
188,There is lumbar facet arthropathy contributing to foraminal narrowing.,0,1
189,Multilevel lumbar spondylosis is evident on the MRI.,0,1
190,A broad-based disc protrusion causes moderate central canal stenosis.,0,1
191,Degenerative changes result in foraminal encroachment at L5-S1.,0,1
192,MRI shows left foraminal disc herniation compressing the L4 nerve.,0,1
193,The imaging demonstrates progressive disc degeneration in the lumbar spine.,0,1
194,Discogenic endplate changes are noted at multiple lumbar levels.,0,1
195,Nerve root sleeve effacement is observed at L4-L5.,0,1
196,Mild posterior disc bulging is present at L2-L3 and L3-L4.,0,1
197,A central disc protrusion is compressing the anterior thecal sac.,0,1
198,The MRI shows asymmetric disc bulging more pronounced on the right.,0,1
199,There is neural foraminal narrowing due to facet joint overgrowth.,0,1
200,An inferiorly migrated disc fragment is compressing the S1 nerve root.,0,1
201,The lumbar vertebral bodies and intervertebral discs appear normal in height and signal intensity.,1,0
202,No disc bulge,1,0
203,The spinal canal and neural foramina remain widely patent without evidence of stenosis.,1,0
204,No marrow signal abnormality is detected within the lumbar vertebral bodies.,1,0
205,Paraspinal soft tissues demonstrate unremarkable appearance on all sequences.,1,0
206,No abnormal enhancement or mass lesion is observed.,1,0
207,The conus medullaris terminates at the L1-L2 level with normal morphology and signal.,1,0
208,No evidence of nerve root compression or impingement is seen.,1,0
209,Alignment of the lumbar spine is well maintained without spondylolisthesis.,1,0
210,Facet joints show no significant degenerative changes.,1,0
211,No epidural or paraspinal collection is present.,1,0
212,Vertebral endplates appear intact without erosive or destructive changes.,1,0
213,Posterior elements are intact with no fracture or lytic lesion.,1,0
214,Normal cerebrospinal fluid signal throughout the spinal canal.,1,0
215,No evidence of prior surgical intervention or hardware is noted.,1,0
216,No abnormal curvature or scoliosis is present.,1,0
217,The sacroiliac joints appear symmetric and unremarkable bilaterally.,1,0
218,Vertebral body heights are preserved with no compression deformities.,1,0
219,No abnormality is identified involving the intervertebral discs.,1,0
220,Normal fatty marrow signal is demonstrated throughout.,1,0
221,The cauda equina nerve roots are unremarkable.,1,0
222,No significant degenerative disc disease is seen.,1,0
223,Posterior epidural fat appears normal in volume and signal.,1,0
224,The vertebral alignment remains physiologic.,1,0
225,No disc desiccation is observed.,1,0
226,The bony structures of the lumbar spine demonstrate normal morphology.,1,0
227,No mass-like lesion is identified within the spinal canal.,1,0
228,The posterior longitudinal ligament is intact without thickening.,1,0
229,No foraminal narrowing is present at any lumbar level.,1,0
230,No abnormality in the paravertebral soft tissues.,1,0
231,The vertebral marrow signal is homogeneous.,1,0
232,No focal lesion is seen involving the spinal cord.,1,0
233,The anterior longitudinal ligament appears intact.,1,0
234,The conus medullaris shows normal position and configuration.,1,0
235,No abnormal enhancement post-contrast administration.,1,0
236,No signs of infection or inflammatory changes.,1,0
237,No evidence of vertebral body edema or contusion.,1,0
238,Intervertebral disc heights are well preserved.,1,0
239,No synovial cyst or facet joint effusion is seen.,1,0
240,No abnormality of the posterior elements is appreciated.,1,0
241,No significant spondylotic changes present.,1,0
242,The paraspinal muscles show normal bulk and signal.,1,0
243,No evidence of sacral fracture or abnormality.,1,0
244,No intraspinal mass lesion identified.,1,0
245,Thecal sac is normal in caliber throughout.,1,0
246,No signs of spinal instability are demonstrated.,1,0
247,Normal alignment of the spinous processes.,1,0
248,No pathologic signal abnormality in the sacrum.,1,0
249,Disc signal intensity appears preserved on T2-weighted images.,1,0
250,No signs of prior trauma.,1,0
251,Vertebral body contours are regular without lytic or sclerotic lesions.,1,0
252,No abnormality involving the coccyx.,1,0
253,No abnormality of the lumbosacral junction.,1,0
254,No ligamentous thickening or ossification.,1,0
255,Normal epidural venous plexus.,1,0
256,No indication of degenerative spondylolisthesis.,1,0
257,The interspinous ligaments are intact.,1,0
258,No high-signal changes suggestive of marrow edema.,1,0
259,No epidural hematoma or abnormal collection.,1,0
260,No nerve root thickening or abnormal enhancement.,1,0
261,No intervertebral disc extrusion or migration.,1,0
262,The vertebral endplates are preserved without Modic changes.,1,0
263,No abnormality detected at the thoracolumbar junction.,1,0
264,No congenital anomalies of the lumbar vertebrae.,1,0
265,Spinal cord signal remains normal throughout.,1,0
266,No evidence of neural foraminal stenosis.,1,0
267,No incidental mass or lesion identified.,1,0
268,Posterior elements well aligned with no pars defects.,1,0
269,No vertebral body collapse or loss of height.,1,0
270,No cystic or solid mass detected.,1,0
271,No abnormal curvature of the lumbar spine.,1,0
272,No degenerative disc changes appreciated.,1,0
273,Paravertebral regions remain unremarkable.,1,0
274,No signs of neoplastic involvement.,1,0
275,The posterior dura appears normal.,1,0
276,No bone marrow replacement process identified.,1,0
277,No fluid collection or abnormal enhancement.,1,0
278,Intervertebral discs are age-appropriate in appearance.,1,0
279,No unusual findings or incidentalomas detected.,1,0
280,No abnormal intraspinal signal change.,1,0
281,Vertebral marrow displays homogeneous fat signal.,1,0
282,The facet joint capsules are intact without effusion.,1,0
283,No annular fissures identified.,1,0
284,Spinal cord conus terminates normally without distortion.,1,0
285,No prevertebral or retroperitoneal abnormality.,1,0
286,No epidural lesion or displacement.,1,0
287,No hyperintense foci on T2 suggestive of edema or tumor.,1,0
288,The sacral nerve roots demonstrate normal appearance.,1,0
289,No abnormalities in the lumbosacral transition vertebrae.,1,0
290,Vertebral heights and alignment are anatomic.,1,0
291,No significant facet arthropathy noted.,1,0
292,No intradural mass lesion.,1,0
293,No pathologic enhancement pattern after gadolinium.,1,0
294,Disc margins remain smooth and continuous.,1,0
295,Thecal sac compression is not evident.,1,0
296,Paraspinal musculature is symmetric.,1,0
297,No significant osteophyte formation.,1,0
298,No acute or chronic fractures detected.,1,0
299,No notable vascular anomalies.,1,0
300,No features suggesting infection or malignancy.,1,0
301,Mild disc desiccation is noted at the L4-L5 and L5-S1 levels.,0,1
302,Central disc protrusion at L5-S1 minimally effaces the thecal sac.,0,1
303,Mild facet arthropathy is present at multiple lumbar levels.,0,1
304,Small annular fissure seen at L4-L5.,0,1
305,Disc bulging observed at L3-L4 without significant stenosis.,0,1
306,Concentric disc bulge at L4-L5 mildly narrows the central canal.,0,1
307,Moderate facet joint hypertrophy at L5-S1 contributes to foraminal narrowing.,0,1
308,Thecal sac impression by a posterior disc protrusion at L5-S1.,0,1
309,Left-sided foraminal stenosis at L4-L5 due to disc-osteophyte complex.,0,1
310,Subtle marrow edema adjacent to the L5-S1 endplates.,0,1
311,Small left paracentral disc protrusion noted at L4-L5.,0,1
312,Ligamentum flavum thickening contributes to mild canal narrowing.,0,1
313,Early degenerative disc disease involving the lower lumbar spine.,0,1
314,Disc bulging with shallow central protrusion at L3-L4.,0,1
315,Bilateral facet hypertrophy at L4-L5 and L5-S1.,0,1
316,Posterior annular tear visible at L5-S1.,0,1
317,No significant central canal stenosis,0,1
318,Disc height loss at L5-S1 with associated Modic type 1 changes.,0,1
319,Right L5 nerve root proximity to a disc bulge at L4-L5.,0,1
320,Early spondylotic changes throughout the lumbar spine.,0,1
321,Mild asymmetric disc bulge toward the left at L4-L5.,0,1
322,Narrowing of bilateral neural foramina at L5-S1.,0,1
323,Posterior disc osteophyte complex noted at L5-S1.,0,1
324,Minor disc extrusion causing mild thecal sac displacement.,0,1
325,Bilateral L5-S1 facet joint arthrosis with small joint effusions.,0,1
326,Focal marrow edema adjacent to the L4 endplate.,0,1
327,Concentric bulging with annular fissuring at L5-S1.,0,1
328,Minor spondylolisthesis at L4-L5 with facet degeneration.,0,1
329,Low-grade central canal stenosis at L4-L5.,0,1
330,Foraminal disc protrusion impinging on the left L5 nerve root.,0,1
331,Diffuse disc desiccation involving multiple lumbar levels.,0,1
332,Central disc extrusion at L5-S1 with inferior migration.,0,1
333,Mild retrolisthesis at L3-L4 on extension images.,0,1
334,Hypertrophic changes of the ligamentum flavum at L5-S1.,0,1
335,Small left foraminal synovial cyst at L4-L5.,0,1
336,Posterior disc displacement resulting in mild foraminal narrowing.,0,1
337,Subtle disc extrusion contacting the right L4 nerve root.,0,1
338,Annular fissures present at L3-L4 and L5-S1.,0,1
339,Advanced disc degeneration noted at L5-S1.,0,1
340,Moderate bilateral foraminal narrowing at L5-S1.,0,1
341,Disc material abutting the right S1 nerve root.,0,1
342,Marrow signal changes adjacent to degenerative endplates.,0,1
343,Disc-osteophyte complex formation noted at L4-L5.,0,1
344,Posterior disc protrusion contacting bilateral nerve roots.,0,1
345,Small Schmorl’s nodes within L3 and L4 vertebral endplates.,0,1
346,Thecal sac indentation by a central disc protrusion.,0,1
347,Moderate facet joint arthropathy at multiple levels.,0,1
348,Disc desiccation and mild disc bulging at L5-S1.,0,1
349,Central disc bulging with mild foraminal narrowing bilaterally.,0,1
350,L5-S1 disc height significantly reduced.,0,1
351,Mild degenerative changes at L4-L5 with facet joint narrowing.,0,1
352,Disc bulging at L4-L5 contributing to slight lateral recess narrowing.,0,1
353,Mild foraminal stenosis bilaterally at L3-L4 secondary to disc bulging.,0,1
354,Mild scoliosis noted,0,1
355,Focal herniation at L5-S1 causing minimal displacement of the nerve roots.,0,1
356,Moderate disc degeneration at L3-L4 with associated annular tear.,0,1
357,Severe disc collapse with associated foraminal stenosis at L5-S1.,0,1
358,Facet joint hypertrophy and thickening observed at L4-L5.,0,1
359,Mild narrowing of the central canal at L4-L5 with posterior bulging.,0,1
360,Posterior osteophytes at L3-L4 contributing to mild canal narrowing.,0,1
361,Central disc protrusion at L2-L3 without significant stenosis.,0,1
362,Partial disc extrusion at L5-S1 with lateral disc bulge.,0,1
363,No significant central canal narrowing,0,1
364,Mild posterior bulging at L5-S1 resulting in nerve root compression.,0,1
365,L5-S1 spondylolisthesis contributing to mild spinal instability.,0,1
366,Mild central disc protrusion at L4-L5 causing slight spinal canal narrowing.,0,1
367,Intervertebral disc herniation at L3-L4 impinging the left L3 nerve root.,0,1
368,Severe disc degeneration at L4-L5 with Schmorl’s nodes.,0,1
369,Moderate facet arthrosis at L2-L3 with mild narrowing of the neural foramina.,0,1
370,Small synovial cyst in the left facet joint at L3-L4,0,1
371,Chronic disc desiccation at L5-S1 with mild spondylolisthesis.,0,1
372,Mild central and paracentral disc protrusion at L3-L4 with associated nerve root compression.,0,1
373,L4-L5 disc space loss with associated foraminal stenosis and facet joint arthropathy.,0,1
374,Abnormal disc height at L4-L5 with mild bulging and associated nerve root encroachment.,0,1
375,Right-sided facet joint osteoarthritis at L5-S1 causing moderate foraminal narrowing.,0,1
376,Posterior disc bulge at L3-L4 with mild narrowing of the central canal.,0,1
377,Mild to moderate stenosis at L4-L5 secondary to disc protrusion and facet arthropathy.,0,1
378,Severe disc desiccation at L3-L4 with associated anterior osteophytes.,0,1
379,A mild bulging disc at L5-S1 causes narrowing of the right neural foramen.,0,1
380,Facet joint degenerative changes at L4-L5 with mild foraminal narrowing.,0,1
381,Annular tear with mild disc bulge at L3-L4 impinging on the L4 nerve root.,0,1
382,Narrowing of the lumbar disc spaces with mild spondylosis at multiple levels.,0,1
383,Diffuse bulging of the disc at L3-L4,0,1
384,Severe disc degeneration at L5-S1 with evidence of nerve root impingement.,0,1
385,The L4-L5 disc shows moderate bulging and mild central canal stenosis.,0,1
386,Left paracentral disc protrusion at L4-L5 with slight compression of the nerve roots.,0,1
387,Minor spondylolisthesis at L4-L5 with accompanying facet arthrosis.,0,1
388,L5-S1 disc bulging with minimal foraminal narrowing and nerve root compression.,0,1
389,Mild disc protrusion at L3-L4 with associated facet joint degeneration.,0,1
390,Disc bulge at L5-S1 with foraminal stenosis causing nerve root encroachment.,0,1
391,Disc degeneration and bulging at L3-L4 contributing to nerve root impingement.,0,1
392,Left-sided disc protrusion at L4-L5 resulting in mild foraminal stenosis.,0,1
393,Central disc extrusion at L5-S1,0,1
394,Spondylotic changes noted at L3-L4 with mild canal narrowing.,0,1
395,Facet joint hypertrophy at L5-S1 causing mild foraminal narrowing bilaterally.,0,1
396,Disc bulging with minor herniation at L4-L5 resulting in bilateral foraminal narrowing.,0,1
397,Posterior disc protrusion at L2-L3 causing mild central canal narrowing.,0,1
398,Diffuse disc bulging at L4-L5 contributing to mild neural foraminal narrowing.,0,1
399,Moderate disc desiccation and degenerative changes at L4-L5 and L5-S1.,0,1
400,Chronic degenerative changes at L3-L4 with mild facet joint osteoarthritis.,0,1
401,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 2-4, 8 show Pfirrmann grade 3. Modic type 2 in discs 1-6. Narrowing in discs 3. Bulging in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5.",0,1
402,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 2-4, 8 show Pfirrmann grade 3. Modic type 2 in discs 1-6. Narrowing in discs 3. Bulging in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5.",0,1
403,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 4. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6.,0,1
404,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 4. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6.,0,1
405,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-6 show Pfirrmann grade 3. Discs 1-4, 7 show Pfirrmann grade 4. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 1-4.",0,1
406,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-6 show Pfirrmann grade 3. Discs 1-4, 7 show Pfirrmann grade 4. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 1-4.",0,1
407,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-2, 5-6 show Pfirrmann grade 2. Discs 7 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-3, 5. Low endplate in discs 2-4. Up endplate in discs 2-4, 6-7. Modic type 2 in discs 2, 4. Narrowing in discs 3-4. Herniation in discs 4.",0,1
408,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-2, 5-6 show Pfirrmann grade 2. Discs 7 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-3, 5. Low endplate in discs 2-4. Up endplate in discs 2-4, 6-7. Modic type 2 in discs 2, 4. Narrowing in discs 3-4. Herniation in discs 4.",0,1
409,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 4. Narrowing in discs 1-7. Bulging in discs 1-2, 5. Modic type 2 in discs 1-7. Low endplate in discs 2-5, 7. Up endplate in discs 3, 7. Herniation in discs 3-4.",0,1
410,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 4. Narrowing in discs 1-7. Bulging in discs 1-2, 5. Modic type 2 in discs 1-7. Low endplate in discs 2-5, 7. Up endplate in discs 3, 7. Herniation in discs 3-4.",0,1
411,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 0 show Pfirrmann grade 1. Discs 6 show Pfirrmann grade 2. Discs 1, 4-5 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2. Bulging in discs 3-5.",0,1
412,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 0 show Pfirrmann grade 1. Discs 6 show Pfirrmann grade 2. Discs 1, 4-5 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2. Bulging in discs 3-5.",0,1
413,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 0 show Pfirrmann grade 1. Discs 6 show Pfirrmann grade 2. Discs 1, 4-5 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2. Bulging in discs 3-5.",0,1
414,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 3-5 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Narrowing in discs 1, 4. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6.",0,1
415,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 3-5 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Narrowing in discs 1, 4. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6.",0,1
416,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Low endplate in discs 2, 5-7. Up endplate in discs 2, 5-6. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 3 in discs 2.",0,1
417,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Low endplate in discs 2, 5-7. Up endplate in discs 2, 5-6. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 3 in discs 2.",0,1
418,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 4, 7 show Pfirrmann grade 3. Discs 5 show Pfirrmann grade 4. Discs 2-3, 6 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1, 4-6. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Herniation in discs 2-3.",0,1
419,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 4, 7 show Pfirrmann grade 3. Discs 5 show Pfirrmann grade 4. Discs 2-3, 6 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1, 4-6. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Herniation in discs 2-3.",0,1
420,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-4, 6-7 show Pfirrmann grade 3. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1, 5. Bulging in discs 1, 3, 5. Low endplate in discs 1, 3, 5, 7. Up endplate in discs 1, 3, 5, 7. Modic type 2 in discs 1-2, 5.",0,1
421,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-4, 6-7 show Pfirrmann grade 3. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1, 5. Bulging in discs 1, 3, 5. Low endplate in discs 1, 3, 5, 7. Up endplate in discs 1, 3, 5, 7. Modic type 2 in discs 1-2, 5.",0,1
422,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-4, 6-7 show Pfirrmann grade 3. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1, 5. Bulging in discs 1, 3, 5. Low endplate in discs 1, 3, 5, 7. Up endplate in discs 1, 3, 5, 7. Modic type 2 in discs 1-2, 5.",0,1
423,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1.,0,1
424,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1.,0,1
425,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 3-5 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2.,0,1
426,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 3-5 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2.,0,1
427,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Bulging in discs 2-4. Low endplate in discs 2-4. Up endplate in discs 2-4. Modic type 2 in discs 2-3.",0,1
428,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Bulging in discs 2-4. Low endplate in discs 2-4. Up endplate in discs 2-4. Modic type 2 in discs 2-3.",0,1
429,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 2. Discs 6 show Pfirrmann grade 3. Discs 2, 5 show Pfirrmann grade 4. Discs 1, 3-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
430,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 2. Discs 6 show Pfirrmann grade 3. Discs 2, 5 show Pfirrmann grade 4. Discs 1, 3-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
431,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 2-5. Spondylolisthesis in discs 2. Low endplate in discs 3-6. Up endplate in discs 3-6. Modic type 2 in discs 3-6.,0,1
432,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 2-5. Spondylolisthesis in discs 2. Low endplate in discs 3-6. Up endplate in discs 3-6. Modic type 2 in discs 3-6.,0,1
433,A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
434,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
435,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
436,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 2-4 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 1, 6 show Pfirrmann grade 5. Narrowing in discs 1, 6-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2. Bulging in discs 3-6. Low endplate in discs 5-7. Up endplate in discs 5-7.",0,1
437,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 2-4 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 1, 6 show Pfirrmann grade 5. Narrowing in discs 1, 6-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2. Bulging in discs 3-6. Low endplate in discs 5-7. Up endplate in discs 5-7.",0,1
438,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 2-4 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 1, 6 show Pfirrmann grade 5. Narrowing in discs 1, 6-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2. Bulging in discs 3-6. Low endplate in discs 5-7. Up endplate in discs 5-7.",0,1
439,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Herniation in discs 1. Bulging in discs 2.,0,1
440,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 3-4 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Modic type 2 in discs 1-3. Up endplate in discs 2.",0,1
441,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 3-4 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Modic type 2 in discs 1-3. Up endplate in discs 2.",0,1
442,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1, 3-6 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Bulging in discs 1-2. Modic type 2 in discs 2.",0,1
443,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1, 3-6 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Bulging in discs 1-2. Modic type 2 in discs 2.",0,1
444,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5 show Pfirrmann grade 1. Discs 2-4, 6-8 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 5. Herniation in discs 1. Bulging in discs 2, 5. Up endplate in discs 7.",0,1
445,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Bulging in discs 1-2. Narrowing in discs 2. Herniation in discs 2. Up endplate in discs 3-6. Low endplate in discs 6.,0,1
446,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4, 6-7 show Pfirrmann grade 2. Discs 1-3, 5 show Pfirrmann grade 3. Narrowing in discs 2. Up endplate in discs 3, 5. Bulging in discs 5. Modic type 2 in discs 5.",0,1
447,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4, 6-7 show Pfirrmann grade 2. Discs 1-3, 5 show Pfirrmann grade 3. Narrowing in discs 2. Up endplate in discs 3, 5. Bulging in discs 5. Modic type 2 in discs 5.",0,1
448,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 1. Narrowing in discs 2, 4. Bulging in discs 2-4. Low endplate in discs 2-7. Up endplate in discs 2-7.",0,1
449,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5, 7-8 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Narrowing in discs 1, 3-4. Bulging in discs 1-4, 6. Low endplate in discs 3-4. Up endplate in discs 3-4. Modic type 2 in discs 3-4.",0,1
450,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5, 7-8 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Narrowing in discs 1, 3-4. Bulging in discs 1-4, 6. Low endplate in discs 3-4. Up endplate in discs 3-4. Modic type 2 in discs 3-4.",0,1
451,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5, 7-8 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Narrowing in discs 1, 3-4. Bulging in discs 1-4, 6. Low endplate in discs 3-4. Up endplate in discs 3-4. Modic type 2 in discs 3-4.",0,1
452,A T1-weighted MRI of a female patient with 7 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 4.,0,1
453,A T2-weighted MRI of a female patient with 7 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 4.,0,1
454,A T2-weighted MRI of a female patient with 7 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 4.,0,1
455,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 3-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 4. Bulging in discs 1. Narrowing in discs 2. Herniation in discs 2.",0,1
456,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 3-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 4. Bulging in discs 1. Narrowing in discs 2. Herniation in discs 2.",0,1
457,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 2. Discs 6-7 show Pfirrmann grade 3. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 2-5. Bulging in discs 2-5. Low endplate in discs 2-7. Up endplate in discs 2-7. Modic type 2 in discs 2-6. Spondylolisthesis in discs 5.,0,1
458,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 2. Discs 6-7 show Pfirrmann grade 3. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 2-5. Bulging in discs 2-5. Low endplate in discs 2-7. Up endplate in discs 2-7. Modic type 2 in discs 2-6. Spondylolisthesis in discs 5.,0,1
459,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
460,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
461,A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 1. Low endplate in discs 1. Narrowing in discs 2-4. Bulging in discs 2-4.,0,1
462,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 2. Discs 2-4, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-4, 6. Spondylolisthesis in discs 1. Modic type 2 in discs 1-5.",0,1
463,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 2. Discs 2-4, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-4, 6. Spondylolisthesis in discs 1. Modic type 2 in discs 1-5.",0,1
464,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 4-8 show Pfirrmann grade 1. Discs 1, 3 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-3. Low endplate in discs 1-7. Up endplate in discs 1-3.",0,1
465,A T1-weighted MRI of a female patient with 6 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 3-7. Up endplate in discs 3-7. Modic type 2 in discs 4.,0,1
466,A T2-weighted MRI of a female patient with 6 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 3-7. Up endplate in discs 3-7. Modic type 2 in discs 4.,0,1
467,A T2-weighted MRI of a female patient with 6 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 3-7. Up endplate in discs 3-7. Modic type 2 in discs 4.,0,1
468,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
469,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
470,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3, 6-7 show Pfirrmann grade 1. Discs 1, 4-5 show Pfirrmann grade 3. Bulging in discs 1, 5. Narrowing in discs 5.",0,1
471,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3, 6-7 show Pfirrmann grade 1. Discs 1, 4-5 show Pfirrmann grade 3. Bulging in discs 1, 5. Narrowing in discs 5.",0,1
472,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Discs 5-7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5-7. Bulging in discs 1-5. Low endplate in discs 1, 5-8. Up endplate in discs 1, 5-7. Modic type 2 in discs 2-7. Spondylolisthesis in discs 5.",0,1
473,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Discs 5-7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5-7. Bulging in discs 1-5. Low endplate in discs 1, 5-8. Up endplate in discs 1, 5-7. Modic type 2 in discs 2-7. Spondylolisthesis in discs 5.",0,1
474,"A T1-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 6-9 show Pfirrmann grade 1. Discs 1-3 show Pfirrmann grade 3. Discs 4-5 show Pfirrmann grade 4. Modic type 2 in discs 1, 3, 7. Narrowing in discs 3-5. Herniation in discs 3. Bulging in discs 4-5. Modic type 3 in discs 5. Up endplate in discs 8.",0,1
475,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 6-9 show Pfirrmann grade 1. Discs 1-3 show Pfirrmann grade 3. Discs 4-5 show Pfirrmann grade 4. Modic type 2 in discs 1, 3, 7. Narrowing in discs 3-5. Herniation in discs 3. Bulging in discs 4-5. Modic type 3 in discs 5. Up endplate in discs 8.",0,1
476,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
477,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 3. Discs 1-5 show Pfirrmann grade 4. Up endplate in discs 2, 4-5. Herniation in discs 2. Spondylolisthesis in discs 2. Bulging in discs 3-5. Narrowing in discs 4. Low endplate in discs 4-5, 7.",0,1
478,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 3. Discs 1-5 show Pfirrmann grade 4. Up endplate in discs 2, 4-5. Herniation in discs 2. Spondylolisthesis in discs 2. Bulging in discs 3-5. Narrowing in discs 4. Low endplate in discs 4-5, 7.",0,1
479,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 3. Discs 1-5 show Pfirrmann grade 4. Up endplate in discs 2, 4-5. Herniation in discs 2. Spondylolisthesis in discs 2. Bulging in discs 3-5. Narrowing in discs 4. Low endplate in discs 4-5, 7.",0,1
480,"A T1-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 6-7, 9 show Pfirrmann grade 3. Discs 5, 8 show Pfirrmann grade 4. Discs 1-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
481,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 6-7, 9 show Pfirrmann grade 3. Discs 5, 8 show Pfirrmann grade 4. Discs 1-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
482,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 1. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Herniation in discs 1. Low endplate in discs 2, 4, 6. Up endplate in discs 3-4, 6. Narrowing in discs 4. Bulging in discs 4.",0,1
483,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 1. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Herniation in discs 1. Low endplate in discs 2, 4, 6. Up endplate in discs 3-4, 6. Narrowing in discs 4. Bulging in discs 4.",0,1
484,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1, 3-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1, 3-5. Up endplate in discs 1, 3-5. Spondylolisthesis in discs 1. Modic type 2 in discs 3-4.",0,1
485,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1, 3-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1, 3-5. Up endplate in discs 1, 3-5. Spondylolisthesis in discs 1. Modic type 2 in discs 3-4.",0,1
486,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Herniation in discs 2.,0,1
487,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Herniation in discs 2.,0,1
488,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Herniation in discs 2.,0,1
489,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 3-4, 6 show Pfirrmann grade 2. Discs 2, 7-8 show Pfirrmann grade 3. Discs 5 show Pfirrmann grade 4. Narrowing in discs 1-8. Bulging in discs 1. Herniation in discs 2. Low endplate in discs 3-7. Up endplate in discs 3-7.",0,1
490,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1.,0,1
491,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1.,0,1
492,A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
493,"A T1-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 0 show Pfirrmann grade 1. Discs 6-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 2-3, 8 show Pfirrmann grade 4. Discs 4-5 show Pfirrmann grade 5. Narrowing in discs 1-5, 8. Bulging in discs 2-8. Spondylolisthesis in discs 2. Low endplate in discs 4-5, 7-8. Up endplate in discs 4-5, 7-8. Modic type 2 in discs 4-5.",0,1
494,"A T2-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 0 show Pfirrmann grade 1. Discs 6-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 2-3, 8 show Pfirrmann grade 4. Discs 4-5 show Pfirrmann grade 5. Narrowing in discs 1-5, 8. Bulging in discs 2-8. Spondylolisthesis in discs 2. Low endplate in discs 4-5, 7-8. Up endplate in discs 4-5, 7-8. Modic type 2 in discs 4-5.",0,1
495,"A T2-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 0 show Pfirrmann grade 1. Discs 6-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 2-3, 8 show Pfirrmann grade 4. Discs 4-5 show Pfirrmann grade 5. Narrowing in discs 1-5, 8. Bulging in discs 2-8. Spondylolisthesis in discs 2. Low endplate in discs 4-5, 7-8. Up endplate in discs 4-5, 7-8. Modic type 2 in discs 4-5.",0,1
496,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 3. Discs 2-4 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1, 3-4. Narrowing in discs 2-4. Low endplate in discs 2-4, 7. Up endplate in discs 2-4, 7. Modic type 1 in discs 2. Herniation in discs 3.",0,1
497,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 3. Discs 2-4 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1, 3-4. Narrowing in discs 2-4. Low endplate in discs 2-4, 7. Up endplate in discs 2-4, 7. Modic type 1 in discs 2. Herniation in discs 3.",0,1
498,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1, 4. Modic type 2 in discs 1-5.",0,1
499,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1, 4. Modic type 2 in discs 1-5.",0,1
500,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 3-5 show Pfirrmann grade 5. Narrowing in discs 1, 3-5. Bulging in discs 1, 3-5. Low endplate in discs 1, 3-5. Up endplate in discs 1, 3-5. Spondylolisthesis in discs 1. Modic type 2 in discs 1-5. Herniation in discs 5.",0,1
501,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 3-5 show Pfirrmann grade 5. Narrowing in discs 1, 3-5. Bulging in discs 1, 3-5. Low endplate in discs 1, 3-5. Up endplate in discs 1, 3-5. Spondylolisthesis in discs 1. Modic type 2 in discs 1-5. Herniation in discs 5.",0,1
502,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 4-5 show Pfirrmann grade 2. Discs 3, 6 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 5. Modic type 2 in discs 1-2. Narrowing in discs 2-3. Bulging in discs 2-3. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2.",0,1
503,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 2. Discs 1, 3 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 5. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Low endplate in discs 1, 4-6. Up endplate in discs 1, 4-6. Modic type 2 in discs 2-5. Herniation in discs 4.",0,1
504,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 2. Discs 1, 3 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 5. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Low endplate in discs 1, 4-6. Up endplate in discs 1, 4-6. Modic type 2 in discs 2-5. Herniation in discs 4.",0,1
505,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 6 show Pfirrmann grade 1. Discs 3, 5 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-4. Low endplate in discs 2, 4. Modic type 2 in discs 2-4. Up endplate in discs 5-6.",0,1
506,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 6 show Pfirrmann grade 1. Discs 3, 5 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-4. Low endplate in discs 2, 4. Modic type 2 in discs 2-4. Up endplate in discs 5-6.",0,1
507,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 1, 3-7 show Pfirrmann grade 4. Discs 2, 8 show Pfirrmann grade 5. Narrowing in discs 1-3, 8. Bulging in discs 1-5, 7-8. Low endplate in discs 1-8. Up endplate in discs 1-8. Modic type 2 in discs 1-8.",0,1
508,"A T2-weighted MRI of a male patient with 7 vertebrae and 8 discs. Discs 1, 3-7 show Pfirrmann grade 4. Discs 2, 8 show Pfirrmann grade 5. Narrowing in discs 1-3, 8. Bulging in discs 1-5, 7-8. Low endplate in discs 1-8. Up endplate in discs 1-8. Modic type 2 in discs 1-8.",0,1
509,"A T2-weighted MRI of a male patient with 7 vertebrae and 8 discs. Discs 1, 3-7 show Pfirrmann grade 4. Discs 2, 8 show Pfirrmann grade 5. Narrowing in discs 1-3, 8. Bulging in discs 1-5, 7-8. Low endplate in discs 1-8. Up endplate in discs 1-8. Modic type 2 in discs 1-8.",0,1
510,A T1-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 1-9 show Pfirrmann grade 5. Narrowing in discs 1-9. Bulging in discs 1-9. Low endplate in discs 1-9. Up endplate in discs 1-9. Spondylolisthesis in discs 1. Modic type 2 in discs 1-9.,0,1
511,A T2-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 1-9 show Pfirrmann grade 5. Narrowing in discs 1-9. Bulging in discs 1-9. Low endplate in discs 1-9. Up endplate in discs 1-9. Spondylolisthesis in discs 1. Modic type 2 in discs 1-9.,0,1
512,A T2-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 1-9 show Pfirrmann grade 5. Narrowing in discs 1-9. Bulging in discs 1-9. Low endplate in discs 1-9. Up endplate in discs 1-9. Spondylolisthesis in discs 1. Modic type 2 in discs 1-9.,0,1
513,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2, 7 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-3, 7. Low endplate in discs 1-3, 5-7. Modic type 2 in discs 1-6. Spondylolisthesis in discs 2. Up endplate in discs 3-4, 6-7.",0,1
514,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2, 7 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-3, 7. Low endplate in discs 1-3, 5-7. Modic type 2 in discs 1-6. Spondylolisthesis in discs 2. Up endplate in discs 3-4, 6-7.",0,1
515,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 1-5 show Pfirrmann grade 3. Bulging in discs 1-5. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2, 4. Narrowing in discs 4.",0,1
516,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 1-5 show Pfirrmann grade 3. Bulging in discs 1-5. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2, 4. Narrowing in discs 4.",0,1
517,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2, 4-6, 8 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 7 show Pfirrmann grade 5. Narrowing in discs 1, 3, 7. Bulging in discs 1-4. Modic type 2 in discs 2-4. Low endplate in discs 6-7. Up endplate in discs 6-7.",0,1
518,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2, 4-6, 8 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 7 show Pfirrmann grade 5. Narrowing in discs 1, 3, 7. Bulging in discs 1-4. Modic type 2 in discs 2-4. Low endplate in discs 6-7. Up endplate in discs 6-7.",0,1
519,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2, 4-6, 8 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 7 show Pfirrmann grade 5. Narrowing in discs 1, 3, 7. Bulging in discs 1-4. Modic type 2 in discs 2-4. Low endplate in discs 6-7. Up endplate in discs 6-7.",0,1
520,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 6-7 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5. Modic type 2 in discs 3.",0,1
521,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 6-7 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5. Modic type 2 in discs 3.",0,1
522,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Herniation in discs 2. Narrowing in discs 3-4. Modic type 2 in discs 3, 6. Up endplate in discs 5.",0,1
523,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Herniation in discs 2. Narrowing in discs 3-4. Modic type 2 in discs 3, 6. Up endplate in discs 5.",0,1
524,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Discs 3-6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3-4. Bulging in discs 1-6. Modic type 2 in discs 1-7. Low endplate in discs 2-7. Up endplate in discs 2-7.",0,1
525,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Discs 3-6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3-4. Bulging in discs 1-6. Modic type 2 in discs 1-7. Low endplate in discs 2-7. Up endplate in discs 2-7.",0,1
526,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-2, 5-7 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-2. Low endplate in discs 1. Spondylolisthesis in discs 1.",0,1
527,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-2, 5-7 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-2. Low endplate in discs 1. Spondylolisthesis in discs 1.",0,1
528,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 1-4. Bulging in discs 2-5. Narrowing in discs 3-5.,0,1
529,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 1-4. Bulging in discs 2-5. Narrowing in discs 3-5.,0,1
530,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-6 show Pfirrmann grade 3. Discs 2, 7-8 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-6. Low endplate in discs 1-4, 6-8. Up endplate in discs 1-4. Modic type 2 in discs 1-8. Narrowing in discs 2-3.",0,1
531,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-6 show Pfirrmann grade 3. Discs 2, 7-8 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-6. Low endplate in discs 1-4, 6-8. Up endplate in discs 1-4. Modic type 2 in discs 1-8. Narrowing in discs 2-3.",0,1
532,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4, 6 show Pfirrmann grade 1. Discs 2, 5 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-2. Low endplate in discs 1-6. Up endplate in discs 1-6. Spondylolisthesis in discs 1. Modic type 2 in discs 1-6. Herniation in discs 3.",0,1
533,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4, 6 show Pfirrmann grade 1. Discs 2, 5 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-2. Low endplate in discs 1-6. Up endplate in discs 1-6. Spondylolisthesis in discs 1. Modic type 2 in discs 1-6. Herniation in discs 3.",0,1
534,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-3, 5-6 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Narrowing in discs 1. Low endplate in discs 1-2, 4-5. Up endplate in discs 1-2, 4-5. Herniation in discs 1. Modic type 2 in discs 1, 4-5. Bulging in discs 2, 4.",0,1
535,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-3, 5-6 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Narrowing in discs 1. Low endplate in discs 1-2, 4-5. Up endplate in discs 1-2, 4-5. Herniation in discs 1. Modic type 2 in discs 1, 4-5. Bulging in discs 2, 4.",0,1
536,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Herniation in discs 2.,0,1
537,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Herniation in discs 2.,0,1
538,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3, 5-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2. Bulging in discs 3-4.",0,1
539,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3, 5-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2. Bulging in discs 3-4.",0,1
540,"A T1-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 7-9 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-5, 7. Modic type 2 in discs 1-5. Up endplate in discs 2-7. Herniation in discs 7.",0,1
541,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 7-9 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-5, 7. Modic type 2 in discs 1-5. Up endplate in discs 2-7. Herniation in discs 7.",0,1
542,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 7-9 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-5, 7. Modic type 2 in discs 1-5. Up endplate in discs 2-7. Herniation in discs 7.",0,1
543,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3, 6. Bulging in discs 1-6. Low endplate in discs 1-3, 6-7. Up endplate in discs 1-3, 6-7. Modic type 2 in discs 2.",0,1
544,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3, 6. Bulging in discs 1-6. Low endplate in discs 1-3, 6-7. Up endplate in discs 1-3, 6-7. Modic type 2 in discs 2.",0,1
545,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-3. Modic type 3 in discs 2.",0,1
546,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-3. Modic type 3 in discs 2.",0,1
547,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 1. Narrowing in discs 1-6. Bulging in discs 1-4. Low endplate in discs 1-7. Up endplate in discs 1-7.,0,1
548,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-4 show Pfirrmann grade 4. Discs 1, 5-7 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.",0,1
549,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-4 show Pfirrmann grade 4. Discs 1, 5-7 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.",0,1
550,"A T1-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1-3, 6, 9 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 1-9. Up endplate in discs 1-9. Modic type 2 in discs 1-9. Narrowing in discs 4-5, 8.",0,1
551,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1-3, 6, 9 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 1-9. Up endplate in discs 1-9. Modic type 2 in discs 1-9. Narrowing in discs 4-5, 8.",0,1
552,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-4. Herniation in discs 2. Low endplate in discs 3. Up endplate in discs 3.,0,1
553,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 1-2. Up endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
554,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 1-2. Up endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
555,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 6-8 show Pfirrmann grade 2. Discs 4-5 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 2-3. Low endplate in discs 2-8. Up endplate in discs 2-8. Modic type 2 in discs 2-7.",0,1
556,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 6-8 show Pfirrmann grade 2. Discs 4-5 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 2-3. Low endplate in discs 2-8. Up endplate in discs 2-8. Modic type 2 in discs 2-7.",0,1
557,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 6-8 show Pfirrmann grade 2. Discs 4-5 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 2-3. Low endplate in discs 2-8. Up endplate in discs 2-8. Modic type 2 in discs 2-7.",0,1
558,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Herniation in discs 1. Modic type 2 in discs 1-7.,0,1
559,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Herniation in discs 1. Modic type 2 in discs 1-7.,0,1
560,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 2. Low endplate in discs 1, 7. Bulging in discs 7.",0,1
561,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 2. Low endplate in discs 1, 7. Bulging in discs 7.",0,1
562,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 7 show Pfirrmann grade 2. Discs 4-6 show Pfirrmann grade 3. Low endplate in discs 2. Up endplate in discs 4, 6. No disc herniation, disc narrowing or disc bulging is observed.",0,1
563,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 7 show Pfirrmann grade 2. Discs 4-6 show Pfirrmann grade 3. Low endplate in discs 2. Up endplate in discs 4, 6. No disc herniation, disc narrowing or disc bulging is observed.",0,1
564,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Bulging in discs 2-4. Low endplate in discs 2-4. Up endplate in discs 2-4. Modic type 2 in discs 2-5. Narrowing in discs 3-4.",0,1
565,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Bulging in discs 2-4. Low endplate in discs 2-4. Up endplate in discs 2-4. Modic type 2 in discs 2-5. Narrowing in discs 3-4.",0,1
566,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 2. Discs 3-4, 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5, 7-8. Bulging in discs 1-5, 7-8. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Modic type 2 in discs 1-2, 5.",0,1
567,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 2. Discs 3-4, 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5, 7-8. Bulging in discs 1-5, 7-8. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Modic type 2 in discs 1-2, 5.",0,1
568,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 2. Discs 3-4, 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5, 7-8. Bulging in discs 1-5, 7-8. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Modic type 2 in discs 1-2, 5.",0,1
569,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 2. No disc herniation, disc narrowing or disc bulging is observed.",1,0
570,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 2. No disc herniation, disc narrowing or disc bulging is observed.",1,0
571,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 3. Discs 1-5, 7-8 show Pfirrmann grade 4. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 2-6, 8. Modic type 2 in discs 2-3, 5. Up endplate in discs 3-8.",0,1
572,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 3. Discs 1-5, 7-8 show Pfirrmann grade 4. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 2-6, 8. Modic type 2 in discs 2-3, 5. Up endplate in discs 3-8.",0,1
573,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 3. Discs 1-5, 7-8 show Pfirrmann grade 4. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 2-6, 8. Modic type 2 in discs 2-3, 5. Up endplate in discs 3-8.",0,1
574,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Discs 1-3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-3. Up endplate in discs 1-3. Modic type 2 in discs 1-3, 5.",0,1
575,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Discs 1-3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-3. Up endplate in discs 1-3. Modic type 2 in discs 1-3, 5.",0,1
576,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 3.,0,1
577,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 3.,0,1
578,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-2. Modic type 2 in discs 1-2.,0,1
579,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-2. Modic type 2 in discs 1-2.,0,1
580,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-2 show Pfirrmann grade 1. Discs 3-6 show Pfirrmann grade 2. Low endplate in discs 2-5. Up endplate in discs 2-6. Modic type 2 in discs 2-5. Spondylolisthesis in discs 6. No disc herniation, disc narrowing or disc bulging is observed.",0,1
581,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-2 show Pfirrmann grade 1. Discs 3-6 show Pfirrmann grade 2. Low endplate in discs 2-5. Up endplate in discs 2-6. Modic type 2 in discs 2-5. Spondylolisthesis in discs 6. No disc herniation, disc narrowing or disc bulging is observed.",0,1
582,A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-4. Low endplate in discs 1-2. Up endplate in discs 1-2.,0,1
583,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 3, 5 show Pfirrmann grade 4. Discs 1-2, 4 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
584,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 3, 5 show Pfirrmann grade 4. Discs 1-2, 4 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
585,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-5 show Pfirrmann grade 1. Discs 1-3, 6 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1. Modic type 2 in discs 1-3. Up endplate in discs 4-6. Low endplate in discs 5-6.",0,1
586,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-5 show Pfirrmann grade 1. Discs 1-3, 6 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1. Modic type 2 in discs 1-3. Up endplate in discs 4-6. Low endplate in discs 5-6.",0,1
587,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Narrowing in discs 1, 3-7. Bulging in discs 1-4. Low endplate in discs 6.",0,1
588,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Narrowing in discs 1, 3-7. Bulging in discs 1-4. Low endplate in discs 6.",0,1
589,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1.,0,1
590,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1.,0,1
591,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1.,0,1
592,A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1. Herniation in discs 1. Bulging in discs 2. Up endplate in discs 5.,0,1
593,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 1. Discs 3-4 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2-3, 7. Bulging in discs 2-4. Low endplate in discs 2, 4-7. Up endplate in discs 2, 4-7.",0,1
594,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 1. Discs 3-4 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2-3, 7. Bulging in discs 2-4. Low endplate in discs 2, 4-7. Up endplate in discs 2, 4-7.",0,1
595,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Bulging in discs 1-3. Up endplate in discs 5-6. Low endplate in discs 6.",0,1
596,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Bulging in discs 1-3. Up endplate in discs 5-6. Low endplate in discs 6.",0,1
597,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 5 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 2-3.",0,1
598,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 5 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 2-3.",0,1
599,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 2-3, 5 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Bulging in discs 1. Modic type 2 in discs 1-2.",0,1
600,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 2-3, 5 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Bulging in discs 1. Modic type 2 in discs 1-2.",0,1
601,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1, 4. Bulging in discs 1-4. Modic type 2 in discs 1-4.",0,1
602,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1, 4. Bulging in discs 1-4. Modic type 2 in discs 1-4.",0,1
603,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1.,0,1
604,A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2-8 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Herniation in discs 1-2. Spondylolisthesis in discs 1-2. Modic type 2 in discs 1. Low endplate in discs 2-3. Up endplate in discs 2-3. Bulging in discs 3-4.,0,1
605,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2-8 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Herniation in discs 1-2. Spondylolisthesis in discs 1-2. Modic type 2 in discs 1. Low endplate in discs 2-3. Up endplate in discs 2-3. Bulging in discs 3-4.,0,1
606,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2-8 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Herniation in discs 1-2. Spondylolisthesis in discs 1-2. Modic type 2 in discs 1. Low endplate in discs 2-3. Up endplate in discs 2-3. Bulging in discs 3-4.,0,1
607,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 2. Discs 3-7 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 5. Bulging in discs 1-6. Narrowing in discs 2-3, 6-7. Low endplate in discs 2-7. Up endplate in discs 2-7.",0,1
608,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 1-4 show Pfirrmann grade 3. Bulging in discs 1-4. Narrowing in discs 2-4. Low endplate in discs 3-4. Up endplate in discs 3-4.,0,1
609,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 1-4 show Pfirrmann grade 3. Bulging in discs 1-4. Narrowing in discs 2-4. Low endplate in discs 3-4. Up endplate in discs 3-4.,0,1
610,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6 show Pfirrmann grade 3. Discs 2-5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2, 4, 7. Bulging in discs 1, 3-5, 7. Low endplate in discs 1, 5. Up endplate in discs 1. Herniation in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2, 4.",0,1
611,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6 show Pfirrmann grade 3. Discs 2-5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2, 4, 7. Bulging in discs 1, 3-5, 7. Low endplate in discs 1, 5. Up endplate in discs 1. Herniation in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2, 4.",0,1
612,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 5-6 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Narrowing in discs 2. Low endplate in discs 2-5. Up endplate in discs 2-5. Herniation in discs 2. Bulging in discs 3-4. Modic type 2 in discs 4-6.",0,1
613,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 5-6 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Narrowing in discs 2. Low endplate in discs 2-5. Up endplate in discs 2-5. Herniation in discs 2. Bulging in discs 3-4. Modic type 2 in discs 4-6.",0,1
614,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 4. Bulging in discs 1. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2.",0,1
615,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 4. Bulging in discs 1. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2.",0,1
616,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Bulging in discs 1, 4. Low endplate in discs 4-5. Up endplate in discs 4-5.",0,1
617,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Bulging in discs 1, 4. Low endplate in discs 4-5. Up endplate in discs 4-5.",0,1
618,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1, 6. Modic type 2 in discs 1, 3.",0,1
619,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1, 6. Modic type 2 in discs 1, 3.",0,1
620,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2. Low endplate in discs 2. Up endplate in discs 2. Spondylolisthesis in discs 2.,0,1
621,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2. Low endplate in discs 2. Up endplate in discs 2. Spondylolisthesis in discs 2.,0,1
622,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 1. Bulging in discs 1-4. Low endplate in discs 3-5, 7. Up endplate in discs 3-5, 7. Narrowing in discs 7.",0,1
623,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-3. Low endplate in discs 1-3. Up endplate in discs 1-3.,0,1
624,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 6-8 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 2-8. Up endplate in discs 2-8. Narrowing in discs 3-5. Modic type 2 in discs 3-6.",0,1
625,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 6-8 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 2-8. Up endplate in discs 2-8. Narrowing in discs 3-5. Modic type 2 in discs 3-6.",0,1
626,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 6-8 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 2-8. Up endplate in discs 2-8. Narrowing in discs 3-5. Modic type 2 in discs 3-6.",0,1
627,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 4-6 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Bulging in discs 1-5. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Narrowing in discs 3-4.,0,1
628,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 4-6 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Bulging in discs 1-5. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Narrowing in discs 3-4.,0,1
629,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2-3. Herniation in discs 2. Modic type 2 in discs 2. Bulging in discs 3-4.",0,1
630,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2-3. Herniation in discs 2. Modic type 2 in discs 2. Bulging in discs 3-4.",0,1
631,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 4 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 2 in discs 1. Bulging in discs 2-4.,0,1
632,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 4 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 2 in discs 1. Bulging in discs 2-4.,0,1
633,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6 show Pfirrmann grade 2. Discs 3-5, 7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-3. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-2.",0,1
634,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6 show Pfirrmann grade 2. Discs 3-5, 7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-3. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-2.",0,1
635,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-4 show Pfirrmann grade 1. Discs 6 show Pfirrmann grade 2. Discs 2, 5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-3. Low endplate in discs 1-2. Up endplate in discs 1, 5. Modic type 2 in discs 1.",0,1
636,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-4 show Pfirrmann grade 1. Discs 6 show Pfirrmann grade 2. Discs 2, 5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-3. Low endplate in discs 1-2. Up endplate in discs 1, 5. Modic type 2 in discs 1.",0,1
637,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2, 5-7 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1, 4 show Pfirrmann grade 5. Narrowing in discs 1, 3-4, 6-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-3, 5, 7. Modic type 3 in discs 4, 6.",0,1
638,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2, 5-7 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1, 4 show Pfirrmann grade 5. Narrowing in discs 1, 3-4, 6-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-3, 5, 7. Modic type 3 in discs 4, 6.",0,1
639,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Bulging in discs 1-4, 6. Narrowing in discs 2-3. Modic type 2 in discs 2-4. Up endplate in discs 3, 5-6. Low endplate in discs 5.",0,1
640,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Bulging in discs 1-4, 6. Narrowing in discs 2-3. Modic type 2 in discs 2-4. Up endplate in discs 3, 5-6. Low endplate in discs 5.",0,1
641,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 6.,0,1
642,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 6.,0,1
643,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Narrowing in discs 2-4. Bulging in discs 2-5. Low endplate in discs 2-6. Up endplate in discs 2-6. Modic type 2 in discs 2-4.,0,1
644,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Narrowing in discs 2-4. Bulging in discs 2-5. Low endplate in discs 2-6. Up endplate in discs 2-6. Modic type 2 in discs 2-4.,0,1
645,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
646,A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
647,A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
648,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-5, 7 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.",0,1
649,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-5, 7 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.",0,1
650,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Bulging in discs 1, 4.",0,1
651,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Bulging in discs 1, 4.",0,1
652,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1. Herniation in discs 1. Modic type 2 in discs 1.,0,1
653,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1. Herniation in discs 1. Modic type 2 in discs 1.,0,1
654,"A T1-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 1-2, 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 5. Bulging in discs 1-4. Modic type 2 in discs 1-4, 6-9. Narrowing in discs 3. Low endplate in discs 3, 6. Up endplate in discs 3, 6. Spondylolisthesis in discs 3.",0,1
655,"A T2-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 1-2, 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 5. Bulging in discs 1-4. Modic type 2 in discs 1-4, 6-9. Narrowing in discs 3. Low endplate in discs 3, 6. Up endplate in discs 3, 6. Spondylolisthesis in discs 3.",0,1
656,"A T2-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 1-2, 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 5. Bulging in discs 1-4. Modic type 2 in discs 1-4, 6-9. Narrowing in discs 3. Low endplate in discs 3, 6. Up endplate in discs 3, 6. Spondylolisthesis in discs 3.",0,1
657,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 1, 5 show Pfirrmann grade 4. Discs 2-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2-5.",0,1
658,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 1, 5 show Pfirrmann grade 4. Discs 2-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2-5.",0,1
659,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
660,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
661,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Herniation in discs 1. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
662,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Herniation in discs 1. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
663,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Herniation in discs 1. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
664,"A T1-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 9 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Discs 5-8 show Pfirrmann grade 4. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-8. Low endplate in discs 2-6. Up endplate in discs 2-6. Herniation in discs 4.",0,1
665,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 9 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Discs 5-8 show Pfirrmann grade 4. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-8. Low endplate in discs 2-6. Up endplate in discs 2-6. Herniation in discs 4.",0,1
666,"A T1-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 1 show Pfirrmann grade 2. Discs 9 show Pfirrmann grade 3. Discs 3-8 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 2-7. Bulging in discs 2-8. Low endplate in discs 2-9. Up endplate in discs 2-8. Herniation in discs 2. Modic type 2 in discs 2, 4.",0,1
667,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 1 show Pfirrmann grade 2. Discs 9 show Pfirrmann grade 3. Discs 3-8 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 2-7. Bulging in discs 2-8. Low endplate in discs 2-9. Up endplate in discs 2-8. Herniation in discs 2. Modic type 2 in discs 2, 4.",0,1
668,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 1 show Pfirrmann grade 2. Discs 9 show Pfirrmann grade 3. Discs 3-8 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 2-7. Bulging in discs 2-8. Low endplate in discs 2-9. Up endplate in discs 2-8. Herniation in discs 2. Modic type 2 in discs 2, 4.",0,1
669,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 4 show Pfirrmann grade 1. Discs 2, 6 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 4. Discs 3 show Pfirrmann grade 5. Bulging in discs 1-5. Narrowing in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5. Modic type 2 in discs 3.",0,1
670,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 4 show Pfirrmann grade 1. Discs 2, 6 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 4. Discs 3 show Pfirrmann grade 5. Bulging in discs 1-5. Narrowing in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5. Modic type 2 in discs 3.",0,1
671,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 3-7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2.",0,1
672,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 3-7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2.",0,1
673,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-7. Modic type 2 in discs 1-5. Low endplate in discs 2, 4-7. Up endplate in discs 2, 4-7.",0,1
674,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-7. Modic type 2 in discs 1-5. Low endplate in discs 2, 4-7. Up endplate in discs 2, 4-7.",0,1
675,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4, 6 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 5 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3, 5. Modic type 2 in discs 1, 3. Bulging in discs 5-6.",0,1
676,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4, 6 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 5 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3, 5. Modic type 2 in discs 1, 3. Bulging in discs 5-6.",0,1
677,A T2-weighted MRI of a female patient with 5 vertebrae and 5 discs. Discs 3-5 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2.,0,1
678,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6-7 show Pfirrmann grade 2. Discs 3-5 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 8 show Pfirrmann grade 5. Narrowing in discs 1-2, 8. Bulging in discs 1-5. Low endplate in discs 1-2, 8. Up endplate in discs 1-2, 7. Modic type 2 in discs 1-2, 6-8.",0,1
679,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6-7 show Pfirrmann grade 2. Discs 3-5 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 8 show Pfirrmann grade 5. Narrowing in discs 1-2, 8. Bulging in discs 1-5. Low endplate in discs 1-2, 8. Up endplate in discs 1-2, 7. Modic type 2 in discs 1-2, 6-8.",0,1
680,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6. Herniation in discs 2.",0,1
681,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6. Herniation in discs 2.",0,1
682,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-3. Modic type 2 in discs 4.,0,1
683,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-3. Modic type 2 in discs 4.,0,1
684,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 3. Discs 1, 4-6 show Pfirrmann grade 4. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 2-7. Herniation in discs 2-3, 6. Modic type 2 in discs 2-3. Up endplate in discs 7.",0,1
685,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 3. Discs 1, 4-6 show Pfirrmann grade 4. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 2-7. Herniation in discs 2-3, 6. Modic type 2 in discs 2-3. Up endplate in discs 7.",0,1
686,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 2. Herniation in discs 2.,0,1
687,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 2. Herniation in discs 2.,0,1
688,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 2. Discs 2-7 show Pfirrmann grade 4. Narrowing in discs 2-7. Bulging in discs 2-6. Spondylolisthesis in discs 3. Low endplate in discs 4-5. Up endplate in discs 4-5.,0,1
689,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 2. Discs 2-7 show Pfirrmann grade 4. Narrowing in discs 2-7. Bulging in discs 2-6. Spondylolisthesis in discs 3. Low endplate in discs 4-5. Up endplate in discs 4-5.,0,1
690,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4 show Pfirrmann grade 2. Discs 5-6 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5-6. Bulging in discs 1-2, 5-6. Herniation in discs 3.",0,1
691,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4 show Pfirrmann grade 2. Discs 5-6 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5-6. Bulging in discs 1-2, 5-6. Herniation in discs 3.",0,1
692,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 1-3. Narrowing in discs 2. Herniation in discs 2. Modic type 2 in discs 2. Up endplate in discs 3. Low endplate in discs 6.,0,1
693,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 1-3. Narrowing in discs 2. Herniation in discs 2. Modic type 2 in discs 2. Up endplate in discs 3. Low endplate in discs 6.,0,1
694,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Narrowing in discs 1-3. Bulging in discs 1-4. Herniation in discs 2. Modic type 2 in discs 2. Low endplate in discs 3. Up endplate in discs 3.,0,1
695,A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Narrowing in discs 1-3. Bulging in discs 1-4. Herniation in discs 2. Modic type 2 in discs 2. Low endplate in discs 3. Up endplate in discs 3.,0,1
696,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 2. Discs 6 show Pfirrmann grade 3. Discs 1-4 show Pfirrmann grade 4. Bulging in discs 1-4. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6. Herniation in discs 2.,0,1
697,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 2. Discs 6 show Pfirrmann grade 3. Discs 1-4 show Pfirrmann grade 4. Bulging in discs 1-4. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6. Herniation in discs 2.,0,1
698,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 2, 6 show Pfirrmann grade 2. Discs 1, 4-5 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 5. Modic type 2 in discs 1-3. Narrowing in discs 2-5. Low endplate in discs 3-5. Up endplate in discs 3-6. Bulging in discs 5.",0,1
699,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 2, 6 show Pfirrmann grade 2. Discs 1, 4-5 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 5. Modic type 2 in discs 1-3. Narrowing in discs 2-5. Low endplate in discs 3-5. Up endplate in discs 3-6. Bulging in discs 5.",0,1
700,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5, 7 show Pfirrmann grade 1. Discs 2-4, 6 show Pfirrmann grade 2. Low endplate in discs 1-2, 7. Up endplate in discs 1, 5-7. Narrowing in discs 2-4. Bulging in discs 2-4, 6-7.",0,1
701,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5, 7 show Pfirrmann grade 1. Discs 2-4, 6 show Pfirrmann grade 2. Low endplate in discs 1-2, 7. Up endplate in discs 1, 5-7. Narrowing in discs 2-4. Bulging in discs 2-4, 6-7.",0,1
702,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-4, 6 show Pfirrmann grade 4. Discs 1-2, 5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-4, 6. Herniation in discs 1-2. Modic type 2 in discs 1-4. Spondylolisthesis in discs 2-4.",0,1
703,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-4, 6 show Pfirrmann grade 4. Discs 1-2, 5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-4, 6. Herniation in discs 1-2. Modic type 2 in discs 1-4. Spondylolisthesis in discs 2-4.",0,1
704,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3 show Pfirrmann grade 3. Discs 1-2, 4-6 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 2-3. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2-5.",0,1
705,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3 show Pfirrmann grade 3. Discs 1-2, 4-6 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 2-3. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2-5.",0,1
706,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 1, 3-5 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Bulging in discs 2-5. Low endplate in discs 3-7. Up endplate in discs 3-7.",0,1
707,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 1, 3-5 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Bulging in discs 2-5. Low endplate in discs 3-7. Up endplate in discs 3-7.",0,1
708,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-3, 5-8 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Narrowing in discs 2, 4. Bulging in discs 2-6. Low endplate in discs 2, 4-7. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2-4. Up endplate in discs 3-7.",0,1
709,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-3, 5-8 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Narrowing in discs 2, 4. Bulging in discs 2-6. Low endplate in discs 2, 4-7. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2-4. Up endplate in discs 3-7.",0,1
710,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 5-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2.",0,1
711,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 5-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2.",0,1
712,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4-5 show Pfirrmann grade 2. Discs 3, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Herniation in discs 1. Modic type 2 in discs 1, 4-5. Up endplate in discs 3, 5-6. Low endplate in discs 5-6.",0,1
713,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4-5 show Pfirrmann grade 2. Discs 3, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Herniation in discs 1. Modic type 2 in discs 1, 4-5. Up endplate in discs 3, 5-6. Low endplate in discs 5-6.",0,1
714,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2, 4, 6 show Pfirrmann grade 3. Discs 3, 5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Modic type 2 in discs 1. Low endplate in discs 3.",0,1
715,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2, 4, 6 show Pfirrmann grade 3. Discs 3, 5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Modic type 2 in discs 1. Low endplate in discs 3.",0,1
716,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2, 4, 6 show Pfirrmann grade 3. Discs 3, 5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Modic type 2 in discs 1. Low endplate in discs 3.",0,1
717,A T1-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-3. Low endplate in discs 3-8. Up endplate in discs 3-8.,0,1
718,A T2-weighted MRI of a female patient with 3 vertebrae and 3 discs. Discs 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-3. Low endplate in discs 3-8. Up endplate in discs 3-8.,0,1
719,A T2-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-3. Low endplate in discs 3-8. Up endplate in discs 3-8.,0,1
720,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 1, 3, 5-6 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-4. Narrowing in discs 2, 4-6. Spondylolisthesis in discs 4. Modic type 2 in discs 4-5.",0,1
721,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 1, 3, 5-6 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-4. Narrowing in discs 2, 4-6. Spondylolisthesis in discs 4. Modic type 2 in discs 4-5.",0,1
722,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-3. Low endplate in discs 1, 3. Up endplate in discs 1, 3. Narrowing in discs 3. Modic type 2 in discs 3.",0,1
723,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-3. Low endplate in discs 1, 3. Up endplate in discs 1, 3. Narrowing in discs 3. Modic type 2 in discs 3.",0,1
724,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-7.",0,1
725,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-7.",0,1
726,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 5. Up endplate in discs 5. Modic type 2 in discs 5.,0,1
727,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 5. Up endplate in discs 5. Modic type 2 in discs 5.,0,1
728,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 2, 5-6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 3.",0,1
729,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 2, 5-6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 3.",0,1
730,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 1. Modic type 2 in discs 5. Low endplate in discs 6-7. Up endplate in discs 6-7. No disc herniation, disc narrowing or disc bulging is observed.",0,1
731,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 1. Modic type 2 in discs 5. Low endplate in discs 6-7. Up endplate in discs 6-7. No disc herniation, disc narrowing or disc bulging is observed.",0,1
732,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
733,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Narrowing in discs 1. Bulging in discs 1.,0,1
734,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 2, 4-5 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-5. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Herniation in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 5.",0,1
735,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 2, 4-5 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-5. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Herniation in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 5.",0,1
736,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Narrowing in discs 1. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Bulging in discs 2, 4.",0,1
737,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Narrowing in discs 1. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Bulging in discs 2, 4.",0,1
738,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Narrowing in discs 1. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Bulging in discs 2, 4.",0,1
739,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Up endplate in discs 1-3, 6-7. Bulging in discs 2. Low endplate in discs 2-4, 6. Narrowing in discs 3.",0,1
740,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Up endplate in discs 1-3, 6-7. Bulging in discs 2. Low endplate in discs 2-4, 6. Narrowing in discs 3.",0,1
741,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5 show Pfirrmann grade 2. Discs 1-4, 6-7 show Pfirrmann grade 3. Bulging in discs 1-3. Low endplate in discs 2-4, 6-7. Up endplate in discs 3-6.",0,1
742,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5 show Pfirrmann grade 2. Discs 1-4, 6-7 show Pfirrmann grade 3. Bulging in discs 1-3. Low endplate in discs 2-4, 6-7. Up endplate in discs 3-6.",0,1
743,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-2. Low endplate in discs 1-3. Up endplate in discs 1-3. Modic type 2 in discs 1-3. Herniation in discs 3. Spondylolisthesis in discs 3.,0,1
744,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-2. Low endplate in discs 1-3. Up endplate in discs 1-3. Modic type 2 in discs 1-3. Herniation in discs 3. Spondylolisthesis in discs 3.,0,1
745,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 3. Discs 1-7 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-7. Low endplate in discs 2-7. Up endplate in discs 2-7. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2, 5-6. Modic type 1 in discs 4.",0,1
746,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 3. Discs 1-7 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-7. Low endplate in discs 2-7. Up endplate in discs 2-7. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2, 5-6. Modic type 1 in discs 4.",0,1
747,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 3. Discs 1-7 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-7. Low endplate in discs 2-7. Up endplate in discs 2-7. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2, 5-6. Modic type 1 in discs 4.",0,1
748,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 7-8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Low endplate in discs 1, 3. Up endplate in discs 1, 3. Modic type 2 in discs 1.",0,1
749,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 7-8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Low endplate in discs 1, 3. Up endplate in discs 1, 3. Modic type 2 in discs 1.",0,1
750,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1, 4-5, 7 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1-5. Modic type 2 in discs 1-3.",0,1
751,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1, 4-5, 7 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1-5. Modic type 2 in discs 1-3.",0,1
752,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1, 4-5, 7 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1-5. Modic type 2 in discs 1-3.",0,1
753,A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 1 show Pfirrmann grade 1. Discs 3-8 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Bulging in discs 2.,0,1
754,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 1 show Pfirrmann grade 1. Discs 3-8 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Bulging in discs 2.,0,1
755,"A T1-weighted MRI of a male patient with 7 vertebrae and 8 discs. Discs 1, 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Bulging in discs 1-7. Narrowing in discs 2, 5, 7. Low endplate in discs 3-7. Up endplate in discs 3, 5-7. Modic type 2 in discs 3, 5. Spondylolisthesis in discs 6.",0,1
756,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Bulging in discs 1-7. Narrowing in discs 2, 5, 7. Low endplate in discs 3-7. Up endplate in discs 3, 5-7. Modic type 2 in discs 3, 5. Spondylolisthesis in discs 6.",0,1
757,"A T2-weighted MRI of a male patient with 7 vertebrae and 8 discs. Discs 1, 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Bulging in discs 1-7. Narrowing in discs 2, 5, 7. Low endplate in discs 3-7. Up endplate in discs 3, 5-7. Modic type 2 in discs 3, 5. Spondylolisthesis in discs 6.",0,1
758,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4, 7 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 2. Narrowing in discs 3. Low endplate in discs 3. Up endplate in discs 3. Herniation in discs 3. Modic type 2 in discs 3.",0,1
759,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4, 7 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 2. Narrowing in discs 3. Low endplate in discs 3. Up endplate in discs 3. Herniation in discs 3. Modic type 2 in discs 3.",0,1
760,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 4, 6-7 show Pfirrmann grade 3. Discs 2-3, 5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5. Bulging in discs 1-5. Low endplate in discs 1, 3, 5. Modic type 2 in discs 1, 3, 5. Herniation in discs 2. Up endplate in discs 5.",0,1
761,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4, 6-7 show Pfirrmann grade 3. Discs 2-3, 5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5. Bulging in discs 1-5. Low endplate in discs 1, 3, 5. Modic type 2 in discs 1, 3, 5. Herniation in discs 2. Up endplate in discs 5.",0,1
762,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 4, 6-7 show Pfirrmann grade 3. Discs 2-3, 5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5. Bulging in discs 1-5. Low endplate in discs 1, 3, 5. Modic type 2 in discs 1, 3, 5. Herniation in discs 2. Up endplate in discs 5.",0,1
763,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-4, 6 show Pfirrmann grade 3. Discs 1, 5, 7 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5, 7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.",0,1
764,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-4, 6 show Pfirrmann grade 3. Discs 1, 5, 7 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5, 7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.",0,1
765,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 4. Narrowing in discs 1-6. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-6. Herniation in discs 1. Modic type 2 in discs 1-6.,0,1
766,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 4. Narrowing in discs 1-6. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-6. Herniation in discs 1. Modic type 2 in discs 1-6.,0,1
767,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Bulging in discs 1-3.,0,1
768,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Bulging in discs 1-3.,0,1
769,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Bulging in discs 1-3.,0,1
770,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3 show Pfirrmann grade 2. Discs 1, 4-6 show Pfirrmann grade 3. Discs 7 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Bulging in discs 1, 3-5. Narrowing in discs 2, 4-7. Low endplate in discs 2, 4-5. Up endplate in discs 2, 4-5. Modic type 2 in discs 2, 4. Modic type 3 in discs 5.",0,1
771,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3 show Pfirrmann grade 2. Discs 1, 4-6 show Pfirrmann grade 3. Discs 7 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Bulging in discs 1, 3-5. Narrowing in discs 2, 4-7. Low endplate in discs 2, 4-5. Up endplate in discs 2, 4-5. Modic type 2 in discs 2, 4. Modic type 3 in discs 5.",0,1
772,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 3. Discs 3-4, 7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-4. Up endplate in discs 1-2, 4-5, 7-8. Modic type 2 in discs 1-4. Low endplate in discs 2-5, 7-8.",0,1
773,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 3. Discs 3-4, 7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-4. Up endplate in discs 1-2, 4-5, 7-8. Modic type 2 in discs 1-4. Low endplate in discs 2-5, 7-8.",0,1
774,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 3. Discs 3-4, 7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-4. Up endplate in discs 1-2, 4-5, 7-8. Modic type 2 in discs 1-4. Low endplate in discs 2-5, 7-8.",0,1
775,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 3-6 show Pfirrmann grade 2. Discs 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2, 7. Bulging in discs 1, 3-5. Low endplate in discs 1-2, 7. Up endplate in discs 1-2, 8. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2. Herniation in discs 2.",0,1
776,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 3-6 show Pfirrmann grade 2. Discs 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2, 7. Bulging in discs 1, 3-5. Low endplate in discs 1-2, 7. Up endplate in discs 1-2, 8. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2. Herniation in discs 2.",0,1
777,"A T1-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 8-9 show Pfirrmann grade 1. Discs 7 show Pfirrmann grade 2. Discs 1, 3-6 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-5. Modic type 2 in discs 1-7. Low endplate in discs 2-3. Up endplate in discs 2-7. Spondylolisthesis in discs 5.",0,1
778,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 8-9 show Pfirrmann grade 1. Discs 7 show Pfirrmann grade 2. Discs 1, 3-6 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-5. Modic type 2 in discs 1-7. Low endplate in discs 2-3. Up endplate in discs 2-7. Spondylolisthesis in discs 5.",0,1
779,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 1, 3-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2. Modic type 2 in discs 2. Low endplate in discs 3.",0,1
780,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 7 show Pfirrmann grade 1. Discs 1, 3-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2. Modic type 2 in discs 2. Low endplate in discs 3.",0,1
781,"A T1-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1, 3-6, 8-9 show Pfirrmann grade 4. Discs 2, 7 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7, 9. Low endplate in discs 2-9. Up endplate in discs 2-9. Modic type 2 in discs 2-9.",0,1
782,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1, 3-6, 8-9 show Pfirrmann grade 4. Discs 2, 7 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7, 9. Low endplate in discs 2-9. Up endplate in discs 2-9. Modic type 2 in discs 2-9.",0,1
783,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1, 3-6, 8-9 show Pfirrmann grade 4. Discs 2, 7 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7, 9. Low endplate in discs 2-9. Up endplate in discs 2-9. Modic type 2 in discs 2-9.",0,1
784,"A T1-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1, 3-9 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Bulging in discs 1-4. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2-3.",0,1
785,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1, 3-9 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Bulging in discs 1-4. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2-3.",0,1
786,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Bulging in discs 1, 3. Herniation in discs 2.",0,1
787,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Bulging in discs 1, 3. Herniation in discs 2.",0,1
788,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 4. Discs 1-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5, 7-8. Modic type 2 in discs 1-3. Modic type 1 in discs 4-5.",0,1
789,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 4. Discs 1-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5, 7-8. Modic type 2 in discs 1-3. Modic type 1 in discs 4-5.",0,1
790,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 3-6 show Pfirrmann grade 3. Discs 2, 7 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1-7. Narrowing in discs 2. Low endplate in discs 2, 7. Up endplate in discs 2, 7.",0,1
791,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 3-6 show Pfirrmann grade 3. Discs 2, 7 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1-7. Narrowing in discs 2. Low endplate in discs 2, 7. Up endplate in discs 2, 7.",0,1
792,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 3-6 show Pfirrmann grade 3. Discs 2, 7 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1-7. Narrowing in discs 2. Low endplate in discs 2, 7. Up endplate in discs 2, 7.",0,1
793,A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 2. Narrowing in discs 1. Bulging in discs 1-3.,0,1
794,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1 show Pfirrmann grade 3. Discs 3, 5, 7-8 show Pfirrmann grade 4. Discs 2, 4, 6 show Pfirrmann grade 5. Narrowing in discs 1-2, 4, 6. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-2, 4, 6. Spondylolisthesis in discs 4.",0,1
795,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1 show Pfirrmann grade 3. Discs 3, 5, 7-8 show Pfirrmann grade 4. Discs 2, 4, 6 show Pfirrmann grade 5. Narrowing in discs 1-2, 4, 6. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-2, 4, 6. Spondylolisthesis in discs 4.",0,1
796,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 1. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Narrowing in discs 2-4.,0,1
797,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 4-7.",0,1
798,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 4-7.",0,1
799,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-5 show Pfirrmann grade 2. Discs 1-3, 6 show Pfirrmann grade 4. Narrowing in discs 1-3, 6. Bulging in discs 1-3. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
800,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-5 show Pfirrmann grade 2. Discs 1-3, 6 show Pfirrmann grade 4. Narrowing in discs 1-3, 6. Bulging in discs 1-3. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
801,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Herniation in discs 1. Bulging in discs 2-3. Low endplate in discs 2-3. Up endplate in discs 2-3. Modic type 2 in discs 2-4.,0,1
802,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Herniation in discs 1. Bulging in discs 2-3. Low endplate in discs 2-3. Up endplate in discs 2-3. Modic type 2 in discs 2-4.,0,1
803,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
804,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 3 in discs 1.,0,1
805,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 3 in discs 1.,0,1
806,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 3 in discs 1.,0,1
807,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-3. Low endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
808,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-3. Low endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
809,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-3. Low endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
810,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 3. Herniation in discs 2. Low endplate in discs 4-5. Up endplate in discs 4-5.",0,1
811,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 3. Herniation in discs 2. Low endplate in discs 4-5. Up endplate in discs 4-5.",0,1
812,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2.,0,1
813,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2.,0,1
814,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2.,0,1
815,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 4. Discs 1-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-7.,0,1
816,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 4. Discs 1-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-7.,0,1
817,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Bulging in discs 1-6. Narrowing in discs 3-4. Low endplate in discs 3-6. Up endplate in discs 3-6. Modic type 2 in discs 3-6.,0,1
818,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Bulging in discs 1-6. Narrowing in discs 3-4. Low endplate in discs 3-6. Up endplate in discs 3-6. Modic type 2 in discs 3-6.,0,1
819,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 7-8 show Pfirrmann grade 4. Discs 1-6 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-8. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.,0,1
820,A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 7-8 show Pfirrmann grade 4. Discs 1-6 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-8. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.,0,1
821,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 1. Discs 4, 6 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-4. Low endplate in discs 1-4, 6. Up endplate in discs 1-4. Modic type 2 in discs 1-4.",0,1
822,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 1. Discs 4, 6 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-4. Low endplate in discs 1-4, 6. Up endplate in discs 1-4. Modic type 2 in discs 1-4.",0,1
823,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 5 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Bulging in discs 1-5. Narrowing in discs 3. Low endplate in discs 5. Up endplate in discs 5. Modic type 2 in discs 5.,0,1
824,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 5 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Bulging in discs 1-5. Narrowing in discs 3. Low endplate in discs 5. Up endplate in discs 5. Modic type 2 in discs 5.,0,1
825,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 2, 4-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Modic type 2 in discs 1-5. Low endplate in discs 2, 4-5. Up endplate in discs 2, 4-5.",0,1
826,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 2, 4-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Modic type 2 in discs 1-5. Low endplate in discs 2, 4-5. Up endplate in discs 2, 4-5.",0,1
827,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 3, 5-6 show Pfirrmann grade 2. Discs 2, 4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Low endplate in discs 1. Herniation in discs 1. Bulging in discs 4.",0,1
828,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 2. Discs 4-7 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Modic type 2 in discs 1, 4-5.",0,1
829,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 2. Discs 4-7 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Modic type 2 in discs 1, 4-5.",0,1
830,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 5-6 show Pfirrmann grade 3. Discs 2-4 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-4. Low endplate in discs 2, 4-5. Up endplate in discs 4-5. Modic type 2 in discs 4.",0,1
831,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 5-6 show Pfirrmann grade 3. Discs 2-4 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-4. Low endplate in discs 2, 4-5. Up endplate in discs 4-5. Modic type 2 in discs 4.",0,1
832,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-6 show Pfirrmann grade 2. Discs 1-4, 7 show Pfirrmann grade 3. Bulging in discs 1-4. Narrowing in discs 3. Low endplate in discs 3-4, 7. Up endplate in discs 3-4, 7.",0,1
833,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-2, 4. Low endplate in discs 1-3, 6. Modic type 2 in discs 1-3, 6. Herniation in discs 3.",0,1
834,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-2, 4. Low endplate in discs 1-3, 6. Modic type 2 in discs 1-3, 6. Herniation in discs 3.",0,1
835,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 1. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 1-3. Narrowing in discs 2-3.,0,1
836,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 1. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 1-3. Narrowing in discs 2-3.,0,1
837,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 2 in discs 1.,0,1
838,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 2 in discs 1.,0,1
839,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5. Low endplate in discs 1-3. Up endplate in discs 1-3, 5. Modic type 2 in discs 1-2.",0,1
840,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5. Low endplate in discs 1-3. Up endplate in discs 1-3, 5. Modic type 2 in discs 1-2.",0,1
841,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 4. Discs 1, 3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5.",0,1
842,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-7 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1-2, 4-5, 7. Up endplate in discs 1-2, 4. Herniation in discs 1-2. Modic type 2 in discs 1-2.",0,1
843,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1-2, 4-5, 7. Up endplate in discs 1-2, 4. Herniation in discs 1-2. Modic type 2 in discs 1-2.",0,1
844,"A T2-weighted MRI of a female patient with 7 vertebrae and 8 discs. Discs 3-7 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1-2, 4-5, 7. Up endplate in discs 1-2, 4. Herniation in discs 1-2. Modic type 2 in discs 1-2.",0,1
845,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2-5, 7-8. Up endplate in discs 2-5, 7-8. Herniation in discs 2.",0,1
846,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 3. Discs 2-5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-7.,0,1
847,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 3. Discs 2-5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-7.,0,1
848,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
849,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
850,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
851,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
852,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
853,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
854,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
855,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
856,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
857,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
858,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
859,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
860,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
861,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
862,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
863,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
864,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
865,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
866,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
867,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
868,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
869,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
870,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
871,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
872,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
873,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
874,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
875,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
876,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
877,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
878,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
879,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
880,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
881,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
882,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
883,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
884,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
885,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
886,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
887,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
888,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
889,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
890,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
891,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
892,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
893,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
894,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
895,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
896,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
897,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
898,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
899,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
900,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
901,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
902,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
903,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
904,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
905,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
906,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
907,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
908,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
909,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
910,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
911,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
912,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
913,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
914,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
915,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
916,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
917,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
918,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
919,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
920,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
921,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
922,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
923,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
924,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
925,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
926,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
927,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
928,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
929,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
930,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
931,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
932,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
933,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
934,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
935,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
936,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
937,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
938,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
939,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
940,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
941,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
942,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
943,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
944,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
945,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
946,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
947,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
948,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
949,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
950,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
951,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
952,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
953,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
954,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
955,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
956,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
957,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
958,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
959,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
960,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
961,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
962,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
963,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
964,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
965,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
966,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
967,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
968,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
969,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
970,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
971,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
972,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
973,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
974,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
975,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
976,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
977,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
978,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
979,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
980,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
981,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
982,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
983,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
984,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
985,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
986,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
987,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
988,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
989,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
990,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
991,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
992,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
993,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
994,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
995,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
996,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
997,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
998,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
999,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1000,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1001,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1002,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1003,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1004,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1005,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1006,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1007,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1008,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1009,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1010,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1011,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1012,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1013,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1014,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1015,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1016,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1017,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1018,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1019,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1020,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1021,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1022,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1023,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1024,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1025,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1026,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1027,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1028,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1029,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1030,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1031,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1032,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1033,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1034,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1035,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1036,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1037,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1038,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1039,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1040,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1041,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1042,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1043,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1044,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1045,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1046,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1047,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1048,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1049,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1050,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1051,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1052,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1053,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1054,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1055,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1056,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1057,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1058,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1059,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1060,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1061,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1062,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1063,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1064,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1065,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1066,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1067,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1068,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1069,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1070,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1071,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1072,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1073,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1074,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1075,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1076,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1077,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1078,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1079,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1080,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1081,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1082,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1083,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1084,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1085,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1086,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1087,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1088,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1089,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1090,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1091,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1092,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1093,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1094,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1095,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1096,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1097,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1098,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1099,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1100,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1101,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1102,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1103,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1104,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1105,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1106,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1107,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1108,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1109,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1110,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1111,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1112,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1113,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1114,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1115,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1116,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1117,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1118,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1119,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1120,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1121,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1122,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1123,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1124,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1125,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1126,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1127,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1128,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1129,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1130,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1131,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1132,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1133,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1134,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1135,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1136,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1137,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1138,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1139,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1140,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1141,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1142,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1143,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1144,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1145,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1146,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1147,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1148,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1149,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1150,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1151,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1152,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1153,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1154,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1155,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1156,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1157,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1158,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1159,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1160,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1161,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1162,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1163,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1164,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1165,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1166,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1167,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1168,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1169,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1170,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1171,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1172,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1173,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1174,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1175,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1176,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1177,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1178,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1179,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1180,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1181,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1182,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1183,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1184,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1185,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1186,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1187,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1188,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1189,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1190,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1191,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1192,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1193,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1194,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1195,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1196,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1197,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1198,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1199,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1200,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1201,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1202,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1203,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1204,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1205,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1206,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1207,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1208,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1209,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1210,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1211,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1212,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1213,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1214,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1215,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1216,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1217,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1218,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1219,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1220,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1221,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1222,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1223,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1224,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1225,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1226,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1227,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1228,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1229,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1230,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1231,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1232,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1233,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1234,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1235,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1236,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1237,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1238,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1239,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1240,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1241,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1242,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1243,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1244,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1245,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1246,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1247,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1248,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1249,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1250,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1251,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1252,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1253,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1254,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1255,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1256,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1257,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1258,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1259,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1260,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1261,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1262,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1263,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1264,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1265,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1266,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1267,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1268,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1269,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1270,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1271,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1272,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1273,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1274,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1275,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1276,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1277,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1278,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1279,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1280,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1281,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1282,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1283,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1284,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1285,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1286,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1287,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1288,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1289,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1290,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1291,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1292,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1293,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1294,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1295,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1296,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1297,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1298,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1299,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1300,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1301,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1302,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1303,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1304,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1305,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1306,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1307,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1308,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1309,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1310,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1311,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1312,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1313,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1314,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1315,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1316,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1317,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1318,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1319,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1320,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1321,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1322,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1323,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1324,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1325,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1326,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1327,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1328,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1329,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1330,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1331,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1332,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1333,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1334,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1335,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1336,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1337,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1338,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1339,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1340,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1341,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1342,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1343,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1344,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1345,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1346,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1347,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0

Overwriting local_data/sentence-label.csv


## If you want to use `spider` dataset, execute the below cells

In [4]:
%%writefile local_data/lbp-train-meta.csv
,imgpath,report,No Finding,LBP
0,local_data/spider/123_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5 show Pfirrmann grade 1. Discs 2-4, 6-8 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 5. Herniation in discs 1. Bulging in discs 2, 5. Up endplate in discs 7.",0,1
1,local_data/spider/33_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4-5 show Pfirrmann grade 2. Discs 3, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Herniation in discs 1. Modic type 2 in discs 1, 4-5. Up endplate in discs 3, 5-6. Low endplate in discs 5-6.",0,1
2,local_data/spider/P96 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
3,local_data/spider/P6-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
4,local_data/spider/257_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 2. Discs 6 show Pfirrmann grade 3. Discs 1-4 show Pfirrmann grade 4. Bulging in discs 1-4. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6. Herniation in discs 2.,0,1
5,local_data/spider/P62-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
6,local_data/spider/50_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 3. Discs 1-7 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-7. Low endplate in discs 2-7. Up endplate in discs 2-7. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2, 5-6. Modic type 1 in discs 4.",0,1
7,local_data/spider/P83 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
8,local_data/spider/67_t1.png,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Bulging in discs 1, 3. Herniation in discs 2.",0,1
9,local_data/spider/131_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
10,local_data/spider/P50-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
11,local_data/spider/P64-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
12,local_data/spider/P53-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
13,local_data/spider/38_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-7.",0,1
14,local_data/spider/58_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-4, 6 show Pfirrmann grade 3. Discs 1, 5, 7 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5, 7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.",0,1
15,local_data/spider/P69-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
16,local_data/spider/12_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 3-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 4. Bulging in discs 1. Narrowing in discs 2. Herniation in discs 2.",0,1
17,local_data/spider/P12-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
18,local_data/spider/P22-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
19,local_data/spider/P46-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
20,local_data/spider/142_t2_SPACE.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 3. Discs 1-5 show Pfirrmann grade 4. Up endplate in discs 2, 4-5. Herniation in discs 2. Spondylolisthesis in discs 2. Bulging in discs 3-5. Narrowing in discs 4. Low endplate in discs 4-5, 7.",0,1
21,local_data/spider/P48-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
22,local_data/spider/83_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Bulging in discs 1-6. Narrowing in discs 3-4. Low endplate in discs 3-6. Up endplate in discs 3-6. Modic type 2 in discs 3-6.,0,1
23,local_data/spider/18_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 2. Discs 3-4, 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5, 7-8. Bulging in discs 1-5, 7-8. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Modic type 2 in discs 1-2, 5.",0,1
24,local_data/spider/P10-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
25,local_data/spider/221_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 4-6 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Bulging in discs 1-5. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Narrowing in discs 3-4.,0,1
26,local_data/spider/P78 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
27,local_data/spider/P28-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
28,local_data/spider/23_t1.png,"A T1-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 9 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Discs 5-8 show Pfirrmann grade 4. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-8. Low endplate in discs 2-6. Up endplate in discs 2-6. Herniation in discs 4.",0,1
29,local_data/spider/50_t2_SPACE.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 3. Discs 1-7 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-7. Low endplate in discs 2-7. Up endplate in discs 2-7. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2, 5-6. Modic type 1 in discs 4.",0,1
30,local_data/spider/P97 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
31,local_data/spider/P60-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
32,local_data/spider/P41-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
33,local_data/spider/121_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 3-4 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Modic type 2 in discs 1-3. Up endplate in discs 2.",0,1
34,local_data/spider/P60-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
35,local_data/spider/P47-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
36,local_data/spider/P70 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
37,local_data/spider/P33-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
38,local_data/spider/P2-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
39,local_data/spider/P43-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
40,local_data/spider/P88 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
41,local_data/spider/P38-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
42,local_data/spider/P54-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
43,local_data/spider/P93 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
44,local_data/spider/P81 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
45,local_data/spider/82_t1.png,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 4. Discs 1-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-7.,0,1
46,local_data/spider/163_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2, 7 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-3, 7. Low endplate in discs 1-3, 5-7. Modic type 2 in discs 1-6. Spondylolisthesis in discs 2. Up endplate in discs 3-4, 6-7.",0,1
47,local_data/spider/95_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 2 in discs 1.,0,1
48,local_data/spider/P65-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
49,local_data/spider/P55-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
50,local_data/spider/104_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-6 show Pfirrmann grade 3. Discs 1-4, 7 show Pfirrmann grade 4. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 1-4.",0,1
51,local_data/spider/42_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Narrowing in discs 1. Bulging in discs 1.,0,1
52,local_data/spider/255_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 1-3. Narrowing in discs 2. Herniation in discs 2. Modic type 2 in discs 2. Up endplate in discs 3. Low endplate in discs 6.,0,1
53,local_data/spider/P16-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
54,local_data/spider/217_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1, 6. Modic type 2 in discs 1, 3.",0,1
55,local_data/spider/P80 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
56,local_data/spider/P82 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
57,local_data/spider/55_t2_SPACE.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 8 discs. Discs 1, 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Bulging in discs 1-7. Narrowing in discs 2, 5, 7. Low endplate in discs 3-7. Up endplate in discs 3, 5-7. Modic type 2 in discs 3, 5. Spondylolisthesis in discs 6.",0,1
58,local_data/spider/170_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 1-4. Bulging in discs 2-5. Narrowing in discs 3-5.,0,1
59,local_data/spider/61_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 3. Discs 3-4, 7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-4. Up endplate in discs 1-2, 4-5, 7-8. Modic type 2 in discs 1-4. Low endplate in discs 2-5, 7-8.",0,1
60,local_data/spider/P11-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
61,local_data/spider/243_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 3-7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2.",0,1
62,local_data/spider/125_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4, 6-7 show Pfirrmann grade 2. Discs 1-3, 5 show Pfirrmann grade 3. Narrowing in discs 2. Up endplate in discs 3, 5. Bulging in discs 5. Modic type 2 in discs 5.",0,1
63,local_data/spider/115_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Bulging in discs 2-4. Low endplate in discs 2-4. Up endplate in discs 2-4. Modic type 2 in discs 2-3.",0,1
64,local_data/spider/P1-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
65,local_data/spider/P18-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
66,local_data/spider/65_t2.png,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1, 3-6, 8-9 show Pfirrmann grade 4. Discs 2, 7 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7, 9. Low endplate in discs 2-9. Up endplate in discs 2-9. Modic type 2 in discs 2-9.",0,1
67,local_data/spider/246_t2.png,A T2-weighted MRI of a female patient with 5 vertebrae and 5 discs. Discs 3-5 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2.,0,1
68,local_data/spider/64_t2.png,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 7 show Pfirrmann grade 1. Discs 1, 3-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2. Modic type 2 in discs 2. Low endplate in discs 3.",0,1
69,local_data/spider/1_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Narrowing in discs 1, 3-7. Bulging in discs 1-4. Low endplate in discs 6.",0,1
70,local_data/spider/P21-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
71,local_data/spider/36_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 1, 3, 5-6 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-4. Narrowing in discs 2, 4-6. Spondylolisthesis in discs 4. Modic type 2 in discs 4-5.",0,1
72,local_data/spider/P87 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
73,local_data/spider/P100 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
74,local_data/spider/P71 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
75,local_data/spider/P51-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
76,local_data/spider/10_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 4, 7 show Pfirrmann grade 3. Discs 5 show Pfirrmann grade 4. Discs 2-3, 6 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1, 4-6. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Herniation in discs 2-3.",0,1
77,local_data/spider/127_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5, 7-8 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Narrowing in discs 1, 3-4. Bulging in discs 1-4, 6. Low endplate in discs 3-4. Up endplate in discs 3-4. Modic type 2 in discs 3-4.",0,1
78,local_data/spider/255_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 1-3. Narrowing in discs 2. Herniation in discs 2. Modic type 2 in discs 2. Up endplate in discs 3. Low endplate in discs 6.,0,1
79,local_data/spider/P26-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
80,local_data/spider/P52-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
81,local_data/spider/179_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3, 6. Bulging in discs 1-6. Low endplate in discs 1-3, 6-7. Up endplate in discs 1-3, 6-7. Modic type 2 in discs 2.",0,1
82,local_data/spider/156_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 3-5 show Pfirrmann grade 5. Narrowing in discs 1, 3-5. Bulging in discs 1, 3-5. Low endplate in discs 1, 3-5. Up endplate in discs 1, 3-5. Spondylolisthesis in discs 1. Modic type 2 in discs 1-5. Herniation in discs 5.",0,1
83,local_data/spider/34_t2_SPACE.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2, 4, 6 show Pfirrmann grade 3. Discs 3, 5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Modic type 2 in discs 1. Low endplate in discs 3.",0,1
84,local_data/spider/137_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
85,local_data/spider/5_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Bulging in discs 1-3.,0,1
86,local_data/spider/P70 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
87,local_data/spider/15_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 2. Discs 1, 3 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 5. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Low endplate in discs 1, 4-6. Up endplate in discs 1, 4-6. Modic type 2 in discs 2-5. Herniation in discs 4.",0,1
88,local_data/spider/P36-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
89,local_data/spider/142_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 3. Discs 1-5 show Pfirrmann grade 4. Up endplate in discs 2, 4-5. Herniation in discs 2. Spondylolisthesis in discs 2. Bulging in discs 3-5. Narrowing in discs 4. Low endplate in discs 4-5, 7.",0,1
90,local_data/spider/P89 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
91,local_data/spider/P67-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
92,local_data/spider/P30-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
93,local_data/spider/P37-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
94,local_data/spider/P44-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
95,local_data/spider/P4-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
96,local_data/spider/P59-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
97,local_data/spider/112_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1.,0,1
98,local_data/spider/P13-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
99,local_data/spider/P88 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
100,local_data/spider/73_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 4-7.",0,1
101,local_data/spider/P17-1.jpeg,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
102,local_data/spider/108_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 3-5 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Narrowing in discs 1, 4. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6.",0,1
103,local_data/spider/P95 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
104,local_data/spider/P29-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
105,local_data/spider/110_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-4, 6-7 show Pfirrmann grade 3. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1, 5. Bulging in discs 1, 3, 5. Low endplate in discs 1, 3, 5, 7. Up endplate in discs 1, 3, 5, 7. Modic type 2 in discs 1-2, 5.",0,1
106,local_data/spider/P41-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
107,local_data/spider/P61-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
108,local_data/spider/P13-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
109,local_data/spider/P58-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
110,local_data/spider/P9-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
111,local_data/spider/175_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3, 5-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2. Bulging in discs 3-4.",0,1
112,local_data/spider/86_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 1. Discs 4, 6 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-4. Low endplate in discs 1-4, 6. Up endplate in discs 1-4. Modic type 2 in discs 1-4.",0,1
113,local_data/spider/250_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-3. Modic type 2 in discs 4.,0,1
114,local_data/spider/22_t1.png,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
115,local_data/spider/P64-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
116,local_data/spider/186_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Herniation in discs 1. Modic type 2 in discs 1-7.,0,1
117,local_data/spider/104_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-6 show Pfirrmann grade 3. Discs 1-4, 7 show Pfirrmann grade 4. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 1-4.",0,1
118,local_data/spider/P57-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
119,local_data/spider/222_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2-3. Herniation in discs 2. Modic type 2 in discs 2. Bulging in discs 3-4.",0,1
120,local_data/spider/129_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 4.,0,1
121,local_data/spider/P33-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
122,local_data/spider/86_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 1. Discs 4, 6 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-4. Low endplate in discs 1-4, 6. Up endplate in discs 1-4. Modic type 2 in discs 1-4.",0,1
123,local_data/spider/130_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 2. Discs 6-7 show Pfirrmann grade 3. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 2-5. Bulging in discs 2-5. Low endplate in discs 2-7. Up endplate in discs 2-7. Modic type 2 in discs 2-6. Spondylolisthesis in discs 5.,0,1
124,local_data/spider/177_t2.png,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 7-9 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-5, 7. Modic type 2 in discs 1-5. Up endplate in discs 2-7. Herniation in discs 7.",0,1
125,local_data/spider/P13-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
126,local_data/spider/160_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 6 show Pfirrmann grade 1. Discs 3, 5 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-4. Low endplate in discs 2, 4. Modic type 2 in discs 2-4. Up endplate in discs 5-6.",0,1
127,local_data/spider/P93 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
128,local_data/spider/P35-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
129,local_data/spider/100_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 2-4, 8 show Pfirrmann grade 3. Modic type 2 in discs 1-6. Narrowing in discs 3. Bulging in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5.",0,1
130,local_data/spider/171_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-6 show Pfirrmann grade 3. Discs 2, 7-8 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-6. Low endplate in discs 1-4, 6-8. Up endplate in discs 1-4. Modic type 2 in discs 1-8. Narrowing in discs 2-3.",0,1
131,local_data/spider/P46-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
132,local_data/spider/P36-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
133,local_data/spider/P81 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
134,local_data/spider/P92 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
135,local_data/spider/223_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 4 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 2 in discs 1. Bulging in discs 2-4.,0,1
136,local_data/spider/220_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 6-8 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 2-8. Up endplate in discs 2-8. Narrowing in discs 3-5. Modic type 2 in discs 3-6.",0,1
137,local_data/spider/P72 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
138,local_data/spider/204_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 5 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 2-3.",0,1
139,local_data/spider/81_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2.,0,1
140,local_data/spider/97_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 4. Discs 1, 3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5.",0,1
141,local_data/spider/162_t1.png,A T1-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 1-9 show Pfirrmann grade 5. Narrowing in discs 1-9. Bulging in discs 1-9. Low endplate in discs 1-9. Up endplate in discs 1-9. Spondylolisthesis in discs 1. Modic type 2 in discs 1-9.,0,1
142,local_data/spider/P21-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
143,local_data/spider/34_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2, 4, 6 show Pfirrmann grade 3. Discs 3, 5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Modic type 2 in discs 1. Low endplate in discs 3.",0,1
144,local_data/spider/P70 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
145,local_data/spider/P86 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
146,local_data/spider/53_t2.png,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 1 show Pfirrmann grade 1. Discs 3-8 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Bulging in discs 2.,0,1
147,local_data/spider/P53-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
148,local_data/spider/200_t1.png,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1.,0,1
149,local_data/spider/P100 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
150,local_data/spider/61_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 3. Discs 3-4, 7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-4. Up endplate in discs 1-2, 4-5, 7-8. Modic type 2 in discs 1-4. Low endplate in discs 2-5, 7-8.",0,1
151,local_data/spider/P89 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
152,local_data/spider/22_t2_SPACE.png,A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
153,local_data/spider/210_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 1-4 show Pfirrmann grade 3. Bulging in discs 1-4. Narrowing in discs 2-4. Low endplate in discs 3-4. Up endplate in discs 3-4.,0,1
154,local_data/spider/P82 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
155,local_data/spider/P1-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
156,local_data/spider/P25-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
157,local_data/spider/1_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Narrowing in discs 1, 3-7. Bulging in discs 1-4. Low endplate in discs 6.",0,1
158,local_data/spider/P4-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
159,local_data/spider/P90 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
160,local_data/spider/212_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6 show Pfirrmann grade 3. Discs 2-5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2, 4, 7. Bulging in discs 1, 3-5, 7. Low endplate in discs 1, 5. Up endplate in discs 1. Herniation in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2, 4.",0,1
161,local_data/spider/P8-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
162,local_data/spider/P34-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
163,local_data/spider/200_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1.,0,1
164,local_data/spider/P52-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
165,local_data/spider/P40-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
166,local_data/spider/P95 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
167,local_data/spider/P21-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
168,local_data/spider/P100 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
169,local_data/spider/201_t2.png,A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1. Herniation in discs 1. Bulging in discs 2. Up endplate in discs 5.,0,1
170,local_data/spider/85_t1.png,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 7-8 show Pfirrmann grade 4. Discs 1-6 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-8. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.,0,1
171,local_data/spider/55_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Bulging in discs 1-7. Narrowing in discs 2, 5, 7. Low endplate in discs 3-7. Up endplate in discs 3, 5-7. Modic type 2 in discs 3, 5. Spondylolisthesis in discs 6.",0,1
172,local_data/spider/101_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 4. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6.,0,1
173,local_data/spider/P18-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
174,local_data/spider/244_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-7. Modic type 2 in discs 1-5. Low endplate in discs 2, 4-7. Up endplate in discs 2, 4-7.",0,1
175,local_data/spider/P88 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
176,local_data/spider/P41-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
177,local_data/spider/227_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Bulging in discs 1-4, 6. Narrowing in discs 2-3. Modic type 2 in discs 2-4. Up endplate in discs 3, 5-6. Low endplate in discs 5.",0,1
178,local_data/spider/35_t1.png,A T1-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-3. Low endplate in discs 3-8. Up endplate in discs 3-8.,0,1
179,local_data/spider/196_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-2 show Pfirrmann grade 1. Discs 3-6 show Pfirrmann grade 2. Low endplate in discs 2-5. Up endplate in discs 2-6. Modic type 2 in discs 2-5. Spondylolisthesis in discs 6. No disc herniation, disc narrowing or disc bulging is observed.",0,1
180,local_data/spider/P95 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
181,local_data/spider/256_t1.png,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Narrowing in discs 1-3. Bulging in discs 1-4. Herniation in discs 2. Modic type 2 in discs 2. Low endplate in discs 3. Up endplate in discs 3.,0,1
182,local_data/spider/68_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 4. Discs 1-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5, 7-8. Modic type 2 in discs 1-3. Modic type 1 in discs 4-5.",0,1
183,local_data/spider/179_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3, 6. Bulging in discs 1-6. Low endplate in discs 1-3, 6-7. Up endplate in discs 1-3, 6-7. Modic type 2 in discs 2.",0,1
184,local_data/spider/210_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 1-4 show Pfirrmann grade 3. Bulging in discs 1-4. Narrowing in discs 2-4. Low endplate in discs 3-4. Up endplate in discs 3-4.,0,1
185,local_data/spider/P40-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
186,local_data/spider/P48-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
187,local_data/spider/81_t1.png,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2.,0,1
188,local_data/spider/P27-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
189,local_data/spider/P23-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
190,local_data/spider/P75 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
191,local_data/spider/228_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 6.,0,1
192,local_data/spider/P5-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
193,local_data/spider/21_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-3. Low endplate in discs 1-3. Up endplate in discs 1-3.,0,1
194,local_data/spider/152_t2_SPACE.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 0 show Pfirrmann grade 1. Discs 6-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 2-3, 8 show Pfirrmann grade 4. Discs 4-5 show Pfirrmann grade 5. Narrowing in discs 1-5, 8. Bulging in discs 2-8. Spondylolisthesis in discs 2. Low endplate in discs 4-5, 7-8. Up endplate in discs 4-5, 7-8. Modic type 2 in discs 4-5.",0,1
195,local_data/spider/P20-5.jpeg,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
196,local_data/spider/P20-2.jpeg,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
197,local_data/spider/41_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
198,local_data/spider/234_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 1-2, 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 5. Bulging in discs 1-4. Modic type 2 in discs 1-4, 6-9. Narrowing in discs 3. Low endplate in discs 3, 6. Up endplate in discs 3, 6. Spondylolisthesis in discs 3.",0,1
199,local_data/spider/P68-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
200,local_data/spider/93_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-2, 4. Low endplate in discs 1-3, 6. Modic type 2 in discs 1-3, 6. Herniation in discs 3.",0,1
201,local_data/spider/181_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-4 show Pfirrmann grade 4. Discs 1, 5-7 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.",0,1
202,local_data/spider/P95 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
203,local_data/spider/P34-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
204,local_data/spider/P11-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
205,local_data/spider/215_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Bulging in discs 1, 4. Low endplate in discs 4-5. Up endplate in discs 4-5.",0,1
206,local_data/spider/52_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1, 4-5, 7 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1-5. Modic type 2 in discs 1-3.",0,1
207,local_data/spider/64_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 1, 3-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2. Modic type 2 in discs 2. Low endplate in discs 3.",0,1
208,local_data/spider/P19-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
209,local_data/spider/30_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 1, 3-5 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Bulging in discs 2-5. Low endplate in discs 3-7. Up endplate in discs 3-7.",0,1
210,local_data/spider/P25-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
211,local_data/spider/66_t1.png,"A T1-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1, 3-9 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Bulging in discs 1-4. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2-3.",0,1
212,local_data/spider/P26-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
213,local_data/spider/69_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 3-6 show Pfirrmann grade 3. Discs 2, 7 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1-7. Narrowing in discs 2. Low endplate in discs 2, 7. Up endplate in discs 2, 7.",0,1
214,local_data/spider/P56-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
215,local_data/spider/P84 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
216,local_data/spider/P79 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
217,local_data/spider/214_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 4. Bulging in discs 1. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2.",0,1
218,local_data/spider/191_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 3. Discs 1-5, 7-8 show Pfirrmann grade 4. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 2-6, 8. Modic type 2 in discs 2-3, 5. Up endplate in discs 3-8.",0,1
219,local_data/spider/P35-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
220,local_data/spider/P1-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
221,local_data/spider/3_t2.png,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 2, 5-6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 3.",0,1
222,local_data/spider/197_t2.png,A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-4. Low endplate in discs 1-2. Up endplate in discs 1-2.,0,1
223,local_data/spider/P61-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
224,local_data/spider/65_t2_SPACE.png,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1, 3-6, 8-9 show Pfirrmann grade 4. Discs 2, 7 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7, 9. Low endplate in discs 2-9. Up endplate in discs 2-9. Modic type 2 in discs 2-9.",0,1
225,local_data/spider/P49-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
226,local_data/spider/198_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 3, 5 show Pfirrmann grade 4. Discs 1-2, 4 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
227,local_data/spider/P90 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
228,local_data/spider/P30-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
229,local_data/spider/191_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 3. Discs 1-5, 7-8 show Pfirrmann grade 4. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 2-6, 8. Modic type 2 in discs 2-3, 5. Up endplate in discs 3-8.",0,1
230,local_data/spider/P11-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
231,local_data/spider/224_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6 show Pfirrmann grade 2. Discs 3-5, 7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-3. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-2.",0,1
232,local_data/spider/P19-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
233,local_data/spider/P37-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
234,local_data/spider/72_t1.png,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 1. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Narrowing in discs 2-4.,0,1
235,local_data/spider/P36-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
236,local_data/spider/P15-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
237,local_data/spider/P87 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
238,local_data/spider/108_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 3-5 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Narrowing in discs 1, 4. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6.",0,1
239,local_data/spider/208_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1.,0,1
240,local_data/spider/P76 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
241,local_data/spider/96_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5. Low endplate in discs 1-3. Up endplate in discs 1-3, 5. Modic type 2 in discs 1-2.",0,1
242,local_data/spider/P32-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
243,local_data/spider/P12-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
244,local_data/spider/P96 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
245,local_data/spider/P60-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
246,local_data/spider/P32-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
247,local_data/spider/P60-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
248,local_data/spider/126_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 1. Narrowing in discs 2, 4. Bulging in discs 2-4. Low endplate in discs 2-7. Up endplate in discs 2-7.",0,1
249,local_data/spider/P64-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
250,local_data/spider/P65-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
251,local_data/spider/P94 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
252,local_data/spider/P37-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
253,local_data/spider/165_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 1-5 show Pfirrmann grade 3. Bulging in discs 1-5. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2, 4. Narrowing in discs 4.",0,1
254,local_data/spider/118_t1.png,A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
255,local_data/spider/P58-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
256,local_data/spider/P72 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
257,local_data/spider/221_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 4-6 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Bulging in discs 1-5. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Narrowing in discs 3-4.,0,1
258,local_data/spider/223_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 4 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 2 in discs 1. Bulging in discs 2-4.,0,1
259,local_data/spider/P43-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
260,local_data/spider/74_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-5 show Pfirrmann grade 2. Discs 1-3, 6 show Pfirrmann grade 4. Narrowing in discs 1-3, 6. Bulging in discs 1-3. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
261,local_data/spider/P43-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
262,local_data/spider/P1-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
263,local_data/spider/P63-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
264,local_data/spider/222_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2-3. Herniation in discs 2. Modic type 2 in discs 2. Bulging in discs 3-4.",0,1
265,local_data/spider/28_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5, 7 show Pfirrmann grade 1. Discs 2-4, 6 show Pfirrmann grade 2. Low endplate in discs 1-2, 7. Up endplate in discs 1, 5-7. Narrowing in discs 2-4. Bulging in discs 2-4, 6-7.",0,1
266,local_data/spider/P77 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
267,local_data/spider/P33-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
268,local_data/spider/P35-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
269,local_data/spider/57_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 4, 6-7 show Pfirrmann grade 3. Discs 2-3, 5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5. Bulging in discs 1-5. Low endplate in discs 1, 3, 5. Modic type 2 in discs 1, 3, 5. Herniation in discs 2. Up endplate in discs 5.",0,1
270,local_data/spider/P22-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
271,local_data/spider/P99 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
272,local_data/spider/P65-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
273,local_data/spider/91_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-6 show Pfirrmann grade 2. Discs 1-4, 7 show Pfirrmann grade 3. Bulging in discs 1-4. Narrowing in discs 3. Low endplate in discs 3-4, 7. Up endplate in discs 3-4, 7.",0,1
274,local_data/spider/P74 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
275,local_data/spider/P1-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
276,local_data/spider/P14-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
277,local_data/spider/P56-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
278,local_data/spider/217_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1, 6. Modic type 2 in discs 1, 3.",0,1
279,local_data/spider/P42-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
280,local_data/spider/136_t2_SPACE.png,A T2-weighted MRI of a female patient with 6 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 3-7. Up endplate in discs 3-7. Modic type 2 in discs 4.,0,1
281,local_data/spider/250_t1.png,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-3. Modic type 2 in discs 4.,0,1
282,local_data/spider/P84 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
283,local_data/spider/71_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1 show Pfirrmann grade 3. Discs 3, 5, 7-8 show Pfirrmann grade 4. Discs 2, 4, 6 show Pfirrmann grade 5. Narrowing in discs 1-2, 4, 6. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-2, 4, 6. Spondylolisthesis in discs 4.",0,1
284,local_data/spider/242_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 4 show Pfirrmann grade 1. Discs 2, 6 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 4. Discs 3 show Pfirrmann grade 5. Bulging in discs 1-5. Narrowing in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5. Modic type 2 in discs 3.",0,1
285,local_data/spider/47_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Up endplate in discs 1-3, 6-7. Bulging in discs 2. Low endplate in discs 2-4, 6. Narrowing in discs 3.",0,1
286,local_data/spider/53_t1.png,A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 1 show Pfirrmann grade 1. Discs 3-8 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Bulging in discs 2.,0,1
287,local_data/spider/37_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-3. Low endplate in discs 1, 3. Up endplate in discs 1, 3. Narrowing in discs 3. Modic type 2 in discs 3.",0,1
288,local_data/spider/P92 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
289,local_data/spider/13_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Discs 5-7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5-7. Bulging in discs 1-5. Low endplate in discs 1, 5-8. Up endplate in discs 1, 5-7. Modic type 2 in discs 2-7. Spondylolisthesis in discs 5.",0,1
290,local_data/spider/44_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 2, 4-5 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-5. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Herniation in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 5.",0,1
291,local_data/spider/P63-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
292,local_data/spider/185_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 6-8 show Pfirrmann grade 2. Discs 4-5 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 2-3. Low endplate in discs 2-8. Up endplate in discs 2-8. Modic type 2 in discs 2-7.",0,1
293,local_data/spider/P89 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
294,local_data/spider/214_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 4. Bulging in discs 1. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2.",0,1
295,local_data/spider/P76 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
296,local_data/spider/203_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Bulging in discs 1-3. Up endplate in discs 5-6. Low endplate in discs 6.",0,1
297,local_data/spider/215_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Bulging in discs 1, 4. Low endplate in discs 4-5. Up endplate in discs 4-5.",0,1
298,local_data/spider/19_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-5 show Pfirrmann grade 1. Discs 1-3, 6 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1. Modic type 2 in discs 1-3. Up endplate in discs 4-6. Low endplate in discs 5-6.",0,1
299,local_data/spider/P3-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
300,local_data/spider/59_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 4. Narrowing in discs 1-6. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-6. Herniation in discs 1. Modic type 2 in discs 1-6.,0,1
301,local_data/spider/P91 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
302,local_data/spider/P67-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
303,local_data/spider/P51-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
304,local_data/spider/241_t2_SPACE.png,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 1 show Pfirrmann grade 2. Discs 9 show Pfirrmann grade 3. Discs 3-8 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 2-7. Bulging in discs 2-8. Low endplate in discs 2-9. Up endplate in discs 2-8. Herniation in discs 2. Modic type 2 in discs 2, 4.",0,1
305,local_data/spider/P8-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
306,local_data/spider/P84 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
307,local_data/spider/149_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1.,0,1
308,local_data/spider/P10-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
309,local_data/spider/P6-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
310,local_data/spider/P22-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
311,local_data/spider/94_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 1. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 1-3. Narrowing in discs 2-3.,0,1
312,local_data/spider/101_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 4. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6.,0,1
313,local_data/spider/P67-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
314,local_data/spider/P16-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
315,local_data/spider/P24-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
316,local_data/spider/242_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 4 show Pfirrmann grade 1. Discs 2, 6 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 4. Discs 3 show Pfirrmann grade 5. Bulging in discs 1-5. Narrowing in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5. Modic type 2 in discs 3.",0,1
317,local_data/spider/207_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1, 4. Bulging in discs 1-4. Modic type 2 in discs 1-4.",0,1
318,local_data/spider/P55-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
319,local_data/spider/8_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 2. Discs 4-7 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Modic type 2 in discs 1, 4-5.",0,1
320,local_data/spider/P94 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
321,local_data/spider/P66-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
322,local_data/spider/P53-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
323,local_data/spider/90_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 5-6 show Pfirrmann grade 3. Discs 2-4 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-4. Low endplate in discs 2, 4-5. Up endplate in discs 4-5. Modic type 2 in discs 4.",0,1
324,local_data/spider/P17-3.jpeg,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
325,local_data/spider/P78 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
326,local_data/spider/18_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 2. Discs 3-4, 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5, 7-8. Bulging in discs 1-5, 7-8. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Modic type 2 in discs 1-2, 5.",0,1
327,local_data/spider/140_t2.png,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 6-9 show Pfirrmann grade 1. Discs 1-3 show Pfirrmann grade 3. Discs 4-5 show Pfirrmann grade 4. Modic type 2 in discs 1, 3, 7. Narrowing in discs 3-5. Herniation in discs 3. Bulging in discs 4-5. Modic type 3 in discs 5. Up endplate in discs 8.",0,1
328,local_data/spider/122_t2.png,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1, 3-6 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Bulging in discs 1-2. Modic type 2 in discs 2.",0,1
329,local_data/spider/P52-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
330,local_data/spider/P48-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
331,local_data/spider/P99 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
332,local_data/spider/245_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4, 6 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 5 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3, 5. Modic type 2 in discs 1, 3. Bulging in discs 5-6.",0,1
333,local_data/spider/P40-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
334,local_data/spider/239_t2_SPACE.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Herniation in discs 1. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
335,local_data/spider/7_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-3. Low endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
336,local_data/spider/P68-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
337,local_data/spider/142_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 3. Discs 1-5 show Pfirrmann grade 4. Up endplate in discs 2, 4-5. Herniation in discs 2. Spondylolisthesis in discs 2. Bulging in discs 3-5. Narrowing in discs 4. Low endplate in discs 4-5, 7.",0,1
338,local_data/spider/120_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Herniation in discs 1. Bulging in discs 2.,0,1
339,local_data/spider/2_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3 show Pfirrmann grade 3. Discs 1-2, 4-6 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 2-3. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2-5.",0,1
340,local_data/spider/106_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 4. Narrowing in discs 1-7. Bulging in discs 1-2, 5. Modic type 2 in discs 1-7. Low endplate in discs 2-5, 7. Up endplate in discs 3, 7. Herniation in discs 3-4.",0,1
341,local_data/spider/200_t2_SPACE.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1.,0,1
342,local_data/spider/116_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 2. Discs 6 show Pfirrmann grade 3. Discs 2, 5 show Pfirrmann grade 4. Discs 1, 3-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
343,local_data/spider/202_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 1. Discs 3-4 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2-3, 7. Bulging in discs 2-4. Low endplate in discs 2, 4-7. Up endplate in discs 2, 4-7.",0,1
344,local_data/spider/87_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 5 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Bulging in discs 1-5. Narrowing in discs 3. Low endplate in discs 5. Up endplate in discs 5. Modic type 2 in discs 5.,0,1
345,local_data/spider/P94 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
346,local_data/spider/138_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3, 6-7 show Pfirrmann grade 1. Discs 1, 4-5 show Pfirrmann grade 3. Bulging in discs 1, 5. Narrowing in discs 5.",0,1
347,local_data/spider/169_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Discs 3-6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3-4. Bulging in discs 1-6. Modic type 2 in discs 1-7. Low endplate in discs 2-7. Up endplate in discs 2-7.",0,1
348,local_data/spider/P77 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
349,local_data/spider/195_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-2. Modic type 2 in discs 1-2.,0,1
350,local_data/spider/P58-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
351,local_data/spider/202_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 1. Discs 3-4 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2-3, 7. Bulging in discs 2-4. Low endplate in discs 2, 4-7. Up endplate in discs 2, 4-7.",0,1
352,local_data/spider/24_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6. Herniation in discs 2.",0,1
353,local_data/spider/P29-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
354,local_data/spider/P69-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
355,local_data/spider/P43-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
356,local_data/spider/P67-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
357,local_data/spider/P74 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
358,local_data/spider/7_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-3. Low endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
359,local_data/spider/146_t2_SPACE.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Herniation in discs 2.,0,1
360,local_data/spider/P50-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
361,local_data/spider/P94 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
362,local_data/spider/P10-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
363,local_data/spider/189_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Bulging in discs 2-4. Low endplate in discs 2-4. Up endplate in discs 2-4. Modic type 2 in discs 2-5. Narrowing in discs 3-4.",0,1
364,local_data/spider/80_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 3. Herniation in discs 2. Low endplate in discs 4-5. Up endplate in discs 4-5.",0,1
365,local_data/spider/P31-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
366,local_data/spider/P62-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
367,local_data/spider/P21-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
368,local_data/spider/4_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-2. Low endplate in discs 1-3. Up endplate in discs 1-3. Modic type 2 in discs 1-3. Herniation in discs 3. Spondylolisthesis in discs 3.,0,1
369,local_data/spider/147_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 3-4, 6 show Pfirrmann grade 2. Discs 2, 7-8 show Pfirrmann grade 3. Discs 5 show Pfirrmann grade 4. Narrowing in discs 1-8. Bulging in discs 1. Herniation in discs 2. Low endplate in discs 3-7. Up endplate in discs 3-7.",0,1
370,local_data/spider/P15-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
371,local_data/spider/P41-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
372,local_data/spider/22_t2.png,A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
373,local_data/spider/181_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2-4 show Pfirrmann grade 4. Discs 1, 5-7 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.",0,1
374,local_data/spider/237_t2.png,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
375,local_data/spider/P79 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
376,local_data/spider/P69-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
377,local_data/spider/P64-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
378,local_data/spider/12_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 3-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 4. Bulging in discs 1. Narrowing in discs 2. Herniation in discs 2.",0,1
379,local_data/spider/173_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-3, 5-6 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Narrowing in discs 1. Low endplate in discs 1-2, 4-5. Up endplate in discs 1-2, 4-5. Herniation in discs 1. Modic type 2 in discs 1, 4-5. Bulging in discs 2, 4.",0,1
380,local_data/spider/P96 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
381,local_data/spider/P69-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
382,local_data/spider/P90 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
383,local_data/spider/184_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 1-2. Up endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
384,local_data/spider/P98 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
385,local_data/spider/P7-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
386,local_data/spider/13_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Discs 5-7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5-7. Bulging in discs 1-5. Low endplate in discs 1, 5-8. Up endplate in discs 1, 5-7. Modic type 2 in discs 2-7. Spondylolisthesis in discs 5.",0,1
387,local_data/spider/144_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 1. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Herniation in discs 1. Low endplate in discs 2, 4, 6. Up endplate in discs 3-4, 6. Narrowing in discs 4. Bulging in discs 4.",0,1
388,local_data/spider/161_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 8 discs. Discs 1, 3-7 show Pfirrmann grade 4. Discs 2, 8 show Pfirrmann grade 5. Narrowing in discs 1-3, 8. Bulging in discs 1-5, 7-8. Low endplate in discs 1-8. Up endplate in discs 1-8. Modic type 2 in discs 1-8.",0,1
389,local_data/spider/65_t1.png,"A T1-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1, 3-6, 8-9 show Pfirrmann grade 4. Discs 2, 7 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7, 9. Low endplate in discs 2-9. Up endplate in discs 2-9. Modic type 2 in discs 2-9.",0,1
390,local_data/spider/85_t2.png,A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 7-8 show Pfirrmann grade 4. Discs 1-6 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-8. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.,0,1
391,local_data/spider/82_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 4. Discs 1-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-7.,0,1
392,local_data/spider/118_t2_SPACE.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
393,local_data/spider/36_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 1, 3, 5-6 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-4. Narrowing in discs 2, 4-6. Spondylolisthesis in discs 4. Modic type 2 in discs 4-5.",0,1
394,local_data/spider/188_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 7 show Pfirrmann grade 2. Discs 4-6 show Pfirrmann grade 3. Low endplate in discs 2. Up endplate in discs 4, 6. No disc herniation, disc narrowing or disc bulging is observed.",0,1
395,local_data/spider/110_t2_SPACE.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-4, 6-7 show Pfirrmann grade 3. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1, 5. Bulging in discs 1, 3, 5. Low endplate in discs 1, 3, 5, 7. Up endplate in discs 1, 3, 5, 7. Modic type 2 in discs 1-2, 5.",0,1
396,local_data/spider/225_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-4 show Pfirrmann grade 1. Discs 6 show Pfirrmann grade 2. Discs 2, 5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-3. Low endplate in discs 1-2. Up endplate in discs 1, 5. Modic type 2 in discs 1.",0,1
397,local_data/spider/P82 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
398,local_data/spider/P13-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
399,local_data/spider/P83 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
400,local_data/spider/117_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 2-5. Spondylolisthesis in discs 2. Low endplate in discs 3-6. Up endplate in discs 3-6. Modic type 2 in discs 3-6.,0,1
401,local_data/spider/124_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Bulging in discs 1-2. Narrowing in discs 2. Herniation in discs 2. Up endplate in discs 3-6. Low endplate in discs 6.,0,1
402,local_data/spider/4_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-2. Low endplate in discs 1-3. Up endplate in discs 1-3. Modic type 2 in discs 1-3. Herniation in discs 3. Spondylolisthesis in discs 3.,0,1
403,local_data/spider/P20-4.jpeg,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
404,local_data/spider/P73 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
405,local_data/spider/P70 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
406,local_data/spider/146_t1.png,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Herniation in discs 2.,0,1
407,local_data/spider/P22-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
408,local_data/spider/166_t2_SPACE.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2, 4-6, 8 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 7 show Pfirrmann grade 5. Narrowing in discs 1, 3, 7. Bulging in discs 1-4. Modic type 2 in discs 2-4. Low endplate in discs 6-7. Up endplate in discs 6-7.",0,1
409,local_data/spider/P30-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
410,local_data/spider/188_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 7 show Pfirrmann grade 2. Discs 4-6 show Pfirrmann grade 3. Low endplate in discs 2. Up endplate in discs 4, 6. No disc herniation, disc narrowing or disc bulging is observed.",0,1
411,local_data/spider/107_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 0 show Pfirrmann grade 1. Discs 6 show Pfirrmann grade 2. Discs 1, 4-5 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2. Bulging in discs 3-5.",0,1
412,local_data/spider/P34-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
413,local_data/spider/P90 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
414,local_data/spider/P27-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
415,local_data/spider/P79 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
416,local_data/spider/P92 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
417,local_data/spider/P63-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
418,local_data/spider/224_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6 show Pfirrmann grade 2. Discs 3-5, 7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-3. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-2.",0,1
419,local_data/spider/98_t2_SPACE.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 8 discs. Discs 3-7 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1-2, 4-5, 7. Up endplate in discs 1-2, 4. Herniation in discs 1-2. Modic type 2 in discs 1-2.",0,1
420,local_data/spider/45_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Narrowing in discs 1. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Bulging in discs 2, 4.",0,1
421,local_data/spider/132_t1.png,A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 1. Low endplate in discs 1. Narrowing in discs 2-4. Bulging in discs 2-4.,0,1
422,local_data/spider/P4-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
423,local_data/spider/P98 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
424,local_data/spider/P72 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
425,local_data/spider/P93 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
426,local_data/spider/81_t2_SPACE.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2.,0,1
427,local_data/spider/88_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 2, 4-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Modic type 2 in discs 1-5. Low endplate in discs 2, 4-5. Up endplate in discs 2, 4-5.",0,1
428,local_data/spider/237_t1.png,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
429,local_data/spider/113_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 3-5 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2.,0,1
430,local_data/spider/P88 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
431,local_data/spider/253_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 2. Discs 2-7 show Pfirrmann grade 4. Narrowing in discs 2-7. Bulging in discs 2-6. Spondylolisthesis in discs 3. Low endplate in discs 4-5. Up endplate in discs 4-5.,0,1
432,local_data/spider/151_t2.png,A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
433,local_data/spider/P71 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
434,local_data/spider/P32-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
435,local_data/spider/219_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 1. Bulging in discs 1-4. Low endplate in discs 3-5, 7. Up endplate in discs 3-5, 7. Narrowing in discs 7.",0,1
436,local_data/spider/159_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 4-5 show Pfirrmann grade 2. Discs 3, 6 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 5. Modic type 2 in discs 1-2. Narrowing in discs 2-3. Bulging in discs 2-3. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2.",0,1
437,local_data/spider/62_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 3-6 show Pfirrmann grade 2. Discs 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2, 7. Bulging in discs 1, 3-5. Low endplate in discs 1-2, 7. Up endplate in discs 1-2, 8. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2. Herniation in discs 2.",0,1
438,local_data/spider/P2-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
439,local_data/spider/P61-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
440,local_data/spider/31_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-3, 5-8 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Narrowing in discs 2, 4. Bulging in discs 2-6. Low endplate in discs 2, 4-7. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2-4. Up endplate in discs 3-7.",0,1
441,local_data/spider/P45-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
442,local_data/spider/P80 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
443,local_data/spider/P73 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
444,local_data/spider/220_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 6-8 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 2-8. Up endplate in discs 2-8. Narrowing in discs 3-5. Modic type 2 in discs 3-6.",0,1
445,local_data/spider/251_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 3. Discs 1, 4-6 show Pfirrmann grade 4. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 2-7. Herniation in discs 2-3, 6. Modic type 2 in discs 2-3. Up endplate in discs 7.",0,1
446,local_data/spider/163_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2, 7 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-3, 7. Low endplate in discs 1-3, 5-7. Modic type 2 in discs 1-6. Spondylolisthesis in discs 2. Up endplate in discs 3-4, 6-7.",0,1
447,local_data/spider/P84 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
448,local_data/spider/P83 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
449,local_data/spider/P28-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
450,local_data/spider/48_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5 show Pfirrmann grade 2. Discs 1-4, 6-7 show Pfirrmann grade 3. Bulging in discs 1-3. Low endplate in discs 2-4, 6-7. Up endplate in discs 3-6.",0,1
451,local_data/spider/P79 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
452,local_data/spider/127_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5, 7-8 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Narrowing in discs 1, 3-4. Bulging in discs 1-4, 6. Low endplate in discs 3-4. Up endplate in discs 3-4. Modic type 2 in discs 3-4.",0,1
453,local_data/spider/P14-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
454,local_data/spider/P45-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
455,local_data/spider/P9-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
456,local_data/spider/P25-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
457,local_data/spider/57_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4, 6-7 show Pfirrmann grade 3. Discs 2-3, 5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5. Bulging in discs 1-5. Low endplate in discs 1, 3, 5. Modic type 2 in discs 1, 3, 5. Herniation in discs 2. Up endplate in discs 5.",0,1
458,local_data/spider/177_t1.png,"A T1-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 7-9 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-5, 7. Modic type 2 in discs 1-5. Up endplate in discs 2-7. Herniation in discs 7.",0,1
459,local_data/spider/P8-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
460,local_data/spider/P26-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
461,local_data/spider/P81 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
462,local_data/spider/186_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Herniation in discs 1. Modic type 2 in discs 1-7.,0,1
463,local_data/spider/P5-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
464,local_data/spider/P47-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
465,local_data/spider/P71 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
466,local_data/spider/P91 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
467,local_data/spider/162_t2_SPACE.png,A T2-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 1-9 show Pfirrmann grade 5. Narrowing in discs 1-9. Bulging in discs 1-9. Low endplate in discs 1-9. Up endplate in discs 1-9. Spondylolisthesis in discs 1. Modic type 2 in discs 1-9.,0,1
468,local_data/spider/P49-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
469,local_data/spider/125_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4, 6-7 show Pfirrmann grade 2. Discs 1-3, 5 show Pfirrmann grade 3. Narrowing in discs 2. Up endplate in discs 3, 5. Bulging in discs 5. Modic type 2 in discs 5.",0,1
470,local_data/spider/129_t2_SPACE.png,A T2-weighted MRI of a female patient with 7 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 4.,0,1
471,local_data/spider/P15-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
472,local_data/spider/256_t2.png,A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Narrowing in discs 1-3. Bulging in discs 1-4. Herniation in discs 2. Modic type 2 in discs 2. Low endplate in discs 3. Up endplate in discs 3.,0,1
473,local_data/spider/P63-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
474,local_data/spider/P64-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
475,local_data/spider/138_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3, 6-7 show Pfirrmann grade 1. Discs 1, 4-5 show Pfirrmann grade 3. Bulging in discs 1, 5. Narrowing in discs 5.",0,1
476,local_data/spider/P76 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
477,local_data/spider/P86 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
478,local_data/spider/160_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 6 show Pfirrmann grade 1. Discs 3, 5 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-4. Low endplate in discs 2, 4. Modic type 2 in discs 2-4. Up endplate in discs 5-6.",0,1
479,local_data/spider/P72 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
480,local_data/spider/109_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Low endplate in discs 2, 5-7. Up endplate in discs 2, 5-6. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 3 in discs 2.",0,1
481,local_data/spider/P12-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
482,local_data/spider/44_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 2, 4-5 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-5. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Herniation in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 5.",0,1
483,local_data/spider/154_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 3. Discs 2-4 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1, 3-4. Narrowing in discs 2-4. Low endplate in discs 2-4, 7. Up endplate in discs 2-4, 7. Modic type 1 in discs 2. Herniation in discs 3.",0,1
484,local_data/spider/P34-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
485,local_data/spider/133_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 2. Discs 2-4, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-4, 6. Spondylolisthesis in discs 1. Modic type 2 in discs 1-5.",0,1
486,local_data/spider/193_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 3.,0,1
487,local_data/spider/P96 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
488,local_data/spider/58_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-4, 6 show Pfirrmann grade 3. Discs 1, 5, 7 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5, 7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7.",0,1
489,local_data/spider/69_t2_SPACE.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 3-6 show Pfirrmann grade 3. Discs 2, 7 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1-7. Narrowing in discs 2. Low endplate in discs 2, 7. Up endplate in discs 2, 7.",0,1
490,local_data/spider/146_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Herniation in discs 2.,0,1
491,local_data/spider/190_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 2. No disc herniation, disc narrowing or disc bulging is observed.",1,0
492,local_data/spider/137_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
493,local_data/spider/P66-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
494,local_data/spider/P99 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
495,local_data/spider/107_t2_SPACE.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 0 show Pfirrmann grade 1. Discs 6 show Pfirrmann grade 2. Discs 1, 4-5 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2. Bulging in discs 3-5.",0,1
496,local_data/spider/143_t1.png,"A T1-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 6-7, 9 show Pfirrmann grade 3. Discs 5, 8 show Pfirrmann grade 4. Discs 1-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
497,local_data/spider/P92 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
498,local_data/spider/P2-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
499,local_data/spider/228_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 6.,0,1
500,local_data/spider/257_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 2. Discs 6 show Pfirrmann grade 3. Discs 1-4 show Pfirrmann grade 4. Bulging in discs 1-4. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6. Herniation in discs 2.,0,1
501,local_data/spider/29_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-4, 6 show Pfirrmann grade 4. Discs 1-2, 5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-4, 6. Herniation in discs 1-2. Modic type 2 in discs 1-4. Spondylolisthesis in discs 2-4.",0,1
502,local_data/spider/P31-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
503,local_data/spider/213_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 5-6 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Narrowing in discs 2. Low endplate in discs 2-5. Up endplate in discs 2-5. Herniation in discs 2. Bulging in discs 3-4. Modic type 2 in discs 4-6.",0,1
504,local_data/spider/185_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 6-8 show Pfirrmann grade 2. Discs 4-5 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 2-3. Low endplate in discs 2-8. Up endplate in discs 2-8. Modic type 2 in discs 2-7.",0,1
505,local_data/spider/107_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 0 show Pfirrmann grade 1. Discs 6 show Pfirrmann grade 2. Discs 1, 4-5 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2. Bulging in discs 3-5.",0,1
506,local_data/spider/241_t1.png,"A T1-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 1 show Pfirrmann grade 2. Discs 9 show Pfirrmann grade 3. Discs 3-8 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 2-7. Bulging in discs 2-8. Low endplate in discs 2-9. Up endplate in discs 2-8. Herniation in discs 2. Modic type 2 in discs 2, 4.",0,1
507,local_data/spider/P39-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
508,local_data/spider/232_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Bulging in discs 1, 4.",0,1
509,local_data/spider/P24-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
510,local_data/spider/P99 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
511,local_data/spider/P83 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
512,local_data/spider/P74 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
513,local_data/spider/25_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 2, 6 show Pfirrmann grade 2. Discs 1, 4-5 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 5. Modic type 2 in discs 1-3. Narrowing in discs 2-5. Low endplate in discs 3-5. Up endplate in discs 3-6. Bulging in discs 5.",0,1
514,local_data/spider/P67-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
515,local_data/spider/P85 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
516,local_data/spider/99_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2-5, 7-8. Up endplate in discs 2-5, 7-8. Herniation in discs 2.",0,1
517,local_data/spider/152_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 0 show Pfirrmann grade 1. Discs 6-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 2-3, 8 show Pfirrmann grade 4. Discs 4-5 show Pfirrmann grade 5. Narrowing in discs 1-5, 8. Bulging in discs 2-8. Spondylolisthesis in discs 2. Low endplate in discs 4-5, 7-8. Up endplate in discs 4-5, 7-8. Modic type 2 in discs 4-5.",0,1
518,local_data/spider/51_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 7-8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Low endplate in discs 1, 3. Up endplate in discs 1, 3. Modic type 2 in discs 1.",0,1
519,local_data/spider/31_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-3, 5-8 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Narrowing in discs 2, 4. Bulging in discs 2-6. Low endplate in discs 2, 4-7. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2-4. Up endplate in discs 3-7.",0,1
520,local_data/spider/152_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 0 show Pfirrmann grade 1. Discs 6-7 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 2-3, 8 show Pfirrmann grade 4. Discs 4-5 show Pfirrmann grade 5. Narrowing in discs 1-5, 8. Bulging in discs 2-8. Spondylolisthesis in discs 2. Low endplate in discs 4-5, 7-8. Up endplate in discs 4-5, 7-8. Modic type 2 in discs 4-5.",0,1
521,local_data/spider/252_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 2. Herniation in discs 2.,0,1
522,local_data/spider/61_t2_SPACE.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 3. Discs 3-4, 7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-4. Up endplate in discs 1-2, 4-5, 7-8. Modic type 2 in discs 1-4. Low endplate in discs 2-5, 7-8.",0,1
523,local_data/spider/198_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 3, 5 show Pfirrmann grade 4. Discs 1-2, 4 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
524,local_data/spider/P78 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
525,local_data/spider/P40-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
526,local_data/spider/60_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3 show Pfirrmann grade 2. Discs 1, 4-6 show Pfirrmann grade 3. Discs 7 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Bulging in discs 1, 3-5. Narrowing in discs 2, 4-7. Low endplate in discs 2, 4-5. Up endplate in discs 2, 4-5. Modic type 2 in discs 2, 4. Modic type 3 in discs 5.",0,1
527,local_data/spider/78_t2_SPACE.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 3 in discs 1.,0,1
528,local_data/spider/P27-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
529,local_data/spider/63_t1.png,"A T1-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 8-9 show Pfirrmann grade 1. Discs 7 show Pfirrmann grade 2. Discs 1, 3-6 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-5. Modic type 2 in discs 1-7. Low endplate in discs 2-3. Up endplate in discs 2-7. Spondylolisthesis in discs 5.",0,1
530,local_data/spider/P66-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
531,local_data/spider/P14-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
532,local_data/spider/112_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1.,0,1
533,local_data/spider/145_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1, 3-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1, 3-5. Up endplate in discs 1, 3-5. Spondylolisthesis in discs 1. Modic type 2 in discs 3-4.",0,1
534,local_data/spider/P42-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
535,local_data/spider/234_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 1-2, 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 5. Bulging in discs 1-4. Modic type 2 in discs 1-4, 6-9. Narrowing in discs 3. Low endplate in discs 3, 6. Up endplate in discs 3, 6. Spondylolisthesis in discs 3.",0,1
536,local_data/spider/P45-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
537,local_data/spider/P24-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
538,local_data/spider/9_t1.png,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 3. Discs 2-5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-7.,0,1
539,local_data/spider/56_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4, 7 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 2. Narrowing in discs 3. Low endplate in discs 3. Up endplate in discs 3. Herniation in discs 3. Modic type 2 in discs 3.",0,1
540,local_data/spider/P75 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
541,local_data/spider/P86 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
542,local_data/spider/205_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 2-3, 5 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Bulging in discs 1. Modic type 2 in discs 1-2.",0,1
543,local_data/spider/P77 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
544,local_data/spider/P73 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
545,local_data/spider/5_t2_SPACE.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Bulging in discs 1-3.,0,1
546,local_data/spider/141_t2_SPACE.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
547,local_data/spider/P61-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
548,local_data/spider/245_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4, 6 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 5 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3, 5. Modic type 2 in discs 1, 3. Bulging in discs 5-6.",0,1
549,local_data/spider/P53-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
550,local_data/spider/P6-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
551,local_data/spider/P45-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
552,local_data/spider/P45-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
553,local_data/spider/28_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5, 7 show Pfirrmann grade 1. Discs 2-4, 6 show Pfirrmann grade 2. Low endplate in discs 1-2, 7. Up endplate in discs 1, 5-7. Narrowing in discs 2-4. Bulging in discs 2-4, 6-7.",0,1
554,local_data/spider/231_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-5, 7 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.",0,1
555,local_data/spider/P27-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
556,local_data/spider/98_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1-2, 4-5, 7. Up endplate in discs 1-2, 4. Herniation in discs 1-2. Modic type 2 in discs 1-2.",0,1
557,local_data/spider/205_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 2-3, 5 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Bulging in discs 1. Modic type 2 in discs 1-2.",0,1
558,local_data/spider/P42-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
559,local_data/spider/P26-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
560,local_data/spider/P5-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
561,local_data/spider/191_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 3. Discs 1-5, 7-8 show Pfirrmann grade 4. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 2-6, 8. Modic type 2 in discs 2-3, 5. Up endplate in discs 3-8.",0,1
562,local_data/spider/P54-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
563,local_data/spider/P51-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
564,local_data/spider/P47-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
565,local_data/spider/P85 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
566,local_data/spider/P77 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
567,local_data/spider/100_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 2-4, 8 show Pfirrmann grade 3. Modic type 2 in discs 1-6. Narrowing in discs 3. Bulging in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5.",0,1
568,local_data/spider/16_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-2, 5-7 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-2. Low endplate in discs 1. Spondylolisthesis in discs 1.",0,1
569,local_data/spider/P33-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
570,local_data/spider/P49-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
571,local_data/spider/166_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2, 4-6, 8 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 7 show Pfirrmann grade 5. Narrowing in discs 1, 3, 7. Bulging in discs 1-4. Modic type 2 in discs 2-4. Low endplate in discs 6-7. Up endplate in discs 6-7.",0,1
572,local_data/spider/P97 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
573,local_data/spider/P9-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
574,local_data/spider/P58-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
575,local_data/spider/P16-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
576,local_data/spider/204_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 5 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 2-3.",0,1
577,local_data/spider/P19-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
578,local_data/spider/66_t2.png,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1, 3-9 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Bulging in discs 1-4. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2-3.",0,1
579,local_data/spider/P16-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
580,local_data/spider/P78 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
581,local_data/spider/17_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-3. Modic type 3 in discs 2.",0,1
582,local_data/spider/45_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Narrowing in discs 1. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Bulging in discs 2, 4.",0,1
583,local_data/spider/P84 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
584,local_data/spider/16_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-2, 5-7 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-2. Low endplate in discs 1. Spondylolisthesis in discs 1.",0,1
585,local_data/spider/P23-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
586,local_data/spider/P80 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
587,local_data/spider/133_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 2. Discs 2-4, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-4, 6. Spondylolisthesis in discs 1. Modic type 2 in discs 1-5.",0,1
588,local_data/spider/P73 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
589,local_data/spider/P7-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
590,local_data/spider/P93 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
591,local_data/spider/233_t1.png,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1. Herniation in discs 1. Modic type 2 in discs 1.,0,1
592,local_data/spider/190_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-8 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 2. No disc herniation, disc narrowing or disc bulging is observed.",1,0
593,local_data/spider/P55-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
594,local_data/spider/P93 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
595,local_data/spider/129_t1.png,A T1-weighted MRI of a female patient with 7 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 4.,0,1
596,local_data/spider/P11-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
597,local_data/spider/P37-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
598,local_data/spider/P6-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
599,local_data/spider/P86 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
600,local_data/spider/P39-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
601,local_data/spider/P10-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
602,local_data/spider/59_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 4. Narrowing in discs 1-6. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-6. Herniation in discs 1. Modic type 2 in discs 1-6.,0,1
603,local_data/spider/40_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 1. Modic type 2 in discs 5. Low endplate in discs 6-7. Up endplate in discs 6-7. No disc herniation, disc narrowing or disc bulging is observed.",0,1
604,local_data/spider/229_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Narrowing in discs 2-4. Bulging in discs 2-5. Low endplate in discs 2-6. Up endplate in discs 2-6. Modic type 2 in discs 2-4.,0,1
605,local_data/spider/P62-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
606,local_data/spider/P87 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
607,local_data/spider/P87 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
608,local_data/spider/156_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 3-5 show Pfirrmann grade 5. Narrowing in discs 1, 3-5. Bulging in discs 1, 3-5. Low endplate in discs 1, 3-5. Up endplate in discs 1, 3-5. Spondylolisthesis in discs 1. Modic type 2 in discs 1-5. Herniation in discs 5.",0,1
609,local_data/spider/P55-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
610,local_data/spider/236_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 1, 5 show Pfirrmann grade 4. Discs 2-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2-5.",0,1
611,local_data/spider/182_t1.png,"A T1-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1-3, 6, 9 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 1-9. Up endplate in discs 1-9. Modic type 2 in discs 1-9. Narrowing in discs 4-5, 8.",0,1
612,local_data/spider/P51-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
613,local_data/spider/P97 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
614,local_data/spider/234_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 1-2, 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 5. Bulging in discs 1-4. Modic type 2 in discs 1-4, 6-9. Narrowing in discs 3. Low endplate in discs 3, 6. Up endplate in discs 3, 6. Spondylolisthesis in discs 3.",0,1
615,local_data/spider/P17-2.jpeg,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
616,local_data/spider/P15-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
617,local_data/spider/62_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 3-6 show Pfirrmann grade 2. Discs 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2, 7. Bulging in discs 1, 3-5. Low endplate in discs 1-2, 7. Up endplate in discs 1-2, 8. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2. Herniation in discs 2.",0,1
618,local_data/spider/78_t1.png,A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 3 in discs 1.,0,1
619,local_data/spider/P32-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
620,local_data/spider/30_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 2. Discs 1, 3-5 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Bulging in discs 2-5. Low endplate in discs 3-7. Up endplate in discs 3-7.",0,1
621,local_data/spider/50_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 3. Discs 1-7 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-7. Low endplate in discs 2-7. Up endplate in discs 2-7. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 2 in discs 2, 5-6. Modic type 1 in discs 4.",0,1
622,local_data/spider/226_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2, 5-7 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1, 4 show Pfirrmann grade 5. Narrowing in discs 1, 3-4, 6-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-3, 5, 7. Modic type 3 in discs 4, 6.",0,1
623,local_data/spider/P59-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
624,local_data/spider/P88 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
625,local_data/spider/P53-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
626,local_data/spider/134_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 4-8 show Pfirrmann grade 1. Discs 1, 3 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-3. Low endplate in discs 1-7. Up endplate in discs 1-3.",0,1
627,local_data/spider/P31-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
628,local_data/spider/254_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4 show Pfirrmann grade 2. Discs 5-6 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5-6. Bulging in discs 1-2, 5-6. Herniation in discs 3.",0,1
629,local_data/spider/225_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-4 show Pfirrmann grade 1. Discs 6 show Pfirrmann grade 2. Discs 2, 5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-3. Low endplate in discs 1-2. Up endplate in discs 1, 5. Modic type 2 in discs 1.",0,1
630,local_data/spider/P24-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
631,local_data/spider/88_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 2, 4-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Modic type 2 in discs 1-5. Low endplate in discs 2, 4-5. Up endplate in discs 2, 4-5.",0,1
632,local_data/spider/P75 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
633,local_data/spider/P66-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
634,local_data/spider/52_t2_SPACE.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1, 4-5, 7 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1-5. Modic type 2 in discs 1-3.",0,1
635,local_data/spider/P31-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
636,local_data/spider/25_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 2, 6 show Pfirrmann grade 2. Discs 1, 4-5 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 5. Modic type 2 in discs 1-3. Narrowing in discs 2-5. Low endplate in discs 3-5. Up endplate in discs 3-6. Bulging in discs 5.",0,1
637,local_data/spider/P100 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
638,local_data/spider/20_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 2. Discs 3-7 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 5. Bulging in discs 1-6. Narrowing in discs 2-3, 6-7. Low endplate in discs 2-7. Up endplate in discs 2-7.",0,1
639,local_data/spider/P56-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
640,local_data/spider/P30-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
641,local_data/spider/207_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1, 4. Bulging in discs 1-4. Modic type 2 in discs 1-4.",0,1
642,local_data/spider/33_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4-5 show Pfirrmann grade 2. Discs 3, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Herniation in discs 1. Modic type 2 in discs 1, 4-5. Up endplate in discs 3, 5-6. Low endplate in discs 5-6.",0,1
643,local_data/spider/P38-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
644,local_data/spider/P99 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
645,local_data/spider/P68-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
646,local_data/spider/P56-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
647,local_data/spider/P11-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
648,local_data/spider/P52-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
649,local_data/spider/P7-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
650,local_data/spider/P66-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
651,local_data/spider/11_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 2-4 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 1, 6 show Pfirrmann grade 5. Narrowing in discs 1, 6-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2. Bulging in discs 3-6. Low endplate in discs 5-7. Up endplate in discs 5-7.",0,1
652,local_data/spider/113_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 3-5 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Modic type 2 in discs 2.,0,1
653,local_data/spider/23_t2.png,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 9 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Discs 5-8 show Pfirrmann grade 4. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-8. Low endplate in discs 2-6. Up endplate in discs 2-6. Herniation in discs 4.",0,1
654,local_data/spider/P54-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
655,local_data/spider/P50-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
656,local_data/spider/39_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 5. Up endplate in discs 5. Modic type 2 in discs 5.,0,1
657,local_data/spider/P76 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
658,local_data/spider/P50-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
659,local_data/spider/29_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-4, 6 show Pfirrmann grade 4. Discs 1-2, 5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-4, 6. Herniation in discs 1-2. Modic type 2 in discs 1-4. Spondylolisthesis in discs 2-4.",0,1
660,local_data/spider/106_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 4. Narrowing in discs 1-7. Bulging in discs 1-2, 5. Modic type 2 in discs 1-7. Low endplate in discs 2-5, 7. Up endplate in discs 3, 7. Herniation in discs 3-4.",0,1
661,local_data/spider/35_t2.png,A T2-weighted MRI of a female patient with 3 vertebrae and 3 discs. Discs 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-3. Low endplate in discs 3-8. Up endplate in discs 3-8.,0,1

Overwriting local_data/lbp-train-meta.csv


In [2]:
%%writefile local_data/lbp-valid-meta.csv
,imgpath,report,No Finding,LBP
0,local_data/spider/P3-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
1,local_data/spider/P54-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
2,local_data/spider/P29-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
3,local_data/spider/249_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6-7 show Pfirrmann grade 2. Discs 3-5 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 8 show Pfirrmann grade 5. Narrowing in discs 1-2, 8. Bulging in discs 1-5. Low endplate in discs 1-2, 8. Up endplate in discs 1-2, 7. Modic type 2 in discs 1-2, 6-8.",0,1
4,local_data/spider/P37-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
5,local_data/spider/P54-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
6,local_data/spider/98_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-7 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2. Low endplate in discs 1-2, 4-5, 7. Up endplate in discs 1-2, 4. Herniation in discs 1-2. Modic type 2 in discs 1-2.",0,1
7,local_data/spider/241_t2.png,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 1 show Pfirrmann grade 2. Discs 9 show Pfirrmann grade 3. Discs 3-8 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 2-7. Bulging in discs 2-8. Low endplate in discs 2-9. Up endplate in discs 2-8. Herniation in discs 2. Modic type 2 in discs 2, 4.",0,1
8,local_data/spider/P28-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
9,local_data/spider/6_t2.png,A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 2. Narrowing in discs 1. Bulging in discs 1-3.,0,1
10,local_data/spider/40_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 1. Modic type 2 in discs 5. Low endplate in discs 6-7. Up endplate in discs 6-7. No disc herniation, disc narrowing or disc bulging is observed.",0,1
11,local_data/spider/P17-4.jpeg,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
12,local_data/spider/P69-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
13,local_data/spider/253_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 2. Discs 2-7 show Pfirrmann grade 4. Narrowing in discs 2-7. Bulging in discs 2-6. Spondylolisthesis in discs 3. Low endplate in discs 4-5. Up endplate in discs 4-5.,0,1
14,local_data/spider/P5-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
15,local_data/spider/P95 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
16,local_data/spider/192_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Discs 1-3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-3. Up endplate in discs 1-3. Modic type 2 in discs 1-3, 5.",0,1
17,local_data/spider/90_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 5-6 show Pfirrmann grade 3. Discs 2-4 show Pfirrmann grade 4. Narrowing in discs 2, 4. Bulging in discs 2-4. Low endplate in discs 2, 4-5. Up endplate in discs 4-5. Modic type 2 in discs 4.",0,1
18,local_data/spider/110_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-4, 6-7 show Pfirrmann grade 3. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1, 5. Bulging in discs 1, 3, 5. Low endplate in discs 1, 3, 5, 7. Up endplate in discs 1, 3, 5, 7. Modic type 2 in discs 1-2, 5.",0,1
19,local_data/spider/P68-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
20,local_data/spider/P91 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
21,local_data/spider/45_t2_SPACE.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Narrowing in discs 1. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Bulging in discs 2, 4.",0,1
22,local_data/spider/P100 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
23,local_data/spider/165_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 1-5 show Pfirrmann grade 3. Bulging in discs 1-5. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2, 4. Narrowing in discs 4.",0,1
24,local_data/spider/P20-3.jpeg,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
25,local_data/spider/P82 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
26,local_data/spider/8_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 2. Discs 4-7 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Modic type 2 in discs 1, 4-5.",0,1
27,local_data/spider/7_t2_SPACE.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-3. Low endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
28,local_data/spider/P27-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
29,local_data/spider/249_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6-7 show Pfirrmann grade 2. Discs 3-5 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 8 show Pfirrmann grade 5. Narrowing in discs 1-2, 8. Bulging in discs 1-5. Low endplate in discs 1-2, 8. Up endplate in discs 1-2, 7. Modic type 2 in discs 1-2, 6-8.",0,1
30,local_data/spider/68_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 4. Discs 1-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5, 7-8. Modic type 2 in discs 1-3. Modic type 1 in discs 4-5.",0,1
31,local_data/spider/252_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 2. Herniation in discs 2.,0,1
32,local_data/spider/94_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4-7 show Pfirrmann grade 1. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 1-3. Narrowing in discs 2-3.,0,1
33,local_data/spider/78_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 3 in discs 1.,0,1
34,local_data/spider/63_t2.png,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 8-9 show Pfirrmann grade 1. Discs 7 show Pfirrmann grade 2. Discs 1, 3-6 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-5. Modic type 2 in discs 1-7. Low endplate in discs 2-3. Up endplate in discs 2-7. Spondylolisthesis in discs 5.",0,1
35,local_data/spider/P46-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
36,local_data/spider/P29-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
37,local_data/spider/187_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 2. Low endplate in discs 1, 7. Bulging in discs 7.",0,1
38,local_data/spider/243_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 3-7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2.",0,1
39,local_data/spider/P70 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
40,local_data/spider/P77 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
41,local_data/spider/P35-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
42,local_data/spider/P82 4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
43,local_data/spider/213_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1, 5-6 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Narrowing in discs 2. Low endplate in discs 2-5. Up endplate in discs 2-5. Herniation in discs 2. Bulging in discs 3-4. Modic type 2 in discs 4-6.",0,1
44,local_data/spider/9_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 3. Discs 2-5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-7.,0,1
45,local_data/spider/P2-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
46,local_data/spider/32_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 5-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2.",0,1
47,local_data/spider/P20-1.jpeg,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
48,local_data/spider/P87 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
49,local_data/spider/83_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Bulging in discs 1-6. Narrowing in discs 3-4. Low endplate in discs 3-6. Up endplate in discs 3-6. Modic type 2 in discs 3-6.,0,1
50,local_data/spider/P13-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
51,local_data/spider/175_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3, 5-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2. Bulging in discs 3-4.",0,1
52,local_data/spider/P62-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
53,local_data/spider/60_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3 show Pfirrmann grade 2. Discs 1, 4-6 show Pfirrmann grade 3. Discs 7 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Bulging in discs 1, 3-5. Narrowing in discs 2, 4-7. Low endplate in discs 2, 4-5. Up endplate in discs 2, 4-5. Modic type 2 in discs 2, 4. Modic type 3 in discs 5.",0,1
54,local_data/spider/P26-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
55,local_data/spider/174_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Herniation in discs 2.,0,1
56,local_data/spider/168_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Herniation in discs 2. Narrowing in discs 3-4. Modic type 2 in discs 3, 6. Up endplate in discs 5.",0,1
57,local_data/spider/P46-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
58,local_data/spider/P19-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
59,local_data/spider/69_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 3-6 show Pfirrmann grade 3. Discs 2, 7 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1-7. Narrowing in discs 2. Low endplate in discs 2, 7. Up endplate in discs 2, 7.",0,1
60,local_data/spider/189_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Bulging in discs 2-4. Low endplate in discs 2-4. Up endplate in discs 2-4. Modic type 2 in discs 2-5. Narrowing in discs 3-4.",0,1
61,local_data/spider/P7-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
62,local_data/spider/183_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-4. Herniation in discs 2. Low endplate in discs 3. Up endplate in discs 3.,0,1
63,local_data/spider/17_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-3. Modic type 3 in discs 2.",0,1
64,local_data/spider/P39-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
65,local_data/spider/75_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Herniation in discs 1. Bulging in discs 2-3. Low endplate in discs 2-3. Up endplate in discs 2-3. Modic type 2 in discs 2-4.,0,1
66,local_data/spider/P80 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
67,local_data/spider/162_t2.png,A T2-weighted MRI of a male patient with 8 vertebrae and 9 discs. Discs 1-9 show Pfirrmann grade 5. Narrowing in discs 1-9. Bulging in discs 1-9. Low endplate in discs 1-9. Up endplate in discs 1-9. Spondylolisthesis in discs 1. Modic type 2 in discs 1-9.,0,1
68,local_data/spider/P29-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
69,local_data/spider/P6-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
70,local_data/spider/87_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 1. Discs 5 show Pfirrmann grade 2. Discs 1-4 show Pfirrmann grade 3. Bulging in discs 1-5. Narrowing in discs 3. Low endplate in discs 5. Up endplate in discs 5. Modic type 2 in discs 5.,0,1
71,local_data/spider/47_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Up endplate in discs 1-3, 6-7. Bulging in discs 2. Low endplate in discs 2-4, 6. Narrowing in discs 3.",0,1
72,local_data/spider/P85 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
73,local_data/spider/209_t1.png,A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2-8 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Herniation in discs 1-2. Spondylolisthesis in discs 1-2. Modic type 2 in discs 1. Low endplate in discs 2-3. Up endplate in discs 2-3. Bulging in discs 3-4.,0,1
74,local_data/spider/203_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Bulging in discs 1-3. Up endplate in discs 5-6. Low endplate in discs 6.",0,1
75,local_data/spider/P71 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
76,local_data/spider/P17-5.jpeg,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
77,local_data/spider/168_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Herniation in discs 2. Narrowing in discs 3-4. Modic type 2 in discs 3, 6. Up endplate in discs 5.",0,1
78,local_data/spider/P63-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
79,local_data/spider/122_t1.png,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1, 3-6 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Bulging in discs 1-2. Modic type 2 in discs 2.",0,1
80,local_data/spider/172_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4, 6 show Pfirrmann grade 1. Discs 2, 5 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-2. Low endplate in discs 1-6. Up endplate in discs 1-6. Spondylolisthesis in discs 1. Modic type 2 in discs 1-6. Herniation in discs 3.",0,1
81,local_data/spider/11_t2.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 2-4 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 1, 6 show Pfirrmann grade 5. Narrowing in discs 1, 6-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2. Bulging in discs 3-6. Low endplate in discs 5-7. Up endplate in discs 5-7.",0,1
82,local_data/spider/218_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2. Low endplate in discs 2. Up endplate in discs 2. Spondylolisthesis in discs 2.,0,1
83,local_data/spider/34_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2, 4, 6 show Pfirrmann grade 3. Discs 3, 5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Modic type 2 in discs 1. Low endplate in discs 3.",0,1
84,local_data/spider/P14-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
85,local_data/spider/57_t2_SPACE.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 4, 6-7 show Pfirrmann grade 3. Discs 2-3, 5 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5. Bulging in discs 1-5. Low endplate in discs 1, 3, 5. Modic type 2 in discs 1, 3, 5. Herniation in discs 2. Up endplate in discs 5.",0,1
86,local_data/spider/P42-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
87,local_data/spider/P90 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
88,local_data/spider/174_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Herniation in discs 2.,0,1
89,local_data/spider/P86 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
90,local_data/spider/P44-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
91,local_data/spider/P56-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
92,local_data/spider/239_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Herniation in discs 1. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
93,local_data/spider/105_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-2, 5-6 show Pfirrmann grade 2. Discs 7 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-3, 5. Low endplate in discs 2-4. Up endplate in discs 2-4, 6-7. Modic type 2 in discs 2, 4. Narrowing in discs 3-4. Herniation in discs 4.",0,1
94,local_data/spider/P36-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
95,local_data/spider/56_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 4, 7 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 2. Discs 1-3 show Pfirrmann grade 3. Bulging in discs 2. Narrowing in discs 3. Low endplate in discs 3. Up endplate in discs 3. Herniation in discs 3. Modic type 2 in discs 3.",0,1
96,local_data/spider/P75 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
97,local_data/spider/P49-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
98,local_data/spider/P18-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
99,local_data/spider/P78 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
100,local_data/spider/P58-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
101,local_data/spider/P8-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
102,local_data/spider/P98 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
103,local_data/spider/P5-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
104,local_data/spider/P44-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
105,local_data/spider/P4-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
106,local_data/spider/P94 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
107,local_data/spider/172_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4, 6 show Pfirrmann grade 1. Discs 2, 5 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Narrowing in discs 1-3. Bulging in discs 1-2. Low endplate in discs 1-6. Up endplate in discs 1-6. Spondylolisthesis in discs 1. Modic type 2 in discs 1-6. Herniation in discs 3.",0,1
108,local_data/spider/P24-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
109,local_data/spider/P8-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
110,local_data/spider/55_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 8 discs. Discs 1, 6-7 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Bulging in discs 1-7. Narrowing in discs 2, 5, 7. Low endplate in discs 3-7. Up endplate in discs 3, 5-7. Modic type 2 in discs 3, 5. Spondylolisthesis in discs 6.",0,1
111,local_data/spider/227_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Bulging in discs 1-4, 6. Narrowing in discs 2-3. Modic type 2 in discs 2-4. Up endplate in discs 3, 5-6. Low endplate in discs 5.",0,1
112,local_data/spider/3_t1.png,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 2, 5-6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 3.",0,1
113,local_data/spider/80_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 3. Herniation in discs 2. Low endplate in discs 4-5. Up endplate in discs 4-5.",0,1
114,local_data/spider/P12-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
115,local_data/spider/52_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1, 4-5, 7 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1-5. Modic type 2 in discs 1-3.",0,1
116,local_data/spider/130_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 2. Discs 6-7 show Pfirrmann grade 3. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 2-5. Bulging in discs 2-5. Low endplate in discs 2-7. Up endplate in discs 2-7. Modic type 2 in discs 2-6. Spondylolisthesis in discs 5.,0,1
117,local_data/spider/109_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Low endplate in discs 2, 5-7. Up endplate in discs 2, 5-6. Herniation in discs 2. Spondylolisthesis in discs 2. Modic type 3 in discs 2.",0,1
118,local_data/spider/185_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1, 6-8 show Pfirrmann grade 2. Discs 4-5 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Bulging in discs 1-4. Narrowing in discs 2-3. Low endplate in discs 2-8. Up endplate in discs 2-8. Modic type 2 in discs 2-7.",0,1
119,local_data/spider/173_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-3, 5-6 show Pfirrmann grade 2. Discs 1, 4 show Pfirrmann grade 3. Narrowing in discs 1. Low endplate in discs 1-2, 4-5. Up endplate in discs 1-2, 4-5. Herniation in discs 1. Modic type 2 in discs 1, 4-5. Bulging in discs 2, 4.",0,1
120,local_data/spider/10_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1, 4, 7 show Pfirrmann grade 3. Discs 5 show Pfirrmann grade 4. Discs 2-3, 6 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 1, 4-6. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-7. Herniation in discs 2-3.",0,1
121,local_data/spider/P80 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
122,local_data/spider/193_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 3.,0,1
123,local_data/spider/P2-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
124,local_data/spider/P68-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
125,local_data/spider/P34-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
126,local_data/spider/P9-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
127,local_data/spider/95_t2.png,A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Low endplate in discs 1. Up endplate in discs 1. Herniation in discs 1. Modic type 2 in discs 1.,0,1
128,local_data/spider/155_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1, 4. Modic type 2 in discs 1-5.",0,1
129,local_data/spider/P57-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
130,local_data/spider/11_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 2-4 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 1, 6 show Pfirrmann grade 5. Narrowing in discs 1, 6-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2. Bulging in discs 3-6. Low endplate in discs 5-7. Up endplate in discs 5-7.",0,1
131,local_data/spider/196_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-2 show Pfirrmann grade 1. Discs 3-6 show Pfirrmann grade 2. Low endplate in discs 2-5. Up endplate in discs 2-6. Modic type 2 in discs 2-5. Spondylolisthesis in discs 6. No disc herniation, disc narrowing or disc bulging is observed.",0,1
132,local_data/spider/169_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 2. Discs 3-6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3-4. Bulging in discs 1-6. Modic type 2 in discs 1-7. Low endplate in discs 2-7. Up endplate in discs 2-7.",0,1
133,local_data/spider/232_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 1. Discs 2-4 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Bulging in discs 1, 4.",0,1
134,local_data/spider/P65-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
135,local_data/spider/P92 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
136,local_data/spider/P72 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
137,local_data/spider/P7-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
138,local_data/spider/P48-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
139,local_data/spider/P59-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
140,local_data/spider/P9-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
141,local_data/spider/P25-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0

Overwriting local_data/lbp-valid-meta.csv


In [5]:
%%writefile local_data/lbp-test-meta.csv
,imgpath,report,No Finding,LBP
0,local_data/spider/209_t2_SPACE.png,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2-8 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Herniation in discs 1-2. Spondylolisthesis in discs 1-2. Modic type 2 in discs 1. Low endplate in discs 2-3. Up endplate in discs 2-3. Bulging in discs 3-4.,0,1
1,local_data/spider/77_t1.png,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
2,local_data/spider/P79 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
3,local_data/spider/P19-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
4,local_data/spider/P16-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
5,local_data/spider/P89 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
6,local_data/spider/P75 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
7,local_data/spider/167_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 6-7 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5. Modic type 2 in discs 3.",0,1
8,local_data/spider/P57-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
9,local_data/spider/P3-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
10,local_data/spider/P96 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
11,local_data/spider/18_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 2. Discs 3-4, 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5, 7-8. Bulging in discs 1-5, 7-8. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Modic type 2 in discs 1-2, 5.",0,1
12,local_data/spider/P23-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
13,local_data/spider/P97 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
14,local_data/spider/P74 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
15,local_data/spider/144_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 1. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Herniation in discs 1. Low endplate in discs 2, 4, 6. Up endplate in discs 3-4, 6. Narrowing in discs 4. Bulging in discs 4.",0,1
16,local_data/spider/254_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4 show Pfirrmann grade 2. Discs 5-6 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5-6. Bulging in discs 1-2, 5-6. Herniation in discs 3.",0,1
17,local_data/spider/67_t2.png,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Bulging in discs 1, 3. Herniation in discs 2.",0,1
18,local_data/spider/P22-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
19,local_data/spider/154_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 3. Discs 2-4 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1, 3-4. Narrowing in discs 2-4. Low endplate in discs 2-4, 7. Up endplate in discs 2-4, 7. Modic type 1 in discs 2. Herniation in discs 3.",0,1
20,local_data/spider/48_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5 show Pfirrmann grade 2. Discs 1-4, 6-7 show Pfirrmann grade 3. Bulging in discs 1-3. Low endplate in discs 2-4, 6-7. Up endplate in discs 3-6.",0,1
21,local_data/spider/187_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 2. Low endplate in discs 1, 7. Bulging in discs 7.",0,1
22,local_data/spider/116_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 2. Discs 6 show Pfirrmann grade 3. Discs 2, 5 show Pfirrmann grade 4. Discs 1, 3-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
23,local_data/spider/P3-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
24,local_data/spider/P48-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
25,local_data/spider/P23-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
26,local_data/spider/121_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 3-4 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Modic type 2 in discs 1-3. Up endplate in discs 2.",0,1
27,local_data/spider/39_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 5. Up endplate in discs 5. Modic type 2 in discs 5.,0,1
28,local_data/spider/P44-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
29,local_data/spider/P91 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
30,local_data/spider/P59-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
31,local_data/spider/P38-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
32,local_data/spider/145_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1, 3-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1, 3-5. Up endplate in discs 1, 3-5. Spondylolisthesis in discs 1. Modic type 2 in discs 3-4.",0,1
33,local_data/spider/105_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-2, 5-6 show Pfirrmann grade 2. Discs 7 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-3, 5. Low endplate in discs 2-4. Up endplate in discs 2-4, 6-7. Modic type 2 in discs 2, 4. Narrowing in discs 3-4. Herniation in discs 4.",0,1
34,local_data/spider/19_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-5 show Pfirrmann grade 1. Discs 1-3, 6 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1. Modic type 2 in discs 1-3. Up endplate in discs 4-6. Low endplate in discs 5-6.",0,1
35,local_data/spider/P44-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
36,local_data/spider/236_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 1, 5 show Pfirrmann grade 4. Discs 2-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2-5.",0,1
37,local_data/spider/P50-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
38,local_data/spider/P18-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
39,local_data/spider/P89 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
40,local_data/spider/161_t2_SPACE.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 8 discs. Discs 1, 3-7 show Pfirrmann grade 4. Discs 2, 8 show Pfirrmann grade 5. Narrowing in discs 1-3, 8. Bulging in discs 1-5, 7-8. Low endplate in discs 1-8. Up endplate in discs 1-8. Modic type 2 in discs 1-8.",0,1
41,local_data/spider/P10-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
42,local_data/spider/71_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1 show Pfirrmann grade 3. Discs 3, 5, 7-8 show Pfirrmann grade 4. Discs 2, 4, 6 show Pfirrmann grade 5. Narrowing in discs 1-2, 4, 6. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-2, 4, 6. Spondylolisthesis in discs 4.",0,1
43,local_data/spider/P35-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
44,local_data/spider/P59-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
45,local_data/spider/209_t2.png,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2-8 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Herniation in discs 1-2. Spondylolisthesis in discs 1-2. Modic type 2 in discs 1. Low endplate in discs 2-3. Up endplate in discs 2-3. Bulging in discs 3-4.,0,1
46,local_data/spider/P40-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
47,local_data/spider/161_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 1, 3-7 show Pfirrmann grade 4. Discs 2, 8 show Pfirrmann grade 5. Narrowing in discs 1-3, 8. Bulging in discs 1-5, 7-8. Low endplate in discs 1-8. Up endplate in discs 1-8. Modic type 2 in discs 1-8.",0,1
48,local_data/spider/5_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Bulging in discs 1-3.,0,1
49,local_data/spider/P43-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
50,local_data/spider/P57-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
51,local_data/spider/P25-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
52,local_data/spider/P15-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
53,local_data/spider/51_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 7-8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Low endplate in discs 1, 3. Up endplate in discs 1, 3. Modic type 2 in discs 1.",0,1
54,local_data/spider/212_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6 show Pfirrmann grade 3. Discs 2-5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2, 4, 7. Bulging in discs 1, 3-5, 7. Low endplate in discs 1, 5. Up endplate in discs 1. Herniation in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2, 4.",0,1
55,local_data/spider/P36-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
56,local_data/spider/155_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1, 4. Modic type 2 in discs 1-5.",0,1
57,local_data/spider/P98 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
58,local_data/spider/P21-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
59,local_data/spider/P83 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
60,local_data/spider/P46-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
61,local_data/spider/37_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-3. Low endplate in discs 1, 3. Up endplate in discs 1, 3. Narrowing in discs 3. Modic type 2 in discs 3.",0,1
62,local_data/spider/226_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2, 5-7 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1, 4 show Pfirrmann grade 5. Narrowing in discs 1, 3-4, 6-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-3, 5, 7. Modic type 3 in discs 4, 6.",0,1
63,local_data/spider/P47-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
64,local_data/spider/24_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6. Herniation in discs 2.",0,1
65,local_data/spider/115_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Bulging in discs 2-4. Low endplate in discs 2-4. Up endplate in discs 2-4. Modic type 2 in discs 2-3.",0,1
66,local_data/spider/P14-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
67,local_data/spider/171_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-6 show Pfirrmann grade 3. Discs 2, 7-8 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-6. Low endplate in discs 1-4, 6-8. Up endplate in discs 1-4. Modic type 2 in discs 1-8. Narrowing in discs 2-3.",0,1
68,local_data/spider/140_t1.png,"A T1-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 6-9 show Pfirrmann grade 1. Discs 1-3 show Pfirrmann grade 3. Discs 4-5 show Pfirrmann grade 4. Modic type 2 in discs 1, 3, 7. Narrowing in discs 3-5. Herniation in discs 3. Bulging in discs 4-5. Modic type 3 in discs 5. Up endplate in discs 8.",0,1
69,local_data/spider/P51-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
70,local_data/spider/195_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-2. Modic type 2 in discs 1-2.,0,1
71,local_data/spider/P91 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
72,local_data/spider/244_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-7. Modic type 2 in discs 1-5. Low endplate in discs 2, 4-7. Up endplate in discs 2, 4-7.",0,1
73,local_data/spider/P28-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
74,local_data/spider/P39-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
75,local_data/spider/P76 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
76,local_data/spider/136_t1.png,A T1-weighted MRI of a female patient with 6 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 3-7. Up endplate in discs 3-7. Modic type 2 in discs 4.,0,1
77,local_data/spider/38_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-7.",0,1
78,local_data/spider/218_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2. Low endplate in discs 2. Up endplate in discs 2. Spondylolisthesis in discs 2.,0,1
79,local_data/spider/P61-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
80,local_data/spider/P97 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
81,local_data/spider/P12-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
82,local_data/spider/233_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1. Herniation in discs 1. Modic type 2 in discs 1.,0,1
83,local_data/spider/220_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 6-8 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 2-8. Up endplate in discs 2-8. Narrowing in discs 3-5. Modic type 2 in discs 3-6.",0,1
84,local_data/spider/P32-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
85,local_data/spider/P23-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
86,local_data/spider/P55-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
87,local_data/spider/96_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5. Low endplate in discs 1-3. Up endplate in discs 1-3, 5. Modic type 2 in discs 1-2.",0,1
88,local_data/spider/117_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 2-5. Spondylolisthesis in discs 2. Low endplate in discs 3-6. Up endplate in discs 3-6. Modic type 2 in discs 3-6.,0,1
89,local_data/spider/P3-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
90,local_data/spider/P39-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
91,local_data/spider/170_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 1-4. Bulging in discs 2-5. Narrowing in discs 3-5.,0,1
92,local_data/spider/P81 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
93,local_data/spider/192_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Discs 1-3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-3. Up endplate in discs 1-3. Modic type 2 in discs 1-3, 5.",0,1
94,local_data/spider/P28-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
95,local_data/spider/P98 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
96,local_data/spider/75_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Herniation in discs 1. Bulging in discs 2-3. Low endplate in discs 2-3. Up endplate in discs 2-3. Modic type 2 in discs 2-4.,0,1
97,local_data/spider/131_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
98,local_data/spider/P62-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
99,local_data/spider/149_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1.,0,1
100,local_data/spider/32_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 5-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2.",0,1
101,local_data/spider/P71 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
102,local_data/spider/251_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 3. Discs 1, 4-6 show Pfirrmann grade 4. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 2-7. Herniation in discs 2-3, 6. Modic type 2 in discs 2-3. Up endplate in discs 7.",0,1
103,local_data/spider/35_t2_SPACE.png,A T2-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-3. Low endplate in discs 3-8. Up endplate in discs 3-8.,0,1
104,local_data/spider/P85 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
105,local_data/spider/89_t2.png,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 3, 5-6 show Pfirrmann grade 2. Discs 2, 4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Low endplate in discs 1. Herniation in discs 1. Bulging in discs 4.",0,1
106,local_data/spider/231_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-5, 7 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.",0,1
107,local_data/spider/127_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5, 7-8 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Narrowing in discs 1, 3-4. Bulging in discs 1-4, 6. Low endplate in discs 3-4. Up endplate in discs 3-4. Modic type 2 in discs 3-4.",0,1
108,local_data/spider/182_t2.png,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1-3, 6, 9 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 1-9. Up endplate in discs 1-9. Modic type 2 in discs 1-9. Narrowing in discs 4-5, 8.",0,1
109,local_data/spider/P57-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
110,local_data/spider/P18-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
111,local_data/spider/P85 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
112,local_data/spider/P47-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
113,local_data/spider/P41-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
114,local_data/spider/136_t2.png,A T2-weighted MRI of a female patient with 6 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 3-7. Up endplate in discs 3-7. Modic type 2 in discs 4.,0,1
115,local_data/spider/118_t2.png,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
116,local_data/spider/P60-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
117,local_data/spider/74_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-5 show Pfirrmann grade 2. Discs 1-3, 6 show Pfirrmann grade 4. Narrowing in discs 1-3, 6. Bulging in discs 1-3. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
118,local_data/spider/P33-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
119,local_data/spider/229_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Narrowing in discs 2-4. Bulging in discs 2-5. Low endplate in discs 2-6. Up endplate in discs 2-6. Modic type 2 in discs 2-4.,0,1
120,local_data/spider/P38-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
121,local_data/spider/P52-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
122,local_data/spider/239_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Herniation in discs 1. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
123,local_data/spider/P65-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
124,local_data/spider/143_t2.png,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 6-7, 9 show Pfirrmann grade 3. Discs 5, 8 show Pfirrmann grade 4. Discs 1-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
125,local_data/spider/2_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3 show Pfirrmann grade 3. Discs 1-2, 4-6 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 2-3. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2-5.",0,1
126,local_data/spider/166_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2, 4-6, 8 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 7 show Pfirrmann grade 5. Narrowing in discs 1, 3, 7. Bulging in discs 1-4. Modic type 2 in discs 2-4. Low endplate in discs 6-7. Up endplate in discs 6-7.",0,1
127,local_data/spider/93_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-2, 4. Low endplate in discs 1-3, 6. Modic type 2 in discs 1-3, 6. Herniation in discs 3.",0,1
128,local_data/spider/P30-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
129,local_data/spider/184_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 1-2. Up endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
130,local_data/spider/P4-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
131,local_data/spider/177_t2_SPACE.png,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 7-9 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-5, 7. Modic type 2 in discs 1-5. Up endplate in discs 2-7. Herniation in discs 7.",0,1
132,local_data/spider/167_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 6-7 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5. Modic type 2 in discs 3.",0,1
133,local_data/spider/P49-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
134,local_data/spider/180_t1.png,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 1. Narrowing in discs 1-6. Bulging in discs 1-4. Low endplate in discs 1-7. Up endplate in discs 1-7.,0,1
135,local_data/spider/P74 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
136,local_data/spider/P81 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
137,local_data/spider/P31-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
138,local_data/spider/P38-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
139,local_data/spider/15_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 2. Discs 1, 3 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 5. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Low endplate in discs 1, 4-6. Up endplate in discs 1, 4-6. Modic type 2 in discs 2-5. Herniation in discs 4.",0,1
140,local_data/spider/73_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 4-7.",0,1
141,local_data/spider/P73 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
142,local_data/spider/P42-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0

Overwriting local_data/lbp-test-meta.csv


In [6]:
%%writefile local_data/sentence-label.csv
,Reports,No Finding,LBP
1,"L4-5: degenerative annular disc bulge is noted more to the left side compressing thecal sac, compressing left nerve root and narrowing right neural foramen. // Evidence of hyperintense signal within the annulus fibrosus at left paramedian/posterolateral area which probably represents a torn annulus.",0,1.0
2,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
3,LSS MRI Features of muscle spasm. small central disc protrusion noted at L5-S1 level abutting the thecal sac. no significant thecal sac or nerve root compression noted.,0,1.0
4,"Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mildly compressing the thecal sac and encroaching exit nerve roots",0,1.0
5,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted ar L4/L5 level ,mildly compressing the thecal sac and both exit nerve roots",0,1.0
6,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots . Wide base disc bulges noted at L5/S1 level, mild compressing thecal sac and both nerve roots",0,1.0
7,"LSS MRI diffuse disc bulge noted at L4-L5 level , compressing the thecal sac and exit canals. small central disc protrusion noted at L5-S1 level. no significant spinal canal narrowing noted.",0,1.0
8,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level , with mild compresing thecal sac, NB: ABout 7 cm, mixed pelvic cyst noted, correlate with US .",0,1.0
9,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
10,LSS MRI no evidence of disc herniation. no significant thecal sac or nerve root compression noted.,1,0.0
11,"LSS MRI : About 3*2 cm lesion with inhomogenous signal intensity noted just posterior to L1 veretbral body ,largely compressing the Rt side of the thecal sac. contrast study is advsied. Diffuse disc bulge noted at L4/L5 level, compresing the thecal sac and encroaching upon both neural canals. Wide base disc bulge noted at L5/S1 level, extending to Lt lateral recess , compressin the thecal sac and nerve roots, more to left side, associated with ligamnetum flavum hyperatrophy.",0,1.0
12,"LSS MRI Rt paracentral disc protrusion noted at L4-L5 level, compressing the thecal sac. Adequate spinal canal.",0,1.0
13,L4-5 broad based central and left paracentral disc protrusion is noted compressing thecal sac and narrowing left neural foramen./// L5-S1: broad based degenerative central disc bulge is noted indenting thecal sac but not causing significant nerve root compression or foraminal narrowing.//// dehydrated L4-5 and L5-S1 intervetrebral discs./// vertebral bone marrow signal reconversion is noted.,0,1.0
14,LSS MRI : Feature of muscle spasm. No significant disc herniation or protrusion seen. Adequate thecal sac,0,1.0
15,Lumbosacral MRI Mild disc bulge noted at L5-S1 level Small right foraminal disc protrusion with annular tear is noted No significant thecal sac or nerve root compression noted,0,1.0
16,"Lumbosacral MRI Features of muscle spasm Dissicating lower disc space noted -Central and left paracentral disc protrusion noted at L4-L5 level compressing the thecal sac and Lt exit nerve root -Diffuse Disc bulges noted at the L5-S1 level , mild compressing the thecal sac and exiting nerve root.",0,1.0
17,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge causing compression of thecal sac, compressing left S1 nerve root within lateral recess, compressing left L5 nerve root within left neural foramen and narrowing right neural foramen./// L4-5: annular disc bulge is noted compressing thecal sac and narrowing both neural foramina/// L3-4: left posterolateral disc bulge is notedslightly narrowing left neural foramen.",0,1.0
18,L1-2: central disc bulge is noted indenting thecal sac/// L3-4: right posterolateral disc bulge is noted with end-plate osteophyte formation causing compression of right nerve root within neural foramen. L5-S1: right paramedian disc bulge is noted indenting thecal sac.,0,1.0
19,"Compression fracture is noted involving D12 body with compression of thecal sac and indentation of spinal cord at this level and maximum posterior displacement of posterior border of vertebral body by about 6.5mm. This fracture is believed to be pathological due to the mottled appearance of vertebral bodies. L4-5: secondary lumbar canal stenosis is noted due to degenerative annular disc bulge along with hypertrophy of facet joints and ligamenta flaca. compression of right L4 nerve root within right neural foramen due to the above mentioned disc bulge is also noted. L5-S1: degenerative annular disc bulge more toward the right side causing compression of thecal sac, compression of right L5 nerve root within right neural foramen and narrowing of left neural foramen. Diffusely altered signal of vertebral bone marrow; further evaluation is advised.",0,1.0
20,"L5-S1: left paramedian disc protrusion on top of degenerative annular disc bulge with end-plate osteophytes formation causing compression of thecal sac, compression of left S1 nerve root within lateral recess, and narrowing of neural foramina.",0,1.0
21,Lumbosacral MRI Features of muscle spasm Central and left paracentral disc protrusion noted at the level of L4-L5 Significantly compressing the thecal sac and the left exit nerve root and desiccative disc material noted .,0,1.0
22,L4-l5 L5-S1: diffuse disc bulges compressing the thecal sac and encroaching exit canals.,0,1.0
23,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0.0
24,L4-5: mild broad based central disc bulge is noted indenting thecal sac.// L5-S1: central disc protrusion is noted indenting thecal sac/// diffuse bone marrow signal reconversion is noted,0,1.0
25,LSS MRI No evidence of disc herniation. No signidant thecal sac or nerve root compression noted.,1,0.0
26,L2-3: broad based right paramedin disc extrusion is noted slightly migrating downwards,0,1.0
27,"MRI OF THE LSS -Feature of muscle spasm. -Central disc protrusion noted at D11/D12, obliterating anterior CSF spaces and compressing spinal cord and encroaching exit nerve canals. -Diffuse disc bulges noted at L4/L5 , mild compressing thecal sac and exit nerve roots . Diffuse disc bulges noted at L5/S1, compressing thecal sac and exit nerve roots more to Lt side",0,1.0
28,"LSS MRI Diffuse disc bulge noted at L5-S1 level, more to the Lt side, compressing the thercal sac and Lt exit canal.",0,1.0
29,"LSS MRI : Grade I spondylolithesis of L5/S1 level, with narrowed disc space . Multlevel dissicating disc materials noted Evidence of previus surgery noted at lumbosacra level. Diffuse disc bulges noted at (D12/L1& L4/L5 & L5/S1 levels, compressing (contrast study is asuperior",0,1.0
30,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
31,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0.0
32,L4-L5: Mild central disc protrusion noted abutting the thecal sac.,0,1.0
33,"LSS MRI (with contrast ):Follow up History of recent & Prevoius operations . Inhomogenous enhancing granulation tissue noted at Lt sided of thecal sac ,compressing the Lt sided thecal sac and exit neural canal (site of operation (L5/S1 )extending posteriorly with small collection . Inhomogenous enhancing granulation noted at L4/L5 left sided .",0,1.0
34,"LSS MRI : There is sequeatrated disc fracgement (ring enhancing ) noted just posterior to L5 vertebral body below L4/L5 level ,at Lt paracentral thecal sac compressing Lt sided thecal sac and exit nerve roots",0,1.0
35,MRI OF THE LSS Features of muscle spasm noted. No evidence of disc heniation noted. No significant thecal sac or nerve root compression noted.,0,1.0
36,Lumbosacral MRI Straightening of the lumbar spine mostly due to muscle spasm Dehydrated discal material Right paracentral disc protrusion noted at the L4-5 level compressing the thecal sac Right foraminal disc protrusion noted at the L5-S1 level compressing the right S1 nerve root Incidental finding of subcutaneous lipoma at the level of the L1 vertebra,0,1.0
37,"LSS MRI Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac. No significant thecal sac or nerve root compression noted. Adequate spinal canal.",0,1.0
38,"Lumbosacral MRI Features of muscle spasm Partially desiccated L4-L5level Central disc protrusion noted at the L4-L5 level, partially caudally migration , significant compressing the thecal sac and encroaching exiting neural canal Widely disc bulge noted at L5-S1 mildly compressing the thecal sac",0,1.0
39,"Lumbosacral MRI Diffuse disc bulges noted at L4-5 and L3-L4 levels, Abutting the thecal sac With mild encroachment of the left exit neural canal at L4-L5 level Features of muscle spasm noted",0,1.0
40,MRI OF THE LSS -Feature of muscle spasm. diffuse disc bulge noted at L3/L4& L4/L5 level mildly compressing thecal sac and exit nerve roots,0,1.0
41,L4-L5: Central disc protrusion compressing the thecal sac. L5-S1: Rt paracentral disc protrusion abutting Rt S1 nerve root.,0,1.0
42,"LSS MRI (without contrast): -Evidence of recent operation noted at lower level (L5/S1 ), with soft tissue edema. -Central and Rt paracentral disc protrusion notedat L5/S1 level, displacing inferiorly the thecal sac and encraoching Rt S1 exit nerve root. -Diffuse disc bulge noted at L4/L5 level extending to Lt lateral recess, compressing the thecal sac and encroaching upon exit neural canal .",0,1.0
43,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0.0
44,"LSS MRI : -Feature of muscle spasm. -Degenerative spondylotic chnages noted . -Multilevel diffuse disc bulges , noteceably noted at L3/L4 & L4/L5 level, compresiing thecal sac and both nerve roots associated with ligamnetum flavum hypertrophy , all leading to secandary spinal canal stenosis. Small Lt paracentral disc protrusion noted at L1/L2 level , mild compressing thecal sac NB : Lt renal cyst noted",0,1.0
45,"LSS MRI : feature of muscle spasm. Difffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both nerve roots",0,1.0
46,"L3-L4, L5-S1: Diffuse disc bulges, more to the Rt side, abutting the thecal sac and encroaching Rt exit neural canal. L4-L5: Lt paracentral and foraminal disc protrusion, compressing the thecal sac and Lt exit neural canal.",0,1.0
47,LSS MRI Mild disc bulges noted at L5-S1 and L2-L3 levels abutting the thecal sac. Adequate spinal canal.,0,1.0
48,"Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3/L4 & and L5-S1levels,  mild compressing the thecal sac and mild encroaching exiting neural canal,associated with annular tear at L5-S1",0,1.0
49,Lumbosacral MRI Diffuse disc bulges noted at L4-L5 and L5-S1 levels with endplate degeneration Compressing the thecal sac,0,1.0
50,LSS MRI No evidence of disc herniation noted. No thecal sac or nerve root compression noted,1,0.0
51,"LSS MRI Enhancing Lt paracentral soft tissue noted at L5-S1 level, mostly granulation tissue, compressing Lt S1 nerve root and thecal sac. no definite fractures or subluxation noted.",0,1.0
52,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level , mildly compressing the thecal sac, and  exit nerve roots",0,1.0
53,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and both nerve roots .  mild diffuse disc bulges noted at L3/L4 level, mild compressing thecal sac.  NB :small tarlove cyst noted at S2 level",0,1.0
54,C SPINE AND LSS MRI Features of muscle spasm. No evidence of disc herniation. No significant thecal sac or nerve root compressioon,0,1.0
55,"LSS MRI Rt paracentral disc protrusion notred at L4-L5 level, compressing the thecal sac and encroaching upon Rt L5 nerrve root. adequate spinal. features of muscle spasm.",0,1.0
56,"LSS MRI L4-L5: Lt paracentral disc protrusion noted, compressing the tyhecal sac and Lt exit canal.",0,1.0
57,"LSS MRI:  Feature of muscle spasm.  Mild diffuse disc bulges noted at L4/L5&L5/S1 levels , mild compressing thecal sac and exit nerve roots .",0,1.0
58,Lumbosacral MRI No evidence of disc herniation No thecal sac or nerve root compression noted,1,0.0
59,L4-L5: Diffuse disc bulge compressing the thecal sac and exit neural canals.,0,1.0
60,L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals. L5-S1: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt S1 nerve root.,0,1.0
61,L.SS MRI :  Central of Lt para central disc protrusion noted at L4/L5 migrating caudally causing thecal sac at Lt nerveroots,0,1.0
62,L5-S1: Lt posterolateral disc protrusion noted. No significant thecal sac or nerve root compression noted.,0,1.0
63,L5-S1: Diffuse disc bulge abutting the thecal sac. No significant thecal sac or nerve root compression noted.,0,1.0
64,Multilevel diffuse disc bulges noted compressing the thecal sac and exit canals. Multilevel schmorl's nodules noted.,0,1.0
65,"LSS MRI :  Feature of muscle spasm .  Multiple schmorls nodules  L3/L4 level , diffuse disc bulge , extending to lateral recess, more to Lt sided, compressing thecal sac and both nerve roots , more to Lt.  L4/L5 & L5/S1levels:  diffuse disc bulge , extending to lateral recess, compressing thecal sac and both nerve roots",0,1.0
66,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0.0
67,"Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level, compressing the thecal sac and right exit neural canal Diffuse disc bulges noted at L4-L5 and L3-L4 levels compressing the thecal sac and exiting neural canals Multilevel endplate degeneration noted",0,1.0
68,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1.0
69,LSS MRI mild disc bulge noted at L4-L5 level abutting the thecal sac.  no significant thecal sac or nerve root compression noted.,0,1.0
70,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
71,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
72,"LSS MRI :  Feature of muscle spasm.  Wide base posteriocentral disc bulge noted at L4/L5 level, copmressing the thecal sac and encroaching upon exit neural canals.  Diffuse disc bulge noted at L3/L4 level, mildly compressing the thecal sac and both nerve roots",0,1.0
73,LSS MRI : Feature of muscle spasm. No evidence of Thecal sac or nerve root roots compression,0,1.0
74,"Lumbosacral MRI Diffuse disc bulge noted at L4-5 level compressing the thecal sac and exit neural canals Left paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and left exit neural,",0,1.0
75,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  End plate degenerative changes noted .  Grade I spondylolisthesis of L5 over S1 , with bone marrow edema and narrowed disc spaces with irregular end plates  Bulky Lt iliopsoas muscle with abnormal signal alteration ? post traumatic , infective.  Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac and exit neural canals.  Adequate Spinal canal noted.",0,1.0
76,LSS MRI Mild disc bulge noted at L5-S1 level with Rt posterolateral tear.  No significant thecal sac or nerve root compression noted.  Adequate spinal canal.,0,1.0
77,"LSS MRI Multilevel, diffuse disc bulges noted (L2-L3, L3-L4 and L4-L5 levels), mildly compressing the thecal sac.",0,1.0
78,no evidence of disc heniation. no significant thecal sac or nerve root compression.,1,0.0
79,"LSS MRI :  Feature of muscle spasm.  L4/L5 level :  Partial dissicating disc material noted ,with diffuse disc bulge causing mild compressing thecal sac and encroaching exit neural canals",0,1.0
80,"L3-L4: Central disc protrusion noted, largely compressing the thecal sac. L4-L5: Rt paracentral disc protrusion noted, compressing the thecal sac and Rt exit neural canal. L5-S1: Mild disc bulge noted.",0,1.0
81,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression .  No disc protrusion or herniation seen  NB :  small tarlovs cyst.  pelvic cyst noted ? correalte with US,0,1.0
82,LSS MRI :  scoliosis of lumbar spine noted.  dissicating disc materials .  Wedging fracture of L2 vertebral body mostly dut to old insult .  Grade II spondylolithesis of L4 over L5 level.  Diffuse disc bulges noted at L5/S1 level mild compressing thecal sac,0,1.0
83,"LSS MRI : faeture of muscle spasm. there is Posterio-central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching both exit neural canal",0,1.0
84,LSS MRI L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit neural canal. Mild disc bulges noted at L3-L4 and L4-L5 levels.,0,1.0
85,LSS MRI Features of muscle spasm. mild disc bulge noted at L4-L5 level.,0,1.0
86,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Mild Diffuse disc bulges noted at L4-L5 level,mildly compressing thecal sac and exit nerve root .",0,1.0
87,"LSS MRI Lt paracentral disc protrusion noted at L5-S1 level, abutting the thecal sac and Lt S1 nerve root.  adequate spinal canal.",0,1.0
88,"LSS MRI: feature of muscle spasm. Wide base focal disc bulge , more to Lt sided, noted at L5/S1 level, compressing thecal sac and both exit nerve roots associated with dehydrated disc material.",0,1.0
89,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
90,Lumbosacral MRI Feature of muscle spasm Multilevels diffuse disc bulging noted noticebly at L3-L4 & L4-L5 levels mild compressing thecal sac and exit nerve roots dissicating disc material noted,0,1.0
91,"LSS MRI Diffuse disc bulge noted at L3-L4 level, more to the Lt side, compressing the thecal sac and exit canals.  Features of muscle spasm.",0,1.0
92,LSS MRI : Feature of muscle spasm Mild difuse bulge disk  L5-S1 noted No significant thecal sac or nerve roots compression seen.,0,1.0
93,L4-L5: Lt posterolateral disc protrusion noted compressing Lt exit canal with Lt L5 nerve root. Mild disc bulges noted at L3-L4 and L5-S1 levels.,0,1.0
94,MRI of the lumbosacral spine  No evidence of disc herniation  No thecal sac or nerve root compression noted  Adequate spinal canal,1,0.0
95,"LSS MRI :  -Feature of muscle spasm  -Multiple schmorls noted and suggetsive limbus veterba at anterio-superior corner of L4 .  -Mild diffuse disc bulges noted at L4/L5 and L5/S1 levels, mild compressing tehcal sac and both nerve roots .",0,1.0
96,LSS MRI :  Feature of muscle spasm.  No thecal sac or nerve root compression noted,0,1.0
97,"LSS MRI :  Feature of muscle spasm.  wide base disc bulges noted at L3/L4 level, compressing thecal sac and encroaching neural exit canals.  Diffuse disc bulges noted at L4/L5 level,mild compressing thecal sac and l exit nerve roots.",0,1.0
98,L4-L5: Central disc protrusion noted compressing the thecal sac.,0,1.0
99,"LSS MRI : -Feature of muscle spasm. -Grade I spondylolisthesis of L4 over L5 level -Diffuse disc bulge noted at L4/L5 level,largely  compressing the thecal sac extending to lateral recess , associated with ligametum flavum hyperatrophy  with encroaching exit neural canals,  all leading to secandary spinal canal stenosis ,as well as dehydrated disc materials",0,1.0
100,"L5-S1: Mild RT PARACENTRAL DISC PROTRUSION NOTED, ABUTTING THE THECAL SAC. L4-L5 and L3-L4: Mild disc bulges noted.",0,1.0
101,Lumbosacral MRI  Mild disc bulge noted at L5-S1 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,0,1.0
102,"LSS MRI  L5-S1: Lt paracentral and foraminal disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  features of muscle spasm noted.",0,1.0
103,"LSS MRI  multilevel diffuse disc bulges (L4-S1) compressing the thecal sac, noticeably at L4-L5 where the Lt exit canal also appeared compressed",0,1.0
104,"LSS MRI Previous surgery noted with transpedicle fixation noted at L3-L4 level. Normal bony alignment. adequate spinal canal with no significant thecal sac or nerve root compression. enhancing posterior parts of L4-L5 disc with ioncreased T2 signal, probably discitis.",0,1.0
105,LSS MRI  Multilevel degenerative end plate changes noted.  mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.  tarlov cyst noted at S2 level,0,1.0
106,"L3-L4: Mild disc bulge noted L4-L5: broad based, central and Lt paracentral disc protrusion, compressing the thecal sac and Lt exit neural canal. L5-S1: Central disc protrusion, compressing the thecal sa",0,1.0
107,"LSS MRI  L3 and L2 hemangiomas.  diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.  mild disc bulge noted at L3-L4 level, compressing the thecal sac and encroaching upon the Lt exit canal.",0,1.0
108,L4-L5: Rt posterolateral disc protrusion with annular tear compressing Rt exit neural canal.,0,1.0
109,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, mildly compressing the thecal sac and exit nerve roots",0,1.0
110,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
111,"LSS MRI :  fea ture of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, compressing thecal sac extending to lateral recess , with encroaching exit neural canals, as well as dehydrated disc materials",0,1.0
112,LSS MRI :  feature of muscle spasm.  Dissicating disc materail at lower level.  Lt foraminal disc protrusion noted at L5/S1 level compressing thecal sac and Lt exit nerve root (S1).  Diffuse disc bulges noted at L4/L5 level mild compressing thecal sac and both nerve roots .,0,1.0
113,"LSS MRI:  Lt paracentral ,wide base ,disc bulge noted at L5/S1 level, compressing the thecal sac and exit neural canal.",0,1.0
114,"LSS MRI : Degenerative changes. Multilevel diffuse bulges noteceably at L2/L3, L3/L4&L4/5 levels, compressing the thecal sac and encroaching both exit nerve roots , associated with ligamentum flavum hyperatrophy, all leading to secandary spinal canal stenosis",0,1.0
115,L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit neural canals.,0,1.0
116,LSS MRI  small central disc protrusion noted at L5-S1 level.  diffuse disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1.0
117,L3-L4 and L4-L5: diffuse disc bulges compressing the thecal sac and exit canals. L5-S1: Mild disc bulge noted abutting the thecal sac and S1 nerve roots.,0,1.0
118,L3-L4 and L4-L5: Mild disc bulges noted. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,0,1.0
119,LSS MRI  Mild disc bulge noted at L4-L5 level.  adequate spinal canal.,0,1.0
120,LSS MRI  Rt paracentral disc protrusion noted compressing the thecal sac and Rt S1 nerve root.,0,1.0
121,"LSS MRI :  Feature of muscle spasm.  No thecal sac or exit nerev roots compression seen .  No disc herniation or protrusion noted .  NB :  pelvic kidney noted ,correlate with US .",0,1.0
122,"LSS MRI :  Feature of muscle spasm.  mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots",0,1.0
123,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mildly compressing thecalsac and nerve roots .",0,1.0
124,MRI OF THE C. SPINE  -Features of muscle spasm noted.  -Multilevel dissecating disc materials.  -Wide base disc bulges extending more to Rt side noted at C4/C5 level with obliterating anterior CSF spaces and intending spinal cord and encroaching exit neural canal more to the Rt  -Multilevel small posterior central disc protrusion noted obliterating anterior CSF spaces with annular tears. .  MRI OF THE LSS  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1 .  -Dissecating disc material L5/S1  -L5-S1 level : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1.0
125,"LSS MRI :  Feature of muscle spasm.  Small Lt paracentral disc protruion noted at D12/L1 , mildly compressing Lt exit nerve root.",0,1.0
126,"LSS MRI  Small Lt paracentral protrusion noted at L5-S1 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.",0,1.0
127,"LSS MRI Central and to the Rt side, disc protrusion noted at L5-S1 level,  compressing the thecal sac and encroaching upon the Rt exit canal. Features of muscle spasm.",0,1.0
128,LSS MRI : Feature of muscle spasm No disc protrusion or herniation noted . No significant thecal sac or nerve roots compression seen .,0,1.0
129,"LSS MRI : feature of muscle spasm. posterior central disc protrusion noted at L5/S1 level, largely compressing thecal sac and encroaching exit neural canals. Small Rt paracentral disc protrusion noted at D11/D12 level, mildy compressing the thecal sac .",0,1.0
130,"LSS MRI  -Feature of muscle spasm.  -Small central disc protrusion , noted at D12/L1 level, obliterating anterior CSF space and indenting spinal cord.  -Partial Dissecating disc material noted at lower levels.  -Wide base disc bulges noted at L4-L5 levels compressing the thecal sac and exit neural canals  -Mild Diffuse disc bulges noted at L5-S1 levels mild compressing the thecal sac and exit neural canals with annular tear.",0,1.0
131,"LSS MRI : Feature of muscle spasm with schmorls nodules . Posterio-central disc protrusion noted at L5/S1 level, compressing thecal sac and encroaching exit nerve canal  with annular tear and dehtdrated disc material . Diffuse disc bulges note at D12/L1 level, obliterating anterior CSF spaces .",0,1.0
132,"L3-L4, L4-L5, L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1.0
133,"L4-L5: Broad based disc protrusion, more to the Lt side, compressing the thecal sac and exit canals (largely the Lt one).",0,1.0
134,Lumbosacral MRI Mild disc bulge noted at L5-S1 level abutting both S1 nerve roots Diffuse disc bulge noted at L4-L5 level compressing the thecal sac and encroaching upon the exit neural canals,0,1.0
135,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals. adequate spinal canal.",0,1.0
136,"LSS MRI :  -Feature of muscle spasm.  -Grade I spondylolisthesis of L5 over S1  -Dissicating disc material noted at lower levels  -Postreior central disc protrusion noted at L4/L5 level, largely compressing thecal sac more to Lt and encroaching exit neural canal.  -Wide base disc bulge noted at L5/S1 level, compressing thecal sac and nerve roots , with modic type II noted & narrowed disc space",0,1.0
137,"Lumbosacral MRI -Diffuse disc bulge noted at L4-5 level , extending more to the right side compressing the thecal sac and encroaching upon the exiting neural canals, more to Rt sided  -Spondylolisthesis grade 1 seen at L5 over S1 -Diffuse disc bulge noted at L5-S1 mild compressing thecal sac and exiting nerve roots -Feature of muscle spasm with dissicating disc material noted at lower level",0,1.0
138,"LSS MRI L4-L5: Diffuse disc bulge compresing the thecal sac and exit canals. L5-S1: Mild disc bulge, more to the Lt side, encroaching on Lt exit canal. Adequate spinal canal.",0,1.0
139,"L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, facet joint and ligamentum flavum hypertrophy, compressing the thecal sac, exit canals  and narrowing the spinal canal. L3-L4: diffuse disc bulge compressing the thecal sac and exit canals.",0,1.0
140,"LSS MRI : feature of muscle spasm. Multilevel diffuse disc bulges mainly at L2/L3,L3/L4 & L4/L5 levels compressing thecal sac and both nerve roots",0,1.0
141,LSS and pelvis MRI:  Evidence of the previous surgery noted  No evidence of significant thecal sac or nerve root compression noted  Adequate thecal sac,1,0.0
142,mild disc bulge at L4_L5 level.,0,1.0
143,"Lumbosacral MRI  -Feature of muscle spasm noted  Modic type II noted at the level of the L4-L5-  -Wide base disc bulge noted at the L4-L5 level ,extending to the lateral recess compressing the thecal sac and encroaching upon exiting neural canals",0,1.0
144,Lumbosacral MRI No evidence of disc herniation The thecal sac or nerve root compression noted,1,0.0
145,feature of muscle spasm.  No significant thecal sac or nerve root compressing,0,1.0
146,LSS MRI :  feature of muscle spasm.  No significant thyecal sac or nerve root compression .  adequate spinal canal,0,1.0
147,LSS MRI :  feature of muscle spasm.  End palte degenerative chnages.  L4/L5 level :narrowed disc space.  Diffuse disc bulges compressing thecal sac and both nerve roots .,0,1.0
148,L4-L5: Mild disc bulge noted. No significant thecal sac or nerve root compression noted.,0,1.0
149,MRI LSS:  feature of muscle spasm  Endplate Modic typeI noted at L4/L5 level with dehyadrated disc material.  Central and to the Rt disc protrusion noted at the L4-L5 level significantly compression thecal sac and exit nerve roots,0,1.0
150,No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0.0
151,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal  and both nerve roots . NB: lipoma of the filum terminale / filar lipoma is noted",0,1.0
152,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0.0
153,"diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit neural canals- more on the Lt side. adequate spinal canal.",0,1.0
154,"LSS MRI :  L3/L4 level :wide based disc bulge noted with a small soft tissue lesion noted at Lt foraminal region ? neurogenic tumour - sequestrated disc . contrast study is recommended  L4/L5 level: diffuse disc bulge , extended to Rt lateral recess , mild compressing thecal sca and encroaching exit nerve root more to RT .",0,1.0
155,"LSS MRI  Diffuse disc bulges, at L4-L5 and L5-S1 levels, compressing the thecal sac and exit neural canals.",0,1.0
156,"LSS MRI :  Feature of muscle spasm.  Wide base focal disc bulge noted at L5/S1 level, compressing the thecal sac with annular tear and mild encroaching upon exit neural canal.",0,1.0
157,"MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Desiccated disc material .  Modec type II of L2/L3 with narrowed disc space .  Diffuse disc bulge ,with Rt foraminal disc protrusion noted at L2/L3 level compressing the thecal sac and exit neural canals.  With spinal canal stenosis noted.  Diffuse disc bulge at L3/L4 & L4/L5 levels , mild compressing thecal sac and encroaching exit nerve root , with relatively secondary spinal canal stenosis",0,1.0
158,Old L1 compression fracture noted. No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,0,1.0
159,"LSS MRI : -Feature of muscel spasm. -Dissicating discal materials.   -Modic type II noted L2/L3 level. -Diffuse disc bulges noted at  L2/L3 & L4/L5 levels ,compressing the thecal sac ,extending to  lateral recess -encroaching upon exit neural canals. -Mild disc bulge noted at L3/L4 level, mildy compressing the thecal sac .",0,1.0
160,"LSS MRI :  Feature of muscle spasm.  dissicating disc materials noted .  Multilevels diffuse disc bulges noted mainly at L4/L5& L5/S1 levels, compressing thecal and encroaching exit neural canal with relatively ligamentum flavum hyperatrophy at L4/L5 level.",0,1.0
161,"LSS MRI  L4-L5: Diffuse disc bulge, more to the Lt side, compressing the thecal sac and Lt exit canal.  Adequate spinal canal.",0,1.0
162,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0.0
163,"Lumbosacral MRI Left paracentral disc protrusion noted at T12 - L1, Compressing the thecal sac and encroaching on the left exit neural canal mild, Diffuse disc bulges noted at L3-L4 and L4-L5 level",0,1.0
164,mild disc bulge noted at L5-S1 level.  adequate spinal canal.,0,1.0
165,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level abutting thecal sac.  No significant thecal sac or nerve roots compression noted .,0,1.0
166,"LSS MRI : Feature of muscle spasm. dissicating disc materials . Multilevel diffuse disc bulges noted , notceably at L4/L5 & L5/S1, level, compressing the thecal sac and encroaching exit neural canal , associated with ligamentum flavum hyperatrophy at L4/L5 level with secandary spinal canal stenosis. NB : small Rt renal  cyst",0,1.0
167,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1.0
168,"LSS MRI Multilevel diffuse disc bulges noted, noticeably at L4-L5 level, where it compresses the thecal sac and exit canals, more to the Rt side.",0,1.0
169,"LSS MRI : -Feature of muscle spasm. Diffuse decreased bone marrow signal intensitie (obese patient ). - Diffuse disc bulge noted, L4/L5 level, mildy compressing thecal sac and encroaching both exit neural canals",0,1.0
170,"LSS MRI : .Feature of muscle spasm. Dissicating disc materials noted . .Multilevel diffuse disc bulges, notceably noted at L3/L4 & L4/L5 levels, compressing the thecal sac and encroaching upon exit neural canals .",0,1.0
171,"LSS MRI  diffuse disc bulges noted at L4-L5 and L5-S1 levels, compressing the thecal sac and exit canals.  the spinal canal appeared adequate.  L1 hemangioma.",0,1.0
172,LSS MRI : Feature of muscle spasm. Adequate thecal sac . No disc protrusion or herniation seen .,0,1.0
173,"MRI OF THE L. SPINE  Feature of muscle spasm.  Dissecating disc material at L5/S1 with narrowed disc space .  Diffuse disc bulge noted at L5/S1 ,mild compressing thecal sac and exit nerve roots.",0,1.0
174,Multilevel diffuse disc bulges with facet joint hypertrophy and ligamentum hypertrophy. At L4-L5 level mild spondylolisthesis noted. Spinal canal appeared narrow at L3-L4 and L4-L5 levels.,0,1.0
175,Multilevel diffuse disc bulges noted with facet joint hypertrophy compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1.0
176,"LSS MRI  Features of muscle spasm noted.  multilevel schmorls nodule noted  Lt foraminal disc protrusion noted at L4-L5 level, compressing the thecal sac and Lt L5 NERVE ROOT.",0,1.0
177,"LSS MRI  Small Rt parantral disc protrusion noted at L4-L5 level, comressing the thecal sac.  adequate spinal canal",0,1.0
178,Lumbosacral MRI Features of muscle spasm Multilevel dissicating  disc material Modic Type II at the L5-S1 Hemangioma of the L1 vertebral body -Diffuse disc bulge with right paracentral disc extrusion noted at the L5-S1 compressing the thecal sac and encroaching upon  neural canal -Diffuse disc bulge noted at the L4-L5 mild compressing the thecal sac and there-,0,1.0
179,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1.0
180,"Lumbosacral MRI Feature of muscle spasm Scoliosis of the lumbar spine convex to the left  side Dissicating  disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3 , L3-L4& L4/L5 levels compressing  thecal sac and encroaching exiting neural canal, associated with secondary spinal canal stenosis",0,1.0
181,Rt paracentral disc protrusion noted compressing the thecal sac.,0,1.0
182,"LSS MRI : .Feaure of muscle spasm. .Multilevels diffuse disc bulges noted nortceably at L2/L3 & L4/L5 level, compressing the thecal sac and enroaching upon exit neural canals.   anterior wedging of D12 vertebral body  associated  with multiple Schmorl's nodes, and end plate irregularity , Dorsal spine MRI is advsied to rule out, Schuerman's disease .",0,1.0
183,L5-S1: Mild Lt paracentral disc protrusion compressing the thecal sac. L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals. Mild multilevel disc bulges noted elsewhere.,0,1.0
184,"LSS MRI : Feature of muscle spasm. Central and to the Lt,  disc protrusion noted  at L5/S1 level relatively compressing thecal sac and encroaching exit neural canal , with dissicating disc material",0,1.0
185,L3-L4 AND L5-S1: Mild disc bulges noted. No significant thecal sac or nerve root compression noted.,0,1.0
186,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1.0
187,"LSS MRI Central disc protrusions noted at L3-L4, L4-L5 and L5-S1 levels, compressing the thecal sac. mild disc bulge noted at L2-L3 level.",0,1.0
188,"LSS MRI : .Feature of muscle spasm. .Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac extending to lateral recess , more to Rt sided with encroaching upon exit neural canal. .Mild disc bulge noted at L5/S1 level, mildy compressing thecal sac.",0,1.0
189,"Multilevel diffuse disc bulges noted with facet joint hypertrophy and hypertrophied ligament flava, compressing the thecal sac and exit neural canals with narrowed spinal canal.",0,1.0
190,LSS MRI  L5-S1: Rt paracentral disc protrusion compressing Rt S1 nerve root.  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals with narrow spinal canal.  L3-L4: Mild disc bulge noted.,0,1.0
191,"L2 osteoprotic compression fracture with no retrobulge into spinal canal.  No evidence of disc herniation or thecal sac compression.  high T2 signal intensity involving the sacral ala (? post trauma, inflamatory,..).",0,1.0
192,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0.0
193,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1.0
194,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0.0
195,MRI LSS Feature of muscle spasm noted Diffuse disc bulge noted at L5-S1 level mild compressing thecal sac and exiting nerve root Dehydrated  disc material noted at the same level,0,1.0
196,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildly compressing the thecal sac and  both exit nerve roots",0,1.0
197,"LSS MRI  Mild disc bulge noted at L5-S1 level, abutting the thecal sac and S1 nerve roots.  Adequate spinal canal.",0,1.0
198,"LSS MRI:  feature of muscle spasm.  Dissicating disc mayetrials.  Lt paracentral disc protruion noted at D12/L1 level, indenting spinal cord.  Multilevel diffuse disc bulges noted at L3/L4&L4/L5 levels , compressing thecal sac and encroaching exit neural canals .",0,1.0
199,L3-L4: Rt foraminal disc protrusion compressing Rt L4 nerve root. Multilevel mild disc bulges noted elsewhere.,0,1.0
200,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level. mildly compressing the thecal sac and exit nerve roots,0,1.0
201,"L5-S1: Mild disc bulge noted. L4-L5: Diffuse disc bulge with Lt posterolateral annular tear, abutting the thecal sac and encroaching Lt exit canal.",0,1.0
202,LSS MRI  L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4 AND L4-L5: Mild disc bulges noted.,0,1.0
203,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
204,L4-L5 AND L5-S1: diffuse disc bulges compressing the thecal sac and exit canals. L3-L4: Mild disc bulge noted.,0,1.0
205,LSS MRI :  Feature of muscel spasm.  Multilevel end plates degenerative changes.  schmorles nodules noted .  No significant disc protrusion or herniation seen.  No thecal sac or nerve roots compression.,0,1.0
206,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 level, mildy compressint the thecal sac and exit nerve root.",0,1.0
207,LSS MRI Multilevel mild disc bulges noted. no significant thecal sac or nerve root compression noted. adequate spinal canal.,0,1.0
208,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0.0
209,L5-S1: Mild disc bulge noted.,0,1.0
210,"LSS MRI : Dissicating disc materials . Narrowed disc spaces noted at L2/L3& L3/L4 levels . Grade I spondylolisthesis of L2 over L3 level   Multilevel diffuse disc bulges noticeably at L3/L4 level ,relatively compressing thecal sac and both nerve roots more at Rt sided .",0,1.0
211,Marked degenerative changes with kyphoscoliosis. Spondylolisthesis noted at L2-L3 level.,0,1.0
212,Lumbosacral MRI  Diffuse disc bulge more to the left side at L5-S1 level compressing the thecal sac and left exit neural canal  Diffuse disc bulge noted at L3-L4 level compressing the thecal sac and encroaching upon exit neural canals,0,1.0
213,"LSS MRI  Feature of muscle spasm.  Partial dissecating disc materials.  Multilevel diffuse disc bulges noted at L2/L3,L3/L4&L5/S1 level, compressing thecal sac and encroaching exit neural canal",0,1.0
214,"LSS MRI:  -Evidence of bone marrow edema involving L1 vertebral body , extending to the Lt sided posterior element ?? bone isotope scan is advised,  -Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots , with ligamentum flavum hyperatrophy , leading to secandary spinal canal stensois .  Narrowed disc space of L5/S1 level, with diffuse disc bulge , more to Lt sided compressing thecal sac and nerve roots.  D12 vertebral body hemangioma ?",0,1.0
215,Lumbosacral MRI Right paracentral and foraminal disc protrusion noted at L5-S1 level compressing the thecal sac and right exiting neural canal Diffuse disc bulge noted at L4-L5 level encroaching  upon the thecal sac and exiting neural canals,0,1.0
216,LSS MRI : feature of muscle spasm. multiple schmorls nodules. Left  sided foraminal lumbar disc protrusion  noted at L5/S1 level compressing Lt sided thecal sac and the exiting nerve root.,0,1.0
217,"LSS MRI L5-S1 level: Lt paracentral disc protrusion noted, compressing the thecal sac and abutting the Lt S1 nerve root.",0,1.0
218,"LSS MRI  L5-S1: central and more to the Lt side, disc protrusion noted, compressing the thecal sac and Lt S1 nerve root.  L4-L5: diffuse disc bulge noted with Rt foraminal disc protrusion, compressing the thecal sac and Rt L5 nerve root",0,1.0
219,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots .",0,1.0
220,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 levels, mild compressing the thecal sac and both nerve roots .",0,1.0
221,LSS MRI  -feature of muscle spasm.  -Modec type II noted at L4/L5 level.  -Diffuse disc bulges noted at L4-L5&L5/S1 levels compressing the thecal sac and exit neural canals.,0,1.0
222,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1.0
223,"L4-L5: Central disc protrusion, compressing the thecal sac.",0,1.0
224,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0.0
225,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L3/L4& L4/L5 levels, mildly compressing the thecal sac and exit nerve roots",0,1.0
226,LSS MRI  mild disc bulge noted at L5-S1 level.  No significant thecal sac or nerve root compresion noted.,0,1.0
227,"MRI OF THE LSS  Feature of muscle spasm.  Spondylolisthesis of L5 over L5 level , with Modec type II endplate .as well as dissecated disc material.  Diffuse disc bulge noted at L4/L5 level compressing thecal sac and both exit nerve root .  Mild diffuse disc bulge noted at L3/L4 level ,mild compressing thecal sac .",0,1.0
228,"LSS MRI :  Feature of muscle spasm.  L1/L2 level:  posterior central disc protrusion largly compressing thecal sac and both exit nerve roots.  L5/S1 level:  Lt paracentral disc protruion , migrating caudally ,compressing thecal sca and Lt exit nerve roots.",0,1.0
229,"LSS MRI : Feature of muscle spasm. Multilevel diffuse disc bulge noted , noticeably at  L3/L4&  at L4/L5 levels, relatively compressing  thecal sac and both exit nerve roots . NB:Tarlov cysts noted mainly at Rt side",0,1.0
230,"LSS MRI L Diffusely reduced signal of the bone marrow noted , correalte with bone isotope scan ?. faeture of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and both nerve roots .",0,1.0
231,"Lumbosacral MRI Diffuse disc bulge noted at L3-L4 level significantly compressing the thecal sac ,extending to Lt lateral recess    and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 level mild compressing the thecal sac nerve roots",0,1.0
232,"LSS MRI L4-L5: Mild spondylolisthesis noted with diffuse disc bulge, compressing the thecal sac and exit canals with narrow spinal canal. diffuse disc bulges noted also at L3-L4 and L2-L3 levels..",0,1.0
233,"Feature of muscle spasm.  Multilevel dissecating disc materials.  Diffuse disc bulge noted at L3/L4 level , compressing thecal and exit nerve roots more to Rt .  Diffuse disc bulge noted at L4/L5 compressing the thecal sac and exit nerves root more to Lt .",0,1.0
234,D SPINE MRI Normal bony alignment. No significant spinal cord or nerve root compression noted..,1,0.0
235,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac.  the spinal canal appeared adequate.",0,1.0
236,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and both nerve roots",0,1.0
237,"MRI OF THE LUMBAR SPINE :  Feature of muscle spasm.  L4/L5 level: central and Rt paracentral disc protrusion noted ,significantly compressing the thecal sac and exit neural canals, associated with annular tear .  L5/S1 level: Rt paracentral disc protrusion noted compressing the thecal sac and exit neural canals.",0,1.0
238,"LSS MRI :  feature of muscle spasm.  Small focal central disc protrusion noted at L5/S1 level, mild compressing thecal sac .",0,1.0
239,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1.0
240,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolisthesis of L5 over S1 level  Dissicating disc materails noted,  Multilevel diffuse disc bulges, notceably at L4/L5 level,compressing thecal sac and nerve roots .  About 3*1.5 cm, Tarlov cyst noted at level of S3 , with impression on posterior aspect.",0,1.0
241,"LSS MRI L5-S1: Diffuse disc bulge noted, compressing the thecal sac and exit canals, sligthly migrating downward. mild spondylolisthesis noted. L4-L5: Lt paracentral disc protrusion noted, compressing the thecal sac and enroaching upon Lt exit canal. mild disc bulge noted at L1-L2 level.",0,1.0
242,"LSS MRI:  Feature of muscle spasm.  Dissicating disc materals at multilevels.  Endpalte modec type II noted  Large L2 veretbral body hemangioma as well assmaller one s noted.  Posteriocentral disc protruion noted at L5/S1 level, compressing thecal sac and encroaching both neural canals.  Diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots.",0,1.0
243,"L4-L5: Lt paracentral disc protrusion, compressing the thecal sac  T12-L1: Diffuse disc bulge noted abutting the cord",0,1.0
244,Multilevel mild disc bulges noted abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.,0,1.0
245,"MRI OF THE LSS  - feature of muscle spasm.  Small Lt foraminal disc protrusion noted at L4-L5 level , compressing the Lt sided nerves canals.",0,1.0
246,LSS MRI :  Feature of musclespasm.  No significant thecal sac or nerve roots compression seen,0,1.0
247,"LSS MRI : -Feature of muscle spasm. -Dissicating disc material at lower levels. -Central disc protrusion noted at L4/L5 level, mild caudally migration, compressing thecal sac and both nerve rootys . -Central and to the the Rt  disc protrusion noted at L5/S1 level, compressing thecal sac and nerve roots",0,1.0
248,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit neural canals.",0,1.0
249,Cervical spine MRI  Features of muscle spasm  No significant thecal sac or nerve root compression  Normal signal intensity of the spinal cord  Lumbosacral MRI  Small vertebral hemangioma of L3  Diffuse disc bulge noted at the L4-L5 mild comparison thecal sac,0,1.0
250,L3-L4: Narrowed disc space with mild disc bulge. L4-L5: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal.,0,1.0
251,Lumbosacral MRI Feature of muscle spasm noted Small posterior central disc protrusion noted at L5-S1 mildly indenting thecal sac and encroaching exiting neural canal Diffuse disc bulge noted at L4-L5 mild compressing the thecal sac and the needle roots,0,1.0
252,"LSS MRI : Feature of muscle spasm. Central and to left disc protrusion noted at L2/L3 level, compressing thecal sac and encroaching exit neural canal . Multilevel diffuse disc bulges noted, noteceably at L4/L5&L5/S1 levels , compressing thecal sac and both nerve roots NB:Cystic dilaition of Rt pelvicalyceal syetems ? correalte with renal US",0,1.0
253,"LSS MRI : feature of muscle spasm. Diffuse disc bulges noted   L4/L5 & L5/S1 levels , mild compressing thecal sac and nerve roots",0,1.0
254,"LSS MRI  Diffuse disc bulge noted at L5-S1 level, more to the Rt side, compressing the thecal sac and Rt exit canal.",0,1.0
255,LSS MRI Small central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1.0
256,"MRI OF THE L. SPINE  Feature of muscle spasm.Desiccating disc materials.  Diffuse disc bulge noted at L4/L5 & L5/S1 levels , mild compressing thecal sac and both exit nerve roots, annular tear.",0,1.0
257,"L4-L5 AND L5-S1: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1.0
258,L5-S1: Central disc protrusion noted compressing the thecal sac and abutting S1 nerve roots.,0,1.0
259,"LSS MRI : -Feature of muscle spasm. -wide base disc bulge noted at L5/S1 level, compressing the theca sac and both nerve roots . -Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroching exit neural canal.",0,1.0
260,LSS MRI Feature of muscle spasm Small posterior central disc protrusion noted at L5-S1 level compressing thecal sac and exit  nerve roots(dehydrated disc matreial noted) Diffuse disc bulge noted at the L4-L5 mild compressing thecal sac and encroaching both neural canals,0,1.0
261,"LSS MRI : Feature of muscle spasm. Partial loss of L1 vertebral body hieght  anteriorly , with no bone marrow edema , mostly old fracture, with no definite thecal sac or exit neural canal compression . -Focal bony bruise noted at S3-S4 vertebral bodies? post traumatic  -Mild diffuse disc bulge noted at L4/L5 level , mild compressing thecal and exit nerve roots ,",0,1.0
262,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -L1 vertebral body hemangioma noted  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing the thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  NB:  Small size Rt kidney (correalte with US )",0,1.0
263,MRI OF THE LSS  Feature of muscle spasm.  Dissecating disc material L5/S 1  L4/L5& L5-S1 levels : diffuse disc bulge noted compressing the thecal sac and exit neural canals associated with annular tear .,0,1.0
264,"LSS MRI : -Feature of muscle spasm. -small vertebral body hemangioma of L4 . -Central and Lt paracentral disc protruion, noted at L4/L5 level , migrated caudally , compressing thecal sac and Lt nerve roots.",0,1.0
265,LSS MRI :  Diffuse disc bulge noted at L3/L4&&L4/S1 level mild compressing thecal sac and exit nerve roots .,0,1.0
266,LSS MRI L4-L5: Diffuse disc bulge noted compressing the thecal sac and exit canals.  L3-L4: Mild disc bulge noted.  Adequate spinal canal,0,1.0
267,"Multilevel mild disc bulges noted, largest at L4-L5 level, where it encroaches the exit neural canals.",0,1.0
268,"Multilevel diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1.0
269,"Multilevel mild disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level (Lt side), abutting the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1.0
270,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0.0
271,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, compressing the  thecal sac and encroaching both exit neural canal , more to Lt sided.",0,1.0
272,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5&L5/S1 levels, mild compressing thecal sac  and both nerve roots",0,1.0
273,LSS MRI  Feature of muscle spasm.  Mild disc bulge noted at L4-L5 level.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,0,1.0
274,"LSS MRI  Multilevel diffuse disc bulges, abutting the thecal sac and exit canals.- L3-L4 & L4-L5.  no significant thecal sac or nerve root compression.  Adequate spinaL canal.",0,1.0
275,LSS MRI :  Feature of muscle spasm.  no significant thecal sac or nerve roots compression .  Tarlov cyst noted at level of S1/S2 level,0,1.0
276,LSS MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,0,1.0
277,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted. -Diffuse disc bulge with small Lt paracentral disc protrusion noted at L2/L3 level, compressing the thecal sac and exit nerve roots . -Diffuse disc bulges noted at L3/L4& L4/L5 levels , compressing the thecal sac and encroaching upon exit neural canals .",0,1.0
278,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0.0
279,"LSS MRI :  Central and Lt paracentral disc protrusion noted at L5/S1 level , largely compressing the thecal sac and S1 Lt exit nerve root.  NB:  small tarlov cyst noted at S2 level .",0,1.0
280,Lumbosacral MRI  L3-L4  Diffuse disc bulge with annular tear more to the right side compressing the thecal sac  L4-L5  Diffuse disc bulge with left paracentral and foraminal protrusion compressing the thecal sac and exit neural canals  L5-S1  Diffuse bulge with annular tear compressing the thecal sac and exit neural canals more to the right side,0,1.0
281,"LSS MRI:  Muscle spasm noted.  Desiccated disc material noted  Multilevel diffuse disc bulge noted, notecibly at L3-L4 and L4-L5 Mildly compressing the thecal sac and significantly encroaching exiting neural canals",0,1.0
282,"LSS MRI  L5-S1: diffuse disc bulge, more to the Lt side.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit canals.  about 1.3 cm, Lt paracentral soft tissue fullness noted posterior to lower L4 vertebra, probably detached disc material. Contrast re-evaluation is recommended.",0,1.0
283,Lumbosacral MRI  No evidence of disc Herniation  No significant thecal sac or nerve root compression noted,1,0.0
284,Lumbosacral MRI Features of muscle spasm No significant thecal sac or nerve root compression No disc protrusion or herniation noted,0,1.0
285,LSS MRI : Feature of muscle spasm. Adequate  thecal sac. No significant  disc protrusion or herniation seen .,0,1.0
286,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression.  No disc protrusion or herniation seen,0,1.0
287,"LSS MRI : Feature of muscle spasm. Multiple vertebral bodies hemangiomas noted . Diffuse disc bulge noted at L4/L5 level, compression the thecal sac and encroaching both exit neural canals.associated with ligamentum flavum hyperatrophy . .Diffuse disc bulge noted at L3/L4 level, mild compressing thecall sac and nerve roots .",0,1.0
288,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 & L5/S1 levels , mildly compressing the thecal sac and encroaching both nerve roots .",0,1.0
289,"LSS MRI  Small Rt paracentral disc protrusion noted at L5-S1 level, mildly compressing the thecal sac.  adequate spinal canal",0,1.0
290,LSS MRI :  Grade I spondylolithesis of L4 over L5 level noted with end palate degenertaive change.,0,1.0
291,"LSS MRI WITH CONTRAST  Previous surgery noted.  mild Rt paracentral inhomogenous enhancement noted at L4-L5 level, mostly postoperative granulatiobn/fibrous tissue.",0,1.0
292,LSS MRI  Mild disc bulge noted at L4-L5 level.  No significant thecal sac or nerve root compression noted.,0,1.0
293,"LSS MRI :  Feature of muscle spasm.  Adequate thecal sac.  No significant disc protrusion or herniation seen .  Diffuse disc bulge noted at D11/D12, obliterating anterior CSf spaces .",0,1.0
294,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals.,0,1.0
295,MRI OF THE LSS  Feature of muscle spasm.  Dissicating disc at L4/L5 level  Lt foraminal disc protrusion noted at L4-L5 level compressing Lt sided (thecal sac and exit nerve canal),0,1.0
296,LSS MRI  L2 hemngioma.  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0.0
297,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals. L3-L4: Diffuse disc bulge noted abutting the thecal sac",0,1.0
298,LSS MRI  No significant changes noted since 04/01/2016,0,1.0
299,L4-L5: Diffuse disc bulge compressing the thecal sac and encroaching exit canals.,0,1.0
300,No evidence of disc herniation.   No significant thecal sac or nerve root compression noted,1,0.0
301,"LSS MRI :  Preveius surgery noted at lower lumbar spine.  Degenerative spondolytic changes .  Lt paracentral disc protrusion noted at L3/L4 level, compressing thecal sac and encroaching exit neural canal.  -Small abnormal signal alteration noted just posterior to L4 vertebral body appeared low on T1&T2 WI, compressing thecal sac and encroaching upon exit neural canal?appearances suggetsive sequestrated disc , calcifiaction ...ect, contrast study is advsied .  -Narrowed L5/S1 level with irrigular end plates and vacuume phenomina .",0,1.0
302,"LSS MRI L4-L5: Diffuse disc bulge noted, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L3-L4: diffuse disc bulge noted compressing the thecal sac and exit canals.  multilevel disc bulges noted elswere with end plate degeneration and schmorls nodules.",0,1.0
303,"LSS MRI :  Feature of muscle spasm.  Lt paracentral wide base disc bulge noted at L4/L5 level , compressing the Rt sided thecal sac and exit nerve root",0,1.0
304,"LSS MRI :  -Multilevel dissicating disc materials noted .  -Degenrative spondolytic chnages  -Diffuse disc bulges noted mostly at L4/L5 level , compressing the thecal sac and encroaching exit neural canals associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis .  -Diffuse disc bulges noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1.0
305,Lumbosacral MRI  Mild disc bulges noted at L4-5 and L5-S1 levels  No significant thecal sac or nerve root compression noted  Multilevel Schmorl's nodules noted  Adequate spinal canal,0,1.0
306,"LSS MRI  mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  the spinal canal appeared adquate.",0,1.0
307,LSS MRI :  Feature of muscle spasm.  No definite thecal sca or nerve roots compression noted .  No disc herniation or protrusion noted .,0,1.0
308,LSS MRI :  Feature of muscle spasm noted  Central and left paracentral disc protrusion noted at L3-L4 and L4-5 disc space compressing the thecal sac and exite nerve roots  Dissicating disc material noted,0,1.0
309,"L2-L3, L3-L4, L4-L5: diffuse disc bulges compressing the thecal sac and encroaching exit canals.",0,1.0
310,"LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1.0
311,L5-S1: Rt paracentral disc protrusion compressing the thecal sac and Rt exit canal. L4-L5: Diffuse disc bulge noted compressing the thecal sac and encroaching exit canals.,0,1.0
312,Previous surgery noted with transpedicle fixation. Enhancing granulation tissue compressing the thecal sac.,0,1.0
313,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, abitting thecal sac.",0,1.0
314,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0.0
315,"LSS MRI  Broad based central disc protrusion noted at L4-L5 level, largely compressng the thecal sac and encroaching upon the exit neural canals.  Diffuse disc bulge noted at L3-L4 level, compressing the thecal sac and exit canals.",0,1.0
316,"LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted mainly at L3/L4 & L4/L5 level, mildly compressing the thecal sac and exit nerve.",0,1.0
317,LSS MRI  dehydrated L4-L5 discal material with features of muscle spasm.  no evidence of disc herniation.  no significant thecal sac or nerve root compression noted.,0,1.0
318,L5-S1: diffuse disc bulge compressing the thecal sac and exit canals.,0,1.0
319,LSS MRI: Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1.0
320,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1.0
321,"LSS MRI  L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals.  L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1.0
322,"LSS MRI :  Feature of muscle spasm.  Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan.  Adequate thecal sac with no exit nerve root compression .  No evidence of disc protrusion or herniation seen .",0,1.0
323,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1.0
324,"LSS MRI  L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate.  large anterior bulge of the same abovementioned disc noted.  posterior osteophytes noted.  Type I end plate degeneration noted.",0,1.0
325,LSS MRI :  Feature of muscle spasm.  No significant thecal sac or nerve roots compression noted .  No disc protrusion or herniation seen .,1,0.0
326,Lumbosacral MRI  Features of muscle spasm  Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1.0
327,LSS MRI :  feature of muscle spasm.  adequate thecal sac.  No significant disc protruion seen,1,0.0
328,"LSS MRI  Central disc protrusion noted at L5-S1 level, compressing the thecal sac  the spinal canal appeared adquate.",0,1.0
329,"LSS MRI :  Feature of muscle spasm.  There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised .  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1.0
330,"LSS MRI :  Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1.0
331,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1.0
332,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1.0
333,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1.0
334,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1.0
335,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1.0
336,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1.0
337,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1 0,1,
338,Lumbosacral MRI  Mild disc bulges noted at L3-L4 and L4-L5 levels  Compressing the thecal sac and encroaching exit neural canals  Facet joint hypertrophy noted at L5-S1,0,1.0
339,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.  Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal.  NB:Multiple small tarlovs cysts noted",0,1.0
340,"LSS MRI  Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root.  adequate spinal canal.",0,1.0
341,MRI OF THE LUMBOSACRAL SPINE  Feature of muscle spasm.  Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1.0
342,"LSS MRI :  Feature of muscle spasm.  Vertebral bodies hemangioma noted at L2&L3 .  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1.0
343,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1.0
344,"LSS MRI  L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal.  L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals.  L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1.0
345,"LSS MRI :  -Feature of muscle spasm.  -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan.  -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal.  Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1.0
346,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1.0
347,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression.,1,0.0
348,"LSS MRI :  There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1.0
349,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1.0
350,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1.0
351,"Feature of muscle spasm.  L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1.0
352,MRI LSS  Feature of muscle spasm  Mild diffuse disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted,1,0.0
353,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
354,"LSS MRI :  feature of muscle spasm.  schorls nodules noted .  Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1.0
355,"LSS MRI :  -Feature of muscle spasm.  -Dgenerative spondolytic changes  -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals.  -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1.0
356,LSS MRI :  faeture of muscle spasm.  No definite thecal sac or nerev root compression .  Small tarlov cyst noted,1,0.0
357,LSS MRI  No evidence of disc herniation.  No thecal sac or nerve root compression noted.,1,0.0
358,LSS MRI  Multilevel mild disc bulges noted.  Adequate spinal canal.  No significant thecal sac or nerve root compression noted.,1,0.0
359,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1.0
360,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1.0
361,"LSS MRI :  Dissicating disc materials.  Grdae I spondylolisthesis of L4 over L5 level.  widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1.0
362,Lumbosacral MRI  Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1.0
363,"LSS MRI :  Feature of muscle spasm.  Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted.  multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots .  scoliosis of Lumbar spine noted, convexity to Ltside",0,1.0
364,LSS MRI Features of muscle spasm noted. Mild disc bulge noted at L5-S1 level. Adequate spinal canal.,0,1.0
365,"Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals, largest at L4-L5 level.",0,1.0
366,"LSS MRI L4-L5: diffuse disc bulge noted, more to the Rt side, compressing the thecal sac and exit neural canals. L3-L4 & L5-S1:diffuse disc bulges noted, compressing the thecal sac and exit neural canals.",0,1.0
367,"LSS MRI : Feature of muscle spasm. Diffuse decreased bone marrow signals intensity noted , correalte with bone isotope scan. Adequate thecal sac with no exit nerve root compression . No evidence of disc protrusion or herniation seen .",0,1.0
368,L4-L5: Central disc protrusion compressing the thecal  sac. L5-S1: Lt paracentral disc protrusion compressing the thecal sac.,0,1.0
369,"LSS MRI L5-S1: diffuse disc bulge noted, compressing the thecal sac and both exit canals. the spinal canal appeared adequate. large anterior bulge of the same abovementioned disc noted. posterior osteophytes noted. Type I end plate degeneration noted.",0,1.0
370,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve roots compression noted . No disc protrusion or herniation seen .,1,0.0
371,Lumbosacral MRI Features of muscle spasm Diffuse disc bulge noted at L4-L5 compressing the thecal sac and encroachment both nerve roots more to the left side,0,1.0
372,LSS MRI : feature of muscle spasm. adequate thecal sac. No significant disc protruion seen,1,0.0
373,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac the spinal canal appeared adquate.",0,1.0
374,"LSS MRI : Feature of muscle spasm. There is posterio-central abnormal extra axial soft tissue lesion noted at level of L3/L4, compressing thecal sac ? sequestrated disc , neuroma.., others .LSS MRI with contrast study is advised . Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both exit nerve roots",0,1.0
375,"LSS MRI : Lt paracentral disc protrusion -sequestrated noted at L4/L5 level ,caudally migrating , compressing the Lt sided thecal sac and exit nerve root , associated with dissicating disc materials",0,1.0
376,Mild disc bulge noted at L4-L5 level noted more to the Rt side. Adequate spinal canal.,0,1.0
377,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and both nerve roots",0,1.0
378,L4-L5 and L5-S1: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1.0
379,"L3-L4, L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit canals.",0,1.0
380,"L4-L5 and L5-S1: diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1.0
381,Multilevel diffuse disc bulges compressing the thecal sac and exit neural canals,0,1.0
382,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1.0
383,Lumbosacral MRI Mild disc bulges noted at L3-L4 and L4-L5 levels Compressing the thecal sac and encroaching exit neural canals Facet joint hypertrophy noted at L5-S1,0,1.0
384,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge with posteriocentral disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal. Diffuse disc bulge ,noted at L3/L4& L4/L5 levels, compressing thecal sac and encroaching exit neural canal. NB:Multiple small tarlovs cysts noted",0,1.0
385,"LSS MRI Small, Rt paracentral disc protrusion noted at L5-S1 level, abutting Rt S1 nerve root. adequate spinal canal.",0,1.0
386,MRI OF THE LUMBOSACRAL SPINE Feature of muscle spasm. Wide base Central disc bulges noted atL4/L5 level compressing thecal sac and encroaching exit nerves roots,0,1.0
387,"LSS MRI : Feature of muscle spasm. Vertebral bodies hemangioma noted at L2&L3 . Diffuse disc bulges noted at L3/L4 & L4/L5 levels, mildly compressing the thecal sac and exit neural canals.",0,1.0
388,Mild disc bulge noted at L4-L5 level. Adequate spinal canal.,0,1.0
389,"LSS MRI L3-L4: Diffuse disc bulge, more to the Rt side, compressing the thecal sac and Rt exit neural canal. L4-L5: Diffuse disc bulge, compressing the thecal sac and exit neural canals. L5-S1: Broad-based disc protrusion noted, more to the Lt side, compressing the thecal sac and Lt exit canal.",0,1.0
390,"LSS MRI : -Feature of muscle spasm. -Generalised decreased bone marrow signals noted ? coreralte with bone marrow isotope scan. -Central disc protrusion noted at L5/S1 level, migrated caudally ,slightly to right side, largely compressing the thecal sac and encroaching exit neural canal. Mild disc bulge noted at L4/L5 level, mildy compressing thecal sac",0,1.0
391,"LSS MRI : Feature of muscle spasm. Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and nerve roots",0,1.0
392,LSS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression.,1,0.0
393,"LSS MRI : There is significant posterior central disc protruion noted at L1/L2 level, largely compressing and filling thecal sac with encroaching exit nerve roots",0,1.0
394,"L4-L5: diffuse disc bulge noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1.0
395,Mild disc bulge noted at L4-L5 level. Adequate spinal canal,0,1.0
396,"Feature of muscle spasm. L4/L5 level: diffuse disc bulge noted , mild compressing the thecal sac and exit neural canals.",0,1.0
397,MRI LSS Feature of muscle spasm Mild diffuse disc bulge noted at L4-L5 level No significant thecal sac or nerve root compression noted,1,0.0
398,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
399,"LSS MRI : feature of muscle spasm. schorls nodules noted . Diffuse disc bulges noted at L4/L5 level , compressing thecal sac extending to lateral recess with encroaching exit neural canals , with dehydrated disc material .",0,1.0
400,"LSS MRI : -Feature of muscle spasm. -Dgenerative spondolytic changes -Rt paracentral disc protrusion noted at L4/L5 level, compressing the thecal sac and exit neural canals. -Diffuse disc bulges noted at L3/L4 level, mild compresing the thecal cac and both nerve roots",0,1.0
401,LSS MRI : faeture of muscle spasm. No definite thecal sac or nerev root compression . Small tarlov cyst noted,1,0.0
402,LSS MRI No evidence of disc herniation. No thecal sac or nerve root compression noted.,1,0.0
403,LSS MRI Multilevel mild disc bulges noted. Adequate spinal canal. No significant thecal sac or nerve root compression noted.,1,0.0
404,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL.,0,1.0
405,"LSS MRI : Feature of muscle spasm. Diffuse disc bulges noted at L4/L5 &L5/S1 levels, mild compressing thecal sac and both nerve roots .",0,1.0
406,"LSS MRI : Dissicating disc materials. Grdae I spondylolisthesis of L4 over L5 level. widw base disc bulge noted at L4/L5 level, compressing thecal sac and exit nerve roots .",0,1.0
407,Lumbosacral MRI Left foraminal disc protrusion noted at L5-S1 compressing the thecal sac and Lt S1 nerve root,0,1.0
408,"LSS MRI : Feature of muscle spasm. Grade I spondylolithesis of L3 over L4 vertebral body as well as modec type I noted. multilevel diffuse disc bulges noted , noceably at L4/L5 level, compressing tehcal sac and both nerve roots . scoliosis of Lumbar spine noted, convexity to Ltside",0,1.0
409,"LSS MRI  fearture of muscle spasm.  L5/S1 level :  Central disc protrusion noted , migrateing caudally , more to Lt side, largely compressing the thecal sac and exit nerve roots  dissicating disc materials .  Modic type II noted .",0,1.0
410,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0.0
411,"LSS MRI :  Feature of Muscle spasm.  Diffuse disc bulges noted noticeably at L3/L4/&L4/L5 levels, compressing thecal sac and encroaqching both exit neural canals, associated with ligamentum flavum hyperatrophy at L4/L5 level.",0,1.0
412,"LSS MRI  Multilevel diffuse disc bulges (L2-L3, L3-L4, L4-L5), compressing the thecal sac and exit canals.  multilevel end plate degenerative changes",0,1.0
413,LSS MRI :  feature of muscle spasm.  Mild diffuse disc bulge noted at L5/S1 level mildly compressing thecal sac,0,1.0
414,"LSS MRI  Mild disc bulges noted at L4-L5 and L5-S1 levels, abutting the thecal sac.  Adequate spinal canal.",0,1.0
415,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
416,"LSS MRI :  faeture of muscle spasm.  L4/L5 level :(disscating disc material ):  Central and Lt paracentral disc protruion migrating dawnwoard ,compressing thecal sac and Lt exit nerve roots .",0,1.0
417,"Compression fracture of L5 vertebra with retrobulge into spinal canal, compressing the thecal sac.",0,1.0
418,Lumbosacral MRI  Central disc protrusion noted at L4-L5 level compressing the thecal sac  Mild disc bulge noted at L5-S1 level  Adequate spinal canal  Features of muscle spasm noted,0,1.0
419,"LSS MRI :  wide base , Rt paracentral ,disc bulge noted at L4/L5 level , compressing thecal sac and Rt exit nerve root.  posterior central disc protrusion noted at L5/S1 level , with mild cephalic migration, compressing the thecal sac and encroaching exit neuralcanal.  NB:Lt Pelvic kidney noted , correalte with ultrasound",0,1.0
420,LSS MRI  No evidence of disc herniation noted.  No thecal sac or nerve root compression noted.,1,0.0
421,"LSS MRI :  faetuer of muscle spasm.  Diffuse disc bulge noted at L4/L5 level, extending to lateral recess, compressing thecal sac and exit neural canals with annular tear .",0,1.0
422,"L3-L4, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, noticeably at L4-L5 level where mild spondylolisthesis noted, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1.0
423,"LSS MRI  L5-S1: Mild disc bulge with Lt paracentral annular tear, compressing the thecal sac and encroaching upon Lt exit neural canal.  L4-L5: Mild central disc protrusion noted compressing the thecal sac.  L3-L4: Mild disc bulge with posterocentral tear.",0,1.0
424,"LSS MRI  Small central disc protrusions noted at L4-L5 amnd L5-S1 levels, compressing the thecal sac.  adequate spinal canal.",0,1.0
425,L3-L4 and L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals L5-S1: Mild disc bulge noted.,0,1.0
426,LSS MRI :  Feature of muscle spasm.  No significant disc protrusion or herniation seen.  No thecal sac or exit nerve root compression noted .  Preserved vertebral bodies hieght and bone marrow signal intensity seen.,1,0.0
427,L4-L5: Mild spondylolisthesis with secondary disc bulge abutting the thecal sac and encroaching exit canals. L1-L2: Posterior osteophytes noted abutting the thecal sac,0,1.0
428,"LSS MRI  Mild disc bulge noted at L4-L5 level, abutting the thecal sac and encroaching upon the exit canals.  L2 hemangioma.  3 cm, Lt ovarian cyst.",0,1.0
429,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L3/L4 & L4/L5 levels, compressing thecal sac and both nerve roots .",0,1.0
430,"LSS MRI :  Cetral & Rt paracentral disc protrusion noted at L4/L5 level , largely compressing the Rt sided thecal sac and exit nerve root.",0,1.0
431,"L4-L5: diffuse disc bulge noted, compressing the thecal sac and exit canals.",0,1.0
432,"LSS MRI :  Lt paracentral disc protrusion noted at L4/L5 level, largely compressing (Lt) thecal sca and exit nerve root",0,1.0
433,"LSS MRI :  .Feature of muscle spasm.  .posterior central disc protrusion noted at L4/L5 level, largely compressing the thecal sac and enrcoaching exit neural canal.",0,1.0
434,"C.SPINE MRI :  Feature of muscle spasm.  Wide base disc bulge noted at C5/C6 level, obliterating anterior CSF spaces , and encroachong exit neural canal.  LSS MRI :  Feature of muscle spasm.  Dehydrated disc materials at lower levels.  Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and encroaching exit neural canal with annular tear .",0,1.0
435,"LSS MRI :  feature of muscle spasm.  multiple schmorls nodules .  dissicating disc material at L5/S1 level.  Rt paracentral disc protruion noted at L5/S1 level ,compression thecal sac and Rt exit nerve root.",0,1.0
436,"LSS MRI :  FEATure of muscle spasm.  Diffuse disc bulge with Lt para central disc protrusion & sequestration , noted at L5/S1, largely compressing the thecal sac and encroaching exit neural canal, with narrowed disc space and dehydrated disc material.",0,1.0
437,LSS MRI  diffuse disc bulges noted at L4-L5 levels compressing the thecal sac.  mild spondylolisthesis of L4 over L5 vertebrae.,0,1.0
438,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulge noted at L4/L5 level , mild compresing thecal sac and nerve roots",0,1.0
439,"LSS MRI :  faeture of muscle spasm.  wide base disc bulge ,with annular tear noted at L5/S1 level, compressing teh thecal sac and both nerve roots",0,1.0
440,Lumbosacral MRI  Features of muscle spasm  No significant thecal sac or nerve root compression,1,0.0
441,Lumbosacral MRI  No evidence of disc herniation  No thecal sac or nerve root compression  Adequate spinal canal,1,0.0
442,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing the thecal sac and encroaching exit neural canals .",0,1.0
443,"LSS MRI :  feature of muscle spasm.  diffuse disc bulges noted at L4/L5 levels, extending to lateral recess, compressing the thecal sac and both nerve roots",0,1.0
444,"LSS MRI  Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and encroaching upon exit canals.  mild disc bulge noted at L5-S1 level.",0,1.0
445,"Lumbosacral MRI:  Feature of muscle spasm  Dissicating lower lumbar disc material noted  L4-L5 level:  Central and right paracentral disc protrusion noted with cephalic migration as well as soft tissue density noted mostly sequestrated disc,  Compressing the thecal sac and encroaching the right exit neural canals.  L5-S1 level:  Wide based disc bulge noted compressing the thecal sac and both nerve root.  Contrast study LSS is advised",0,1.0
446,"LSS MRI  L4-L5: diffuse disc bulge, more to the Lt side, compressing the thecal sac and encroaching upon Lt exit canal.  L5-S1: Mild disc bulge noted.",0,1.0
447,"LSS MRI :  Feature of muscle spasm.-  -multiple schmorls nodules noted  -Dissicating disc materials at lower levels.  -posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and exit nerve roots more to Lt side.  -Diffuse disc bulge with small central protrusion noted at L4/L5 level compressing thecal sac and both nerve roots .",0,1.0
448,"LSS MRI :  Feature of muscle spasm.  Wide base disc bulge noted at L5/S1 level, compressing thecal sac and encroaching exit neural canals.  SI JOINTS MRI:  Normal smooth articular surface of both scaro iliac joints  Normal bone marrow signals intensities.  No bony erosion or peri-osteal reaction.  Intact visualised muscles",0,1.0
449,"LSS MRI :  posterior central disc protrusion noted at L5/S1 level, crossing & migrating caudally to Rt thecal sac, largely compressing the thecal sac and Rt S1 nerve root.",0,1.0
450,"Nild diffuse disc bulge noted at L4/L5 level, mild compressing thecal sac and exit nerve root.  Featrure of muscle spam.",0,1.0
451,"LSS MRI :  -Feature of muscle spasm.  -Discating disc material with degeneraive changes  -Multilevel diffuse disc bulges noted , notceably at L3/L4 level , compressing  thecal sac and encroaching upon exit neural canals asoociated with ligementum flavum hyperatrophy , all leading to secandary spinal canal stenosis",0,1.0
452,"LSS MRI :  Evidnece of tranpedicular fixation screws noted at lower dorsal and upper lumbar spines with fractures verretbal bodies (post traumatic ).  -Diffuse disc bulges noted at L4/L5 level compressing thecal sac and both nerve roots associated with ligamnetum flavum hyperatrophy , all leading to secandary spinal canal stenosis, as well as and to lesser degree at L3/L4 level.  Grade I spondylolithesis of L4 over L5 level",0,1.0
453,LSS MRI :  feature of muscle spasm.  No thecal sac or nerve roots compression seen.  No significant disc protrusion or herniation noted .,1,0.0
454,Lumbosacral MRI  Mild disc bulge noted at L4-L5 level  No significant thecal sac or nerve root compression noted  Adequate spinal canal,1,0.0
455,"LSS MRI :  Feature of muscle spasm.  -Lt paracentral disc protrusion noted at L5/S1 level, compressing Lt thecal sac and exit nerve root.(dissicaring disc material noted )  -Wide base disc bulge noted at L4/L5 level, mildly compressing thecal sac and nerve roots",0,1.0
456,"Lumbosacral MRI  Central and left paracentral disc protrusion noted at L4/L5 level , largely compressing the thecal sac and encroaching upon exiting neural canals.",0,1.0
457,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material noted at L5/S1 level.  wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and encroaching upon exit neural canals",0,1.0
458,LSS MRI  Central disc protrusion noted at L5-S1 level compressing the thecal sac.,0,1.0
459,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0.0
460,"LSS MRI :  -Feature of muscle spasm.  -Multiple schmorls nodules.  -Diffuse disc bulge noted at L4/L5 level, extending to lateral recess , more to Rt sided ,compressing the thecal sac and both nerve roots .  -Diffuse disc bulges with small posterior central disc protrusion noted at L5/S1 level, compressing thecal sac and both nerve roots",0,1.0
461,"LSS MRI :  -Feature of muscle spasm.  -Degenerative spondylotic changes.  -Multilevel diffuse disc bulge noted, noteably at L3/L4 & L4/L5 level, compressing thecal sac and encroaching both exit neural canals associated with ligamnetum flavum hypertrophy , all leading releatively secandary spinal canal stenosis .  -Diffuse disc bulges with small Rt paracentral disc protrusion noted at L5/S1 level , compressing thecal sac and exit nerve roots",0,1.0
462,"LSS MRI :  feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 level, compressing thecal sac and both nerve roots .  Mild disc bulges noted at L5/S1 level.",0,1.0
463,"LSS MRI :  Feature of muscle spasm.  central slightly to Rt disc protrusion noted at L5/S1 , compressing Lt thecal sac and exit nerve root.",0,1.0
464,LSS MRI :  Feature of muscle spasm.  Mild diffuse disc bulge noted at L4/L5 level .  No significant thecal sac or nerve root compression notred,1,0.0
465,LSS MRI  No evidence of disc herniation noted.  No significant thecal sac or nerve root compression noted.,1,0.0
466,"LSS MRI  No evidence of disc herniation.  no thecal sac or nerve root compresion noted.  small hypointense T1 and T2 lesion noted, probably schmorls nodule",1,0.0
467,"LSS MRI  Mild Lt paracentral disc protrusion noted at L1-L2 level, compressing the thecal sac.  no evidence of disc herniation at other levels.",1,0.0
468,LSS MRI  No evidence of disc herniation.  No significant thecal sac or nerve root compression,1,0.0
469,LSS MRI:  feature of muscle spasm.  No significant thecal sac or nerve roots.  NB:relatively Decreased bone marrow signals noted ? correalte with bone isotope scan,1,0.0
470,L4-L5: Mild disc bulge noted. L5-S1: Mild Lt paracentral disc protrusion noted abutting the thecal sac and encroaching LT EXIT CANAL,0,1.0
471,"Cervical MRI :  -feature of muscle spasm.  -dehydrated discal materials.  -diffuse disc bulges noted at C5/C6 & C6/C7 levels , compressing anterior CSF spaces and encroaching upon both exit nerve canals.  LSS MRI :  -Diffuse disc bulge noted at L4/L5 level compressing the thecal sac and both exit nerve roots.  NB:the sacral canal is occupied by a central extradural and non divided cyst with fine wall and without neural element or herniation of meninges.The cyst is very expansive but doses not transgress margins of the sacrum  No connection is visualized between the cyst and the thecal sac.  There is no other significant abnormality.  appearances mostly represent occult intrasacral meningocele (OIM) or sacral meningeal cyst ..",0,1.0
472,"LSS MRI :  -Degenerative changes noted with dissicating disc materials.  -Multilevel diffuse disc bulges noted , noteceably at L4/L5 level and L2/L3 levels, compressing the thecal sca and encroaching upon exit neural canals , associated with ligamentum flavum hyperatrophy , relatively causing secandary spinal canal stnosis NB:  Small Rt renal cyst noted .",0,1.0
473,LSS MRI  No evidence of disc herniation.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.,1,0.0
474,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
475,"LSS MRI :  Feature of muscle spasm.  Diffuse disc bulges noted at L4/L5 & L5/S1 levels, compressing the thecal sac and encroaching both exit neural canals",0,1.0
476,"LSS MRI  Small central disc protrusion noted at L5-S1 level, compressing the thecal sac.  adequate spinal canal.",0,1.0
477,"LSS MRI :  Feature of Muscle spasm.  Modec Type I , noted at L5/S1 level.  Diffuse disc bulges noted L5/S1 level , compressing thecal sac and both exit nerve roots.",0,1.0
478,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1.0
479,LSS AND C. SPINE MRI  Features of muscle spasm noted in both c. spine and LSS.  mild disc bulge noted at L4-L5 level.,0,1.0
480,"L5-S1: Lt paracentral disc protrusion, compressing the thecal sac. L4-L5: Mild disc bulge with annular tear.",0,1.0
481,"LSS MRI :  Feature of muscle spasm  Relatively generalised decreased bone marrow signals ? correalte with bone isotope scan..  diffuse disc bulge, noted at L4/L5 level, compressing the thecal sac and exit neural canals",0,1.0
482,"LSS MRI :  -There is anterior wedging fracture of L5 vertebral body noted with bone marrow edema involving the rest of vertebral body extending to pedicles with evidence of paravertebral soft tissue componant with no posterior bulges ??fracture , however bone isotope scan is advised to rule out secandary depostis.  -Multlevel diffuse disc bulges noted , at L2/L3 , L3/L4 & L5/S1 levels ,compresiing the thecal sac and both exit nerve roots.  grade I spodylolisthesis of L5 over S1 level",0,1.0
483,"HIP JOINT & SIJ MRI :  Feature of Lt sided scaroilitis noted(infective = septic sacroiliatis ,TB, Brucellosis ...etc), correalte with lab results and needs clinical correaltion .  Preserved Both femoral head noted  LSS MRI :  Feature of muscle spasm .  No significant thecal sac or nerve roots compression seen .  Evidence of left sided gluteal signal alteration noted (old insult )",1,0.0
484,Lumbosacral MRI  Mild disc bulge noted at L3-L4 level  No evidence of disc Herniation at other levels noted  Adequate the spinal canal,1,0.0
485,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
486,"LSS MRI  diffuse disc bulge with annular tear noted at L4-L5 level, abutting the thecal sac.  no significant thecal sac or nerve root compression noted.  adequate spinal canal.",1,0.0
487,"LSS MRI  diffuse disc bulge, more to the Rt side, noted at L4-L5 level, compressing the thecal sac and Rt exit neural canal.",0,1.0
488,Lumbosacral MRI  Diffuse disc bulge noted at L4-L5 level  Mildly compressing the thecal sac and the exit neural canals  The spinal canal appears adequate,0,1.0
489,"Multilevel diffuse disc bulges compressing the thecal sac and encroaching exit neural canals, largest at L2-L3 level.",0,1.0
490,"LSS MRI :  -Feature of muscle spasm.  -Diffuse disc bulge noted at L5/S1 level, extending to lateral recess, more to Lt the sided the thecal sac and encraoching upon exit neural canals.  -Diffuse disc bulge noted at L4/L5 level, compressing the thecal sac and nerve roots associated with annular tear",0,1.0
491,"Lumbosacral MRI  Multilevel diffuse disc bulges noted at L5-S1, L4-L5 and L3-L4 levels  Compressing the thecal sac and exiting neural canals  the Spinal canal appears narrowed at the L4-L5 leve, By disc bulge and the facet joint hypertrophy  Multilevel endplate degeneration noted",0,1.0
492,"LSS MRI :  Feature of muscle spasm.  Dissicating disc material at L5/S1 level .  Diffuse disc bulge noted at L5/S1 level, compressing thecal sac and encroaching both exit neural canal.  Mild diffuse disc bulge noted at L4/L5 level mild comprssing thecal sac & nerve roots .",0,1.0
493,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0.0
494,LSS MRI  Mild disc bulge noted at L2-L3 level abutting the thecal sac.  features of muscle spasm.,0,1.0
495,"LSS MRI : feature of degenerative chaanges . Mutlilevel diffuse disc bulges notceably at L3/L4 &L4/L5 levels, compressing the thecal sac and encroaching exit neural canal associated with ligamtum flavum hyperatrophy , all leading to secandary spinal canal stenosis.",0,1.0
496,"LSS MRI : -Feature of muscle spasm. -Modic type II end plate degenerative changes of L4/L5 level noted . -Posterior central disc protrusion noted at L4/L5 level , compressing the thecal sca extending to lateral recess encroaching upon both exit neural canals . -Diffuse disc bulge with annular tear noted at L3/L4 level, mild compressing thecal sac and exit nerve roots NB :About 4 cm , ovarian cyst noted .",0,1.0
497,"LSS MRI : .Feature of muscle spasm. .dehydrated disc materials noted at lower levels . .wide base posterior central disc bulge with annular tear noted at L4/L5 level, extending to Lt lateral recess, compressing the thecal sac and encroaching exit neural canal. .Mild diffuse dic bulges noted at L3/L4 & L5/mpressing the thecal sac",0,1.0
498,"LSS MRI : Feature of muscle spasm. Central and Lt paracentral disc prortusion noted at L5/S1 level, compressing the thecal sac and Lt (S1) exit nerve root.",0,1.0
499,No evidence of disc herniation. No significant thecal sac or nerve root compression noted,1,0.0
500,"LSS MRI Central disc protrusion noted at L5-S1 level, compressing the thecal sac. features of muscle spasm.",0,1.0
501,"Lumbosacral MRI Central disc protrusion and more to the right side, noted at L4-L5 level, largely compressing the thecal sac and right exit neural canal Left paracentral disc protrusion noted at L2-L3 level, compressing the left side of the thecal sac",0,1.0
502,"LSS MRI : Feature of muscle spasm. Dissicating disc material at L4/L5 level: Diffuse disc bulge noted at L4/L5 level , extending to lateral recess, compressing thecal sac , and largely encroaching upon exit neural canal associated with annular tear .",0,1.0
503,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials at L4/L5 & L5/S1 levels Diffuse disc bulge noted at L4/L5 level, compressing the theca sac and encroaching upon exit neural canal , with annular tear . -Small posterior central disc bulge noted at L5/S1 level, mildly compressing the thecal sac and exit nerve roots",0,1.0
504,"LSS MRI : Feature of muscle spasm. Partial dissicating disc material noted at L4/L5 level Diffuse disc bulg noted at L4/L5 level, compressing thecal sac andd both nerve roots",0,1.0
505,"Lumbosacral MRI Feature of muscle spasm Desiccative disc material noted at the lower levels Multiple Schmorl's nodule noted Modic type II noted at the L5-S1 L4-L5 leve:l wide based disc bulge noted ,mildly compressing the thecal sac and exiting nerve root L5-S1 level: Diffuse disc bulge noted compressing the thecal sac and encroaching upon the neural canals",0,1.0
506,"LSS MRI : feature of muscle spasm. diffuse disc bulges noted at L4/L5& L5/S1 levels, mildly compressing the thecal sac and both exit nerve roots .",0,1.0
507,"MRI OF THE LUMBAR SPINE : Feature of muscle spasm. L4/L5 level: diffuse bulge noted, with annular tear mild compressing the thecal sac . No significant thecal sac compression . RT KNEE MRI : Tear of posterior horn of medial Meniscus. Joint effusion noted . No other abnormality seen",0,1.0
508,"Multilevel Diffuse disc bulges with facet joint hypertrophy noted compressing the thecal sac and exit neural canals, noticeably at L4-L5 level.",0,1.0
509,LSS MRI Mild diosc bulges noted at L4-L5 and L5-S1 levels. No sigificant thecal sac or nerve root compressuion noted. adequate spinal canal,0,1.0
510,LSS MRI Multilevel mild disc bulges noted. No significant thecal sac or nerve root compression noted. Tarlov cyst- S2 level.,0,1.0
511,"LSS MRI: Feature of muscle spasm. Diffuse disc bulges noted at L4/L5&L5/S1 level, mildly compressing thecal sac and exit nerve roots",0,1.0
512,"LSS MRI L5-S1: Rt paracentral disc protrusion, compressing the thecal sac and Rt exit neural canal. mild disc bulge noted at L4-L5 level.",0,1.0
513,LSS AND PELVIS MRI No evidence of disc herniation. No significant thecal sac or nerve root compression noted. preserved SI joints. features of PID noted.,1,0.0
514,"LSS MRI L4-L5: Diffuse disc bulge noted, compressing the thecal nsac and both exit canals. L5-S1: Mild disc bulge noted",0,1.0
515,L5-S1: Lt paracentral disc protrusion compressing the thecal sac and Lt exit neural canal.,0,1.0
516,"LSS MRI : Feature of muscle spasm. Multiple schomrls nodules Wide base disc bulge noted at L4/L5 level, compressing thecal sac more at Rt side with encroaching exit neural canal . Mild diffuse disc bulge noted at L5/S1 level, mild compressing thecal sca and both nerve roots",0,1.0
517,"C.SPINE MRI : feature of muscle spasm. Rt paracentral disc protrusion with annular tear noted at C5/C6 level, ablitertaing anterior CSF space with indenting spinal cord with mild encroaching exit neural canal. LSS MRI (contrast): Evidnece of previus opertaion at Rt sided (L4/L5 level): Central and RT paracentral inhomogenous enhancing struture noted at L4/L5 level, compressing thecal sac and Rt exit neural canal , mostly post opertaive fibrous -granulation tissue",0,1.0
518,LSS MRI no evidence of disc herniation. no thecal sac or nerve root compression.,1,0.0
519,"Lumbosacral MRI Features of muscle spasm Dissicating disc material noted Multilevel diffuse disc bulge noted mainly at L2-L3, L3-L4 and L4-L5 levels compressing thecal sac and encroaching exiting neural canals Relatively secondary spinal Stenosis noted at the L4-L5 Level",0,1.0
520,"LSS MRI : features of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and exit nerve roots. .diffusely reduced signal of bone marrow noted, mostly non-specific - however, correlate with bone isotope scan .",0,1.0
521,C4-C5 mild disc bulge noted compressing anterior CSF space with features iof m. spasm. L4-L5: Diffuse disc bulge compressing the thecal sac and exit canals.,0,1.0
522,"LSS MRI : Feature of muscle spasm.- -Dissicating disc materials noetd at L4/L5&L5/S1 levels. Posteriocentral disc protrusion noted at L5/S1 level, largely compressing the thecal and encroaching both neural canals. mild diffuse disc bulge noted at L4/L5 level.",0,1.0
523,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials .- -Rt para central disc protrotion , slight caudal migratition noted at L4/L5 level, compressing Rt thecal sac & L5 nerve root .",0,1.0
524,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1.0
525,"LSS MRI : Feature of muscle spasm. Wide base disc bulge noted at L5/S1 level, mildy compressing the thecal sac and exit nerve roots associated with annular tear",0,1.0
526,L4-l5: Diffuse disc bulge noted abutting the thecal sac and encroaching exit canals.,0,1.0
527,"LSS MRI : Mild diffuse disc bulge noted at L4/L5 level, mildly compressing thecal sac.",0,1.0
528,LSS MRI : Feature of muscle spasm. No significant thecal sac or nerve root compression seen . No disc herniation or protruion noted.,1,0.0
529,"L5-S1, L4-L5: Diffuse disc bulges noted with facet joint hypertrophy, compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate.",0,1.0
530,LSS MRI No evidence of disc herniation .No significant thecal sac or nerve root compression noted.,1,0.0
531,Lumbosacral MRI Mild disc bulge at the L5-S1 level Abutting the thecal sac No significant thecal sac or nerve root compression noted,0,1.0
532,"LSS MRI : Diffuse disc bulge with Central and to the Rt disc protrusion ,noted at L5/S1 level, compressing thecal sac and encroaching exit neural canal.",0,1.0
533,"LSS MRI : -Feature of muscle spasm. -Dissicating disc materials noted . -Narrowed disc space of L5/S1 level -Multilevel diffuse disc bulges noteceably at L4/L5 &L5/S1 levels, compressing the thecal sac and encroaching upon exit neural canals with ligamentum flavum hyperatrophy. NB:Small hypointense lesion noted at S2 vertebral body , mostly bony scleorois like bony island .",0,1.0
534,No evidence of disc herniation. No significant thecal sac or nerve root compression noted.,1,0.0
535,"LSS MRI : Feature of muscle spasm. Mild diffuse disc bulge noted at L4/L5 level, mild cpmpressing thecal sac and nervce roots . NB : -About 4 cm, sub mucosal fibroid noted .",0,1.0
536,Lumbosacral MRI There is inhomogeneously enhancing soft tissue in the left paracentral and foraminal area at L4-L5 level Mostly fibrosis compressing the thecal sac and left exit neural canal,0,1.0
537,"LSS MRI Features of muscle spasm. Mild disc bulge noted at L5-S1 level, abutting the thecal sac. No signicant thecal sac or nerve root compression noted.",0,1.0
538,L3-L4: Posterior osteophytes abutting the thecal sac with narrow disc space.,0,1.0
539,"LSS MRI Diffuse disc bulges noted at L3-L4 and L4-L5 levels, compressing the thecal sac and exit canals.",0,1.0
540,No evidence of disc herniation. No significant thecal sac or nerve root compression noted. Adequate spinal canal.,1,0.0
541,Mild disc bulge noted at L4-L5 and L1-L2 level. No significant thecal sac or nerve root compression noted.,0,1.0
542,Lumbosacral spine MRI No evidence of disc herniation No significant thecal sac or nerve root compression noted Adequate spinal canal,1,0.0
543,"LSS MRI : Feature of muscle spasm. .Central disc protrusion more to Rt sided noted at L5/S1 level ,largely compressing the thecal sac and exit nerve roots with narrowed disc space . .Diffuse disc bulge noted at L4/L5 level, mildly compressing the tehcal sac and both nerve roots associated with annular tear .",0,1.0
544,LSS MRI No evidence of disc herniation noted. No significant thecal sac or nerve root compression noted.,1,0.0
545,"LSS MRI : Feature of muscel spasm. Generalised decreased bone marrow signals intensities noted ? correalte with bone isotope scan. Mild diffuse disc bulges noted at L4/L5 level , abutting thecal sac .",0,1.0
546,L3-L4: Mild Rt paracentral disc protrusion abutting the thecal sac. L4-L5: Mild central disc protrusion abutting the thecal sac. L5-S1: Mild Lt paracentral disc protrusion abutting the thecal sac.,0,1.0
547,"LSS MRI : faeture of muscle spasm. L5/S1 level :(dissicating disc material ): Central disc protruion relatively upwatd migration ,mild compressing thecal sac and exit nerve root .",0,1.0
548,. MRI LSS Feature of muscle spasm Dissicating disc material noted at L4-L5 and L5-S1 wide base disc bulge noted at the L4-L5 level there is mild compressing thecal sac and encroaching the neural canals,0,1.0
549,Mild disc bulges noted at L23-L3 and L4-L5 levels.,0,1.0
550,"L4-L5: Diffuse disc bulge with Lt paracentral disc protrusion, compressing the thecal sac and encroaching Lt exit canal. L5-S1: Mild disc bulge noted.",0,1.0
551,Lumbosacral MRI Feature of muscle spasm Diffuse disc bulge noted at L3-L4 and L4-5 levels with mild compressing thecal sac and encroaching upon exiting neural canals Small vertebral bodies hemangioma noted,0,1.0
552,"LSS MRI Diffuse disc bulge noted at L4-L5 level, compressing the thecal sac and exit canals.",0,1.0
553,L4-L5: Diffuse disc bulges noted compressing the thecal sac and encroaching exit neural canals,0,1.0
554,"LSS MRI : Feature of muscle spasm . diffuse disc bulges noted at L4/L5 level, mild compressing thecal sac and nerve roots .",0,1.0
555,"LSS MRI Mutlivel diffuse disc bulges noted in the lower LSS, noticeably at L4-L5, compressing the thecal sac and encroaching upon exit canals",0,1.0
556,"L3-L4, L4-L5: Mild disc bulges noted compressing the thecal sac and encroaching exit canals. The spinal canal is still adequate",0,1.0
557,"LSS MRI : wedging fractures of L2& L4 vetrebal bodies , with bone marrow edema without significant thecal sac. Multilevel diffuse disc bulges notecibly at L4/L5 levels , compressing thecal sac and both exit nerves roots with relatively secandary spinal canal stenosis",0,1.0
558,"L3-L4, L4-L5 AND L5-S1: Diffuse disc bulges noted, compressing the thecal sac and exit canals.",0,1.0
559,"LSS MRI : Feature of muscle spasm. Preveius operation at L4/L5 level ( fenestration). Diffuse disc bulge noted at L4/L5 level, mildly compressing the thecal sac and encroaching upon exit neural canal with annular tear . .No definite abnormal enhancing lesion could be seen .",0,1.0
560,"MRI OF THE L. SPINE Feature of muscle spasm. Desiccating disc materials with degenerative changes . Diffuse disc bulge noted at L4/L5 ,with posterior osteophytes compressing thecal sac and both exit nerve roots, associated with ligamnetum flavum hypertrophy , all leading to secondary spinal canal stenosis . Diffuse disc bulge noted at L5/S1 level, mild compressing thecal sac and exit nerve roost D11/D12. showed ligamnetum flavum hyperatrophy",0,1.0
561,No abnormalities detected in the lumbar spine.,1,0.0
562,The MRI shows normal spinal alignment with no disc herniation.,1,0.0
563,There are no signs of nerve compression or other abnormalities.,1,0.0
564,The lumbar vertebrae are intact with no evidence of fractures.,1,0.0
565,No bulging or herniation of intervertebral discs noted.,1,0.0
566,No abnormal findings in the lumbar region on imaging.,1,0.0
567,The MRI reveals a normal spine with no pathological changes.,1,0.0
568,No signs of degenerative disc disease or spinal stenosis.,1,0.0
569,The vertebral bodies and intervertebral discs are normal.,1,0.0
570,There is no evidence of scoliosis or other deformities.,1,0.0
571,The spinal cord appears normal with no signs of injury.,1,0.0
572,There is no evidence of osteoarthritis in the lumbar spine.,1,0.0
573,The MRI shows normal curvature and alignment of the spine.,1,0.0
574,There are no abnormal disc spaces or vertebral body changes.,1,0.0
575,No signs of bone fractures or tumors in the lumbar region.,1,0.0
576,The intervertebral discs appear healthy and well-aligned.,1,0.0
577,There is no evidence of swelling or inflammation in the spine.,1,0.0
578,No signs of spinal cord compression or other abnormalities.,1,0.0
579,The MRI scan shows no signs of disc degeneration.,1,0.0
580,There is no bulging or rupturing of the intervertebral discs.,1,0.0
581,No pathological findings detected in the lumbar MRI.,1,0.0
582,The vertebrae in the lumbar spine are healthy and intact.,1,0.0
583,There is no evidence of herniation or spinal deformity.,1,0.0
584,Normal intervertebral disc spacing is observed in the MRI.,1,0.0
585,No signs of abnormal bone growth or spurs in the lumbar region.,1,0.0
586,The spine appears healthy with no signs of degeneration.,1,0.0
587,The MRI reveals a normal disc-to-vertebral body ratio.,1,0.0
588,There are no abnormal changes to the vertebral column.,1,0.0
589,No evidence of scoliosis or abnormal curvature in the lumbar spine.,1,0.0
590,The lumbar spine is structurally intact with no herniation.,1,0.0
591,The discs in the lumbar spine show no signs of bulging or prolapse.,1,0.0
592,The MRI confirms a normal and healthy lumbar spine.,1,0.0
593,There are no irregularities in the spinal alignment or bone structure.,1,0.0
594,No signs of inflammation in the lumbar intervertebral discs.,1,0.0
595,The MRI scan does not show any nerve compression.,1,0.0
596,The lumbar spine appears symmetrical with normal curvature.,1,0.0
597,No abnormalities in the soft tissues surrounding the spine.,1,0.0
598,There are no signs of narrowing of the spinal canal.,1,0.0
599,The MRI results show a normal lumbar vertebral body structure.,1,0.0
600,No signs of disc bulging or damage to the spinal cord.,1,0.0
601,The vertebral bodies are intact with no signs of fractures.,1,0.0
602,The spinal column appears normal with no deformities.,1,0.0
603,There are no lesions or masses in the lumbar region.,1,0.0
604,The lumbar discs are well-spaced and free from degeneration.,1,0.0
605,No abnormal bone formations are present in the MRI.,1,0.0
606,The MRI reveals a healthy spinal alignment with no evidence of damage.,1,0.0
607,There are no signs of disc herniation or prolapse in the lumbar spine.,1,0.0
608,The lumbar spine shows no evidence of wear and tear.,1,0.0
609,No abnormalities in the spine's bony structures.,1,0.0
610,The lumbar region shows normal joint and bone structures.,1,0.0
611,There are no unusual growths or abnormalities in the lumbar MRI.,1,0.0
612,The MRI confirms the absence of degenerative changes in the spine.,1,0.0
613,No evidence of narrowing or compression in the spinal canal.,1,0.0
614,The vertebral joints show no signs of arthritis or degeneration.,1,0.0
615,The lumbar MRI appears unremarkable with no pathological findings.,1,0.0
616,The lumbar spinal cord demonstrates no abnormal signal.,1,0.0
617,There is normal signal intensity in the intervertebral discs.,1,0.0
618,No evidence of disc desiccation or collapse.,1,0.0
619,No posterior disc protrusion is noted in any lumbar levels.,1,0.0
620,The lumbar vertebral alignment is within normal limits.,1,0.0
621,No mass effect is present within the spinal canal.,1,0.0
622,No abnormal enhancement detected post-contrast administration.,1,0.0
623,No abnormalities identified in paraspinal soft tissues.,1,0.0
624,The sacroiliac joints appear unremarkable.,1,0.0
625,No significant disc space narrowing is seen.,1,0.0
626,The bony structures appear normal in density and contour.,1,0.0
627,No evidence of spondylolisthesis in the lumbar region.,1,0.0
628,The thecal sac is normal in size without compression.,1,0.0
629,The lumbar neural foramina are widely patent.,1,0.0
630,No abnormal marrow signal in the lumbar vertebrae.,1,0.0
631,There is no epidural or paraspinal mass.,1,0.0
632,The conus medullaris is normally positioned.,1,0.0
633,No abnormal motion artifact distorting image quality.,1,0.0
634,The vertebral endplates appear intact and regular.,1,0.0
635,No Schmorl's nodes or endplate irregularities observed.,1,0.0
636,No abnormal soft tissue masses detected.,1,0.0
637,No signs of infection or inflammatory process in the lumbar spine.,1,0.0
638,Normal spinal canal diameter throughout lumbar region.,1,0.0
639,The MRI shows maintained lumbar lordosis.,1,0.0
640,No evidence of traumatic injury in the lumbar area.,1,0.0
641,The facet joints appear well-maintained without degeneration.,1,0.0
642,The discs demonstrate normal hydration signal on T2 images.,1,0.0
643,No evidence of congenital anomalies in the lumbar spine.,1,0.0
644,No signs of subluxation or instability are present.,1,0.0
645,The posterior elements of the spine are intact and normal.,1,0.0
646,The posterior longitudinal ligament appears intact.,1,0.0
647,The anterior longitudinal ligament shows no abnormality.,1,0.0
648,No vertebral collapse or compression fractures noted.,1,0.0
649,No signs of previous spinal surgery or instrumentation.,1,0.0
650,The MRI reveals no suspicious lesions in the lumbar spine.,1,0.0
651,The lumbar interspinous ligaments appear intact.,1,0.0
652,No foraminal narrowing at any lumbar levels.,1,0.0
653,No abnormality noted in adjacent pelvic structures.,1,0.0
654,The vertebral cortical margins are intact and smooth.,1,0.0
655,No abnormal signal is seen in adjacent muscles.,1,0.0
656,The epidural fat planes are preserved and symmetrical.,1,0.0
657,No spinal cord tethering or abnormal positioning detected.,1,0.0
658,The MRI scan reveals no acute or chronic pathology.,1,0.0
659,"No spinal masses, cysts, or hemangiomas detected.",1,0.0
660,"No disc herniation, protrusion, or extrusion identified.",1,0.0
661,The MRI reveals signs of disc herniation in the lumbar spine.,0,1.0
662,There is evidence of nerve root compression in the lower back.,0,1.0
663,Degenerative disc disease is seen in the lumbar region.,0,1.0
664,The imaging shows signs of spinal stenosis in the lumbar spine.,0,1.0
665,The lumbar vertebrae show signs of mild fractures or degeneration.,0,1.0
666,There is bulging of the intervertebral disc at L4-L5.,0,1.0
667,MRI suggests a herniated disc at L5-S1 with nerve root impingement.,0,1.0
668,Signs of lumbar spondylosis and facet joint osteoarthritis.,0,1.0
669,There is evidence of a lumbar disc bulge causing nerve compression.,0,1.0
670,The MRI indicates a possible spinal cord abnormality due to herniated disc.,0,1.0
671,The MRI scan shows disc degeneration at L3-L4.,0,1.0
672,There are signs of nerve compression at L4-L5 and L5-S1.,0,1.0
673,Spinal canal narrowing is observed in the lumbar region.,0,1.0
674,The MRI reveals significant herniation at the L4-L5 disc.,0,1.0
675,Degenerative changes in the lumbar spine are noted.,0,1.0
676,There is a narrowing of the intervertebral space at L5-S1.,0,1.0
677,The lumbar vertebrae show signs of wear and tear.,0,1.0
678,Signs of facet joint arthritis in the lumbar spine.,0,1.0
679,The MRI shows evidence of spondylolisthesis at L5.,0,1.0
680,There is abnormal disc bulging with nerve compression in the lumbar spine.,0,1.0
681,The MRI shows lumbar spine deformities due to spondylosis.,0,1.0
682,Herniated discs are observed at multiple lumbar levels.,0,1.0
683,There is evidence of lumbar spinal instability due to disc damage.,0,1.0
684,The imaging shows signs of spinal stenosis causing nerve compression.,0,1.0
685,There is moderate degeneration of the lumbar intervertebral discs.,0,1.0
686,The MRI suggests lumbar spine fractures with mild displacement.,0,1.0
687,The vertebral bodies show signs of degenerative changes in the lumbar spine.,0,1.0
688,There is significant disc herniation at L3-L4 affecting the nerve roots.,0,1.0
689,Signs of severe disc degeneration and nerve impingement at L5-S1.,0,1.0
690,The MRI scan shows severe degenerative changes in the lumbar spine.,0,1.0
691,There is bulging of the disc at L4-L5 with mild spinal stenosis.,0,1.0
692,The MRI shows evidence of spinal canal narrowing at L2-L3.,0,1.0
693,The lumbar spine shows signs of severe degenerative changes.,0,1.0
694,A herniated disc is pressing on the spinal nerve at L5-S1.,0,1.0
695,MRI indicates moderate degeneration at L3-L4 with narrowing of the disc space.,0,1.0
696,The MRI shows a herniated lumbar disc causing nerve root impingement.,0,1.0
697,There is evidence of facet joint arthritis at L4-L5 and L5-S1.,0,1.0
698,The MRI suggests degenerative changes in the lower lumbar spine.,0,1.0
699,Disc bulging is observed at L3-L4 and L5-S1 levels.,0,1.0
700,There is nerve compression and spinal stenosis at multiple lumbar levels.,0,1.0
701,The MRI indicates advanced degenerative disc disease in the lumbar spine.,0,1.0
702,There is evidence of chronic lumbar pain from disc herniation.,0,1.0
703,The MRI scan shows significant disc protrusion at L4-L5.,0,1.0
704,The lumbar spine shows disc herniation with nerve root compression.,0,1.0
705,There is severe stenosis of the lumbar spinal canal.,0,1.0
706,The MRI indicates bulging discs causing pressure on the spinal nerves.,0,1.0
707,There are signs of chronic lumbar pain and disk degeneration.,0,1.0
708,The MRI suggests herniated discs causing spinal cord pressure.,0,1.0
709,Moderate narrowing of the lumbar neural foramina is present.,0,1.0
710,Degenerative spondylolisthesis is observed at L4-L5.,0,1.0
711,Disc desiccation is evident at multiple lumbar levels.,0,1.0
712,Endplate changes are noted at L5-S1 due to chronic degeneration.,0,1.0
713,Facet joint hypertrophy is seen at L3-L4.,0,1.0
714,Spinal canal effacement is observed due to disc protrusion.,0,1.0
715,A disc extrusion compressing the thecal sac is present at L5-S1.,0,1.0
716,There is moderate nerve impingement bilaterally at L4-L5.,0,1.0
717,Anterior disc osteophyte complexes are present at L3-L4.,0,1.0
718,A lumbar disc herniation is causing foraminal narrowing at L4-L5.,0,1.0
719,The MRI shows degenerative retrolisthesis at L2-L3.,0,1.0
720,Disc protrusions are seen at three consecutive lumbar levels.,0,1.0
721,The lumbar MRI reveals paracentral disc herniation at L5-S1.,0,1.0
722,There is posterior disc displacement causing mild nerve impingement.,0,1.0
723,A broad-based disc bulge is compressing the nerve roots at L3-L4.,0,1.0
724,There is evidence of mild spondylolisthesis at L5-S1.,0,1.0
725,The MRI reveals prominent disc degeneration throughout the lumbar spine.,0,1.0
726,Advanced degenerative disc changes are evident at L4-L5.,0,1.0
727,A right paracentral disc herniation is compressing the L5 nerve.,0,1.0
728,There is posterior vertebral body osteophyte formation.,0,1.0
729,The imaging shows lumbar disc prolapse with thecal sac compression.,0,1.0
730,The MRI scan reveals bilateral foraminal stenosis.,0,1.0
731,There is ligamentum flavum thickening contributing to canal stenosis.,0,1.0
732,Disc space collapse is noted at L5-S1.,0,1.0
733,MRI reveals a large disc protrusion affecting neural structures.,0,1.0
734,A large disc herniation at L3-L4 compresses the adjacent nerve root.,0,1.0
735,The MRI reveals facet joint hypertrophy contributing to spinal stenosis.,0,1.0
736,Evidence of disc protrusion is causing significant nerve impingement.,0,1.0
737,A lumbar vertebral body compression fracture is seen at L2.,0,1.0
738,The lumbar discs show marked desiccation with disc space narrowing.,0,1.0
739,There is bilateral nerve root compression at L5-S1.,0,1.0
740,The MRI shows lumbar disc prolapse at L4-L5.,0,1.0
741,A sequestered disc fragment is compressing the left L5 nerve root.,0,1.0
742,There is degenerative retrolisthesis at L3-L4 with associated stenosis.,0,1.0
743,Posterior disc herniation is observed at L2-L3 with mild canal narrowing.,0,1.0
744,MRI reveals advanced discogenic changes at multiple lumbar levels.,0,1.0
745,Disc extrusion at L5-S1 is impinging on the thecal sac.,0,1.0
746,An annular tear is identified at the L4-L5 intervertebral disc.,0,1.0
747,Significant disc bulging is seen at the L3-L4 level.,0,1.0
748,There is lumbar facet arthropathy contributing to foraminal narrowing.,0,1.0
749,Multilevel lumbar spondylosis is evident on the MRI.,0,1.0
750,A broad-based disc protrusion causes moderate central canal stenosis.,0,1.0
751,Degenerative changes result in foraminal encroachment at L5-S1.,0,1.0
752,MRI shows left foraminal disc herniation compressing the L4 nerve.,0,1.0
753,The imaging demonstrates progressive disc degeneration in the lumbar spine.,0,1.0
754,Discogenic endplate changes are noted at multiple lumbar levels.,0,1.0
755,Nerve root sleeve effacement is observed at L4-L5.,0,1.0
756,Mild posterior disc bulging is present at L2-L3 and L3-L4.,0,1.0
757,A central disc protrusion is compressing the anterior thecal sac.,0,1.0
758,The MRI shows asymmetric disc bulging more pronounced on the right.,0,1.0
759,There is neural foraminal narrowing due to facet joint overgrowth.,0,1.0
760,An inferiorly migrated disc fragment is compressing the S1 nerve root.,0,1.0
761,The lumbar vertebral bodies and intervertebral discs appear normal in height and signal intensity.,1,0.0
762,No disc bulge,1,0.0
763,The spinal canal and neural foramina remain widely patent without evidence of stenosis.,1,0.0
764,No marrow signal abnormality is detected within the lumbar vertebral bodies.,1,0.0
765,Paraspinal soft tissues demonstrate unremarkable appearance on all sequences.,1,0.0
766,No abnormal enhancement or mass lesion is observed.,1,0.0
767,The conus medullaris terminates at the L1-L2 level with normal morphology and signal.,1,0.0
768,No evidence of nerve root compression or impingement is seen.,1,0.0
769,Alignment of the lumbar spine is well maintained without spondylolisthesis.,1,0.0
770,Facet joints show no significant degenerative changes.,1,0.0
771,No epidural or paraspinal collection is present.,1,0.0
772,Vertebral endplates appear intact without erosive or destructive changes.,1,0.0
773,Posterior elements are intact with no fracture or lytic lesion.,1,0.0
774,Normal cerebrospinal fluid signal throughout the spinal canal.,1,0.0
775,No evidence of prior surgical intervention or hardware is noted.,1,0.0
776,No abnormal curvature or scoliosis is present.,1,0.0
777,The sacroiliac joints appear symmetric and unremarkable bilaterally.,1,0.0
778,Vertebral body heights are preserved with no compression deformities.,1,0.0
779,No abnormality is identified involving the intervertebral discs.,1,0.0
780,Normal fatty marrow signal is demonstrated throughout.,1,0.0
781,The cauda equina nerve roots are unremarkable.,1,0.0
782,No significant degenerative disc disease is seen.,1,0.0
783,Posterior epidural fat appears normal in volume and signal.,1,0.0
784,The vertebral alignment remains physiologic.,1,0.0
785,No disc desiccation is observed.,1,0.0
786,The bony structures of the lumbar spine demonstrate normal morphology.,1,0.0
787,No mass-like lesion is identified within the spinal canal.,1,0.0
788,The posterior longitudinal ligament is intact without thickening.,1,0.0
789,No foraminal narrowing is present at any lumbar level.,1,0.0
790,No abnormality in the paravertebral soft tissues.,1,0.0
791,The vertebral marrow signal is homogeneous.,1,0.0
792,No focal lesion is seen involving the spinal cord.,1,0.0
793,The anterior longitudinal ligament appears intact.,1,0.0
794,The conus medullaris shows normal position and configuration.,1,0.0
795,No abnormal enhancement post-contrast administration.,1,0.0
796,No signs of infection or inflammatory changes.,1,0.0
797,No evidence of vertebral body edema or contusion.,1,0.0
798,Intervertebral disc heights are well preserved.,1,0.0
799,No synovial cyst or facet joint effusion is seen.,1,0.0
800,No abnormality of the posterior elements is appreciated.,1,0.0
801,No significant spondylotic changes present.,1,0.0
802,The paraspinal muscles show normal bulk and signal.,1,0.0
803,No evidence of sacral fracture or abnormality.,1,0.0
804,No intraspinal mass lesion identified.,1,0.0
805,Thecal sac is normal in caliber throughout.,1,0.0
806,No signs of spinal instability are demonstrated.,1,0.0
807,Normal alignment of the spinous processes.,1,0.0
808,No pathologic signal abnormality in the sacrum.,1,0.0
809,Disc signal intensity appears preserved on T2-weighted images.,1,0.0
810,No signs of prior trauma.,1,0.0
811,Vertebral body contours are regular without lytic or sclerotic lesions.,1,0.0
812,No abnormality involving the coccyx.,1,0.0
813,No abnormality of the lumbosacral junction.,1,0.0
814,No ligamentous thickening or ossification.,1,0.0
815,Normal epidural venous plexus.,1,0.0
816,No indication of degenerative spondylolisthesis.,1,0.0
817,The interspinous ligaments are intact.,1,0.0
818,No high-signal changes suggestive of marrow edema.,1,0.0
819,No epidural hematoma or abnormal collection.,1,0.0
820,No nerve root thickening or abnormal enhancement.,1,0.0
821,No intervertebral disc extrusion or migration.,1,0.0
822,The vertebral endplates are preserved without Modic changes.,1,0.0
823,No abnormality detected at the thoracolumbar junction.,1,0.0
824,No congenital anomalies of the lumbar vertebrae.,1,0.0
825,Spinal cord signal remains normal throughout.,1,0.0
826,No evidence of neural foraminal stenosis.,1,0.0
827,No incidental mass or lesion identified.,1,0.0
828,Posterior elements well aligned with no pars defects.,1,0.0
829,No vertebral body collapse or loss of height.,1,0.0
830,No cystic or solid mass detected.,1,0.0
831,No abnormal curvature of the lumbar spine.,1,0.0
832,No degenerative disc changes appreciated.,1,0.0
833,Paravertebral regions remain unremarkable.,1,0.0
834,No signs of neoplastic involvement.,1,0.0
835,The posterior dura appears normal.,1,0.0
836,No bone marrow replacement process identified.,1,0.0
837,No fluid collection or abnormal enhancement.,1,0.0
838,Intervertebral discs are age-appropriate in appearance.,1,0.0
839,No unusual findings or incidentalomas detected.,1,0.0
840,No abnormal intraspinal signal change.,1,0.0
841,Vertebral marrow displays homogeneous fat signal.,1,0.0
842,The facet joint capsules are intact without effusion.,1,0.0
843,No annular fissures identified.,1,0.0
844,Spinal cord conus terminates normally without distortion.,1,0.0
845,No prevertebral or retroperitoneal abnormality.,1,0.0
846,No epidural lesion or displacement.,1,0.0
847,No hyperintense foci on T2 suggestive of edema or tumor.,1,0.0
848,The sacral nerve roots demonstrate normal appearance.,1,0.0
849,No abnormalities in the lumbosacral transition vertebrae.,1,0.0
850,Vertebral heights and alignment are anatomic.,1,0.0
851,No significant facet arthropathy noted.,1,0.0
852,No intradural mass lesion.,1,0.0
853,No pathologic enhancement pattern after gadolinium.,1,0.0
854,Disc margins remain smooth and continuous.,1,0.0
855,Thecal sac compression is not evident.,1,0.0
856,Paraspinal musculature is symmetric.,1,0.0
857,No significant osteophyte formation.,1,0.0
858,No acute or chronic fractures detected.,1,0.0
859,No notable vascular anomalies.,1,0.0
860,No features suggesting infection or malignancy.,1,0.0
861,Mild disc desiccation is noted at the L4-L5 and L5-S1 levels.,0,1.0
862,Central disc protrusion at L5-S1 minimally effaces the thecal sac.,0,1.0
863,Mild facet arthropathy is present at multiple lumbar levels.,0,1.0
864,Small annular fissure seen at L4-L5.,0,1.0
865,Disc bulging observed at L3-L4 without significant stenosis.,0,1.0
866,Concentric disc bulge at L4-L5 mildly narrows the central canal.,0,1.0
867,Moderate facet joint hypertrophy at L5-S1 contributes to foraminal narrowing.,0,1.0
868,Thecal sac impression by a posterior disc protrusion at L5-S1.,0,1.0
869,Left-sided foraminal stenosis at L4-L5 due to disc-osteophyte complex.,0,1.0
870,Subtle marrow edema adjacent to the L5-S1 endplates.,0,1.0
871,Small left paracentral disc protrusion noted at L4-L5.,0,1.0
872,Ligamentum flavum thickening contributes to mild canal narrowing.,0,1.0
873,Early degenerative disc disease involving the lower lumbar spine.,0,1.0
874,Disc bulging with shallow central protrusion at L3-L4.,0,1.0
875,Bilateral facet hypertrophy at L4-L5 and L5-S1.,0,1.0
876,Posterior annular tear visible at L5-S1.,0,1.0
877,No significant central canal stenosis,0,1.0
878,Disc height loss at L5-S1 with associated Modic type 1 changes.,0,1.0
879,Right L5 nerve root proximity to a disc bulge at L4-L5.,0,1.0
880,Early spondylotic changes throughout the lumbar spine.,0,1.0
881,Mild asymmetric disc bulge toward the left at L4-L5.,0,1.0
882,Narrowing of bilateral neural foramina at L5-S1.,0,1.0
883,Posterior disc osteophyte complex noted at L5-S1.,0,1.0
884,Minor disc extrusion causing mild thecal sac displacement.,0,1.0
885,Bilateral L5-S1 facet joint arthrosis with small joint effusions.,0,1.0
886,Focal marrow edema adjacent to the L4 endplate.,0,1.0
887,Concentric bulging with annular fissuring at L5-S1.,0,1.0
888,Minor spondylolisthesis at L4-L5 with facet degeneration.,0,1.0
889,Low-grade central canal stenosis at L4-L5.,0,1.0
890,Foraminal disc protrusion impinging on the left L5 nerve root.,0,1.0
891,Diffuse disc desiccation involving multiple lumbar levels.,0,1.0
892,Central disc extrusion at L5-S1 with inferior migration.,0,1.0
893,Mild retrolisthesis at L3-L4 on extension images.,0,1.0
894,Hypertrophic changes of the ligamentum flavum at L5-S1.,0,1.0
895,Small left foraminal synovial cyst at L4-L5.,0,1.0
896,Posterior disc displacement resulting in mild foraminal narrowing.,0,1.0
897,Subtle disc extrusion contacting the right L4 nerve root.,0,1.0
898,Annular fissures present at L3-L4 and L5-S1.,0,1.0
899,Advanced disc degeneration noted at L5-S1.,0,1.0
900,Moderate bilateral foraminal narrowing at L5-S1.,0,1.0
901,Disc material abutting the right S1 nerve root.,0,1.0
902,Marrow signal changes adjacent to degenerative endplates.,0,1.0
903,Disc-osteophyte complex formation noted at L4-L5.,0,1.0
904,Posterior disc protrusion contacting bilateral nerve roots.,0,1.0
905,Small Schmorls nodes within L3 and L4 vertebral endplates.,0,1.0
906,Thecal sac indentation by a central disc protrusion.,0,1.0
907,Moderate facet joint arthropathy at multiple levels.,0,1.0
908,Disc desiccation and mild disc bulging at L5-S1.,0,1.0
909,Central disc bulging with mild foraminal narrowing bilaterally.,0,1.0
910,L5-S1 disc height significantly reduced.,0,1.0
911,Mild degenerative changes at L4-L5 with facet joint narrowing.,0,1.0
912,Disc bulging at L4-L5 contributing to slight lateral recess narrowing.,0,1.0
913,Mild foraminal stenosis bilaterally at L3-L4 secondary to disc bulging.,0,1.0
914,Mild scoliosis noted,0,1.0
915,Focal herniation at L5-S1 causing minimal displacement of the nerve roots.,0,1.0
916,Moderate disc degeneration at L3-L4 with associated annular tear.,0,1.0
917,Severe disc collapse with associated foraminal stenosis at L5-S1.,0,1.0
918,Facet joint hypertrophy and thickening observed at L4-L5.,0,1.0
919,Mild narrowing of the central canal at L4-L5 with posterior bulging.,0,1.0
920,Posterior osteophytes at L3-L4 contributing to mild canal narrowing.,0,1.0
921,Central disc protrusion at L2-L3 without significant stenosis.,0,1.0
922,Partial disc extrusion at L5-S1 with lateral disc bulge.,0,1.0
923,No significant central canal narrowing,0,1.0
924,Mild posterior bulging at L5-S1 resulting in nerve root compression.,0,1.0
925,L5-S1 spondylolisthesis contributing to mild spinal instability.,0,1.0
926,Mild central disc protrusion at L4-L5 causing slight spinal canal narrowing.,0,1.0
927,Intervertebral disc herniation at L3-L4 impinging the left L3 nerve root.,0,1.0
928,Severe disc degeneration at L4-L5 with Schmorls nodes.,0,1.0
929,Moderate facet arthrosis at L2-L3 with mild narrowing of the neural foramina.,0,1.0
930,Small synovial cyst in the left facet joint at L3-L4,0,1.0
931,Chronic disc desiccation at L5-S1 with mild spondylolisthesis.,0,1.0
932,Mild central and paracentral disc protrusion at L3-L4 with associated nerve root compression.,0,1.0
933,L4-L5 disc space loss with associated foraminal stenosis and facet joint arthropathy.,0,1.0
934,Abnormal disc height at L4-L5 with mild bulging and associated nerve root encroachment.,0,1.0
935,Right-sided facet joint osteoarthritis at L5-S1 causing moderate foraminal narrowing.,0,1.0
936,Posterior disc bulge at L3-L4 with mild narrowing of the central canal.,0,1.0
937,Mild to moderate stenosis at L4-L5 secondary to disc protrusion and facet arthropathy.,0,1.0
938,Severe disc desiccation at L3-L4 with associated anterior osteophytes.,0,1.0
939,A mild bulging disc at L5-S1 causes narrowing of the right neural foramen.,0,1.0
940,Facet joint degenerative changes at L4-L5 with mild foraminal narrowing.,0,1.0
941,Annular tear with mild disc bulge at L3-L4 impinging on the L4 nerve root.,0,1.0
942,Narrowing of the lumbar disc spaces with mild spondylosis at multiple levels.,0,1.0
943,Diffuse bulging of the disc at L3-L4,0,1.0
944,Severe disc degeneration at L5-S1 with evidence of nerve root impingement.,0,1.0
945,The L4-L5 disc shows moderate bulging and mild central canal stenosis.,0,1.0
946,Left paracentral disc protrusion at L4-L5 with slight compression of the nerve roots.,0,1.0
947,Minor spondylolisthesis at L4-L5 with accompanying facet arthrosis.,0,1.0
948,L5-S1 disc bulging with minimal foraminal narrowing and nerve root compression.,0,1.0
949,Mild disc protrusion at L3-L4 with associated facet joint degeneration.,0,1.0
950,Disc bulge at L5-S1 with foraminal stenosis causing nerve root encroachment.,0,1.0
951,Disc degeneration and bulging at L3-L4 contributing to nerve root impingement.,0,1.0
952,Left-sided disc protrusion at L4-L5 resulting in mild foraminal stenosis.,0,1.0
953,Central disc extrusion at L5-S1,0,1.0
954,Spondylotic changes noted at L3-L4 with mild canal narrowing.,0,1.0
955,Facet joint hypertrophy at L5-S1 causing mild foraminal narrowing bilaterally.,0,1.0
956,Disc bulging with minor herniation at L4-L5 resulting in bilateral foraminal narrowing.,0,1.0
957,Posterior disc protrusion at L2-L3 causing mild central canal narrowing.,0,1.0
958,Diffuse disc bulging at L4-L5 contributing to mild neural foraminal narrowing.,0,1.0
959,Moderate disc desiccation and degenerative changes at L4-L5 and L5-S1.,0,1.0
960,Chronic degenerative changes at L3-L4 with mild facet joint osteoarthritis.,0,1.0

Overwriting local_data/sentence-label.csv


## Overwriting codes

In [18]:
%%writefile 'medclip/dataset.py'

import re
import random
from collections import defaultdict
import pdb
from typing import Union, List, Optional

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
from torchvision import transforms

from transformers import AutoTokenizer
from transformers import CLIPFeatureExtractor, CLIPProcessor
from transformers.utils import TensorType
from transformers.feature_extraction_utils import BatchFeature
from transformers.image_utils import is_torch_tensor

# from nltk.tokenize import RegexpTokenizer
import nltk
from PIL import Image
from sklearn.preprocessing import OrdinalEncoder


from .prompts import process_class_prompts, process_class_prompts_for_tuning
from .prompts import generate_chexpert_class_prompts
from . import constants

class MedCLIPFeatureExtractor(CLIPFeatureExtractor):
    def __init__(self, 
        do_resize=True, 
        size=224, 
        resample=Image.BICUBIC, 
        do_center_crop=True, 
        crop_size=224, 
        do_normalize=True, 
        image_mean=constants.IMG_MEAN, 
        image_std=constants.IMG_STD, 
        do_convert_rgb=False,
        do_pad_square=True,
        **kwargs):
        super().__init__(do_resize, size, resample, do_center_crop, crop_size, do_normalize, image_mean, image_std, do_convert_rgb, **kwargs)
        self.do_pad_square = do_pad_square
    
    def __call__(self, 
        images: Union[Image.Image, np.ndarray, "torch.Tensor", List[Image.Image], List[np.ndarray], List["torch.Tensor"]], 
        return_tensors: Optional[Union[str, TensorType]] = None, 
        **kwargs) -> BatchFeature:
        """
        Main method to prepare for the model one or several image(s).

        <Tip warning={true}>

        NumPy arrays and PyTorch tensors are converted to PIL images when resizing, so the most efficient is to pass
        PIL images.

        </Tip>

        Args:
            images (`PIL.Image.Image`, `np.ndarray`, `torch.Tensor`, `List[PIL.Image.Image]`, `List[np.ndarray]`, `List[torch.Tensor]`):
                The image or batch of images to be prepared. Each image can be a PIL image, NumPy array or PyTorch
                tensor. In case of a NumPy array/PyTorch tensor, each image should be of shape (C, H, W), where C is a
                number of channels, H and W are image height and width.

            return_tensors (`str` or [`~utils.TensorType`], *optional*, defaults to `'np'`):
                If set, will return tensors of a particular framework. Acceptable values are:

                - `'tf'`: Return TensorFlow `tf.constant` objects.
                - `'pt'`: Return PyTorch `torch.Tensor` objects.
                - `'np'`: Return NumPy `np.ndarray` objects.
                - `'jax'`: Return JAX `jnp.ndarray` objects.

        Returns:
            [`BatchFeature`]: A [`BatchFeature`] with the following fields:

            - **pixel_values** -- Pixel values to be fed to a model.
        """
        # Input type checking for clearer error
        valid_images = False

        # Check that images has a valid type
        if isinstance(images, (Image.Image, np.ndarray)) or is_torch_tensor(images):
            valid_images = True
        elif isinstance(images, (list, tuple)):
            if len(images) == 0 or isinstance(images[0], (Image.Image, np.ndarray)) or is_torch_tensor(images[0]):
                valid_images = True

        if not valid_images:
            raise ValueError(
                "Images must of type `PIL.Image.Image`, `np.ndarray` or `torch.Tensor` (single example), "
                "`List[PIL.Image.Image]`, `List[np.ndarray]` or `List[torch.Tensor]` (batch of examples)."
            )

        is_batched = bool(
            isinstance(images, (list, tuple))
            and (isinstance(images[0], (Image.Image, np.ndarray)) or is_torch_tensor(images[0]))
        )

        if not is_batched:
            images = [images]

        # transformations (convert rgb + resizing + center cropping + normalization)
        if self.do_convert_rgb:
            images = [self.convert_rgb(image) for image in images]

        if self.do_pad_square:
            images = [self.pad_img(image,min_size=self.size) for image in images]
        
        if self.do_resize and self.size is not None and self.resample is not None:
            images = [
                self.resize(image=image, size=self.size, resample=self.resample)
                for image in images
            ]
        if self.do_center_crop and self.crop_size is not None:
            images = [self.center_crop(image, self.crop_size) for image in images]
        if self.do_normalize:
            images = [self.normalize(image=image, mean=self.image_mean, std=self.image_std) for image in images]

        # add a RGB dim for each image
        images_ = []
        for image in images:
            if len(image.shape) == 2:
                image = image[None]
            images_.append(image)
        images = images_

        # return as BatchFeature
        data = {"pixel_values": images}
        encoded_inputs = BatchFeature(data=data, tensor_type=return_tensors)

        return encoded_inputs

    def pad_img(self, img, min_size=224, fill_color=0):
        '''pad img to square.
        '''
        x, y = img.size
        size = max(min_size, x, y)
        new_im = Image.new('L', (size, size), fill_color)
        new_im.paste(img, (int((size - x) / 2), int((size - y) / 2)))
        return new_im

class MedCLIPProcessor(CLIPProcessor):
    '''
    A processor that takes input images and texts and provides inputs for
    `MedCLIPModel`.
    '''
    feature_extractor_class = "CLIPFeatureExtractor"
    tokenizer_class = ("BertTokenizer", "BertTokenizerFast")
    def __init__(self):
        feature_extractor = MedCLIPFeatureExtractor()
        tokenizer = AutoTokenizer.from_pretrained(constants.BERT_TYPE)
        tokenizer.model_max_length = 77
        super().__init__(feature_extractor, tokenizer)

class ImageTextContrastiveDataset(Dataset):
    _labels_ = ['No Finding', 'LBP']
    def __init__(self, datalist=['lbp-train'], imgtransform=None) -> None:
        '''support data list in lbp-train
        '''
        super().__init__()
        # imgpath, subject_id, report, labels...
        df_list = []
        for data in datalist:
            filename = f'./local_data/{data}-meta.csv'
            print('load data from', filename)
            df = pd.read_csv(filename, index_col=0)
            df_list.append(df)
        self.df = pd.concat(df_list, axis=0).reset_index(drop=True)

        # split raw reports and process into sentences
        self.df = self.create_sent_segments(self.df)

        # could try contrast, brightness, fog
        if imgtransform is None:
            self.transform = transforms.Compose([
                # transforms.RandomHorizontalFlip(0.5),
                # transforms.ColorJitter(0.1,0.1),
                transforms.ToTensor(),
                transforms.Resize((constants.IMG_SIZE,constants.IMG_SIZE)),
                transforms.Normalize(mean=[0.5862785803043838],std=[0.27950088968644304])],
            )
        else:
            self.transform = imgtransform

        # use labeled sentences as prompts for chexpert training
        self.sentence_label = pd.read_csv('./local_data/sentence-label.csv', index_col=0).fillna(0)
        # self.sentence_label = pd.read_csv('./local_data/sentence-label.csv').fillna(0)
        print('load sentence prompts from ./local_data/sentence-label.csv')
        self._preprocess_sentence_label()
        self._build_prompt_sentence()

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img = Image.open(row.imgpath)

        img = self._pad_img(img) # pad image to square
        img = self.transform(img).unsqueeze(1)
        report = row.report # original sentences list
        img_label = row[self._labels_].values # image corresponds to text labels
        if len(report) == 0: # no report available
            # sample class prompts as augmentation
            report, text_label = self.sample_sent_prompts(row)
        else:
            # randomly sample one sentence
            sent_ix = random.randint(0, len(report)-1)
            report = report[sent_ix]
            # we need to use sentence-level label instead
            # maintain a sentence dictionary
            # index sentence dictionary label during training, if not found, return all zero
            # **supervision from unpaired texts and images**
            if report in self.sent_label_dict: # find the sentence
                text_label = self.sent_label_dict[report]
            else:
                text_label = np.zeros(len(img_label))
                text_label[0] = 1
        # print(f"In ImageTextContrastiveDataset: img_label = {img_label}")
        # print(f"In ImageTextContrastiveDataset: text_label = {text_label}")
        return img, report, img_label, text_label

    def __len__(self):
        return len(self.df)

    def _pad_img(self, img, min_size=224, fill_color=0):
        '''pad img to square.
        '''
        x, y = img.size
        size = max(min_size, x, y)
        new_im = Image.new('L', (size, size), fill_color)
        new_im.paste(img, (int((size - x) / 2), int((size - y) / 2)))
        return new_im

    def sample_sent_prompts(self, row):
        # do prompt sampling
        if (row[self._labels_] == 0).all(): # no label available, use no finding
            sampled_sent = self.sentence_label[self.sentence_label['No Finding'] > 0].sample()
            report = sampled_sent['report'].values[0][0]
            label = sampled_sent[self._labels_].values[0]
        else:
            # get prompt sentence x * 0 = 0, 1 * -1 = -1, 1 * 1 = 1, -1 * -1 = 1
            bool_sent_label = self.prompt_sentence_label[self._labels_] *  row[self._labels_]
            bool_sent_label[bool_sent_label < 0] = 0
            sents = self.prompt_sentence_label.loc[~(bool_sent_label.iloc[:,1:] == 0).all(1)]
            if len(sents) == 0: # only no finding
                sampled_sent = self.prompt_sentence_label[self.prompt_sentence_label['No Finding']==1].sample()
            else:
                # random sample
                sampled_sent = sents.sample()
            report = sampled_sent['report'].values[0]
            label = sampled_sent[self._labels_].values.flatten()
        return report, label

    def create_sent_segments(self, df):
        '''do preprocessing to split raw reports into sentence segments for
        sentence-image contrastive pretraining.
        '''
        df['report'] = df['report'].apply(self._split_report_into_segment)
        return df

    def _split_report_into_segment(self, report):
        '''clean up raw reports into sentences
        '''
        if pd.isnull(report):
            return []
        else:
            report = report.replace('\n',' ')
            # splitter = re.compile("[0-9]+\.")
            splitter = re.compile("[0-9]+\.+[^0-9]")
            report = splitter.split(report)
            reports = [point.split(". ") for point in report]
            # reports = [point.split(".") for point in report]
            reports = [sent for point in reports for sent in point]
            study_sent = []
            for sent in reports:
                if len(sent) == 0:
                    continue

                sent = sent.replace("\ufffd\ufffd", " ")
                # tokenizer = RegexpTokenizer(r"\w+")
                # tokens = tokenizer.tokenize(sent.lower())

                tokens = tokens = nltk.wordpunct_tokenize(sent.lower())
                
                if len(tokens) <= 1:
                    continue

                # filter tokens for current sentence
                included_tokens = []
                for t in tokens:
                    t = t.encode("ascii", "ignore").decode("ascii")
                    if len(t) > 0:
                        included_tokens.append(t)
                if len(included_tokens) > 4: # only include relative long sentences
                    study_sent.append(" ".join(included_tokens))
            return study_sent

    def _preprocess_sentence_label(self):
        self.sentence_label = self.sentence_label.drop_duplicates(subset='Reports')
        self.sentence_label = self.sentence_label[self.sentence_label['Reports'].map(len)>2].reset_index(drop=True)
        self.sentence_label['report'] = self.sentence_label['Reports']
        self.sentence_label = self.sentence_label.drop('Reports', axis=1)
        self.sentence_label = self.create_sent_segments(self.sentence_label)
        self.sentence_label = self.sentence_label[(self.sentence_label['report'].map(len)==1)]
        # self.sentence_label['report'] = np.concatenate(self.sentence_label['report'].values)
        self.sentence_label = self.sentence_label.explode('report').reset_index(drop=True)

        # build dict from dataframe
        # print("_preprocess_sentence_label:",self.sentence_label.columns)
        
        keys = self.sentence_label['report'].values
        vals = self.sentence_label.drop(['report'],axis=1).fillna(0).values
        self.sent_label_dict = dict(zip(keys,vals))
        # print("_preprocess_sentence_label: ", vals)

    def _build_prompt_sentence(self, n = 200):
        print('build prompt sentences.')
        sentence_label = self.sentence_label.copy()
        new_sent_list = []
        LBP_TASKS = ["No Finding", "LBP"]
        for task in LBP_TASKS:
            sub_sent_df = sentence_label[sentence_label[task] == 1]
            if len(sub_sent_df) < n: new_sent_list.append(sub_sent_df)
            else: new_sent_list.append(sub_sent_df.sample(n))

        new_sent_df = pd.concat(new_sent_list, axis=0)
        new_sent_df = new_sent_df.drop_duplicates()
        self.prompt_sentence_label = new_sent_df

class ImageTextContrastiveCollator:
    def __init__(self, use_eda=True):
        '''Args:
        use_EDA: easy data augmentation from textaugment
        '''
        if use_eda:
            import nltk
            nltk.download('stopwords')
            nltk.download('omw-1.4')
            nltk.download('wordnet')
            from textaugment import EDA
            self.eda = EDA()
        else:
            self.eda = None

        self.tokenizer = AutoTokenizer.from_pretrained(constants.BERT_TYPE)
        self.tokenizer.model_max_length = 77
    def __call__(self, batch):
        inputs = defaultdict(list)
        report_list = []
        report_aug_list = []
        for data in batch:
            inputs['pixel_values'].append(data[0])
            if self.eda is not None:
                eda_aug = random.choice([self.eda.synonym_replacement, self.eda.random_swap, self.eda.random_deletion])
                text_aug = eda_aug(data[1])
                if isinstance(text_aug, list): text_aug = ' '.join(text_aug)
                report_aug_list.append(text_aug)
            report_list.append(data[1])
            inputs['img_labels'].append(data[2])
            inputs['text_labels'].append(data[3])
        text_inputs = self.tokenizer(report_list, truncation=True, padding=True, return_tensors='pt')
        inputs['pixel_values'] = torch.cat(inputs['pixel_values'], 0)
        if inputs['pixel_values'].shape[1] == 1: inputs['pixel_values'] = inputs['pixel_values'].repeat((1,3,1,1))
        inputs['img_labels'] = torch.tensor(np.stack(inputs['img_labels']).astype(float))
        inputs['text_labels'] = torch.tensor(np.stack(inputs['text_labels']).astype(float))
        inputs['input_ids'] = text_inputs['input_ids']
        inputs['attention_mask'] = text_inputs['attention_mask']
        if len(report_aug_list) > 0:
            aug_text_inputs = self.tokenizer(report_aug_list, truncation=True, padding=True, return_tensors='pt')
            inputs['aug_input_ids'] =  aug_text_inputs['input_ids']
            inputs['aug_attention_mask'] = aug_text_inputs['attention_mask']

        # print("ImageTextContrastiveCollator text_labels: ", inputs['text_labels'])
        # print("ImageTextContrastiveCollator img_labels: ", inputs['img_labels'])
        return inputs

class ZeroShotImageDataset(Dataset):
    def __init__(self,
        datalist=['lbp-valid'],
        class_names=None,
        imgtransform=None,
        ) -> None:
        '''support data list in mimic-5x200, chexpert-5x200, rsna-balanced-test, covid-test
        args:
            imgtransform: a torchvision transform
            cls_prompts: a dict of prompt sentences, cls:[sent1, sent2, ..],
        '''
        super().__init__()

        if imgtransform is None:
            self.transform = transforms.Compose([
                transforms.Resize((constants.IMG_SIZE,constants.IMG_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5862785803043838],std=[0.27950088968644304])]
            )
        else:
            self.transform = imgtransform

        self.class_names = class_names

        # imgpath, subject_id, report, labels...(14 labels)
        df_list = []
        for data in datalist:
            filename = f'./local_data/{data}-meta.csv'
            print('load data from', filename)
            df = pd.read_csv(filename, index_col=0)
            df_list.append(df)
        self.df = pd.concat(df_list, axis=0).reset_index(drop=True)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img = Image.open(row.imgpath)
        img = self._pad_img(img)
        img = self.transform(img).unsqueeze(1)
        label = pd.DataFrame(row[self.class_names]).transpose()
        # label = row[self.class_names].values
        # print(f"label in ZeroShotImageDataset: {label}")
        # print(f"class name: {self.class_names}")
        # print("row")
        # print(row[self.class_names])
        return img, label

    def _pad_img(self, img, min_size=224, fill_color=0):
        '''pad img to square.
        '''
        x, y = img.size
        size = max(min_size, x, y)
        new_im = Image.new('L', (size, size), fill_color)
        new_im.paste(img, (int((size - x) / 2), int((size - y) / 2)))
        return new_im

    def __len__(self):
        return len(self.df)

class ZeroShotImageCollator:
    def __init__(self, mode, cls_prompts=None, n_prompt=5):
        # initialize tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(constants.BERT_TYPE)
        self.tokenizer.model_max_length = 77
        assert mode in ['multiclass','multilabel','binary']
        self.mode = mode

        if cls_prompts is None:
            raise NotImplementedError
        else:
            self.cls_prompts = cls_prompts

        # process cls prompts into texts indices
        self.prompt_texts_inputs = process_class_prompts(self.cls_prompts)

    def __call__(self, batch):
        inputs = defaultdict(list)
        for data in batch:
            inputs['pixel_values'].append(data[0])
            inputs['labels'].append(data[1])

        inputs['labels'] = pd.concat(inputs['labels']).astype(int).values
        if self.mode in ['multiclass','binary']:
            inputs['labels'] = torch.tensor(inputs['labels'].argmax(1), dtype=int)
        else:
            inputs['labels'] = torch.tensor(inputs['labels'], dtype=float)

        inputs['pixel_values'] = torch.cat(inputs['pixel_values'], 0)
        if inputs['pixel_values'].shape[1] == 1: inputs['pixel_values'] = inputs['pixel_values'].repeat((1,3,1,1))

        # print(f"Label in ZeroShotImageCollator: {inputs['labels']}")
        return {
            'pixel_values': inputs['pixel_values'],
            'prompt_inputs': self.prompt_texts_inputs,
            'labels': inputs['labels'],
            }

class SuperviseImageDataset(Dataset):
    def __init__(self,
        datalist=['lbp-valid'],
        class_names=None,
        imgtransform=None,
        ) -> None:
        '''support data list in mimic-5x200, mimic-5x200-finetune, chexpert-5x200, chexpert-5x200-finetune,
        rsna-balanced-test, rsna-balanced-train, covid-test, covid-train, covid-0.1-train
        args:
            imgtransform: a torchvision transform
            class_names: a list of class names
        '''
        super().__init__()
        if imgtransform is None:
            self.transform = transforms.Compose([
                transforms.Resize((constants.IMG_SIZE,constants.IMG_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5862785803043838],std=[0.27950088968644304])]
            )
        else:
            self.transform = imgtransform

        self.class_names = class_names

        # imgpath, subject_id, report, labels...(14 labels)
        df_list = []
        for data in datalist:
            filename = f'./local_data/{data}-meta.csv'
            print('load data from', filename)
            df = pd.read_csv(filename, index_col=0)
            df_list.append(df)
        self.df = pd.concat(df_list, axis=0).reset_index(drop=True)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        print(row)
        img = Image.open(row.imgpath)
        img = self._pad_img(img)
        img = self.transform(img).unsqueeze(1)
        label = pd.DataFrame(row[self.class_names]).transpose()
        return img, label

    def _pad_img(self, img, min_size=224, fill_color=0):
        '''pad img to square.
        '''
        x, y = img.size
        size = max(min_size, x, y)
        new_im = Image.new('L', (size, size), fill_color)
        new_im.paste(img, (int((size - x) / 2), int((size - y) / 2)))
        return new_im

    def __len__(self):
        return len(self.df)

class SuperviseImageCollator:
    def __init__(self, mode):
        assert mode in ['multiclass','multilabel','binary']
        self.mode = mode

    def __call__(self, batch):
        inputs = defaultdict(list)
        for data in batch:
            inputs['pixel_values'].append(data[0])
            inputs['labels'].append(data[1])
        inputs['labels'] = pd.concat(inputs['labels']).astype(int).values

        if self.mode in ['multiclass','binary']:
            inputs['labels'] = torch.tensor(inputs['labels'].argmax(1), dtype=int)
        else:
            inputs['labels'] = torch.tensor(inputs['labels'], dtype=float)

        inputs['pixel_values'] = torch.cat(inputs['pixel_values'], 0)
        if inputs['pixel_values'].shape[1] == 1: inputs['pixel_values'] = inputs['pixel_values'].repeat((1,3,1,1))
        return {
            'pixel_values': inputs['pixel_values'],
            'labels': inputs['labels'],
            }


class PromptTuningImageDataset(Dataset):
    def __init__(self,
                 datalist=['lbp-valid'],
                 class_names=None,
                 imgtransform=None,
                 ) -> None:
        '''support data list in mimic-5x200, mimic-5x200-finetune, chexpert-5x200, chexpert-5x200-finetune,
        rsna-balanced-test, rsna-balanced-train, covid-test, covid-train, covid-0.1-train
        args:
            imgtransform: a torchvision transform
            cls_prompts: a dict of prompt sentences, cls:[sent1, sent2, ..],
        '''
        super().__init__()

        if imgtransform is None:
            self.transform = transforms.Compose([
                transforms.Resize((constants.IMG_SIZE, constants.IMG_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5862785803043838], std=[0.27950088968644304])]
            )
        else:
            self.transform = imgtransform

        self.class_names = class_names

        # imgpath, subject_id, report, labels...(14 labels)
        df_list = []
        for data in datalist:
            filename = f'./local_data/{data}-meta.csv'
            print('load data from', filename)
            df = pd.read_csv(filename, index_col=0)
            df_list.append(df)
        self.df = pd.concat(df_list, axis=0).reset_index(drop=True)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        print(row)
        img = Image.open(row.imgpath)
        img = self._pad_img(img)
        img = self.transform(img).unsqueeze(1)
        label = pd.DataFrame(row[self.class_names]).transpose()
        return img, label

    def _pad_img(self, img, min_size=224, fill_color=0):
        '''pad img to square.
        '''
        x, y = img.size
        size = max(min_size, x, y)
        new_im = Image.new('L', (size, size), fill_color)
        new_im.paste(img, (int((size - x) / 2), int((size - y) / 2)))
        return new_im

    def __len__(self):
        return len(self.df)


class PromptTuningImageCollator:
    def __init__(self, mode, cls_prompts=None, n_prompt=5, n_context=16, class_specific_context=False):
        assert mode in ['multiclass', 'multilabel', 'binary']
        self.mode = mode

        if cls_prompts is None:
            raise NotImplementedError
        else:
            self.cls_prompts = cls_prompts

        # process cls prompts into texts indices
        self.prompt_texts_inputs = process_class_prompts_for_tuning(self.cls_prompts,
                                                                    n_context=n_context,
                                                                    class_specific_context=class_specific_context)

    def __call__(self, batch):
        inputs = defaultdict(list)
        for data in batch:
            inputs['pixel_values'].append(data[0])
            inputs['labels'].append(data[1])

        inputs['labels'] = pd.concat(inputs['labels']).astype(int).values
        if self.mode in ['multiclass', 'binary']:
            inputs['labels'] = torch.tensor(inputs['labels'].argmax(1), dtype=int)
        else:
            inputs['labels'] = torch.tensor(inputs['labels'], dtype=float)

        inputs['pixel_values'] = torch.cat(inputs['pixel_values'], 0)
        if inputs['pixel_values'].shape[1] == 1: inputs['pixel_values'] = inputs['pixel_values'].repeat((1, 3, 1, 1))
        return {
            'pixel_values': inputs['pixel_values'],
            'prompt_inputs': self.prompt_texts_inputs,
            'labels': inputs['labels'],
        }


Overwriting medclip/dataset.py


In [1]:
%%writefile "medclip/trainer.py"
import os
import json
import pdb
from typing import List, Dict, Tuple, Iterable, Type, Union, Callable, Optional
from collections import defaultdict
import math

import numpy as np
import torch
from torch import nn
from torch import device, Tensor
from tqdm.autonotebook import trange
from torch.utils.data import DataLoader
from torch.optim import Optimizer
from torch import distributed as dist
import transformers

WEIGHTS_NAME = "pytorch_model.bin"

class Trainer:
    '''trainer for single-gpu training.
    '''
    def __init__(self, args=None):
        pass

    def train(self,
        model,
        train_objectives: Iterable[Tuple[DataLoader, nn.Module]],
        eval_dataloader = None,
        evaluator=None,
        epochs: int = 1,
        steps_per_epoch = None,
        scheduler: str = 'WarmupCosine',
        warmup_steps: int = 10000,
        warmup_ratio: float = 0.01,
        optimizer_class: Type[Optimizer] = torch.optim.AdamW,
        optimizer_params : Dict[str, object]= {'lr': 2e-5},
        weight_decay: float = 0.01,
        evaluation_steps: int = 100,
        save_steps : int = 100,
        output_path: str = None,
        save_best_model: bool = True,
        max_grad_norm: float = 1,
        use_amp: bool = False,
        accumulation_steps: int = 1,
        callback: Callable[[float, int, int], None] = None,
        show_progress_bar: bool = True,
        checkpoint_path: str = None,
        checkpoint_save_total_limit: int = 0,
        load_best_model_at_last: bool = True,
        ):
        '''
        output_path: model save path
        checkpoint_path: model load and continue to learn path
        '''
        self.best_score = -9999999
        self.accumulation_steps = accumulation_steps
        if use_amp:
            from torch.cuda.amp import autocast
            scaler = torch.cuda.amp.GradScaler()

        self.score_logs = defaultdict(list)
        self.evaluator = evaluator
        self.eval_dataloader = eval_dataloader

        dataloaders = [dataloader for dataloader,_,_ in train_objectives]
        if steps_per_epoch is None or steps_per_epoch == 0:
            steps_per_epoch = min([len(dataloader) for dataloader in dataloaders])
        num_train_steps = int((steps_per_epoch) * epochs)
        warmup_steps = math.ceil(num_train_steps * warmup_ratio) #10% of train data for warm-up

        loss_models = [loss for _, loss,_ in train_objectives]
        train_weights = [weight for _,_,weight in train_objectives]

        # Prepare optimizers
        optimizers = []
        schedulers = []
        for loss_model in loss_models:
            param_optimizer = list(loss_model.named_parameters())

            no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
            ]

            optimizer = optimizer_class(optimizer_grouped_parameters, **optimizer_params)
            scheduler_obj = self._get_scheduler(optimizer, scheduler=scheduler, warmup_steps=warmup_steps, t_total=num_train_steps)

            optimizers.append(optimizer)
            schedulers.append(scheduler_obj)

        # map models to devices
        model = model.cuda()

        # execute training on multiple GPUs
        global_step = 0
        data_iterators = [iter(dataloader) for dataloader in dataloaders]

        num_train_objectives = len(train_objectives)

        skip_scheduler = False
        train_loss_dict = defaultdict(list)
        for epoch in trange(epochs, desc="Epoch", disable=not show_progress_bar):
            training_steps = 0
            for train_iter in trange(steps_per_epoch, desc="Iteration", smoothing=0.05, disable=not show_progress_bar):

                # check if model parameters keep same
                for train_idx in range(num_train_objectives):
                    loss_model = loss_models[train_idx]
                    loss_model.zero_grad()
                    loss_model.train()

                    loss_weight = train_weights[train_idx]
                    optimizer = optimizers[train_idx]
                    scheduler = schedulers[train_idx]
                    data_iterator = data_iterators[train_idx]

                    try:
                        data = next(data_iterator)
                    except StopIteration:
                        # for train_idx in range(num_train_objectives):
                        if '_build_prompt_sentence' in dir(dataloaders[train_idx].dataset):
                            dataloaders[train_idx].dataset._build_prompt_sentence()
                        data_iterator = iter(dataloaders[train_idx])
                        data_iterators[train_idx] = data_iterator
                        data = next(data_iterator)

                    if use_amp:
                        with autocast():
                            loss_model_return = loss_model(**data)
                        loss_value = loss_weight * loss_model_return['loss_value']
                        loss_value = loss_value
                        scale_before_step = scaler.get_scale()
                        scaler.scale(loss_value).backward()
                        scaler.unscale_(optimizer)
                        torch.nn.utils.clip_grad_norm_(loss_model.parameters(), max_grad_norm)
                        scaler.step(optimizer)
                        scaler.update()
                        skip_scheduler = scaler.get_scale() != scale_before_step
                    else:
                        loss_model_return = loss_model(**data)
                        loss_value = loss_weight * loss_model_return['loss_value'] / self.accumulation_steps
                        loss_value.backward()
                        torch.nn.utils.clip_grad_norm_(loss_model.parameters(), max_grad_norm)
                        optimizer.step()

                    train_loss_dict[train_idx].append(loss_value.item())
                    optimizer.zero_grad()

                if not skip_scheduler:
                    scheduler.step()

                training_steps += 1
                global_step += 1

                if evaluation_steps>0 and global_step % evaluation_steps == 0:
                    print('\n######### Train Loss #########')
                    for key in train_loss_dict.keys():
                        print('{} {:.4f} \n'.format(key, np.mean(train_loss_dict[key])))
                    train_loss_dict = defaultdict(list)

                    #TODO: update prompt sentences
                    # for train_idx in range(num_train_objectives):
                    #     if '_build_prompt_sentence' in dir(dataloaders[train_idx].dataset):
                    #         dataloaders[train_idx].dataset._build_prompt_sentence()

                # if evaluation_steps > 0 and global_step % evaluation_steps == 0 and self.evaluator is not None:
                #     scores = self.evaluator.evaluate()
                #     print(f'\n######### Eval {global_step} #########')
                #     for key in scores.keys():
                #         if key in ['acc','auc']:
                #             print('{}: {:.4f}'.format(key, scores[key]))
                #     # save_dir =  os.path.join(output_path, f'{global_step}/')
                #     # self._save_ckpt(model, save_dir)

                #     # score logs save the list of scores
                #     self.score_logs['global_step'].append(global_step)
                #     for key in scores.keys():
                #         if key in ['acc','auc']:
                #             self.score_logs[key].append(scores[key])

                if evaluation_steps > 0 and global_step % evaluation_steps == 0 and self.evaluator is not None:
                    scores = self.evaluator.evaluate()
                    print(f'\n######### Eval {global_step} #########')
                    for key in scores.keys():
                        if key in ['acc','auc']:
                            print('{}: {:.4f}'.format(key, scores[key]))
                
                    # Track logs
                    self.score_logs['global_step'].append(global_step)
                    for key in scores.keys():
                        if key in ['acc','auc']:
                            self.score_logs[key].append(scores[key])
                
                    # Compute average score for checkpoint selection
                    current_avg_score = np.mean([scores[k] for k in scores if k in ['acc', 'auc']])
                
                    if current_avg_score > self.best_score:
                        print(f'New best score: {current_avg_score:.4f} at step {global_step}. Saving model...')
                        self.best_score = current_avg_score
                
                        if output_path is not None:
                            best_save_path = os.path.join(output_path, 'best')
                            if not os.path.exists(best_save_path): os.makedirs(best_save_path)
                            self._save_ckpt(model, best_save_path)


                if self.evaluator is None and global_step % save_steps == 0:
                    state_dict = model.state_dict()
                    save_dir =  os.path.join(output_path, f'{global_step}/')
                    self._save_ckpt(model, save_dir)
                    print('model saved to', os.path.join(output_path, WEIGHTS_NAME))

        if save_best_model:
            import pandas as pd
            from distutils.dir_util import copy_tree
            # res = pd.DataFrame(self.score_logs)
            # res = res.set_index('global_step')
            # # take the average column best
            # best_iter = res.mean(1).idxmax()
            # best_save_path = os.path.join(output_path, './best')
            # if not os.path.exists(best_save_path): os.makedirs(best_save_path)
            # best_origin_path = os.path.join(output_path, f'./{best_iter}')
            # print(f'save best checkpoint at iter {best_iter} to', best_save_path)
            # copy_tree(best_origin_path, best_save_path)

            res = pd.DataFrame(self.score_logs)
            res = res.set_index('global_step')
            best_iter = res.mean(1).idxmax()
            best_save_path = os.path.join(output_path, './best')
            if not os.path.exists(best_save_path): os.makedirs(best_save_path)
            print(f'saving best model at iter {best_iter} to {best_save_path}')
            torch.save(model.state_dict(), os.path.join(best_save_path, WEIGHTS_NAME))


        if eval_dataloader is None and output_path is not None:   #No evaluator, but output path: save final model version
            state_dict = model.state_dict()
            torch.save(state_dict, os.path.join(output_path, WEIGHTS_NAME))
            print('model saved to', os.path.join(output_path, WEIGHTS_NAME))

        if eval_dataloader is not None and load_best_model_at_last and save_best_model and evaluator is not None:
            state_dict = torch.load(os.path.join(best_save_path, WEIGHTS_NAME))
            model.load_state_dict(state_dict)
            print(f'load best checkpoint at last from {best_save_path}')

    @staticmethod
    def _get_scheduler(optimizer, scheduler: str, warmup_steps: int, t_total: int):
        """
        Returns the correct learning rate scheduler. Available scheduler: constantlr, warmupconstant, warmuplinear, warmupcosine, warmupcosinewithhardrestarts
        """
        scheduler = scheduler.lower()
        if scheduler == 'constantlr':
            return transformers.get_constant_schedule(optimizer)
        elif scheduler == 'warmupconstant':
            return transformers.get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)
        elif scheduler == 'warmuplinear':
            return transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
        elif scheduler == 'warmupcosine':
            return transformers.get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
        elif scheduler == 'warmupcosinewithhardrestarts':
            return transformers.get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)
        else:
            raise ValueError("Unknown scheduler {}".format(scheduler))

    def _save_ckpt(self, model, save_dir):
        if not os.path.exists(save_dir): os.makedirs(save_dir)
        state_dict = model.state_dict()
        torch.save(state_dict, os.path.join(save_dir, WEIGHTS_NAME))


Overwriting medclip/trainer.py


In [20]:
%%writefile "medclip/prompts.py"
import random
import pdb
from collections import defaultdict

from transformers import AutoTokenizer

from . import constants

# def generate_class_prompts(df_sent, task=None, n=100):
#     '''args:
#     df_sent: pd.DataFrame with sentence labels, columns=['Reports', 'task1', 'task2',...]
#     task: the specified task to build prompts
#     n: number of prompts for each task
#     '''
#     df_sent = df_sent.fillna(0)
#     df_sent = df_sent.loc[df_sent['Reports'].map(len)>4].reset_index(drop=True)
#     prompts = {}
#     all_tasks = df_sent.columns.tolist()[1:]
#     if task is not None:
#         if isinstance(task, list):
#             target_tasks = task
#         else:
#             target_tasks = [task]
#     else:
#         target_tasks = all_tasks

#     print("Target tasks: ", target_tasks)
#     print("df_sent: ", df_sent)

#     for task in target_tasks:
#         other_tasks = [t for t in all_tasks if t != task]
#         df_sub_sent = df_sent[(df_sent[task] == 1) & (df_sent[other_tasks] == 0).all(1)]
#         if n is not None:
#             if len(df_sub_sent) > n: df_sub_sent = df_sub_sent.sample(n)
#         prompts[task] = df_sub_sent['Reports'].values.tolist()
#     print("From prompts.py file: ", prompts)
#     return prompts

def generate_class_prompts(df_sent, task=None, n=100):
    df_sent = df_sent.fillna(0)
    df_sent = df_sent[df_sent['Reports'].map(len) > 4].reset_index(drop=True)
    prompts = {}

    all_tasks = df_sent.columns.tolist()[1:]
    target_tasks = task if isinstance(task, list) else [task] if task else all_tasks

    for t in target_tasks:
        df_task = df_sent[df_sent[t] == 1]
        if n is not None and len(df_task) > n:
            df_task = df_task.sample(n)
        prompts[t] = df_task['Reports'].tolist()

    # print("In prompts.py: ", prompts)
    return prompts


def generate_chexpert_class_prompts(n = None):
    """Generate text prompts for each CheXpert classification task
    Parameters
    ----------
    n:  int
        number of prompts per class
    Returns
    -------
    class prompts : dict
        dictionary of class to prompts
    """

    prompts = {}
    for k, v in constants.CHEXPERT_CLASS_PROMPTS.items():
        cls_prompts = []
        keys = list(v.keys())

        # severity
        for k0 in v[keys[0]]:
            # subtype
            for k1 in v[keys[1]]:
                # location
                for k2 in v[keys[2]]:
                    cls_prompts.append(f"{k0} {k1} {k2}")

        # randomly sample n prompts for zero-shot classification
        # TODO: we shall make use all the candidate prompts for autoprompt tuning
        if n is not None and n < len(cls_prompts):
            prompts[k] = random.sample(cls_prompts, n)
        else:
            prompts[k] = cls_prompts
        print(f'sample {len(prompts[k])} num of prompts for {k} from total {len(cls_prompts)}')
    return prompts

def generate_covid_class_prompts(n = None):
    prompts = {}
    for k, v in constants.COVID_CLASS_PROMPTS.items():
        cls_prompts = []
        keys = list(v.keys())

        for k0 in v[keys[0]]:
            for k1 in v[keys[1]]:
                for k2 in v[keys[2]]:
                    for k3 in v[keys[3]]:
                        cls_prompts.append(f"{k0} {k1} {k2} {k3}")

        # randomly sample n prompts for zero-shot classification
        if n is not None and n < len(cls_prompts):
            prompts[k] = random.sample(cls_prompts, n)
        else:
            prompts[k] = cls_prompts
        print(f'sample {len(prompts[k])} num of prompts for {k} from total {len(cls_prompts)}')
    return prompts

def generate_rsna_class_prompts(n = None):
    prompts = {}
    for k, v in constants.RSNA_CLASS_PROMPTS.items():
        cls_prompts = []
        keys = list(v.keys())

        for k0 in v[keys[0]]:
            for k1 in v[keys[1]]:
                for k2 in v[keys[2]]:
                    cls_prompts.append(f"{k0} {k1} {k2}")

        # randomly sample n prompts for zero-shot classification
        if n is not None and n < len(cls_prompts):
            prompts[k] = random.sample(cls_prompts, n)
        else:
            prompts[k] = cls_prompts
        print(f'sample {len(prompts[k])} num of prompts for {k} from total {len(cls_prompts)}')
    return prompts

def process_class_prompts(cls_prompts):
    tokenizer = AutoTokenizer.from_pretrained(constants.BERT_TYPE)
    tokenizer.model_max_length = 77
    cls_prompt_inputs = defaultdict()
    for k,v in cls_prompts.items():
        # print(f"process_class_prompts: k = {k}, v = {v}")
        text_inputs = tokenizer(v, truncation=True, padding=True, return_tensors='pt')
        cls_prompt_inputs[k] = text_inputs
    return cls_prompt_inputs


def process_class_prompts_for_tuning(cls_prompts, n_context, class_specific_context):
    tokenizer = AutoTokenizer.from_pretrained(constants.BERT_TYPE)
    tokenizer.model_max_length = 77

    if class_specific_context:
        context = [f'<prompt_{k}_{i}>' for i in range(n_context) for k in cls_prompts]
        num_added_tokens = tokenizer.add_tokens(context)
        assert num_added_tokens == n_context * len(cls_prompts)
    else:
        context = [f'<prompt_{i}>' for i in range(n_context)]
        num_added_tokens = tokenizer.add_tokens(context)
        assert num_added_tokens == n_context

    cls_prompt_inputs = defaultdict()
    for k, v in cls_prompts.items():
        if class_specific_context:
            prefix = ' '.join([f'<prompt_{k}_{i}>' for i in range(n_context)])
        else:
            prefix = ' '.join([f'<prompt_{i}>' for i in range(n_context)])
        context_v = [f'{prefix} {i}' for i in v]
        text_inputs = tokenizer(context_v, truncation=True, padding=True, return_tensors='pt')
        cls_prompt_inputs[k] = text_inputs
    return cls_prompt_inputs



Overwriting medclip/prompts.py


In [21]:
%%writefile "medclip/evaluator.py"
import pdb

import pandas as pd
import numpy as np
from sklearn import multiclass
import torch
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import confusion_matrix, classification_report

from tqdm import tqdm

from . import constants

class Evaluator:
    '''do evaluation on chexpert5x200 zero-shot classification
    '''
    def __init__(self,
        medclip_clf,
        eval_dataloader=None,
        mode=None,
        ) -> None:
        '''specify class_names if doing zero-shot classification.
        mode: `binary`, 'multiclass`, or `multilabel`,
        if set None, the method will automatically decide from data.
        recommend to set explicitly to avoid errors.
        '''
        self.clf = medclip_clf
        self.mode = mode
        self.eval_dataloader = eval_dataloader
    
    def evaluate(self, eval_dataloader=None):
        self.clf.eval()
        if self.eval_dataloader is None and eval_dataloader is not None: self.eval_dataloader = eval_dataloader
        else: eval_dataloader = self.eval_dataloader
        pred_list = []
        label_list = []
        
        for data in tqdm(eval_dataloader, desc='Evaluation'):
            with torch.no_grad():
                outputs = self.clf(**data)
                pred = outputs['logits']
            pred_list.append(pred)
            label_list.append(data['labels'])
            # print("data label: ", data['labels'])

        # print(f"evaluate(): pred_list = {pred_list}")
        # print(f"evaluate(): label_list = {label_list}")
        
        pred_list = torch.cat(pred_list, 0)
        labels = torch.cat(label_list, 0).cpu().detach().numpy()

        pred = pred_list.cpu().detach().numpy()        
        outputs = {'pred':pred, 'labels':labels}

        if self.mode is None:
            if len(labels.shape) == 1:
                if len(np.unique(labels)) == 2:
                    self.mode = 'binary'
                else:
                    self.mode = 'multiclass'
            else:
                self.mode = 'multilabel'
            print(f'no mode specified, will pick mode `{self.mode}` by data.')

        # print(f"Label: {labels}")

        if self.mode == 'binary':
            if pred.shape[1] == 1:
                pred_score = torch.tensor(pred).sigmoid().numpy().flatten()
                auc = roc_auc_score(labels, pred_score)
                outputs['auc'] = auc
                pred_label = np.ones(len(pred))
                pred_label[pred_score<0.5] = 0
                acc = (pred_label == labels).mean()
                outputs['acc'] = acc
                

            else: # have 2 outputs
                pred_score = torch.tensor(pred).sigmoid().numpy()
                pred_label = np.argmax(pred_score, 1)
                acc = (pred_label == labels).mean()
                outputs['acc'] = acc

                # cnf_matrix = confusion_matrix(labels, pred_label)
                # res = self.process_confusion_matrix(cnf_matrix)
                # outputs.update(res)

            res = classification_report(labels, pred_label, output_dict=True)
            res = res['macro avg']
            res.pop('support')
            outputs.update(res)

        if self.mode == 'multiclass':
            pred_label = pred.argmax(1)
            acc = (pred_label == labels).mean()
            outputs['acc'] = acc
            res = classification_report(labels, pred_label, output_dict=True)
            res = res['macro avg']
            res.pop('support')
            outputs.update(res)

            # cnf_matrix = confusion_matrix(labels, pred_label)
            # res = self.process_confusion_matrix(cnf_matrix)
            # outputs.update(res)
        
        if self.mode == 'multilabel':
            pred_score = torch.tensor(pred).sigmoid().numpy()
            auroc_list, auprc_list = [], []
            for i in range(pred_score.shape[1]):
                y_cls = labels[:, i]
                pred_cls = pred_score[:, i]
                auprc_list.append(average_precision_score(y_cls, pred_cls))
                auroc_list.append(roc_auc_score(y_cls, pred_cls))
            outputs['auc'] = np.mean(auroc_list)
            outputs['auprc'] = np.mean(auprc_list)
        return outputs
    
    def process_confusion_matrix(self, cnf_matrix):
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        outputs = {}
        # Sensitivity, hit rate, recall, or true positive rate
        outputs['tpr'] = TP/(TP+FN)
        # Specificity or true negative rate
        outputs['tnr'] = TN/(TN+FP) 
        # Precision or positive predictive value
        outputs['ppv'] = TP/(TP+FP)
        # Negative predictive value
        outputs['npv'] = TN/(TN+FN)
        # Fall out or false positive rate
        outputs['fpr'] = FP/(FP+TN)
        # False negative rate
        outputs['fnr'] = FN/(TP+FN)
        # False discovery rate
        outputs['fdr'] = FP/(TP+FP)

        # Overall accuracy for each class
        # outputs['acc'] = (TP+TN)/(TP+FP+FN+TN)
        if cnf_matrix.shape[0] > 2: # multiclass
            for k,v in outputs.items(): # take macro avg over each class
                outputs[k] = np.mean(v)
        else:
            for k,v in outputs.items(): # take macro avg over each class
                outputs[k] = v[1]
        return outputs


Overwriting medclip/evaluator.py


In [22]:
%%writefile "medclip/losses.py"
from torch import nn
import torch.nn.functional as F
import torch
import pdb
import numpy as np

class ImageTextContrastiveLoss(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
    def forward(self,
        input_ids=None,
        pixel_values=None,
        attention_mask=None,
        img_labels=None,
        text_labels=None,
        aug_input_ids=None,
        aug_attention_mask=None,
        **kwargs,
        ):
        '''args:
        labels: the image corresponds to which classes of diagnoses
        text_labels: the text corresponds to which classes of diagnoses
        '''
        if img_labels is None or text_labels is None:
            '''use hard clip loss as the original clip
            '''
            outputs = self.model(
                    input_ids=input_ids,
                    pixel_values=pixel_values,
                    attention_mask=attention_mask,
                    return_loss=True,
                    )
        else:
            '''use soft clip loss
            '''
            outputs = self.model(
                    input_ids=input_ids,
                    pixel_values=pixel_values,
                    attention_mask=attention_mask,
                    return_loss=False,
                    )

            # get logits
            logits = outputs['logits']

            # compute soft-labels, -1: negative, 0: uncertain, 1: positive
            # in the original data: 1: positive, 0: negative, -1: uncertain, NA: not mentioned
            # print(f"img_labels: {img_labels}")
            # print(f"text_labels: {text_labels}")
            label_sim = torch.matmul(img_labels, text_labels.T)
            label_sim = label_sim.to(logits.device)

            if aug_input_ids is not None:
                aug_text_embeds = self.model.encode_text(aug_input_ids, aug_attention_mask)
                img_embeds = outputs['img_embeds']
                logits_aug = self.model.compute_logits(img_embeds, aug_text_embeds)
                aug_loss_value = self._soft_clip_loss(logits_aug, label_sim)
                loss_value = self._soft_clip_loss(logits, label_sim)
                outputs['loss_value'] = (aug_loss_value + loss_value) / 2
            else:
                outputs['loss_value'] = self._soft_clip_loss(logits, label_sim)

        return_res = {
            'loss_value': outputs['loss_value'],
        }
        return return_res

    def _soft_clip_loss(self, logits_per_img, soft_label):
        '''take labels of images and sentences as a softlabel
        e.g., image_label = [1, 0, 1, -1], sentence_label = [0, 0, 1, -1]
        this pair has similarity as: 1 * 0 + 0 * 0 + 1 * 1 + -1 * -1 = 2.
        We will clamp the similarity into [-1,1], and take softmax as a soft-label.
        '''
        # when using InfoNCE-like loss
        image_loss = self._soft_xent_loss(logits_per_img, F.softmax(soft_label,1))
        caption_loss = self._soft_xent_loss(logits_per_img.T, F.softmax(soft_label.T,1))
        return (image_loss + caption_loss) / 2

        # when using multilabel bce loss
        # image_loss = self._soft_bce_loss(logits_per_img, soft_label)
        # return image_loss

    def _soft_xent_loss(self, input, target):
        logprobs = torch.nn.functional.log_softmax(input, dim = 1)
        return  -(target * logprobs).sum() / input.shape[0]

    def _soft_bce_loss(self, input, target):
        return nn.functional.binary_cross_entropy_with_logits(input, target)


class ImageSuperviseLoss(nn.Module):
    def __init__(self,
        model,
        loss_fn=None,
        ):
        super().__init__()
        self.model = model
        self.mode = model.mode
        if loss_fn is None:
            if self.mode in ['multilabel','binary']:
                self.loss_fn = nn.BCEWithLogitsLoss()
            else:
                self.loss_fn = nn.CrossEntropyLoss()
        else:
            self.loss_fn = loss_fn

    def forward(self,
        pixel_values,
        labels=None,
        **kwargs):
        outputs = self.model(pixel_values=pixel_values, labels=labels, return_loss=True)
        # mix_x, y_a, y_b, lamb = self.mixup_data(pixel_values, labels)
        # outputs = self.model(pixel_values=mix_x, labels=labels, return_loss=False)
        # y_a = y_a.cuda()
        # y_b = y_b.cuda()
        # loss = self.mixup_criterion(self.loss_fn, outputs['logits'], y_a, y_b, lamb)
        # outputs['loss_value'] = loss
        return outputs

    def mixup_data(self, x, y, alpha=0.3):
        if alpha > 0: lamb = np.random.beta(alpha, alpha)
        else: lamb = 1
        batch_size = x.shape[0]
        index = torch.randperm(batch_size).to(x.device)
        mixed_x = lamb * x + (1 - lamb) * x[index, :]
        y_a, y_b = y, y[index]
        return mixed_x, y_a, y_b, lamb

    def mixup_criterion(self, criterion, pred, y_a, y_b, lamb):
        return lamb * criterion(pred, y_a) + (1- lamb) * criterion(pred, y_b)


Overwriting medclip/losses.py


In [1]:
# %load run_medclip_pretrain.py
import pdb, os
import random

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

from medclip.modeling_medclip import MedCLIPModel, PromptClassifier, MedCLIPVisionModel, MedCLIPVisionModelViT
from medclip.dataset import ImageTextContrastiveDataset, ZeroShotImageDataset
from medclip.dataset import ImageTextContrastiveCollator, ZeroShotImageCollator
from medclip.losses import ImageTextContrastiveLoss
from medclip.trainer import Trainer
from medclip.evaluator import Evaluator
from medclip import constants
from medclip.prompts import generate_class_prompts, generate_chexpert_class_prompts, generate_covid_class_prompts

# set random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
os.environ['PYTHONASHSEED'] = str(seed)
os.environ['TOKENIZERS_PARALLELISM']='false'

# set cuda devices
os.environ['CUDA_VISIBLE_DEVICES']='0'
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# set training configurations
train_config = {
    'batch_size': 128,
    'num_epochs': 20,
    'warmup': 0.1, # the first 10% of training steps are used for warm-up
    'lr': 2e-5,
    'weight_decay': 1e-4,
    'eval_batch_size': 256,
    'eval_steps': 20,
    'save_steps': 20,
}

# only pretrain on chexpert train data and mimic-cxr data
# do zero-shot training on chexpert-5x200 and iuxray
datalist = [
    'lbp-train',
]

transform = transforms.Compose([
                transforms.RandomHorizontalFlip(0.5),
                transforms.ColorJitter(0.2,0.2),
                transforms.RandomAffine(degrees=10, scale=(0.8,1.1), translate=(0.0625,0.0625)),
                transforms.Resize((256, 256)),
                transforms.RandomCrop((constants.IMG_SIZE, constants.IMG_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[constants.IMG_MEAN],std=[constants.IMG_STD])],
            )

traindata = ImageTextContrastiveDataset(datalist=datalist, imgtransform=transform)
train_collate_fn = ImageTextContrastiveCollator()
trainloader = DataLoader(traindata,
    batch_size=train_config['batch_size'],
    collate_fn=train_collate_fn,
    shuffle=True,
    pin_memory=True,
    num_workers=12,
    )

# build medclip model
model = MedCLIPModel(vision_cls=MedCLIPVisionModel)
model.cuda()

# build evaluator
# cls_prompts = generate_chexpert_class_prompts(n=10)
df_sent = pd.read_csv('./local_data/sentence-label.csv')
cls_prompts = generate_class_prompts(df_sent, task = ['No Finding', 'LBP'], n = 10)
# print(cls_prompts)

val_data = ZeroShotImageDataset(['lbp-valid'],
    class_names=["No Finding", "LBP"])
val_collate_fn = ZeroShotImageCollator(cls_prompts=cls_prompts,
    mode='multiclass')
eval_dataloader = DataLoader(val_data,
    batch_size=train_config['eval_batch_size'],
    collate_fn=val_collate_fn,
    shuffle=False,
    pin_memory=True,
    num_workers=4,
    )
medclip_clf = PromptClassifier(model)
evaluator = Evaluator(
    medclip_clf=medclip_clf,
    eval_dataloader=eval_dataloader,
    mode='multiclass',
)

# build loss models and start training
loss_model = ImageTextContrastiveLoss(model)
loss_model.cuda()
train_objectives = [
    (trainloader, loss_model, 1),
]
model_save_path = f'./checkpoints/vision_text_pretrain'
trainer = Trainer()
trainer.train(
    model,
    train_objectives=train_objectives,
    warmup_ratio=train_config['warmup'],
    epochs=train_config['num_epochs'],
    optimizer_params={'lr':train_config['lr']},
    output_path=model_save_path,
    evaluation_steps=train_config['eval_steps'],
    weight_decay=train_config['weight_decay'],
    save_steps=train_config['save_steps'],
    evaluator=evaluator,
    eval_dataloader=eval_dataloader,
    use_amp=True,
    )
print('done')


load data from ./local_data/lbp-train-meta.csv
load sentence prompts from ./local_data/sentence-label.csv
build prompt sentences.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' ar

load data from ./local_data/lbp-valid-meta.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

...


######### Eval 360 #########
acc: 0.8373


Iteration:   0%|          | 0/19 [00:00<?, ?it/s]

build prompt sentences.

######### Train Loss #########
0 4.6589 





Evaluation:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluation:  50%|█████     | 1/2 [00:01<00:01,  1.46s/it]

Evaluation: 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]



######### Eval 380 #########
acc: 0.8373
saving best model at iter 40 to ./checkpoints/vision_text_pretrain/./best
load best checkpoint at last from ./checkpoints/vision_text_pretrain/./best
done


In [17]:
%%writefile /kaggle/working/MedCLIP/local_data/lbp-test-spider-meta.csv
,imgpath,report,No Finding,LBP
0,local_data/spider-lumbar-mri/209_t2_SPACE.png,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2-8 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Herniation in discs 1-2. Spondylolisthesis in discs 1-2. Modic type 2 in discs 1. Low endplate in discs 2-3. Up endplate in discs 2-3. Bulging in discs 3-4.,0,1
1,local_data/spider-lumbar-mri/77_t1.png,"A T1-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.",1,0
2,local_data/spider-lumbar-mri/P79 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
3,local_data/spider-lumbar-mri/P19-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
4,local_data/spider-lumbar-mri/P16-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
5,local_data/spider-lumbar-mri/P89 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
6,local_data/spider-lumbar-mri/P75 5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
7,local_data/spider-lumbar-mri/167_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 6-7 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5. Modic type 2 in discs 3.",0,1
8,local_data/spider-lumbar-mri/P57-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
9,local_data/spider-lumbar-mri/P3-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
10,local_data/spider-lumbar-mri/P96 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
11,local_data/spider-lumbar-mri/18_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 6 show Pfirrmann grade 2. Discs 3-4, 7-8 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1, 5 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-5, 7-8. Bulging in discs 1-5, 7-8. Low endplate in discs 1, 5. Up endplate in discs 1, 5. Modic type 2 in discs 1-2, 5.",0,1
12,local_data/spider-lumbar-mri/P23-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
13,local_data/spider-lumbar-mri/P97 3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
14,local_data/spider-lumbar-mri/P74 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
15,local_data/spider-lumbar-mri/144_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5 show Pfirrmann grade 1. Discs 2-3, 6 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Herniation in discs 1. Low endplate in discs 2, 4, 6. Up endplate in discs 3-4, 6. Narrowing in discs 4. Bulging in discs 4.",0,1
16,local_data/spider-lumbar-mri/254_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 4 show Pfirrmann grade 2. Discs 5-6 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 5. Narrowing in discs 1, 3, 5-6. Bulging in discs 1-2, 5-6. Herniation in discs 3.",0,1
17,local_data/spider-lumbar-mri/67_t2.png,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 2. Discs 1-2 show Pfirrmann grade 3. Bulging in discs 1, 3. Herniation in discs 2.",0,1
18,local_data/spider-lumbar-mri/P22-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
19,local_data/spider-lumbar-mri/154_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 3. Discs 2-4 show Pfirrmann grade 4. Bulging in discs 1-4. Modic type 2 in discs 1, 3-4. Narrowing in discs 2-4. Low endplate in discs 2-4, 7. Up endplate in discs 2-4, 7. Modic type 1 in discs 2. Herniation in discs 3.",0,1
20,local_data/spider-lumbar-mri/48_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5 show Pfirrmann grade 2. Discs 1-4, 6-7 show Pfirrmann grade 3. Bulging in discs 1-3. Low endplate in discs 2-4, 6-7. Up endplate in discs 3-6.",0,1
21,local_data/spider-lumbar-mri/187_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 2. Low endplate in discs 1, 7. Bulging in discs 7.",0,1
22,local_data/spider-lumbar-mri/116_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 2. Discs 6 show Pfirrmann grade 3. Discs 2, 5 show Pfirrmann grade 4. Discs 1, 3-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
23,local_data/spider-lumbar-mri/P3-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
24,local_data/spider-lumbar-mri/P48-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
25,local_data/spider-lumbar-mri/P23-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
26,local_data/spider-lumbar-mri/121_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 1. Discs 3-4 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Modic type 2 in discs 1-3. Up endplate in discs 2.",0,1
27,local_data/spider-lumbar-mri/39_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Narrowing in discs 2. Low endplate in discs 5. Up endplate in discs 5. Modic type 2 in discs 5.,0,1
28,local_data/spider-lumbar-mri/P44-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
29,local_data/spider-lumbar-mri/P91 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
30,local_data/spider-lumbar-mri/P59-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
31,local_data/spider-lumbar-mri/P38-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
32,local_data/spider-lumbar-mri/145_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 3. Discs 1, 3-5 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1, 3-5. Up endplate in discs 1, 3-5. Spondylolisthesis in discs 1. Modic type 2 in discs 3-4.",0,1
33,local_data/spider-lumbar-mri/105_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-2, 5-6 show Pfirrmann grade 2. Discs 7 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-3, 5. Low endplate in discs 2-4. Up endplate in discs 2-4, 6-7. Modic type 2 in discs 2, 4. Narrowing in discs 3-4. Herniation in discs 4.",0,1
34,local_data/spider-lumbar-mri/19_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-5 show Pfirrmann grade 1. Discs 1-3, 6 show Pfirrmann grade 3. Narrowing in discs 1. Bulging in discs 1. Modic type 2 in discs 1-3. Up endplate in discs 4-6. Low endplate in discs 5-6.",0,1
35,local_data/spider-lumbar-mri/P44-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
36,local_data/spider-lumbar-mri/236_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 1, 5 show Pfirrmann grade 4. Discs 2-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2-5.",0,1
37,local_data/spider-lumbar-mri/P50-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
38,local_data/spider-lumbar-mri/P18-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
39,local_data/spider-lumbar-mri/P89 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
40,local_data/spider-lumbar-mri/161_t2_SPACE.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 8 discs. Discs 1, 3-7 show Pfirrmann grade 4. Discs 2, 8 show Pfirrmann grade 5. Narrowing in discs 1-3, 8. Bulging in discs 1-5, 7-8. Low endplate in discs 1-8. Up endplate in discs 1-8. Modic type 2 in discs 1-8.",0,1
41,local_data/spider-lumbar-mri/P10-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
42,local_data/spider-lumbar-mri/71_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1 show Pfirrmann grade 3. Discs 3, 5, 7-8 show Pfirrmann grade 4. Discs 2, 4, 6 show Pfirrmann grade 5. Narrowing in discs 1-2, 4, 6. Bulging in discs 1-6. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-2, 4, 6. Spondylolisthesis in discs 4.",0,1
43,local_data/spider-lumbar-mri/P35-2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
44,local_data/spider-lumbar-mri/P59-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
45,local_data/spider-lumbar-mri/209_t2.png,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2-8 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Herniation in discs 1-2. Spondylolisthesis in discs 1-2. Modic type 2 in discs 1. Low endplate in discs 2-3. Up endplate in discs 2-3. Bulging in discs 3-4.,0,1
46,local_data/spider-lumbar-mri/P40-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
47,local_data/spider-lumbar-mri/161_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 1, 3-7 show Pfirrmann grade 4. Discs 2, 8 show Pfirrmann grade 5. Narrowing in discs 1-3, 8. Bulging in discs 1-5, 7-8. Low endplate in discs 1-8. Up endplate in discs 1-8. Modic type 2 in discs 1-8.",0,1
48,local_data/spider-lumbar-mri/5_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1-7 show Pfirrmann grade 2. Bulging in discs 1-3.,0,1
49,local_data/spider-lumbar-mri/P43-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
50,local_data/spider-lumbar-mri/P57-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
51,local_data/spider-lumbar-mri/P25-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
52,local_data/spider-lumbar-mri/P15-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
53,local_data/spider-lumbar-mri/51_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 7-8 show Pfirrmann grade 1. Discs 5-6 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-4. Low endplate in discs 1, 3. Up endplate in discs 1, 3. Modic type 2 in discs 1.",0,1
54,local_data/spider-lumbar-mri/212_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 6 show Pfirrmann grade 3. Discs 2-5, 7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2, 4, 7. Bulging in discs 1, 3-5, 7. Low endplate in discs 1, 5. Up endplate in discs 1. Herniation in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1-2, 4.",0,1
55,local_data/spider-lumbar-mri/P36-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
56,local_data/spider-lumbar-mri/155_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 3. Discs 2-3 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1, 3. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1, 4. Modic type 2 in discs 1-5.",0,1
57,local_data/spider-lumbar-mri/P98 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
58,local_data/spider-lumbar-mri/P21-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
59,local_data/spider-lumbar-mri/P83 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
60,local_data/spider-lumbar-mri/P46-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
61,local_data/spider-lumbar-mri/37_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Bulging in discs 1-3. Low endplate in discs 1, 3. Up endplate in discs 1, 3. Narrowing in discs 3. Modic type 2 in discs 3.",0,1
62,local_data/spider-lumbar-mri/226_t2.png,"A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 2, 5-7 show Pfirrmann grade 3. Discs 3 show Pfirrmann grade 4. Discs 1, 4 show Pfirrmann grade 5. Narrowing in discs 1, 3-4, 6-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Modic type 2 in discs 1-3, 5, 7. Modic type 3 in discs 4, 6.",0,1
63,local_data/spider-lumbar-mri/P47-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
64,local_data/spider-lumbar-mri/24_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 8 show Pfirrmann grade 1. Discs 7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-6. Up endplate in discs 1-6. Modic type 2 in discs 1-6. Herniation in discs 2.",0,1
65,local_data/spider-lumbar-mri/115_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 5-7 show Pfirrmann grade 2. Discs 2-4 show Pfirrmann grade 3. Bulging in discs 2-4. Low endplate in discs 2-4. Up endplate in discs 2-4. Modic type 2 in discs 2-3.",0,1
66,local_data/spider-lumbar-mri/P14-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
67,local_data/spider-lumbar-mri/171_t1.png,"A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 3-6 show Pfirrmann grade 3. Discs 2, 7-8 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-6. Low endplate in discs 1-4, 6-8. Up endplate in discs 1-4. Modic type 2 in discs 1-8. Narrowing in discs 2-3.",0,1
68,local_data/spider-lumbar-mri/140_t1.png,"A T1-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 6-9 show Pfirrmann grade 1. Discs 1-3 show Pfirrmann grade 3. Discs 4-5 show Pfirrmann grade 4. Modic type 2 in discs 1, 3, 7. Narrowing in discs 3-5. Herniation in discs 3. Bulging in discs 4-5. Modic type 3 in discs 5. Up endplate in discs 8.",0,1
69,local_data/spider-lumbar-mri/P51-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
70,local_data/spider-lumbar-mri/195_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-2. Modic type 2 in discs 1-2.,0,1
71,local_data/spider-lumbar-mri/P91 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
72,local_data/spider-lumbar-mri/244_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 5-7 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-7. Modic type 2 in discs 1-5. Low endplate in discs 2, 4-7. Up endplate in discs 2, 4-7.",0,1
73,local_data/spider-lumbar-mri/P28-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
74,local_data/spider-lumbar-mri/P39-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
75,local_data/spider-lumbar-mri/P76 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
76,local_data/spider-lumbar-mri/136_t1.png,A T1-weighted MRI of a female patient with 6 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 3-7. Up endplate in discs 3-7. Modic type 2 in discs 4.,0,1
77,local_data/spider-lumbar-mri/38_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Discs 1-2 show Pfirrmann grade 5. Narrowing in discs 1-2, 4-7. Bulging in discs 1-7. Low endplate in discs 1-7. Up endplate in discs 1-7. Spondylolisthesis in discs 1. Modic type 2 in discs 1-7.",0,1
78,local_data/spider-lumbar-mri/218_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 2. Low endplate in discs 2. Up endplate in discs 2. Spondylolisthesis in discs 2.,0,1
79,local_data/spider-lumbar-mri/P61-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
80,local_data/spider-lumbar-mri/P97 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
81,local_data/spider-lumbar-mri/P12-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
82,local_data/spider-lumbar-mri/233_t2.png,A T2-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1-2. Bulging in discs 1. Herniation in discs 1. Modic type 2 in discs 1.,0,1
83,local_data/spider-lumbar-mri/220_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-2, 6-8 show Pfirrmann grade 4. Discs 3-5 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 2-8. Up endplate in discs 2-8. Narrowing in discs 3-5. Modic type 2 in discs 3-6.",0,1
84,local_data/spider-lumbar-mri/P32-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
85,local_data/spider-lumbar-mri/P23-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
86,local_data/spider-lumbar-mri/P55-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
87,local_data/spider-lumbar-mri/96_t2.png,"A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 6-8 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-5. Low endplate in discs 1-3. Up endplate in discs 1-3, 5. Modic type 2 in discs 1-2.",0,1
88,local_data/spider-lumbar-mri/117_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3-6 show Pfirrmann grade 3. Discs 1-2 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 2-5. Spondylolisthesis in discs 2. Low endplate in discs 3-6. Up endplate in discs 3-6. Modic type 2 in discs 3-6.,0,1
89,local_data/spider-lumbar-mri/P3-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
90,local_data/spider-lumbar-mri/P39-1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
91,local_data/spider-lumbar-mri/170_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 6-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 2. Discs 2-5 show Pfirrmann grade 3. Low endplate in discs 1-4. Up endplate in discs 1-4. Modic type 2 in discs 1-4. Bulging in discs 2-5. Narrowing in discs 3-5.,0,1
92,local_data/spider-lumbar-mri/P81 2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
93,local_data/spider-lumbar-mri/192_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 6 show Pfirrmann grade 2. Discs 5 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Discs 1-3 show Pfirrmann grade 5. Narrowing in discs 1-4. Bulging in discs 1-5. Low endplate in discs 1-3. Up endplate in discs 1-3. Modic type 2 in discs 1-3, 5.",0,1
94,local_data/spider-lumbar-mri/P28-3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
95,local_data/spider-lumbar-mri/P98 4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
96,local_data/spider-lumbar-mri/75_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 2 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Herniation in discs 1. Bulging in discs 2-3. Low endplate in discs 2-3. Up endplate in discs 2-3. Modic type 2 in discs 2-4.,0,1
97,local_data/spider-lumbar-mri/131_t1.png,A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1-2 show Pfirrmann grade 3. Narrowing in discs 1-2. Bulging in discs 1-2.,0,1
98,local_data/spider-lumbar-mri/P62-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
99,local_data/spider-lumbar-mri/149_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2-6 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 3. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1.,0,1
100,local_data/spider-lumbar-mri/32_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 2, 5-6 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 2. Discs 1 show Pfirrmann grade 3. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1-2. Low endplate in discs 2. Up endplate in discs 2. Herniation in discs 2. Modic type 2 in discs 2.",0,1
101,local_data/spider-lumbar-mri/P71 1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
102,local_data/spider-lumbar-mri/251_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 7 show Pfirrmann grade 3. Discs 1, 4-6 show Pfirrmann grade 4. Discs 2-3 show Pfirrmann grade 5. Narrowing in discs 1-7. Bulging in discs 2-7. Herniation in discs 2-3, 6. Modic type 2 in discs 2-3. Up endplate in discs 7.",0,1
103,local_data/spider-lumbar-mri/35_t2_SPACE.png,A T2-weighted MRI of a female patient with 8 vertebrae and 9 discs. Discs 4-9 show Pfirrmann grade 1. Discs 3 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Bulging in discs 1-3. Low endplate in discs 3-8. Up endplate in discs 3-8.,0,1
104,local_data/spider-lumbar-mri/P85 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
105,local_data/spider-lumbar-mri/89_t2.png,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. Discs 3, 5-6 show Pfirrmann grade 2. Discs 2, 4 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Low endplate in discs 1. Herniation in discs 1. Bulging in discs 4.",0,1
106,local_data/spider-lumbar-mri/231_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-5, 7 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-3. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.",0,1
107,local_data/spider-lumbar-mri/127_t2_SPACE.png,"A T2-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 5, 7-8 show Pfirrmann grade 2. Discs 2, 6 show Pfirrmann grade 3. Discs 1, 3-4 show Pfirrmann grade 4. Narrowing in discs 1, 3-4. Bulging in discs 1-4, 6. Low endplate in discs 3-4. Up endplate in discs 3-4. Modic type 2 in discs 3-4.",0,1
108,local_data/spider-lumbar-mri/182_t2.png,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 1-3, 6, 9 show Pfirrmann grade 3. Discs 5, 7-8 show Pfirrmann grade 4. Discs 4 show Pfirrmann grade 5. Bulging in discs 1-5. Low endplate in discs 1-9. Up endplate in discs 1-9. Modic type 2 in discs 1-9. Narrowing in discs 4-5, 8.",0,1
109,local_data/spider-lumbar-mri/P57-1.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
110,local_data/spider-lumbar-mri/P18-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
111,local_data/spider-lumbar-mri/P85 3.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
112,local_data/spider-lumbar-mri/P47-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
113,local_data/spider-lumbar-mri/P41-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
114,local_data/spider-lumbar-mri/136_t2.png,A T2-weighted MRI of a female patient with 6 vertebrae and 7 discs. Discs 2-7 show Pfirrmann grade 3. Discs 1 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 3-7. Up endplate in discs 3-7. Modic type 2 in discs 4.,0,1
115,local_data/spider-lumbar-mri/118_t2.png,A T2-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 5-8 show Pfirrmann grade 2. Discs 3-4 show Pfirrmann grade 3. Discs 2 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Modic type 2 in discs 1.,0,1
116,local_data/spider-lumbar-mri/P60-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
117,local_data/spider-lumbar-mri/74_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-5 show Pfirrmann grade 2. Discs 1-3, 6 show Pfirrmann grade 4. Narrowing in discs 1-3, 6. Bulging in discs 1-3. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
118,local_data/spider-lumbar-mri/P33-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
119,local_data/spider-lumbar-mri/229_t1.png,A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 1-6 show Pfirrmann grade 3. Narrowing in discs 2-4. Bulging in discs 2-5. Low endplate in discs 2-6. Up endplate in discs 2-6. Modic type 2 in discs 2-4.,0,1
120,local_data/spider-lumbar-mri/P38-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
121,local_data/spider-lumbar-mri/P52-2.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
122,local_data/spider-lumbar-mri/239_t1.png,"A T1-weighted MRI of a female patient with 7 vertebrae and 7 discs. Discs 1 show Pfirrmann grade 1. Discs 2-3, 5-7 show Pfirrmann grade 2. Discs 4 show Pfirrmann grade 4. Narrowing in discs 1. Bulging in discs 1-4. Herniation in discs 1. Low endplate in discs 4-6. Up endplate in discs 4-6.",0,1
123,local_data/spider-lumbar-mri/P65-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
124,local_data/spider-lumbar-mri/143_t2.png,"A T2-weighted MRI of a male patient with 9 vertebrae and 9 discs. Discs 6-7, 9 show Pfirrmann grade 3. Discs 5, 8 show Pfirrmann grade 4. Discs 1-4 show Pfirrmann grade 5. Narrowing in discs 1-5. Bulging in discs 1-5. Low endplate in discs 1-5. Up endplate in discs 1-5. Modic type 2 in discs 1-5.",0,1
125,local_data/spider-lumbar-mri/2_t2.png,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 3 show Pfirrmann grade 3. Discs 1-2, 4-6 show Pfirrmann grade 4. Bulging in discs 1-5. Narrowing in discs 2-3. Low endplate in discs 2-5. Up endplate in discs 2-5. Modic type 2 in discs 2-5.",0,1
126,local_data/spider-lumbar-mri/166_t1.png,"A T1-weighted MRI of a male patient with 8 vertebrae and 8 discs. Discs 2, 4-6, 8 show Pfirrmann grade 3. Discs 1, 3 show Pfirrmann grade 4. Discs 7 show Pfirrmann grade 5. Narrowing in discs 1, 3, 7. Bulging in discs 1-4. Modic type 2 in discs 2-4. Low endplate in discs 6-7. Up endplate in discs 6-7.",0,1
127,local_data/spider-lumbar-mri/93_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 4-6 show Pfirrmann grade 3. Discs 1-3 show Pfirrmann grade 5. Narrowing in discs 1-3. Bulging in discs 1-2, 4. Low endplate in discs 1-3, 6. Modic type 2 in discs 1-3, 6. Herniation in discs 3.",0,1
128,local_data/spider-lumbar-mri/P30-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
129,local_data/spider-lumbar-mri/184_t2.png,A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 3-7 show Pfirrmann grade 1. Discs 1 show Pfirrmann grade 4. Discs 2 show Pfirrmann grade 5. Narrowing in discs 1-2. Bulging in discs 1-2. Low endplate in discs 1-2. Up endplate in discs 1-2. Spondylolisthesis in discs 1. Modic type 2 in discs 2.,0,1
130,local_data/spider-lumbar-mri/P4-4.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
131,local_data/spider-lumbar-mri/177_t2_SPACE.png,"A T2-weighted MRI of a female patient with 9 vertebrae and 9 discs. Discs 7-9 show Pfirrmann grade 3. Discs 1, 6 show Pfirrmann grade 4. Discs 2-5 show Pfirrmann grade 5. Narrowing in discs 1-6. Bulging in discs 1-7. Low endplate in discs 1-5, 7. Modic type 2 in discs 1-5. Up endplate in discs 2-7. Herniation in discs 7.",0,1
132,local_data/spider-lumbar-mri/167_t1.png,"A T1-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 1, 6-7 show Pfirrmann grade 1. Discs 4-5 show Pfirrmann grade 2. Discs 2-3 show Pfirrmann grade 3. Narrowing in discs 2. Bulging in discs 3-4. Low endplate in discs 3-5. Up endplate in discs 3-5. Modic type 2 in discs 3.",0,1
133,local_data/spider-lumbar-mri/P49-3.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
134,local_data/spider-lumbar-mri/180_t1.png,A T1-weighted MRI of a female patient with 8 vertebrae and 8 discs. Discs 1-8 show Pfirrmann grade 1. Narrowing in discs 1-6. Bulging in discs 1-4. Low endplate in discs 1-7. Up endplate in discs 1-7.,0,1
135,local_data/spider-lumbar-mri/P74 5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
136,local_data/spider-lumbar-mri/P81 1.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
137,local_data/spider-lumbar-mri/P31-5.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
138,local_data/spider-lumbar-mri/P38-4.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
139,local_data/spider-lumbar-mri/15_t1.png,"A T1-weighted MRI of a male patient with 6 vertebrae and 6 discs. Discs 5-6 show Pfirrmann grade 2. Discs 1, 3 show Pfirrmann grade 3. Discs 2, 4 show Pfirrmann grade 5. Narrowing in discs 1-2, 4. Bulging in discs 1-4. Low endplate in discs 1, 4-6. Up endplate in discs 1, 4-6. Modic type 2 in discs 2-5. Herniation in discs 4.",0,1
140,local_data/spider-lumbar-mri/73_t2.png,"A T2-weighted MRI of a male patient with 7 vertebrae and 7 discs. Discs 2-3 show Pfirrmann grade 3. Discs 4-7 show Pfirrmann grade 4. Discs 1 show Pfirrmann grade 5. Narrowing in discs 1. Bulging in discs 1-4. Low endplate in discs 1. Up endplate in discs 1. Spondylolisthesis in discs 1. Modic type 2 in discs 1, 4-7.",0,1
141,local_data/spider-lumbar-mri/P73 2.JPEG,"A T2-weighted MRI of a female patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0
142,local_data/spider-lumbar-mri/P42-5.JPEG,"A T2-weighted MRI of a male patient with 6 vertebrae and 6 discs. All discs show Pfirrmann grade 1. No disc herniation, disc narrowing or disc bulging is observed.Endplates are normal across all levels.No abnormalities are noted.",1,0

Writing /kaggle/working/MedCLIP/local_data/lbp-test-spider-meta.csv


In [ ]:
ls -l ./checkpoints/vision_text_pretrain/best

## **Training MedCLIP and classification head**

In [2]:
# set random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
os.environ['PYTHONASHSEED'] = str(seed)
os.environ['TOKENIZERS_PARALLELISM']='false'

# set cuda devices
os.environ['CUDA_VISIBLE_DEVICES']='0'
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# set training configurations
train_config = {
    'batch_size': 128,
    'num_epochs': 50,
    'warmup': 0.1, # the first 10% of training steps are used for warm-up
    'lr': 2e-5,
    'weight_decay': 1e-4,
    'eval_batch_size': 128,
    'eval_steps': 32,
    'save_steps': 32,
}

# only pretrain on chexpert train data and mimic-cxr data
# do zero-shot training on chexpert-5x200 and iuxray
datalist = [
    'lbp-train',
]

transform = transforms.Compose([
                transforms.RandomHorizontalFlip(0.5),
                transforms.ColorJitter(0.2,0.2),
                transforms.RandomAffine(degrees=10, scale=(0.8,1.1), translate=(0.0625,0.0625)),
                transforms.Resize((256, 256)),
                transforms.RandomCrop((constants.IMG_SIZE, constants.IMG_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[constants.IMG_MEAN],std=[constants.IMG_STD])],
            )

traindata = ImageTextContrastiveDataset(datalist=datalist, imgtransform=transform)
train_collate_fn = ImageTextContrastiveCollator()
trainloader = DataLoader(traindata,
    batch_size=train_config['batch_size'],
    collate_fn=train_collate_fn,
    shuffle=True,
    pin_memory=True,
    num_workers=12,
    )

# build medclip model
model = MedCLIPModel(vision_cls=MedCLIPVisionModelSwin)
model.cuda()

# build evaluator
# cls_prompts = generate_chexpert_class_prompts(n=10)
df_sent = pd.read_csv('./local_data/sentence-label.csv')
cls_prompts = generate_class_prompts(df_sent, task = ['No Finding', 'LBP'], n = 10)
# print(cls_prompts)

val_data = ZeroShotImageDataset(['lbp-valid'],
    class_names=["No Finding", "LBP"])
val_collate_fn = ZeroShotImageCollator(cls_prompts=cls_prompts,
    mode='multiclass')
eval_dataloader = DataLoader(val_data,
    batch_size=train_config['eval_batch_size'],
    collate_fn=val_collate_fn,
    shuffle=False,
    pin_memory=True,
    num_workers=4,
    )
medclip_clf = PromptClassifier(model)
evaluator = Evaluator(
    medclip_clf=medclip_clf,
    eval_dataloader=eval_dataloader,
    mode='multiclass',
)

# build loss models and start training
loss_model = ImageTextContrastiveLoss(model)
loss_model.cuda()
train_objectives = [
    (trainloader, loss_model, 1),
]
model_save_path = f'./checkpoints/vision_text_pretrain'
trainer = Trainer()
trainer.train(
    model,
    train_objectives=train_objectives,
    warmup_ratio=train_config['warmup'],
    epochs=train_config['num_epochs'],
    optimizer_params={'lr':train_config['lr']},
    output_path=model_save_path,
    evaluation_steps=train_config['eval_steps'],
    weight_decay=train_config['weight_decay'],
    save_steps=train_config['save_steps'],
    evaluator=evaluator,
    eval_dataloader=eval_dataloader,
    use_amp=True,
    )
print('done')

load data from ./local_data/lbp-train-meta.csv
load sentence prompts from ./local_data/sentence-label.csv
build prompt sentences.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.we

load data from ./local_data/lbp-valid-meta.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: khang-vo872003 (khang-vo872003-ho-chi-minh-city-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

...

build prompt sentences.

######### Train Loss #########
0 4.7639 





Evaluation:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluation:  25%|██▌       | 1/4 [00:01<00:03,  1.24s/it]

Evaluation:  50%|█████     | 2/4 [00:01<00:01,  1.09it/s]

Evaluation:  75%|███████▌  | 3/4 [00:02<00:00,  1.22it/s]

Evaluation: 100%|██████████| 4/4 [00:03<00:00,  1.19it/s]



######### Eval 928 #########
acc: 0.7679


Iteration:   0%|          | 0/19 [00:00<?, ?it/s]

build prompt sentences.
saving best model at iter 128 to ./checkpoints/vision_text_pretrain/./best
load best checkpoint at last from ./checkpoints/vision_text_pretrain/./best
done


In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from PIL import Image
from medclip import MedCLIPModel, MedCLIPVisionModel, MedCLIPProcessor
from torch.utils.data import WeightedRandomSampler
import numpy as np

# Load MedCLIP model and processor
model = MedCLIPModel(vision_cls=MedCLIPVisionModelSwin)
model.from_pretrained('./checkpoints/vision_text_pretrain/best')
model.cuda()
model.eval()
processor = MedCLIPProcessor()

# Custom dataset
class MRIDataset(Dataset):
    def __init__(self, csv_path):
        self.data = pd.read_csv(csv_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image = Image.open(row['imgpath']).convert("RGB")
        text = row['report']
        label = torch.tensor([row['No Finding'], row['LBP']], dtype=torch.float32)
        inputs = processor(text=[text], images=[image], return_tensors="pt", padding=True)
        with torch.no_grad():
            img_emb = model.encode_image(inputs['pixel_values'].cuda(), use_projection=False)
            text_emb = model.encode_text(inputs['input_ids'].cuda(), inputs['attention_mask'].cuda(), use_projection=False)
        emb = torch.cat([img_emb.squeeze(0), text_emb.squeeze(0)], dim=-1)
        return emb, label


def create_balanced_sampler(dataset):
    # Extract all labels
    labels = [int(dataset.data.iloc[i]['LBP']) for i in range(len(dataset))]
    
    # Count label distribution
    class_counts = np.bincount(labels)
    class_weights = 1. / class_counts
    weights = [class_weights[label] for label in labels]
    
    sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)
    return sampler


# Dataloaders
train_dataset = MRIDataset('./local_data/lbp-train-meta.csv')
valid_dataset = MRIDataset('./local_data/lbp-valid-meta.csv')
test_dataset  = MRIDataset('./local_data/lbp-test-meta.csv')

train_sampler = create_balanced_sampler(train_dataset)
valid_sampler = create_balanced_sampler(valid_dataset)

train_loader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler)
valid_loader = DataLoader(valid_dataset, batch_size=64, sampler=valid_sampler)
test_loader  = DataLoader(test_dataset, batch_size=64)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a m

load model weight from: ./checkpoints/vision_text_pretrain/best


In [4]:
# MLP Classifier
class MLPClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2)  # 2 classes: No Finding, LBP
        )

    def forward(self, x):
        return self.fc(x)

# Instantiate model
embedding_dim = train_dataset[0][0].shape[0]
clf = MLPClassifier(input_dim=embedding_dim).cuda()
print(embedding_dim)

# Loss & optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(clf.parameters(), lr=1e-4)

1024


In [5]:
# Training loop
for epoch in range(10):
    clf.train()
    train_loss = 0
    for emb, label in train_loader:
        emb, label = emb.cuda(), label.cuda()
        optimizer.zero_grad()
        outputs = clf(emb)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    print(f"Epoch {epoch+1}, Train Loss: {train_loss / len(train_loader):.4f}")

    # Validation
    clf.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for emb, label in valid_loader:
            emb, label = emb.cuda(), label.cuda()
            outputs = clf(emb)
            preds = torch.argmax(outputs, dim=1)
            labels = torch.argmax(label, dim=1)
            correct += (preds == labels).sum().item()
            total += label.size(0)
    print(f"Validation Accuracy: {correct / total:.4f}")
# torch.save(clf.state_dict(), './mlp_classifier.pth')
torch.save(clf.state_dict(), './mlp_classifier.pth')
print("✅ Saved MLP classifier to ./mlp_classifier.pth")

Token indices sequence length is longer than the specified maximum sequence length for this model (81 > 77). Running this sequence through the model will result in indexing errors


Epoch 1, Train Loss: 0.6882
Validation Accuracy: 0.7063
Epoch 2, Train Loss: 0.6750
Validation Accuracy: 0.9425
Epoch 3, Train Loss: 0.6562
Validation Accuracy: 0.9306
Epoch 4, Train Loss: 0.6306
Validation Accuracy: 0.9226
Epoch 5, Train Loss: 0.5971
Validation Accuracy: 0.9087
Epoch 6, Train Loss: 0.5614
Validation Accuracy: 0.9385
Epoch 7, Train Loss: 0.5187
Validation Accuracy: 0.9325
Epoch 8, Train Loss: 0.4841
Validation Accuracy: 0.9464
Epoch 9, Train Loss: 0.4439
Validation Accuracy: 0.9623
Epoch 10, Train Loss: 0.4020
Validation Accuracy: 0.9583
✅ Saved MLP classifier to ./mlp_classifier.pth


In [6]:
# Testing — get final performance + predictions
clf.eval()
pred_classes = []
true_classes = []
indices = []
idx = 0

with torch.no_grad():
    for emb, label in test_loader:
        batch_size = emb.size(0)
        emb, label = emb.cuda(), label.cuda()
        outputs = clf(emb)
        preds = torch.argmax(outputs, dim=1)
        labels = torch.argmax(label, dim=1)
        
        pred_classes.extend(preds.cpu().tolist())
        true_classes.extend(labels.cpu().tolist())
        
        indices.extend(range(idx, idx + batch_size))
        idx += batch_size

from sklearn.metrics import classification_report
print(classification_report(true_classes, pred_classes, target_names=['No Finding', 'LBP']))

test_results = pd.DataFrame({
    'index': indices,
    'predicted_class': pred_classes,
    'true_class': true_classes
})
test_results.to_csv('./lbp_test_predictions.csv', index=False)
print("✅ Test predictions saved to ./lbp_test_predictions.csv")

              precision    recall  f1-score   support

  No Finding       0.64      0.88      0.74        66
         LBP       0.98      0.93      0.95       462

    accuracy                           0.92       528
   macro avg       0.81      0.90      0.85       528
weighted avg       0.94      0.92      0.93       528

✅ Test predictions saved to ./lbp_test_predictions.csv
